In [508]:
import pandas as pd
import ast
import spacy
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_trf")
from numpy import average, round
from collections import defaultdict
from openpyxl import load_workbook

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7d014530e610>>
Traceback (most recent call last):
  File "/home/rafael/miniconda3/envs/langchain/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
def somar_scores_tokens_por_posicao(*listas):
    """
    Função que recebe uma ou mais listas de dicionários contendo informações sobre tokens e scores.
    Retorna duas listas: uma contendo os tokens com os maiores scores em cada posição e outra contendo os maiores scores.

    Parâmetros:
    *listas: Uma ou mais listas de dicionários. Cada dicionário contém as chaves 'token' e 'score'.

    Retorno:
    Uma tupla contendo duas listas: a primeira lista contém os tokens com os maiores scores em cada posição e a segunda lista contém os maiores scores.

    Exemplo de uso:
    lista1 = [{'token': 'a', 'score': 0.5}, {'token': 'b', 'score': 0.8}, {'token': 'c', 'score': 0.3}]
    lista2 = [{'token': 'x', 'score': 0.7}, {'token': 'y', 'score': 0.2}, {'token': 'z', 'score': 0.9}]
    maiores_tokens, scores_maiores_tokens = somar_scores_tokens_por_posicao(lista1, lista2)
    print(maiores_tokens)  # Output: ['b', 'x', 'z']
    print(scores_maiores_tokens)  # Output: [1.3, 0.7, 1.2]
    """
    
    maiores_tokens = []
    scores_maiores_tokens = []

    # Itera sobre as listas simultaneamente
    for itens in zip(*listas):
        maior_token = None
        maior_score = float('-inf')

        # Dicionário para armazenar os scores por token
        scores_por_token = {}

        # Itera sobre os itens em cada posição
        for item in itens:
            token = item['token']
            score = item['score']

            # Soma os scores para cada token
            if token in scores_por_token:
                scores_por_token[token] += score
            else:
                scores_por_token[token] = score
        
        # Encontra o token com o maior score na posição atual
        for token, score in scores_por_token.items():
            if score > maior_score:
                maior_token = token
                maior_score = score
        
        # Adiciona o token com o maior score à lista de maiores tokens
        maiores_tokens.append(maior_token)
        scores_maiores_tokens.append(round(maior_score,2))

    return maiores_tokens, scores_maiores_tokens

In [ ]:
def create_results_list(data, true_labels, num_iterations):
    """
    Create a results list based on whether each corresponding true_label is present in the corresponding sublist of the data.

    Parameters:
    - data: List of sublists to search through
    - true_labels: List of true_labels to check for in each sublist
    - num_iterations: Number of iterations to consider for each sublist

    Returns:
    - results_list: List of 1s and 0s based on the presence of true_labels in the sublists

    results_list = []
    for sublist, true_label in zip(data, true_labels):
        results_list.append(1 if true_label in sublist[:num_iterations] else 0)
    """
    results_list = [1 if true_label in sublist[:num_iterations] else 0 for sublist, true_label in zip(data, true_labels)]
    indices = [sublist.index(true_label) if true_label in sublist[:num_iterations] else None for sublist, true_label in zip(data, true_labels)]
    return results_list, indices, round(average(results_list), 3)

def ranking(row, columns):
    """
    Função que realiza o ranqueamento dos tokens em uma linha de dados, com base nas colunas fornecidas.

    Parâmetros:
    - row: dict - Dicionário contendo as informações da linha de dados.
    - columns: list - Lista das colunas que contêm os tokens a serem ranqueados.

    Retorna:
    - list - Lista de dicionários contendo os tokens ranqueados, ordenados pelo score em ordem decrescente.
    """

    tokens_combinados = {}
    for col in columns:
        for token_info in row[col]:
            token = token_info['token']
            score = token_info['score']
            if token in tokens_combinados:
                tokens_combinados[token]['score'] += score
            else:
                tokens_combinados[token] = {'token': token, 'score': score}
    return sorted(list(tokens_combinados.values()), key=lambda x: x['score'], reverse=True)

def ranking_dataframe(df, columns):
    """
    Retorna uma lista de tokens finais para cada linha do dataframe, com base no ranking calculado usando as colunas fornecidas.

    Parâmetros:
    - df: DataFrame - O dataframe de entrada contendo os dados.
    - columns: list - Uma lista de colunas usadas para calcular o ranking.

    Retorno:
    - tokens_finais: list - Uma lista de tokens finais para cada linha do dataframe.
    """
    tokens_finais = [ranking(row, columns) for _, row in df.iterrows()]
    return tokens_finais


# Função para converter uma string para uma lista
def str_to_list(x):
    """
    Converte uma string em uma lista.

    Parâmetros:
    x (str): A string a ser convertida.

    Retorna:
    list: A lista resultante da conversão da string.
          Se a conversão falhar, retorna a string original.
    """
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return x
    
# Função para extrair os tokens
def extrair_tokens(lista_dicts):
    """
    Função que extrai os tokens de uma lista de dicionários.

    Parâmetros:
    - lista_dicts (list): Uma lista de dicionários contendo a chave 'token'.

    Retorna:
    - Uma lista contendo os valores da chave 'token' de cada dicionário.

    Exemplo:
    >>> lista = [{'token': 'Olá'}, {'token': 'mundo'}, {'token': '!'}]
    >>> extrair_tokens(lista)
    ['Olá', 'mundo', '!']
    """
    return [item['token'] for item in lista_dicts]

In [ ]:
def calcular_pontuacao_borda(lista):
    # Dicionário para armazenar as pontuações do Borda Count
    borda_scores = defaultdict(int)

    # Ordenar a lista com base nos escores em ordem decrescente
    lista_ordenada = sorted(lista, key=lambda x: x['score'], reverse=True)

    # Aplicar o método de Borda Count
    for i, item in enumerate(lista_ordenada):
        borda_scores[item['token']] += len(lista_ordenada) - i

    # Multiplicar o valor do Borda Count pelo valor do score
    for item in lista:
        item['score'] *= borda_scores[item['token']]
        # Arredondar o score para três casas decimais
        item['score'] = round(item['score'], 3)

    return lista

def aplicar_calculo_borda_dataframe(df, columns):
    """
    Aplica o cálculo de Borda Count a um DataFrame.

    Parâmetros:
    - df: DataFrame do pandas

    Retorna:
    - DataFrame com os scores calculados usando o método de Borda Count.
    """
    # Aplicar a função para calcular os scores para cada célula do DataFrame
    for coluna in columns:
        df[coluna] = df[coluna].apply(calcular_pontuacao_borda)
    
    return df

def posicao_borda(lista, min_score=5):
    # Dicionário para armazenar as pontuações de Borda
    borda_scores = defaultdict(int)
    
    # Ordenar a lista com base nos escores em ordem decrescente
    lista_ordenada = sorted(lista, key=lambda x: x['score'], reverse=True)
    
    # Definir o número inicial de pontos baseado no tamanho da lista
    tamanho_lista = len(lista_ordenada)
    
    # Aplicar o método de Borda Count
    for i, item in enumerate(lista_ordenada):
        # Calcular a pontuação de Borda, limitando a pontuação mínima ao valor de min_score
        pontuacao_borda = max(tamanho_lista - i, min_score)
        borda_scores[item['token']] += pontuacao_borda

    # Converter o dicionário de pontuações em uma lista de dicionários
    borda_resultado = [{'token': token, 'score': score} for token, score in borda_scores.items()]

    # Ordenar o resultado final com base nas pontuações em ordem decrescente
    borda_resultado = sorted(borda_resultado, key=lambda x: x['score'], reverse=True)

    return borda_resultado

def posicao_borda2(lista):
    # Dicionário para armazenar as pontuações
    borda_scores = defaultdict(int)

    # Ordenar a lista com base nos escores em ordem decrescente
    lista_ordenada = sorted(lista, key=lambda x: x['score'], reverse=True)

    # Aplicar o método de Borda Count
    for i, item in enumerate(lista_ordenada):
        borda_scores[item['token']] += len(lista_ordenada) - i

    # Converter o dicionário de pontuações em uma lista de dicionários
    borda_resultado = [{'token': token, 'score': score} for token, score in borda_scores.items()]

    # Ordenar o resultado final com base nas pontuações
    borda_resultado = sorted(borda_resultado, key=lambda x: x['score'], reverse=True)

    return borda_resultado

def aplicar_posicao_borda_dataframe(df, columns):
    """
    Aplica o cálculo de Borda Count a um DataFrame.

    Parâmetros:
    - df: DataFrame do pandas
    - columns: lista de colunas para aplicar o cálculo

    Retorna:
    - DataFrame com os scores calculados usando o método de Borda Count.
    """
    for coluna in columns:
        df[coluna] = df[coluna].apply(posicao_borda)
    
    return df

In [ ]:
'''def extrair_tokens_gpe(lista_tokens):
    tokens_gpe = []
    for token_info in lista_tokens:
        token = token_info['token']
        doc_capitalize = nlp(token.capitalize())
        doc_lower = nlp(token.lower())
        for doc in [doc_capitalize, doc_lower]:
            for ent in doc.ents:
                if ent.label_ == 'GPE':
                    tokens_gpe.append({'token': token.lower(), 'score': token_info['score']})
                    break  # Sair do loop interno se encontrou uma entidade GPE
            else:
                continue  # Continuar para a próxima iteração do loop externo se não encontrou uma entidade GPE
            break  # Sair do loop externo se encontrou uma entidade GPE
    return tokens_gpe

def extrair_tokens_org(lista_tokens):
    tokens_org = []
    for token_info in lista_tokens:
        token = token_info['token']
        doc_capitalize = nlp(token.capitalize())
        doc_lower = nlp(token.lower())
        for doc in [doc_capitalize, doc_lower]:
            for ent in doc.ents:
                if ent.label_ == 'ORG':
                    tokens_org.append({'token': token.lower(), 'score': token_info['score']})
                    break  # Sair do loop interno se encontrou uma entidade ORG
            else:
                continue  # Continuar para a próxima iteração do loop externo se não encontrou uma entidade ORG
            break  # Sair do loop externo se encontrou uma entidade ORG
    return tokens_org

def extrair_tokens_norp(lista_tokens):
    tokens_norp = []
    for token_info in lista_tokens:
        token = token_info['token']
        doc_capitalize = nlp(token.capitalize())
        doc_lower = nlp(token.lower())
        for doc in [doc_capitalize, doc_lower]:
            for ent in doc.ents:
                if ent.label_ == 'NORP':
                    tokens_norp.append({'token': token.lower(), 'score': token_info['score']})
                    break  # Sair do loop interno se encontrou uma entidade NORP
            else:
                continue  # Continuar para a próxima iteração do loop externo se não encontrou uma entidade NORP
            break  # Sair do loop externo se encontrou uma entidade NORP
    return tokens_norp'''

"def extrair_tokens_gpe(lista_tokens):\n    tokens_gpe = []\n    for token_info in lista_tokens:\n        token = token_info['token']\n        doc_capitalize = nlp(token.capitalize())\n        doc_lower = nlp(token.lower())\n        for doc in [doc_capitalize, doc_lower]:\n            for ent in doc.ents:\n                if ent.label_ == 'GPE':\n                    tokens_gpe.append({'token': token.lower(), 'score': token_info['score']})\n                    break  # Sair do loop interno se encontrou uma entidade GPE\n            else:\n                continue  # Continuar para a próxima iteração do loop externo se não encontrou uma entidade GPE\n            break  # Sair do loop externo se encontrou uma entidade GPE\n    return tokens_gpe\n\ndef extrair_tokens_org(lista_tokens):\n    tokens_org = []\n    for token_info in lista_tokens:\n        token = token_info['token']\n        doc_capitalize = nlp(token.capitalize())\n        doc_lower = nlp(token.lower())\n        for doc in [d

In [ ]:
'''def extrair_tokens_gpe(lista_tokens):
    tokens_gpe = []
    for token_info in lista_tokens:
        token = token_info['token']
        doc_capitalize = nlp(f"Allan Peiper (born 26 April 1960 in {token}, Australia) is a former Professional cyclist, who competed in five Tour de France cycle races.")
        for ent in doc_capitalize.ents:
            if str(ent).lower() == token.lower() and ent.label_ == 'GPE':
                tokens_gpe.append({'token': token.lower(), 'score': token_info['score']})
                break  # Sair do loop interno se encontrou a terceira entidade GPE
    return tokens_gpe


# Função para substituir a máscara em uma string com um token específico
def substituir_prompt(column, row, token):
    return column[row].replace('[MASK]', token)

# Função para extrair tokens que são reconhecidos como entidades nomeadas
def extrair_tokens_gpe2(lista_tokens, novo, row, column_name):
    tokens = []
    for token_info in lista_tokens:
        token = token_info['token']
        doc = nlp(substituir_prompt(novo[column_name], row, token))
        for ent in doc.ents:
            if str(ent).lower() == token.lower() and ent.label_ == 'GPE':
                tokens.append({'token': token.lower(), 'score': token_info['score']})
                break  # Sair do loop interno se encontrou a entidade
    return tokens


def extrair_tokens_org(lista_tokens):
    tokens_org = []
    for token_info in lista_tokens:
        token = token_info['token'].capitalize()
        doc_capitalize = nlp(f"Villa Ocampo is the former house of Victoria Ocampo (1890 – 1979), one of Argentina's greatest cultural figures, founder and director of {token.capitalize()}.")
        for ent in doc_capitalize.ents:
            if str(ent).lower() == token.lower() and ent.label_ == 'ORG':
                tokens_org.append({'token': token.lower(), 'score': token_info['score']})
                break  # Sair do loop interno se encontrou a terceira entidade org
    return tokens_org

# Função para extrair tokens que são reconhecidos como entidades nomeadas
def extrair_tokens_org2(lista_tokens, novo, row, column_name):
    tokens = []
    for token_info in lista_tokens:
        token = token_info['token']
        doc = nlp(substituir_prompt(novo[column_name], row, token))
        for ent in doc.ents:
            if str(ent).lower() == token.lower() and ent.label_ == 'ORG':
                tokens.append({'token': token.lower(), 'score': token_info['score']})
                break  # Sair do loop interno se encontrou a entidade
    return tokens
'''

'def extrair_tokens_gpe(lista_tokens):\n    tokens_gpe = []\n    for token_info in lista_tokens:\n        token = token_info[\'token\']\n        doc_capitalize = nlp(f"Allan Peiper (born 26 April 1960 in {token}, Australia) is a former Professional cyclist, who competed in five Tour de France cycle races.")\n        for ent in doc_capitalize.ents:\n            if str(ent).lower() == token.lower() and ent.label_ == \'GPE\':\n                tokens_gpe.append({\'token\': token.lower(), \'score\': token_info[\'score\']})\n                break  # Sair do loop interno se encontrou a terceira entidade GPE\n    return tokens_gpe\n\n\n# Função para substituir a máscara em uma string com um token específico\ndef substituir_prompt(column, row, token):\n    return column[row].replace(\'[MASK]\', token)\n\n# Função para extrair tokens que são reconhecidos como entidades nomeadas\ndef extrair_tokens_gpe2(lista_tokens, novo, row, column_name):\n    tokens = []\n    for token_info in lista_tokens:\n

In [ ]:
# Função para converter uma string para uma lista
def str_to_list(x):
    """
    Converte uma string em uma lista.

    Parâmetros:
    x (str): A string a ser convertida.

    Retorna:
    list: A lista resultante da conversão da string.
          Se a conversão falhar, retorna a string original.
    """
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return x
    
# Função para substituir a máscara em uma string com um token específico
def substituir_prompt(sentence, token):
    return sentence.replace('[MASK]', token)

# Função para extrair tokens que são reconhecidos como entidades nomeadas
def extrair_tokens_gpe2(sentence, token_info, nlp):
    token = token_info['token']
    doc = nlp(substituir_prompt(sentence, token))
    for ent in doc.ents:
        if str(ent).lower() == token.lower() and ent.label_ == 'GPE':
            return {'token': token.lower(), 'score': token_info['score']}
    return None

# Função para extrair tokens que são reconhecidos como entidades nomeadas
def extrair_tokens_org2(sentence, token_info, nlp):
    token = token_info['token']
    doc = nlp(substituir_prompt(sentence, token))
    for ent in doc.ents:
        if str(ent).lower() == token.lower() and ent.label_ == 'ORG':
            return {'token': token.lower(), 'score': token_info['score']}
    return None

# Função para extrair tokens que são reconhecidos como entidades nomeadas
def extrair_tokens_lang2(sentence, token_info, nlp):
    token = token_info['token']
    doc = nlp(substituir_prompt(sentence, token))
    for ent in doc.ents:
        if str(ent).lower() == token.lower() and ent.label_ == 'LANGUAGE':
            return {'token': token.lower(), 'score': token_info['score']}
    return None

# Função principal para aplicar extração de tokens em um DataFrame
def extrair_tokens_dataframe(dataframe, masked_columns, original_columns, nlp, extrator):
    # Inicializar um novo DataFrame para armazenar os tokens extraídos
    tokens_extraidos = {col: [] for col in original_columns}
    
    # Iterar sobre cada linha do DataFrame
    for _, row in dataframe.iterrows():
        for masked_col, original_col in zip(masked_columns, original_columns):
            tokens = []
            for token_info in row[original_col]:
                extracted_token = extrator(row[masked_col], token_info, nlp)
                if extracted_token:
                    tokens.append(extracted_token)
            tokens_extraidos[original_col].append(tokens)
    
    # Criar um novo DataFrame com os tokens extraídos
    tokens_dataframe = pd.DataFrame(tokens_extraidos)
    
    return tokens_dataframe


## Teste

In [ ]:
prompt_bert = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/t5_bornIn.csv')
prompt_bert.drop(columns=['Unnamed: 0'], inplace=True)

bornIn_bert = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/bert/outputs_bornIn_bert_v2.csv')
bornIn_bert.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_bert = bornIn_bert.map(str_to_list)

In [ ]:
prompt_bert.columns

Index(['sub_label', 'template', 'obj_label', 'masked_sentence',
       'T5_paraphrased_filled_sentence_0', 'T5_paraphrased_filled_sentence_1',
       'T5_paraphrased_filled_sentence_2', 'T5_paraphrased_filled_sentence_3'],
      dtype='object')

In [ ]:
# Creating a new DataFrame from 'prompt' with specified columns
novo = prompt_bert[['masked_sentence', 'T5_paraphrased_filled_sentence_0', 
               'T5_paraphrased_filled_sentence_1','T5_paraphrased_filled_sentence_2', 
               'T5_paraphrased_filled_sentence_3']].copy()

# Using .loc to avoid SettingWithCopyWarning
novo.loc[:, 'original'] = bornIn_bert['original']
novo.loc[:, 't5_0'] = bornIn_bert['t5_0']
novo.loc[:, 't5_1'] = bornIn_bert['t5_1']
novo.loc[:, 't5_2'] = bornIn_bert['t5_2']
novo.loc[:, 't5_3'] = bornIn_bert['t5_3']

In [ ]:
raw_text = 'The official language of Azad Kashmir is urdu.'

text1= nlp(raw_text)

for word in text1.ents:
    print(word.text,word.label_)

Azad Kashmir GPE
urdu LANGUAGE


# Real

#### Bert

In [ ]:
bornIn_bert = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/bert/outputs_bornIn_bert_v2.csv')
bornIn_bert.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_bert = bornIn_bert.map(str_to_list)
#bornIn_bert = bornIn_bert[:10]
diedIn_bert = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/bert/outputs_diedIn_bert_v2.csv')
diedIn_bert.drop(columns=['Unnamed: 0'], inplace=True)
diedIn_bert = diedIn_bert.map(str_to_list)
#diedIn_bert = diedIn_bert[:10]
capital_bert = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/bert/outputs_capital_bert_v2.csv')
capital_bert.drop(columns=['Unnamed: 0'], inplace=True)
capital_bert = capital_bert.map(str_to_list)
#capital_bert = capital_bert[:10]
worksfor_bert = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/bert/outputs_worksfor_bert_v2.csv')
worksfor_bert.drop(columns=['Unnamed: 0'], inplace=True)
worksfor_bert = worksfor_bert.map(str_to_list)
#worksfor_bert = worksfor_bert[:10]
citizen_bert = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/bert/outputs_citizen_bert_v2.csv')
citizen_bert.drop(columns=['Unnamed: 0'], inplace=True)
citizen_bert = citizen_bert.map(str_to_list)
#citizen_bert = citizen_bert[:10]

In [ ]:
language_bert = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/bert/outputs_plays_music_bert_v2.csv')
language_bert.drop(columns=['Unnamed: 0'], inplace=True)
language_bert = language_bert.map(str_to_list)

In [ ]:
bornIn_bert_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/bert/v2/outputs_bornIn_t5_bert_one_shoot_v2.csv')
bornIn_bert_one.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_bert_one = bornIn_bert_one.map(str_to_list)
#bornIn_bert_one = bornIn_bert_one[:10]
diedIn_bert_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/bert/v2/outputs_diedIn_t5_bert_one_shoot_v2.csv')
diedIn_bert_one.drop(columns=['Unnamed: 0'], inplace=True)
diedIn_bert_one = diedIn_bert_one.map(str_to_list)
#diedIn_bert_one = diedIn_bert_one[:10]
capital_bert_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/bert/v2/outputs_capital_t5_bert_one_shoot_v2.csv')
capital_bert_one.drop(columns=['Unnamed: 0'], inplace=True)
capital_bert_one = capital_bert_one.map(str_to_list)
#capital_bert_one = capital_bert_one[:10]
worksfor_bert_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/bert/v2/outputs_worksfor_t5_bert_one_shoot_v2.csv')
worksfor_bert_one.drop(columns=['Unnamed: 0'], inplace=True)
worksfor_bert_one = worksfor_bert_one.map(str_to_list)
#worksfor_bert_one = worksfor_bert_one[:10]
citizen_bert_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/bert/v2/outputs_citizen_t5_bert_one_shoot_v2.csv')
citizen_bert_one.drop(columns=['Unnamed: 0'], inplace=True)
citizen_bert_one = citizen_bert_one.map(str_to_list)
#citizen_bert_one = citizen_bert_one[:10]

In [ ]:
language_bert_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/bert/v2/outputs_plays_music_t5_bert_one_shoot_v2.csv')
language_bert_one.drop(columns=['Unnamed: 0'], inplace=True)
language_bert_one = language_bert_one.map(str_to_list)
#language_bert_one = language_bert_one[:10]

In [ ]:
bornIn_bert_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/bert/v2/outputs_bornIn_t5_bert_few_shoot_v2.csv')
bornIn_bert_few.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_bert_few = bornIn_bert_few.map(str_to_list)
#bornIn_bert_few = bornIn_bert_few[:10]
diedIn_bert_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/bert/v2/outputs_diedIn_t5_bert_few_shoot_v2.csv')
diedIn_bert_few.drop(columns=['Unnamed: 0'], inplace=True)
diedIn_bert_few = diedIn_bert_few.map(str_to_list)
#diedIn_bert_few = diedIn_bert_few[:10]
capital_bert_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/bert/v2/outputs_capital_t5_bert_few_shoot_v2.csv')
capital_bert_few.drop(columns=['Unnamed: 0'], inplace=True)
capital_bert_few = capital_bert_few.map(str_to_list)
#capital_bert_few = capital_bert_few[:10]
worksfor_bert_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/bert/v2/outputs_worksfor_t5_bert_few_shoot_v2.csv')
worksfor_bert_few.drop(columns=['Unnamed: 0'], inplace=True)
worksfor_bert_few = worksfor_bert_few.map(str_to_list)
#worksfor_bert_few = worksfor_bert_few[:10]
citizen_bert_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/bert/v2/outputs_citizen_t5_bert_few_shoot_v2.csv')
citizen_bert_few.drop(columns=['Unnamed: 0'], inplace=True)
citizen_bert_few = citizen_bert_few.map(str_to_list)
#citizen_bert_few = citizen_bert_few[:10]

In [ ]:
language_bert_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/bert/v2/outputs_plays_music_t5_bert_few_shoot_v2.csv')
language_bert_few.drop(columns=['Unnamed: 0'], inplace=True)
language_bert_few = language_bert_few.map(str_to_list)
#language_bert_few = language_bert_few[:10]

#### Roberta

In [ ]:
bornIn_roberta = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/roberta/outputs_bornIn_roberta_v2.csv')
bornIn_roberta.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_roberta = bornIn_roberta.map(str_to_list)
#bornIn_roberta = bornIn_roberta[:10]
diedIn_roberta = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/roberta/outputs_diedIn_roberta_v2.csv')
diedIn_roberta.drop(columns=['Unnamed: 0'], inplace=True)
diedIn_roberta = diedIn_roberta.map(str_to_list)
#diedIn_roberta = diedIn_roberta[:10]
capital_roberta = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/roberta/outputs_capital_roberta_v2.csv')
capital_roberta.drop(columns=['Unnamed: 0'], inplace=True)
capital_roberta = capital_roberta.map(str_to_list)
#capital_roberta = capital_roberta[:10]
worksfor_roberta = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/roberta/outputs_worksfor_roberta_v2.csv')
worksfor_roberta.drop(columns=['Unnamed: 0'], inplace=True)
worksfor_roberta = worksfor_roberta.map(str_to_list)
#worksfor_roberta = worksfor_roberta[:10]
citizen_roberta = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/roberta/outputs_citizen_roberta_v2.csv')
citizen_roberta.drop(columns=['Unnamed: 0'], inplace=True)
citizen_roberta = citizen_roberta.map(str_to_list)
#citizen_roberta = citizen_roberta[:10]

In [ ]:
language_roberta = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/roberta/outputs_plays_music_roberta_v2.csv')
language_roberta.drop(columns=['Unnamed: 0'], inplace=True)
language_roberta = language_roberta.map(str_to_list)

In [ ]:
bornIn_roberta_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/roberta/v2/outputs_bornIn_t5_roberta_one_shoot_v2.csv')
bornIn_roberta_one.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_roberta_one = bornIn_roberta_one.map(str_to_list)
#bornIn_roberta_one = bornIn_roberta_one[:10]
diedIn_roberta_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/roberta/v2/outputs_diedIn_t5_roberta_one_shoot_v2.csv')
diedIn_roberta_one.drop(columns=['Unnamed: 0'], inplace=True)
diedIn_roberta_one = diedIn_roberta_one.map(str_to_list)
#diedIn_roberta_one = diedIn_roberta_one[:10]
capital_roberta_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/roberta/v2/outputs_capital_t5_roberta_one_shoot_v2.csv')
capital_roberta_one.drop(columns=['Unnamed: 0'], inplace=True)
capital_roberta_one = capital_roberta_one.map(str_to_list)
#capital_roberta_one = capital_roberta_one[:10]
worksfor_roberta_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/roberta/v2/outputs_worksfor_t5_roberta_one_shoot_v2.csv')
worksfor_roberta_one.drop(columns=['Unnamed: 0'], inplace=True)
worksfor_roberta_one = worksfor_roberta_one.map(str_to_list)
#worksfor_roberta_one = worksfor_roberta_one[:10]
citizen_roberta_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/roberta/v2/outputs_citizen_t5_roberta_one_shoot_v2.csv')
citizen_roberta_one.drop(columns=['Unnamed: 0'], inplace=True)
citizen_roberta_one = citizen_roberta_one.map(str_to_list)
#citizen_roberta_one = citizen_roberta_one[:10]

In [ ]:
language_roberta_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/roberta/v2/outputs_plays_music_t5_roberta_one_shoot_v2.csv')
language_roberta_one.drop(columns=['Unnamed: 0'], inplace=True)
language_roberta_one = language_roberta_one.map(str_to_list)
#language_roberta_one = language_roberta_one[:10]

In [ ]:
bornIn_roberta_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/roberta/v2/outputs_bornIn_t5_roberta_few_shoot_v2.csv')
bornIn_roberta_few.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_roberta_few = bornIn_roberta_few.map(str_to_list)
#bornIn_roberta_few = bornIn_roberta_few[:10]
diedIn_roberta_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/roberta/v2/outputs_diedIn_t5_roberta_few_shoot_v2.csv')
diedIn_roberta_few.drop(columns=['Unnamed: 0'], inplace=True)
diedIn_roberta_few = diedIn_roberta_few.map(str_to_list)
#diedIn_roberta_few = diedIn_roberta_few[:10]
capital_roberta_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/roberta/v2/outputs_capital_t5_roberta_few_shoot_v2.csv')
capital_roberta_few.drop(columns=['Unnamed: 0'], inplace=True)
capital_roberta_few = capital_roberta_few.map(str_to_list)
#capital_roberta_few = capital_roberta_few[:10]
worksfor_roberta_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/roberta/v2/outputs_worksfor_t5_roberta_few_shoot_v2.csv')
worksfor_roberta_few.drop(columns=['Unnamed: 0'], inplace=True)
worksfor_roberta_few = worksfor_roberta_few.map(str_to_list)
#worksfor_roberta_few = worksfor_roberta_few[:10]
citizen_roberta_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/roberta/v2/outputs_citizen_t5_roberta_few_shoot_v2.csv')
citizen_roberta_few.drop(columns=['Unnamed: 0'], inplace=True)
citizen_roberta_few = citizen_roberta_few.map(str_to_list)
#citizen_roberta_few = citizen_roberta_few[:10]

In [ ]:
language_roberta_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/roberta/v2/outputs_plays_music_t5_roberta_few_shoot_v2.csv')
language_roberta_few.drop(columns=['Unnamed: 0'], inplace=True)
language_roberta_few = language_roberta_few.map(str_to_list)
#language_roberta_few = language_roberta_few[:10]

#### Electra

In [ ]:
bornIn_electra = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/electra/outputs_bornIn_electra_v2.csv')
bornIn_electra.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_electra = bornIn_electra.map(str_to_list)
#bornIn_electra = bornIn_electra[:10]
diedIn_electra = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/electra/outputs_diedIn_electra_v2.csv')
diedIn_electra.drop(columns=['Unnamed: 0'], inplace=True)
diedIn_electra = diedIn_electra.map(str_to_list)
#diedIn_electra = diedIn_electra[:10]
capital_electra = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/electra/outputs_capital_electra_v2.csv')
capital_electra.drop(columns=['Unnamed: 0'], inplace=True)
capital_electra = capital_electra.map(str_to_list)
#capital_electra = capital_electra[:10]
worksfor_electra = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/electra/outputs_worksfor_electra_v2.csv')
worksfor_electra.drop(columns=['Unnamed: 0'], inplace=True)
worksfor_electra = worksfor_electra.map(str_to_list)
#worksfor_electra = worksfor_electra[:10]
citizen_electra = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/electra/outputs_citizen_electra_v2.csv')
citizen_electra.drop(columns=['Unnamed: 0'], inplace=True)
citizen_electra = citizen_electra.map(str_to_list)
#citizen_electra = citizen_electra[:10]

In [ ]:
language_electra = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/electra/outputs_plays_music_electra_v2.csv')
language_electra.drop(columns=['Unnamed: 0'], inplace=True)
language_electra = language_electra.map(str_to_list)

In [ ]:
bornIn_electra_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/electra/v2/outputs_bornIn_t5_electra_one_shoot_v2.csv')
bornIn_electra_one.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_electra_one = bornIn_electra_one.map(str_to_list)
#bornIn_electra_one = bornIn_electra_one[:10]
diedIn_electra_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/electra/v2/outputs_diedIn_t5_electra_one_shoot_v2.csv')
diedIn_electra_one.drop(columns=['Unnamed: 0'], inplace=True)
diedIn_electra_one = diedIn_electra_one.map(str_to_list)
#diedIn_electra_one = diedIn_electra_one[:10]
capital_electra_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/electra/v2/outputs_capital_t5_electra_one_shoot_v2.csv')
capital_electra_one.drop(columns=['Unnamed: 0'], inplace=True)
capital_electra_one = capital_electra_one.map(str_to_list)
#capital_electra_one = capital_electra_one[:10]
worksfor_electra_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/electra/v2/outputs_worksfor_t5_electra_one_shoot_v2.csv')
worksfor_electra_one.drop(columns=['Unnamed: 0'], inplace=True)
worksfor_electra_one = worksfor_electra_one.map(str_to_list)
#worksfor_electra_one = worksfor_electra_one[:10]
citizen_electra_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/electra/v2/outputs_citizen_t5_electra_one_shoot_v2.csv')
citizen_electra_one.drop(columns=['Unnamed: 0'], inplace=True)
citizen_electra_one = citizen_electra_one.map(str_to_list)
#citizen_electra_one = citizen_electra_one[:10]

In [ ]:
language_electra_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/electra/v2/outputs_plays_music_t5_electra_one_shoot_v2.csv')
language_electra_one.drop(columns=['Unnamed: 0'], inplace=True)
language_electra_one = language_electra_one.map(str_to_list)
#language_electra_one = language_electra_one[:10]

In [ ]:
bornIn_electra_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/electra/v2/outputs_bornIn_t5_electra_few_shoot_v2.csv')
bornIn_electra_few.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_electra_few = bornIn_electra_few.map(str_to_list)
#bornIn_electra_few = bornIn_electra_few[:10]
diedIn_electra_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/electra/v2/outputs_diedIn_t5_electra_few_shoot_v2.csv')
diedIn_electra_few.drop(columns=['Unnamed: 0'], inplace=True)
diedIn_electra_few = diedIn_electra_few.map(str_to_list)
#diedIn_electra_few = diedIn_electra_few[:10]
capital_electra_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/electra/v2/outputs_capital_t5_electra_few_shoot_v2.csv')
capital_electra_few.drop(columns=['Unnamed: 0'], inplace=True)
capital_electra_few = capital_electra_few.map(str_to_list)
#capital_electra_few = capital_electra_few[:10]
worksfor_electra_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/electra/v2/outputs_worksfor_t5_electra_few_shoot_v2.csv')
worksfor_electra_few.drop(columns=['Unnamed: 0'], inplace=True)
worksfor_electra_few = worksfor_electra_few.map(str_to_list)
#worksfor_electra_few = worksfor_electra_few[:10]
citizen_electra_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/electra/v2/outputs_citizen_t5_electra_few_shoot_v2.csv')
citizen_electra_few.drop(columns=['Unnamed: 0'], inplace=True)
citizen_electra_few = citizen_electra_few.map(str_to_list)
#citizen_electra_few = citizen_electra_few[:10]

In [ ]:
language_electra_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/electra/v2/outputs_plays_music_t5_electra_few_shoot_v2.csv')
language_electra_few.drop(columns=['Unnamed: 0'], inplace=True)
language_electra_few = language_electra_few.map(str_to_list)
#language_electra_few = language_electra_few[:10]

# Spacy

## Zero-Shot

### Bert

In [ ]:
prompt_bornIn = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/t5_bornIn.csv')
prompt_bornIn.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_bert = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/bert/outputs_bornIn_bert_v2.csv')
bornIn_bert.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_bert = bornIn_bert.map(str_to_list)

prompt_diedIn = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/t5_diedIn.csv')
prompt_diedIn.drop(columns=['Unnamed: 0'], inplace=True)
diedIn_bert = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/bert/outputs_diedIn_bert_v2.csv')
diedIn_bert.drop(columns=['Unnamed: 0'], inplace=True)
diedIn_bert = diedIn_bert.map(str_to_list)

prompt_capital = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/t5_capital.csv')
prompt_capital.drop(columns=['Unnamed: 0'], inplace=True)
capital_bert = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/bert/outputs_capital_bert_v2.csv')
capital_bert.drop(columns=['Unnamed: 0'], inplace=True)
capital_bert = capital_bert.map(str_to_list)

prompt_citizen = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/t5_citizen.csv')
prompt_citizen.drop(columns=['Unnamed: 0'], inplace=True)
citizen_bert = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/bert/outputs_citizen_bert_v2.csv')
citizen_bert.drop(columns=['Unnamed: 0'], inplace=True)
citizen_bert = citizen_bert.map(str_to_list)

prompt_worksfor = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/t5_worksfor.csv').iloc[:379]
prompt_worksfor.drop(columns=['Unnamed: 0'], inplace=True)
worksfor_bert = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/bert/outputs_worksfor_bert_v2.csv')
worksfor_bert.drop(columns=['Unnamed: 0'], inplace=True)
worksfor_bert = worksfor_bert.map(str_to_list)

prompt_language = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/t5_language.csv')
prompt_language.drop(columns=['Unnamed: 0'], inplace=True)
language_bert = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/bert/outputs_language_bert_v2.csv')
language_bert.drop(columns=['Unnamed: 0'], inplace=True)
language_bert = language_bert.map(str_to_list)

In [ ]:
def substituir_x(row):
    return row['template'].replace('[X]', row['sub_label'])

def substituir_y(row):
    return row['triple_NL'].replace('[Y]', '[MASK]')

In [ ]:
# Born In
prompt_bornIn.loc[:, 'triple_NL'] = prompt_bornIn.apply(substituir_x, axis=1)
prompt_bornIn.loc[:, 'triple_NL'] = prompt_bornIn.apply(substituir_y, axis=1)

# Died In
prompt_diedIn.loc[:, 'triple_NL'] = prompt_diedIn.apply(substituir_x, axis=1)
prompt_diedIn.loc[:, 'triple_NL'] = prompt_diedIn.apply(substituir_y, axis=1)

# Capital
prompt_capital.loc[:, 'triple_NL'] = prompt_capital.apply(substituir_x, axis=1)
prompt_capital.loc[:, 'triple_NL'] = prompt_capital.apply(substituir_y, axis=1)

# Works For
prompt_worksfor.loc[:, 'triple_NL'] = prompt_worksfor.apply(substituir_x, axis=1)
prompt_worksfor.loc[:, 'triple_NL'] = prompt_worksfor.apply(substituir_y, axis=1)

# Citizen
prompt_citizen.loc[:, 'triple_NL'] = prompt_citizen.apply(substituir_x, axis=1)
prompt_citizen.loc[:, 'triple_NL'] = prompt_citizen.apply(substituir_y, axis=1)

# Language
prompt_language.loc[:, 'triple_NL'] = prompt_language.apply(substituir_x, axis=1)
prompt_language.loc[:, 'triple_NL'] = prompt_language.apply(substituir_y, axis=1)

# Language
prompt_language.loc[:, 'triple_NL'] = prompt_language.apply(substituir_x, axis=1)
prompt_language.loc[:, 'triple_NL'] = prompt_language.apply(substituir_y, axis=1)

#### Language

In [ ]:
# Creating a new DataFrame from 'prompt' with specified columns
bornIn_bert_spacy = prompt_bornIn[['masked_sentence', 'T5_paraphrased_filled_sentence_0', 
               'T5_paraphrased_filled_sentence_1','T5_paraphrased_filled_sentence_2', 
               'T5_paraphrased_filled_sentence_3']].copy()

# Using .loc to avoid SettingWithCopyWarning
bornIn_bert_spacy.loc[:, 'original'] = bornIn_bert['original']
bornIn_bert_spacy.loc[:, 't5_0'] = bornIn_bert['t5_0']
bornIn_bert_spacy.loc[:, 't5_1'] = bornIn_bert['t5_1']
bornIn_bert_spacy.loc[:, 't5_2'] = bornIn_bert['t5_2']
bornIn_bert_spacy.loc[:, 't5_3'] = bornIn_bert['t5_3']

# Creating a new DataFrame from 'prompt' with specified columns
diedIn_bert_spacy = prompt_diedIn[['masked_sentence', 'T5_paraphrased_filled_sentence_0', 
               'T5_paraphrased_filled_sentence_1','T5_paraphrased_filled_sentence_2', 
               'T5_paraphrased_filled_sentence_3']].copy()

# Using .loc to avoid SettingWithCopyWarning
diedIn_bert_spacy.loc[:, 'original'] = diedIn_bert['original']
diedIn_bert_spacy.loc[:, 't5_0'] = diedIn_bert['t5_0']
diedIn_bert_spacy.loc[:, 't5_1'] = diedIn_bert['t5_1']
diedIn_bert_spacy.loc[:, 't5_2'] = diedIn_bert['t5_2']
diedIn_bert_spacy.loc[:, 't5_3'] = diedIn_bert['t5_3']

# Creating a new DataFrame from 'prompt' with specified columns
capital_bert_spacy = prompt_capital[['masked_sentence', 'T5_paraphrased_filled_sentence_0', 
               'T5_paraphrased_filled_sentence_1','T5_paraphrased_filled_sentence_2', 
               'T5_paraphrased_filled_sentence_3']].copy()

# Using .loc to avoid SettingWithCopyWarning
capital_bert_spacy.loc[:, 'original'] = capital_bert['original']
capital_bert_spacy.loc[:, 't5_0'] = capital_bert['t5_0']
capital_bert_spacy.loc[:, 't5_1'] = capital_bert['t5_1']
capital_bert_spacy.loc[:, 't5_2'] = capital_bert['t5_2']
capital_bert_spacy.loc[:, 't5_3'] = capital_bert['t5_3']

# Creating a new DataFrame from 'prompt' with specified columns
citizen_bert_spacy = prompt_citizen[['masked_sentence', 'T5_paraphrased_filled_sentence_0', 
               'T5_paraphrased_filled_sentence_1','T5_paraphrased_filled_sentence_2', 
               'T5_paraphrased_filled_sentence_3']].copy()

# Using .loc to avoid SettingWithCopyWarning
citizen_bert_spacy.loc[:, 'original'] = citizen_bert['original']
citizen_bert_spacy.loc[:, 't5_0'] = citizen_bert['t5_0']
citizen_bert_spacy.loc[:, 't5_1'] = citizen_bert['t5_1']
citizen_bert_spacy.loc[:, 't5_2'] = citizen_bert['t5_2']
citizen_bert_spacy.loc[:, 't5_3'] = citizen_bert['t5_3']

# Creating a new DataFrame from 'prompt' with specified columns
worksfor_bert_spacy = prompt_worksfor[['masked_sentence', 'T5_paraphrased_filled_sentence_0', 
               'T5_paraphrased_filled_sentence_1','T5_paraphrased_filled_sentence_2', 
               'T5_paraphrased_filled_sentence_3']].copy()

# Using .loc to avoid SettingWithCopyWarning
worksfor_bert_spacy.loc[:, 'original'] = worksfor_bert['original']
worksfor_bert_spacy.loc[:, 't5_0'] = worksfor_bert['t5_0']
worksfor_bert_spacy.loc[:, 't5_1'] = worksfor_bert['t5_1']
worksfor_bert_spacy.loc[:, 't5_2'] = worksfor_bert['t5_2']
worksfor_bert_spacy.loc[:, 't5_3'] = worksfor_bert['t5_3']


# Creating a new DataFrame from 'prompt' with specified columns
language_bert_spacy = prompt_language[['masked_sentence', 'T5_paraphrased_filled_sentence_0', 
               'T5_paraphrased_filled_sentence_1','T5_paraphrased_filled_sentence_2', 
               'T5_paraphrased_filled_sentence_3']].copy()

# Using .loc to avoid SettingWithCopyWarning
language_bert_spacy.loc[:, 'original'] = language_bert['original']
language_bert_spacy.loc[:, 't5_0'] = language_bert['t5_0']
language_bert_spacy.loc[:, 't5_1'] = language_bert['t5_1']
language_bert_spacy.loc[:, 't5_2'] = language_bert['t5_2']
language_bert_spacy.loc[:, 't5_3'] = language_bert['t5_3']

In [ ]:
# Replicando a coluna 'triple_NL' cinco vezes
bornIn_bert_spacy = pd.concat([prompt_bornIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
bornIn_bert_spacy.columns = new_columns


# Using .loc to avoid SettingWithCopyWarning
bornIn_bert_spacy.loc[:, 'original'] = bornIn_bert['original']
bornIn_bert_spacy.loc[:, 't5_0'] = bornIn_bert['t5_0']
bornIn_bert_spacy.loc[:, 't5_1'] = bornIn_bert['t5_1']
bornIn_bert_spacy.loc[:, 't5_2'] = bornIn_bert['t5_2']
bornIn_bert_spacy.loc[:, 't5_3'] = bornIn_bert['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
diedIn_bert_spacy = pd.concat([prompt_diedIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
diedIn_bert_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
diedIn_bert_spacy.loc[:, 'original'] = diedIn_bert['original']
diedIn_bert_spacy.loc[:, 't5_0'] = diedIn_bert['t5_0']
diedIn_bert_spacy.loc[:, 't5_1'] = diedIn_bert['t5_1']
diedIn_bert_spacy.loc[:, 't5_2'] = diedIn_bert['t5_2']
diedIn_bert_spacy.loc[:, 't5_3'] = diedIn_bert['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
capital_bert_spacy = pd.concat([prompt_capital['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
capital_bert_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
capital_bert_spacy.loc[:, 'original'] = capital_bert['original']
capital_bert_spacy.loc[:, 't5_0'] = capital_bert['t5_0']
capital_bert_spacy.loc[:, 't5_1'] = capital_bert['t5_1']
capital_bert_spacy.loc[:, 't5_2'] = capital_bert['t5_2']
capital_bert_spacy.loc[:, 't5_3'] = capital_bert['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
citizen_bert_spacy = pd.concat([prompt_citizen['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
citizen_bert_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
citizen_bert_spacy.loc[:, 'original'] = citizen_bert['original']
citizen_bert_spacy.loc[:, 't5_0'] = citizen_bert['t5_0']
citizen_bert_spacy.loc[:, 't5_1'] = citizen_bert['t5_1']
citizen_bert_spacy.loc[:, 't5_2'] = citizen_bert['t5_2']
citizen_bert_spacy.loc[:, 't5_3'] = citizen_bert['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
worksfor_bert_spacy = pd.concat([prompt_worksfor['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
worksfor_bert_spacy.columns = new_columns
# Using .loc to avoid SettingWithCopyWarning
worksfor_bert_spacy.loc[:, 'original'] = worksfor_bert['original']
worksfor_bert_spacy.loc[:, 't5_0'] = worksfor_bert['t5_0']
worksfor_bert_spacy.loc[:, 't5_1'] = worksfor_bert['t5_1']
worksfor_bert_spacy.loc[:, 't5_2'] = worksfor_bert['t5_2']
worksfor_bert_spacy.loc[:, 't5_3'] = worksfor_bert['t5_3']


# Replicando a coluna 'triple_NL' cinco vezes
language_bert_spacy = pd.concat([prompt_language['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
language_bert_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
language_bert_spacy.loc[:, 'original'] = language_bert['original']
language_bert_spacy.loc[:, 't5_0'] = language_bert['t5_0']
language_bert_spacy.loc[:, 't5_1'] = language_bert['t5_1']
language_bert_spacy.loc[:, 't5_2'] = language_bert['t5_2']
language_bert_spacy.loc[:, 't5_3'] = language_bert['t5_3']

In [ ]:
# Exemplo de uso
# Supondo que temos um DataFrame `novo` com as colunas especificadas
masked_columns = [
    'triple_NL_1',
    'triple_NL_2',
    'triple_NL_3',
    'triple_NL_4',
    'triple_NL_5'
]

# As colunas que contêm os tokens originais
original_columns = [
    'original', 
    't5_0', 
    't5_1', 
    't5_2', 
    't5_3'
]

# Chamar a função para extrair tokens
bornIn_bert_spacy = extrair_tokens_dataframe(bornIn_bert_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
diedIn_bert_spacy = extrair_tokens_dataframe(diedIn_bert_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
capital_bert_spacy = extrair_tokens_dataframe(capital_bert_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
citizen_bert_spacy = extrair_tokens_dataframe(citizen_bert_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
worksfor_bert_spacy = extrair_tokens_dataframe(worksfor_bert_spacy, masked_columns, original_columns, nlp, extrair_tokens_org2)
language_bert_spacy = extrair_tokens_dataframe(language_bert_spacy, masked_columns, original_columns, nlp, extrair_tokens_lang2)

### Roberta

In [ ]:
bornIn_roberta = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/roberta/outputs_bornIn_roberta_v2.csv')
bornIn_roberta.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_roberta = bornIn_roberta.map(str_to_list)


diedIn_roberta = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/roberta/outputs_diedIn_roberta_v2.csv')
diedIn_roberta.drop(columns=['Unnamed: 0'], inplace=True)
diedIn_roberta = diedIn_roberta.map(str_to_list)


capital_roberta = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/roberta/outputs_capital_roberta_v2.csv')
capital_roberta.drop(columns=['Unnamed: 0'], inplace=True)
capital_roberta = capital_roberta.map(str_to_list)


citizen_roberta = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/roberta/outputs_citizen_roberta_v2.csv')
citizen_roberta.drop(columns=['Unnamed: 0'], inplace=True)
citizen_roberta = citizen_roberta.map(str_to_list)


worksfor_roberta = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/roberta/outputs_worksfor_roberta_v2.csv')
worksfor_roberta.drop(columns=['Unnamed: 0'], inplace=True)
worksfor_roberta = worksfor_roberta.map(str_to_list)

language_roberta = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/roberta/outputs_language_roberta_v2.csv')
language_roberta.drop(columns=['Unnamed: 0'], inplace=True)
language_roberta = language_roberta.map(str_to_list)

In [ ]:
# Replicando a coluna 'triple_NL' cinco vezes
bornIn_roberta_spacy = pd.concat([prompt_bornIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
bornIn_roberta_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
bornIn_roberta_spacy.loc[:, 'original'] = bornIn_roberta['original']
bornIn_roberta_spacy.loc[:, 't5_0'] = bornIn_roberta['t5_0']
bornIn_roberta_spacy.loc[:, 't5_1'] = bornIn_roberta['t5_1']
bornIn_roberta_spacy.loc[:, 't5_2'] = bornIn_roberta['t5_2']
bornIn_roberta_spacy.loc[:, 't5_3'] = bornIn_roberta['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
diedIn_roberta_spacy = pd.concat([prompt_diedIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
diedIn_roberta_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
diedIn_roberta_spacy.loc[:, 'original'] = diedIn_roberta['original']
diedIn_roberta_spacy.loc[:, 't5_0'] = diedIn_roberta['t5_0']
diedIn_roberta_spacy.loc[:, 't5_1'] = diedIn_roberta['t5_1']
diedIn_roberta_spacy.loc[:, 't5_2'] = diedIn_roberta['t5_2']
diedIn_roberta_spacy.loc[:, 't5_3'] = diedIn_roberta['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
capital_roberta_spacy = pd.concat([prompt_capital['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
capital_roberta_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
capital_roberta_spacy.loc[:, 'original'] = capital_roberta['original']
capital_roberta_spacy.loc[:, 't5_0'] = capital_roberta['t5_0']
capital_roberta_spacy.loc[:, 't5_1'] = capital_roberta['t5_1']
capital_roberta_spacy.loc[:, 't5_2'] = capital_roberta['t5_2']
capital_roberta_spacy.loc[:, 't5_3'] = capital_roberta['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
citizen_roberta_spacy = pd.concat([prompt_citizen['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
citizen_roberta_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
citizen_roberta_spacy.loc[:, 'original'] = citizen_roberta['original']
citizen_roberta_spacy.loc[:, 't5_0'] = citizen_roberta['t5_0']
citizen_roberta_spacy.loc[:, 't5_1'] = citizen_roberta['t5_1']
citizen_roberta_spacy.loc[:, 't5_2'] = citizen_roberta['t5_2']
citizen_roberta_spacy.loc[:, 't5_3'] = citizen_roberta['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
worksfor_roberta_spacy = pd.concat([prompt_worksfor['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
worksfor_roberta_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
worksfor_roberta_spacy.loc[:, 'original'] = worksfor_roberta['original']
worksfor_roberta_spacy.loc[:, 't5_0'] = worksfor_roberta['t5_0']
worksfor_roberta_spacy.loc[:, 't5_1'] = worksfor_roberta['t5_1']
worksfor_roberta_spacy.loc[:, 't5_2'] = worksfor_roberta['t5_2']
worksfor_roberta_spacy.loc[:, 't5_3'] = worksfor_roberta['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
language_roberta_spacy = pd.concat([prompt_language['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
language_roberta_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
language_roberta_spacy.loc[:, 'original'] = language_roberta['original']
language_roberta_spacy.loc[:, 't5_0'] = language_roberta['t5_0']
language_roberta_spacy.loc[:, 't5_1'] = language_roberta['t5_1']
language_roberta_spacy.loc[:, 't5_2'] = language_roberta['t5_2']
language_roberta_spacy.loc[:, 't5_3'] = language_roberta['t5_3']

In [ ]:
# Exemplo de uso
# Supondo que temos um DataFrame `novo` com as colunas especificadas
masked_columns = [
    'triple_NL_1',
    'triple_NL_2',
    'triple_NL_3',
    'triple_NL_4',
    'triple_NL_5'
]

# As colunas que contêm os tokens originais
original_columns = [
    'original', 
    't5_0', 
    't5_1', 
    't5_2', 
    't5_3'
]

# Chamar a função para extrair tokens
bornIn_roberta_spacy = extrair_tokens_dataframe(bornIn_roberta_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
diedIn_roberta_spacy = extrair_tokens_dataframe(diedIn_roberta_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
capital_roberta_spacy = extrair_tokens_dataframe(capital_roberta_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
citizen_roberta_spacy = extrair_tokens_dataframe(citizen_roberta_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
worksfor_roberta_spacy = extrair_tokens_dataframe(worksfor_roberta_spacy, masked_columns, original_columns, nlp, extrair_tokens_org2)
language_roberta_spacy = extrair_tokens_dataframe(language_roberta_spacy, masked_columns, original_columns, nlp, extrair_tokens_lang2)

### Electra

In [ ]:
bornIn_electra = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/electra/outputs_bornIn_electra_v2.csv')
bornIn_electra.drop(columns=['Unnamed: 0'], inplace=True)
bornIn_electra = bornIn_electra.map(str_to_list)


diedIn_electra = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/electra/outputs_diedIn_electra_v2.csv')
diedIn_electra.drop(columns=['Unnamed: 0'], inplace=True)
diedIn_electra = diedIn_electra.map(str_to_list)


capital_electra = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/electra/outputs_capital_electra_v2.csv')
capital_electra.drop(columns=['Unnamed: 0'], inplace=True)
capital_electra = capital_electra.map(str_to_list)


citizen_electra = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/electra/outputs_citizen_electra_v2.csv')
citizen_electra.drop(columns=['Unnamed: 0'], inplace=True)
citizen_electra = citizen_electra.map(str_to_list)


worksfor_electra = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/electra/outputs_worksfor_electra_v2.csv')
worksfor_electra.drop(columns=['Unnamed: 0'], inplace=True)
worksfor_electra = worksfor_electra.map(str_to_list)

language_electra = pd.read_csv('/home/rafael/tese/code/data_v2/outputs/electra/outputs_language_electra_v2.csv')
language_electra.drop(columns=['Unnamed: 0'], inplace=True)
language_electra = language_electra.map(str_to_list)

In [ ]:
# Replicando a coluna 'triple_NL' cinco vezes
bornIn_electra_spacy = pd.concat([prompt_bornIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
bornIn_electra_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
bornIn_electra_spacy.loc[:, 'original'] = bornIn_electra['original']
bornIn_electra_spacy.loc[:, 't5_0'] = bornIn_electra['t5_0']
bornIn_electra_spacy.loc[:, 't5_1'] = bornIn_electra['t5_1']
bornIn_electra_spacy.loc[:, 't5_2'] = bornIn_electra['t5_2']
bornIn_electra_spacy.loc[:, 't5_3'] = bornIn_electra['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
diedIn_electra_spacy = pd.concat([prompt_diedIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
diedIn_electra_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
diedIn_electra_spacy.loc[:, 'original'] = diedIn_electra['original']
diedIn_electra_spacy.loc[:, 't5_0'] = diedIn_electra['t5_0']
diedIn_electra_spacy.loc[:, 't5_1'] = diedIn_electra['t5_1']
diedIn_electra_spacy.loc[:, 't5_2'] = diedIn_electra['t5_2']
diedIn_electra_spacy.loc[:, 't5_3'] = diedIn_electra['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
capital_electra_spacy = pd.concat([prompt_capital['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
capital_electra_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
capital_electra_spacy.loc[:, 'original'] = capital_electra['original']
capital_electra_spacy.loc[:, 't5_0'] = capital_electra['t5_0']
capital_electra_spacy.loc[:, 't5_1'] = capital_electra['t5_1']
capital_electra_spacy.loc[:, 't5_2'] = capital_electra['t5_2']
capital_electra_spacy.loc[:, 't5_3'] = capital_electra['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
citizen_electra_spacy = pd.concat([prompt_citizen['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
citizen_electra_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
citizen_electra_spacy.loc[:, 'original'] = citizen_electra['original']
citizen_electra_spacy.loc[:, 't5_0'] = citizen_electra['t5_0']
citizen_electra_spacy.loc[:, 't5_1'] = citizen_electra['t5_1']
citizen_electra_spacy.loc[:, 't5_2'] = citizen_electra['t5_2']
citizen_electra_spacy.loc[:, 't5_3'] = citizen_electra['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
worksfor_electra_spacy = pd.concat([prompt_worksfor['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
worksfor_electra_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
worksfor_electra_spacy.loc[:, 'original'] = worksfor_electra['original']
worksfor_electra_spacy.loc[:, 't5_0'] = worksfor_electra['t5_0']
worksfor_electra_spacy.loc[:, 't5_1'] = worksfor_electra['t5_1']
worksfor_electra_spacy.loc[:, 't5_2'] = worksfor_electra['t5_2']
worksfor_electra_spacy.loc[:, 't5_3'] = worksfor_electra['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
language_electra_spacy = pd.concat([prompt_language['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
language_electra_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
language_electra_spacy.loc[:, 'original'] = language_electra['original']
language_electra_spacy.loc[:, 't5_0'] = language_electra['t5_0']
language_electra_spacy.loc[:, 't5_1'] = language_electra['t5_1']
language_electra_spacy.loc[:, 't5_2'] = language_electra['t5_2']
language_electra_spacy.loc[:, 't5_3'] = language_electra['t5_3']

In [ ]:
# Exemplo de uso
# Supondo que temos um DataFrame `novo` com as colunas especificadas
masked_columns = [
    'triple_NL_1',
    'triple_NL_2',
    'triple_NL_3',
    'triple_NL_4',
    'triple_NL_5'
]

# As colunas que contêm os tokens originais
original_columns = [
    'original', 
    't5_0', 
    't5_1', 
    't5_2', 
    't5_3'
]

# Chamar a função para extrair tokens
bornIn_electra_spacy = extrair_tokens_dataframe(bornIn_electra_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
diedIn_electra_spacy = extrair_tokens_dataframe(diedIn_electra_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
capital_electra_spacy = extrair_tokens_dataframe(capital_electra_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
citizen_electra_spacy = extrair_tokens_dataframe(citizen_electra_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
worksfor_electra_spacy = extrair_tokens_dataframe(worksfor_electra_spacy, masked_columns, original_columns, nlp, extrair_tokens_org2)
language_electra_spacy = extrair_tokens_dataframe(language_electra_spacy, masked_columns, original_columns, nlp, extrair_tokens_lang2)

## One-Shot

### Bert

In [ ]:
bornIn_bert_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/bert/v2/outputs_bornIn_t5_bert_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
bornIn_bert_one = bornIn_bert_one.map(str_to_list)


diedIn_bert_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/bert/v2/outputs_diedIn_t5_bert_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
diedIn_bert_one = diedIn_bert_one.map(str_to_list)


capital_bert_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/bert/v2/outputs_capital_t5_bert_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
capital_bert_one = capital_bert_one.map(str_to_list)


worksfor_bert_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/bert/v2/outputs_worksfor_t5_bert_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
worksfor_bert_one = worksfor_bert_one.map(str_to_list)


citizen_bert_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/bert/v2/outputs_citizen_t5_bert_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
citizen_bert_one = citizen_bert_one.map(str_to_list)

language_bert_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/bert/v2/outputs_language_t5_bert_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
language_bert_one = language_bert_one.map(str_to_list)

In [ ]:
# Replicando a coluna 'triple_NL' cinco vezes
bornIn_bert_one_spacy = pd.concat([prompt_bornIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
bornIn_bert_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
bornIn_bert_one_spacy.loc[:, 'original'] = bornIn_bert_one['original']
bornIn_bert_one_spacy.loc[:, 't5_0'] = bornIn_bert_one['t5_0']
bornIn_bert_one_spacy.loc[:, 't5_1'] = bornIn_bert_one['t5_1']
bornIn_bert_one_spacy.loc[:, 't5_2'] = bornIn_bert_one['t5_2']
bornIn_bert_one_spacy.loc[:, 't5_3'] = bornIn_bert_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
diedIn_bert_one_spacy = pd.concat([prompt_diedIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
diedIn_bert_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
diedIn_bert_one_spacy.loc[:, 'original'] = diedIn_bert_one['original']
diedIn_bert_one_spacy.loc[:, 't5_0'] = diedIn_bert_one['t5_0']
diedIn_bert_one_spacy.loc[:, 't5_1'] = diedIn_bert_one['t5_1']
diedIn_bert_one_spacy.loc[:, 't5_2'] = diedIn_bert_one['t5_2']
diedIn_bert_one_spacy.loc[:, 't5_3'] = diedIn_bert_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
capital_bert_one_spacy = pd.concat([prompt_capital['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
capital_bert_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
capital_bert_one_spacy.loc[:, 'original'] = capital_bert_one['original']
capital_bert_one_spacy.loc[:, 't5_0'] = capital_bert_one['t5_0']
capital_bert_one_spacy.loc[:, 't5_1'] = capital_bert_one['t5_1']
capital_bert_one_spacy.loc[:, 't5_2'] = capital_bert_one['t5_2']
capital_bert_one_spacy.loc[:, 't5_3'] = capital_bert_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
citizen_bert_one_spacy = pd.concat([prompt_citizen['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
citizen_bert_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
citizen_bert_one_spacy.loc[:, 'original'] = citizen_bert_one['original']
citizen_bert_one_spacy.loc[:, 't5_0'] = citizen_bert_one['t5_0']
citizen_bert_one_spacy.loc[:, 't5_1'] = citizen_bert_one['t5_1']
citizen_bert_one_spacy.loc[:, 't5_2'] = citizen_bert_one['t5_2']
citizen_bert_one_spacy.loc[:, 't5_3'] = citizen_bert_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
worksfor_bert_one_spacy = pd.concat([prompt_worksfor['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
worksfor_bert_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
worksfor_bert_one_spacy.loc[:, 'original'] = worksfor_bert_one['original']
worksfor_bert_one_spacy.loc[:, 't5_0'] = worksfor_bert_one['t5_0']
worksfor_bert_one_spacy.loc[:, 't5_1'] = worksfor_bert_one['t5_1']
worksfor_bert_one_spacy.loc[:, 't5_2'] = worksfor_bert_one['t5_2']
worksfor_bert_one_spacy.loc[:, 't5_3'] = worksfor_bert_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
language_bert_one_spacy = pd.concat([prompt_language['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
language_bert_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
language_bert_one_spacy.loc[:, 'original'] = language_bert_one['original']
language_bert_one_spacy.loc[:, 't5_0'] = language_bert_one['t5_0']
language_bert_one_spacy.loc[:, 't5_1'] = language_bert_one['t5_1']
language_bert_one_spacy.loc[:, 't5_2'] = language_bert_one['t5_2']
language_bert_one_spacy.loc[:, 't5_3'] = language_bert_one['t5_3']

In [ ]:
# Exemplo de uso
# Supondo que temos um DataFrame `novo` com as colunas especificadas
masked_columns = [
    'triple_NL_1',
    'triple_NL_2',
    'triple_NL_3',
    'triple_NL_4',
    'triple_NL_5'
]
# As colunas que contêm os tokens originais
original_columns = [
    'original', 
    't5_0', 
    't5_1', 
    't5_2', 
    't5_3'
]

# Chamar a função para extrair tokens
bornIn_bert_one_spacy = extrair_tokens_dataframe(bornIn_bert_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
diedIn_bert_one_spacy = extrair_tokens_dataframe(diedIn_bert_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
capital_bert_one_spacy = extrair_tokens_dataframe(capital_bert_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
citizen_bert_one_spacy = extrair_tokens_dataframe(citizen_bert_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
worksfor_bert_one_spacy = extrair_tokens_dataframe(worksfor_bert_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_org2)
language_bert_one_spacy = extrair_tokens_dataframe(language_bert_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_lang2)

### Roberta

In [ ]:
bornIn_roberta_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/roberta/v2/outputs_bornIn_t5_roberta_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
bornIn_roberta_one = bornIn_roberta_one.map(str_to_list)


diedIn_roberta_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/roberta/v2/outputs_diedIn_t5_roberta_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
diedIn_roberta_one = diedIn_roberta_one.map(str_to_list)


capital_roberta_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/roberta/v2/outputs_capital_t5_roberta_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
capital_roberta_one = capital_roberta_one.map(str_to_list)


worksfor_roberta_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/roberta/v2/outputs_worksfor_t5_roberta_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
worksfor_roberta_one = worksfor_roberta_one.map(str_to_list)


citizen_roberta_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/roberta/v2/outputs_citizen_t5_roberta_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
citizen_roberta_one = citizen_roberta_one.map(str_to_list)

language_roberta_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/roberta/v2/outputs_language_t5_roberta_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
language_roberta_one = language_roberta_one.map(str_to_list)

In [ ]:
# Replicando a coluna 'triple_NL' cinco vezes
bornIn_roberta_one_spacy = pd.concat([prompt_bornIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
bornIn_roberta_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
bornIn_roberta_one_spacy.loc[:, 'original'] = bornIn_roberta_one['original']
bornIn_roberta_one_spacy.loc[:, 't5_0'] = bornIn_roberta_one['t5_0']
bornIn_roberta_one_spacy.loc[:, 't5_1'] = bornIn_roberta_one['t5_1']
bornIn_roberta_one_spacy.loc[:, 't5_2'] = bornIn_roberta_one['t5_2']
bornIn_roberta_one_spacy.loc[:, 't5_3'] = bornIn_roberta_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
diedIn_roberta_one_spacy = pd.concat([prompt_diedIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
diedIn_roberta_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
diedIn_roberta_one_spacy.loc[:, 'original'] = diedIn_roberta_one['original']
diedIn_roberta_one_spacy.loc[:, 't5_0'] = diedIn_roberta_one['t5_0']
diedIn_roberta_one_spacy.loc[:, 't5_1'] = diedIn_roberta_one['t5_1']
diedIn_roberta_one_spacy.loc[:, 't5_2'] = diedIn_roberta_one['t5_2']
diedIn_roberta_one_spacy.loc[:, 't5_3'] = diedIn_roberta_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
capital_roberta_one_spacy = pd.concat([prompt_capital['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
capital_roberta_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
capital_roberta_one_spacy.loc[:, 'original'] = capital_roberta_one['original']
capital_roberta_one_spacy.loc[:, 't5_0'] = capital_roberta_one['t5_0']
capital_roberta_one_spacy.loc[:, 't5_1'] = capital_roberta_one['t5_1']
capital_roberta_one_spacy.loc[:, 't5_2'] = capital_roberta_one['t5_2']
capital_roberta_one_spacy.loc[:, 't5_3'] = capital_roberta_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
citizen_roberta_one_spacy = pd.concat([prompt_citizen['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
citizen_roberta_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
citizen_roberta_one_spacy.loc[:, 'original'] = citizen_roberta_one['original']
citizen_roberta_one_spacy.loc[:, 't5_0'] = citizen_roberta_one['t5_0']
citizen_roberta_one_spacy.loc[:, 't5_1'] = citizen_roberta_one['t5_1']
citizen_roberta_one_spacy.loc[:, 't5_2'] = citizen_roberta_one['t5_2']
citizen_roberta_one_spacy.loc[:, 't5_3'] = citizen_roberta_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
worksfor_roberta_one_spacy = pd.concat([prompt_worksfor['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
worksfor_roberta_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
worksfor_roberta_one_spacy.loc[:, 'original'] = worksfor_roberta_one['original']
worksfor_roberta_one_spacy.loc[:, 't5_0'] = worksfor_roberta_one['t5_0']
worksfor_roberta_one_spacy.loc[:, 't5_1'] = worksfor_roberta_one['t5_1']
worksfor_roberta_one_spacy.loc[:, 't5_2'] = worksfor_roberta_one['t5_2']
worksfor_roberta_one_spacy.loc[:, 't5_3'] = worksfor_roberta_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
language_roberta_one_spacy = pd.concat([prompt_language['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
language_roberta_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
language_roberta_one_spacy.loc[:, 'original'] = language_roberta_one['original']
language_roberta_one_spacy.loc[:, 't5_0'] = language_roberta_one['t5_0']
language_roberta_one_spacy.loc[:, 't5_1'] = language_roberta_one['t5_1']
language_roberta_one_spacy.loc[:, 't5_2'] = language_roberta_one['t5_2']
language_roberta_one_spacy.loc[:, 't5_3'] = language_roberta_one['t5_3']

In [ ]:
# Exemplo de uso
# Supondo que temos um DataFrame `novo` com as colunas especificadas
masked_columns = [
    'triple_NL_1',
    'triple_NL_2',
    'triple_NL_3',
    'triple_NL_4',
    'triple_NL_5'
]

# As colunas que contêm os tokens originais
original_columns = [
    'original', 
    't5_0', 
    't5_1', 
    't5_2', 
    't5_3'
]

# Chamar a função para extrair tokens
bornIn_roberta_one_spacy = extrair_tokens_dataframe(bornIn_roberta_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
diedIn_roberta_one_spacy = extrair_tokens_dataframe(diedIn_roberta_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
capital_roberta_one_spacy = extrair_tokens_dataframe(capital_roberta_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
citizen_roberta_one_spacy = extrair_tokens_dataframe(citizen_roberta_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
worksfor_roberta_one_spacy = extrair_tokens_dataframe(worksfor_roberta_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_org2)
language_roberta_one_spacy = extrair_tokens_dataframe(language_roberta_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_lang2)

### Electra

In [ ]:
bornIn_electra_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/electra/v2/outputs_bornIn_t5_electra_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
bornIn_electra_one = bornIn_electra_one.map(str_to_list)


diedIn_electra_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/electra/v2/outputs_diedIn_t5_electra_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
diedIn_electra_one = diedIn_electra_one.map(str_to_list)


capital_electra_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/electra/v2/outputs_capital_t5_electra_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
capital_electra_one = capital_electra_one.map(str_to_list)


worksfor_electra_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/electra/v2/outputs_worksfor_t5_electra_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
worksfor_electra_one = worksfor_electra_one.map(str_to_list)


citizen_electra_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/electra/v2/outputs_citizen_t5_electra_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
citizen_electra_one = citizen_electra_one.map(str_to_list)

language_electra_one = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/one/electra/v2/outputs_language_t5_electra_one_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
language_electra_one = language_electra_one.map(str_to_list)

In [ ]:
# Replicando a coluna 'triple_NL' cinco vezes
bornIn_electra_one_spacy = pd.concat([prompt_bornIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
bornIn_electra_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
bornIn_electra_one_spacy.loc[:, 'original'] = bornIn_electra_one['original']
bornIn_electra_one_spacy.loc[:, 't5_0'] = bornIn_electra_one['t5_0']
bornIn_electra_one_spacy.loc[:, 't5_1'] = bornIn_electra_one['t5_1']
bornIn_electra_one_spacy.loc[:, 't5_2'] = bornIn_electra_one['t5_2']
bornIn_electra_one_spacy.loc[:, 't5_3'] = bornIn_electra_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
diedIn_electra_one_spacy = pd.concat([prompt_diedIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
diedIn_electra_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
diedIn_electra_one_spacy.loc[:, 'original'] = diedIn_electra_one['original']
diedIn_electra_one_spacy.loc[:, 't5_0'] = diedIn_electra_one['t5_0']
diedIn_electra_one_spacy.loc[:, 't5_1'] = diedIn_electra_one['t5_1']
diedIn_electra_one_spacy.loc[:, 't5_2'] = diedIn_electra_one['t5_2']
diedIn_electra_one_spacy.loc[:, 't5_3'] = diedIn_electra_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
capital_electra_one_spacy = pd.concat([prompt_capital['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
capital_electra_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
capital_electra_one_spacy.loc[:, 'original'] = capital_electra_one['original']
capital_electra_one_spacy.loc[:, 't5_0'] = capital_electra_one['t5_0']
capital_electra_one_spacy.loc[:, 't5_1'] = capital_electra_one['t5_1']
capital_electra_one_spacy.loc[:, 't5_2'] = capital_electra_one['t5_2']
capital_electra_one_spacy.loc[:, 't5_3'] = capital_electra_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
citizen_electra_one_spacy = pd.concat([prompt_citizen['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
citizen_electra_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
citizen_electra_one_spacy.loc[:, 'original'] = citizen_electra_one['original']
citizen_electra_one_spacy.loc[:, 't5_0'] = citizen_electra_one['t5_0']
citizen_electra_one_spacy.loc[:, 't5_1'] = citizen_electra_one['t5_1']
citizen_electra_one_spacy.loc[:, 't5_2'] = citizen_electra_one['t5_2']
citizen_electra_one_spacy.loc[:, 't5_3'] = citizen_electra_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
worksfor_electra_one_spacy = pd.concat([prompt_worksfor['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
worksfor_electra_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
worksfor_electra_one_spacy.loc[:, 'original'] = worksfor_electra_one['original']
worksfor_electra_one_spacy.loc[:, 't5_0'] = worksfor_electra_one['t5_0']
worksfor_electra_one_spacy.loc[:, 't5_1'] = worksfor_electra_one['t5_1']
worksfor_electra_one_spacy.loc[:, 't5_2'] = worksfor_electra_one['t5_2']
worksfor_electra_one_spacy.loc[:, 't5_3'] = worksfor_electra_one['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
language_electra_one_spacy = pd.concat([prompt_language['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
language_electra_one_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
language_electra_one_spacy.loc[:, 'original'] = language_electra_one['original']
language_electra_one_spacy.loc[:, 't5_0'] = language_electra_one['t5_0']
language_electra_one_spacy.loc[:, 't5_1'] = language_electra_one['t5_1']
language_electra_one_spacy.loc[:, 't5_2'] = language_electra_one['t5_2']
language_electra_one_spacy.loc[:, 't5_3'] = language_electra_one['t5_3']

In [ ]:
# Exemplo de uso
# Supondo que temos um DataFrame `novo` com as colunas especificadas
masked_columns = [
    'triple_NL_1',
    'triple_NL_2',
    'triple_NL_3',
    'triple_NL_4',
    'triple_NL_5'
]

# As colunas que contêm os tokens originais
original_columns = [
    'original', 
    't5_0', 
    't5_1', 
    't5_2', 
    't5_3'
]

# Chamar a função para extrair tokens
bornIn_electra_one_spacy = extrair_tokens_dataframe(bornIn_electra_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
diedIn_electra_one_spacy = extrair_tokens_dataframe(diedIn_electra_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
capital_electra_one_spacy = extrair_tokens_dataframe(capital_electra_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
citizen_electra_one_spacy = extrair_tokens_dataframe(citizen_electra_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
worksfor_electra_one_spacy = extrair_tokens_dataframe(worksfor_electra_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_org2)
language_electra_one_spacy = extrair_tokens_dataframe(language_electra_one_spacy, masked_columns, original_columns, nlp, extrair_tokens_lang2)

## Few-Shot

### Bert

In [ ]:
bornIn_bert_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/bert/v2/outputs_bornIn_t5_bert_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
bornIn_bert_few = bornIn_bert_few.map(str_to_list)

diedIn_bert_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/bert/v2/outputs_diedIn_t5_bert_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
diedIn_bert_few = diedIn_bert_few.map(str_to_list)


capital_bert_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/bert/v2/outputs_capital_t5_bert_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
capital_bert_few = capital_bert_few.map(str_to_list)


worksfor_bert_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/bert/v2/outputs_worksfor_t5_bert_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
worksfor_bert_few = worksfor_bert_few.map(str_to_list)


citizen_bert_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/bert/v2/outputs_citizen_t5_bert_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
citizen_bert_few = citizen_bert_few.map(str_to_list)

language_bert_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/bert/v2/outputs_language_t5_bert_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
language_bert_few = language_bert_few.map(str_to_list)

In [ ]:
# Replicando a coluna 'triple_NL' cinco vezes
bornIn_bert_few_spacy = pd.concat([prompt_bornIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
bornIn_bert_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
bornIn_bert_few_spacy.loc[:, 'original'] = bornIn_bert_few['original']
bornIn_bert_few_spacy.loc[:, 't5_0'] = bornIn_bert_few['t5_0']
bornIn_bert_few_spacy.loc[:, 't5_1'] = bornIn_bert_few['t5_1']
bornIn_bert_few_spacy.loc[:, 't5_2'] = bornIn_bert_few['t5_2']
bornIn_bert_few_spacy.loc[:, 't5_3'] = bornIn_bert_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
diedIn_bert_few_spacy = pd.concat([prompt_diedIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
diedIn_bert_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
diedIn_bert_few_spacy.loc[:, 'original'] = diedIn_bert_few['original']
diedIn_bert_few_spacy.loc[:, 't5_0'] = diedIn_bert_few['t5_0']
diedIn_bert_few_spacy.loc[:, 't5_1'] = diedIn_bert_few['t5_1']
diedIn_bert_few_spacy.loc[:, 't5_2'] = diedIn_bert_few['t5_2']
diedIn_bert_few_spacy.loc[:, 't5_3'] = diedIn_bert_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
capital_bert_few_spacy = pd.concat([prompt_capital['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
capital_bert_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
capital_bert_few_spacy.loc[:, 'original'] = capital_bert_few['original']
capital_bert_few_spacy.loc[:, 't5_0'] = capital_bert_few['t5_0']
capital_bert_few_spacy.loc[:, 't5_1'] = capital_bert_few['t5_1']
capital_bert_few_spacy.loc[:, 't5_2'] = capital_bert_few['t5_2']
capital_bert_few_spacy.loc[:, 't5_3'] = capital_bert_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
citizen_bert_few_spacy = pd.concat([prompt_citizen['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
citizen_bert_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
citizen_bert_few_spacy.loc[:, 'original'] = citizen_bert_few['original']
citizen_bert_few_spacy.loc[:, 't5_0'] = citizen_bert_few['t5_0']
citizen_bert_few_spacy.loc[:, 't5_1'] = citizen_bert_few['t5_1']
citizen_bert_few_spacy.loc[:, 't5_2'] = citizen_bert_few['t5_2']
citizen_bert_few_spacy.loc[:, 't5_3'] = citizen_bert_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
worksfor_bert_few_spacy = pd.concat([prompt_worksfor['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
worksfor_bert_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
worksfor_bert_few_spacy.loc[:, 'original'] = worksfor_bert_few['original']
worksfor_bert_few_spacy.loc[:, 't5_0'] = worksfor_bert_few['t5_0']
worksfor_bert_few_spacy.loc[:, 't5_1'] = worksfor_bert_few['t5_1']
worksfor_bert_few_spacy.loc[:, 't5_2'] = worksfor_bert_few['t5_2']
worksfor_bert_few_spacy.loc[:, 't5_3'] = worksfor_bert_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
language_bert_few_spacy = pd.concat([prompt_language['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
language_bert_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
language_bert_few_spacy.loc[:, 'original'] = language_bert_few['original']
language_bert_few_spacy.loc[:, 't5_0'] = language_bert_few['t5_0']
language_bert_few_spacy.loc[:, 't5_1'] = language_bert_few['t5_1']
language_bert_few_spacy.loc[:, 't5_2'] = language_bert_few['t5_2']
language_bert_few_spacy.loc[:, 't5_3'] = language_bert_few['t5_3']

In [ ]:
# Exemplo de uso
# Supondo que temos um DataFrame `novo` com as colunas especificadas
masked_columns = [
    'triple_NL_1',
    'triple_NL_2',
    'triple_NL_3',
    'triple_NL_4',
    'triple_NL_5'
]

# As colunas que contêm os tokens originais
original_columns = [
    'original', 
    't5_0', 
    't5_1', 
    't5_2', 
    't5_3'
]

# Chamar a função para extrair tokens
bornIn_bert_few_spacy = extrair_tokens_dataframe(bornIn_bert_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
diedIn_bert_few_spacy = extrair_tokens_dataframe(diedIn_bert_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
capital_bert_few_spacy = extrair_tokens_dataframe(capital_bert_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
citizen_bert_few_spacy = extrair_tokens_dataframe(citizen_bert_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
worksfor_bert_few_spacy = extrair_tokens_dataframe(worksfor_bert_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
language_bert_few_spacy = extrair_tokens_dataframe(language_bert_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_lang2)

### Roberta

In [ ]:
bornIn_roberta_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/roberta/v2/outputs_bornIn_t5_roberta_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
bornIn_roberta_few = bornIn_roberta_few.map(str_to_list)


diedIn_roberta_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/roberta/v2/outputs_diedIn_t5_roberta_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
diedIn_roberta_few = diedIn_roberta_few.map(str_to_list)


capital_roberta_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/roberta/v2/outputs_capital_t5_roberta_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
capital_roberta_few = capital_roberta_few.map(str_to_list)


worksfor_roberta_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/roberta/v2/outputs_worksfor_t5_roberta_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
worksfor_roberta_few = worksfor_roberta_few.map(str_to_list)


citizen_roberta_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/roberta/v2/outputs_citizen_t5_roberta_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
citizen_roberta_few = citizen_roberta_few.map(str_to_list)

language_roberta_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/roberta/v2/outputs_language_t5_roberta_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
language_roberta_few = language_roberta_few.map(str_to_list)

In [ ]:
# Replicando a coluna 'triple_NL' cinco vezes
bornIn_roberta_few_spacy = pd.concat([prompt_bornIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
bornIn_roberta_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
bornIn_roberta_few_spacy.loc[:, 'original'] = bornIn_roberta_few['original']
bornIn_roberta_few_spacy.loc[:, 't5_0'] = bornIn_roberta_few['t5_0']
bornIn_roberta_few_spacy.loc[:, 't5_1'] = bornIn_roberta_few['t5_1']
bornIn_roberta_few_spacy.loc[:, 't5_2'] = bornIn_roberta_few['t5_2']
bornIn_roberta_few_spacy.loc[:, 't5_3'] = bornIn_roberta_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
diedIn_roberta_few_spacy = pd.concat([prompt_diedIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
diedIn_roberta_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
diedIn_roberta_few_spacy.loc[:, 'original'] = diedIn_roberta_few['original']
diedIn_roberta_few_spacy.loc[:, 't5_0'] = diedIn_roberta_few['t5_0']
diedIn_roberta_few_spacy.loc[:, 't5_1'] = diedIn_roberta_few['t5_1']
diedIn_roberta_few_spacy.loc[:, 't5_2'] = diedIn_roberta_few['t5_2']
diedIn_roberta_few_spacy.loc[:, 't5_3'] = diedIn_roberta_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
capital_roberta_few_spacy = pd.concat([prompt_capital['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
capital_roberta_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
capital_roberta_few_spacy.loc[:, 'original'] = capital_roberta_few['original']
capital_roberta_few_spacy.loc[:, 't5_0'] = capital_roberta_few['t5_0']
capital_roberta_few_spacy.loc[:, 't5_1'] = capital_roberta_few['t5_1']
capital_roberta_few_spacy.loc[:, 't5_2'] = capital_roberta_few['t5_2']
capital_roberta_few_spacy.loc[:, 't5_3'] = capital_roberta_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
citizen_roberta_few_spacy = pd.concat([prompt_citizen['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
citizen_roberta_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
citizen_roberta_few_spacy.loc[:, 'original'] = citizen_roberta_few['original']
citizen_roberta_few_spacy.loc[:, 't5_0'] = citizen_roberta_few['t5_0']
citizen_roberta_few_spacy.loc[:, 't5_1'] = citizen_roberta_few['t5_1']
citizen_roberta_few_spacy.loc[:, 't5_2'] = citizen_roberta_few['t5_2']
citizen_roberta_few_spacy.loc[:, 't5_3'] = citizen_roberta_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
worksfor_roberta_few_spacy = pd.concat([prompt_worksfor['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
worksfor_roberta_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
worksfor_roberta_few_spacy.loc[:, 'original'] = worksfor_roberta_few['original']
worksfor_roberta_few_spacy.loc[:, 't5_0'] = worksfor_roberta_few['t5_0']
worksfor_roberta_few_spacy.loc[:, 't5_1'] = worksfor_roberta_few['t5_1']
worksfor_roberta_few_spacy.loc[:, 't5_2'] = worksfor_roberta_few['t5_2']
worksfor_roberta_few_spacy.loc[:, 't5_3'] = worksfor_roberta_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
language_roberta_few_spacy = pd.concat([prompt_language['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
language_roberta_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
language_roberta_few_spacy.loc[:, 'original'] = language_roberta_few['original']
language_roberta_few_spacy.loc[:, 't5_0'] = language_roberta_few['t5_0']
language_roberta_few_spacy.loc[:, 't5_1'] = language_roberta_few['t5_1']
language_roberta_few_spacy.loc[:, 't5_2'] = language_roberta_few['t5_2']
language_roberta_few_spacy.loc[:, 't5_3'] = language_roberta_few['t5_3']

In [ ]:
# Exemplo de uso
# Supondo que temos um DataFrame `novo` com as colunas especificadas
masked_columns = [
    'triple_NL_1',
    'triple_NL_2',
    'triple_NL_3',
    'triple_NL_4',
    'triple_NL_5'
]

# As colunas que contêm os tokens originais
original_columns = [
    'original', 
    't5_0', 
    't5_1', 
    't5_2', 
    't5_3'
]

# Chamar a função para extrair tokens
bornIn_roberta_few_spacy = extrair_tokens_dataframe(bornIn_roberta_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
diedIn_roberta_few_spacy = extrair_tokens_dataframe(diedIn_roberta_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
capital_roberta_few_spacy = extrair_tokens_dataframe(capital_roberta_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
citizen_roberta_few_spacy = extrair_tokens_dataframe(citizen_roberta_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
worksfor_roberta_few_spacy = extrair_tokens_dataframe(worksfor_roberta_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_org2)
language_roberta_few_spacy = extrair_tokens_dataframe(language_roberta_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_lang2)

### Electra

In [ ]:
bornIn_electra_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/electra/v2/outputs_bornIn_t5_electra_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
bornIn_electra_few = bornIn_electra_few.map(str_to_list)


diedIn_electra_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/electra/v2/outputs_diedIn_t5_electra_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
diedIn_electra_few = diedIn_electra_few.map(str_to_list)


capital_electra_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/electra/v2/outputs_capital_t5_electra_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
capital_electra_few = capital_electra_few.map(str_to_list)


worksfor_electra_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/electra/v2/outputs_worksfor_t5_electra_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
worksfor_electra_few = worksfor_electra_few.map(str_to_list)


citizen_electra_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/electra/v2/outputs_citizen_t5_electra_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
citizen_electra_few = citizen_electra_few.map(str_to_list)

language_electra_few = pd.read_csv('/home/rafael/tese/code/data/novos_data/dados/outputs/One_Few-Shoot/few/electra/v2/outputs_language_t5_electra_few_shoot_v2.csv')\
    .drop(columns=['Unnamed: 0'])
language_electra_few = language_electra_few.map(str_to_list)

In [ ]:
# Replicando a coluna 'triple_NL' cinco vezes
bornIn_electra_few_spacy = pd.concat([prompt_bornIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
bornIn_electra_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
bornIn_electra_few_spacy.loc[:, 'original'] = bornIn_electra_few['original']
bornIn_electra_few_spacy.loc[:, 't5_0'] = bornIn_electra_few['t5_0']
bornIn_electra_few_spacy.loc[:, 't5_1'] = bornIn_electra_few['t5_1']
bornIn_electra_few_spacy.loc[:, 't5_2'] = bornIn_electra_few['t5_2']
bornIn_electra_few_spacy.loc[:, 't5_3'] = bornIn_electra_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
diedIn_electra_few_spacy = pd.concat([prompt_diedIn['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
diedIn_electra_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
diedIn_electra_few_spacy.loc[:, 'original'] = diedIn_electra_few['original']
diedIn_electra_few_spacy.loc[:, 't5_0'] = diedIn_electra_few['t5_0']
diedIn_electra_few_spacy.loc[:, 't5_1'] = diedIn_electra_few['t5_1']
diedIn_electra_few_spacy.loc[:, 't5_2'] = diedIn_electra_few['t5_2']
diedIn_electra_few_spacy.loc[:, 't5_3'] = diedIn_electra_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
capital_electra_few_spacy = pd.concat([prompt_capital['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
capital_electra_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
capital_electra_few_spacy.loc[:, 'original'] = capital_electra_few['original']
capital_electra_few_spacy.loc[:, 't5_0'] = capital_electra_few['t5_0']
capital_electra_few_spacy.loc[:, 't5_1'] = capital_electra_few['t5_1']
capital_electra_few_spacy.loc[:, 't5_2'] = capital_electra_few['t5_2']
capital_electra_few_spacy.loc[:, 't5_3'] = capital_electra_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
citizen_electra_few_spacy = pd.concat([prompt_citizen['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
citizen_electra_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
citizen_electra_few_spacy.loc[:, 'original'] = citizen_electra_few['original']
citizen_electra_few_spacy.loc[:, 't5_0'] = citizen_electra_few['t5_0']
citizen_electra_few_spacy.loc[:, 't5_1'] = citizen_electra_few['t5_1']
citizen_electra_few_spacy.loc[:, 't5_2'] = citizen_electra_few['t5_2']
citizen_electra_few_spacy.loc[:, 't5_3'] = citizen_electra_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
worksfor_electra_few_spacy = pd.concat([prompt_worksfor['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
worksfor_electra_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
worksfor_electra_few_spacy.loc[:, 'original'] = worksfor_electra_few['original']
worksfor_electra_few_spacy.loc[:, 't5_0'] = worksfor_electra_few['t5_0']
worksfor_electra_few_spacy.loc[:, 't5_1'] = worksfor_electra_few['t5_1']
worksfor_electra_few_spacy.loc[:, 't5_2'] = worksfor_electra_few['t5_2']
worksfor_electra_few_spacy.loc[:, 't5_3'] = worksfor_electra_few['t5_3']

# Replicando a coluna 'triple_NL' cinco vezes
language_electra_few_spacy = pd.concat([prompt_language['triple_NL']] * 5, axis=1)

# Renomeando as novas colunas
new_columns = [f'triple_NL_{i+1}' for i in range(5)]
language_electra_few_spacy.columns = new_columns

# Using .loc to avoid SettingWithCopyWarning
language_electra_few_spacy.loc[:, 'original'] = language_electra_few['original']
language_electra_few_spacy.loc[:, 't5_0'] = language_electra_few['t5_0']
language_electra_few_spacy.loc[:, 't5_1'] = language_electra_few['t5_1']
language_electra_few_spacy.loc[:, 't5_2'] = language_electra_few['t5_2']
language_electra_few_spacy.loc[:, 't5_3'] = language_electra_few['t5_3']

In [ ]:
# Exemplo de uso
# Supondo que temos um DataFrame `novo` com as colunas especificadas
masked_columns = [
    'triple_NL_1',
    'triple_NL_2',
    'triple_NL_3',
    'triple_NL_4',
    'triple_NL_5'
]

# As colunas que contêm os tokens originais
original_columns = [
    'original', 
    't5_0', 
    't5_1', 
    't5_2', 
    't5_3'
]

# Chamar a função para extrair tokens
bornIn_electra_few_spacy = extrair_tokens_dataframe(bornIn_electra_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
diedIn_electra_few_spacy = extrair_tokens_dataframe(diedIn_electra_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
capital_electra_few_spacy = extrair_tokens_dataframe(capital_electra_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
citizen_electra_few_spacy = extrair_tokens_dataframe(citizen_electra_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_gpe2)
worksfor_electra_few_spacy = extrair_tokens_dataframe(worksfor_electra_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_org2)
language_electra_few_spacy = extrair_tokens_dataframe(language_electra_few_spacy, masked_columns, original_columns, nlp, extrair_tokens_lang2)

# Escolha do melhor score

## Spacy

### Bert

##### BornIn

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_bert_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_bert_spacy.loc[:, bornIn_bert_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_bert_spacy[idx] = lista_linha

tokens_bornIn_bert_spacy = []
scores_bornIn_bert_spacy = []
for lista in listas_bornIn_bert_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_bert_spacy.append(tokens)
    scores_bornIn_bert_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_bert_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_bert_one_spacy.loc[:, bornIn_bert_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_bert_one_spacy[idx] = lista_linha

tokens_bornIn_bert_one_spacy = []
scores_bornIn_bert_one_spacy = []
for lista in listas_bornIn_bert_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_bert_one_spacy.append(tokens)
    scores_bornIn_bert_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_bert_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_bert_few_spacy.loc[:, bornIn_bert_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_bert_few_spacy[idx] = lista_linha

tokens_bornIn_bert_few_spacy = []
scores_bornIn_bert_few_spacy = []
for lista in listas_bornIn_bert_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_bert_few_spacy.append(tokens)
    scores_bornIn_bert_few_spacy.append(scores)

##### DiedIn

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_bert_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_bert_spacy.loc[:, diedIn_bert_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_bert_spacy[idx] = lista_linha

tokens_diedIn_bert_spacy = []
scores_diedIn_bert_spacy = []
for lista in listas_diedIn_bert_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_bert_spacy.append(tokens)
    scores_diedIn_bert_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_bert_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_bert_one_spacy.loc[:, diedIn_bert_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_bert_one_spacy[idx] = lista_linha

tokens_diedIn_bert_one_spacy = []
scores_diedIn_bert_one_spacy = []
for lista in listas_diedIn_bert_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_bert_one_spacy.append(tokens)
    scores_diedIn_bert_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_bert_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_bert_few_spacy.loc[:, diedIn_bert_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_bert_few_spacy[idx] = lista_linha

tokens_diedIn_bert_few_spacy = []
scores_diedIn_bert_few_spacy = []
for lista in listas_diedIn_bert_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_bert_few_spacy.append(tokens)
    scores_diedIn_bert_few_spacy.append(scores)

##### Worksfor

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_bert_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_bert_spacy.loc[:, worksfor_bert_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_bert_spacy[idx] = lista_linha

tokens_worksfor_bert_spacy = []
scores_worksfor_bert_spacy = []
for lista in listas_worksfor_bert_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_bert_spacy.append(tokens)
    scores_worksfor_bert_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_bert_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_bert_one_spacy.loc[:, worksfor_bert_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_bert_one_spacy[idx] = lista_linha

tokens_worksfor_bert_one_spacy = []
scores_worksfor_bert_one_spacy = []
for lista in listas_worksfor_bert_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_bert_one_spacy.append(tokens)
    scores_worksfor_bert_one_spacy.append(scores)

In [ ]:
worksfor_bert_one_spacy.columns 

Index(['original', 't5_0', 't5_1', 't5_2', 't5_3'], dtype='object')

In [ ]:
worksfor_bert_few_spacy.columns

Index(['original', 't5_0', 't5_1', 't5_2', 't5_3'], dtype='object')

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_bert_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_bert_few_spacy.loc[:, worksfor_bert_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        print(coluna, valor)
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_bert_few_spacy[idx] = lista_linha

tokens_worksfor_bert_few_spacy = []
scores_worksfor_bert_few_spacy = []
for lista in listas_worksfor_bert_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_bert_few_spacy.append(tokens)
    scores_worksfor_bert_few_spacy.append(scores)

original []
t5_0 []
t5_1 []
t5_2 []
t5_3 []
original []
t5_0 []
t5_1 []
t5_2 []
t5_3 []
original []
t5_0 []
t5_1 []
t5_2 []
t5_3 []
original []
t5_0 []
t5_1 []
t5_2 []
t5_3 []
original []
t5_0 []
t5_1 []
t5_2 [{'token': 'uk', 'score': 0.001}]
t5_3 []
original []
t5_0 []
t5_1 []
t5_2 []
t5_3 []
original []
t5_0 []
t5_1 []
t5_2 [{'token': 'uk', 'score': 0.001}]
t5_3 []
original []
t5_0 []
t5_1 []
t5_2 []
t5_3 []
original []
t5_0 [{'token': 'berkeley', 'score': 0.01}]
t5_1 [{'token': 'berkeley', 'score': 0.006}]
t5_2 []
t5_3 []
original []
t5_0 []
t5_1 []
t5_2 []
t5_3 []
original []
t5_0 []
t5_1 []
t5_2 []
t5_3 []
original []
t5_0 []
t5_1 []
t5_2 []
t5_3 []
original [{'token': 'gandhi', 'score': 0.067}, {'token': 'britain', 'score': 0.022}]
t5_0 [{'token': 'washington', 'score': 0.063}, {'token': 'lincoln', 'score': 0.036}, {'token': 'gandhi', 'score': 0.023}]
t5_1 [{'token': 'gandhi', 'score': 0.035}, {'token': 'melbourne', 'score': 0.016}]
t5_2 [{'token': 'gandhi', 'score': 0.081}, {'to

In [ ]:
worksfor_bert_few_spacy

,original,t5_0,t5_1,t5_2,t5_3
0,[],[],[],[],[]
1,[],[],[],[],[]
2,[],[],[],[],[]
3,[],[],[],[],[]
4,[],[],[],"[{'token': 'uk', 'score': 0.001}]",[]
...,...,...,...,...,...
374,[],[],[],"[{'token': 'bavaria', 'score': 0.028}]",[]
375,"[{'token': 'punjab', 'score': 0.057}]","[{'token': 'punjab', 'score': 0.021}]","[{'token': 'punjab', 'score': 0.011}]","[{'token': 'punjab', 'score': 0.065}]",[]
376,"[{'token': 'seattle', 'score': 0.062}]","[{'token': 'dupont', 'score': 0.022}]",[],[],[]
377,[],[],[],[],[]


##### Citizen

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_bert_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_bert_spacy.loc[:, citizen_bert_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_bert_spacy[idx] = lista_linha

tokens_citizen_bert_spacy = []
scores_citizen_bert_spacy = []
for lista in listas_citizen_bert_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_bert_spacy.append(tokens)
    scores_citizen_bert_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_bert_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_bert_one_spacy.loc[:, citizen_bert_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_bert_one_spacy[idx] = lista_linha

tokens_citizen_bert_one_spacy = []
scores_citizen_bert_one_spacy = []
for lista in listas_citizen_bert_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_bert_one_spacy.append(tokens)
    scores_citizen_bert_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_bert_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_bert_few_spacy.loc[:, citizen_bert_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_bert_few_spacy[idx] = lista_linha

tokens_citizen_bert_few_spacy = []
scores_citizen_bert_few_spacy = []
for lista in listas_citizen_bert_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_bert_few_spacy.append(tokens)
    scores_citizen_bert_few_spacy.append(scores)

##### Capital

In [ ]:
# Dicionário para armazenar as listas
listas_capital_bert_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_bert_spacy.loc[:, capital_bert_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_bert_spacy[idx] = lista_linha

tokens_capital_bert_spacy = []
scores_capital_bert_spacy = []
for lista in listas_capital_bert_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_bert_spacy.append(tokens)
    scores_capital_bert_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_bert_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_bert_one_spacy.loc[:, capital_bert_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_bert_one_spacy[idx] = lista_linha

tokens_capital_bert_one_spacy = []
scores_capital_bert_one_spacy = []
for lista in listas_capital_bert_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_bert_one_spacy.append(tokens)
    scores_capital_bert_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_bert_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_bert_few_spacy.loc[:, capital_bert_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_bert_few_spacy[idx] = lista_linha

tokens_capital_bert_few_spacy = []
scores_capital_bert_few_spacy = []
for lista in listas_capital_bert_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_bert_few_spacy.append(tokens)
    scores_capital_bert_few_spacy.append(scores)

##### Language

In [ ]:
# Dicionário para armazenar as listas
listas_language_bert_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_bert_spacy.loc[:, language_bert_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_bert_spacy[idx] = lista_linha

tokens_language_bert_spacy = []
scores_language_bert_spacy = []
for lista in listas_language_bert_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_bert_spacy.append(tokens)
    scores_language_bert_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_bert_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_bert_one_spacy.loc[:, capital_bert_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_bert_one_spacy[idx] = lista_linha

tokens_capital_bert_one_spacy = []
scores_capital_bert_one_spacy = []
for lista in listas_capital_bert_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_bert_one_spacy.append(tokens)
    scores_capital_bert_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_bert_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_bert_few_spacy.loc[:, capital_bert_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_bert_few_spacy[idx] = lista_linha

tokens_capital_bert_few_spacy = []
scores_capital_bert_few_spacy = []
for lista in listas_capital_bert_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_bert_few_spacy.append(tokens)
    scores_capital_bert_few_spacy.append(scores)

### Roberta

##### BornIn

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_roberta_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_roberta_spacy.loc[:, bornIn_roberta_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_roberta_spacy[idx] = lista_linha

tokens_bornIn_roberta_spacy = []
scores_bornIn_roberta_spacy = []
for lista in listas_bornIn_roberta_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_roberta_spacy.append(tokens)
    scores_bornIn_roberta_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_roberta_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_roberta_one_spacy.loc[:, bornIn_roberta_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_roberta_one_spacy[idx] = lista_linha

tokens_bornIn_roberta_one_spacy = []
scores_bornIn_roberta_one_spacy = []
for lista in listas_bornIn_roberta_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_roberta_one_spacy.append(tokens)
    scores_bornIn_roberta_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_roberta_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_roberta_few_spacy.loc[:, bornIn_roberta_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_roberta_few_spacy[idx] = lista_linha

tokens_bornIn_roberta_few_spacy = []
scores_bornIn_roberta_few_spacy = []
for lista in listas_bornIn_roberta_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_roberta_few_spacy.append(tokens)
    scores_bornIn_roberta_few_spacy.append(scores)

##### DiedIn

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_roberta_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_roberta_spacy.loc[:, diedIn_roberta_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_roberta_spacy[idx] = lista_linha

tokens_diedIn_roberta_spacy = []
scores_diedIn_roberta_spacy = []
for lista in listas_diedIn_roberta_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_roberta_spacy.append(tokens)
    scores_diedIn_roberta_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_roberta_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_roberta_one_spacy.loc[:, diedIn_roberta_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_roberta_one_spacy[idx] = lista_linha

tokens_diedIn_roberta_one_spacy = []
scores_diedIn_roberta_one_spacy = []
for lista in listas_diedIn_roberta_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_roberta_one_spacy.append(tokens)
    scores_diedIn_roberta_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_roberta_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_roberta_few_spacy.loc[:, diedIn_roberta_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_roberta_few_spacy[idx] = lista_linha

tokens_diedIn_roberta_few_spacy = []
scores_diedIn_roberta_few_spacy = []
for lista in listas_diedIn_roberta_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_roberta_few_spacy.append(tokens)
    scores_diedIn_roberta_few_spacy.append(scores)

##### Worksfor

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_roberta_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_roberta_spacy.loc[:, worksfor_roberta_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_roberta_spacy[idx] = lista_linha

tokens_worksfor_roberta_spacy = []
scores_worksfor_roberta_spacy = []
for lista in listas_worksfor_roberta_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_roberta_spacy.append(tokens)
    scores_worksfor_roberta_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_roberta_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_roberta_one_spacy.loc[:, worksfor_roberta_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_roberta_one_spacy[idx] = lista_linha

tokens_worksfor_roberta_one_spacy = []
scores_worksfor_roberta_one_spacy = []
for lista in listas_worksfor_roberta_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_roberta_one_spacy.append(tokens)
    scores_worksfor_roberta_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_roberta_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_roberta_few_spacy.loc[:, worksfor_roberta_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_roberta_few_spacy[idx] = lista_linha

tokens_worksfor_roberta_few_spacy = []
scores_worksfor_roberta_few_spacy = []
for lista in listas_worksfor_roberta_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_roberta_few_spacy.append(tokens)
    scores_worksfor_roberta_few_spacy.append(scores)

##### Citizen

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_roberta_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_roberta_spacy.loc[:, citizen_roberta_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_roberta_spacy[idx] = lista_linha

tokens_citizen_roberta_spacy = []
scores_citizen_roberta_spacy = []
for lista in listas_citizen_roberta_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_roberta_spacy.append(tokens)
    scores_citizen_roberta_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_roberta_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_roberta_one_spacy.loc[:, citizen_roberta_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_roberta_one_spacy[idx] = lista_linha

tokens_citizen_roberta_one_spacy = []
scores_citizen_roberta_one_spacy = []
for lista in listas_citizen_roberta_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_roberta_one_spacy.append(tokens)
    scores_citizen_roberta_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_roberta_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_roberta_few_spacy.loc[:, citizen_roberta_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_roberta_few_spacy[idx] = lista_linha

tokens_citizen_roberta_few_spacy = []
scores_citizen_roberta_few_spacy = []
for lista in listas_citizen_roberta_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_roberta_few_spacy.append(tokens)
    scores_citizen_roberta_few_spacy.append(scores)

##### Capital

In [ ]:
# Dicionário para armazenar as listas
listas_capital_roberta_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_roberta_spacy.loc[:, capital_roberta_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_roberta_spacy[idx] = lista_linha

tokens_capital_roberta_spacy = []
scores_capital_roberta_spacy = []
for lista in listas_capital_roberta_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_roberta_spacy.append(tokens)
    scores_capital_roberta_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_roberta_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_roberta_one_spacy.loc[:, capital_roberta_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_roberta_one_spacy[idx] = lista_linha

tokens_capital_roberta_one_spacy = []
scores_capital_roberta_one_spacy = []
for lista in listas_capital_roberta_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_roberta_one_spacy.append(tokens)
    scores_capital_roberta_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_roberta_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_roberta_few_spacy.loc[:, capital_roberta_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_roberta_few_spacy[idx] = lista_linha

tokens_capital_roberta_few_spacy = []
scores_capital_roberta_few_spacy = []
for lista in listas_capital_roberta_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_roberta_few_spacy.append(tokens)
    scores_capital_roberta_few_spacy.append(scores)

##### Language

In [ ]:
# Dicionário para armazenar as listas
listas_language_roberta_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_roberta_spacy.loc[:, language_roberta_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_roberta_spacy[idx] = lista_linha

tokens_language_roberta_spacy = []
scores_language_roberta_spacy = []
for lista in listas_language_roberta_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_roberta_spacy.append(tokens)
    scores_language_roberta_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_language_roberta_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_roberta_one_spacy.loc[:, language_roberta_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_roberta_one_spacy[idx] = lista_linha

tokens_language_roberta_one_spacy = []
scores_language_roberta_one_spacy = []
for lista in listas_language_roberta_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_roberta_one_spacy.append(tokens)
    scores_language_roberta_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_language_roberta_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_roberta_few_spacy.loc[:, language_roberta_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_roberta_few_spacy[idx] = lista_linha

tokens_language_roberta_few_spacy = []
scores_language_roberta_few_spacy = []
for lista in listas_language_roberta_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_roberta_few_spacy.append(tokens)
    scores_language_roberta_few_spacy.append(scores)

### Electra

##### BornIn

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_electra_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_electra_spacy.loc[:, bornIn_electra_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_electra_spacy[idx] = lista_linha

tokens_bornIn_electra_spacy = []
scores_bornIn_electra_spacy = []
for lista in listas_bornIn_electra_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_electra_spacy.append(tokens)
    scores_bornIn_electra_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_electra_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_electra_one_spacy.loc[:, bornIn_electra_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_electra_one_spacy[idx] = lista_linha

tokens_bornIn_electra_one_spacy = []
scores_bornIn_electra_one_spacy = []
for lista in listas_bornIn_electra_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_electra_one_spacy.append(tokens)
    scores_bornIn_electra_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_electra_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_electra_few_spacy.loc[:, bornIn_electra_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_electra_few_spacy[idx] = lista_linha

tokens_bornIn_electra_few_spacy = []
scores_bornIn_electra_few_spacy = []
for lista in listas_bornIn_electra_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_electra_few_spacy.append(tokens)
    scores_bornIn_electra_few_spacy.append(scores)

##### DiedIn

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_electra_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_electra_spacy.loc[:, diedIn_electra_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_electra_spacy[idx] = lista_linha

tokens_diedIn_electra_spacy = []
scores_diedIn_electra_spacy = []
for lista in listas_diedIn_electra_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_electra_spacy.append(tokens)
    scores_diedIn_electra_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_electra_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_electra_one_spacy.loc[:, diedIn_electra_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_electra_one_spacy[idx] = lista_linha

tokens_diedIn_electra_one_spacy = []
scores_diedIn_electra_one_spacy = []
for lista in listas_diedIn_electra_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_electra_one_spacy.append(tokens)
    scores_diedIn_electra_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_electra_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_electra_few_spacy.loc[:, diedIn_electra_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_electra_few_spacy[idx] = lista_linha

tokens_diedIn_electra_few_spacy = []
scores_diedIn_electra_few_spacy = []
for lista in listas_diedIn_electra_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_electra_few_spacy.append(tokens)
    scores_diedIn_electra_few_spacy.append(scores)

##### Worksfor

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_electra_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_electra_spacy.loc[:, worksfor_electra_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_electra_spacy[idx] = lista_linha

tokens_worksfor_electra_spacy = []
scores_worksfor_electra_spacy = []
for lista in listas_worksfor_electra_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_electra_spacy.append(tokens)
    scores_worksfor_electra_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_electra_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_electra_one_spacy.loc[:, worksfor_electra_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_electra_one_spacy[idx] = lista_linha

tokens_worksfor_electra_one_spacy = []
scores_worksfor_electra_one_spacy = []
for lista in listas_worksfor_electra_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_electra_one_spacy.append(tokens)
    scores_worksfor_electra_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_electra_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_electra_few_spacy.loc[:, worksfor_electra_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_electra_few_spacy[idx] = lista_linha

tokens_worksfor_electra_few_spacy = []
scores_worksfor_electra_few_spacy = []
for lista in listas_worksfor_electra_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_electra_few_spacy.append(tokens)
    scores_worksfor_electra_few_spacy.append(scores)

##### Citizen

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_electra_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_electra_spacy.loc[:, citizen_electra_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_electra_spacy[idx] = lista_linha

tokens_citizen_electra_spacy = []
scores_citizen_electra_spacy = []
for lista in listas_citizen_electra_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_electra_spacy.append(tokens)
    scores_citizen_electra_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_electra_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_electra_one_spacy.loc[:, citizen_electra_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_electra_one_spacy[idx] = lista_linha

tokens_citizen_electra_one_spacy = []
scores_citizen_electra_one_spacy = []
for lista in listas_citizen_electra_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_electra_one_spacy.append(tokens)
    scores_citizen_electra_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_electra_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_electra_few_spacy.loc[:, citizen_electra_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_electra_few_spacy[idx] = lista_linha

tokens_citizen_electra_few_spacy = []
scores_citizen_electra_few_spacy = []
for lista in listas_citizen_electra_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_electra_few_spacy.append(tokens)
    scores_citizen_electra_few_spacy.append(scores)

##### Capital

In [ ]:
# Dicionário para armazenar as listas
listas_capital_electra_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_electra_spacy.loc[:, capital_electra_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_electra_spacy[idx] = lista_linha

tokens_capital_electra_spacy = []
scores_capital_electra_spacy = []
for lista in listas_capital_electra_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_electra_spacy.append(tokens)
    scores_capital_electra_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_electra_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_electra_one_spacy.loc[:, capital_electra_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_electra_one_spacy[idx] = lista_linha

tokens_capital_electra_one_spacy = []
scores_capital_electra_one_spacy = []
for lista in listas_capital_electra_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_electra_one_spacy.append(tokens)
    scores_capital_electra_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_electra_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_electra_few_spacy.loc[:, capital_electra_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_electra_few_spacy[idx] = lista_linha

tokens_capital_electra_few_spacy = []
scores_capital_electra_few_spacy = []
for lista in listas_capital_electra_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_electra_few_spacy.append(tokens)
    scores_capital_electra_few_spacy.append(scores)

##### Language

In [ ]:
# Dicionário para armazenar as listas
listas_language_electra_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_electra_spacy.loc[:, language_electra_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_electra_spacy[idx] = lista_linha

tokens_language_electra_spacy = []
scores_language_electra_spacy = []
for lista in listas_language_electra_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_electra_spacy.append(tokens)
    scores_language_electra_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_language_electra_one_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_electra_one_spacy.loc[:, language_electra_one_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_electra_one_spacy[idx] = lista_linha

tokens_language_electra_one_spacy = []
scores_language_electra_one_spacy = []
for lista in listas_language_electra_one_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_electra_one_spacy.append(tokens)
    scores_language_electra_one_spacy.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_language_electra_few_spacy = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_electra_few_spacy.loc[:, language_electra_few_spacy.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_electra_few_spacy[idx] = lista_linha

tokens_language_electra_few_spacy = []
scores_language_electra_few_spacy = []
for lista in listas_language_electra_few_spacy.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_electra_few_spacy.append(tokens)
    scores_language_electra_few_spacy.append(scores)

## Normal

### Bert

##### BornIn

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_bert = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_bert.loc[:, bornIn_bert.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_bert[idx] = lista_linha

tokens_bornIn_bert = []
scores_bornIn_bert = []
for lista in listas_bornIn_bert.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_bert.append(tokens)
    scores_bornIn_bert.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_bert_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_bert_one.loc[:, bornIn_bert_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_bert_one[idx] = lista_linha

tokens_bornIn_bert_one = []
scores_bornIn_bert_one = []
for lista in listas_bornIn_bert_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_bert_one.append(tokens)
    scores_bornIn_bert_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_bert_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_bert_few.loc[:, bornIn_bert_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_bert_few[idx] = lista_linha

tokens_bornIn_bert_few = []
scores_bornIn_bert_few = []
for lista in listas_bornIn_bert_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_bert_few.append(tokens)
    scores_bornIn_bert_few.append(scores)

##### DiedIn

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_bert = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_bert.loc[:, diedIn_bert.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_bert[idx] = lista_linha

tokens_diedIn_bert = []
scores_diedIn_bert = []
for lista in listas_diedIn_bert.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_bert.append(tokens)
    scores_diedIn_bert.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_bert_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_bert_one.loc[:, diedIn_bert_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_bert_one[idx] = lista_linha

tokens_diedIn_bert_one = []
scores_diedIn_bert_one = []
for lista in listas_diedIn_bert_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_bert_one.append(tokens)
    scores_diedIn_bert_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_bert_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_bert_few.loc[:, diedIn_bert_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_bert_few[idx] = lista_linha

tokens_diedIn_bert_few = []
scores_diedIn_bert_few = []
for lista in listas_diedIn_bert_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_bert_few.append(tokens)
    scores_diedIn_bert_few.append(scores)

##### Worksfor

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_bert = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_bert.loc[:, worksfor_bert.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_bert[idx] = lista_linha

tokens_worksfor_bert = []
scores_worksfor_bert = []
for lista in listas_worksfor_bert.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_bert.append(tokens)
    scores_worksfor_bert.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_bert_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_bert_one.loc[:, worksfor_bert_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_bert_one[idx] = lista_linha

tokens_worksfor_bert_one = []
scores_worksfor_bert_one = []
for lista in listas_worksfor_bert_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_bert_one.append(tokens)
    scores_worksfor_bert_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_bert_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_bert_few.loc[:, worksfor_bert_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_bert_few[idx] = lista_linha

tokens_worksfor_bert_few = []
scores_worksfor_bert_few = []
for lista in listas_worksfor_bert_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_bert_few.append(tokens)
    scores_worksfor_bert_few.append(scores)

##### Citizen

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_bert = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_bert.loc[:, citizen_bert.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_bert[idx] = lista_linha

tokens_citizen_bert = []
scores_citizen_bert = []
for lista in listas_citizen_bert.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_bert.append(tokens)
    scores_citizen_bert.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_bert_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_bert_one.loc[:, citizen_bert_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_bert_one[idx] = lista_linha

tokens_citizen_bert_one = []
scores_citizen_bert_one = []
for lista in listas_citizen_bert_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_bert_one.append(tokens)
    scores_citizen_bert_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_bert_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_bert_few.loc[:, citizen_bert_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_bert_few[idx] = lista_linha

tokens_citizen_bert_few = []
scores_citizen_bert_few = []
for lista in listas_citizen_bert_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_bert_few.append(tokens)
    scores_citizen_bert_few.append(scores)

##### Capital

In [ ]:
# Dicionário para armazenar as listas
listas_capital_bert = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_bert.loc[:, capital_bert.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_bert[idx] = lista_linha

tokens_capital_bert = []
scores_capital_bert = []
for lista in listas_capital_bert.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_bert.append(tokens)
    scores_capital_bert.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_bert_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_bert_one.loc[:, capital_bert_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_bert_one[idx] = lista_linha

tokens_capital_bert_one = []
scores_capital_bert_one = []
for lista in listas_capital_bert_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_bert_one.append(tokens)
    scores_capital_bert_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_bert_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_bert_few.loc[:, capital_bert_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_bert_few[idx] = lista_linha

tokens_capital_bert_few = []
scores_capital_bert_few = []
for lista in listas_capital_bert_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_bert_few.append(tokens)
    scores_capital_bert_few.append(scores)

##### Language

In [ ]:
# Dicionário para armazenar as listas
listas_language_bert = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_bert.loc[:, language_bert.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_bert[idx] = lista_linha

tokens_language_bert = []
scores_language_bert = []
for lista in listas_language_bert.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_bert.append(tokens)
    scores_language_bert.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_language_bert_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_bert_one.loc[:, language_bert_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_bert_one[idx] = lista_linha

tokens_language_bert_one = []
scores_language_bert_one = []
for lista in listas_language_bert_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_bert_one.append(tokens)
    scores_language_bert_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_language_bert_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_bert_few.loc[:, language_bert_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_bert_few[idx] = lista_linha

tokens_language_bert_few = []
scores_language_bert_few = []
for lista in listas_language_bert_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_bert_few.append(tokens)
    scores_language_bert_few.append(scores)

### Roberta

##### BornIn

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_roberta = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_roberta.loc[:, bornIn_roberta.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_roberta[idx] = lista_linha

tokens_bornIn_roberta = []
scores_bornIn_roberta = []
for lista in listas_bornIn_roberta.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_roberta.append(tokens)
    scores_bornIn_roberta.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_roberta_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_roberta_one.loc[:, bornIn_roberta_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_roberta_one[idx] = lista_linha

tokens_bornIn_roberta_one = []
scores_bornIn_roberta_one = []
for lista in listas_bornIn_roberta_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_roberta_one.append(tokens)
    scores_bornIn_roberta_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_roberta_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_roberta_few.loc[:, bornIn_roberta_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_roberta_few[idx] = lista_linha

tokens_bornIn_roberta_few = []
scores_bornIn_roberta_few = []
for lista in listas_bornIn_roberta_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_roberta_few.append(tokens)
    scores_bornIn_roberta_few.append(scores)

##### DiedIn

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_roberta = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_roberta.loc[:, diedIn_roberta.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_roberta[idx] = lista_linha

tokens_diedIn_roberta = []
scores_diedIn_roberta = []
for lista in listas_diedIn_roberta.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_roberta.append(tokens)
    scores_diedIn_roberta.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_roberta_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_roberta_one.loc[:, diedIn_roberta_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_roberta_one[idx] = lista_linha

tokens_diedIn_roberta_one = []
scores_diedIn_roberta_one = []
for lista in listas_diedIn_roberta_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_roberta_one.append(tokens)
    scores_diedIn_roberta_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_roberta_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_roberta_few.loc[:, diedIn_roberta_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_roberta_few[idx] = lista_linha

tokens_diedIn_roberta_few = []
scores_diedIn_roberta_few = []
for lista in listas_diedIn_roberta_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_roberta_few.append(tokens)
    scores_diedIn_roberta_few.append(scores)

##### Worksfor

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_roberta = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_roberta.loc[:, worksfor_roberta.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_roberta[idx] = lista_linha

tokens_worksfor_roberta = []
scores_worksfor_roberta = []
for lista in listas_worksfor_roberta.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_roberta.append(tokens)
    scores_worksfor_roberta.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_roberta_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_roberta_one.loc[:, worksfor_roberta_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_roberta_one[idx] = lista_linha

tokens_worksfor_roberta_one = []
scores_worksfor_roberta_one = []
for lista in listas_worksfor_roberta_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_roberta_one.append(tokens)
    scores_worksfor_roberta_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_roberta_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_roberta_few.loc[:, worksfor_roberta_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_roberta_few[idx] = lista_linha

tokens_worksfor_roberta_few = []
scores_worksfor_roberta_few = []
for lista in listas_worksfor_roberta_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_roberta_few.append(tokens)
    scores_worksfor_roberta_few.append(scores)

##### Citizen

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_roberta = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_roberta.loc[:, citizen_roberta.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_roberta[idx] = lista_linha

tokens_citizen_roberta = []
scores_citizen_roberta = []
for lista in listas_citizen_roberta.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_roberta.append(tokens)
    scores_citizen_roberta.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_roberta_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_roberta_one.loc[:, citizen_roberta_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_roberta_one[idx] = lista_linha

tokens_citizen_roberta_one = []
scores_citizen_roberta_one = []
for lista in listas_citizen_roberta_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_roberta_one.append(tokens)
    scores_citizen_roberta_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_roberta_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_roberta_few.loc[:, citizen_roberta_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_roberta_few[idx] = lista_linha

tokens_citizen_roberta_few = []
scores_citizen_roberta_few = []
for lista in listas_citizen_roberta_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_roberta_few.append(tokens)
    scores_citizen_roberta_few.append(scores)

##### Capital

In [ ]:
# Dicionário para armazenar as listas
listas_capital_roberta = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_roberta.loc[:, capital_roberta.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_roberta[idx] = lista_linha

tokens_capital_roberta = []
scores_capital_roberta = []
for lista in listas_capital_roberta.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_roberta.append(tokens)
    scores_capital_roberta.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_roberta_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_roberta_one.loc[:, capital_roberta_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_roberta_one[idx] = lista_linha

tokens_capital_roberta_one = []
scores_capital_roberta_one = []
for lista in listas_capital_roberta_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_roberta_one.append(tokens)
    scores_capital_roberta_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_roberta_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_roberta_few.loc[:, capital_roberta_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_roberta_few[idx] = lista_linha

tokens_capital_roberta_few = []
scores_capital_roberta_few = []
for lista in listas_capital_roberta_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_roberta_few.append(tokens)
    scores_capital_roberta_few.append(scores)

##### Language

In [ ]:
# Dicionário para armazenar as listas
listas_language_roberta = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_roberta.loc[:, language_roberta.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_roberta[idx] = lista_linha

tokens_language_roberta = []
scores_language_roberta = []
for lista in listas_language_roberta.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_roberta.append(tokens)
    scores_language_roberta.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_language_roberta_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_roberta_one.loc[:, language_roberta_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_roberta_one[idx] = lista_linha

tokens_language_roberta_one = []
scores_language_roberta_one = []
for lista in listas_language_roberta_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_roberta_one.append(tokens)
    scores_language_roberta_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_language_roberta_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_roberta_few.loc[:, language_roberta_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_roberta_few[idx] = lista_linha

tokens_language_roberta_few = []
scores_language_roberta_few = []
for lista in listas_language_roberta_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_roberta_few.append(tokens)
    scores_language_roberta_few.append(scores)

### Electra

##### BornIn

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_electra = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_electra.loc[:, bornIn_electra.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_electra[idx] = lista_linha

tokens_bornIn_electra = []
scores_bornIn_electra = []
for lista in listas_bornIn_electra.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_electra.append(tokens)
    scores_bornIn_electra.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_electra_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_electra_one.loc[:, bornIn_electra_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_electra_one[idx] = lista_linha

tokens_bornIn_electra_one = []
scores_bornIn_electra_one = []
for lista in listas_bornIn_electra_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_electra_one.append(tokens)
    scores_bornIn_electra_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_bornIn_electra_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in bornIn_electra_few.loc[:, bornIn_electra_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_bornIn_electra_few[idx] = lista_linha

tokens_bornIn_electra_few = []
scores_bornIn_electra_few = []
for lista in listas_bornIn_electra_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_bornIn_electra_few.append(tokens)
    scores_bornIn_electra_few.append(scores)

##### DiedIn

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_electra = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_electra.loc[:, diedIn_electra.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_electra[idx] = lista_linha

tokens_diedIn_electra = []
scores_diedIn_electra = []
for lista in listas_diedIn_electra.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_electra.append(tokens)
    scores_diedIn_electra.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_electra_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_electra_one.loc[:, diedIn_electra_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_electra_one[idx] = lista_linha

tokens_diedIn_electra_one = []
scores_diedIn_electra_one = []
for lista in listas_diedIn_electra_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_electra_one.append(tokens)
    scores_diedIn_electra_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_diedIn_electra_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in diedIn_electra_few.loc[:, diedIn_electra_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_diedIn_electra_few[idx] = lista_linha

tokens_diedIn_electra_few = []
scores_diedIn_electra_few = []
for lista in listas_diedIn_electra_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_diedIn_electra_few.append(tokens)
    scores_diedIn_electra_few.append(scores)

##### Worksfor

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_electra = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_electra.loc[:, worksfor_electra.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_electra[idx] = lista_linha

tokens_worksfor_electra = []
scores_worksfor_electra = []
for lista in listas_worksfor_electra.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_electra.append(tokens)
    scores_worksfor_electra.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_electra_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_electra_one.loc[:, worksfor_electra_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_electra_one[idx] = lista_linha

tokens_worksfor_electra_one = []
scores_worksfor_electra_one = []
for lista in listas_worksfor_electra_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_electra_one.append(tokens)
    scores_worksfor_electra_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_worksfor_electra_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in worksfor_electra_few.loc[:, worksfor_electra_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_worksfor_electra_few[idx] = lista_linha

tokens_worksfor_electra_few = []
scores_worksfor_electra_few = []
for lista in listas_worksfor_electra_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_worksfor_electra_few.append(tokens)
    scores_worksfor_electra_few.append(scores)

##### Citizen

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_electra = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_electra.loc[:, citizen_electra.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_electra[idx] = lista_linha

tokens_citizen_electra = []
scores_citizen_electra = []
for lista in listas_citizen_electra.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_electra.append(tokens)
    scores_citizen_electra.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_electra_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_electra_one.loc[:, citizen_electra_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_electra_one[idx] = lista_linha

tokens_citizen_electra_one = []
scores_citizen_electra_one = []
for lista in listas_citizen_electra_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_electra_one.append(tokens)
    scores_citizen_electra_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_citizen_electra_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in citizen_electra_few.loc[:, citizen_electra_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_citizen_electra_few[idx] = lista_linha

tokens_citizen_electra_few = []
scores_citizen_electra_few = []
for lista in listas_citizen_electra_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_citizen_electra_few.append(tokens)
    scores_citizen_electra_few.append(scores)

##### Capital

In [ ]:
# Dicionário para armazenar as listas
listas_capital_electra = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_electra.loc[:, capital_electra.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_electra[idx] = lista_linha

tokens_capital_electra = []
scores_capital_electra = []
for lista in listas_capital_electra.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_electra.append(tokens)
    scores_capital_electra.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_electra_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_electra_one.loc[:, capital_electra_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_electra_one[idx] = lista_linha

tokens_capital_electra_one = []
scores_capital_electra_one = []
for lista in listas_capital_electra_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_electra_one.append(tokens)
    scores_capital_electra_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_capital_electra_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in capital_electra_few.loc[:, capital_electra_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_capital_electra_few[idx] = lista_linha

tokens_capital_electra_few = []
scores_capital_electra_few = []
for lista in listas_capital_electra_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_capital_electra_few.append(tokens)
    scores_capital_electra_few.append(scores)

##### Language

In [ ]:
# Dicionário para armazenar as listas
listas_language_electra = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_electra.loc[:, language_electra.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_electra[idx] = lista_linha

tokens_language_electra = []
scores_language_electra = []
for lista in listas_language_electra.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_electra.append(tokens)
    scores_language_electra.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_language_electra_one = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_electra_one.loc[:, language_electra_one.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_electra_one[idx] = lista_linha

tokens_language_electra_one = []
scores_language_electra_one = []
for lista in listas_language_electra_one.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_electra_one.append(tokens)
    scores_language_electra_one.append(scores)

In [ ]:
# Dicionário para armazenar as listas
listas_language_electra_few = {}

# Iterar sobre as linhas do DataFrame
for idx, row in language_electra_few.loc[:, language_electra_few.columns != 'answers'].iterrows():
    # Criar uma lista para a linha atual
    lista_linha = []
    # Iterar sobre as colunas da linha atual
    for coluna, valor in row.items():
        # Adicionar o valor da coluna atual à lista da linha
        lista_linha.append(valor)
    # Adicionar a lista da linha ao dicionário, usando o índice da linha como chave
    listas_language_electra_few[idx] = lista_linha

tokens_language_electra_few = []
scores_language_electra_few = []
for lista in listas_language_electra_few.values():
    tokens, scores = somar_scores_tokens_por_posicao(*lista)
    tokens_language_electra_few.append(tokens)
    scores_language_electra_few.append(scores)

# Ranking

## Bert

#### BornIn

In [ ]:
# Normal
ranking_bornIn_bert_normal_tokens = []
ranking_bornIn_bert_normal = ranking_dataframe(bornIn_bert, bornIn_bert.columns[1:])
for i in range(len(ranking_bornIn_bert_normal)):
    ranking_bornIn_bert_normal_tokens.append(extrair_tokens(ranking_bornIn_bert_normal[i]))

# One Shoot
ranking_bornIn_bert_normal_tokens_one = []
ranking_bornIn_bert_normal_one = ranking_dataframe(bornIn_bert_one, bornIn_bert_one.columns[1:])
for i in range(len(ranking_bornIn_bert_normal_one)):
    ranking_bornIn_bert_normal_tokens_one.append(extrair_tokens(ranking_bornIn_bert_normal_one[i]))

# Few Shoot
ranking_bornIn_bert_normal_tokens_few = []
ranking_bornIn_bert_normal_few = ranking_dataframe(bornIn_bert_few, bornIn_bert_few.columns[1:])
for i in range(len(ranking_bornIn_bert_normal_few)):
    ranking_bornIn_bert_normal_tokens_few.append(extrair_tokens(ranking_bornIn_bert_normal_few[i]))

In [ ]:
# Spacy
ranking_bornIn_bert_spacy_tokens = []
ranking_bornIn_bert_spacy = ranking_dataframe(bornIn_bert_spacy, bornIn_bert_spacy.columns[1:])
for i in range(len(ranking_bornIn_bert_spacy)):
    ranking_bornIn_bert_spacy_tokens.append(extrair_tokens(ranking_bornIn_bert_spacy[i]))

# One Shoot
ranking_bornIn_bert_spacy_tokens_one = []
ranking_bornIn_bert_one_spacy = ranking_dataframe(bornIn_bert_one_spacy, bornIn_bert_one_spacy.columns[1:])
for i in range(len(ranking_bornIn_bert_one_spacy)):
    ranking_bornIn_bert_spacy_tokens_one.append(extrair_tokens(ranking_bornIn_bert_one_spacy[i]))

# Few Shoot
ranking_bornIn_bert_spacy_tokens_few = []
ranking_bornIn_bert_few_spacy = ranking_dataframe(bornIn_bert_few_spacy, bornIn_bert_few_spacy.columns[1:])
for i in range(len(ranking_bornIn_bert_few_spacy)):
    ranking_bornIn_bert_spacy_tokens_few.append(extrair_tokens(ranking_bornIn_bert_few_spacy[i]))

#### DiedIn

In [ ]:
# Normal
ranking_diedIn_bert_normal_tokens = []
ranking_diedIn_bert_normal = ranking_dataframe(diedIn_bert, diedIn_bert.columns[1:])
for i in range(len(ranking_diedIn_bert_normal)):
    ranking_diedIn_bert_normal_tokens.append(extrair_tokens(ranking_diedIn_bert_normal[i]))

# One Shoot
ranking_diedIn_bert_normal_tokens_one = []
ranking_diedIn_bert_normal_one = ranking_dataframe(diedIn_bert_one, diedIn_bert_one.columns[1:])
for i in range(len(ranking_diedIn_bert_normal_one)):
    ranking_diedIn_bert_normal_tokens_one.append(extrair_tokens(ranking_diedIn_bert_normal_one[i]))

# Few Shoot
ranking_diedIn_bert_normal_tokens_few = []
ranking_diedIn_bert_normal_few = ranking_dataframe(diedIn_bert_few, diedIn_bert_few.columns[1:])
for i in range(len(ranking_diedIn_bert_normal_few)):
    ranking_diedIn_bert_normal_tokens_few.append(extrair_tokens(ranking_diedIn_bert_normal_few[i]))

In [ ]:
# Spacy
ranking_diedIn_bert_spacy_tokens = []
ranking_diedIn_bert_spacy = ranking_dataframe(diedIn_bert_spacy, diedIn_bert_spacy.columns[1:])
for i in range(len(ranking_diedIn_bert_spacy)):
    ranking_diedIn_bert_spacy_tokens.append(extrair_tokens(ranking_diedIn_bert_spacy[i]))

# One Shoot
ranking_diedIn_bert_spacy_tokens_one = []
ranking_diedIn_bert_one_spacy = ranking_dataframe(diedIn_bert_one_spacy, diedIn_bert_one_spacy.columns[1:])
for i in range(len(ranking_diedIn_bert_one_spacy)):
    ranking_diedIn_bert_spacy_tokens_one.append(extrair_tokens(ranking_diedIn_bert_one_spacy[i]))

# Few Shoot
ranking_diedIn_bert_spacy_tokens_few = []
ranking_diedIn_bert_few_spacy = ranking_dataframe(diedIn_bert_few_spacy, diedIn_bert_few_spacy.columns[1:])
for i in range(len(ranking_diedIn_bert_few_spacy)):
    ranking_diedIn_bert_spacy_tokens_few.append(extrair_tokens(ranking_diedIn_bert_few_spacy[i]))

#### Worksfor

In [ ]:
# Normal
ranking_worksfor_bert_normal_tokens = []
ranking_worksfor_bert_normal = ranking_dataframe(worksfor_bert, worksfor_bert.columns[1:])
for i in range(len(ranking_worksfor_bert_normal)):
    ranking_worksfor_bert_normal_tokens.append(extrair_tokens(ranking_worksfor_bert_normal[i]))

# One Shoot
ranking_worksfor_bert_normal_tokens_one = []
ranking_worksfor_bert_normal_one = ranking_dataframe(worksfor_bert_one, worksfor_bert_one.columns[1:])
for i in range(len(ranking_worksfor_bert_normal_one)):
    ranking_worksfor_bert_normal_tokens_one.append(extrair_tokens(ranking_worksfor_bert_normal_one[i]))

# Few Shoot
ranking_worksfor_bert_normal_tokens_few = []
ranking_worksfor_bert_normal_few = ranking_dataframe(worksfor_bert_few, worksfor_bert_few.columns[1:])
for i in range(len(ranking_worksfor_bert_normal_few)):
    ranking_worksfor_bert_normal_tokens_few.append(extrair_tokens(ranking_worksfor_bert_normal_few[i]))

In [ ]:
# Spacy
ranking_worksfor_bert_spacy_tokens = []
ranking_worksfor_bert_spacy = ranking_dataframe(worksfor_bert_spacy, worksfor_bert_spacy.columns[1:])
for i in range(len(ranking_worksfor_bert_spacy)):
    ranking_worksfor_bert_spacy_tokens.append(extrair_tokens(ranking_worksfor_bert_spacy[i]))

# One Shoot
ranking_worksfor_bert_spacy_tokens_one = []
ranking_worksfor_bert_one_spacy = ranking_dataframe(worksfor_bert_one_spacy, worksfor_bert_one_spacy.columns[1:])
for i in range(len(ranking_worksfor_bert_one_spacy)):
    ranking_worksfor_bert_spacy_tokens_one.append(extrair_tokens(ranking_worksfor_bert_one_spacy[i]))

# Few Shoot
ranking_worksfor_bert_spacy_tokens_few = []
ranking_worksfor_bert_few_spacy = ranking_dataframe(worksfor_bert_few_spacy, worksfor_bert_few_spacy.columns[1:])
for i in range(len(ranking_worksfor_bert_few_spacy)):
    ranking_worksfor_bert_spacy_tokens_few.append(extrair_tokens(ranking_worksfor_bert_few_spacy[i]))

#### Capital

In [ ]:
# Normal
ranking_capital_bert_normal_tokens = []
ranking_capital_bert_normal = ranking_dataframe(capital_bert, capital_bert.columns[1:])
for i in range(len(ranking_capital_bert_normal)):
    ranking_capital_bert_normal_tokens.append(extrair_tokens(ranking_capital_bert_normal[i]))

# One Shoot
ranking_capital_bert_normal_tokens_one = []
ranking_capital_bert_normal_one = ranking_dataframe(capital_bert_one, capital_bert_one.columns[1:])
for i in range(len(ranking_capital_bert_normal_one)):
    ranking_capital_bert_normal_tokens_one.append(extrair_tokens(ranking_capital_bert_normal_one[i]))

# Few Shoot
ranking_capital_bert_normal_tokens_few = []
ranking_capital_bert_normal_few = ranking_dataframe(capital_bert_few, capital_bert_few.columns[1:])
for i in range(len(ranking_capital_bert_normal_few)):
    ranking_capital_bert_normal_tokens_few.append(extrair_tokens(ranking_capital_bert_normal_few[i]))

In [ ]:
# Spacy
ranking_capital_bert_spacy_tokens = []
ranking_capital_bert_spacy = ranking_dataframe(capital_bert_spacy, capital_bert_spacy.columns[1:])
for i in range(len(ranking_capital_bert_spacy)):
    ranking_capital_bert_spacy_tokens.append(extrair_tokens(ranking_capital_bert_spacy[i]))

# One Shoot
ranking_capital_bert_spacy_tokens_one = []
ranking_capital_bert_one_spacy = ranking_dataframe(capital_bert_one_spacy, capital_bert_one_spacy.columns[1:])
for i in range(len(ranking_capital_bert_one_spacy)):
    ranking_capital_bert_spacy_tokens_one.append(extrair_tokens(ranking_capital_bert_one_spacy[i]))

# Few Shoot
ranking_capital_bert_spacy_tokens_few = []
ranking_capital_bert_few_spacy = ranking_dataframe(capital_bert_few_spacy, capital_bert_few_spacy.columns[1:])
for i in range(len(ranking_capital_bert_few_spacy)):
    ranking_capital_bert_spacy_tokens_few.append(extrair_tokens(ranking_capital_bert_few_spacy[i]))

#### Citizen

In [ ]:
# Normal
ranking_citizen_bert_normal_tokens = []
ranking_citizen_bert_normal = ranking_dataframe(citizen_bert, citizen_bert.columns[1:])
for i in range(len(ranking_citizen_bert_normal)):
    ranking_citizen_bert_normal_tokens.append(extrair_tokens(ranking_citizen_bert_normal[i]))

# One Shoot
ranking_citizen_bert_normal_tokens_one = []
ranking_citizen_bert_normal_one = ranking_dataframe(citizen_bert_one, citizen_bert_one.columns[1:])
for i in range(len(ranking_citizen_bert_normal_one)):
    ranking_citizen_bert_normal_tokens_one.append(extrair_tokens(ranking_citizen_bert_normal_one[i]))

# Few Shoot
ranking_citizen_bert_normal_tokens_few = []
ranking_citizen_bert_normal_few = ranking_dataframe(citizen_bert_few, citizen_bert_few.columns[1:])
for i in range(len(ranking_citizen_bert_normal_few)):
    ranking_citizen_bert_normal_tokens_few.append(extrair_tokens(ranking_citizen_bert_normal_few[i]))

In [ ]:
# Spacy
ranking_citizen_bert_spacy_tokens = []
ranking_citizen_bert_spacy = ranking_dataframe(citizen_bert_spacy, citizen_bert_spacy.columns[1:])
for i in range(len(ranking_citizen_bert_spacy)):
    ranking_citizen_bert_spacy_tokens.append(extrair_tokens(ranking_citizen_bert_spacy[i]))

# One Shoot
ranking_citizen_bert_spacy_tokens_one = []
ranking_citizen_bert_one_spacy = ranking_dataframe(citizen_bert_one_spacy, citizen_bert_one_spacy.columns[1:])
for i in range(len(ranking_citizen_bert_one_spacy)):
    ranking_citizen_bert_spacy_tokens_one.append(extrair_tokens(ranking_citizen_bert_one_spacy[i]))

# Few Shoot
ranking_citizen_bert_spacy_tokens_few = []
ranking_citizen_bert_few_spacy = ranking_dataframe(citizen_bert_few_spacy, citizen_bert_few_spacy.columns[1:])
for i in range(len(ranking_citizen_bert_few_spacy)):
    ranking_citizen_bert_spacy_tokens_few.append(extrair_tokens(ranking_citizen_bert_few_spacy[i]))

#### Language

In [ ]:
# Normal
ranking_language_bert_normal_tokens = []
ranking_language_bert_normal = ranking_dataframe(language_bert, language_bert.columns[1:])
for i in range(len(ranking_language_bert_normal)):
    ranking_language_bert_normal_tokens.append(extrair_tokens(ranking_language_bert_normal[i]))

# One Shoot
ranking_language_bert_normal_tokens_one = []
ranking_language_bert_normal_one = ranking_dataframe(language_bert_one, language_bert_one.columns[1:])
for i in range(len(ranking_language_bert_normal_one)):
    ranking_language_bert_normal_tokens_one.append(extrair_tokens(ranking_language_bert_normal_one[i]))

# Few Shoot
ranking_language_bert_normal_tokens_few = []
ranking_language_bert_normal_few = ranking_dataframe(language_bert_few, language_bert_few.columns[1:])
for i in range(len(ranking_language_bert_normal_few)):
    ranking_language_bert_normal_tokens_few.append(extrair_tokens(ranking_language_bert_normal_few[i]))

In [ ]:
# Spacy
ranking_language_bert_spacy_tokens = []
ranking_language_bert_spacy = ranking_dataframe(language_bert_spacy, language_bert_spacy.columns[1:])
for i in range(len(ranking_language_bert_spacy)):
    ranking_language_bert_spacy_tokens.append(extrair_tokens(ranking_language_bert_spacy[i]))

# One Shoot
ranking_language_bert_spacy_tokens_one = []
ranking_language_bert_one_spacy = ranking_dataframe(language_bert_one_spacy, language_bert_one_spacy.columns[1:])
for i in range(len(ranking_language_bert_one_spacy)):
    ranking_language_bert_spacy_tokens_one.append(extrair_tokens(ranking_language_bert_one_spacy[i]))

# Few Shoot
ranking_language_bert_spacy_tokens_few = []
ranking_language_bert_few_spacy = ranking_dataframe(language_bert_few_spacy, language_bert_few_spacy.columns[1:])
for i in range(len(ranking_language_bert_few_spacy)):
    ranking_language_bert_spacy_tokens_few.append(extrair_tokens(ranking_language_bert_few_spacy[i]))

## Roberta

#### BornIn

In [ ]:
# Normal
ranking_bornIn_roberta_normal_tokens = []
ranking_bornIn_roberta_normal = ranking_dataframe(bornIn_roberta, bornIn_roberta.columns[1:])
for i in range(len(ranking_bornIn_roberta_normal)):
    ranking_bornIn_roberta_normal_tokens.append(extrair_tokens(ranking_bornIn_roberta_normal[i]))

# One Shoot
ranking_bornIn_roberta_normal_tokens_one = []
ranking_bornIn_roberta_normal_one = ranking_dataframe(bornIn_roberta_one, bornIn_roberta_one.columns[1:])
for i in range(len(ranking_bornIn_roberta_normal_one)):
    ranking_bornIn_roberta_normal_tokens_one.append(extrair_tokens(ranking_bornIn_roberta_normal_one[i]))

# Few Shoot
ranking_bornIn_roberta_normal_tokens_few = []
ranking_bornIn_roberta_normal_few = ranking_dataframe(bornIn_roberta_few, bornIn_roberta_few.columns[1:])
for i in range(len(ranking_bornIn_roberta_normal_few)):
    ranking_bornIn_roberta_normal_tokens_few.append(extrair_tokens(ranking_bornIn_roberta_normal_few[i]))

In [ ]:
# Spacy
ranking_bornIn_roberta_spacy_tokens = []
ranking_bornIn_roberta_spacy = ranking_dataframe(bornIn_roberta_spacy, bornIn_roberta_spacy.columns[1:])
for i in range(len(ranking_bornIn_roberta_spacy)):
    ranking_bornIn_roberta_spacy_tokens.append(extrair_tokens(ranking_bornIn_roberta_spacy[i]))

# One Shoot
ranking_bornIn_roberta_spacy_tokens_one = []
ranking_bornIn_roberta_one_spacy = ranking_dataframe(bornIn_roberta_one_spacy, bornIn_roberta_one_spacy.columns[1:])
for i in range(len(ranking_bornIn_roberta_one_spacy)):
    ranking_bornIn_roberta_spacy_tokens_one.append(extrair_tokens(ranking_bornIn_roberta_one_spacy[i]))

# Few Shoot
ranking_bornIn_roberta_spacy_tokens_few = []
ranking_bornIn_roberta_few_spacy = ranking_dataframe(bornIn_roberta_few_spacy, bornIn_roberta_few_spacy.columns[1:])
for i in range(len(ranking_bornIn_roberta_few_spacy)):
    ranking_bornIn_roberta_spacy_tokens_few.append(extrair_tokens(ranking_bornIn_roberta_few_spacy[i]))

#### DiedIn

In [ ]:
# Normal
ranking_diedIn_roberta_normal_tokens = []
ranking_diedIn_roberta_normal = ranking_dataframe(diedIn_roberta, diedIn_roberta.columns[1:])
for i in range(len(ranking_diedIn_roberta_normal)):
    ranking_diedIn_roberta_normal_tokens.append(extrair_tokens(ranking_diedIn_roberta_normal[i]))

# One Shoot
ranking_diedIn_roberta_normal_tokens_one = []
ranking_diedIn_roberta_normal_one = ranking_dataframe(diedIn_roberta_one, diedIn_roberta_one.columns[1:])
for i in range(len(ranking_diedIn_roberta_normal_one)):
    ranking_diedIn_roberta_normal_tokens_one.append(extrair_tokens(ranking_diedIn_roberta_normal_one[i]))

# Few Shoot
ranking_diedIn_roberta_normal_tokens_few = []
ranking_diedIn_roberta_normal_few = ranking_dataframe(diedIn_roberta_few, diedIn_roberta_few.columns[1:])
for i in range(len(ranking_diedIn_roberta_normal_few)):
    ranking_diedIn_roberta_normal_tokens_few.append(extrair_tokens(ranking_diedIn_roberta_normal_few[i]))

In [ ]:
# Spacy
ranking_diedIn_roberta_spacy_tokens = []
ranking_diedIn_roberta_spacy = ranking_dataframe(diedIn_roberta_spacy, diedIn_roberta_spacy.columns[1:])
for i in range(len(ranking_diedIn_roberta_spacy)):
    ranking_diedIn_roberta_spacy_tokens.append(extrair_tokens(ranking_diedIn_roberta_spacy[i]))

# One Shoot
ranking_diedIn_roberta_spacy_tokens_one = []
ranking_diedIn_roberta_one_spacy = ranking_dataframe(diedIn_roberta_one_spacy, diedIn_roberta_one_spacy.columns[1:])
for i in range(len(ranking_diedIn_roberta_one_spacy)):
    ranking_diedIn_roberta_spacy_tokens_one.append(extrair_tokens(ranking_diedIn_roberta_one_spacy[i]))

# Few Shoot
ranking_diedIn_roberta_spacy_tokens_few = []
ranking_diedIn_roberta_few_spacy = ranking_dataframe(diedIn_roberta_few_spacy, diedIn_roberta_few_spacy.columns[1:])
for i in range(len(ranking_diedIn_roberta_few_spacy)):
    ranking_diedIn_roberta_spacy_tokens_few.append(extrair_tokens(ranking_diedIn_roberta_few_spacy[i]))

#### Worksfor

In [ ]:
# Normal
ranking_worksfor_roberta_normal_tokens = []
ranking_worksfor_roberta_normal = ranking_dataframe(worksfor_roberta, worksfor_roberta.columns[1:])
for i in range(len(ranking_worksfor_roberta_normal)):
    ranking_worksfor_roberta_normal_tokens.append(extrair_tokens(ranking_worksfor_roberta_normal[i]))

# One Shoot
ranking_worksfor_roberta_normal_tokens_one = []
ranking_worksfor_roberta_normal_one = ranking_dataframe(worksfor_roberta_one, worksfor_roberta_one.columns[1:])
for i in range(len(ranking_worksfor_roberta_normal_one)):
    ranking_worksfor_roberta_normal_tokens_one.append(extrair_tokens(ranking_worksfor_roberta_normal_one[i]))

# Few Shoot
ranking_worksfor_roberta_normal_tokens_few = []
ranking_worksfor_roberta_normal_few = ranking_dataframe(worksfor_roberta_few, worksfor_roberta_few.columns[1:])
for i in range(len(ranking_worksfor_roberta_normal_few)):
    ranking_worksfor_roberta_normal_tokens_few.append(extrair_tokens(ranking_worksfor_roberta_normal_few[i]))

In [ ]:
# Spacy
ranking_worksfor_roberta_spacy_tokens = []
ranking_worksfor_roberta_spacy = ranking_dataframe(worksfor_roberta_spacy, worksfor_roberta_spacy.columns[1:])
for i in range(len(ranking_worksfor_roberta_spacy)):
    ranking_worksfor_roberta_spacy_tokens.append(extrair_tokens(ranking_worksfor_roberta_spacy[i]))

# One Shoot
ranking_worksfor_roberta_spacy_tokens_one = []
ranking_worksfor_roberta_one_spacy = ranking_dataframe(worksfor_roberta_one_spacy, worksfor_roberta_one_spacy.columns[1:])
for i in range(len(ranking_worksfor_roberta_one_spacy)):
    ranking_worksfor_roberta_spacy_tokens_one.append(extrair_tokens(ranking_worksfor_roberta_one_spacy[i]))

# Few Shoot
ranking_worksfor_roberta_spacy_tokens_few = []
ranking_worksfor_roberta_few_spacy = ranking_dataframe(worksfor_roberta_few_spacy, worksfor_roberta_few_spacy.columns[1:])
for i in range(len(ranking_worksfor_roberta_few_spacy)):
    ranking_worksfor_roberta_spacy_tokens_few.append(extrair_tokens(ranking_worksfor_roberta_few_spacy[i]))

#### Capital

In [ ]:
# Normal
ranking_capital_roberta_normal_tokens = []
ranking_capital_roberta_normal = ranking_dataframe(capital_roberta, capital_roberta.columns[1:])
for i in range(len(ranking_capital_roberta_normal)):
    ranking_capital_roberta_normal_tokens.append(extrair_tokens(ranking_capital_roberta_normal[i]))

# One Shoot
ranking_capital_roberta_normal_tokens_one = []
ranking_capital_roberta_normal_one = ranking_dataframe(capital_roberta_one, capital_roberta_one.columns[1:])
for i in range(len(ranking_capital_roberta_normal_one)):
    ranking_capital_roberta_normal_tokens_one.append(extrair_tokens(ranking_capital_roberta_normal_one[i]))

# Few Shoot
ranking_capital_roberta_normal_tokens_few = []
ranking_capital_roberta_normal_few = ranking_dataframe(capital_roberta_few, capital_roberta_few.columns[1:])
for i in range(len(ranking_capital_roberta_normal_few)):
    ranking_capital_roberta_normal_tokens_few.append(extrair_tokens(ranking_capital_roberta_normal_few[i]))

In [ ]:
# Spacy
ranking_capital_roberta_spacy_tokens = []
ranking_capital_roberta_spacy = ranking_dataframe(capital_roberta_spacy, capital_roberta_spacy.columns[1:])
for i in range(len(ranking_capital_roberta_spacy)):
    ranking_capital_roberta_spacy_tokens.append(extrair_tokens(ranking_capital_roberta_spacy[i]))

# One Shoot
ranking_capital_roberta_spacy_tokens_one = []
ranking_capital_roberta_one_spacy = ranking_dataframe(capital_roberta_one_spacy, capital_roberta_one_spacy.columns[1:])
for i in range(len(ranking_capital_roberta_one_spacy)):
    ranking_capital_roberta_spacy_tokens_one.append(extrair_tokens(ranking_capital_roberta_one_spacy[i]))

# Few Shoot
ranking_capital_roberta_spacy_tokens_few = []
ranking_capital_roberta_few_spacy = ranking_dataframe(capital_roberta_few_spacy, capital_roberta_few_spacy.columns[1:])
for i in range(len(ranking_capital_roberta_few_spacy)):
    ranking_capital_roberta_spacy_tokens_few.append(extrair_tokens(ranking_capital_roberta_few_spacy[i]))

#### Citizen

In [ ]:
# Normal
ranking_citizen_roberta_normal_tokens = []
ranking_citizen_roberta_normal = ranking_dataframe(citizen_roberta, citizen_roberta.columns[1:])
for i in range(len(ranking_citizen_roberta_normal)):
    ranking_citizen_roberta_normal_tokens.append(extrair_tokens(ranking_citizen_roberta_normal[i]))

# One Shoot
ranking_citizen_roberta_normal_tokens_one = []
ranking_citizen_roberta_normal_one = ranking_dataframe(citizen_roberta_one, citizen_roberta_one.columns[1:])
for i in range(len(ranking_citizen_roberta_normal_one)):
    ranking_citizen_roberta_normal_tokens_one.append(extrair_tokens(ranking_citizen_roberta_normal_one[i]))

# Few Shoot
ranking_citizen_roberta_normal_tokens_few = []
ranking_citizen_roberta_normal_few = ranking_dataframe(citizen_roberta_few, citizen_roberta_few.columns[1:])
for i in range(len(ranking_citizen_roberta_normal_few)):
    ranking_citizen_roberta_normal_tokens_few.append(extrair_tokens(ranking_citizen_roberta_normal_few[i]))

In [ ]:
# Spacy
ranking_citizen_roberta_spacy_tokens = []
ranking_citizen_roberta_spacy = ranking_dataframe(citizen_roberta_spacy, citizen_roberta_spacy.columns[1:])
for i in range(len(ranking_citizen_roberta_spacy)):
    ranking_citizen_roberta_spacy_tokens.append(extrair_tokens(ranking_citizen_roberta_spacy[i]))

# One Shoot
ranking_citizen_roberta_spacy_tokens_one = []
ranking_citizen_roberta_one_spacy = ranking_dataframe(citizen_roberta_one_spacy, citizen_roberta_one_spacy.columns[1:])
for i in range(len(ranking_citizen_roberta_one_spacy)):
    ranking_citizen_roberta_spacy_tokens_one.append(extrair_tokens(ranking_citizen_roberta_one_spacy[i]))

# Few Shoot
ranking_citizen_roberta_spacy_tokens_few = []
ranking_citizen_roberta_few_spacy = ranking_dataframe(citizen_roberta_few_spacy, citizen_roberta_few_spacy.columns[1:])
for i in range(len(ranking_citizen_roberta_few_spacy)):
    ranking_citizen_roberta_spacy_tokens_few.append(extrair_tokens(ranking_citizen_roberta_few_spacy[i]))

#### Language

In [ ]:
# Normal
ranking_language_roberta_normal_tokens = []
ranking_language_roberta_normal = ranking_dataframe(language_roberta, language_roberta.columns[1:])
for i in range(len(ranking_language_roberta_normal)):
    ranking_language_roberta_normal_tokens.append(extrair_tokens(ranking_language_roberta_normal[i]))

# One Shoot
ranking_language_roberta_normal_tokens_one = []
ranking_language_roberta_normal_one = ranking_dataframe(language_roberta_one, language_roberta_one.columns[1:])
for i in range(len(ranking_language_roberta_normal_one)):
    ranking_language_roberta_normal_tokens_one.append(extrair_tokens(ranking_language_roberta_normal_one[i]))

# Few Shoot
ranking_language_roberta_normal_tokens_few = []
ranking_language_roberta_normal_few = ranking_dataframe(language_roberta_few, language_roberta_few.columns[1:])
for i in range(len(ranking_language_roberta_normal_few)):
    ranking_language_roberta_normal_tokens_few.append(extrair_tokens(ranking_language_roberta_normal_few[i]))

In [ ]:
# Spacy
ranking_language_roberta_spacy_tokens = []
ranking_language_roberta_spacy = ranking_dataframe(language_roberta_spacy, language_roberta_spacy.columns[1:])
for i in range(len(ranking_language_roberta_spacy)):
    ranking_language_roberta_spacy_tokens.append(extrair_tokens(ranking_language_roberta_spacy[i]))

# One Shoot
ranking_language_roberta_spacy_tokens_one = []
ranking_language_roberta_one_spacy = ranking_dataframe(language_roberta_one_spacy, language_roberta_one_spacy.columns[1:])
for i in range(len(ranking_language_roberta_one_spacy)):
    ranking_language_roberta_spacy_tokens_one.append(extrair_tokens(ranking_language_roberta_one_spacy[i]))

# Few Shoot
ranking_language_roberta_spacy_tokens_few = []
ranking_language_roberta_few_spacy = ranking_dataframe(language_roberta_few_spacy, language_roberta_few_spacy.columns[1:])
for i in range(len(ranking_language_roberta_few_spacy)):
    ranking_language_roberta_spacy_tokens_few.append(extrair_tokens(ranking_language_roberta_few_spacy[i]))

## Electra

#### BornIn

In [ ]:
# Normal
ranking_bornIn_electra_normal_tokens = []
ranking_bornIn_electra_normal = ranking_dataframe(bornIn_electra, bornIn_electra.columns[1:])
for i in range(len(ranking_bornIn_electra_normal)):
    ranking_bornIn_electra_normal_tokens.append(extrair_tokens(ranking_bornIn_electra_normal[i]))

# One Shoot
ranking_bornIn_electra_normal_tokens_one = []
ranking_bornIn_electra_normal_one = ranking_dataframe(bornIn_electra_one, bornIn_electra_one.columns[1:])
for i in range(len(ranking_bornIn_electra_normal_one)):
    ranking_bornIn_electra_normal_tokens_one.append(extrair_tokens(ranking_bornIn_electra_normal_one[i]))

# Few Shoot
ranking_bornIn_electra_normal_tokens_few = []
ranking_bornIn_electra_normal_few = ranking_dataframe(bornIn_electra_few, bornIn_electra_few.columns[1:])
for i in range(len(ranking_bornIn_electra_normal_few)):
    ranking_bornIn_electra_normal_tokens_few.append(extrair_tokens(ranking_bornIn_electra_normal_few[i]))

In [ ]:
# Spacy
ranking_bornIn_electra_spacy_tokens = []
ranking_bornIn_electra_spacy = ranking_dataframe(bornIn_electra_spacy, bornIn_electra_spacy.columns[1:])
for i in range(len(ranking_bornIn_electra_spacy)):
    ranking_bornIn_electra_spacy_tokens.append(extrair_tokens(ranking_bornIn_electra_spacy[i]))

# One Shoot
ranking_bornIn_electra_spacy_tokens_one = []
ranking_bornIn_electra_one_spacy = ranking_dataframe(bornIn_electra_one_spacy, bornIn_electra_one_spacy.columns[1:])
for i in range(len(ranking_bornIn_electra_one_spacy)):
    ranking_bornIn_electra_spacy_tokens_one.append(extrair_tokens(ranking_bornIn_electra_one_spacy[i]))

# Few Shoot
ranking_bornIn_electra_spacy_tokens_few = []
ranking_bornIn_electra_few_spacy = ranking_dataframe(bornIn_electra_few_spacy, bornIn_electra_few_spacy.columns[1:])
for i in range(len(ranking_bornIn_electra_few_spacy)):
    ranking_bornIn_electra_spacy_tokens_few.append(extrair_tokens(ranking_bornIn_electra_few_spacy[i]))

#### DiedIn

In [ ]:
# Normal
ranking_diedIn_electra_normal_tokens = []
ranking_diedIn_electra_normal = ranking_dataframe(diedIn_electra, diedIn_electra.columns[1:])
for i in range(len(ranking_diedIn_electra_normal)):
    ranking_diedIn_electra_normal_tokens.append(extrair_tokens(ranking_diedIn_electra_normal[i]))

# One Shoot
ranking_diedIn_electra_normal_tokens_one = []
ranking_diedIn_electra_normal_one = ranking_dataframe(diedIn_electra_one, diedIn_electra_one.columns[1:])
for i in range(len(ranking_diedIn_electra_normal_one)):
    ranking_diedIn_electra_normal_tokens_one.append(extrair_tokens(ranking_diedIn_electra_normal_one[i]))

# Few Shoot
ranking_diedIn_electra_normal_tokens_few = []
ranking_diedIn_electra_normal_few = ranking_dataframe(diedIn_electra_few, diedIn_electra_few.columns[1:])
for i in range(len(ranking_diedIn_electra_normal_few)):
    ranking_diedIn_electra_normal_tokens_few.append(extrair_tokens(ranking_diedIn_electra_normal_few[i]))

In [ ]:
# Spacy
ranking_diedIn_electra_spacy_tokens = []
ranking_diedIn_electra_spacy = ranking_dataframe(diedIn_electra_spacy, diedIn_electra_spacy.columns[1:])
for i in range(len(ranking_diedIn_electra_spacy)):
    ranking_diedIn_electra_spacy_tokens.append(extrair_tokens(ranking_diedIn_electra_spacy[i]))

# One Shoot
ranking_diedIn_electra_spacy_tokens_one = []
ranking_diedIn_electra_one_spacy = ranking_dataframe(diedIn_electra_one_spacy, diedIn_electra_one_spacy.columns[1:])
for i in range(len(ranking_diedIn_electra_one_spacy)):
    ranking_diedIn_electra_spacy_tokens_one.append(extrair_tokens(ranking_diedIn_electra_one_spacy[i]))

# Few Shoot
ranking_diedIn_electra_spacy_tokens_few = []
ranking_diedIn_electra_few_spacy = ranking_dataframe(diedIn_electra_few_spacy, diedIn_electra_few_spacy.columns[1:])
for i in range(len(ranking_diedIn_electra_few_spacy)):
    ranking_diedIn_electra_spacy_tokens_few.append(extrair_tokens(ranking_diedIn_electra_few_spacy[i]))

#### Worksfor

In [ ]:
# Normal
ranking_worksfor_electra_normal_tokens = []
ranking_worksfor_electra_normal = ranking_dataframe(worksfor_electra, worksfor_electra.columns[1:])
for i in range(len(ranking_worksfor_electra_normal)):
    ranking_worksfor_electra_normal_tokens.append(extrair_tokens(ranking_worksfor_electra_normal[i]))

# One Shoot
ranking_worksfor_electra_normal_tokens_one = []
ranking_worksfor_electra_normal_one = ranking_dataframe(worksfor_electra_one, worksfor_electra_one.columns[1:])
for i in range(len(ranking_worksfor_electra_normal_one)):
    ranking_worksfor_electra_normal_tokens_one.append(extrair_tokens(ranking_worksfor_electra_normal_one[i]))

# Few Shoot
ranking_worksfor_electra_normal_tokens_few = []
ranking_worksfor_electra_normal_few = ranking_dataframe(worksfor_electra_few, worksfor_electra_few.columns[1:])
for i in range(len(ranking_worksfor_electra_normal_few)):
    ranking_worksfor_electra_normal_tokens_few.append(extrair_tokens(ranking_worksfor_electra_normal_few[i]))

In [ ]:
# Spacy
ranking_worksfor_electra_spacy_tokens = []
ranking_worksfor_electra_spacy = ranking_dataframe(worksfor_electra_spacy, worksfor_electra_spacy.columns[1:])
for i in range(len(ranking_worksfor_electra_spacy)):
    ranking_worksfor_electra_spacy_tokens.append(extrair_tokens(ranking_worksfor_electra_spacy[i]))

# One Shoot
ranking_worksfor_electra_spacy_tokens_one = []
ranking_worksfor_electra_one_spacy = ranking_dataframe(worksfor_electra_one_spacy, worksfor_electra_one_spacy.columns[1:])
for i in range(len(ranking_worksfor_electra_one_spacy)):
    ranking_worksfor_electra_spacy_tokens_one.append(extrair_tokens(ranking_worksfor_electra_one_spacy[i]))

# Few Shoot
ranking_worksfor_electra_spacy_tokens_few = []
ranking_worksfor_electra_few_spacy = ranking_dataframe(worksfor_electra_few_spacy, worksfor_electra_few_spacy.columns[1:])
for i in range(len(ranking_worksfor_electra_few_spacy)):
    ranking_worksfor_electra_spacy_tokens_few.append(extrair_tokens(ranking_worksfor_electra_few_spacy[i]))

#### Capital

In [ ]:
# Normal
ranking_capital_electra_normal_tokens = []
ranking_capital_electra_normal = ranking_dataframe(capital_electra, capital_electra.columns[1:])
for i in range(len(ranking_capital_electra_normal)):
    ranking_capital_electra_normal_tokens.append(extrair_tokens(ranking_capital_electra_normal[i]))

# One Shoot
ranking_capital_electra_normal_tokens_one = []
ranking_capital_electra_normal_one = ranking_dataframe(capital_electra_one, capital_electra_one.columns[1:])
for i in range(len(ranking_capital_electra_normal_one)):
    ranking_capital_electra_normal_tokens_one.append(extrair_tokens(ranking_capital_electra_normal_one[i]))

# Few Shoot
ranking_capital_electra_normal_tokens_few = []
ranking_capital_electra_normal_few = ranking_dataframe(capital_electra_few, capital_electra_few.columns[1:])
for i in range(len(ranking_capital_electra_normal_few)):
    ranking_capital_electra_normal_tokens_few.append(extrair_tokens(ranking_capital_electra_normal_few[i]))

In [ ]:
# Spacy
ranking_capital_electra_spacy_tokens = []
ranking_capital_electra_spacy = ranking_dataframe(capital_electra_spacy, capital_electra_spacy.columns[1:])
for i in range(len(ranking_capital_electra_spacy)):
    ranking_capital_electra_spacy_tokens.append(extrair_tokens(ranking_capital_electra_spacy[i]))

# One Shoot
ranking_capital_electra_spacy_tokens_one = []
ranking_capital_electra_one_spacy = ranking_dataframe(capital_electra_one_spacy, capital_electra_one_spacy.columns[1:])
for i in range(len(ranking_capital_electra_one_spacy)):
    ranking_capital_electra_spacy_tokens_one.append(extrair_tokens(ranking_capital_electra_one_spacy[i]))

# Few Shoot
ranking_capital_electra_spacy_tokens_few = []
ranking_capital_electra_few_spacy = ranking_dataframe(capital_electra_few_spacy, capital_electra_few_spacy.columns[1:])
for i in range(len(ranking_capital_electra_few_spacy)):
    ranking_capital_electra_spacy_tokens_few.append(extrair_tokens(ranking_capital_electra_few_spacy[i]))

#### Citizen

In [ ]:
# Normal
ranking_citizen_electra_normal_tokens = []
ranking_citizen_electra_normal = ranking_dataframe(citizen_electra, citizen_electra.columns[1:])
for i in range(len(ranking_citizen_electra_normal)):
    ranking_citizen_electra_normal_tokens.append(extrair_tokens(ranking_citizen_electra_normal[i]))

# One Shoot
ranking_citizen_electra_normal_tokens_one = []
ranking_citizen_electra_normal_one = ranking_dataframe(citizen_electra_one, citizen_electra_one.columns[1:])
for i in range(len(ranking_citizen_electra_normal_one)):
    ranking_citizen_electra_normal_tokens_one.append(extrair_tokens(ranking_citizen_electra_normal_one[i]))

# Few Shoot
ranking_citizen_electra_normal_tokens_few = []
ranking_citizen_electra_normal_few = ranking_dataframe(citizen_electra_few, citizen_electra_few.columns[1:])
for i in range(len(ranking_citizen_electra_normal_few)):
    ranking_citizen_electra_normal_tokens_few.append(extrair_tokens(ranking_citizen_electra_normal_few[i]))

In [ ]:
# Spacy
ranking_citizen_electra_spacy_tokens = []
ranking_citizen_electra_spacy = ranking_dataframe(citizen_electra_spacy, citizen_electra_spacy.columns[1:])
for i in range(len(ranking_citizen_electra_spacy)):
    ranking_citizen_electra_spacy_tokens.append(extrair_tokens(ranking_citizen_electra_spacy[i]))

# One Shoot
ranking_citizen_electra_spacy_tokens_one = []
ranking_citizen_electra_one_spacy = ranking_dataframe(citizen_electra_one_spacy, citizen_electra_one_spacy.columns[1:])
for i in range(len(ranking_citizen_electra_one_spacy)):
    ranking_citizen_electra_spacy_tokens_one.append(extrair_tokens(ranking_citizen_electra_one_spacy[i]))

# Few Shoot
ranking_citizen_electra_spacy_tokens_few = []
ranking_citizen_electra_few_spacy = ranking_dataframe(citizen_electra_few_spacy, citizen_electra_few_spacy.columns[1:])
for i in range(len(ranking_citizen_electra_few_spacy)):
    ranking_citizen_electra_spacy_tokens_few.append(extrair_tokens(ranking_citizen_electra_few_spacy[i]))

#### Language

In [ ]:
# Normal
ranking_language_electra_normal_tokens = []
ranking_language_electra_normal = ranking_dataframe(language_electra, language_electra.columns[1:])
for i in range(len(ranking_language_electra_normal)):
    ranking_language_electra_normal_tokens.append(extrair_tokens(ranking_language_electra_normal[i]))

# One Shoot
ranking_language_electra_normal_tokens_one = []
ranking_language_electra_normal_one = ranking_dataframe(language_electra_one, language_electra_one.columns[1:])
for i in range(len(ranking_language_electra_normal_one)):
    ranking_language_electra_normal_tokens_one.append(extrair_tokens(ranking_language_electra_normal_one[i]))

# Few Shoot
ranking_language_electra_normal_tokens_few = []
ranking_language_electra_normal_few = ranking_dataframe(language_electra_few, language_electra_few.columns[1:])
for i in range(len(ranking_language_electra_normal_few)):
    ranking_language_electra_normal_tokens_few.append(extrair_tokens(ranking_language_electra_normal_few[i]))

In [ ]:
# Spacy
ranking_language_electra_spacy_tokens = []
ranking_language_electra_spacy = ranking_dataframe(language_electra_spacy, language_electra_spacy.columns[1:])
for i in range(len(ranking_language_electra_spacy)):
    ranking_language_electra_spacy_tokens.append(extrair_tokens(ranking_language_electra_spacy[i]))

# One Shoot
ranking_language_electra_spacy_tokens_one = []
ranking_language_electra_one_spacy = ranking_dataframe(language_electra_one_spacy, language_electra_one_spacy.columns[1:])
for i in range(len(ranking_language_electra_one_spacy)):
    ranking_language_electra_spacy_tokens_one.append(extrair_tokens(ranking_language_electra_one_spacy[i]))

# Few Shoot
ranking_language_electra_spacy_tokens_few = []
ranking_language_electra_few_spacy = ranking_dataframe(language_electra_few_spacy, language_electra_few_spacy.columns[1:])
for i in range(len(ranking_language_electra_few_spacy)):
    ranking_language_electra_spacy_tokens_few.append(extrair_tokens(ranking_language_electra_few_spacy[i]))

# Borda

## Bert

#### BornIn

In [ ]:
bornIn_bert_borda_score = aplicar_calculo_borda_dataframe(bornIn_bert, bornIn_bert.columns[1:])
bornIn_bert_borda_score_one = aplicar_calculo_borda_dataframe(bornIn_bert_one, bornIn_bert_one.columns[1:])
bornIn_bert_borda_score_few = aplicar_calculo_borda_dataframe(bornIn_bert_few, bornIn_bert_few.columns[1:])

bornIn_bert_borda_tokens_score = []
ranking_bornIn_bert_borda_score = ranking_dataframe(bornIn_bert_borda_score, bornIn_bert_borda_score.columns[1:])
for i in range(len(ranking_bornIn_bert_borda_score)):
    bornIn_bert_borda_tokens_score.append(extrair_tokens(ranking_bornIn_bert_borda_score[i]))

bornIn_bert_borda_tokens_score_one = []
ranking_bornIn_bert_borda_score_one = ranking_dataframe(bornIn_bert_borda_score_one, bornIn_bert_borda_score_one.columns[1:])
for i in range(len(ranking_bornIn_bert_borda_score_one)):
    bornIn_bert_borda_tokens_score_one.append(extrair_tokens(ranking_bornIn_bert_borda_score_one[i]))

bornIn_bert_borda_tokens_score_few = []
ranking_bornIn_bert_borda_score_few = ranking_dataframe(bornIn_bert_borda_score_few, bornIn_bert_borda_score_few.columns[1:])
for i in range(len(ranking_bornIn_bert_borda_score_few)):
    bornIn_bert_borda_tokens_score_few.append(extrair_tokens(ranking_bornIn_bert_borda_score_few[i]))

In [ ]:
bornIn_bert_borda_score_spacy = aplicar_calculo_borda_dataframe(bornIn_bert_spacy, bornIn_bert_spacy.columns[1:])
bornIn_bert_borda_score_spacy_one = aplicar_calculo_borda_dataframe(bornIn_bert_one_spacy, bornIn_bert_one_spacy.columns[1:])
bornIn_bert_borda_score_spacy_few = aplicar_calculo_borda_dataframe(bornIn_bert_few_spacy, bornIn_bert_few_spacy.columns[1:])

bornIn_bert_borda_spacy_tokens_score = []
ranking_bornIn_bert_borda_spacy_score = ranking_dataframe(bornIn_bert_borda_score_spacy, bornIn_bert_borda_score_spacy.columns[1:])
for i in range(len(ranking_bornIn_bert_borda_spacy_score)):
    bornIn_bert_borda_spacy_tokens_score.append(extrair_tokens(ranking_bornIn_bert_borda_spacy_score[i]))

bornIn_bert_borda_spacy_tokens_score_one = []
ranking_bornIn_bert_borda_spacy_score_one = ranking_dataframe(bornIn_bert_borda_score_spacy_one, bornIn_bert_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_bornIn_bert_borda_spacy_score_one)):
    bornIn_bert_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_bornIn_bert_borda_spacy_score_one[i]))

bornIn_bert_borda_spacy_tokens_score_few = []
ranking_bornIn_bert_borda_spacy_score_few = ranking_dataframe(bornIn_bert_borda_score_spacy_few, bornIn_bert_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_bornIn_bert_borda_spacy_score_few)):
    bornIn_bert_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_bornIn_bert_borda_spacy_score_few[i]))

#### DiedIn

In [ ]:
diedIn_bert_borda_score = aplicar_calculo_borda_dataframe(diedIn_bert, diedIn_bert.columns[1:])
diedIn_bert_borda_score_one = aplicar_calculo_borda_dataframe(diedIn_bert_one, diedIn_bert_one.columns[1:])
diedIn_bert_borda_score_few = aplicar_calculo_borda_dataframe(diedIn_bert_few, diedIn_bert_few.columns[1:])

diedIn_bert_borda_tokens_score = []
ranking_diedIn_bert_borda_score = ranking_dataframe(diedIn_bert_borda_score, diedIn_bert_borda_score.columns[1:])
for i in range(len(ranking_diedIn_bert_borda_score)):
    diedIn_bert_borda_tokens_score.append(extrair_tokens(ranking_diedIn_bert_borda_score[i]))

diedIn_bert_borda_tokens_score_one = []
ranking_diedIn_bert_borda_score_one = ranking_dataframe(diedIn_bert_borda_score_one, diedIn_bert_borda_score_one.columns[1:])
for i in range(len(ranking_diedIn_bert_borda_score_one)):
    diedIn_bert_borda_tokens_score_one.append(extrair_tokens(ranking_diedIn_bert_borda_score_one[i]))

diedIn_bert_borda_tokens_score_few = []
ranking_diedIn_bert_borda_score_few = ranking_dataframe(diedIn_bert_borda_score_few, diedIn_bert_borda_score_few.columns[1:])
for i in range(len(ranking_diedIn_bert_borda_score_few)):
    diedIn_bert_borda_tokens_score_few.append(extrair_tokens(ranking_diedIn_bert_borda_score_few[i]))

In [ ]:
diedIn_bert_borda_score_spacy = aplicar_calculo_borda_dataframe(diedIn_bert_spacy, diedIn_bert_spacy.columns[1:])
diedIn_bert_borda_score_spacy_one = aplicar_calculo_borda_dataframe(diedIn_bert_one_spacy, diedIn_bert_one_spacy.columns[1:])
diedIn_bert_borda_score_spacy_few = aplicar_calculo_borda_dataframe(diedIn_bert_few_spacy, diedIn_bert_few_spacy.columns[1:])

diedIn_bert_borda_spacy_tokens_score = []
ranking_diedIn_bert_borda_spacy_score = ranking_dataframe(diedIn_bert_borda_score_spacy, diedIn_bert_borda_score_spacy.columns[1:])
for i in range(len(ranking_diedIn_bert_borda_spacy_score)):
    diedIn_bert_borda_spacy_tokens_score.append(extrair_tokens(ranking_diedIn_bert_borda_spacy_score[i]))

diedIn_bert_borda_spacy_tokens_score_one = []
ranking_diedIn_bert_borda_spacy_score_one = ranking_dataframe(diedIn_bert_borda_score_spacy_one, diedIn_bert_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_diedIn_bert_borda_spacy_score_one)):
    diedIn_bert_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_diedIn_bert_borda_spacy_score_one[i]))

diedIn_bert_borda_spacy_tokens_score_few = []
ranking_diedIn_bert_borda_spacy_score_few = ranking_dataframe(diedIn_bert_borda_score_spacy_few, diedIn_bert_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_diedIn_bert_borda_spacy_score_few)):
    diedIn_bert_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_diedIn_bert_borda_spacy_score_few[i]))

#### Worksfor

In [ ]:
worksfor_bert_borda_score = aplicar_calculo_borda_dataframe(worksfor_bert, worksfor_bert.columns[1:])
worksfor_bert_borda_score_one = aplicar_calculo_borda_dataframe(worksfor_bert_one, worksfor_bert_one.columns[1:])
worksfor_bert_borda_score_few = aplicar_calculo_borda_dataframe(worksfor_bert_few, worksfor_bert_few.columns[1:])

worksfor_bert_borda_tokens_score = []
ranking_worksfor_bert_borda_score = ranking_dataframe(worksfor_bert_borda_score, worksfor_bert_borda_score.columns[1:])
for i in range(len(ranking_worksfor_bert_borda_score)):
    worksfor_bert_borda_tokens_score.append(extrair_tokens(ranking_worksfor_bert_borda_score[i]))

worksfor_bert_borda_tokens_score_one = []
ranking_worksfor_bert_borda_score_one = ranking_dataframe(worksfor_bert_borda_score_one, worksfor_bert_borda_score_one.columns[1:])
for i in range(len(ranking_worksfor_bert_borda_score_one)):
    worksfor_bert_borda_tokens_score_one.append(extrair_tokens(ranking_worksfor_bert_borda_score_one[i]))

worksfor_bert_borda_tokens_score_few = []
ranking_worksfor_bert_borda_score_few = ranking_dataframe(worksfor_bert_borda_score_few, worksfor_bert_borda_score_few.columns[1:])
for i in range(len(ranking_worksfor_bert_borda_score_few)):
    worksfor_bert_borda_tokens_score_few.append(extrair_tokens(ranking_worksfor_bert_borda_score_few[i]))

In [ ]:
worksfor_bert_borda_score_spacy = aplicar_calculo_borda_dataframe(worksfor_bert_spacy, worksfor_bert_spacy.columns[1:])
worksfor_bert_borda_score_spacy_one = aplicar_calculo_borda_dataframe(worksfor_bert_one_spacy, worksfor_bert_one_spacy.columns[1:])
worksfor_bert_borda_score_spacy_few = aplicar_calculo_borda_dataframe(worksfor_bert_few_spacy, worksfor_bert_few_spacy.columns[1:])

worksfor_bert_borda_spacy_tokens_score = []
ranking_worksfor_bert_borda_spacy_score = ranking_dataframe(worksfor_bert_borda_score_spacy, worksfor_bert_borda_score_spacy.columns[1:])
for i in range(len(ranking_worksfor_bert_borda_spacy_score)):
    worksfor_bert_borda_spacy_tokens_score.append(extrair_tokens(ranking_worksfor_bert_borda_spacy_score[i]))

worksfor_bert_borda_spacy_tokens_score_one = []
ranking_worksfor_bert_borda_spacy_score_one = ranking_dataframe(worksfor_bert_borda_score_spacy_one, worksfor_bert_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_worksfor_bert_borda_spacy_score_one)):
    worksfor_bert_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_worksfor_bert_borda_spacy_score_one[i]))

worksfor_bert_borda_spacy_tokens_score_few = []
ranking_worksfor_bert_borda_spacy_score_few = ranking_dataframe(worksfor_bert_borda_score_spacy_few, worksfor_bert_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_worksfor_bert_borda_spacy_score_few)):
    worksfor_bert_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_worksfor_bert_borda_spacy_score_few[i]))

#### Capital

In [ ]:
capital_bert_borda_score = aplicar_calculo_borda_dataframe(capital_bert, capital_bert.columns[1:])
capital_bert_borda_score_one = aplicar_calculo_borda_dataframe(capital_bert_one, capital_bert_one.columns[1:])
capital_bert_borda_score_few = aplicar_calculo_borda_dataframe(capital_bert_few, capital_bert_few.columns[1:])

capital_bert_borda_tokens_score = []
ranking_capital_bert_borda_score = ranking_dataframe(capital_bert_borda_score, capital_bert_borda_score.columns[1:])
for i in range(len(ranking_capital_bert_borda_score)):
    capital_bert_borda_tokens_score.append(extrair_tokens(ranking_capital_bert_borda_score[i]))

capital_bert_borda_tokens_score_one = []
ranking_capital_bert_borda_score_one = ranking_dataframe(capital_bert_borda_score_one, capital_bert_borda_score_one.columns[1:])
for i in range(len(ranking_capital_bert_borda_score_one)):
    capital_bert_borda_tokens_score_one.append(extrair_tokens(ranking_capital_bert_borda_score_one[i]))

capital_bert_borda_tokens_score_few = []
ranking_capital_bert_borda_score_few = ranking_dataframe(capital_bert_borda_score_few, capital_bert_borda_score_few.columns[1:])
for i in range(len(ranking_capital_bert_borda_score_few)):
    capital_bert_borda_tokens_score_few.append(extrair_tokens(ranking_capital_bert_borda_score_few[i]))

In [ ]:
capital_bert_borda_score_spacy = aplicar_calculo_borda_dataframe(capital_bert_spacy, capital_bert_spacy.columns[1:])
capital_bert_borda_score_spacy_one = aplicar_calculo_borda_dataframe(capital_bert_one_spacy, capital_bert_one_spacy.columns[1:])
capital_bert_borda_score_spacy_few = aplicar_calculo_borda_dataframe(capital_bert_few_spacy, capital_bert_few_spacy.columns[1:])

capital_bert_borda_spacy_tokens_score = []
ranking_capital_bert_borda_spacy_score = ranking_dataframe(capital_bert_borda_score_spacy, capital_bert_borda_score_spacy.columns[1:])
for i in range(len(ranking_capital_bert_borda_spacy_score)):
    capital_bert_borda_spacy_tokens_score.append(extrair_tokens(ranking_capital_bert_borda_spacy_score[i]))

capital_bert_borda_spacy_tokens_score_one = []
ranking_capital_bert_borda_spacy_score_one = ranking_dataframe(capital_bert_borda_score_spacy_one, capital_bert_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_capital_bert_borda_spacy_score_one)):
    capital_bert_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_capital_bert_borda_spacy_score_one[i]))

capital_bert_borda_spacy_tokens_score_few = []
ranking_capital_bert_borda_spacy_score_few = ranking_dataframe(capital_bert_borda_score_spacy_few, capital_bert_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_capital_bert_borda_spacy_score_few)):
    capital_bert_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_capital_bert_borda_spacy_score_few[i]))

#### Citizen

In [ ]:
citizen_bert_borda_score = aplicar_calculo_borda_dataframe(citizen_bert, citizen_bert.columns[1:])
citizen_bert_borda_score_one = aplicar_calculo_borda_dataframe(citizen_bert_one, citizen_bert_one.columns[1:])
citizen_bert_borda_score_few = aplicar_calculo_borda_dataframe(citizen_bert_few, citizen_bert_few.columns[1:])

citizen_bert_borda_tokens_score = []
ranking_citizen_bert_borda_score = ranking_dataframe(citizen_bert_borda_score, citizen_bert_borda_score.columns[1:])
for i in range(len(ranking_citizen_bert_borda_score)):
    citizen_bert_borda_tokens_score.append(extrair_tokens(ranking_citizen_bert_borda_score[i]))

citizen_bert_borda_tokens_score_one = []
ranking_citizen_bert_borda_score_one = ranking_dataframe(citizen_bert_borda_score_one, citizen_bert_borda_score_one.columns[1:])
for i in range(len(ranking_citizen_bert_borda_score_one)):
    citizen_bert_borda_tokens_score_one.append(extrair_tokens(ranking_citizen_bert_borda_score_one[i]))

citizen_bert_borda_tokens_score_few = []
ranking_citizen_bert_borda_score_few = ranking_dataframe(citizen_bert_borda_score_few, citizen_bert_borda_score_few.columns[1:])
for i in range(len(ranking_citizen_bert_borda_score_few)):
    citizen_bert_borda_tokens_score_few.append(extrair_tokens(ranking_citizen_bert_borda_score_few[i]))

In [ ]:
citizen_bert_borda_score_spacy = aplicar_calculo_borda_dataframe(citizen_bert_spacy, citizen_bert_spacy.columns[1:])
citizen_bert_borda_score_spacy_one = aplicar_calculo_borda_dataframe(citizen_bert_one_spacy, citizen_bert_one_spacy.columns[1:])
citizen_bert_borda_score_spacy_few = aplicar_calculo_borda_dataframe(citizen_bert_few_spacy, citizen_bert_few_spacy.columns[1:])

citizen_bert_borda_spacy_tokens_score = []
ranking_citizen_bert_borda_spacy_score = ranking_dataframe(citizen_bert_borda_score_spacy, citizen_bert_borda_score_spacy.columns[1:])
for i in range(len(ranking_citizen_bert_borda_spacy_score)):
    citizen_bert_borda_spacy_tokens_score.append(extrair_tokens(ranking_citizen_bert_borda_spacy_score[i]))

citizen_bert_borda_spacy_tokens_score_one = []
ranking_citizen_bert_borda_spacy_score_one = ranking_dataframe(citizen_bert_borda_score_spacy_one, citizen_bert_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_citizen_bert_borda_spacy_score_one)):
    citizen_bert_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_citizen_bert_borda_spacy_score_one[i]))

citizen_bert_borda_spacy_tokens_score_few = []
ranking_citizen_bert_borda_spacy_score_few = ranking_dataframe(citizen_bert_borda_score_spacy_few, citizen_bert_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_citizen_bert_borda_spacy_score_few)):
    citizen_bert_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_citizen_bert_borda_spacy_score_few[i]))

#### Language

In [ ]:
language_bert_borda_score = aplicar_calculo_borda_dataframe(language_bert, language_bert.columns[1:])
language_bert_borda_score_one = aplicar_calculo_borda_dataframe(language_bert_one, language_bert_one.columns[1:])
language_bert_borda_score_few = aplicar_calculo_borda_dataframe(language_bert_few, language_bert_few.columns[1:])

language_bert_borda_tokens_score = []
ranking_language_bert_borda_score = ranking_dataframe(language_bert_borda_score, language_bert_borda_score.columns[1:])
for i in range(len(ranking_language_bert_borda_score)):
    language_bert_borda_tokens_score.append(extrair_tokens(ranking_language_bert_borda_score[i]))

language_bert_borda_tokens_score_one = []
ranking_language_bert_borda_score_one = ranking_dataframe(language_bert_borda_score_one, language_bert_borda_score_one.columns[1:])
for i in range(len(ranking_language_bert_borda_score_one)):
    language_bert_borda_tokens_score_one.append(extrair_tokens(ranking_language_bert_borda_score_one[i]))

language_bert_borda_tokens_score_few = []
ranking_language_bert_borda_score_few = ranking_dataframe(language_bert_borda_score_few, language_bert_borda_score_few.columns[1:])
for i in range(len(ranking_language_bert_borda_score_few)):
    language_bert_borda_tokens_score_few.append(extrair_tokens(ranking_language_bert_borda_score_few[i]))

In [ ]:
language_bert_borda_score_spacy = aplicar_calculo_borda_dataframe(language_bert_spacy, language_bert_spacy.columns[1:])
language_bert_borda_score_spacy_one = aplicar_calculo_borda_dataframe(language_bert_one_spacy, language_bert_one_spacy.columns[1:])
language_bert_borda_score_spacy_few = aplicar_calculo_borda_dataframe(language_bert_few_spacy, language_bert_few_spacy.columns[1:])

language_bert_borda_spacy_tokens_score = []
ranking_language_bert_borda_spacy_score = ranking_dataframe(language_bert_borda_score_spacy, language_bert_borda_score_spacy.columns[1:])
for i in range(len(ranking_language_bert_borda_spacy_score)):
    language_bert_borda_spacy_tokens_score.append(extrair_tokens(ranking_language_bert_borda_spacy_score[i]))

language_bert_borda_spacy_tokens_score_one = []
ranking_language_bert_borda_spacy_score_one = ranking_dataframe(language_bert_borda_score_spacy_one, language_bert_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_language_bert_borda_spacy_score_one)):
    language_bert_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_language_bert_borda_spacy_score_one[i]))

language_bert_borda_spacy_tokens_score_few = []
ranking_language_bert_borda_spacy_score_few = ranking_dataframe(language_bert_borda_score_spacy_few, language_bert_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_language_bert_borda_spacy_score_few)):
    language_bert_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_language_bert_borda_spacy_score_few[i]))

## Roberta

#### BornIn

In [ ]:
bornIn_roberta_borda_score = aplicar_calculo_borda_dataframe(bornIn_roberta, bornIn_roberta.columns[1:])
bornIn_roberta_borda_score_one = aplicar_calculo_borda_dataframe(bornIn_roberta_one, bornIn_roberta_one.columns[1:])
bornIn_roberta_borda_score_few = aplicar_calculo_borda_dataframe(bornIn_roberta_few, bornIn_roberta_few.columns[1:])

bornIn_roberta_borda_tokens_score = []
ranking_bornIn_roberta_borda_score = ranking_dataframe(bornIn_roberta_borda_score, bornIn_roberta_borda_score.columns[1:])
for i in range(len(ranking_bornIn_roberta_borda_score)):
    bornIn_roberta_borda_tokens_score.append(extrair_tokens(ranking_bornIn_roberta_borda_score[i]))

bornIn_roberta_borda_tokens_score_one = []
ranking_bornIn_roberta_borda_score_one = ranking_dataframe(bornIn_roberta_borda_score_one, bornIn_roberta_borda_score_one.columns[1:])
for i in range(len(ranking_bornIn_roberta_borda_score_one)):
    bornIn_roberta_borda_tokens_score_one.append(extrair_tokens(ranking_bornIn_roberta_borda_score_one[i]))

bornIn_roberta_borda_tokens_score_few = []
ranking_bornIn_roberta_borda_score_few = ranking_dataframe(bornIn_roberta_borda_score_few, bornIn_roberta_borda_score_few.columns[1:])
for i in range(len(ranking_bornIn_roberta_borda_score_few)):
    bornIn_roberta_borda_tokens_score_few.append(extrair_tokens(ranking_bornIn_roberta_borda_score_few[i]))

In [ ]:
bornIn_roberta_borda_score_spacy = aplicar_calculo_borda_dataframe(bornIn_roberta_spacy, bornIn_roberta_spacy.columns[1:])
bornIn_roberta_borda_score_spacy_one = aplicar_calculo_borda_dataframe(bornIn_roberta_one_spacy, bornIn_roberta_one_spacy.columns[1:])
bornIn_roberta_borda_score_spacy_few = aplicar_calculo_borda_dataframe(bornIn_roberta_few_spacy, bornIn_roberta_few_spacy.columns[1:])

bornIn_roberta_borda_spacy_tokens_score = []
ranking_bornIn_roberta_borda_spacy_score = ranking_dataframe(bornIn_roberta_borda_score_spacy, bornIn_roberta_borda_score_spacy.columns[1:])
for i in range(len(ranking_bornIn_roberta_borda_spacy_score)):
    bornIn_roberta_borda_spacy_tokens_score.append(extrair_tokens(ranking_bornIn_roberta_borda_spacy_score[i]))

bornIn_roberta_borda_spacy_tokens_score_one = []
ranking_bornIn_roberta_borda_spacy_score_one = ranking_dataframe(bornIn_roberta_borda_score_spacy_one, bornIn_roberta_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_bornIn_roberta_borda_spacy_score_one)):
    bornIn_roberta_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_bornIn_roberta_borda_spacy_score_one[i]))

bornIn_roberta_borda_spacy_tokens_score_few = []
ranking_bornIn_roberta_borda_spacy_score_few = ranking_dataframe(bornIn_roberta_borda_score_spacy_few, bornIn_roberta_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_bornIn_roberta_borda_spacy_score_few)):
    bornIn_roberta_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_bornIn_roberta_borda_spacy_score_few[i]))

#### DiedIn

In [ ]:
diedIn_roberta_borda_score = aplicar_calculo_borda_dataframe(diedIn_roberta, diedIn_roberta.columns[1:])
diedIn_roberta_borda_score_one = aplicar_calculo_borda_dataframe(diedIn_roberta_one, diedIn_roberta_one.columns[1:])
diedIn_roberta_borda_score_few = aplicar_calculo_borda_dataframe(diedIn_roberta_few, diedIn_roberta_few.columns[1:])

diedIn_roberta_borda_tokens_score = []
ranking_diedIn_roberta_borda_score = ranking_dataframe(diedIn_roberta_borda_score, diedIn_roberta_borda_score.columns[1:])
for i in range(len(ranking_diedIn_roberta_borda_score)):
    diedIn_roberta_borda_tokens_score.append(extrair_tokens(ranking_diedIn_roberta_borda_score[i]))

diedIn_roberta_borda_tokens_score_one = []
ranking_diedIn_roberta_borda_score_one = ranking_dataframe(diedIn_roberta_borda_score_one, diedIn_roberta_borda_score_one.columns[1:])
for i in range(len(ranking_diedIn_roberta_borda_score_one)):
    diedIn_roberta_borda_tokens_score_one.append(extrair_tokens(ranking_diedIn_roberta_borda_score_one[i]))

diedIn_roberta_borda_tokens_score_few = []
ranking_diedIn_roberta_borda_score_few = ranking_dataframe(diedIn_roberta_borda_score_few, diedIn_roberta_borda_score_few.columns[1:])
for i in range(len(ranking_diedIn_roberta_borda_score_few)):
    diedIn_roberta_borda_tokens_score_few.append(extrair_tokens(ranking_diedIn_roberta_borda_score_few[i]))

In [ ]:
diedIn_roberta_borda_score_spacy = aplicar_calculo_borda_dataframe(diedIn_roberta_spacy, diedIn_roberta_spacy.columns[1:])
diedIn_roberta_borda_score_spacy_one = aplicar_calculo_borda_dataframe(diedIn_roberta_one_spacy, diedIn_roberta_one_spacy.columns[1:])
diedIn_roberta_borda_score_spacy_few = aplicar_calculo_borda_dataframe(diedIn_roberta_few_spacy, diedIn_roberta_few_spacy.columns[1:])

diedIn_roberta_borda_spacy_tokens_score = []
ranking_diedIn_roberta_borda_spacy_score = ranking_dataframe(diedIn_roberta_borda_score_spacy, diedIn_roberta_borda_score_spacy.columns[1:])
for i in range(len(ranking_diedIn_roberta_borda_spacy_score)):
    diedIn_roberta_borda_spacy_tokens_score.append(extrair_tokens(ranking_diedIn_roberta_borda_spacy_score[i]))

diedIn_roberta_borda_spacy_tokens_score_one = []
ranking_diedIn_roberta_borda_spacy_score_one = ranking_dataframe(diedIn_roberta_borda_score_spacy_one, diedIn_roberta_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_diedIn_roberta_borda_spacy_score_one)):
    diedIn_roberta_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_diedIn_roberta_borda_spacy_score_one[i]))

diedIn_roberta_borda_spacy_tokens_score_few = []
ranking_diedIn_roberta_borda_spacy_score_few = ranking_dataframe(diedIn_roberta_borda_score_spacy_few, diedIn_roberta_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_diedIn_roberta_borda_spacy_score_few)):
    diedIn_roberta_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_diedIn_roberta_borda_spacy_score_few[i]))

#### Worksfor

In [ ]:
worksfor_roberta_borda_score = aplicar_calculo_borda_dataframe(worksfor_roberta, worksfor_roberta.columns[1:])
worksfor_roberta_borda_score_one = aplicar_calculo_borda_dataframe(worksfor_roberta_one, worksfor_roberta_one.columns[1:])
worksfor_roberta_borda_score_few = aplicar_calculo_borda_dataframe(worksfor_roberta_few, worksfor_roberta_few.columns[1:])

worksfor_roberta_borda_tokens_score = []
ranking_worksfor_roberta_borda_score = ranking_dataframe(worksfor_roberta_borda_score, worksfor_roberta_borda_score.columns[1:])
for i in range(len(ranking_worksfor_roberta_borda_score)):
    worksfor_roberta_borda_tokens_score.append(extrair_tokens(ranking_worksfor_roberta_borda_score[i]))

worksfor_roberta_borda_tokens_score_one = []
ranking_worksfor_roberta_borda_score_one = ranking_dataframe(worksfor_roberta_borda_score_one, worksfor_roberta_borda_score_one.columns[1:])
for i in range(len(ranking_worksfor_roberta_borda_score_one)):
    worksfor_roberta_borda_tokens_score_one.append(extrair_tokens(ranking_worksfor_roberta_borda_score_one[i]))

worksfor_roberta_borda_tokens_score_few = []
ranking_worksfor_roberta_borda_score_few = ranking_dataframe(worksfor_roberta_borda_score_few, worksfor_roberta_borda_score_few.columns[1:])
for i in range(len(ranking_worksfor_roberta_borda_score_few)):
    worksfor_roberta_borda_tokens_score_few.append(extrair_tokens(ranking_worksfor_roberta_borda_score_few[i]))

In [ ]:
worksfor_roberta_borda_score_spacy = aplicar_calculo_borda_dataframe(worksfor_roberta_spacy, worksfor_roberta_spacy.columns[1:])
worksfor_roberta_borda_score_spacy_one = aplicar_calculo_borda_dataframe(worksfor_roberta_one_spacy, worksfor_roberta_one_spacy.columns[1:])
worksfor_roberta_borda_score_spacy_few = aplicar_calculo_borda_dataframe(worksfor_roberta_few_spacy, worksfor_roberta_few_spacy.columns[1:])

worksfor_roberta_borda_spacy_tokens_score = []
ranking_worksfor_roberta_borda_spacy_score = ranking_dataframe(worksfor_roberta_borda_score_spacy, worksfor_roberta_borda_score_spacy.columns[1:])
for i in range(len(ranking_worksfor_roberta_borda_spacy_score)):
    worksfor_roberta_borda_spacy_tokens_score.append(extrair_tokens(ranking_worksfor_roberta_borda_spacy_score[i]))

worksfor_roberta_borda_spacy_tokens_score_one = []
ranking_worksfor_roberta_borda_spacy_score_one = ranking_dataframe(worksfor_roberta_borda_score_spacy_one, worksfor_roberta_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_worksfor_roberta_borda_spacy_score_one)):
    worksfor_roberta_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_worksfor_roberta_borda_spacy_score_one[i]))

worksfor_roberta_borda_spacy_tokens_score_few = []
ranking_worksfor_roberta_borda_spacy_score_few = ranking_dataframe(worksfor_roberta_borda_score_spacy_few, worksfor_roberta_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_worksfor_roberta_borda_spacy_score_few)):
    worksfor_roberta_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_worksfor_roberta_borda_spacy_score_few[i]))

#### Capital

In [ ]:
capital_roberta_borda_score = aplicar_calculo_borda_dataframe(capital_roberta, capital_roberta.columns[1:])
capital_roberta_borda_score_one = aplicar_calculo_borda_dataframe(capital_roberta_one, capital_roberta_one.columns[1:])
capital_roberta_borda_score_few = aplicar_calculo_borda_dataframe(capital_roberta_few, capital_roberta_few.columns[1:])

capital_roberta_borda_tokens_score = []
ranking_capital_roberta_borda_score = ranking_dataframe(capital_roberta_borda_score, capital_roberta_borda_score.columns[1:])
for i in range(len(ranking_capital_roberta_borda_score)):
    capital_roberta_borda_tokens_score.append(extrair_tokens(ranking_capital_roberta_borda_score[i]))

capital_roberta_borda_tokens_score_one = []
ranking_capital_roberta_borda_score_one = ranking_dataframe(capital_roberta_borda_score_one, capital_roberta_borda_score_one.columns[1:])
for i in range(len(ranking_capital_roberta_borda_score_one)):
    capital_roberta_borda_tokens_score_one.append(extrair_tokens(ranking_capital_roberta_borda_score_one[i]))

capital_roberta_borda_tokens_score_few = []
ranking_capital_roberta_borda_score_few = ranking_dataframe(capital_roberta_borda_score_few, capital_roberta_borda_score_few.columns[1:])
for i in range(len(ranking_capital_roberta_borda_score_few)):
    capital_roberta_borda_tokens_score_few.append(extrair_tokens(ranking_capital_roberta_borda_score_few[i]))

In [ ]:
capital_roberta_borda_score_spacy = aplicar_calculo_borda_dataframe(capital_roberta_spacy, capital_roberta_spacy.columns[1:])
capital_roberta_borda_score_spacy_one = aplicar_calculo_borda_dataframe(capital_roberta_one_spacy, capital_roberta_one_spacy.columns[1:])
capital_roberta_borda_score_spacy_few = aplicar_calculo_borda_dataframe(capital_roberta_few_spacy, capital_roberta_few_spacy.columns[1:])

capital_roberta_borda_spacy_tokens_score = []
ranking_capital_roberta_borda_spacy_score = ranking_dataframe(capital_roberta_borda_score_spacy, capital_roberta_borda_score_spacy.columns[1:])
for i in range(len(ranking_capital_roberta_borda_spacy_score)):
    capital_roberta_borda_spacy_tokens_score.append(extrair_tokens(ranking_capital_roberta_borda_spacy_score[i]))

capital_roberta_borda_spacy_tokens_score_one = []
ranking_capital_roberta_borda_spacy_score_one = ranking_dataframe(capital_roberta_borda_score_spacy_one, capital_roberta_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_capital_roberta_borda_spacy_score_one)):
    capital_roberta_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_capital_roberta_borda_spacy_score_one[i]))

capital_roberta_borda_spacy_tokens_score_few = []
ranking_capital_roberta_borda_spacy_score_few = ranking_dataframe(capital_roberta_borda_score_spacy_few, capital_roberta_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_capital_roberta_borda_spacy_score_few)):
    capital_roberta_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_capital_roberta_borda_spacy_score_few[i]))

#### Citizen

In [ ]:
citizen_roberta_borda_score = aplicar_calculo_borda_dataframe(citizen_roberta, citizen_roberta.columns[1:])
citizen_roberta_borda_score_one = aplicar_calculo_borda_dataframe(citizen_roberta_one, citizen_roberta_one.columns[1:])
citizen_roberta_borda_score_few = aplicar_calculo_borda_dataframe(citizen_roberta_few, citizen_roberta_few.columns[1:])

citizen_roberta_borda_tokens_score = []
ranking_citizen_roberta_borda_score = ranking_dataframe(citizen_roberta_borda_score, citizen_roberta_borda_score.columns[1:])
for i in range(len(ranking_citizen_roberta_borda_score)):
    citizen_roberta_borda_tokens_score.append(extrair_tokens(ranking_citizen_roberta_borda_score[i]))

citizen_roberta_borda_tokens_score_one = []
ranking_citizen_roberta_borda_score_one = ranking_dataframe(citizen_roberta_borda_score_one, citizen_roberta_borda_score_one.columns[1:])
for i in range(len(ranking_citizen_roberta_borda_score_one)):
    citizen_roberta_borda_tokens_score_one.append(extrair_tokens(ranking_citizen_roberta_borda_score_one[i]))

citizen_roberta_borda_tokens_score_few = []
ranking_citizen_roberta_borda_score_few = ranking_dataframe(citizen_roberta_borda_score_few, citizen_roberta_borda_score_few.columns[1:])
for i in range(len(ranking_citizen_roberta_borda_score_few)):
    citizen_roberta_borda_tokens_score_few.append(extrair_tokens(ranking_citizen_roberta_borda_score_few[i]))

In [ ]:
citizen_roberta_borda_score_spacy = aplicar_calculo_borda_dataframe(citizen_roberta_spacy, citizen_roberta_spacy.columns[1:])
citizen_roberta_borda_score_spacy_one = aplicar_calculo_borda_dataframe(citizen_roberta_one_spacy, citizen_roberta_one_spacy.columns[1:])
citizen_roberta_borda_score_spacy_few = aplicar_calculo_borda_dataframe(citizen_roberta_few_spacy, citizen_roberta_few_spacy.columns[1:])

citizen_roberta_borda_spacy_tokens_score = []
ranking_citizen_roberta_borda_spacy_score = ranking_dataframe(citizen_roberta_borda_score_spacy, citizen_roberta_borda_score_spacy.columns[1:])
for i in range(len(ranking_citizen_roberta_borda_spacy_score)):
    citizen_roberta_borda_spacy_tokens_score.append(extrair_tokens(ranking_citizen_roberta_borda_spacy_score[i]))

citizen_roberta_borda_spacy_tokens_score_one = []
ranking_citizen_roberta_borda_spacy_score_one = ranking_dataframe(citizen_roberta_borda_score_spacy_one, citizen_roberta_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_citizen_roberta_borda_spacy_score_one)):
    citizen_roberta_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_citizen_roberta_borda_spacy_score_one[i]))

citizen_roberta_borda_spacy_tokens_score_few = []
ranking_citizen_roberta_borda_spacy_score_few = ranking_dataframe(citizen_roberta_borda_score_spacy_few, citizen_roberta_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_citizen_roberta_borda_spacy_score_few)):
    citizen_roberta_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_citizen_roberta_borda_spacy_score_few[i]))

#### Language

In [ ]:
language_roberta_borda_score = aplicar_calculo_borda_dataframe(language_roberta, language_roberta.columns[1:])
language_roberta_borda_score_one = aplicar_calculo_borda_dataframe(language_roberta_one, language_roberta_one.columns[1:])
language_roberta_borda_score_few = aplicar_calculo_borda_dataframe(language_roberta_few, language_roberta_few.columns[1:])

language_roberta_borda_tokens_score = []
ranking_language_roberta_borda_score = ranking_dataframe(language_roberta_borda_score, language_roberta_borda_score.columns[1:])
for i in range(len(ranking_language_roberta_borda_score)):
    language_roberta_borda_tokens_score.append(extrair_tokens(ranking_language_roberta_borda_score[i]))

language_roberta_borda_tokens_score_one = []
ranking_language_roberta_borda_score_one = ranking_dataframe(language_roberta_borda_score_one, language_roberta_borda_score_one.columns[1:])
for i in range(len(ranking_language_roberta_borda_score_one)):
    language_roberta_borda_tokens_score_one.append(extrair_tokens(ranking_language_roberta_borda_score_one[i]))

language_roberta_borda_tokens_score_few = []
ranking_language_roberta_borda_score_few = ranking_dataframe(language_roberta_borda_score_few, language_roberta_borda_score_few.columns[1:])
for i in range(len(ranking_language_roberta_borda_score_few)):
    language_roberta_borda_tokens_score_few.append(extrair_tokens(ranking_language_roberta_borda_score_few[i]))

In [ ]:
language_roberta_borda_score_spacy = aplicar_calculo_borda_dataframe(language_roberta_spacy, language_roberta_spacy.columns[1:])
language_roberta_borda_score_spacy_one = aplicar_calculo_borda_dataframe(language_roberta_one_spacy, language_roberta_one_spacy.columns[1:])
language_roberta_borda_score_spacy_few = aplicar_calculo_borda_dataframe(language_roberta_few_spacy, language_roberta_few_spacy.columns[1:])

language_roberta_borda_spacy_tokens_score = []
ranking_language_roberta_borda_spacy_score = ranking_dataframe(language_roberta_borda_score_spacy, language_roberta_borda_score_spacy.columns[1:])
for i in range(len(ranking_language_roberta_borda_spacy_score)):
    language_roberta_borda_spacy_tokens_score.append(extrair_tokens(ranking_language_roberta_borda_spacy_score[i]))

language_roberta_borda_spacy_tokens_score_one = []
ranking_language_roberta_borda_spacy_score_one = ranking_dataframe(language_roberta_borda_score_spacy_one, language_roberta_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_language_roberta_borda_spacy_score_one)):
    language_roberta_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_language_roberta_borda_spacy_score_one[i]))

language_roberta_borda_spacy_tokens_score_few = []
ranking_language_roberta_borda_spacy_score_few = ranking_dataframe(language_roberta_borda_score_spacy_few, language_roberta_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_language_roberta_borda_spacy_score_few)):
    language_roberta_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_language_roberta_borda_spacy_score_few[i]))

## Electra

#### BornIn

In [ ]:
bornIn_electra_borda_score = aplicar_calculo_borda_dataframe(bornIn_electra, bornIn_electra.columns[1:])
bornIn_electra_borda_score_one = aplicar_calculo_borda_dataframe(bornIn_electra_one, bornIn_electra_one.columns[1:])
bornIn_electra_borda_score_few = aplicar_calculo_borda_dataframe(bornIn_electra_few, bornIn_electra_few.columns[1:])

bornIn_electra_borda_tokens_score = []
ranking_bornIn_electra_borda_score = ranking_dataframe(bornIn_electra_borda_score, bornIn_electra_borda_score.columns[1:])
for i in range(len(ranking_bornIn_electra_borda_score)):
    bornIn_electra_borda_tokens_score.append(extrair_tokens(ranking_bornIn_electra_borda_score[i]))

bornIn_electra_borda_tokens_score_one = []
ranking_bornIn_electra_borda_score_one = ranking_dataframe(bornIn_electra_borda_score_one, bornIn_electra_borda_score_one.columns[1:])
for i in range(len(ranking_bornIn_electra_borda_score_one)):
    bornIn_electra_borda_tokens_score_one.append(extrair_tokens(ranking_bornIn_electra_borda_score_one[i]))

bornIn_electra_borda_tokens_score_few = []
ranking_bornIn_electra_borda_score_few = ranking_dataframe(bornIn_electra_borda_score_few, bornIn_electra_borda_score_few.columns[1:])
for i in range(len(ranking_bornIn_electra_borda_score_few)):
    bornIn_electra_borda_tokens_score_few.append(extrair_tokens(ranking_bornIn_electra_borda_score_few[i]))

In [ ]:
bornIn_electra_borda_score_spacy = aplicar_calculo_borda_dataframe(bornIn_electra_spacy, bornIn_electra_spacy.columns[1:])
bornIn_electra_borda_score_spacy_one = aplicar_calculo_borda_dataframe(bornIn_electra_one_spacy, bornIn_electra_one_spacy.columns[1:])
bornIn_electra_borda_score_spacy_few = aplicar_calculo_borda_dataframe(bornIn_electra_few_spacy, bornIn_electra_few_spacy.columns[1:])

bornIn_electra_borda_spacy_tokens_score = []
ranking_bornIn_electra_borda_spacy_score = ranking_dataframe(bornIn_electra_borda_score_spacy, bornIn_electra_borda_score_spacy.columns[1:])
for i in range(len(ranking_bornIn_electra_borda_spacy_score)):
    bornIn_electra_borda_spacy_tokens_score.append(extrair_tokens(ranking_bornIn_electra_borda_spacy_score[i]))

bornIn_electra_borda_spacy_tokens_score_one = []
ranking_bornIn_electra_borda_spacy_score_one = ranking_dataframe(bornIn_electra_borda_score_spacy_one, bornIn_electra_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_bornIn_electra_borda_spacy_score_one)):
    bornIn_electra_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_bornIn_electra_borda_spacy_score_one[i]))

bornIn_electra_borda_spacy_tokens_score_few = []
ranking_bornIn_electra_borda_spacy_score_few = ranking_dataframe(bornIn_electra_borda_score_spacy_few, bornIn_electra_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_bornIn_electra_borda_spacy_score_few)):
    bornIn_electra_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_bornIn_electra_borda_spacy_score_few[i]))

#### DiedIn

In [ ]:
diedIn_electra_borda_score = aplicar_calculo_borda_dataframe(diedIn_electra, diedIn_electra.columns[1:])
diedIn_electra_borda_score_one = aplicar_calculo_borda_dataframe(diedIn_electra_one, diedIn_electra_one.columns[1:])
diedIn_electra_borda_score_few = aplicar_calculo_borda_dataframe(diedIn_electra_few, diedIn_electra_few.columns[1:])

diedIn_electra_borda_tokens_score = []
ranking_diedIn_electra_borda_score = ranking_dataframe(diedIn_electra_borda_score, diedIn_electra_borda_score.columns[1:])
for i in range(len(ranking_diedIn_electra_borda_score)):
    diedIn_electra_borda_tokens_score.append(extrair_tokens(ranking_diedIn_electra_borda_score[i]))

diedIn_electra_borda_tokens_score_one = []
ranking_diedIn_electra_borda_score_one = ranking_dataframe(diedIn_electra_borda_score_one, diedIn_electra_borda_score_one.columns[1:])
for i in range(len(ranking_diedIn_electra_borda_score_one)):
    diedIn_electra_borda_tokens_score_one.append(extrair_tokens(ranking_diedIn_electra_borda_score_one[i]))

diedIn_electra_borda_tokens_score_few = []
ranking_diedIn_electra_borda_score_few = ranking_dataframe(diedIn_electra_borda_score_few, diedIn_electra_borda_score_few.columns[1:])
for i in range(len(ranking_diedIn_electra_borda_score_few)):
    diedIn_electra_borda_tokens_score_few.append(extrair_tokens(ranking_diedIn_electra_borda_score_few[i]))

In [ ]:
diedIn_electra_borda_score_spacy = aplicar_calculo_borda_dataframe(diedIn_electra_spacy, diedIn_electra_spacy.columns[1:])
diedIn_electra_borda_score_spacy_one = aplicar_calculo_borda_dataframe(diedIn_electra_one_spacy, diedIn_electra_one_spacy.columns[1:])
diedIn_electra_borda_score_spacy_few = aplicar_calculo_borda_dataframe(diedIn_electra_few_spacy, diedIn_electra_few_spacy.columns[1:])

diedIn_electra_borda_spacy_tokens_score = []
ranking_diedIn_electra_borda_spacy_score = ranking_dataframe(diedIn_electra_borda_score_spacy, diedIn_electra_borda_score_spacy.columns[1:])
for i in range(len(ranking_diedIn_electra_borda_spacy_score)):
    diedIn_electra_borda_spacy_tokens_score.append(extrair_tokens(ranking_diedIn_electra_borda_spacy_score[i]))

diedIn_electra_borda_spacy_tokens_score_one = []
ranking_diedIn_electra_borda_spacy_score_one = ranking_dataframe(diedIn_electra_borda_score_spacy_one, diedIn_electra_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_diedIn_electra_borda_spacy_score_one)):
    diedIn_electra_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_diedIn_electra_borda_spacy_score_one[i]))

diedIn_electra_borda_spacy_tokens_score_few = []
ranking_diedIn_electra_borda_spacy_score_few = ranking_dataframe(diedIn_electra_borda_score_spacy_few, diedIn_electra_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_diedIn_electra_borda_spacy_score_few)):
    diedIn_electra_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_diedIn_electra_borda_spacy_score_few[i]))

#### Worksfor

In [ ]:
worksfor_electra_borda_score = aplicar_calculo_borda_dataframe(worksfor_electra, worksfor_electra.columns[1:])
worksfor_electra_borda_score_one = aplicar_calculo_borda_dataframe(worksfor_electra_one, worksfor_electra_one.columns[1:])
worksfor_electra_borda_score_few = aplicar_calculo_borda_dataframe(worksfor_electra_few, worksfor_electra_few.columns[1:])

worksfor_electra_borda_tokens_score = []
ranking_worksfor_electra_borda_score = ranking_dataframe(worksfor_electra_borda_score, worksfor_electra_borda_score.columns[1:])
for i in range(len(ranking_worksfor_electra_borda_score)):
    worksfor_electra_borda_tokens_score.append(extrair_tokens(ranking_worksfor_electra_borda_score[i]))

worksfor_electra_borda_tokens_score_one = []
ranking_worksfor_electra_borda_score_one = ranking_dataframe(worksfor_electra_borda_score_one, worksfor_electra_borda_score_one.columns[1:])
for i in range(len(ranking_worksfor_electra_borda_score_one)):
    worksfor_electra_borda_tokens_score_one.append(extrair_tokens(ranking_worksfor_electra_borda_score_one[i]))

worksfor_electra_borda_tokens_score_few = []
ranking_worksfor_electra_borda_score_few = ranking_dataframe(worksfor_electra_borda_score_few, worksfor_electra_borda_score_few.columns[1:])
for i in range(len(ranking_worksfor_electra_borda_score_few)):
    worksfor_electra_borda_tokens_score_few.append(extrair_tokens(ranking_worksfor_electra_borda_score_few[i]))

In [ ]:
worksfor_electra_borda_score_spacy = aplicar_calculo_borda_dataframe(worksfor_electra_spacy, worksfor_electra_spacy.columns[1:])
worksfor_electra_borda_score_spacy_one = aplicar_calculo_borda_dataframe(worksfor_electra_one_spacy, worksfor_electra_one_spacy.columns[1:])
worksfor_electra_borda_score_spacy_few = aplicar_calculo_borda_dataframe(worksfor_electra_few_spacy, worksfor_electra_few_spacy.columns[1:])

worksfor_electra_borda_spacy_tokens_score = []
ranking_worksfor_electra_borda_spacy_score = ranking_dataframe(worksfor_electra_borda_score_spacy, worksfor_electra_borda_score_spacy.columns[1:])
for i in range(len(ranking_worksfor_electra_borda_spacy_score)):
    worksfor_electra_borda_spacy_tokens_score.append(extrair_tokens(ranking_worksfor_electra_borda_spacy_score[i]))

worksfor_electra_borda_spacy_tokens_score_one = []
ranking_worksfor_electra_borda_spacy_score_one = ranking_dataframe(worksfor_electra_borda_score_spacy_one, worksfor_electra_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_worksfor_electra_borda_spacy_score_one)):
    worksfor_electra_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_worksfor_electra_borda_spacy_score_one[i]))

worksfor_electra_borda_spacy_tokens_score_few = []
ranking_worksfor_electra_borda_spacy_score_few = ranking_dataframe(worksfor_electra_borda_score_spacy_few, worksfor_electra_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_worksfor_electra_borda_spacy_score_few)):
    worksfor_electra_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_worksfor_electra_borda_spacy_score_few[i]))

#### Capital

In [ ]:
capital_electra_borda_score = aplicar_calculo_borda_dataframe(capital_electra, capital_electra.columns[1:])
capital_electra_borda_score_one = aplicar_calculo_borda_dataframe(capital_electra_one, capital_electra_one.columns[1:])
capital_electra_borda_score_few = aplicar_calculo_borda_dataframe(capital_electra_few, capital_electra_few.columns[1:])

capital_electra_borda_tokens_score = []
ranking_capital_electra_borda_score = ranking_dataframe(capital_electra_borda_score, capital_electra_borda_score.columns[1:])
for i in range(len(ranking_capital_electra_borda_score)):
    capital_electra_borda_tokens_score.append(extrair_tokens(ranking_capital_electra_borda_score[i]))

capital_electra_borda_tokens_score_one = []
ranking_capital_electra_borda_score_one = ranking_dataframe(capital_electra_borda_score_one, capital_electra_borda_score_one.columns[1:])
for i in range(len(ranking_capital_electra_borda_score_one)):
    capital_electra_borda_tokens_score_one.append(extrair_tokens(ranking_capital_electra_borda_score_one[i]))

capital_electra_borda_tokens_score_few = []
ranking_capital_electra_borda_score_few = ranking_dataframe(capital_electra_borda_score_few, capital_electra_borda_score_few.columns[1:])
for i in range(len(ranking_capital_electra_borda_score_few)):
    capital_electra_borda_tokens_score_few.append(extrair_tokens(ranking_capital_electra_borda_score_few[i]))

In [ ]:
capital_electra_borda_score_spacy = aplicar_calculo_borda_dataframe(capital_electra_spacy, capital_electra_spacy.columns[1:])
capital_electra_borda_score_spacy_one = aplicar_calculo_borda_dataframe(capital_electra_one_spacy, capital_electra_one_spacy.columns[1:])
capital_electra_borda_score_spacy_few = aplicar_calculo_borda_dataframe(capital_electra_few_spacy, capital_electra_few_spacy.columns[1:])

capital_electra_borda_spacy_tokens_score = []
ranking_capital_electra_borda_spacy_score = ranking_dataframe(capital_electra_borda_score_spacy, capital_electra_borda_score_spacy.columns[1:])
for i in range(len(ranking_capital_electra_borda_spacy_score)):
    capital_electra_borda_spacy_tokens_score.append(extrair_tokens(ranking_capital_electra_borda_spacy_score[i]))

capital_electra_borda_spacy_tokens_score_one = []
ranking_capital_electra_borda_spacy_score_one = ranking_dataframe(capital_electra_borda_score_spacy_one, capital_electra_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_capital_electra_borda_spacy_score_one)):
    capital_electra_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_capital_electra_borda_spacy_score_one[i]))

capital_electra_borda_spacy_tokens_score_few = []
ranking_capital_electra_borda_spacy_score_few = ranking_dataframe(capital_electra_borda_score_spacy_few, capital_electra_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_capital_electra_borda_spacy_score_few)):
    capital_electra_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_capital_electra_borda_spacy_score_few[i]))

#### Citizen

In [ ]:
citizen_electra_borda_score = aplicar_calculo_borda_dataframe(citizen_electra, citizen_electra.columns[1:])
citizen_electra_borda_score_one = aplicar_calculo_borda_dataframe(citizen_electra_one, citizen_electra_one.columns[1:])
citizen_electra_borda_score_few = aplicar_calculo_borda_dataframe(citizen_electra_few, citizen_electra_few.columns[1:])

citizen_electra_borda_tokens_score = []
ranking_citizen_electra_borda_score = ranking_dataframe(citizen_electra_borda_score, citizen_electra_borda_score.columns[1:])
for i in range(len(ranking_citizen_electra_borda_score)):
    citizen_electra_borda_tokens_score.append(extrair_tokens(ranking_citizen_electra_borda_score[i]))

citizen_electra_borda_tokens_score_one = []
ranking_citizen_electra_borda_score_one = ranking_dataframe(citizen_electra_borda_score_one, citizen_electra_borda_score_one.columns[1:])
for i in range(len(ranking_citizen_electra_borda_score_one)):
    citizen_electra_borda_tokens_score_one.append(extrair_tokens(ranking_citizen_electra_borda_score_one[i]))

citizen_electra_borda_tokens_score_few = []
ranking_citizen_electra_borda_score_few = ranking_dataframe(citizen_electra_borda_score_few, citizen_electra_borda_score_few.columns[1:])
for i in range(len(ranking_citizen_electra_borda_score_few)):
    citizen_electra_borda_tokens_score_few.append(extrair_tokens(ranking_citizen_electra_borda_score_few[i]))

In [ ]:
citizen_electra_borda_score_spacy = aplicar_calculo_borda_dataframe(citizen_electra_spacy, citizen_electra_spacy.columns[1:])
citizen_electra_borda_score_spacy_one = aplicar_calculo_borda_dataframe(citizen_electra_one_spacy, citizen_electra_one_spacy.columns[1:])
citizen_electra_borda_score_spacy_few = aplicar_calculo_borda_dataframe(citizen_electra_few_spacy, citizen_electra_few_spacy.columns[1:])

citizen_electra_borda_spacy_tokens_score = []
ranking_citizen_electra_borda_spacy_score = ranking_dataframe(citizen_electra_borda_score_spacy, citizen_electra_borda_score_spacy.columns[1:])
for i in range(len(ranking_citizen_electra_borda_spacy_score)):
    citizen_electra_borda_spacy_tokens_score.append(extrair_tokens(ranking_citizen_electra_borda_spacy_score[i]))

citizen_electra_borda_spacy_tokens_score_one = []
ranking_citizen_electra_borda_spacy_score_one = ranking_dataframe(citizen_electra_borda_score_spacy_one, citizen_electra_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_citizen_electra_borda_spacy_score_one)):
    citizen_electra_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_citizen_electra_borda_spacy_score_one[i]))

citizen_electra_borda_spacy_tokens_score_few = []
ranking_citizen_electra_borda_spacy_score_few = ranking_dataframe(citizen_electra_borda_score_spacy_few, citizen_electra_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_citizen_electra_borda_spacy_score_few)):
    citizen_electra_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_citizen_electra_borda_spacy_score_few[i]))

#### Language

In [ ]:
language_electra_borda_score = aplicar_calculo_borda_dataframe(language_electra, language_electra.columns[1:])
language_electra_borda_score_one = aplicar_calculo_borda_dataframe(language_electra_one, language_electra_one.columns[1:])
language_electra_borda_score_few = aplicar_calculo_borda_dataframe(language_electra_few, language_electra_few.columns[1:])

language_electra_borda_tokens_score = []
ranking_language_electra_borda_score = ranking_dataframe(language_electra_borda_score, language_electra_borda_score.columns[1:])
for i in range(len(ranking_language_electra_borda_score)):
    language_electra_borda_tokens_score.append(extrair_tokens(ranking_language_electra_borda_score[i]))

language_electra_borda_tokens_score_one = []
ranking_language_electra_borda_score_one = ranking_dataframe(language_electra_borda_score_one, language_electra_borda_score_one.columns[1:])
for i in range(len(ranking_language_electra_borda_score_one)):
    language_electra_borda_tokens_score_one.append(extrair_tokens(ranking_language_electra_borda_score_one[i]))

language_electra_borda_tokens_score_few = []
ranking_language_electra_borda_score_few = ranking_dataframe(language_electra_borda_score_few, language_electra_borda_score_few.columns[1:])
for i in range(len(ranking_language_electra_borda_score_few)):
    language_electra_borda_tokens_score_few.append(extrair_tokens(ranking_language_electra_borda_score_few[i]))

In [ ]:
language_electra_borda_score_spacy = aplicar_calculo_borda_dataframe(language_electra_spacy, language_electra_spacy.columns[1:])
language_electra_borda_score_spacy_one = aplicar_calculo_borda_dataframe(language_electra_one_spacy, language_electra_one_spacy.columns[1:])
language_electra_borda_score_spacy_few = aplicar_calculo_borda_dataframe(language_electra_few_spacy, language_electra_few_spacy.columns[1:])

language_electra_borda_spacy_tokens_score = []
ranking_language_electra_borda_spacy_score = ranking_dataframe(language_electra_borda_score_spacy, language_electra_borda_score_spacy.columns[1:])
for i in range(len(ranking_language_electra_borda_spacy_score)):
    language_electra_borda_spacy_tokens_score.append(extrair_tokens(ranking_language_electra_borda_spacy_score[i]))

language_electra_borda_spacy_tokens_score_one = []
ranking_language_electra_borda_spacy_score_one = ranking_dataframe(language_electra_borda_score_spacy_one, language_electra_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_language_electra_borda_spacy_score_one)):
    language_electra_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_language_electra_borda_spacy_score_one[i]))

language_electra_borda_spacy_tokens_score_few = []
ranking_language_electra_borda_spacy_score_few = ranking_dataframe(language_electra_borda_score_spacy_few, language_electra_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_language_electra_borda_spacy_score_few)):
    language_electra_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_language_electra_borda_spacy_score_few[i]))

# Prompts do TRex

## Bert

#### BornIn

In [ ]:
masked_bornIn_bert_original = bornIn_bert['original'].apply(extrair_tokens).tolist()
masked_bornIn_bert_original_spacy = bornIn_bert_spacy['original'].apply(extrair_tokens).tolist()
masked_bornIn_bert_original_one = bornIn_bert_one['original'].apply(extrair_tokens).tolist()
masked_bornIn_bert_original_spacy_one = bornIn_bert_one_spacy['original'].apply(extrair_tokens).tolist()
masked_bornIn_bert_original_few = bornIn_bert_few['original'].apply(extrair_tokens).tolist()
masked_bornIn_bert_original_spacy_few = bornIn_bert_few_spacy['original'].apply(extrair_tokens).tolist()

#### DiedIn

In [ ]:
masked_diedIn_bert_original = diedIn_bert['original'].apply(extrair_tokens).tolist()
masked_diedIn_bert_original_spacy = diedIn_bert_spacy['original'].apply(extrair_tokens).tolist()
masked_diedIn_bert_original_one = diedIn_bert_one['original'].apply(extrair_tokens).tolist()
masked_diedIn_bert_original_spacy_one = diedIn_bert_one_spacy['original'].apply(extrair_tokens).tolist()
masked_diedIn_bert_original_few = diedIn_bert_few['original'].apply(extrair_tokens).tolist()
masked_diedIn_bert_original_spacy_few = diedIn_bert_few_spacy['original'].apply(extrair_tokens).tolist()

#### Woksfor

In [ ]:
masked_worksfor_bert_original = worksfor_bert['original'].apply(extrair_tokens).tolist()
masked_worksfor_bert_original_spacy = worksfor_bert_spacy['original'].apply(extrair_tokens).tolist()
masked_worksfor_bert_original_one = worksfor_bert_one['original'].apply(extrair_tokens).tolist()
masked_worksfor_bert_original_spacy_one = worksfor_bert_one_spacy['original'].apply(extrair_tokens).tolist()
masked_worksfor_bert_original_few = worksfor_bert_few['original'].apply(extrair_tokens).tolist()
masked_worksfor_bert_original_spacy_few = worksfor_bert_few_spacy['original'].apply(extrair_tokens).tolist()

#### Capital

In [ ]:
masked_capital_bert_original = capital_bert['original'].apply(extrair_tokens).tolist()
masked_capital_bert_original_spacy = capital_bert_spacy['original'].apply(extrair_tokens).tolist()
masked_capital_bert_original_one = capital_bert_one['original'].apply(extrair_tokens).tolist()
masked_capital_bert_original_spacy_one = capital_bert_one_spacy['original'].apply(extrair_tokens).tolist()
masked_capital_bert_original_few = capital_bert_few['original'].apply(extrair_tokens).tolist()
masked_capital_bert_original_spacy_few = capital_bert_few_spacy['original'].apply(extrair_tokens).tolist()

#### Citizen

In [ ]:
masked_citizen_bert_original = citizen_bert['original'].apply(extrair_tokens).tolist()
masked_citizen_bert_original_spacy = citizen_bert_spacy['original'].apply(extrair_tokens).tolist()
masked_citizen_bert_original_one = citizen_bert_one['original'].apply(extrair_tokens).tolist()
masked_citizen_bert_original_spacy_one = citizen_bert_one_spacy['original'].apply(extrair_tokens).tolist()
masked_citizen_bert_original_few = citizen_bert_few['original'].apply(extrair_tokens).tolist()
masked_citizen_bert_original_spacy_few = citizen_bert_few_spacy['original'].apply(extrair_tokens).tolist()

#### Language

In [ ]:
masked_language_bert_original = language_bert['original'].apply(extrair_tokens).tolist()
masked_language_bert_original_spacy = language_bert_spacy['original'].apply(extrair_tokens).tolist()
masked_language_bert_original_one = language_bert_one['original'].apply(extrair_tokens).tolist()
masked_language_bert_original_spacy_one = language_bert_one_spacy['original'].apply(extrair_tokens).tolist()
masked_language_bert_original_few = language_bert_few['original'].apply(extrair_tokens).tolist()
masked_language_bert_original_spacy_few = language_bert_few_spacy['original'].apply(extrair_tokens).tolist()

## Roberta

#### BornIn

In [ ]:
masked_bornIn_roberta_original = bornIn_roberta['original'].apply(extrair_tokens).tolist()
masked_bornIn_roberta_original_spacy = bornIn_roberta_spacy['original'].apply(extrair_tokens).tolist()
masked_bornIn_roberta_original_one = bornIn_roberta_one['original'].apply(extrair_tokens).tolist()
masked_bornIn_roberta_original_spacy_one = bornIn_roberta_one_spacy['original'].apply(extrair_tokens).tolist()
masked_bornIn_roberta_original_few = bornIn_roberta_few['original'].apply(extrair_tokens).tolist()
masked_bornIn_roberta_original_spacy_few = bornIn_roberta_few_spacy['original'].apply(extrair_tokens).tolist()

#### DiedIn

In [ ]:
masked_diedIn_roberta_original = diedIn_roberta['original'].apply(extrair_tokens).tolist()
masked_diedIn_roberta_original_spacy = diedIn_roberta_spacy['original'].apply(extrair_tokens).tolist()
masked_diedIn_roberta_original_one = diedIn_roberta_one['original'].apply(extrair_tokens).tolist()
masked_diedIn_roberta_original_spacy_one = diedIn_roberta_one_spacy['original'].apply(extrair_tokens).tolist()
masked_diedIn_roberta_original_few = diedIn_roberta_few['original'].apply(extrair_tokens).tolist()
masked_diedIn_roberta_original_spacy_few = diedIn_roberta_few_spacy['original'].apply(extrair_tokens).tolist()

#### Woksfor

In [ ]:
masked_worksfor_roberta_original = worksfor_roberta['original'].apply(extrair_tokens).tolist()
masked_worksfor_roberta_original_spacy = worksfor_roberta_spacy['original'].apply(extrair_tokens).tolist()
masked_worksfor_roberta_original_one = worksfor_roberta_one['original'].apply(extrair_tokens).tolist()
masked_worksfor_roberta_original_spacy_one = worksfor_roberta_one_spacy['original'].apply(extrair_tokens).tolist()
masked_worksfor_roberta_original_few = worksfor_roberta_few['original'].apply(extrair_tokens).tolist()
masked_worksfor_roberta_original_spacy_few = worksfor_roberta_few_spacy['original'].apply(extrair_tokens).tolist()

#### Capital

In [ ]:
masked_capital_roberta_original = capital_roberta['original'].apply(extrair_tokens).tolist()
masked_capital_roberta_original_spacy = capital_roberta_spacy['original'].apply(extrair_tokens).tolist()
masked_capital_roberta_original_one = capital_roberta_one['original'].apply(extrair_tokens).tolist()
masked_capital_roberta_original_spacy_one = capital_roberta_one_spacy['original'].apply(extrair_tokens).tolist()
masked_capital_roberta_original_few = capital_roberta_few['original'].apply(extrair_tokens).tolist()
masked_capital_roberta_original_spacy_few = capital_roberta_few_spacy['original'].apply(extrair_tokens).tolist()

#### Citizen

In [ ]:
masked_citizen_roberta_original = citizen_roberta['original'].apply(extrair_tokens).tolist()
masked_citizen_roberta_original_spacy = citizen_roberta_spacy['original'].apply(extrair_tokens).tolist()
masked_citizen_roberta_original_one = citizen_roberta_one['original'].apply(extrair_tokens).tolist()
masked_citizen_roberta_original_spacy_one = citizen_roberta_one_spacy['original'].apply(extrair_tokens).tolist()
masked_citizen_roberta_original_few = citizen_roberta_few['original'].apply(extrair_tokens).tolist()
masked_citizen_roberta_original_spacy_few = citizen_roberta_few_spacy['original'].apply(extrair_tokens).tolist()

#### Language

In [ ]:
masked_language_roberta_original = language_roberta['original'].apply(extrair_tokens).tolist()
masked_language_roberta_original_spacy = language_roberta_spacy['original'].apply(extrair_tokens).tolist()
masked_language_roberta_original_one = language_roberta_one['original'].apply(extrair_tokens).tolist()
masked_language_roberta_original_spacy_one = language_roberta_one_spacy['original'].apply(extrair_tokens).tolist()
masked_language_roberta_original_few = language_roberta_few['original'].apply(extrair_tokens).tolist()
masked_language_roberta_original_spacy_few = language_roberta_few_spacy['original'].apply(extrair_tokens).tolist()

## Electra

#### BornIn

In [ ]:
masked_bornIn_electra_original = bornIn_electra['original'].apply(extrair_tokens).tolist()
masked_bornIn_electra_original_spacy = bornIn_electra_spacy['original'].apply(extrair_tokens).tolist()
masked_bornIn_electra_original_one = bornIn_electra_one['original'].apply(extrair_tokens).tolist()
masked_bornIn_electra_original_spacy_one = bornIn_electra_one_spacy['original'].apply(extrair_tokens).tolist()
masked_bornIn_electra_original_few = bornIn_electra_few['original'].apply(extrair_tokens).tolist()
masked_bornIn_electra_original_spacy_few = bornIn_electra_few_spacy['original'].apply(extrair_tokens).tolist()

#### DiedIn

In [ ]:
masked_diedIn_electra_original = diedIn_electra['original'].apply(extrair_tokens).tolist()
masked_diedIn_electra_original_spacy = diedIn_electra_spacy['original'].apply(extrair_tokens).tolist()
masked_diedIn_electra_original_one = diedIn_electra_one['original'].apply(extrair_tokens).tolist()
masked_diedIn_electra_original_spacy_one = diedIn_electra_one_spacy['original'].apply(extrair_tokens).tolist()
masked_diedIn_electra_original_few = diedIn_electra_few['original'].apply(extrair_tokens).tolist()
masked_diedIn_electra_original_spacy_few = diedIn_electra_few_spacy['original'].apply(extrair_tokens).tolist()

#### Woksfor

In [ ]:
masked_worksfor_electra_original = worksfor_electra['original'].apply(extrair_tokens).tolist()
masked_worksfor_electra_original_spacy = worksfor_electra_spacy['original'].apply(extrair_tokens).tolist()
masked_worksfor_electra_original_one = worksfor_electra_one['original'].apply(extrair_tokens).tolist()
masked_worksfor_electra_original_spacy_one = worksfor_electra_one_spacy['original'].apply(extrair_tokens).tolist()
masked_worksfor_electra_original_few = worksfor_electra_few['original'].apply(extrair_tokens).tolist()
masked_worksfor_electra_original_spacy_few = worksfor_electra_few_spacy['original'].apply(extrair_tokens).tolist()

#### Capital

In [ ]:
masked_capital_electra_original = capital_electra['original'].apply(extrair_tokens).tolist()
masked_capital_electra_original_spacy = capital_electra_spacy['original'].apply(extrair_tokens).tolist()
masked_capital_electra_original_one = capital_electra_one['original'].apply(extrair_tokens).tolist()
masked_capital_electra_original_spacy_one = capital_electra_one_spacy['original'].apply(extrair_tokens).tolist()
masked_capital_electra_original_few = capital_electra_few['original'].apply(extrair_tokens).tolist()
masked_capital_electra_original_spacy_few = capital_electra_few_spacy['original'].apply(extrair_tokens).tolist()

#### Citizen

In [ ]:
masked_citizen_electra_original = citizen_electra['original'].apply(extrair_tokens).tolist()
masked_citizen_electra_original_spacy = citizen_electra_spacy['original'].apply(extrair_tokens).tolist()
masked_citizen_electra_original_one = citizen_electra_one['original'].apply(extrair_tokens).tolist()
masked_citizen_electra_original_spacy_one = citizen_electra_one_spacy['original'].apply(extrair_tokens).tolist()
masked_citizen_electra_original_few = citizen_electra_few['original'].apply(extrair_tokens).tolist()
masked_citizen_electra_original_spacy_few = citizen_electra_few_spacy['original'].apply(extrair_tokens).tolist()

#### Citizen

In [ ]:
masked_language_electra_original = language_electra['original'].apply(extrair_tokens).tolist()
masked_language_electra_original_spacy = language_electra_spacy['original'].apply(extrair_tokens).tolist()
masked_language_electra_original_one = language_electra_one['original'].apply(extrair_tokens).tolist()
masked_language_electra_original_spacy_one = language_electra_one_spacy['original'].apply(extrair_tokens).tolist()
masked_language_electra_original_few = language_electra_few['original'].apply(extrair_tokens).tolist()
masked_language_electra_original_spacy_few = language_electra_few_spacy['original'].apply(extrair_tokens).tolist()

## Merged Models

### BornIn

In [ ]:
# Criar um DataFrame com as listas
bornIn_merged_model = pd.DataFrame({
    'answers': bornIn_bert.answers,
    'bert_original': bornIn_bert['original'].tolist(),
    'roberta_original': bornIn_roberta['original'].tolist(),
    'electra_original': bornIn_electra['original'].tolist()
})

# Criar um DataFrame com as listas
bornIn_merged_model_one = pd.DataFrame({
    'answers': bornIn_bert_one.answers,
    'bert_original': bornIn_bert_one['original'].tolist(),
    'roberta_original': bornIn_roberta_one['original'].tolist(),
    'electra_original': bornIn_electra_one['original'].tolist()
})


# Criar um DataFrame com as listas
bornIn_merged_model_few = pd.DataFrame({
    'answers': bornIn_bert_few.answers,
    'bert_original': bornIn_bert_few['original'].tolist(),
    'roberta_original': bornIn_roberta_few['original'].tolist(),
    'electra_original': bornIn_electra_few['original'].tolist()
})

# Normal
ranking_bornIn_merged_model_normal_tokens = []
ranking_bornin_merged_model_normal = ranking_dataframe(bornIn_merged_model, bornIn_merged_model.columns[1:])
for i in range(len(ranking_bornin_merged_model_normal)):
    ranking_bornIn_merged_model_normal_tokens.append(extrair_tokens(ranking_bornin_merged_model_normal[i]))

bornIn_merged_model_borda_score = aplicar_calculo_borda_dataframe(bornIn_merged_model, bornIn_merged_model.columns[1:])

bornIn_merged_model_borda_tokens_score = []
ranking_bornIn_merged_model_borda_score = ranking_dataframe(bornIn_merged_model_borda_score, bornIn_merged_model_borda_score.columns[1:])
for i in range(len(ranking_bornIn_merged_model_borda_score)):
    bornIn_merged_model_borda_tokens_score.append(extrair_tokens(ranking_bornIn_merged_model_borda_score[i]))


# One Shoot
ranking_bornIn_merged_model_normal_tokens_one = []
ranking_bornin_merged_model_normal_one = ranking_dataframe(bornIn_merged_model_one, bornIn_merged_model_one.columns[1:])
for i in range(len(ranking_bornin_merged_model_normal_one)):
    ranking_bornIn_merged_model_normal_tokens_one.append(extrair_tokens(ranking_bornin_merged_model_normal_one[i]))

bornIn_merged_model_borda_score_one = aplicar_calculo_borda_dataframe(bornIn_merged_model_one, bornIn_merged_model_one.columns[1:])

bornIn_merged_model_borda_tokens_score_one = []
ranking_bornIn_merged_model_borda_score_one = ranking_dataframe(bornIn_merged_model_borda_score_one, bornIn_merged_model_borda_score_one.columns[1:])
for i in range(len(ranking_bornIn_merged_model_borda_score_one)):
    bornIn_merged_model_borda_tokens_score_one.append(extrair_tokens(ranking_bornIn_merged_model_borda_score_one[i]))


# Few Shoot
ranking_bornIn_merged_model_normal_tokens_few = []
ranking_bornin_merged_model_normal_few = ranking_dataframe(bornIn_merged_model_few, bornIn_merged_model_few.columns[1:])
for i in range(len(ranking_bornin_merged_model_normal_few)):
    ranking_bornIn_merged_model_normal_tokens_few.append(extrair_tokens(ranking_bornin_merged_model_normal_few[i]))

bornIn_merged_model_borda_score_few = aplicar_calculo_borda_dataframe(bornIn_merged_model_few, bornIn_merged_model_few.columns[1:])

bornIn_merged_model_borda_tokens_score_few = []
ranking_bornIn_merged_model_borda_score_few = ranking_dataframe(bornIn_merged_model_borda_score_few, bornIn_merged_model_borda_score_few.columns[1:])
for i in range(len(ranking_bornIn_merged_model_borda_score_few)):
    bornIn_merged_model_borda_tokens_score_few.append(extrair_tokens(ranking_bornIn_merged_model_borda_score_few[i]))

In [ ]:
# Criar um DataFrame com as listas
bornIn_merged_model_spacy = pd.DataFrame({
    'answers': bornIn_bert.answers,
    'bert_original': bornIn_bert_spacy['original'].tolist(),
    'roberta_original': bornIn_roberta_spacy['original'].tolist(),
    'electra_original': bornIn_electra_spacy['original'].tolist()
})


# Criar um DataFrame com as listas
bornIn_merged_model_spacy_one = pd.DataFrame({
    'answers': bornIn_bert.answers,
    'bert_original': bornIn_bert_one_spacy['original'].tolist(),
    'roberta_original': bornIn_roberta_one_spacy['original'].tolist(),
    'electra_original': bornIn_electra_one_spacy['original'].tolist()
})

# Criar um DataFrame com as listas
bornIn_merged_model_spacy_few = pd.DataFrame({
    'answers': bornIn_bert.answers,
    'bert_original': bornIn_bert_few_spacy['original'].tolist(),
    'roberta_original': bornIn_roberta_few_spacy['original'].tolist(),
    'electra_original': bornIn_electra_few_spacy['original'].tolist()
})

# Spacy
ranking_bornin_merged_model_spacy_tokens = []
ranking_bornin_merged_model_spacy = ranking_dataframe(bornIn_merged_model_spacy, bornIn_merged_model_spacy.columns[1:])
for i in range(len(ranking_bornin_merged_model_spacy)):
    ranking_bornin_merged_model_spacy_tokens.append(extrair_tokens(ranking_bornin_merged_model_spacy[i]))

bornIn_merged_model_borda_score_spacy = aplicar_calculo_borda_dataframe(bornIn_merged_model_spacy, bornIn_merged_model_spacy.columns[1:])

bornIn_merged_model_borda_spacy_tokens_score = []
ranking_bornIn_merged_model_borda_spacy_score = ranking_dataframe(bornIn_merged_model_borda_score_spacy, bornIn_merged_model_borda_score_spacy.columns[1:])
for i in range(len(ranking_bornIn_merged_model_borda_spacy_score)):
    bornIn_merged_model_borda_spacy_tokens_score.append(extrair_tokens(ranking_bornIn_merged_model_borda_spacy_score[i]))

# One Shoot
ranking_bornin_merged_model_spacy_tokens_one = []
ranking_bornin_merged_model_spacy_one = ranking_dataframe(bornIn_merged_model_spacy_one, bornIn_merged_model_spacy_one.columns[1:])
for i in range(len(ranking_bornin_merged_model_spacy_one)):
    ranking_bornin_merged_model_spacy_tokens_one.append(extrair_tokens(ranking_bornin_merged_model_spacy_one[i]))

bornIn_merged_model_borda_score_spacy_one = aplicar_calculo_borda_dataframe(bornIn_merged_model_spacy_one, bornIn_merged_model_spacy_one.columns[1:])

bornIn_merged_model_borda_spacy_tokens_score_one = []
ranking_bornIn_merged_model_borda_spacy_score_one = ranking_dataframe(bornIn_merged_model_borda_score_spacy_one, bornIn_merged_model_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_bornIn_merged_model_borda_spacy_score_one)):
    bornIn_merged_model_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_bornIn_merged_model_borda_spacy_score_one[i]))

# Few Shoot
ranking_bornin_merged_model_spacy_tokens_few = []
ranking_bornin_merged_model_spacy_few = ranking_dataframe(bornIn_merged_model_spacy_few, bornIn_merged_model_spacy_few.columns[1:])
for i in range(len(ranking_bornin_merged_model_spacy_few)):
    ranking_bornin_merged_model_spacy_tokens_few.append(extrair_tokens(ranking_bornin_merged_model_spacy_few[i]))

bornIn_merged_model_borda_score_spacy_few = aplicar_calculo_borda_dataframe(bornIn_merged_model_spacy_few, bornIn_merged_model_spacy_few.columns[1:])

bornIn_merged_model_borda_spacy_tokens_score_few = []
ranking_bornIn_merged_model_borda_spacy_score_few = ranking_dataframe(bornIn_merged_model_borda_score_spacy_few, bornIn_merged_model_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_bornIn_merged_model_borda_spacy_score_few)):
    bornIn_merged_model_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_bornIn_merged_model_borda_spacy_score_few[i]))

### DiedIn

In [ ]:
# Criar um DataFrame com as listas
diedIn_merged_model = pd.DataFrame({
    'answers': diedIn_bert.answers,
    'bert_original': diedIn_bert['original'].tolist(),
    'roberta_original': diedIn_roberta['original'].tolist(),
    'electra_original': diedIn_electra['original'].tolist()
})

# Criar um DataFrame com as listas
diedIn_merged_model_one = pd.DataFrame({
    'answers': diedIn_bert.answers,
    'bert_original': diedIn_bert_one['original'].tolist(),
    'roberta_original': diedIn_roberta_one['original'].tolist(),
    'electra_original': diedIn_electra_one['original'].tolist()
})


# Criar um DataFrame com as listas
diedIn_merged_model_few = pd.DataFrame({
    'answers': diedIn_bert.answers,
    'bert_original': diedIn_bert_few['original'].tolist(),
    'roberta_original': diedIn_roberta_few['original'].tolist(),
    'electra_original': diedIn_electra_few['original'].tolist()
})

# Normal
ranking_diedIn_merged_model_normal_tokens = []
ranking_diedIn_merged_model_normal = ranking_dataframe(diedIn_merged_model, diedIn_merged_model.columns[1:])
for i in range(len(ranking_diedIn_merged_model_normal)):
    ranking_diedIn_merged_model_normal_tokens.append(extrair_tokens(ranking_diedIn_merged_model_normal[i]))

diedIn_merged_model_borda_score = aplicar_calculo_borda_dataframe(diedIn_merged_model, diedIn_merged_model.columns[1:])

diedIn_merged_model_borda_tokens_score = []
ranking_diedIn_merged_model_borda_score = ranking_dataframe(diedIn_merged_model_borda_score, diedIn_merged_model_borda_score.columns[1:])
for i in range(len(ranking_diedIn_merged_model_borda_score)):
    diedIn_merged_model_borda_tokens_score.append(extrair_tokens(ranking_diedIn_merged_model_borda_score[i]))


# One Shoot
ranking_diedIn_merged_model_normal_tokens_one = []
ranking_diedIn_merged_model_normal_one = ranking_dataframe(diedIn_merged_model_one, diedIn_merged_model_one.columns[1:])
for i in range(len(ranking_diedIn_merged_model_normal_one)):
    ranking_diedIn_merged_model_normal_tokens_one.append(extrair_tokens(ranking_diedIn_merged_model_normal_one[i]))

diedIn_merged_model_borda_score_one = aplicar_calculo_borda_dataframe(diedIn_merged_model_one, diedIn_merged_model_one.columns[1:])

diedIn_merged_model_borda_tokens_score_one = []
ranking_diedIn_merged_model_borda_score_one = ranking_dataframe(diedIn_merged_model_borda_score_one, diedIn_merged_model_borda_score_one.columns[1:])
for i in range(len(ranking_diedIn_merged_model_borda_score_one)):
    diedIn_merged_model_borda_tokens_score_one.append(extrair_tokens(ranking_diedIn_merged_model_borda_score_one[i]))


# Few Shoot
ranking_diedIn_merged_model_normal_tokens_few = []
ranking_diedIn_merged_model_normal_few = ranking_dataframe(diedIn_merged_model_few, diedIn_merged_model_few.columns[1:])
for i in range(len(ranking_diedIn_merged_model_normal_few)):
    ranking_diedIn_merged_model_normal_tokens_few.append(extrair_tokens(ranking_diedIn_merged_model_normal_few[i]))

diedIn_merged_model_borda_score_few = aplicar_calculo_borda_dataframe(diedIn_merged_model_few, diedIn_merged_model_few.columns[1:])

diedIn_merged_model_borda_tokens_score_few = []
ranking_diedIn_merged_model_borda_score_few = ranking_dataframe(diedIn_merged_model_borda_score_few, diedIn_merged_model_borda_score_few.columns[1:])
for i in range(len(ranking_diedIn_merged_model_borda_score_few)):
    diedIn_merged_model_borda_tokens_score_few.append(extrair_tokens(ranking_diedIn_merged_model_borda_score_few[i]))

In [ ]:
# Criar um DataFrame com as listas
diedIn_merged_model_spacy = pd.DataFrame({
    'answers': diedIn_bert.answers,
    'bert_original': diedIn_bert_spacy['original'].tolist(),
    'roberta_original': diedIn_roberta_spacy['original'].tolist(),
    'electra_original': diedIn_electra_spacy['original'].tolist()
})


# Criar um DataFrame com as listas
diedIn_merged_model_spacy_one = pd.DataFrame({
    'answers': diedIn_bert.answers,
    'bert_original': diedIn_bert_one_spacy['original'].tolist(),
    'roberta_original': diedIn_roberta_one_spacy['original'].tolist(),
    'electra_original': diedIn_electra_one_spacy['original'].tolist()
})

# Criar um DataFrame com as listas
diedIn_merged_model_spacy_few = pd.DataFrame({
    'answers': diedIn_bert.answers,
    'bert_original': diedIn_bert_few_spacy['original'].tolist(),
    'roberta_original': diedIn_roberta_few_spacy['original'].tolist(),
    'electra_original': diedIn_electra_few_spacy['original'].tolist()
})

# Spacy
ranking_diedIn_merged_model_spacy_tokens = []
ranking_diedIn_merged_model_spacy = ranking_dataframe(diedIn_merged_model_spacy, diedIn_merged_model_spacy.columns[1:])
for i in range(len(ranking_diedIn_merged_model_spacy)):
    ranking_diedIn_merged_model_spacy_tokens.append(extrair_tokens(ranking_diedIn_merged_model_spacy[i]))

diedIn_merged_model_borda_score_spacy = aplicar_calculo_borda_dataframe(diedIn_merged_model_spacy, diedIn_merged_model_spacy.columns[1:])

diedIn_merged_model_borda_spacy_tokens_score = []
ranking_diedIn_merged_model_borda_spacy_score = ranking_dataframe(diedIn_merged_model_borda_score_spacy, diedIn_merged_model_borda_score_spacy.columns[1:])
for i in range(len(ranking_diedIn_merged_model_borda_spacy_score)):
    diedIn_merged_model_borda_spacy_tokens_score.append(extrair_tokens(ranking_diedIn_merged_model_borda_spacy_score[i]))

# One Shoot
ranking_diedIn_merged_model_spacy_tokens_one = []
ranking_diedIn_merged_model_spacy_one = ranking_dataframe(diedIn_merged_model_spacy_one, diedIn_merged_model_spacy_one.columns[1:])
for i in range(len(ranking_diedIn_merged_model_spacy_one)):
    ranking_diedIn_merged_model_spacy_tokens_one.append(extrair_tokens(ranking_diedIn_merged_model_spacy_one[i]))
diedIn_merged_model_borda_score_spacy_one = aplicar_calculo_borda_dataframe(diedIn_merged_model_spacy_one, diedIn_merged_model_spacy_one.columns[1:])

diedIn_merged_model_borda_spacy_tokens_score_one = []
ranking_diedIn_merged_model_borda_spacy_score_one = ranking_dataframe(diedIn_merged_model_borda_score_spacy_one, diedIn_merged_model_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_diedIn_merged_model_borda_spacy_score_one)):
    diedIn_merged_model_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_diedIn_merged_model_borda_spacy_score_one[i]))

# Few Shoot
ranking_diedIn_merged_model_spacy_tokens_few = []
ranking_diedIn_merged_model_spacy_few = ranking_dataframe(diedIn_merged_model_spacy_few, diedIn_merged_model_spacy_few.columns[1:])
for i in range(len(ranking_diedIn_merged_model_spacy_few)):
    ranking_diedIn_merged_model_spacy_tokens_few.append(extrair_tokens(ranking_diedIn_merged_model_spacy_few[i]))

diedIn_merged_model_borda_score_spacy_few = aplicar_calculo_borda_dataframe(diedIn_merged_model_spacy_few, diedIn_merged_model_spacy_few.columns[1:])

diedIn_merged_model_borda_spacy_tokens_score_few = []
ranking_diedIn_merged_model_borda_spacy_score_few = ranking_dataframe(diedIn_merged_model_borda_score_spacy_few, diedIn_merged_model_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_diedIn_merged_model_borda_spacy_score_few)):
    diedIn_merged_model_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_diedIn_merged_model_borda_spacy_score_few[i]))

### WorksFor

In [ ]:
# Criar um DataFrame com as listas
worksfor_merged_model = pd.DataFrame({
    'answers': worksfor_bert.answers,
    'bert_original': worksfor_bert['original'].tolist(),
    'roberta_original': worksfor_roberta['original'].tolist(),
    'electra_original': worksfor_electra['original'].tolist()
})

# Criar um DataFrame com as listas
worksfor_merged_model_one = pd.DataFrame({
    'answers': worksfor_bert.answers,
    'bert_original': worksfor_bert_one['original'].tolist(),
    'roberta_original': worksfor_roberta_one['original'].tolist(),
    'electra_original': worksfor_electra_one['original'].tolist()
})


# Criar um DataFrame com as listas
worksfor_merged_model_few = pd.DataFrame({
    'answers': worksfor_bert.answers,
    'bert_original': worksfor_bert_few['original'].tolist(),
    'roberta_original': worksfor_roberta_few['original'].tolist(),
    'electra_original': worksfor_electra_few['original'].tolist()
})

# Normal
ranking_worksfor_merged_model_normal_tokens = []
ranking_worksfor_merged_model_normal = ranking_dataframe(worksfor_merged_model, worksfor_merged_model.columns[1:])
for i in range(len(ranking_worksfor_merged_model_normal)):
    ranking_worksfor_merged_model_normal_tokens.append(extrair_tokens(ranking_worksfor_merged_model_normal[i]))

worksfor_merged_model_borda_score = aplicar_calculo_borda_dataframe(worksfor_merged_model, worksfor_merged_model.columns[1:])

worksfor_merged_model_borda_tokens_score = []
ranking_worksfor_merged_model_borda_score = ranking_dataframe(worksfor_merged_model_borda_score, worksfor_merged_model_borda_score.columns[1:])
for i in range(len(ranking_worksfor_merged_model_borda_score)):
    worksfor_merged_model_borda_tokens_score.append(extrair_tokens(ranking_worksfor_merged_model_borda_score[i]))


# One Shoot
ranking_worksfor_merged_model_normal_tokens_one = []
ranking_worksfor_merged_model_normal_one = ranking_dataframe(worksfor_merged_model_one, worksfor_merged_model_one.columns[1:])
for i in range(len(ranking_worksfor_merged_model_normal_one)):
    ranking_worksfor_merged_model_normal_tokens_one.append(extrair_tokens(ranking_worksfor_merged_model_normal_one[i]))

worksfor_merged_model_borda_score_one = aplicar_calculo_borda_dataframe(worksfor_merged_model_one, worksfor_merged_model_one.columns[1:])

worksfor_merged_model_borda_tokens_score_one = []
ranking_worksfor_merged_model_borda_score_one = ranking_dataframe(worksfor_merged_model_borda_score_one, worksfor_merged_model_borda_score_one.columns[1:])
for i in range(len(ranking_worksfor_merged_model_borda_score_one)):
    worksfor_merged_model_borda_tokens_score_one.append(extrair_tokens(ranking_worksfor_merged_model_borda_score_one[i]))


# Few Shoot
ranking_worksfor_merged_model_normal_tokens_few = []
ranking_worksfor_merged_model_normal_few = ranking_dataframe(worksfor_merged_model_few, worksfor_merged_model_few.columns[1:])
for i in range(len(ranking_worksfor_merged_model_normal_few)):
    ranking_worksfor_merged_model_normal_tokens_few.append(extrair_tokens(ranking_worksfor_merged_model_normal_few[i]))

worksfor_merged_model_borda_score_few = aplicar_calculo_borda_dataframe(worksfor_merged_model_few, worksfor_merged_model_few.columns[1:])

worksfor_merged_model_borda_tokens_score_few = []
ranking_worksfor_merged_model_borda_score_few = ranking_dataframe(worksfor_merged_model_borda_score_few, worksfor_merged_model_borda_score_few.columns[1:])
for i in range(len(ranking_worksfor_merged_model_borda_score_few)):
    worksfor_merged_model_borda_tokens_score_few.append(extrair_tokens(ranking_worksfor_merged_model_borda_score_few[i]))

In [ ]:
# Criar um DataFrame com as listas
worksfor_merged_model_spacy = pd.DataFrame({
    'answers': worksfor_bert.answers,
    'bert_original': worksfor_bert_spacy['original'].tolist(),
    'roberta_original': worksfor_roberta_spacy['original'].tolist(),
    'electra_original': worksfor_electra_spacy['original'].tolist()
})


# Criar um DataFrame com as listas
worksfor_merged_model_spacy_one = pd.DataFrame({
    'answers': worksfor_bert.answers,
    'bert_original': worksfor_bert_one_spacy['original'].tolist(),
    'roberta_original': worksfor_roberta_one_spacy['original'].tolist(),
    'electra_original': worksfor_electra_one_spacy['original'].tolist()
})

# Criar um DataFrame com as listas
worksfor_merged_model_spacy_few = pd.DataFrame({
    'answers': worksfor_bert.answers,
    'bert_original': worksfor_bert_few_spacy['original'].tolist(),
    'roberta_original': worksfor_roberta_few_spacy['original'].tolist(),
    'electra_original': worksfor_electra_few_spacy['original'].tolist()
})

# Spacy
ranking_worksfor_merged_model_spacy_tokens = []
ranking_worksfor_merged_model_spacy = ranking_dataframe(worksfor_merged_model_spacy, worksfor_merged_model_spacy.columns[1:])
for i in range(len(ranking_worksfor_merged_model_spacy)):
    ranking_worksfor_merged_model_spacy_tokens.append(extrair_tokens(ranking_worksfor_merged_model_spacy[i]))

worksfor_merged_model_borda_score_spacy = aplicar_calculo_borda_dataframe(worksfor_merged_model_spacy, worksfor_merged_model_spacy.columns[1:])

worksfor_merged_model_borda_spacy_tokens_score = []
ranking_worksfor_merged_model_borda_spacy_score = ranking_dataframe(worksfor_merged_model_borda_score_spacy, worksfor_merged_model_borda_score_spacy.columns[1:])
for i in range(len(ranking_worksfor_merged_model_borda_spacy_score)):
    worksfor_merged_model_borda_spacy_tokens_score.append(extrair_tokens(ranking_worksfor_merged_model_borda_spacy_score[i]))

# One Shoot
ranking_worksfor_merged_model_spacy_tokens_one = []
ranking_worksfor_merged_model_spacy_one = ranking_dataframe(worksfor_merged_model_spacy_one, worksfor_merged_model_spacy_one.columns[1:])
for i in range(len(ranking_worksfor_merged_model_spacy_one)):
    ranking_worksfor_merged_model_spacy_tokens_one.append(extrair_tokens(ranking_worksfor_merged_model_spacy_one[i]))

worksfor_merged_model_borda_score_spacy_one = aplicar_calculo_borda_dataframe(worksfor_merged_model_spacy_one, worksfor_merged_model_spacy_one.columns[1:])

worksfor_merged_model_borda_spacy_tokens_score_one = []
ranking_worksfor_merged_model_borda_spacy_score_one = ranking_dataframe(worksfor_merged_model_borda_score_spacy_one, worksfor_merged_model_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_worksfor_merged_model_borda_spacy_score_one)):
    worksfor_merged_model_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_worksfor_merged_model_borda_spacy_score_one[i]))

# Few Shoot
ranking_worksfor_merged_model_spacy_tokens_few = []
ranking_worksfor_merged_model_spacy_few = ranking_dataframe(worksfor_merged_model_spacy_few, worksfor_merged_model_spacy_few.columns[1:])
for i in range(len(ranking_worksfor_merged_model_spacy_few)):
    ranking_worksfor_merged_model_spacy_tokens_few.append(extrair_tokens(ranking_worksfor_merged_model_spacy_few[i]))

worksfor_merged_model_borda_score_spacy_few = aplicar_calculo_borda_dataframe(worksfor_merged_model_spacy_few, worksfor_merged_model_spacy_few.columns[1:])

worksfor_merged_model_borda_spacy_tokens_score_few = []
ranking_worksfor_merged_model_borda_spacy_score_few = ranking_dataframe(worksfor_merged_model_borda_score_spacy_few, worksfor_merged_model_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_worksfor_merged_model_borda_spacy_score_few)):
    worksfor_merged_model_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_worksfor_merged_model_borda_spacy_score_few[i]))

### Capital

In [ ]:
# Criar um DataFrame com as listas
capital_merged_model = pd.DataFrame({
    'answers': capital_bert.answers,
    'bert_original': capital_bert['original'].tolist(),
    'roberta_original': capital_roberta['original'].tolist(),
    'electra_original': capital_electra['original'].tolist()
})

# Criar um DataFrame com as listas
capital_merged_model_one = pd.DataFrame({
    'answers': capital_bert.answers,
    'bert_original': capital_bert_one['original'].tolist(),
    'roberta_original': capital_roberta_one['original'].tolist(),
    'electra_original': capital_electra_one['original'].tolist()
})


# Criar um DataFrame com as listas
capital_merged_model_few = pd.DataFrame({
    'answers': capital_bert.answers,
    'bert_original': capital_bert_few['original'].tolist(),
    'roberta_original': capital_roberta_few['original'].tolist(),
    'electra_original': capital_electra_few['original'].tolist()
})

# Normal
ranking_capital_merged_model_normal_tokens = []
ranking_capital_merged_model_normal = ranking_dataframe(capital_merged_model, capital_merged_model.columns[1:])
for i in range(len(ranking_capital_merged_model_normal)):
    ranking_capital_merged_model_normal_tokens.append(extrair_tokens(ranking_capital_merged_model_normal[i]))

capital_merged_model_borda_score = aplicar_calculo_borda_dataframe(capital_merged_model, capital_merged_model.columns[1:])

capital_merged_model_borda_tokens_score = []
ranking_capital_merged_model_borda_score = ranking_dataframe(capital_merged_model_borda_score, capital_merged_model_borda_score.columns[1:])
for i in range(len(ranking_capital_merged_model_borda_score)):
    capital_merged_model_borda_tokens_score.append(extrair_tokens(ranking_capital_merged_model_borda_score[i]))


# One Shoot
ranking_capital_merged_model_normal_tokens_one = []
ranking_capital_merged_model_normal_one = ranking_dataframe(capital_merged_model_one, capital_merged_model_one.columns[1:])
for i in range(len(ranking_capital_merged_model_normal_one)):
    ranking_capital_merged_model_normal_tokens_one.append(extrair_tokens(ranking_capital_merged_model_normal_one[i]))

capital_merged_model_borda_score_one = aplicar_calculo_borda_dataframe(capital_merged_model_one, capital_merged_model_one.columns[1:])

capital_merged_model_borda_tokens_score_one = []
ranking_capital_merged_model_borda_score_one = ranking_dataframe(capital_merged_model_borda_score_one, capital_merged_model_borda_score_one.columns[1:])
for i in range(len(ranking_capital_merged_model_borda_score_one)):
    capital_merged_model_borda_tokens_score_one.append(extrair_tokens(ranking_capital_merged_model_borda_score_one[i]))


# Few Shoot
ranking_capital_merged_model_normal_tokens_few = []
ranking_capital_merged_model_normal_few = ranking_dataframe(capital_merged_model_few, capital_merged_model_few.columns[1:])
for i in range(len(ranking_capital_merged_model_normal_few)):
    ranking_capital_merged_model_normal_tokens_few.append(extrair_tokens(ranking_capital_merged_model_normal_few[i]))

capital_merged_model_borda_score_few = aplicar_calculo_borda_dataframe(capital_merged_model_few, capital_merged_model_few.columns[1:])

capital_merged_model_borda_tokens_score_few = []
ranking_capital_merged_model_borda_score_few = ranking_dataframe(capital_merged_model_borda_score_few, capital_merged_model_borda_score_few.columns[1:])
for i in range(len(ranking_capital_merged_model_borda_score_few)):
    capital_merged_model_borda_tokens_score_few.append(extrair_tokens(ranking_capital_merged_model_borda_score_few[i]))

In [ ]:
# Criar um DataFrame com as listas
capital_merged_model_spacy = pd.DataFrame({
    'answers': capital_bert.answers,
    'bert_original': capital_bert_spacy['original'].tolist(),
    'roberta_original': capital_roberta_spacy['original'].tolist(),
    'electra_original': capital_electra_spacy['original'].tolist()
})


# Criar um DataFrame com as listas
capital_merged_model_spacy_one = pd.DataFrame({
    'answers': capital_bert.answers,
    'bert_original': capital_bert_one_spacy['original'].tolist(),
    'roberta_original': capital_roberta_one_spacy['original'].tolist(),
    'electra_original': capital_electra_one_spacy['original'].tolist()
})

# Criar um DataFrame com as listas
capital_merged_model_spacy_few = pd.DataFrame({
    'answers': capital_bert.answers,
    'bert_original': capital_bert_few_spacy['original'].tolist(),
    'roberta_original': capital_roberta_few_spacy['original'].tolist(),
    'electra_original': capital_electra_few_spacy['original'].tolist()
})

# Spacy
ranking_capital_merged_model_spacy_tokens = []
ranking_capital_merged_model_spacy = ranking_dataframe(capital_merged_model_spacy, capital_merged_model_spacy.columns[1:])
for i in range(len(ranking_capital_merged_model_spacy)):
    ranking_capital_merged_model_spacy_tokens.append(extrair_tokens(ranking_capital_merged_model_spacy[i]))

capital_merged_model_borda_score_spacy = aplicar_calculo_borda_dataframe(capital_merged_model_spacy, capital_merged_model_spacy.columns[1:])

capital_merged_model_borda_spacy_tokens_score = []
ranking_capital_merged_model_borda_spacy_score = ranking_dataframe(capital_merged_model_borda_score_spacy, capital_merged_model_borda_score_spacy.columns[1:])
for i in range(len(ranking_capital_merged_model_borda_spacy_score)):
    capital_merged_model_borda_spacy_tokens_score.append(extrair_tokens(ranking_capital_merged_model_borda_spacy_score[i]))

# One Shoot
ranking_capital_merged_model_spacy_tokens_one = []
ranking_capital_merged_model_spacy_one = ranking_dataframe(capital_merged_model_spacy_one, capital_merged_model_spacy_one.columns[1:])
for i in range(len(ranking_capital_merged_model_spacy_one)):
    ranking_capital_merged_model_spacy_tokens_one.append(extrair_tokens(ranking_capital_merged_model_spacy_one[i]))

capital_merged_model_borda_score_spacy_one = aplicar_calculo_borda_dataframe(capital_merged_model_spacy_one, capital_merged_model_spacy_one.columns[1:])

capital_merged_model_borda_spacy_tokens_score_one = []
ranking_capital_merged_model_borda_spacy_score_one = ranking_dataframe(capital_merged_model_borda_score_spacy_one, capital_merged_model_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_capital_merged_model_borda_spacy_score_one)):
    capital_merged_model_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_capital_merged_model_borda_spacy_score_one[i]))

# Few Shoot
ranking_capital_merged_model_spacy_tokens_few = []
ranking_capital_merged_model_spacy_few = ranking_dataframe(capital_merged_model_spacy_few, capital_merged_model_spacy_few.columns[1:])
for i in range(len(ranking_capital_merged_model_spacy_few)):
    ranking_capital_merged_model_spacy_tokens_few.append(extrair_tokens(ranking_capital_merged_model_spacy_few[i]))

capital_merged_model_borda_score_spacy_few = aplicar_calculo_borda_dataframe(capital_merged_model_spacy_few, capital_merged_model_spacy_few.columns[1:])

capital_merged_model_borda_spacy_tokens_score_few = []
ranking_capital_merged_model_borda_spacy_score_few = ranking_dataframe(capital_merged_model_borda_score_spacy_few, capital_merged_model_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_capital_merged_model_borda_spacy_score_few)):
    capital_merged_model_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_capital_merged_model_borda_spacy_score_few[i]))

### Citizen

In [ ]:
# Criar um DataFrame com as listas
citizen_merged_model = pd.DataFrame({
    'answers': citizen_bert.answers,
    'bert_original': citizen_bert['original'].tolist(),
    'roberta_original': citizen_roberta['original'].tolist(),
    'electra_original': citizen_electra['original'].tolist()
})

# Criar um DataFrame com as listas
citizen_merged_model_one = pd.DataFrame({
    'answers': citizen_bert.answers,
    'bert_original': citizen_bert_one['original'].tolist(),
    'roberta_original': citizen_roberta_one['original'].tolist(),
    'electra_original': citizen_electra_one['original'].tolist()
})


# Criar um DataFrame com as listas
citizen_merged_model_few = pd.DataFrame({
    'answers': citizen_bert.answers,
    'bert_original': citizen_bert_few['original'].tolist(),
    'roberta_original': citizen_roberta_few['original'].tolist(),
    'electra_original': citizen_electra_few['original'].tolist()
})

# Normal
ranking_citizen_merged_model_normal_tokens = []
ranking_citizen_merged_model_normal = ranking_dataframe(citizen_merged_model, citizen_merged_model.columns[1:])
for i in range(len(ranking_citizen_merged_model_normal)):
    ranking_citizen_merged_model_normal_tokens.append(extrair_tokens(ranking_citizen_merged_model_normal[i]))

citizen_merged_model_borda_score = aplicar_calculo_borda_dataframe(citizen_merged_model, citizen_merged_model.columns[1:])

citizen_merged_model_borda_tokens_score = []
ranking_citizen_merged_model_borda_score = ranking_dataframe(citizen_merged_model_borda_score, citizen_merged_model_borda_score.columns[1:])
for i in range(len(ranking_citizen_merged_model_borda_score)):
    citizen_merged_model_borda_tokens_score.append(extrair_tokens(ranking_citizen_merged_model_borda_score[i]))


# One Shoot
ranking_citizen_merged_model_normal_tokens_one = []
ranking_citizen_merged_model_normal_one = ranking_dataframe(citizen_merged_model_one, citizen_merged_model_one.columns[1:])
for i in range(len(ranking_citizen_merged_model_normal_one)):
    ranking_citizen_merged_model_normal_tokens_one.append(extrair_tokens(ranking_citizen_merged_model_normal_one[i]))

citizen_merged_model_borda_score_one = aplicar_calculo_borda_dataframe(citizen_merged_model_one, citizen_merged_model_one.columns[1:])

citizen_merged_model_borda_tokens_score_one = []
ranking_citizen_merged_model_borda_score_one = ranking_dataframe(citizen_merged_model_borda_score_one, citizen_merged_model_borda_score_one.columns[1:])
for i in range(len(ranking_citizen_merged_model_borda_score_one)):
    citizen_merged_model_borda_tokens_score_one.append(extrair_tokens(ranking_citizen_merged_model_borda_score_one[i]))


# Few Shoot
ranking_citizen_merged_model_normal_tokens_few = []
ranking_citizen_merged_model_normal_few = ranking_dataframe(citizen_merged_model_few, citizen_merged_model_few.columns[1:])
for i in range(len(ranking_citizen_merged_model_normal_few)):
    ranking_citizen_merged_model_normal_tokens_few.append(extrair_tokens(ranking_citizen_merged_model_normal_few[i]))

citizen_merged_model_borda_score_few = aplicar_calculo_borda_dataframe(citizen_merged_model_few, citizen_merged_model_few.columns[1:])

citizen_merged_model_borda_tokens_score_few = []
ranking_citizen_merged_model_borda_score_few = ranking_dataframe(citizen_merged_model_borda_score_few, citizen_merged_model_borda_score_few.columns[1:])
for i in range(len(ranking_citizen_merged_model_borda_score_few)):
    citizen_merged_model_borda_tokens_score_few.append(extrair_tokens(ranking_citizen_merged_model_borda_score_few[i]))

In [ ]:
# Criar um DataFrame com as listas
citizen_merged_model_spacy = pd.DataFrame({
    'answers': citizen_bert.answers,
    'bert_original': citizen_bert_spacy['original'].tolist(),
    'roberta_original': citizen_roberta_spacy['original'].tolist(),
    'electra_original': citizen_electra_spacy['original'].tolist()
})


# Criar um DataFrame com as listas
citizen_merged_model_spacy_one = pd.DataFrame({
    'answers': citizen_bert.answers,
    'bert_original': citizen_bert_one_spacy['original'].tolist(),
    'roberta_original': citizen_roberta_one_spacy['original'].tolist(),
    'electra_original': citizen_electra_one_spacy['original'].tolist()
})

# Criar um DataFrame com as listas
citizen_merged_model_spacy_few = pd.DataFrame({
    'answers': citizen_bert.answers,
    'bert_original': citizen_bert_few_spacy['original'].tolist(),
    'roberta_original': citizen_roberta_few_spacy['original'].tolist(),
    'electra_original': citizen_electra_few_spacy['original'].tolist()
})

# Spacy
ranking_citizen_merged_model_spacy_tokens = []
ranking_citizen_merged_model_spacy = ranking_dataframe(citizen_merged_model_spacy, citizen_merged_model_spacy.columns[1:])
for i in range(len(ranking_citizen_merged_model_spacy)):
    ranking_citizen_merged_model_spacy_tokens.append(extrair_tokens(ranking_citizen_merged_model_spacy[i]))

citizen_merged_model_borda_score_spacy = aplicar_calculo_borda_dataframe(citizen_merged_model_spacy, citizen_merged_model_spacy.columns[1:])

citizen_merged_model_borda_spacy_tokens_score = []
ranking_citizen_merged_model_borda_spacy_score = ranking_dataframe(citizen_merged_model_borda_score_spacy, citizen_merged_model_borda_score_spacy.columns[1:])
for i in range(len(ranking_citizen_merged_model_borda_spacy_score)):
    citizen_merged_model_borda_spacy_tokens_score.append(extrair_tokens(ranking_citizen_merged_model_borda_spacy_score[i]))

# One Shoot
ranking_citizen_merged_model_spacy_tokens_one = []
ranking_citizen_merged_model_spacy_one = ranking_dataframe(citizen_merged_model_spacy_one, citizen_merged_model_spacy_one.columns[1:])
for i in range(len(ranking_citizen_merged_model_spacy_one)):
    ranking_citizen_merged_model_spacy_tokens_one.append(extrair_tokens(ranking_citizen_merged_model_spacy_one[i]))

citizen_merged_model_borda_score_spacy_one = aplicar_calculo_borda_dataframe(citizen_merged_model_spacy_one, citizen_merged_model_spacy_one.columns[1:])

citizen_merged_model_borda_spacy_tokens_score_one = []
ranking_citizen_merged_model_borda_spacy_score_one = ranking_dataframe(citizen_merged_model_borda_score_spacy_one, citizen_merged_model_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_citizen_merged_model_borda_spacy_score_one)):
    citizen_merged_model_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_citizen_merged_model_borda_spacy_score_one[i]))

# Few Shoot
ranking_citizen_merged_model_spacy_tokens_few = []
ranking_citizen_merged_model_spacy_few = ranking_dataframe(citizen_merged_model_spacy_few, citizen_merged_model_spacy_few.columns[1:])
for i in range(len(ranking_citizen_merged_model_spacy_few)):
    ranking_citizen_merged_model_spacy_tokens_few.append(extrair_tokens(ranking_citizen_merged_model_spacy_few[i]))

citizen_merged_model_borda_score_spacy_few = aplicar_calculo_borda_dataframe(citizen_merged_model_spacy_few, citizen_merged_model_spacy_few.columns[1:])

citizen_merged_model_borda_spacy_tokens_score_few = []
ranking_citizen_merged_model_borda_spacy_score_few = ranking_dataframe(citizen_merged_model_borda_score_spacy_few, citizen_merged_model_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_citizen_merged_model_borda_spacy_score_few)):
    citizen_merged_model_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_citizen_merged_model_borda_spacy_score_few[i]))

### Language

In [ ]:
# Criar um DataFrame com as listas
language_merged_model = pd.DataFrame({
    'answers': language_bert.answers,
    'bert_original': language_bert['original'].tolist(),
    'roberta_original': language_roberta['original'].tolist(),
    'electra_original': language_electra['original'].tolist()
})

# Criar um DataFrame com as listas
language_merged_model_one = pd.DataFrame({
    'answers': language_bert.answers,
    'bert_original': language_bert_one['original'].tolist(),
    'roberta_original': language_roberta_one['original'].tolist(),
    'electra_original': language_electra_one['original'].tolist()
})


# Criar um DataFrame com as listas
language_merged_model_few = pd.DataFrame({
    'answers': language_bert.answers,
    'bert_original': language_bert_few['original'].tolist(),
    'roberta_original': language_roberta_few['original'].tolist(),
    'electra_original': language_electra_few['original'].tolist()
})

# Normal
ranking_language_merged_model_normal_tokens = []
ranking_language_merged_model_normal = ranking_dataframe(language_merged_model, language_merged_model.columns[1:])
for i in range(len(ranking_language_merged_model_normal)):
    ranking_language_merged_model_normal_tokens.append(extrair_tokens(ranking_language_merged_model_normal[i]))

language_merged_model_borda_score = aplicar_calculo_borda_dataframe(language_merged_model, language_merged_model.columns[1:])

language_merged_model_borda_tokens_score = []
ranking_language_merged_model_borda_score = ranking_dataframe(language_merged_model_borda_score, language_merged_model_borda_score.columns[1:])
for i in range(len(ranking_language_merged_model_borda_score)):
    language_merged_model_borda_tokens_score.append(extrair_tokens(ranking_language_merged_model_borda_score[i]))


# One Shoot
ranking_language_merged_model_normal_tokens_one = []
ranking_language_merged_model_normal_one = ranking_dataframe(language_merged_model_one, language_merged_model_one.columns[1:])
for i in range(len(ranking_language_merged_model_normal_one)):
    ranking_language_merged_model_normal_tokens_one.append(extrair_tokens(ranking_language_merged_model_normal_one[i]))

language_merged_model_borda_score_one = aplicar_calculo_borda_dataframe(language_merged_model_one, language_merged_model_one.columns[1:])

language_merged_model_borda_tokens_score_one = []
ranking_language_merged_model_borda_score_one = ranking_dataframe(language_merged_model_borda_score_one, language_merged_model_borda_score_one.columns[1:])
for i in range(len(ranking_language_merged_model_borda_score_one)):
    language_merged_model_borda_tokens_score_one.append(extrair_tokens(ranking_language_merged_model_borda_score_one[i]))


# Few Shoot
ranking_language_merged_model_normal_tokens_few = []
ranking_language_merged_model_normal_few = ranking_dataframe(language_merged_model_few, language_merged_model_few.columns[1:])
for i in range(len(ranking_language_merged_model_normal_few)):
    ranking_language_merged_model_normal_tokens_few.append(extrair_tokens(ranking_language_merged_model_normal_few[i]))

language_merged_model_borda_score_few = aplicar_calculo_borda_dataframe(language_merged_model_few, language_merged_model_few.columns[1:])

language_merged_model_borda_tokens_score_few = []
ranking_language_merged_model_borda_score_few = ranking_dataframe(language_merged_model_borda_score_few, language_merged_model_borda_score_few.columns[1:])
for i in range(len(ranking_language_merged_model_borda_score_few)):
    language_merged_model_borda_tokens_score_few.append(extrair_tokens(ranking_language_merged_model_borda_score_few[i]))

In [ ]:
# Criar um DataFrame com as listas
language_merged_model_spacy = pd.DataFrame({
    'answers': language_bert.answers,
    'bert_original': language_bert_spacy['original'].tolist(),
    'roberta_original': language_roberta_spacy['original'].tolist(),
    'electra_original': language_electra_spacy['original'].tolist()
})


# Criar um DataFrame com as listas
language_merged_model_spacy_one = pd.DataFrame({
    'answers': language_bert.answers,
    'bert_original': language_bert_one_spacy['original'].tolist(),
    'roberta_original': language_roberta_one_spacy['original'].tolist(),
    'electra_original': language_electra_one_spacy['original'].tolist()
})

# Criar um DataFrame com as listas
language_merged_model_spacy_few = pd.DataFrame({
    'answers': language_bert.answers,
    'bert_original': language_bert_few_spacy['original'].tolist(),
    'roberta_original': language_roberta_few_spacy['original'].tolist(),
    'electra_original': language_electra_few_spacy['original'].tolist()
})

# Spacy
ranking_language_merged_model_spacy_tokens = []
ranking_language_merged_model_spacy = ranking_dataframe(language_merged_model_spacy, language_merged_model_spacy.columns[1:])
for i in range(len(ranking_language_merged_model_spacy)):
    ranking_language_merged_model_spacy_tokens.append(extrair_tokens(ranking_language_merged_model_spacy[i]))

language_merged_model_borda_score_spacy = aplicar_calculo_borda_dataframe(language_merged_model_spacy, language_merged_model_spacy.columns[1:])

language_merged_model_borda_spacy_tokens_score = []
ranking_language_merged_model_borda_spacy_score = ranking_dataframe(language_merged_model_borda_score_spacy, language_merged_model_borda_score_spacy.columns[1:])
for i in range(len(ranking_language_merged_model_borda_spacy_score)):
    language_merged_model_borda_spacy_tokens_score.append(extrair_tokens(ranking_language_merged_model_borda_spacy_score[i]))

# One Shoot
ranking_language_merged_model_spacy_tokens_one = []
ranking_language_merged_model_spacy_one = ranking_dataframe(language_merged_model_spacy_one, language_merged_model_spacy_one.columns[1:])
for i in range(len(ranking_language_merged_model_spacy_one)):
    ranking_language_merged_model_spacy_tokens_one.append(extrair_tokens(ranking_language_merged_model_spacy_one[i]))

language_merged_model_borda_score_spacy_one = aplicar_calculo_borda_dataframe(language_merged_model_spacy_one, language_merged_model_spacy_one.columns[1:])

language_merged_model_borda_spacy_tokens_score_one = []
ranking_language_merged_model_borda_spacy_score_one = ranking_dataframe(language_merged_model_borda_score_spacy_one, language_merged_model_borda_score_spacy_one.columns[1:])
for i in range(len(ranking_language_merged_model_borda_spacy_score_one)):
    language_merged_model_borda_spacy_tokens_score_one.append(extrair_tokens(ranking_language_merged_model_borda_spacy_score_one[i]))

# Few Shoot
ranking_language_merged_model_spacy_tokens_few = []
ranking_language_merged_model_spacy_few = ranking_dataframe(language_merged_model_spacy_few, language_merged_model_spacy_few.columns[1:])
for i in range(len(ranking_language_merged_model_spacy_few)):
    ranking_language_merged_model_spacy_tokens_few.append(extrair_tokens(ranking_language_merged_model_spacy_few[i]))

language_merged_model_borda_score_spacy_few = aplicar_calculo_borda_dataframe(language_merged_model_spacy_few, language_merged_model_spacy_few.columns[1:])

language_merged_model_borda_spacy_tokens_score_few = []
ranking_language_merged_model_borda_spacy_score_few = ranking_dataframe(language_merged_model_borda_score_spacy_few, language_merged_model_borda_score_spacy_few.columns[1:])
for i in range(len(ranking_language_merged_model_borda_spacy_score_few)):
    language_merged_model_borda_spacy_tokens_score_few.append(extrair_tokens(ranking_language_merged_model_borda_spacy_score_few[i]))

# Resultados

## Bert

#### BornIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_bornIn_bert_top1_ranking_spacy, _, average_bornIn_bert_top1_ranking_spacy = create_results_list(
    ranking_bornIn_bert_spacy_tokens, bornIn_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Spacy: {average_bornIn_bert_top1_ranking_spacy}')

results_list_bornIn_bert_top1_ranking_normal, _, average_bornIn_bert_top1_ranking_normal = create_results_list(
    ranking_bornIn_bert_normal_tokens, bornIn_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Normal: {average_bornIn_bert_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_bornIn_bert_top1_borda_normal, indicies_borda_bornIn_bert, average_bornIn_bert_top1_borda_normal = create_results_list(
    bornIn_bert_borda_tokens_score, bornIn_bert.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Born In Normal: {average_bornIn_bert_top1_borda_normal}')
print('\n')

results_list_bornIn_bert_top1_borda_spacy, indicies_borda_bornIn_bert_spacy, average_bornIn_bert_top1_borda_spacy = create_results_list(
    bornIn_bert_borda_spacy_tokens_score, bornIn_bert.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Born In spacy: {average_bornIn_bert_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top1_spacy, _, average_bornIn_bert_top1_spacy = create_results_list(
    masked_bornIn_bert_original_spacy, bornIn_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Spacy: {average_bornIn_bert_top1_spacy}')

results_list_bornIn_bert_top1, _, average_bornIn_bert_top1 = create_results_list(
    masked_bornIn_bert_original, bornIn_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Normal: {average_bornIn_bert_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Born In Spacy: 0.346
Avg Precision K = 1 of the ranking of Born In Normal: 0.362


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Born In Normal: 0.359


Borda Spacy Score
Avg Precision K = 1 of the borda of Born In spacy: 0.35


Sem ranking
Avg Precision K = 1 of Born In Spacy: 0.4
Avg Precision K = 1 of Born In Normal: 0.393


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_bornIn_bert_top5_ranking_spacy, _, average_bornIn_bert_top5_ranking_spacy = create_results_list(
    ranking_bornIn_bert_spacy_tokens, bornIn_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Spacy: {average_bornIn_bert_top5_ranking_spacy}')

results_list_bornIn_bert_top5_ranking_normal, _, average_bornIn_bert_top5_ranking_normal = create_results_list(
    ranking_bornIn_bert_normal_tokens, bornIn_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Normal: {average_bornIn_bert_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_bornIn_bert_top5_borda_normal, indicies_borda_bornIn_bert, average_bornIn_bert_top5_borda_normal = create_results_list(
    bornIn_bert_borda_tokens_score, bornIn_bert.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Born In Normal: {average_bornIn_bert_top5_borda_normal}')
print('\n')

results_list_bornIn_bert_top5_borda_spacy, indicies_borda_bornIn_bert_spacy, average_bornIn_bert_top5_borda_spacy = create_results_list(
    bornIn_bert_borda_spacy_tokens_score, bornIn_bert.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Born In spacy: {average_bornIn_bert_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top5_spacy, _, average_bornIn_bert_top5_spacy = create_results_list(
    masked_bornIn_bert_original_spacy, bornIn_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Spacy: {average_bornIn_bert_top5_spacy}')

results_list_bornIn_bert_top5, _, average_bornIn_bert_top5 = create_results_list(
    masked_bornIn_bert_original, bornIn_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Normal: {average_bornIn_bert_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Born In Spacy: 0.639
Avg Precision K = 5 of the ranking of Born In Normal: 0.651


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Born In Normal: 0.647


Borda Spacy Score
Avg Precision K = 5 of the borda of Born In spacy: 0.638


Sem ranking
Avg Precision K = 5 of Born In Spacy: 0.625
Avg Precision K = 5 of Born In Normal: 0.62


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_bornIn_bert_top10_ranking_spacy, _, average_bornIn_bert_top10_ranking_spacy = create_results_list(
    ranking_bornIn_bert_spacy_tokens, bornIn_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Spacy: {average_bornIn_bert_top10_ranking_spacy}')

results_list_bornIn_bert_top10_ranking_normal, _, average_bornIn_bert_top10_ranking_normal = create_results_list(
    ranking_bornIn_bert_normal_tokens, bornIn_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Normal: {average_bornIn_bert_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_bornIn_bert_top10_borda_normal, indicies_borda_bornIn_bert, average_bornIn_bert_top10_borda_normal = create_results_list(
    bornIn_bert_borda_tokens_score, bornIn_bert.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Born In Normal: {average_bornIn_bert_top10_borda_normal}')
print('\n')

results_list_bornIn_bert_top10_borda_spacy, indicies_borda_bornIn_bert_spacy, average_bornIn_bert_top10_borda_spacy = create_results_list(
    bornIn_bert_borda_spacy_tokens_score, bornIn_bert.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Born In spacy: {average_bornIn_bert_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top10_spacy, _, average_bornIn_bert_top10_spacy = create_results_list(
    masked_bornIn_bert_original_spacy, bornIn_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Spacy: {average_bornIn_bert_top10_spacy}')

results_list_bornIn_bert_top10, _, average_bornIn_bert_top10 = create_results_list(
    masked_bornIn_bert_original, bornIn_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Normal: {average_bornIn_bert_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Born In Spacy: 0.703
Avg Precision K = 10 of the ranking of Born In Normal: 0.729


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Born In Normal: 0.725


Borda Spacy Score
Avg Precision K = 10 of the borda of Born In spacy: 0.701


Sem ranking
Avg Precision K = 10 of Born In Spacy: 0.677
Avg Precision K = 10 of Born In Normal: 0.684


#### DiedIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_diedIn_bert_top1_ranking_spacy, _, average_diedIn_bert_top1_ranking_spacy = create_results_list(
    ranking_diedIn_bert_spacy_tokens, diedIn_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy: {average_diedIn_bert_top1_ranking_spacy}')

results_list_diedIn_bert_top1_ranking_normal, _, average_diedIn_bert_top1_ranking_normal = create_results_list(
    ranking_diedIn_bert_normal_tokens, diedIn_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal: {average_diedIn_bert_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_diedIn_bert_top1_borda_normal, indicies_borda_diedIn_bert, average_diedIn_bert_top1_borda_normal = create_results_list(
    diedIn_bert_borda_tokens_score, diedIn_bert.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal: {average_diedIn_bert_top1_borda_normal}')
print('\n')

results_list_diedIn_bert_top1_borda_spacy, indicies_borda_diedIn_bert_spacy, average_diedIn_bert_top1_borda_spacy = create_results_list(
    diedIn_bert_borda_spacy_tokens_score, diedIn_bert.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In spacy: {average_diedIn_bert_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top1_spacy, _, average_diedIn_bert_top1_spacy = create_results_list(
    masked_diedIn_bert_original_spacy, diedIn_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy: {average_diedIn_bert_top1_spacy}')

results_list_diedIn_bert_top1, _, average_diedIn_bert_top1 = create_results_list(
    masked_diedIn_bert_original, diedIn_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_diedIn_bert_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy: 0.469
Avg Precision K = 1 of the ranking of died In Normal: 0.483


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal: 0.479


Borda Spacy Score
Avg Precision K = 1 of the borda of died In spacy: 0.47


Sem ranking
Avg Precision K = 1 of died In Spacy: 0.503
Avg Precision K = 1 of died In Normal: 0.482


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_diedIn_bert_top5_ranking_spacy, _, average_diedIn_bert_top5_ranking_spacy = create_results_list(
    ranking_diedIn_bert_spacy_tokens, diedIn_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy: {average_diedIn_bert_top5_ranking_spacy}')

results_list_diedIn_bert_top5_ranking_normal, _, average_diedIn_bert_top5_ranking_normal = create_results_list(
    ranking_diedIn_bert_normal_tokens, diedIn_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal: {average_diedIn_bert_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_diedIn_bert_top5_borda_normal, indicies_borda_diedIn_bert, average_diedIn_bert_top5_borda_normal = create_results_list(
    diedIn_bert_borda_tokens_score, diedIn_bert.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal: {average_diedIn_bert_top5_borda_normal}')
print('\n')

results_list_diedIn_bert_top5_borda_spacy, indicies_borda_diedIn_bert_spacy, average_diedIn_bert_top5_borda_spacy = create_results_list(
    diedIn_bert_borda_spacy_tokens_score, diedIn_bert.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In spacy: {average_diedIn_bert_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top5_spacy, _, average_diedIn_bert_top5_spacy = create_results_list(
    masked_diedIn_bert_original_spacy, diedIn_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy: {average_diedIn_bert_top5_spacy}')

results_list_diedIn_bert_top5, _, average_diedIn_bert_top5 = create_results_list(
    masked_diedIn_bert_original, diedIn_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_diedIn_bert_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy: 0.716
Avg Precision K = 5 of the ranking of died In Normal: 0.753


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal: 0.756


Borda Spacy Score
Avg Precision K = 5 of the borda of died In spacy: 0.716


Sem ranking
Avg Precision K = 5 of died In Spacy: 0.714
Avg Precision K = 5 of died In Normal: 0.72


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_diedIn_bert_top10_ranking_spacy, _, average_diedIn_bert_top10_ranking_spacy = create_results_list(
    ranking_diedIn_bert_spacy_tokens, diedIn_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy: {average_diedIn_bert_top10_ranking_spacy}')

results_list_diedIn_bert_top10_ranking_normal, _, average_diedIn_bert_top10_ranking_normal = create_results_list(
    ranking_diedIn_bert_normal_tokens, diedIn_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal: {average_diedIn_bert_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_diedIn_bert_top10_borda_normal, indicies_borda_diedIn_bert, average_diedIn_bert_top10_borda_normal = create_results_list(
    diedIn_bert_borda_tokens_score, diedIn_bert.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal: {average_diedIn_bert_top10_borda_normal}')
print('\n')

results_list_diedIn_bert_top10_borda_spacy, indicies_borda_diedIn_bert_spacy, average_diedIn_bert_top10_borda_spacy = create_results_list(
    diedIn_bert_borda_spacy_tokens_score, diedIn_bert.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In spacy: {average_diedIn_bert_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top10_spacy, _, average_diedIn_bert_top10_spacy = create_results_list(
    masked_diedIn_bert_original_spacy, diedIn_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy: {average_diedIn_bert_top10_spacy}')

results_list_diedIn_bert_top10, _, average_diedIn_bert_top10 = create_results_list(
    masked_diedIn_bert_original, diedIn_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_diedIn_bert_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy: 0.79
Avg Precision K = 10 of the ranking of died In Normal: 0.827


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal: 0.82


Borda Spacy Score
Avg Precision K = 10 of the borda of died In spacy: 0.791


Sem ranking
Avg Precision K = 10 of died In Spacy: 0.755
Avg Precision K = 10 of died In Normal: 0.78


#### Worksfor

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_worksfor_bert_top1_ranking_spacy, _, average_worksfor_bert_top1_ranking_spacy = create_results_list(
    ranking_worksfor_bert_spacy_tokens, worksfor_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Works For Spacy: {average_worksfor_bert_top1_ranking_spacy}')

results_list_worksfor_bert_top1_ranking_normal, _, average_worksfor_bert_top1_ranking_normal = create_results_list(
    ranking_worksfor_bert_normal_tokens, worksfor_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Works For Normal: {average_worksfor_bert_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_worksfor_bert_top1_borda_normal, indicies_borda_worksfor_bert, average_worksfor_bert_top1_borda_normal = create_results_list(
    worksfor_bert_borda_tokens_score, worksfor_bert.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Works For Normal: {average_worksfor_bert_top1_borda_normal}')
print('\n')

results_list_worksfor_bert_top1_borda_spacy, indicies_borda_worksfor_bert_spacy, average_worksfor_bert_top1_borda_spacy = create_results_list(
    worksfor_bert_borda_spacy_tokens_score, worksfor_bert.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Works For spacy: {average_worksfor_bert_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top1_spacy, _, average_worksfor_bert_top1_spacy = create_results_list(
    masked_worksfor_bert_original_spacy, worksfor_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Works For Spacy: {average_worksfor_bert_top1_spacy}')

results_list_worksfor_bert_top1, _, average_worksfor_bert_top1 = create_results_list(
    masked_worksfor_bert_original, worksfor_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Works For Normal: {average_worksfor_bert_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Works For Spacy: 0.599
Avg Precision K = 1 of the ranking of Works For Normal: 0.525


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Works For Normal: 0.522


Borda Spacy Score
Avg Precision K = 1 of the borda of Works For spacy: 0.604


Sem ranking
Avg Precision K = 1 of Works For Spacy: 0.586
Avg Precision K = 1 of Works For Normal: 0.491


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_worksfor_bert_top5_ranking_spacy, _, average_worksfor_bert_top5_ranking_spacy = create_results_list(
    ranking_worksfor_bert_spacy_tokens, worksfor_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Works For Spacy: {average_worksfor_bert_top5_ranking_spacy}')

results_list_worksfor_bert_top5_ranking_normal, _, average_worksfor_bert_top5_ranking_normal = create_results_list(
    ranking_worksfor_bert_normal_tokens, worksfor_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Works For Normal: {average_worksfor_bert_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_worksfor_bert_top5_borda_normal, indicies_borda_worksfor_bert, average_worksfor_bert_top5_borda_normal = create_results_list(
    worksfor_bert_borda_tokens_score, worksfor_bert.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Works For Normal: {average_worksfor_bert_top5_borda_normal}')
print('\n')

results_list_worksfor_bert_top5_borda_spacy, indicies_borda_worksfor_bert_spacy, average_worksfor_bert_top5_borda_spacy = create_results_list(
    worksfor_bert_borda_spacy_tokens_score, worksfor_bert.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Works For spacy: {average_worksfor_bert_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top5_spacy, _, average_worksfor_bert_top5_spacy = create_results_list(
    masked_worksfor_bert_original_spacy, worksfor_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Works For Spacy: {average_worksfor_bert_top5_spacy}')

results_list_worksfor_bert_top5, _, average_worksfor_bert_top5 = create_results_list(
    masked_worksfor_bert_original, worksfor_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Works For Normal: {average_worksfor_bert_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Works For Spacy: 0.821
Avg Precision K = 5 of the ranking of Works For Normal: 0.799


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Works For Normal: 0.799


Borda Spacy Score
Avg Precision K = 5 of the borda of Works For spacy: 0.815


Sem ranking
Avg Precision K = 5 of Works For Spacy: 0.81
Avg Precision K = 5 of Works For Normal: 0.784


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_worksfor_bert_top10_ranking_spacy, _, average_worksfor_bert_top10_ranking_spacy = create_results_list(
    ranking_worksfor_bert_spacy_tokens, worksfor_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Works For Spacy: {average_worksfor_bert_top10_ranking_spacy}')

results_list_worksfor_bert_top10_ranking_normal, _, average_worksfor_bert_top10_ranking_normal = create_results_list(
    ranking_worksfor_bert_normal_tokens, worksfor_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Works For Normal: {average_worksfor_bert_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_worksfor_bert_top10_borda_normal, indicies_borda_worksfor_bert, average_worksfor_bert_top10_borda_normal = create_results_list(
    worksfor_bert_borda_tokens_score, worksfor_bert.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Works For Normal: {average_worksfor_bert_top10_borda_normal}')
print('\n')

results_list_worksfor_bert_top10_borda_spacy, indicies_borda_worksfor_bert_spacy, average_worksfor_bert_top10_borda_spacy = create_results_list(
    worksfor_bert_borda_spacy_tokens_score, worksfor_bert.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Works For spacy: {average_worksfor_bert_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top10_spacy, _, average_worksfor_bert_top10_spacy = create_results_list(
    masked_worksfor_bert_original_spacy, worksfor_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Works For Spacy: {average_worksfor_bert_top10_spacy}')

results_list_worksfor_bert_top10, _, average_worksfor_bert_top10 = create_results_list(
    masked_worksfor_bert_original, worksfor_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Works For Normal: {average_worksfor_bert_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Works For Spacy: 0.847
Avg Precision K = 10 of the ranking of Works For Normal: 0.873


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Works For Normal: 0.879


Borda Spacy Score
Avg Precision K = 10 of the borda of Works For spacy: 0.844


Sem ranking
Avg Precision K = 10 of Works For Spacy: 0.815
Avg Precision K = 10 of Works For Normal: 0.858


#### Capital

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_capital_bert_top1_ranking_spacy, _, average_capital_bert_top1_ranking_spacy = create_results_list(
    ranking_capital_bert_spacy_tokens, capital_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Capital_bert Spacy: {average_capital_bert_top1_ranking_spacy}')

results_list_capital_bert_top1_ranking_normal, _, average_capital_bert_top1_ranking_normal = create_results_list(
    ranking_capital_bert_normal_tokens, capital_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Capital_bert Normal: {average_capital_bert_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_capital_bert_top1_borda_normal, indicies_borda_capital_bert, average_capital_bert_top1_borda_normal = create_results_list(
    capital_bert_borda_tokens_score, capital_bert.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Capital_bert Normal: {average_capital_bert_top1_borda_normal}')
print('\n')

results_list_capital_bert_top1_borda_spacy, indicies_borda_capital_bert_spacy, average_capital_bert_top1_borda_spacy = create_results_list(
    capital_bert_borda_spacy_tokens_score, capital_bert.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Capital_bert spacy: {average_capital_bert_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top1_spacy, _, average_capital_bert_top1_spacy = create_results_list(
    masked_capital_bert_original_spacy, capital_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Capital_bert Spacy: {average_capital_bert_top1_spacy}')

results_list_capital_bert_top1, _, average_capital_bert_top1 = create_results_list(
    masked_capital_bert_original, capital_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Capital_bert Normal: {average_capital_bert_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Capital_bert Spacy: 0.465
Avg Precision K = 1 of the ranking of Capital_bert Normal: 0.517


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Capital_bert Normal: 0.515


Borda Spacy Score
Avg Precision K = 1 of the borda of Capital_bert spacy: 0.457


Sem ranking
Avg Precision K = 1 of Capital_bert Spacy: 0.554
Avg Precision K = 1 of Capital_bert Normal: 0.545


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_capital_bert_top5_ranking_spacy, _, average_capital_bert_top5_ranking_spacy = create_results_list(
    ranking_capital_bert_spacy_tokens, capital_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Capital_bert Spacy: {average_capital_bert_top5_ranking_spacy}')

results_list_capital_bert_top5_ranking_normal, _, average_capital_bert_top5_ranking_normal = create_results_list(
    ranking_capital_bert_normal_tokens, capital_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Capital_bert Normal: {average_capital_bert_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_capital_bert_top5_borda_normal, indicies_borda_capital_bert, average_capital_bert_top5_borda_normal = create_results_list(
    capital_bert_borda_tokens_score, capital_bert.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Capital_bert Normal: {average_capital_bert_top5_borda_normal}')
print('\n')

results_list_capital_bert_top5_borda_spacy, indicies_borda_capital_bert_spacy, average_capital_bert_top5_borda_spacy = create_results_list(
    capital_bert_borda_spacy_tokens_score, capital_bert.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Capital_bert spacy: {average_capital_bert_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top5_spacy, _, average_capital_bert_top5_spacy = create_results_list(
    masked_capital_bert_original_spacy, capital_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Capital_bert Spacy: {average_capital_bert_top5_spacy}')

results_list_capital_bert_top5, _, average_capital_bert_top5 = create_results_list(
    masked_capital_bert_original, capital_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Capital_bert Normal: {average_capital_bert_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Capital_bert Spacy: 0.654
Avg Precision K = 5 of the ranking of Capital_bert Normal: 0.756


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Capital_bert Normal: 0.755


Borda Spacy Score
Avg Precision K = 5 of the borda of Capital_bert spacy: 0.652


Sem ranking
Avg Precision K = 5 of Capital_bert Spacy: 0.722
Avg Precision K = 5 of Capital_bert Normal: 0.75


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_capital_bert_top10_ranking_spacy, _, average_capital_bert_top10_ranking_spacy = create_results_list(
    ranking_capital_bert_spacy_tokens, capital_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Capital_bert Spacy: {average_capital_bert_top10_ranking_spacy}')

results_list_capital_bert_top10_ranking_normal, _, average_capital_bert_top10_ranking_normal = create_results_list(
    ranking_capital_bert_normal_tokens, capital_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Capital_bert Normal: {average_capital_bert_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_capital_bert_top10_borda_normal, indicies_borda_capital_bert, average_capital_bert_top10_borda_normal = create_results_list(
    capital_bert_borda_tokens_score, capital_bert.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Capital_bert Normal: {average_capital_bert_top10_borda_normal}')
print('\n')

results_list_capital_bert_top10_borda_spacy, indicies_borda_capital_bert_spacy, average_capital_bert_top10_borda_spacy = create_results_list(
    capital_bert_borda_spacy_tokens_score, capital_bert.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Capital_bert spacy: {average_capital_bert_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top10_spacy, _, average_capital_bert_top10_spacy = create_results_list(
    masked_capital_bert_original_spacy, capital_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Capital_bert Spacy: {average_capital_bert_top10_spacy}')

results_list_capital_bert_top10, _, average_capital_bert_top10 = create_results_list(
    masked_capital_bert_original, capital_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Capital_bert Normal: {average_capital_bert_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Capital_bert Spacy: 0.717
Avg Precision K = 10 of the ranking of Capital_bert Normal: 0.81


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Capital_bert Normal: 0.812


Borda Spacy Score
Avg Precision K = 10 of the borda of Capital_bert spacy: 0.717


Sem ranking
Avg Precision K = 10 of Capital_bert Spacy: 0.736
Avg Precision K = 10 of Capital_bert Normal: 0.785


#### Citizen

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_citizen_bert_top1_ranking_spacy, _, average_citizen_bert_top1_ranking_spacy = create_results_list(
    ranking_citizen_bert_spacy_tokens, citizen_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of citizen_bert Spacy: {average_citizen_bert_top1_ranking_spacy}')

results_list_citizen_bert_top1_ranking_normal, _, average_citizen_bert_top1_ranking_normal = create_results_list(
    ranking_citizen_bert_normal_tokens, citizen_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of citizen_bert Normal: {average_citizen_bert_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_bert_top1_borda_normal, indicies_borda_citizen_bert, average_citizen_bert_top1_borda_normal = create_results_list(
    citizen_bert_borda_tokens_score, citizen_bert.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of citizen_bert Normal: {average_citizen_bert_top1_borda_normal}')
print('\n')

results_list_citizen_bert_top1_borda_spacy, indicies_borda_citizen_bert_spacy, average_citizen_bert_top1_borda_spacy = create_results_list(
    citizen_bert_borda_spacy_tokens_score, citizen_bert.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of citizen_bert spacy: {average_citizen_bert_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top1_spacy, _, average_citizen_bert_top1_spacy = create_results_list(
    masked_citizen_bert_original_spacy, citizen_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of citizen_bert Spacy: {average_citizen_bert_top1_spacy}')

results_list_citizen_bert_top1, _, average_citizen_bert_top1 = create_results_list(
    masked_citizen_bert_original, citizen_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of citizen_bert Normal: {average_citizen_bert_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of citizen_bert Spacy: 0.445
Avg Precision K = 1 of the ranking of citizen_bert Normal: 0.632


Borda Count
Borda Score
Avg Precision K = 1 of the borda of citizen_bert Normal: 0.632


Borda Spacy Score
Avg Precision K = 1 of the borda of citizen_bert spacy: 0.442


Sem ranking
Avg Precision K = 1 of citizen_bert Spacy: 0.423
Avg Precision K = 1 of citizen_bert Normal: 0.58


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_citizen_bert_top5_ranking_spacy, _, average_citizen_bert_top5_ranking_spacy = create_results_list(
    ranking_citizen_bert_spacy_tokens, citizen_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of citizen_bert Spacy: {average_citizen_bert_top5_ranking_spacy}')

results_list_citizen_bert_top5_ranking_normal, _, average_citizen_bert_top5_ranking_normal = create_results_list(
    ranking_citizen_bert_normal_tokens, citizen_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of citizen_bert Normal: {average_citizen_bert_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_bert_top5_borda_normal, indicies_borda_citizen_bert, average_citizen_bert_top5_borda_normal = create_results_list(
    citizen_bert_borda_tokens_score, citizen_bert.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of citizen_bert Normal: {average_citizen_bert_top5_borda_normal}')
print('\n')

results_list_citizen_bert_top5_borda_spacy, indicies_borda_citizen_bert_spacy, average_citizen_bert_top5_borda_spacy = create_results_list(
    citizen_bert_borda_spacy_tokens_score, citizen_bert.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of citizen_bert spacy: {average_citizen_bert_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top5_spacy, _, average_citizen_bert_top5_spacy = create_results_list(
    masked_citizen_bert_original_spacy, citizen_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of citizen_bert Spacy: {average_citizen_bert_top5_spacy}')

results_list_citizen_bert_top5, _, average_citizen_bert_top5 = create_results_list(
    masked_citizen_bert_original, citizen_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of citizen_bert Normal: {average_citizen_bert_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of citizen_bert Spacy: 0.544
Avg Precision K = 5 of the ranking of citizen_bert Normal: 0.826


Borda Count
Borda Score
Avg Precision K = 5 of the borda of citizen_bert Normal: 0.825


Borda Spacy Score
Avg Precision K = 5 of the borda of citizen_bert spacy: 0.544


Sem ranking
Avg Precision K = 5 of citizen_bert Spacy: 0.488
Avg Precision K = 5 of citizen_bert Normal: 0.732


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_citizen_bert_top10_ranking_spacy, _, average_citizen_bert_top10_ranking_spacy = create_results_list(
    ranking_citizen_bert_spacy_tokens, citizen_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of citizen_bert Spacy: {average_citizen_bert_top10_ranking_spacy}')

results_list_citizen_bert_top10_ranking_normal, _, average_citizen_bert_top10_ranking_normal = create_results_list(
    ranking_citizen_bert_normal_tokens, citizen_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of citizen_bert Normal: {average_citizen_bert_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_bert_top10_borda_normal, indicies_borda_citizen_bert, average_citizen_bert_top10_borda_normal = create_results_list(
    citizen_bert_borda_tokens_score, citizen_bert.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of citizen_bert Normal: {average_citizen_bert_top10_borda_normal}')
print('\n')

results_list_citizen_bert_top10_borda_spacy, indicies_borda_citizen_bert_spacy, average_citizen_bert_top10_borda_spacy = create_results_list(
    citizen_bert_borda_spacy_tokens_score, citizen_bert.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of citizen_bert spacy: {average_citizen_bert_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top10_spacy, _, average_citizen_bert_top10_spacy = create_results_list(
    masked_citizen_bert_original_spacy, citizen_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of citizen_bert Spacy: {average_citizen_bert_top10_spacy}')

results_list_citizen_bert_top10, _, average_citizen_bert_top10 = create_results_list(
    masked_citizen_bert_original, citizen_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of citizen_bert Normal: {average_citizen_bert_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of citizen_bert Spacy: 0.562
Avg Precision K = 10 of the ranking of citizen_bert Normal: 0.879


Borda Count
Borda Score
Avg Precision K = 10 of the borda of citizen_bert Normal: 0.881


Borda Spacy Score
Avg Precision K = 10 of the borda of citizen_bert spacy: 0.562


Sem ranking
Avg Precision K = 10 of citizen_bert Spacy: 0.491
Avg Precision K = 10 of citizen_bert Normal: 0.802


#### Language

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_language_bert_top1_ranking_spacy, _, average_language_bert_top1_ranking_spacy = create_results_list(
    ranking_language_bert_spacy_tokens, language_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of language_bert Spacy: {average_language_bert_top1_ranking_spacy}')

results_list_language_bert_top1_ranking_normal, _, average_language_bert_top1_ranking_normal = create_results_list(
    ranking_language_bert_normal_tokens, language_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of language_bert Normal: {average_language_bert_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_bert_top1_borda_normal, indicies_borda_language_bert, average_language_bert_top1_borda_normal = create_results_list(
    language_bert_borda_tokens_score, language_bert.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of language_bert Normal: {average_language_bert_top1_borda_normal}')
print('\n')

results_list_language_bert_top1_borda_spacy, indicies_borda_language_bert_spacy, average_language_bert_top1_borda_spacy = create_results_list(
    language_bert_borda_spacy_tokens_score, language_bert.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of language_bert spacy: {average_language_bert_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_bert_top1_spacy, _, average_language_bert_top1_spacy = create_results_list(
    masked_language_bert_original_spacy, language_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language_bert Spacy: {average_language_bert_top1_spacy}')

results_list_language_bert_top1, _, average_language_bert_top1 = create_results_list(
    masked_language_bert_original, language_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language_bert Normal: {average_language_bert_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of language_bert Spacy: 0.456
Avg Precision K = 1 of the ranking of language_bert Normal: 0.676


Borda Count
Borda Score
Avg Precision K = 1 of the borda of language_bert Normal: 0.674


Borda Spacy Score
Avg Precision K = 1 of the borda of language_bert spacy: 0.451


Sem ranking
Avg Precision K = 1 of language_bert Spacy: 0.674
Avg Precision K = 1 of language_bert Normal: 0.738


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_language_bert_top5_ranking_spacy, _, average_language_bert_top5_ranking_spacy = create_results_list(
    ranking_language_bert_spacy_tokens, language_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of language_bert Spacy: {average_language_bert_top5_ranking_spacy}')

results_list_language_bert_top5_ranking_normal, _, average_language_bert_top5_ranking_normal = create_results_list(
    ranking_language_bert_normal_tokens, language_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of language_bert Normal: {average_language_bert_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_bert_top5_borda_normal, indicies_borda_language_bert, average_language_bert_top5_borda_normal = create_results_list(
    language_bert_borda_tokens_score, language_bert.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of language_bert Normal: {average_language_bert_top5_borda_normal}')
print('\n')

results_list_language_bert_top5_borda_spacy, indicies_borda_language_bert_spacy, average_language_bert_top5_borda_spacy = create_results_list(
    language_bert_borda_spacy_tokens_score, language_bert.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of language_bert spacy: {average_language_bert_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_bert_top5_spacy, _, average_language_bert_top5_spacy = create_results_list(
    masked_language_bert_original_spacy, language_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language_bert Spacy: {average_language_bert_top5_spacy}')

results_list_language_bert_top5, _, average_language_bert_top5 = create_results_list(
    masked_language_bert_original, language_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language_bert Normal: {average_language_bert_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of language_bert Spacy: 0.611
Avg Precision K = 5 of the ranking of language_bert Normal: 0.903


Borda Count
Borda Score
Avg Precision K = 5 of the borda of language_bert Normal: 0.902


Borda Spacy Score
Avg Precision K = 5 of the borda of language_bert spacy: 0.616


Sem ranking
Avg Precision K = 5 of language_bert Spacy: 0.77
Avg Precision K = 5 of language_bert Normal: 0.885


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_language_bert_top10_ranking_spacy, _, average_language_bert_top10_ranking_spacy = create_results_list(
    ranking_language_bert_spacy_tokens, language_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of language_bert Spacy: {average_language_bert_top10_ranking_spacy}')

results_list_language_bert_top10_ranking_normal, _, average_language_bert_top10_ranking_normal = create_results_list(
    ranking_language_bert_normal_tokens, language_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of language_bert Normal: {average_language_bert_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_bert_top10_borda_normal, indicies_borda_language_bert, average_language_bert_top10_borda_normal = create_results_list(
    language_bert_borda_tokens_score, language_bert.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of language_bert Normal: {average_language_bert_top10_borda_normal}')
print('\n')

results_list_language_bert_top10_borda_spacy, indicies_borda_language_bert_spacy, average_language_bert_top10_borda_spacy = create_results_list(
    language_bert_borda_spacy_tokens_score, language_bert.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of language_bert spacy: {average_language_bert_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_bert_top10_spacy, _, average_language_bert_top10_spacy = create_results_list(
    masked_language_bert_original_spacy, language_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language_bert Spacy: {average_language_bert_top10_spacy}')

results_list_language_bert_top10, _, average_language_bert_top10 = create_results_list(
    masked_language_bert_original, language_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language_bert Normal: {average_language_bert_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of language_bert Spacy: 0.643
Avg Precision K = 10 of the ranking of language_bert Normal: 0.937


Borda Count
Borda Score
Avg Precision K = 10 of the borda of language_bert Normal: 0.935


Borda Spacy Score
Avg Precision K = 10 of the borda of language_bert spacy: 0.644


Sem ranking
Avg Precision K = 10 of language_bert Spacy: 0.779
Avg Precision K = 10 of language_bert Normal: 0.916


## Roberta

#### BornIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_bornIn_roberta_top1_ranking_spacy, _, average_bornIn_roberta_top1_ranking_spacy = create_results_list(
    ranking_bornIn_roberta_spacy_tokens, bornIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Spacy: {average_bornIn_roberta_top1_ranking_spacy}')

results_list_bornIn_roberta_top1_ranking_normal, _, average_bornIn_roberta_top1_ranking_normal = create_results_list(
    ranking_bornIn_roberta_normal_tokens, bornIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Normal: {average_bornIn_roberta_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_bornIn_roberta_top1_borda_normal, indicies_borda_bornIn_roberta, average_bornIn_roberta_top1_borda_normal = create_results_list(
    bornIn_roberta_borda_tokens_score, bornIn_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Born In Normal: {average_bornIn_roberta_top1_borda_normal}')
print('\n')

results_list_bornIn_roberta_top1_borda_spacy, indicies_borda_bornIn_roberta_spacy, average_bornIn_roberta_top1_borda_spacy = create_results_list(
    bornIn_roberta_borda_spacy_tokens_score, bornIn_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Born In spacy: {average_bornIn_roberta_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_bornIn_roberta_top1_spacy, _, average_bornIn_roberta_top1_spacy = create_results_list(
    masked_bornIn_roberta_original_spacy, bornIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Spacy: {average_bornIn_roberta_top1_spacy}')

results_list_bornIn_roberta_top1, _, average_bornIn_roberta_top1 = create_results_list(
    masked_bornIn_roberta_original, bornIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Normal: {average_bornIn_roberta_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Born In Spacy: 0.29
Avg Precision K = 1 of the ranking of Born In Normal: 0.299


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Born In Normal: 0.299


Borda Spacy Score
Avg Precision K = 1 of the borda of Born In spacy: 0.294


Sem ranking
Avg Precision K = 1 of Born In Spacy: 0.337
Avg Precision K = 1 of Born In Normal: 0.329


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_bornIn_roberta_top5_ranking_spacy, _, average_bornIn_roberta_top5_ranking_spacy = create_results_list(
    ranking_bornIn_roberta_spacy_tokens, bornIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Spacy: {average_bornIn_roberta_top5_ranking_spacy}')

results_list_bornIn_roberta_top5_ranking_normal, _, average_bornIn_roberta_top5_ranking_normal = create_results_list(
    ranking_bornIn_roberta_normal_tokens, bornIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Normal: {average_bornIn_roberta_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_bornIn_roberta_top5_borda_normal, indicies_borda_bornIn_roberta, average_bornIn_roberta_top5_borda_normal = create_results_list(
    bornIn_roberta_borda_tokens_score, bornIn_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Born In Normal: {average_bornIn_roberta_top5_borda_normal}')
print('\n')

results_list_bornIn_roberta_top5_borda_spacy, indicies_borda_bornIn_roberta_spacy, average_bornIn_roberta_top5_borda_spacy = create_results_list(
    bornIn_roberta_borda_spacy_tokens_score, bornIn_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Born In spacy: {average_bornIn_roberta_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_bornIn_roberta_top5_spacy, _, average_bornIn_roberta_top5_spacy = create_results_list(
    masked_bornIn_roberta_original_spacy, bornIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Spacy: {average_bornIn_roberta_top5_spacy}')

results_list_bornIn_roberta_top5, _, average_bornIn_roberta_top5 = create_results_list(
    masked_bornIn_roberta_original, bornIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Normal: {average_bornIn_roberta_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Born In Spacy: 0.51
Avg Precision K = 5 of the ranking of Born In Normal: 0.529


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Born In Normal: 0.527


Borda Spacy Score
Avg Precision K = 5 of the borda of Born In spacy: 0.511


Sem ranking
Avg Precision K = 5 of Born In Spacy: 0.528
Avg Precision K = 5 of Born In Normal: 0.52


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_bornIn_roberta_top10_ranking_spacy, _, average_bornIn_roberta_top10_ranking_spacy = create_results_list(
    ranking_bornIn_roberta_spacy_tokens, bornIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Spacy: {average_bornIn_roberta_top10_ranking_spacy}')

results_list_bornIn_roberta_top10_ranking_normal, _, average_bornIn_roberta_top10_ranking_normal = create_results_list(
    ranking_bornIn_roberta_normal_tokens, bornIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Normal: {average_bornIn_roberta_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_bornIn_roberta_top10_borda_normal, indicies_borda_bornIn_roberta, average_bornIn_roberta_top10_borda_normal = create_results_list(
    bornIn_roberta_borda_tokens_score, bornIn_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Born In Normal: {average_bornIn_roberta_top10_borda_normal}')
print('\n')

results_list_bornIn_roberta_top10_borda_spacy, indicies_borda_bornIn_roberta_spacy, average_bornIn_roberta_top10_borda_spacy = create_results_list(
    bornIn_roberta_borda_spacy_tokens_score, bornIn_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Born In spacy: {average_bornIn_roberta_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_bornIn_roberta_top10_spacy, _, average_bornIn_roberta_top10_spacy = create_results_list(
    masked_bornIn_roberta_original_spacy, bornIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Spacy: {average_bornIn_roberta_top10_spacy}')

results_list_bornIn_roberta_top10, _, average_bornIn_roberta_top10 = create_results_list(
    masked_bornIn_roberta_original, bornIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Normal: {average_bornIn_roberta_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Born In Spacy: 0.575
Avg Precision K = 10 of the ranking of Born In Normal: 0.601


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Born In Normal: 0.602


Borda Spacy Score
Avg Precision K = 10 of the borda of Born In spacy: 0.578


Sem ranking
Avg Precision K = 10 of Born In Spacy: 0.571
Avg Precision K = 10 of Born In Normal: 0.577


#### DiedIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_diedIn_roberta_top1_ranking_spacy, _, average_diedIn_roberta_top1_ranking_spacy = create_results_list(
    ranking_diedIn_roberta_spacy_tokens, diedIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy: {average_diedIn_roberta_top1_ranking_spacy}')

results_list_diedIn_roberta_top1_ranking_normal, _, average_diedIn_roberta_top1_ranking_normal = create_results_list(
    ranking_diedIn_roberta_normal_tokens, diedIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal: {average_diedIn_roberta_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_diedIn_roberta_top1_borda_normal, indicies_borda_diedIn_roberta, average_diedIn_roberta_top1_borda_normal = create_results_list(
    diedIn_roberta_borda_tokens_score, diedIn_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal: {average_diedIn_roberta_top1_borda_normal}')
print('\n')

results_list_diedIn_roberta_top1_borda_spacy, indicies_borda_diedIn_roberta_spacy, average_diedIn_roberta_top1_borda_spacy = create_results_list(
    diedIn_roberta_borda_spacy_tokens_score, diedIn_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In spacy: {average_diedIn_roberta_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_diedIn_roberta_top1_spacy, _, average_diedIn_roberta_top1_spacy = create_results_list(
    masked_diedIn_roberta_original_spacy, diedIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy: {average_diedIn_roberta_top1_spacy}')

results_list_diedIn_roberta_top1, _, average_diedIn_roberta_top1 = create_results_list(
    masked_diedIn_roberta_original, diedIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_diedIn_roberta_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy: 0.412
Avg Precision K = 1 of the ranking of died In Normal: 0.424


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal: 0.421


Borda Spacy Score
Avg Precision K = 1 of the borda of died In spacy: 0.41


Sem ranking
Avg Precision K = 1 of died In Spacy: 0.441
Avg Precision K = 1 of died In Normal: 0.421


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_diedIn_roberta_top5_ranking_spacy, _, average_diedIn_roberta_top5_ranking_spacy = create_results_list(
    ranking_diedIn_roberta_spacy_tokens, diedIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy: {average_diedIn_roberta_top5_ranking_spacy}')

results_list_diedIn_roberta_top5_ranking_normal, _, average_diedIn_roberta_top5_ranking_normal = create_results_list(
    ranking_diedIn_roberta_normal_tokens, diedIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal: {average_diedIn_roberta_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_diedIn_roberta_top5_borda_normal, indicies_borda_diedIn_roberta, average_diedIn_roberta_top5_borda_normal = create_results_list(
    diedIn_roberta_borda_tokens_score, diedIn_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal: {average_diedIn_roberta_top5_borda_normal}')
print('\n')

results_list_diedIn_roberta_top5_borda_spacy, indicies_borda_diedIn_roberta_spacy, average_diedIn_roberta_top5_borda_spacy = create_results_list(
    diedIn_roberta_borda_spacy_tokens_score, diedIn_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In spacy: {average_diedIn_roberta_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_diedIn_roberta_top5_spacy, _, average_diedIn_roberta_top5_spacy = create_results_list(
    masked_diedIn_roberta_original_spacy, diedIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy: {average_diedIn_roberta_top5_spacy}')

results_list_diedIn_roberta_top5, _, average_diedIn_roberta_top5 = create_results_list(
    masked_diedIn_roberta_original, diedIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_diedIn_roberta_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy: 0.634
Avg Precision K = 5 of the ranking of died In Normal: 0.659


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal: 0.659


Borda Spacy Score
Avg Precision K = 5 of the borda of died In spacy: 0.636


Sem ranking
Avg Precision K = 5 of died In Spacy: 0.64
Avg Precision K = 5 of died In Normal: 0.631


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_diedIn_roberta_top10_ranking_spacy, _, average_diedIn_roberta_top10_ranking_spacy = create_results_list(
    ranking_diedIn_roberta_spacy_tokens, diedIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy: {average_diedIn_roberta_top10_ranking_spacy}')

results_list_diedIn_roberta_top10_ranking_normal, _, average_diedIn_roberta_top10_ranking_normal = create_results_list(
    ranking_diedIn_roberta_normal_tokens, diedIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal: {average_diedIn_roberta_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_diedIn_roberta_top10_borda_normal, indicies_borda_diedIn_roberta, average_diedIn_roberta_top10_borda_normal = create_results_list(
    diedIn_roberta_borda_tokens_score, diedIn_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal: {average_diedIn_roberta_top10_borda_normal}')
print('\n')

results_list_diedIn_roberta_top10_borda_spacy, indicies_borda_diedIn_roberta_spacy, average_diedIn_roberta_top10_borda_spacy = create_results_list(
    diedIn_roberta_borda_spacy_tokens_score, diedIn_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In spacy: {average_diedIn_roberta_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_diedIn_roberta_top10_spacy, _, average_diedIn_roberta_top10_spacy = create_results_list(
    masked_diedIn_roberta_original_spacy, diedIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy: {average_diedIn_roberta_top10_spacy}')

results_list_diedIn_roberta_top10, _, average_diedIn_roberta_top10 = create_results_list(
    masked_diedIn_roberta_original, diedIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_diedIn_roberta_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy: 0.678
Avg Precision K = 10 of the ranking of died In Normal: 0.715


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal: 0.716


Borda Spacy Score
Avg Precision K = 10 of the borda of died In spacy: 0.68


Sem ranking
Avg Precision K = 10 of died In Spacy: 0.667
Avg Precision K = 10 of died In Normal: 0.689


#### Worksfor

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_worksfor_roberta_top1_ranking_spacy, _, average_worksfor_roberta_top1_ranking_spacy = create_results_list(
    ranking_worksfor_roberta_spacy_tokens, worksfor_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Works For Spacy: {average_worksfor_roberta_top1_ranking_spacy}')

results_list_worksfor_roberta_top1_ranking_normal, _, average_worksfor_roberta_top1_ranking_normal = create_results_list(
    ranking_worksfor_roberta_normal_tokens, worksfor_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Works For Normal: {average_worksfor_roberta_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_worksfor_roberta_top1_borda_normal, indicies_borda_worksfor_roberta, average_worksfor_roberta_top1_borda_normal = create_results_list(
    worksfor_roberta_borda_tokens_score, worksfor_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Works For Normal: {average_worksfor_roberta_top1_borda_normal}')
print('\n')

results_list_worksfor_roberta_top1_borda_spacy, indicies_borda_worksfor_roberta_spacy, average_worksfor_roberta_top1_borda_spacy = create_results_list(
    worksfor_roberta_borda_spacy_tokens_score, worksfor_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Works For spacy: {average_worksfor_roberta_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_worksfor_roberta_top1_spacy, _, average_worksfor_roberta_top1_spacy = create_results_list(
    masked_worksfor_roberta_original_spacy, worksfor_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Works For Spacy: {average_worksfor_roberta_top1_spacy}')

results_list_worksfor_roberta_top1, _, average_worksfor_roberta_top1 = create_results_list(
    masked_worksfor_roberta_original, worksfor_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Works For Normal: {average_worksfor_roberta_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Works For Spacy: 0.609
Avg Precision K = 1 of the ranking of Works For Normal: 0.58


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Works For Normal: 0.573


Borda Spacy Score
Avg Precision K = 1 of the borda of Works For spacy: 0.609


Sem ranking
Avg Precision K = 1 of Works For Spacy: 0.639
Avg Precision K = 1 of Works For Normal: 0.594


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_worksfor_roberta_top5_ranking_spacy, _, average_worksfor_roberta_top5_ranking_spacy = create_results_list(
    ranking_worksfor_roberta_spacy_tokens, worksfor_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Works For Spacy: {average_worksfor_roberta_top5_ranking_spacy}')

results_list_worksfor_roberta_top5_ranking_normal, _, average_worksfor_roberta_top5_ranking_normal = create_results_list(
    ranking_worksfor_roberta_normal_tokens, worksfor_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Works For Normal: {average_worksfor_roberta_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_worksfor_roberta_top5_borda_normal, indicies_borda_worksfor_roberta, average_worksfor_roberta_top5_borda_normal = create_results_list(
    worksfor_roberta_borda_tokens_score, worksfor_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Works For Normal: {average_worksfor_roberta_top5_borda_normal}')
print('\n')

results_list_worksfor_roberta_top5_borda_spacy, indicies_borda_worksfor_roberta_spacy, average_worksfor_roberta_top5_borda_spacy = create_results_list(
    worksfor_roberta_borda_spacy_tokens_score, worksfor_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Works For spacy: {average_worksfor_roberta_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_worksfor_roberta_top5_spacy, _, average_worksfor_roberta_top5_spacy = create_results_list(
    masked_worksfor_roberta_original_spacy, worksfor_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Works For Spacy: {average_worksfor_roberta_top5_spacy}')

results_list_worksfor_roberta_top5, _, average_worksfor_roberta_top5 = create_results_list(
    masked_worksfor_roberta_original, worksfor_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Works For Normal: {average_worksfor_roberta_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Works For Spacy: 0.823
Avg Precision K = 5 of the ranking of Works For Normal: 0.828


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Works For Normal: 0.823


Borda Spacy Score
Avg Precision K = 5 of the borda of Works For spacy: 0.818


Sem ranking
Avg Precision K = 5 of Works For Spacy: 0.797
Avg Precision K = 5 of Works For Normal: 0.792


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_worksfor_roberta_top10_ranking_spacy, _, average_worksfor_roberta_top10_ranking_spacy = create_results_list(
    ranking_worksfor_roberta_spacy_tokens, worksfor_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Works For Spacy: {average_worksfor_roberta_top10_ranking_spacy}')

results_list_worksfor_roberta_top10_ranking_normal, _, average_worksfor_roberta_top10_ranking_normal = create_results_list(
    ranking_worksfor_roberta_normal_tokens, worksfor_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Works For Normal: {average_worksfor_roberta_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_worksfor_roberta_top10_borda_normal, indicies_borda_worksfor_roberta, average_worksfor_roberta_top10_borda_normal = create_results_list(
    worksfor_roberta_borda_tokens_score, worksfor_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Works For Normal: {average_worksfor_roberta_top10_borda_normal}')
print('\n')

results_list_worksfor_roberta_top10_borda_spacy, indicies_borda_worksfor_roberta_spacy, average_worksfor_roberta_top10_borda_spacy = create_results_list(
    worksfor_roberta_borda_spacy_tokens_score, worksfor_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Works For spacy: {average_worksfor_roberta_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_worksfor_roberta_top10_spacy, _, average_worksfor_roberta_top10_spacy = create_results_list(
    masked_worksfor_roberta_original_spacy, worksfor_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Works For Spacy: {average_worksfor_roberta_top10_spacy}')

results_list_worksfor_roberta_top10, _, average_worksfor_roberta_top10 = create_results_list(
    masked_worksfor_roberta_original, worksfor_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Works For Normal: {average_worksfor_roberta_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Works For Spacy: 0.865
Avg Precision K = 10 of the ranking of Works For Normal: 0.894


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Works For Normal: 0.902


Borda Spacy Score
Avg Precision K = 10 of the borda of Works For spacy: 0.865


Sem ranking
Avg Precision K = 10 of Works For Spacy: 0.813
Avg Precision K = 10 of Works For Normal: 0.855


#### Capital

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_capital_roberta_top1_ranking_spacy, _, average_capital_roberta_top1_ranking_spacy = create_results_list(
    ranking_capital_roberta_spacy_tokens, capital_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Capital_roberta Spacy: {average_capital_roberta_top1_ranking_spacy}')

results_list_capital_roberta_top1_ranking_normal, _, average_capital_roberta_top1_ranking_normal = create_results_list(
    ranking_capital_roberta_normal_tokens, capital_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Capital_roberta Normal: {average_capital_roberta_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_capital_roberta_top1_borda_normal, indicies_borda_capital_roberta, average_capital_roberta_top1_borda_normal = create_results_list(
    capital_roberta_borda_tokens_score, capital_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Capital_roberta Normal: {average_capital_roberta_top1_borda_normal}')
print('\n')

results_list_capital_roberta_top1_borda_spacy, indicies_borda_capital_roberta_spacy, average_capital_roberta_top1_borda_spacy = create_results_list(
    capital_roberta_borda_spacy_tokens_score, capital_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Capital_roberta spacy: {average_capital_roberta_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_capital_roberta_top1_spacy, _, average_capital_roberta_top1_spacy = create_results_list(
    masked_capital_roberta_original_spacy, capital_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Capital_roberta Spacy: {average_capital_roberta_top1_spacy}')

results_list_capital_roberta_top1, _, average_capital_roberta_top1 = create_results_list(
    masked_capital_roberta_original, capital_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Capital_roberta Normal: {average_capital_roberta_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Capital_roberta Spacy: 0.322
Avg Precision K = 1 of the ranking of Capital_roberta Normal: 0.348


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Capital_roberta Normal: 0.338


Borda Spacy Score
Avg Precision K = 1 of the borda of Capital_roberta spacy: 0.317


Sem ranking
Avg Precision K = 1 of Capital_roberta Spacy: 0.374
Avg Precision K = 1 of Capital_roberta Normal: 0.367


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_capital_roberta_top5_ranking_spacy, _, average_capital_roberta_top5_ranking_spacy = create_results_list(
    ranking_capital_roberta_spacy_tokens, capital_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Capital_roberta Spacy: {average_capital_roberta_top5_ranking_spacy}')

results_list_capital_roberta_top5_ranking_normal, _, average_capital_roberta_top5_ranking_normal = create_results_list(
    ranking_capital_roberta_normal_tokens, capital_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Capital_roberta Normal: {average_capital_roberta_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_capital_roberta_top5_borda_normal, indicies_borda_capital_roberta, average_capital_roberta_top5_borda_normal = create_results_list(
    capital_roberta_borda_tokens_score, capital_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Capital_roberta Normal: {average_capital_roberta_top5_borda_normal}')
print('\n')

results_list_capital_roberta_top5_borda_spacy, indicies_borda_capital_roberta_spacy, average_capital_roberta_top5_borda_spacy = create_results_list(
    capital_roberta_borda_spacy_tokens_score, capital_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Capital_roberta spacy: {average_capital_roberta_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_capital_roberta_top5_spacy, _, average_capital_roberta_top5_spacy = create_results_list(
    masked_capital_roberta_original_spacy, capital_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Capital_roberta Spacy: {average_capital_roberta_top5_spacy}')

results_list_capital_roberta_top5, _, average_capital_roberta_top5 = create_results_list(
    masked_capital_roberta_original, capital_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Capital_roberta Normal: {average_capital_roberta_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Capital_roberta Spacy: 0.46
Avg Precision K = 5 of the ranking of Capital_roberta Normal: 0.518


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Capital_roberta Normal: 0.517


Borda Spacy Score
Avg Precision K = 5 of the borda of Capital_roberta spacy: 0.46


Sem ranking
Avg Precision K = 5 of Capital_roberta Spacy: 0.472
Avg Precision K = 5 of Capital_roberta Normal: 0.498


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_capital_roberta_top10_ranking_spacy, _, average_capital_roberta_top10_ranking_spacy = create_results_list(
    ranking_capital_roberta_spacy_tokens, capital_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Capital_roberta Spacy: {average_capital_roberta_top10_ranking_spacy}')

results_list_capital_roberta_top10_ranking_normal, _, average_capital_roberta_top10_ranking_normal = create_results_list(
    ranking_capital_roberta_normal_tokens, capital_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Capital_roberta Normal: {average_capital_roberta_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_capital_roberta_top10_borda_normal, indicies_borda_capital_roberta, average_capital_roberta_top10_borda_normal = create_results_list(
    capital_roberta_borda_tokens_score, capital_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Capital_roberta Normal: {average_capital_roberta_top10_borda_normal}')
print('\n')

results_list_capital_roberta_top10_borda_spacy, indicies_borda_capital_roberta_spacy, average_capital_roberta_top10_borda_spacy = create_results_list(
    capital_roberta_borda_spacy_tokens_score, capital_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Capital_roberta spacy: {average_capital_roberta_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_capital_roberta_top10_spacy, _, average_capital_roberta_top10_spacy = create_results_list(
    masked_capital_roberta_original_spacy, capital_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Capital_roberta Spacy: {average_capital_roberta_top10_spacy}')

results_list_capital_roberta_top10, _, average_capital_roberta_top10 = create_results_list(
    masked_capital_roberta_original, capital_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Capital_roberta Normal: {average_capital_roberta_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Capital_roberta Spacy: 0.485
Avg Precision K = 10 of the ranking of Capital_roberta Normal: 0.548


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Capital_roberta Normal: 0.55


Borda Spacy Score
Avg Precision K = 10 of the borda of Capital_roberta spacy: 0.483


Sem ranking
Avg Precision K = 10 of Capital_roberta Spacy: 0.481
Avg Precision K = 10 of Capital_roberta Normal: 0.519


#### Citizen

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_citizen_roberta_top1_ranking_spacy, _, average_citizen_roberta_top1_ranking_spacy = create_results_list(
    ranking_citizen_bert_spacy_tokens, citizen_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of citizen_roberta Spacy: {average_citizen_roberta_top1_ranking_spacy}')

results_list_citizen_roberta_top1_ranking_normal, _, average_citizen_roberta_top1_ranking_normal = create_results_list(
    ranking_citizen_roberta_normal_tokens, citizen_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of citizen_roberta Normal: {average_citizen_roberta_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_roberta_top1_borda_normal, indicies_borda_citizen_roberta, average_citizen_roberta_top1_borda_normal = create_results_list(
    citizen_roberta_borda_tokens_score, citizen_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of citizen_roberta Normal: {average_citizen_roberta_top1_borda_normal}')
print('\n')

results_list_citizen_roberta_top1_borda_spacy, indicies_borda_citizen_roberta_spacy, average_citizen_roberta_top1_borda_spacy = create_results_list(
    citizen_roberta_borda_spacy_tokens_score, citizen_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of citizen_roberta spacy: {average_citizen_roberta_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_roberta_top1_spacy, _, average_citizen_roberta_top1_spacy = create_results_list(
    masked_citizen_roberta_original_spacy, citizen_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of citizen_roberta Spacy: {average_citizen_roberta_top1_spacy}')

results_list_citizen_roberta_top1, _, average_citizen_roberta_top1 = create_results_list(
    masked_citizen_roberta_original, citizen_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of citizen_roberta Normal: {average_citizen_roberta_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of citizen_roberta Spacy: 0.445
Avg Precision K = 1 of the ranking of citizen_roberta Normal: 0.62


Borda Count
Borda Score
Avg Precision K = 1 of the borda of citizen_roberta Normal: 0.615


Borda Spacy Score
Avg Precision K = 1 of the borda of citizen_roberta spacy: 0.442


Sem ranking
Avg Precision K = 1 of citizen_roberta Spacy: 0.409
Avg Precision K = 1 of citizen_roberta Normal: 0.567


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_citizen_roberta_top5_ranking_spacy, _, average_citizen_roberta_top5_ranking_spacy = create_results_list(
    ranking_citizen_bert_spacy_tokens, citizen_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of citizen_roberta Spacy: {average_citizen_roberta_top5_ranking_spacy}')

results_list_citizen_roberta_top5_ranking_normal, _, average_citizen_roberta_top5_ranking_normal = create_results_list(
    ranking_citizen_roberta_normal_tokens, citizen_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of citizen_roberta Normal: {average_citizen_roberta_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_roberta_top5_borda_normal, indicies_borda_citizen_roberta, average_citizen_roberta_top5_borda_normal = create_results_list(
    citizen_roberta_borda_tokens_score, citizen_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of citizen_roberta Normal: {average_citizen_roberta_top5_borda_normal}')
print('\n')

results_list_citizen_roberta_top5_borda_spacy, indicies_borda_citizen_roberta_spacy, average_citizen_roberta_top5_borda_spacy = create_results_list(
    citizen_roberta_borda_spacy_tokens_score, citizen_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of citizen_roberta spacy: {average_citizen_roberta_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_roberta_top5_spacy, _, average_citizen_roberta_top5_spacy = create_results_list(
    masked_citizen_roberta_original_spacy, citizen_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of citizen_roberta Spacy: {average_citizen_roberta_top5_spacy}')

results_list_citizen_roberta_top5, _, average_citizen_roberta_top5 = create_results_list(
    masked_citizen_roberta_original, citizen_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of citizen_roberta Normal: {average_citizen_roberta_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of citizen_roberta Spacy: 0.544
Avg Precision K = 5 of the ranking of citizen_roberta Normal: 0.822


Borda Count
Borda Score
Avg Precision K = 5 of the borda of citizen_roberta Normal: 0.822


Borda Spacy Score
Avg Precision K = 5 of the borda of citizen_roberta spacy: 0.542


Sem ranking
Avg Precision K = 5 of citizen_roberta Spacy: 0.472
Avg Precision K = 5 of citizen_roberta Normal: 0.709


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_citizen_roberta_top10_ranking_spacy, _, average_citizen_roberta_top10_ranking_spacy = create_results_list(
    ranking_citizen_bert_spacy_tokens, citizen_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of citizen_roberta Spacy: {average_citizen_roberta_top10_ranking_spacy}')

results_list_citizen_roberta_top10_ranking_normal, _, average_citizen_roberta_top10_ranking_normal = create_results_list(
    ranking_citizen_roberta_normal_tokens, citizen_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of citizen_roberta Normal: {average_citizen_roberta_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_roberta_top10_borda_normal, indicies_borda_citizen_roberta, average_citizen_roberta_top10_borda_normal = create_results_list(
    citizen_roberta_borda_tokens_score, citizen_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of citizen_roberta Normal: {average_citizen_roberta_top10_borda_normal}')
print('\n')

results_list_citizen_roberta_top10_borda_spacy, indicies_borda_citizen_roberta_spacy, average_citizen_roberta_top10_borda_spacy = create_results_list(
    citizen_roberta_borda_spacy_tokens_score, citizen_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of citizen_roberta spacy: {average_citizen_roberta_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_roberta_top10_spacy, _, average_citizen_roberta_top10_spacy = create_results_list(
    masked_citizen_roberta_original_spacy, citizen_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of citizen_roberta Spacy: {average_citizen_roberta_top10_spacy}')

results_list_citizen_roberta_top10, _, average_citizen_roberta_top10 = create_results_list(
    masked_citizen_roberta_original, citizen_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of citizen_roberta Normal: {average_citizen_roberta_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of citizen_roberta Spacy: 0.562
Avg Precision K = 10 of the ranking of citizen_roberta Normal: 0.871


Borda Count
Borda Score
Avg Precision K = 10 of the borda of citizen_roberta Normal: 0.872


Borda Spacy Score
Avg Precision K = 10 of the borda of citizen_roberta spacy: 0.556


Sem ranking
Avg Precision K = 10 of citizen_roberta Spacy: 0.472
Avg Precision K = 10 of citizen_roberta Normal: 0.763


#### Language

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_language_roberta_top1_ranking_spacy, _, average_language_roberta_top1_ranking_spacy = create_results_list(
    ranking_language_bert_spacy_tokens, language_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of language_roberta Spacy: {average_language_roberta_top1_ranking_spacy}')

results_list_language_roberta_top1_ranking_normal, _, average_language_roberta_top1_ranking_normal = create_results_list(
    ranking_citizen_roberta_normal_tokens, language_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of language_roberta Normal: {average_language_roberta_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_roberta_top1_borda_normal, indicies_borda_language_roberta, average_language_roberta_top1_borda_normal = create_results_list(
    language_roberta_borda_tokens_score, language_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of language_roberta Normal: {average_language_roberta_top1_borda_normal}')
print('\n')

results_list_language_roberta_top1_borda_spacy, indicies_borda_language_roberta_spacy, average_language_roberta_top1_borda_spacy = create_results_list(
    language_roberta_borda_spacy_tokens_score, language_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of language_roberta spacy: {average_language_roberta_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_roberta_top1_spacy, _, average_language_roberta_top1_spacy = create_results_list(
    masked_language_roberta_original_spacy, language_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language_roberta Spacy: {average_language_roberta_top1_spacy}')

results_list_language_roberta_top1, _, average_language_roberta_top1 = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language_roberta Normal: {average_language_roberta_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of language_roberta Spacy: 0.456
Avg Precision K = 1 of the ranking of language_roberta Normal: 0.004


Borda Count
Borda Score
Avg Precision K = 1 of the borda of language_roberta Normal: 0.604


Borda Spacy Score
Avg Precision K = 1 of the borda of language_roberta spacy: 0.427


Sem ranking
Avg Precision K = 1 of language_roberta Spacy: 0.614
Avg Precision K = 1 of language_roberta Normal: 0.639


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_language_roberta_top5_ranking_spacy, _, average_language_roberta_top5_ranking_spacy = create_results_list(
    ranking_language_bert_spacy_tokens, language_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of language_roberta Spacy: {average_language_roberta_top5_ranking_spacy}')

results_list_language_roberta_top5_ranking_normal, _, average_language_roberta_top5_ranking_normal = create_results_list(
    ranking_language_roberta_normal_tokens, language_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of language_roberta Normal: {average_language_roberta_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_roberta_top5_borda_normal, indicies_borda_language_roberta, average_language_roberta_top5_borda_normal = create_results_list(
    language_roberta_borda_tokens_score, language_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of language_roberta Normal: {average_language_roberta_top5_borda_normal}')
print('\n')

results_list_language_roberta_top5_borda_spacy, indicies_borda_language_roberta_spacy, average_language_roberta_top5_borda_spacy = create_results_list(
    language_roberta_borda_spacy_tokens_score, language_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of language_roberta spacy: {average_language_roberta_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_roberta_top5_spacy, _, average_language_roberta_top5_spacy = create_results_list(
    masked_language_roberta_original_spacy, language_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language_roberta Spacy: {average_language_roberta_top5_spacy}')

results_list_language_roberta_top5, _, average_language_roberta_top5 = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language_roberta Normal: {average_language_roberta_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of language_roberta Spacy: 0.611
Avg Precision K = 5 of the ranking of language_roberta Normal: 0.826


Borda Count
Borda Score
Avg Precision K = 5 of the borda of language_roberta Normal: 0.827


Borda Spacy Score
Avg Precision K = 5 of the borda of language_roberta spacy: 0.567


Sem ranking
Avg Precision K = 5 of language_roberta Spacy: 0.702
Avg Precision K = 5 of language_roberta Normal: 0.785


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_language_roberta_top10_ranking_spacy, _, average_language_roberta_top10_ranking_spacy = create_results_list(
    ranking_language_bert_spacy_tokens, language_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of language_roberta Spacy: {average_language_roberta_top10_ranking_spacy}')

results_list_language_roberta_top10_ranking_normal, _, average_language_roberta_top10_ranking_normal = create_results_list(
    ranking_language_roberta_normal_tokens, language_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of language_roberta Normal: {average_language_roberta_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_roberta_top10_borda_normal, indicies_borda_language_roberta, average_language_roberta_top10_borda_normal = create_results_list(
    language_roberta_borda_tokens_score, language_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of language_roberta Normal: {average_language_roberta_top10_borda_normal}')
print('\n')

results_list_language_roberta_top10_borda_spacy, indicies_borda_language_roberta_spacy, average_language_roberta_top10_borda_spacy = create_results_list(
    language_roberta_borda_spacy_tokens_score, language_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of language_roberta spacy: {average_language_roberta_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_roberta_top10_spacy, _, average_language_roberta_top10_spacy = create_results_list(
    masked_language_roberta_original_spacy, language_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language_roberta Spacy: {average_language_roberta_top10_spacy}')

results_list_language_roberta_top10, _, average_language_roberta_top10 = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language_roberta Normal: {average_language_roberta_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of language_roberta Spacy: 0.643
Avg Precision K = 10 of the ranking of language_roberta Normal: 0.854


Borda Count
Borda Score
Avg Precision K = 10 of the borda of language_roberta Normal: 0.853


Borda Spacy Score
Avg Precision K = 10 of the borda of language_roberta spacy: 0.593


Sem ranking
Avg Precision K = 10 of language_roberta Spacy: 0.703
Avg Precision K = 10 of language_roberta Normal: 0.831


## Electra

#### BornIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_bornIn_electra_top1_ranking_spacy, _, average_bornIn_electra_top1_ranking_spacy = create_results_list(
    ranking_bornIn_electra_spacy_tokens, bornIn_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Spacy: {average_bornIn_electra_top1_ranking_spacy}')

results_list_bornIn_electra_top1_ranking_normal, _, average_bornIn_electra_top1_ranking_normal = create_results_list(
    ranking_bornIn_electra_normal_tokens, bornIn_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Normal: {average_bornIn_electra_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_bornIn_electra_top1_borda_normal, indicies_borda_bornIn_electra, average_bornIn_electra_top1_borda_normal = create_results_list(
    bornIn_electra_borda_tokens_score, bornIn_electra.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Born In Normal: {average_bornIn_electra_top1_borda_normal}')
print('\n')

results_list_bornIn_electra_top1_borda_spacy, indicies_borda_bornIn_electra_spacy, average_bornIn_electra_top1_borda_spacy = create_results_list(
    bornIn_electra_borda_spacy_tokens_score, bornIn_electra.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Born In spacy: {average_bornIn_electra_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_bornIn_electra_top1_spacy, _, average_bornIn_electra_top1_spacy = create_results_list(
    masked_bornIn_electra_original_spacy, bornIn_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Spacy: {average_bornIn_electra_top1_spacy}')

results_list_bornIn_electra_top1, _, average_bornIn_electra_top1 = create_results_list(
    masked_bornIn_electra_original, bornIn_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Normal: {average_bornIn_electra_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Born In Spacy: 0.299
Avg Precision K = 1 of the ranking of Born In Normal: 0.309


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Born In Normal: 0.31


Borda Spacy Score
Avg Precision K = 1 of the borda of Born In spacy: 0.31


Sem ranking
Avg Precision K = 1 of Born In Spacy: 0.295
Avg Precision K = 1 of Born In Normal: 0.284


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_bornIn_electra_top5_ranking_spacy, _, average_bornIn_electra_top5_ranking_spacy = create_results_list(
    ranking_bornIn_electra_spacy_tokens, bornIn_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Spacy: {average_bornIn_electra_top5_ranking_spacy}')

results_list_bornIn_electra_top5_ranking_normal, _, average_bornIn_electra_top5_ranking_normal = create_results_list(
    ranking_bornIn_electra_normal_tokens, bornIn_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Normal: {average_bornIn_electra_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_bornIn_electra_top5_borda_normal, indicies_borda_bornIn_electra, average_bornIn_electra_top5_borda_normal = create_results_list(
    bornIn_electra_borda_tokens_score, bornIn_electra.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Born In Normal: {average_bornIn_electra_top5_borda_normal}')
print('\n')

results_list_bornIn_electra_top5_borda_spacy, indicies_borda_bornIn_electra_spacy, average_bornIn_electra_top5_borda_spacy = create_results_list(
    bornIn_electra_borda_spacy_tokens_score, bornIn_electra.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Born In spacy: {average_bornIn_electra_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_bornIn_electra_top5_spacy, _, average_bornIn_electra_top5_spacy = create_results_list(
    masked_bornIn_electra_original_spacy, bornIn_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Spacy: {average_bornIn_electra_top5_spacy}')

results_list_bornIn_electra_top5, _, average_bornIn_electra_top5 = create_results_list(
    masked_bornIn_electra_original, bornIn_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Normal: {average_bornIn_electra_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Born In Spacy: 0.577
Avg Precision K = 5 of the ranking of Born In Normal: 0.579


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Born In Normal: 0.579


Borda Spacy Score
Avg Precision K = 5 of the borda of Born In spacy: 0.576


Sem ranking
Avg Precision K = 5 of Born In Spacy: 0.513
Avg Precision K = 5 of Born In Normal: 0.505


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_bornIn_electra_top10_ranking_spacy, _, average_bornIn_electra_top10_ranking_spacy = create_results_list(
    ranking_bornIn_electra_spacy_tokens, bornIn_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Spacy: {average_bornIn_electra_top10_ranking_spacy}')

results_list_bornIn_electra_top10_ranking_normal, _, average_bornIn_electra_top10_ranking_normal = create_results_list(
    ranking_bornIn_electra_normal_tokens, bornIn_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Normal: {average_bornIn_electra_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_bornIn_electra_top10_borda_normal, indicies_borda_bornIn_electra, average_bornIn_electra_top10_borda_normal = create_results_list(
    bornIn_electra_borda_tokens_score, bornIn_electra.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Born In Normal: {average_bornIn_electra_top10_borda_normal}')
print('\n')

results_list_bornIn_electra_top10_borda_spacy, indicies_borda_bornIn_electra_spacy, average_bornIn_electra_top10_borda_spacy = create_results_list(
    bornIn_electra_borda_spacy_tokens_score, bornIn_electra.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Born In spacy: {average_bornIn_electra_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_bornIn_electra_top10_spacy, _, average_bornIn_electra_top10_spacy = create_results_list(
    masked_bornIn_electra_original_spacy, bornIn_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Spacy: {average_bornIn_electra_top10_spacy}')

results_list_bornIn_electra_top10, _, average_bornIn_electra_top10 = create_results_list(
    masked_bornIn_electra_original, bornIn_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Normal: {average_bornIn_electra_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Born In Spacy: 0.653
Avg Precision K = 10 of the ranking of Born In Normal: 0.66


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Born In Normal: 0.657


Borda Spacy Score
Avg Precision K = 10 of the borda of Born In spacy: 0.656


Sem ranking
Avg Precision K = 10 of Born In Spacy: 0.586
Avg Precision K = 10 of Born In Normal: 0.589


#### DiedIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_diedIn_electra_top1_ranking_spacy, _, average_diedIn_electra_top1_ranking_spacy = create_results_list(
    ranking_diedIn_electra_spacy_tokens, diedIn_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy: {average_diedIn_electra_top1_ranking_spacy}')

results_list_diedIn_electra_top1_ranking_normal, _, average_diedIn_electra_top1_ranking_normal = create_results_list(
    ranking_diedIn_electra_normal_tokens, diedIn_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal: {average_diedIn_electra_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_diedIn_electra_top1_borda_normal, indicies_borda_diedIn_electra, average_diedIn_electra_top1_borda_normal = create_results_list(
    diedIn_electra_borda_tokens_score, diedIn_electra.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal: {average_diedIn_electra_top1_borda_normal}')
print('\n')

results_list_diedIn_electra_top1_borda_spacy, indicies_borda_diedIn_electra_spacy, average_diedIn_electra_top1_borda_spacy = create_results_list(
    diedIn_electra_borda_spacy_tokens_score, diedIn_electra.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In spacy: {average_diedIn_electra_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_diedIn_electra_top1_spacy, _, average_diedIn_electra_top1_spacy = create_results_list(
    masked_diedIn_electra_original_spacy, diedIn_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy: {average_diedIn_electra_top1_spacy}')

results_list_diedIn_electra_top1, _, average_diedIn_electra_top1 = create_results_list(
    masked_diedIn_electra_original, diedIn_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_diedIn_electra_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy: 0.364
Avg Precision K = 1 of the ranking of died In Normal: 0.355


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal: 0.351


Borda Spacy Score
Avg Precision K = 1 of the borda of died In spacy: 0.359


Sem ranking
Avg Precision K = 1 of died In Spacy: 0.327
Avg Precision K = 1 of died In Normal: 0.3


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_diedIn_electra_top5_ranking_spacy, _, average_diedIn_electra_top5_ranking_spacy = create_results_list(
    ranking_diedIn_electra_spacy_tokens, diedIn_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy: {average_diedIn_electra_top5_ranking_spacy}')

results_list_diedIn_electra_top5_ranking_normal, _, average_diedIn_electra_top5_ranking_normal = create_results_list(
    ranking_diedIn_electra_normal_tokens, diedIn_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal: {average_diedIn_electra_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_diedIn_electra_top5_borda_normal, indicies_borda_diedIn_electra, average_diedIn_electra_top5_borda_normal = create_results_list(
    diedIn_electra_borda_tokens_score, diedIn_electra.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal: {average_diedIn_electra_top5_borda_normal}')
print('\n')

results_list_diedIn_electra_top5_borda_spacy, indicies_borda_diedIn_electra_spacy, average_diedIn_electra_top5_borda_spacy = create_results_list(
    diedIn_electra_borda_spacy_tokens_score, diedIn_electra.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In spacy: {average_diedIn_electra_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_diedIn_electra_top5_spacy, _, average_diedIn_electra_top5_spacy = create_results_list(
    masked_diedIn_electra_original_spacy, diedIn_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy: {average_diedIn_electra_top5_spacy}')

results_list_diedIn_electra_top5, _, average_diedIn_electra_top5 = create_results_list(
    masked_diedIn_electra_original, diedIn_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_diedIn_electra_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy: 0.634
Avg Precision K = 5 of the ranking of died In Normal: 0.629


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal: 0.624


Borda Spacy Score
Avg Precision K = 5 of the borda of died In spacy: 0.629


Sem ranking
Avg Precision K = 5 of died In Spacy: 0.58
Avg Precision K = 5 of died In Normal: 0.57


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_diedIn_electra_top10_ranking_spacy, _, average_diedIn_electra_top10_ranking_spacy = create_results_list(
    ranking_diedIn_electra_spacy_tokens, diedIn_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy: {average_diedIn_electra_top10_ranking_spacy}')

results_list_diedIn_electra_top10_ranking_normal, _, average_diedIn_electra_top10_ranking_normal = create_results_list(
    ranking_diedIn_electra_normal_tokens, diedIn_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal: {average_diedIn_electra_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_diedIn_electra_top10_borda_normal, indicies_borda_diedIn_electra, average_diedIn_electra_top10_borda_normal = create_results_list(
    diedIn_electra_borda_tokens_score, diedIn_electra.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal: {average_diedIn_electra_top10_borda_normal}')
print('\n')

results_list_diedIn_electra_top10_borda_spacy, indicies_borda_diedIn_electra_spacy, average_diedIn_electra_top10_borda_spacy = create_results_list(
    diedIn_electra_borda_spacy_tokens_score, diedIn_electra.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In spacy: {average_diedIn_electra_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_diedIn_electra_top10_spacy, _, average_diedIn_electra_top10_spacy = create_results_list(
    masked_diedIn_electra_original_spacy, diedIn_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy: {average_diedIn_electra_top10_spacy}')

results_list_diedIn_electra_top10, _, average_diedIn_electra_top10 = create_results_list(
    masked_diedIn_electra_original, diedIn_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_diedIn_electra_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy: 0.706
Avg Precision K = 10 of the ranking of died In Normal: 0.729


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal: 0.729


Borda Spacy Score
Avg Precision K = 10 of the borda of died In spacy: 0.712


Sem ranking
Avg Precision K = 10 of died In Spacy: 0.631
Avg Precision K = 10 of died In Normal: 0.652


#### Worksfor

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_worksfor_electra_top1_ranking_spacy, _, average_worksfor_electra_top1_ranking_spacy = create_results_list(
    ranking_worksfor_electra_spacy_tokens, worksfor_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Works For Spacy: {average_worksfor_electra_top1_ranking_spacy}')

results_list_worksfor_electra_top1_ranking_normal, _, average_worksfor_electra_top1_ranking_normal = create_results_list(
    ranking_worksfor_electra_normal_tokens, worksfor_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Works For Normal: {average_worksfor_electra_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_worksfor_electra_top1_borda_normal, indicies_borda_worksfor_electra, average_worksfor_electra_top1_borda_normal = create_results_list(
    worksfor_electra_borda_tokens_score, worksfor_electra.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Works For Normal: {average_worksfor_electra_top1_borda_normal}')
print('\n')

results_list_worksfor_electra_top1_borda_spacy, indicies_borda_worksfor_electra_spacy, average_worksfor_electra_top1_borda_spacy = create_results_list(
    worksfor_electra_borda_spacy_tokens_score, worksfor_electra.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Works For spacy: {average_worksfor_electra_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_worksfor_electra_top1_spacy, _, average_worksfor_electra_top1_spacy = create_results_list(
    masked_worksfor_electra_original_spacy, worksfor_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Works For Spacy: {average_worksfor_electra_top1_spacy}')

results_list_worksfor_electra_top1, _, average_worksfor_electra_top1 = create_results_list(
    masked_worksfor_electra_original, worksfor_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Works For Normal: {average_worksfor_electra_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Works For Spacy: 0.491
Avg Precision K = 1 of the ranking of Works For Normal: 0.404


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Works For Normal: 0.393


Borda Spacy Score
Avg Precision K = 1 of the borda of Works For spacy: 0.491


Sem ranking
Avg Precision K = 1 of Works For Spacy: 0.464
Avg Precision K = 1 of Works For Normal: 0.391


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_worksfor_electra_top5_ranking_spacy, _, average_worksfor_electra_top5_ranking_spacy = create_results_list(
    ranking_worksfor_electra_spacy_tokens, worksfor_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Works For Spacy: {average_worksfor_electra_top5_ranking_spacy}')

results_list_worksfor_electra_top5_ranking_normal, _, average_worksfor_electra_top5_ranking_normal = create_results_list(
    ranking_worksfor_electra_normal_tokens, worksfor_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Works For Normal: {average_worksfor_electra_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_worksfor_electra_top5_borda_normal, indicies_borda_worksfor_electra, average_worksfor_electra_top5_borda_normal = create_results_list(
    worksfor_electra_borda_tokens_score, worksfor_electra.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Works For Normal: {average_worksfor_electra_top5_borda_normal}')
print('\n')

results_list_worksfor_electra_top5_borda_spacy, indicies_borda_worksfor_electra_spacy, average_worksfor_electra_top5_borda_spacy = create_results_list(
    worksfor_electra_borda_spacy_tokens_score, worksfor_electra.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Works For spacy: {average_worksfor_electra_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_worksfor_electra_top5_spacy, _, average_worksfor_electra_top5_spacy = create_results_list(
    masked_worksfor_electra_original_spacy, worksfor_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Works For Spacy: {average_worksfor_electra_top5_spacy}')

results_list_worksfor_electra_top5, _, average_worksfor_electra_top5 = create_results_list(
    masked_worksfor_electra_original, worksfor_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Works For Normal: {average_worksfor_electra_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Works For Spacy: 0.715
Avg Precision K = 5 of the ranking of Works For Normal: 0.665


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Works For Normal: 0.652


Borda Spacy Score
Avg Precision K = 5 of the borda of Works For spacy: 0.712


Sem ranking
Avg Precision K = 5 of Works For Spacy: 0.686
Avg Precision K = 5 of Works For Normal: 0.628


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_worksfor_electra_top10_ranking_spacy, _, average_worksfor_electra_top10_ranking_spacy = create_results_list(
    ranking_worksfor_electra_spacy_tokens, worksfor_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Works For Spacy: {average_worksfor_electra_top10_ranking_spacy}')

results_list_worksfor_electra_top10_ranking_normal, _, average_worksfor_electra_top10_ranking_normal = create_results_list(
    ranking_worksfor_electra_normal_tokens, worksfor_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Works For Normal: {average_worksfor_electra_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_worksfor_electra_top10_borda_normal, indicies_borda_worksfor_electra, average_worksfor_electra_top10_borda_normal = create_results_list(
    worksfor_electra_borda_tokens_score, worksfor_electra.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Works For Normal: {average_worksfor_electra_top10_borda_normal}')
print('\n')

results_list_worksfor_electra_top10_borda_spacy, indicies_borda_worksfor_electra_spacy, average_worksfor_electra_top10_borda_spacy = create_results_list(
    worksfor_electra_borda_spacy_tokens_score, worksfor_electra.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Works For spacy: {average_worksfor_electra_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_worksfor_electra_top10_spacy, _, average_worksfor_electra_top10_spacy = create_results_list(
    masked_worksfor_electra_original_spacy, worksfor_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Works For Spacy: {average_worksfor_electra_top10_spacy}')

results_list_worksfor_electra_top10, _, average_worksfor_electra_top10 = create_results_list(
    masked_worksfor_electra_original, worksfor_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Works For Normal: {average_worksfor_electra_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Works For Spacy: 0.773
Avg Precision K = 10 of the ranking of Works For Normal: 0.749


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Works For Normal: 0.741


Borda Spacy Score
Avg Precision K = 10 of the borda of Works For spacy: 0.77


Sem ranking
Avg Precision K = 10 of Works For Spacy: 0.699
Avg Precision K = 10 of Works For Normal: 0.728


#### Capital

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_capital_electra_top1_ranking_spacy, _, average_capital_electra_top1_ranking_spacy = create_results_list(
    ranking_capital_electra_spacy_tokens, capital_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Capital_electra Spacy: {average_capital_electra_top1_ranking_spacy}')

results_list_capital_electra_top1_ranking_normal, _, average_capital_electra_top1_ranking_normal = create_results_list(
    ranking_capital_electra_normal_tokens, capital_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Capital_electra Normal: {average_capital_electra_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_capital_electra_top1_borda_normal, indicies_borda_capital_electra, average_capital_electra_top1_borda_normal = create_results_list(
    capital_electra_borda_tokens_score, capital_electra.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Capital_electra Normal: {average_capital_electra_top1_borda_normal}')
print('\n')

results_list_capital_electra_top1_borda_spacy, indicies_borda_capital_electra_spacy, average_capital_electra_top1_borda_spacy = create_results_list(
    capital_electra_borda_spacy_tokens_score, capital_electra.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Capital_electra spacy: {average_capital_electra_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_capital_electra_top1_spacy, _, average_capital_electra_top1_spacy = create_results_list(
    masked_capital_electra_original_spacy, capital_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Capital_electra Spacy: {average_capital_electra_top1_spacy}')

results_list_capital_electra_top1, _, average_capital_electra_top1 = create_results_list(
    masked_capital_electra_original, capital_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Capital_electra Normal: {average_capital_electra_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Capital_electra Spacy: 0.352
Avg Precision K = 1 of the ranking of Capital_electra Normal: 0.372


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Capital_electra Normal: 0.369


Borda Spacy Score
Avg Precision K = 1 of the borda of Capital_electra spacy: 0.352


Sem ranking
Avg Precision K = 1 of Capital_electra Spacy: 0.375
Avg Precision K = 1 of Capital_electra Normal: 0.365


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_capital_electra_top5_ranking_spacy, _, average_capital_electra_top5_ranking_spacy = create_results_list(
    ranking_capital_electra_spacy_tokens, capital_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Capital_electra Spacy: {average_capital_electra_top5_ranking_spacy}')

results_list_capital_electra_top5_ranking_normal, _, average_capital_electra_top5_ranking_normal = create_results_list(
    ranking_capital_electra_normal_tokens, capital_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Capital_electra Normal: {average_capital_electra_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_capital_electra_top5_borda_normal, indicies_borda_capital_electra, average_capital_electra_top5_borda_normal = create_results_list(
    capital_electra_borda_tokens_score, capital_electra.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Capital_electra Normal: {average_capital_electra_top5_borda_normal}')
print('\n')

results_list_capital_electra_top5_borda_spacy, indicies_borda_capital_electra_spacy, average_capital_electra_top5_borda_spacy = create_results_list(
    capital_electra_borda_spacy_tokens_score, capital_electra.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Capital_electra spacy: {average_capital_electra_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_capital_electra_top5_spacy, _, average_capital_electra_top5_spacy = create_results_list(
    masked_capital_electra_original_spacy, capital_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Capital_electra Spacy: {average_capital_electra_top5_spacy}')

results_list_capital_electra_top5, _, average_capital_electra_top5 = create_results_list(
    masked_capital_electra_original, capital_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Capital_electra Normal: {average_capital_electra_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Capital_electra Spacy: 0.548
Avg Precision K = 5 of the ranking of Capital_electra Normal: 0.613


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Capital_electra Normal: 0.616


Borda Spacy Score
Avg Precision K = 5 of the borda of Capital_electra spacy: 0.543


Sem ranking
Avg Precision K = 5 of Capital_electra Spacy: 0.54
Avg Precision K = 5 of Capital_electra Normal: 0.554


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_capital_electra_top10_ranking_spacy, _, average_capital_electra_top10_ranking_spacy = create_results_list(
    ranking_capital_electra_spacy_tokens, capital_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Capital_electra Spacy: {average_capital_electra_top10_ranking_spacy}')

results_list_capital_electra_top10_ranking_normal, _, average_capital_electra_top10_ranking_normal = create_results_list(
    ranking_capital_electra_normal_tokens, capital_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Capital_electra Normal: {average_capital_electra_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_capital_electra_top10_borda_normal, indicies_borda_capital_electra, average_capital_electra_top10_borda_normal = create_results_list(
    capital_electra_borda_tokens_score, capital_electra.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Capital_electra Normal: {average_capital_electra_top10_borda_normal}')
print('\n')

results_list_capital_electra_top10_borda_spacy, indicies_borda_capital_electra_spacy, average_capital_electra_top10_borda_spacy = create_results_list(
    capital_electra_borda_spacy_tokens_score, capital_electra.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Capital_electra spacy: {average_capital_electra_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_capital_electra_top10_spacy, _, average_capital_electra_top10_spacy = create_results_list(
    masked_capital_electra_original_spacy, capital_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Capital_electra Spacy: {average_capital_electra_top10_spacy}')

results_list_capital_electra_top10, _, average_capital_electra_top10 = create_results_list(
    masked_capital_electra_original, capital_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Capital_electra Normal: {average_capital_electra_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Capital_electra Spacy: 0.605
Avg Precision K = 10 of the ranking of Capital_electra Normal: 0.681


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Capital_electra Normal: 0.68


Borda Spacy Score
Avg Precision K = 10 of the borda of Capital_electra spacy: 0.606


Sem ranking
Avg Precision K = 10 of Capital_electra Spacy: 0.584
Avg Precision K = 10 of Capital_electra Normal: 0.615


#### Citizen

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_citizen_electra_top1_ranking_spacy, _, average_citizen_electra_top1_ranking_spacy = create_results_list(
    ranking_citizen_electra_spacy_tokens, citizen_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of citizen_electra Spacy: {average_citizen_electra_top1_ranking_spacy}')

results_list_citizen_electra_top1_ranking_normal, _, average_citizen_electra_top1_ranking_normal = create_results_list(
    ranking_citizen_electra_normal_tokens, citizen_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of citizen_electra Normal: {average_citizen_electra_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_electra_top1_borda_normal, indicies_borda_citizen_electra, average_citizen_electra_top1_borda_normal = create_results_list(
    citizen_electra_borda_tokens_score, citizen_electra.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of citizen_electra Normal: {average_citizen_electra_top1_borda_normal}')
print('\n')

results_list_citizen_electra_top1_borda_spacy, indicies_borda_citizen_electra_spacy, average_citizen_electra_top1_borda_spacy = create_results_list(
    citizen_electra_borda_spacy_tokens_score, citizen_electra.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of citizen_electra spacy: {average_citizen_electra_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_electra_top1_spacy, _, average_citizen_electra_top1_spacy = create_results_list(
    masked_citizen_electra_original_spacy, citizen_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of citizen_electra Spacy: {average_citizen_electra_top1_spacy}')

results_list_citizen_electra_top1, _, average_citizen_electra_top1 = create_results_list(
    masked_citizen_electra_original, citizen_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of citizen_electra Normal: {average_citizen_electra_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of citizen_electra Spacy: 0.366
Avg Precision K = 1 of the ranking of citizen_electra Normal: 0.519


Borda Count
Borda Score
Avg Precision K = 1 of the borda of citizen_electra Normal: 0.517


Borda Spacy Score
Avg Precision K = 1 of the borda of citizen_electra spacy: 0.365


Sem ranking
Avg Precision K = 1 of citizen_electra Spacy: 0.327
Avg Precision K = 1 of citizen_electra Normal: 0.483


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_citizen_electra_top5_ranking_spacy, _, average_citizen_electra_top5_ranking_spacy = create_results_list(
    ranking_citizen_electra_spacy_tokens, citizen_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of citizen_electra Spacy: {average_citizen_electra_top5_ranking_spacy}')

results_list_citizen_electra_top5_ranking_normal, _, average_citizen_electra_top5_ranking_normal = create_results_list(
    ranking_citizen_electra_normal_tokens, citizen_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of citizen_electra Normal: {average_citizen_electra_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_electra_top5_borda_normal, indicies_borda_citizen_electra, average_citizen_electra_top5_borda_normal = create_results_list(
    citizen_electra_borda_tokens_score, citizen_electra.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of citizen_electra Normal: {average_citizen_electra_top5_borda_normal}')
print('\n')

results_list_citizen_electra_top5_borda_spacy, indicies_borda_citizen_electra_spacy, average_citizen_electra_top5_borda_spacy = create_results_list(
    citizen_electra_borda_spacy_tokens_score, citizen_electra.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of citizen_electra spacy: {average_citizen_electra_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_electra_top5_spacy, _, average_citizen_electra_top5_spacy = create_results_list(
    masked_citizen_electra_original_spacy, citizen_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of citizen_electra Spacy: {average_citizen_electra_top5_spacy}')

results_list_citizen_electra_top5, _, average_citizen_electra_top5 = create_results_list(
    masked_citizen_electra_original, citizen_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of citizen_electra Normal: {average_citizen_electra_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of citizen_electra Spacy: 0.497
Avg Precision K = 5 of the ranking of citizen_electra Normal: 0.718


Borda Count
Borda Score
Avg Precision K = 5 of the borda of citizen_electra Normal: 0.719


Borda Spacy Score
Avg Precision K = 5 of the borda of citizen_electra spacy: 0.496


Sem ranking
Avg Precision K = 5 of citizen_electra Spacy: 0.396
Avg Precision K = 5 of citizen_electra Normal: 0.602


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_citizen_electra_top10_ranking_spacy, _, average_citizen_electra_top10_ranking_spacy = create_results_list(
    ranking_citizen_electra_spacy_tokens, citizen_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of citizen_electra Spacy: {average_citizen_electra_top10_ranking_spacy}')

results_list_citizen_electra_top10_ranking_normal, _, average_citizen_electra_top10_ranking_normal = create_results_list(
    ranking_citizen_electra_normal_tokens, citizen_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of citizen_electra Normal: {average_citizen_electra_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_electra_top10_borda_normal, indicies_borda_citizen_electra, average_citizen_electra_top10_borda_normal = create_results_list(
    citizen_electra_borda_tokens_score, citizen_electra.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of citizen_electra Normal: {average_citizen_electra_top10_borda_normal}')
print('\n')

results_list_citizen_electra_top10_borda_spacy, indicies_borda_citizen_electra_spacy, average_citizen_electra_top10_borda_spacy = create_results_list(
    citizen_electra_borda_spacy_tokens_score, citizen_electra.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of citizen_electra spacy: {average_citizen_electra_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_electra_top10_spacy, _, average_citizen_electra_top10_spacy = create_results_list(
    masked_citizen_electra_original_spacy, citizen_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of citizen_electra Spacy: {average_citizen_electra_top10_spacy}')

results_list_citizen_electra_top10, _, average_citizen_electra_top10 = create_results_list(
    masked_citizen_electra_original, citizen_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of citizen_electra Normal: {average_citizen_electra_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of citizen_electra Spacy: 0.518
Avg Precision K = 10 of the ranking of citizen_electra Normal: 0.778


Borda Count
Borda Score
Avg Precision K = 10 of the borda of citizen_electra Normal: 0.774


Borda Spacy Score
Avg Precision K = 10 of the borda of citizen_electra spacy: 0.517


Sem ranking
Avg Precision K = 10 of citizen_electra Spacy: 0.401
Avg Precision K = 10 of citizen_electra Normal: 0.662


#### Language

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_language_roberta_top1_ranking_spacy, _, average_language_roberta_top1_ranking_spacy = create_results_list(
    ranking_language_bert_spacy_tokens, language_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of language_roberta Spacy: {average_language_roberta_top1_ranking_spacy}')

results_list_language_roberta_top1_ranking_normal, _, average_language_roberta_top1_ranking_normal = create_results_list(
    ranking_language_roberta_normal_tokens, language_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of language_roberta Normal: {average_language_roberta_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_roberta_top1_borda_normal, indicies_borda_language_roberta, average_language_roberta_top1_borda_normal = create_results_list(
    language_roberta_borda_tokens_score, language_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of language_roberta Normal: {average_language_roberta_top1_borda_normal}')
print('\n')

results_list_language_roberta_top1_borda_spacy, indicies_borda_language_roberta_spacy, average_language_roberta_top1_borda_spacy = create_results_list(
    language_roberta_borda_spacy_tokens_score, language_roberta.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of language_roberta spacy: {average_language_roberta_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_roberta_top1_spacy, _, average_language_roberta_top1_spacy = create_results_list(
    masked_language_roberta_original_spacy, language_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language_roberta Spacy: {average_language_roberta_top1_spacy}')

results_list_language_roberta_top1, _, average_language_roberta_top1 = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language_roberta Normal: {average_language_roberta_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of language_roberta Spacy: 0.456
Avg Precision K = 1 of the ranking of language_roberta Normal: 0.606


Borda Count
Borda Score
Avg Precision K = 1 of the borda of language_roberta Normal: 0.604


Borda Spacy Score
Avg Precision K = 1 of the borda of language_roberta spacy: 0.427


Sem ranking
Avg Precision K = 1 of language_roberta Spacy: 0.614
Avg Precision K = 1 of language_roberta Normal: 0.639


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_language_roberta_top5_ranking_spacy, _, average_language_roberta_top5_ranking_spacy = create_results_list(
    ranking_language_bert_spacy_tokens, language_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of language_roberta Spacy: {average_language_roberta_top5_ranking_spacy}')

results_list_language_roberta_top5_ranking_normal, _, average_language_roberta_top5_ranking_normal = create_results_list(
    ranking_language_roberta_normal_tokens, language_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of language_roberta Normal: {average_language_roberta_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_roberta_top5_borda_normal, indicies_borda_language_roberta, average_language_roberta_top5_borda_normal = create_results_list(
    language_roberta_borda_tokens_score, language_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of language_roberta Normal: {average_language_roberta_top5_borda_normal}')
print('\n')

results_list_language_roberta_top5_borda_spacy, indicies_borda_language_roberta_spacy, average_language_roberta_top5_borda_spacy = create_results_list(
    language_roberta_borda_spacy_tokens_score, language_roberta.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of language_roberta spacy: {average_language_roberta_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_roberta_top5_spacy, _, average_language_roberta_top5_spacy = create_results_list(
    masked_language_roberta_original_spacy, language_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language_roberta Spacy: {average_language_roberta_top5_spacy}')

results_list_language_roberta_top5, _, average_language_roberta_top5 = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language_roberta Normal: {average_language_roberta_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of language_roberta Spacy: 0.611
Avg Precision K = 5 of the ranking of language_roberta Normal: 0.826


Borda Count
Borda Score
Avg Precision K = 5 of the borda of language_roberta Normal: 0.827


Borda Spacy Score
Avg Precision K = 5 of the borda of language_roberta spacy: 0.567


Sem ranking
Avg Precision K = 5 of language_roberta Spacy: 0.702
Avg Precision K = 5 of language_roberta Normal: 0.785


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_language_roberta_top10_ranking_spacy, _, average_language_roberta_top10_ranking_spacy = create_results_list(
    ranking_language_bert_spacy_tokens, language_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of language_roberta Spacy: {average_language_roberta_top10_ranking_spacy}')

results_list_language_roberta_top10_ranking_normal, _, average_language_roberta_top10_ranking_normal = create_results_list(
    ranking_language_roberta_normal_tokens, language_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of language_roberta Normal: {average_language_roberta_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_roberta_top10_borda_normal, indicies_borda_language_roberta, average_language_roberta_top10_borda_normal = create_results_list(
    language_roberta_borda_tokens_score, language_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of language_roberta Normal: {average_language_roberta_top10_borda_normal}')
print('\n')

results_list_language_roberta_top10_borda_spacy, indicies_borda_language_roberta_spacy, average_language_roberta_top10_borda_spacy = create_results_list(
    language_roberta_borda_spacy_tokens_score, language_roberta.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of language_roberta spacy: {average_language_roberta_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_roberta_top10_spacy, _, average_language_roberta_top10_spacy = create_results_list(
    masked_language_roberta_original_spacy, language_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language_roberta Spacy: {average_language_roberta_top10_spacy}')

results_list_language_roberta_top10, _, average_language_roberta_top10 = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language_roberta Normal: {average_language_roberta_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of language_roberta Spacy: 0.643
Avg Precision K = 10 of the ranking of language_roberta Normal: 0.854


Borda Count
Borda Score
Avg Precision K = 10 of the borda of language_roberta Normal: 0.853


Borda Spacy Score
Avg Precision K = 10 of the borda of language_roberta spacy: 0.593


Sem ranking
Avg Precision K = 10 of language_roberta Spacy: 0.703
Avg Precision K = 10 of language_roberta Normal: 0.831


#### Citizen

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_citizen_electra_top1_ranking_spacy, _, average_citizen_electra_top1_ranking_spacy = create_results_list(
    ranking_citizen_electra_spacy_tokens, citizen_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of citizen_electra Spacy: {average_citizen_electra_top1_ranking_spacy}')

results_list_citizen_electra_top1_ranking_normal, _, average_citizen_electra_top1_ranking_normal = create_results_list(
    ranking_citizen_electra_normal_tokens, citizen_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of citizen_electra Normal: {average_citizen_electra_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_electra_top1_borda_normal, indicies_borda_citizen_electra, average_citizen_electra_top1_borda_normal = create_results_list(
    citizen_electra_borda_tokens_score, citizen_electra.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of citizen_electra Normal: {average_citizen_electra_top1_borda_normal}')
print('\n')

results_list_citizen_electra_top1_borda_spacy, indicies_borda_citizen_electra_spacy, average_citizen_electra_top1_borda_spacy = create_results_list(
    citizen_electra_borda_spacy_tokens_score, citizen_electra.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of citizen_electra spacy: {average_citizen_electra_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_electra_top1_spacy, _, average_citizen_electra_top1_spacy = create_results_list(
    masked_citizen_electra_original_spacy, citizen_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of citizen_electra Spacy: {average_citizen_electra_top1_spacy}')

results_list_citizen_electra_top1, _, average_citizen_electra_top1 = create_results_list(
    masked_citizen_electra_original, citizen_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of citizen_electra Normal: {average_citizen_electra_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of citizen_electra Spacy: 0.366
Avg Precision K = 1 of the ranking of citizen_electra Normal: 0.519


Borda Count
Borda Score
Avg Precision K = 1 of the borda of citizen_electra Normal: 0.517


Borda Spacy Score
Avg Precision K = 1 of the borda of citizen_electra spacy: 0.365


Sem ranking
Avg Precision K = 1 of citizen_electra Spacy: 0.327
Avg Precision K = 1 of citizen_electra Normal: 0.483


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_citizen_electra_top5_ranking_spacy, _, average_citizen_electra_top5_ranking_spacy = create_results_list(
    ranking_citizen_electra_spacy_tokens, citizen_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of citizen_electra Spacy: {average_citizen_electra_top5_ranking_spacy}')

results_list_citizen_electra_top5_ranking_normal, _, average_citizen_electra_top5_ranking_normal = create_results_list(
    ranking_citizen_electra_normal_tokens, citizen_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of citizen_electra Normal: {average_citizen_electra_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_electra_top5_borda_normal, indicies_borda_citizen_electra, average_citizen_electra_top5_borda_normal = create_results_list(
    citizen_electra_borda_tokens_score, citizen_electra.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of citizen_electra Normal: {average_citizen_electra_top5_borda_normal}')
print('\n')

results_list_citizen_electra_top5_borda_spacy, indicies_borda_citizen_electra_spacy, average_citizen_electra_top5_borda_spacy = create_results_list(
    citizen_electra_borda_spacy_tokens_score, citizen_electra.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of citizen_electra spacy: {average_citizen_electra_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_electra_top5_spacy, _, average_citizen_electra_top5_spacy = create_results_list(
    masked_citizen_electra_original_spacy, citizen_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of citizen_electra Spacy: {average_citizen_electra_top5_spacy}')

results_list_citizen_electra_top5, _, average_citizen_electra_top5 = create_results_list(
    masked_citizen_electra_original, citizen_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of citizen_electra Normal: {average_citizen_electra_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of citizen_electra Spacy: 0.497
Avg Precision K = 5 of the ranking of citizen_electra Normal: 0.718


Borda Count
Borda Score
Avg Precision K = 5 of the borda of citizen_electra Normal: 0.719


Borda Spacy Score
Avg Precision K = 5 of the borda of citizen_electra spacy: 0.496


Sem ranking
Avg Precision K = 5 of citizen_electra Spacy: 0.396
Avg Precision K = 5 of citizen_electra Normal: 0.602


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_citizen_electra_top10_ranking_spacy, _, average_citizen_electra_top10_ranking_spacy = create_results_list(
    ranking_citizen_electra_spacy_tokens, citizen_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of citizen_electra Spacy: {average_citizen_electra_top10_ranking_spacy}')

results_list_citizen_electra_top10_ranking_normal, _, average_citizen_electra_top10_ranking_normal = create_results_list(
    ranking_citizen_electra_normal_tokens, citizen_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of citizen_electra Normal: {average_citizen_electra_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_electra_top10_borda_normal, indicies_borda_citizen_electra, average_citizen_electra_top10_borda_normal = create_results_list(
    citizen_electra_borda_tokens_score, citizen_electra.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of citizen_electra Normal: {average_citizen_electra_top10_borda_normal}')
print('\n')

results_list_citizen_electra_top10_borda_spacy, indicies_borda_citizen_electra_spacy, average_citizen_electra_top10_borda_spacy = create_results_list(
    citizen_electra_borda_spacy_tokens_score, citizen_electra.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of citizen_electra spacy: {average_citizen_electra_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_electra_top10_spacy, _, average_citizen_electra_top10_spacy = create_results_list(
    masked_citizen_electra_original_spacy, citizen_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of citizen_electra Spacy: {average_citizen_electra_top10_spacy}')

results_list_citizen_electra_top10, _, average_citizen_electra_top10 = create_results_list(
    masked_citizen_electra_original, citizen_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of citizen_electra Normal: {average_citizen_electra_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of citizen_electra Spacy: 0.518
Avg Precision K = 10 of the ranking of citizen_electra Normal: 0.778


Borda Count
Borda Score
Avg Precision K = 10 of the borda of citizen_electra Normal: 0.774


Borda Spacy Score
Avg Precision K = 10 of the borda of citizen_electra spacy: 0.517


Sem ranking
Avg Precision K = 10 of citizen_electra Spacy: 0.401
Avg Precision K = 10 of citizen_electra Normal: 0.662


#### Language

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_language_electra_top1_ranking_spacy, _, average_language_electra_top1_ranking_spacy = create_results_list(
    ranking_language_electra_spacy_tokens, language_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of language_electra Spacy: {average_language_electra_top1_ranking_spacy}')

results_list_language_electra_top1_ranking_normal, _, average_language_electra_top1_ranking_normal = create_results_list(
    ranking_language_electra_normal_tokens, language_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of language_electra Normal: {average_language_electra_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_electra_top1_borda_normal, indicies_borda_language_electra, average_language_electra_top1_borda_normal = create_results_list(
    language_electra_borda_tokens_score, language_electra.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of language_electra Normal: {average_language_electra_top1_borda_normal}')
print('\n')

results_list_language_electra_top1_borda_spacy, indicies_borda_language_electra_spacy, average_language_electra_top1_borda_spacy = create_results_list(
    language_electra_borda_spacy_tokens_score, language_electra.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of language_electra spacy: {average_language_electra_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_electra_top1_spacy, _, average_language_electra_top1_spacy = create_results_list(
    masked_language_electra_original_spacy, language_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language_electra Spacy: {average_language_electra_top1_spacy}')

results_list_language_electra_top1, _, average_language_electra_top1 = create_results_list(
    masked_language_electra_original, language_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language_electra Normal: {average_language_electra_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of language_electra Spacy: 0.36
Avg Precision K = 1 of the ranking of language_electra Normal: 0.439


Borda Count
Borda Score
Avg Precision K = 1 of the borda of language_electra Normal: 0.431


Borda Spacy Score
Avg Precision K = 1 of the borda of language_electra spacy: 0.36


Sem ranking
Avg Precision K = 1 of language_electra Spacy: 0.455
Avg Precision K = 1 of language_electra Normal: 0.429


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_language_electra_top5_ranking_spacy, _, average_language_electra_top5_ranking_spacy = create_results_list(
    ranking_language_electra_spacy_tokens, language_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of language_electra Spacy: {average_language_electra_top5_ranking_spacy}')

results_list_language_electra_top5_ranking_normal, _, average_language_electra_top5_ranking_normal = create_results_list(
    ranking_language_electra_normal_tokens, language_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of language_electra Normal: {average_language_electra_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_electra_top5_borda_normal, indicies_borda_language_electra, average_language_electra_top5_borda_normal = create_results_list(
    language_electra_borda_tokens_score, language_electra.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of language_electra Normal: {average_language_electra_top5_borda_normal}')
print('\n')

results_list_language_electra_top5_borda_spacy, indicies_borda_language_electra_spacy, average_language_electra_top5_borda_spacy = create_results_list(
    language_electra_borda_spacy_tokens_score, language_electra.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of language_electra spacy: {average_language_electra_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_electra_top5_spacy, _, average_language_electra_top5_spacy = create_results_list(
    masked_language_electra_original_spacy, language_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language_electra Spacy: {average_language_electra_top5_spacy}')

results_list_language_electra_top5, _, average_language_electra_top5 = create_results_list(
    masked_language_electra_original, language_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language_electra Normal: {average_language_electra_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of language_electra Spacy: 0.546
Avg Precision K = 5 of the ranking of language_electra Normal: 0.748


Borda Count
Borda Score
Avg Precision K = 5 of the borda of language_electra Normal: 0.739


Borda Spacy Score
Avg Precision K = 5 of the borda of language_electra spacy: 0.548


Sem ranking
Avg Precision K = 5 of language_electra Spacy: 0.636
Avg Precision K = 5 of language_electra Normal: 0.699


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_language_electra_top10_ranking_spacy, _, average_language_electra_top10_ranking_spacy = create_results_list(
    ranking_language_electra_spacy_tokens, language_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of language_electra Spacy: {average_language_electra_top10_ranking_spacy}')

results_list_language_electra_top10_ranking_normal, _, average_language_electra_top10_ranking_normal = create_results_list(
    ranking_language_electra_normal_tokens, language_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of language_electra Normal: {average_language_electra_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_electra_top10_borda_normal, indicies_borda_language_electra, average_language_electra_top10_borda_normal = create_results_list(
    language_electra_borda_tokens_score, language_electra.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of language_electra Normal: {average_language_electra_top10_borda_normal}')
print('\n')

results_list_language_electra_top10_borda_spacy, indicies_borda_language_electra_spacy, average_language_electra_top10_borda_spacy = create_results_list(
    language_electra_borda_spacy_tokens_score, language_electra.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of language_electra spacy: {average_language_electra_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_electra_top10_spacy, _, average_language_electra_top10_spacy = create_results_list(
    masked_language_electra_original_spacy, language_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language_electra Spacy: {average_language_electra_top10_spacy}')

results_list_language_electra_top10, _, average_language_electra_top10 = create_results_list(
    masked_language_electra_original, language_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language_electra Normal: {average_language_electra_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of language_electra Spacy: 0.6
Avg Precision K = 10 of the ranking of language_electra Normal: 0.849


Borda Count
Borda Score
Avg Precision K = 10 of the borda of language_electra Normal: 0.848


Borda Spacy Score
Avg Precision K = 10 of the borda of language_electra spacy: 0.6


Sem ranking
Avg Precision K = 10 of language_electra Spacy: 0.657
Avg Precision K = 10 of language_electra Normal: 0.781


## Merged Models

#### BornIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_bornIn_merged_model_top1_ranking_spacy, _, average_bornIn_merged_model_top1_ranking_spacy = create_results_list(
    ranking_bornin_merged_model_spacy_tokens, bornIn_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Spacy: {average_bornIn_merged_model_top1_ranking_spacy}')

results_list_bornIn_merged_model_top1_ranking_normal, _, average_bornIn_merged_model_top1_ranking_normal = create_results_list(
    ranking_bornIn_merged_model_normal_tokens, bornIn_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Normal: {average_bornIn_merged_model_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_bornIn_merged_model_top1_borda_normal, indicies_borda_bornIn_merged_model, average_bornIn_merged_model_top1_borda_normal = create_results_list(
    bornIn_merged_model_borda_tokens_score, bornIn_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Born In Normal: {average_bornIn_merged_model_top1_borda_normal}')
print('\n')

results_list_bornIn_merged_model_top1_borda_spacy, indicies_borda_bornIn_merged_model_spacy, average_bornIn_merged_model_top1_borda_spacy = create_results_list(
    bornIn_merged_model_borda_spacy_tokens_score, bornIn_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Born In spacy: {average_bornIn_merged_model_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top1_spacy, _, average_bornIn_bert_top1_spacy = create_results_list(
    masked_bornIn_bert_original_spacy, bornIn_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Bert Spacy: {average_bornIn_bert_top1_spacy}')

results_list_bornIn_bert_top1, _, average_bornIn_bert_top1 = create_results_list(
    masked_bornIn_bert_original, bornIn_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Bert Normal: {average_bornIn_bert_top1}')

results_list_bornIn_roberta_top1_spacy, _, average_bornIn_roberta_top1_spacy = create_results_list(
    masked_bornIn_roberta_original_spacy, bornIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In roberta Spacy: {average_bornIn_roberta_top1_spacy}')

results_list_bornIn_roberta_top1, _, average_bornIn_roberta_top1 = create_results_list(
    masked_bornIn_roberta_original, bornIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In roberta Normal: {average_bornIn_roberta_top1}')

results_list_bornIn_electra_top1_spacy, _, average_bornIn_electra_top1_spacy = create_results_list(
    masked_bornIn_electra_original_spacy, bornIn_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In electra Spacy: {average_bornIn_electra_top1_spacy}')

results_list_bornIn_electra_top1, _, average_bornIn_electra_top1 = create_results_list(
    masked_bornIn_electra_original, bornIn_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In electra Normal: {average_bornIn_electra_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Born In Spacy: 0.413
Avg Precision K = 1 of the ranking of Born In Normal: 0.399


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Born In Normal: 0.4


Borda Spacy Score
Avg Precision K = 1 of the borda of Born In spacy: 0.414


Sem ranking
Avg Precision K = 1 of Born In Bert Spacy: 0.4
Avg Precision K = 1 of Born In Bert Normal: 0.393
Avg Precision K = 1 of Born In roberta Spacy: 0.337
Avg Precision K = 1 of Born In roberta Normal: 0.329
Avg Precision K = 1 of Born In electra Spacy: 0.295
Avg Precision K = 1 of Born In electra Normal: 0.284


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_bornIn_merged_model_top5_ranking_spacy, _, average_bornIn_merged_model_top5_ranking_spacy = create_results_list(
    ranking_bornin_merged_model_spacy_tokens, bornIn_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Spacy: {average_bornIn_merged_model_top5_ranking_spacy}')

results_list_bornIn_merged_model_top5_ranking_normal, _, average_bornIn_merged_model_top5_ranking_normal = create_results_list(
    ranking_bornIn_merged_model_normal_tokens, bornIn_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Normal: {average_bornIn_merged_model_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_bornIn_merged_model_top5_borda_normal, indicies_borda_bornIn_merged_model, average_bornIn_merged_model_top5_borda_normal = create_results_list(
    bornIn_merged_model_borda_tokens_score, bornIn_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Born In Normal: {average_bornIn_merged_model_top5_borda_normal}')
print('\n')

results_list_bornIn_merged_model_top5_borda_spacy, indicies_borda_bornIn_merged_model_spacy, average_bornIn_merged_model_top5_borda_spacy = create_results_list(
    bornIn_merged_model_borda_spacy_tokens_score, bornIn_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Born In spacy: {average_bornIn_merged_model_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top5_spacy, _, average_bornIn_bert_top5_spacy = create_results_list(
    masked_bornIn_bert_original_spacy, bornIn_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Bert Spacy: {average_bornIn_bert_top5_spacy}')

results_list_bornIn_bert_top5, _, average_bornIn_bert_top5 = create_results_list(
    masked_bornIn_bert_original, bornIn_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Bert Normal: {average_bornIn_bert_top5}')

results_list_bornIn_roberta_top5_spacy, _, average_bornIn_roberta_top5_spacy = create_results_list(
    masked_bornIn_roberta_original_spacy, bornIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In roberta Spacy: {average_bornIn_roberta_top5_spacy}')

results_list_bornIn_roberta_top5, _, average_bornIn_roberta_top5 = create_results_list(
    masked_bornIn_roberta_original, bornIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In roberta Normal: {average_bornIn_roberta_top5}')

results_list_bornIn_electra_top5_spacy, _, average_bornIn_electra_top5_spacy = create_results_list(
    masked_bornIn_electra_original_spacy, bornIn_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In electra Spacy: {average_bornIn_electra_top5_spacy}')

results_list_bornIn_electra_top5, _, average_bornIn_electra_top5 = create_results_list(
    masked_bornIn_electra_original, bornIn_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In electra Normal: {average_bornIn_electra_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Born In Spacy: 0.622
Avg Precision K = 5 of the ranking of Born In Normal: 0.608


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Born In Normal: 0.61


Borda Spacy Score
Avg Precision K = 5 of the borda of Born In spacy: 0.622


Sem ranking
Avg Precision K = 5 of Born In Bert Spacy: 0.625
Avg Precision K = 5 of Born In Bert Normal: 0.62
Avg Precision K = 5 of Born In roberta Spacy: 0.528
Avg Precision K = 5 of Born In roberta Normal: 0.52
Avg Precision K = 5 of Born In electra Spacy: 0.513
Avg Precision K = 5 of Born In electra Normal: 0.505


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_bornIn_merged_model_top10_ranking_spacy, _, average_bornIn_merged_model_top10_ranking_spacy = create_results_list(
    ranking_bornin_merged_model_spacy_tokens, bornIn_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Spacy: {average_bornIn_merged_model_top10_ranking_spacy}')

results_list_bornIn_merged_model_top10_ranking_normal, _, average_bornIn_merged_model_top10_ranking_normal = create_results_list(
    ranking_bornIn_merged_model_normal_tokens, bornIn_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Normal: {average_bornIn_merged_model_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_bornIn_merged_model_top10_borda_normal, indicies_borda_bornIn_merged_model, average_bornIn_merged_model_top10_borda_normal = create_results_list(
    bornIn_merged_model_borda_tokens_score, bornIn_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Born In Normal: {average_bornIn_merged_model_top10_borda_normal}')
print('\n')

results_list_bornIn_merged_model_top10_borda_spacy, indicies_borda_bornIn_merged_model_spacy, average_bornIn_merged_model_top10_borda_spacy = create_results_list(
    bornIn_merged_model_borda_spacy_tokens_score, bornIn_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Born In spacy: {average_bornIn_merged_model_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top10_spacy, _, average_bornIn_bert_top10_spacy = create_results_list(
    masked_bornIn_bert_original_spacy, bornIn_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Bert Spacy: {average_bornIn_bert_top10_spacy}')

results_list_bornIn_bert_top10, _, average_bornIn_bert_top10 = create_results_list(
    masked_bornIn_bert_original, bornIn_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Bert Normal: {average_bornIn_bert_top10}')

results_list_bornIn_roberta_top10_spacy, _, average_bornIn_roberta_top10_spacy = create_results_list(
    masked_bornIn_roberta_original_spacy, bornIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In roberta Spacy: {average_bornIn_roberta_top10_spacy}')

results_list_bornIn_roberta_top10, _, average_bornIn_roberta_top10 = create_results_list(
    masked_bornIn_roberta_original, bornIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In roberta Normal: {average_bornIn_roberta_top10}')

results_list_bornIn_electra_top10_spacy, _, average_bornIn_electra_top10_spacy = create_results_list(
    masked_bornIn_electra_original_spacy, bornIn_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In electra Spacy: {average_bornIn_electra_top10_spacy}')

results_list_bornIn_electra_top10, _, average_bornIn_electra_top10 = create_results_list(
    masked_bornIn_electra_original, bornIn_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In electra Normal: {average_bornIn_electra_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Born In Spacy: 0.708
Avg Precision K = 10 of the ranking of Born In Normal: 0.692


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Born In Normal: 0.694


Borda Spacy Score
Avg Precision K = 10 of the borda of Born In spacy: 0.704


Sem ranking
Avg Precision K = 10 of Born In Bert Spacy: 0.677
Avg Precision K = 10 of Born In Bert Normal: 0.684
Avg Precision K = 10 of Born In roberta Spacy: 0.571
Avg Precision K = 10 of Born In roberta Normal: 0.577
Avg Precision K = 10 of Born In electra Spacy: 0.586
Avg Precision K = 10 of Born In electra Normal: 0.589


#### DiedIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_diedIn_merged_model_top1_ranking_spacy, _, average_diedIn_merged_model_top1_ranking_spacy = create_results_list(
    ranking_diedIn_merged_model_spacy_tokens, diedIn_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Died In Spacy: {average_diedIn_merged_model_top1_ranking_spacy}')

results_list_diedIn_merged_model_top1_ranking_normal, _, average_diedIn_merged_model_top1_ranking_normal = create_results_list(
    ranking_diedIn_merged_model_normal_tokens, diedIn_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Died In Normal: {average_diedIn_merged_model_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_diedIn_merged_model_top1_borda_normal, indicies_borda_diedIn_merged_model, average_diedIn_merged_model_top1_borda_normal = create_results_list(
    diedIn_merged_model_borda_tokens_score, diedIn_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Died In Normal: {average_diedIn_merged_model_top1_borda_normal}')
print('\n')

results_list_diedIn_merged_model_top1_borda_spacy, indicies_borda_diedIn_merged_model_spacy, average_diedIn_merged_model_top1_borda_spacy = create_results_list(
    diedIn_merged_model_borda_spacy_tokens_score, diedIn_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Died In spacy: {average_diedIn_merged_model_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top1_spacy, _, average_diedIn_bert_top1_spacy = create_results_list(
    masked_diedIn_bert_original_spacy, diedIn_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Died In Bert Spacy: {average_diedIn_bert_top1_spacy}')

results_list_diedIn_bert_top1, _, average_diedIn_bert_top1 = create_results_list(
    masked_diedIn_bert_original, diedIn_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Died In Bert Normal: {average_diedIn_bert_top1}')

results_list_diedIn_roberta_top1_spacy, _, average_diedIn_roberta_top1_spacy = create_results_list(
    masked_diedIn_roberta_original_spacy, diedIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Died In roberta Spacy: {average_diedIn_roberta_top1_spacy}')

results_list_diedIn_roberta_top1, _, average_diedIn_roberta_top1 = create_results_list(
    masked_diedIn_roberta_original, diedIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Died In roberta Normal: {average_diedIn_roberta_top1}')

results_list_diedIn_electra_top1_spacy, _, average_diedIn_electra_top1_spacy = create_results_list(
    masked_diedIn_electra_original_spacy, diedIn_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Died In electra Spacy: {average_diedIn_electra_top1_spacy}')

results_list_diedIn_electra_top1, _, average_diedIn_electra_top1 = create_results_list(
    masked_diedIn_electra_original, diedIn_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Died In electra Normal: {average_diedIn_electra_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Died In Spacy: 0.503
Avg Precision K = 1 of the ranking of Died In Normal: 0.47


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Died In Normal: 0.47


Borda Spacy Score
Avg Precision K = 1 of the borda of Died In spacy: 0.502


Sem ranking
Avg Precision K = 1 of Died In Bert Spacy: 0.503
Avg Precision K = 1 of Died In Bert Normal: 0.482
Avg Precision K = 1 of Died In roberta Spacy: 0.441
Avg Precision K = 1 of Died In roberta Normal: 0.421
Avg Precision K = 1 of Died In electra Spacy: 0.327
Avg Precision K = 1 of Died In electra Normal: 0.3


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_diedIn_merged_model_top5_ranking_spacy, _, average_diedIn_merged_model_top5_ranking_spacy = create_results_list(
    ranking_diedIn_merged_model_spacy_tokens, diedIn_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Died In Spacy: {average_diedIn_merged_model_top5_ranking_spacy}')

results_list_diedIn_merged_model_top5_ranking_normal, _, average_diedIn_merged_model_top5_ranking_normal = create_results_list(
    ranking_diedIn_merged_model_normal_tokens, diedIn_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Died In Normal: {average_diedIn_merged_model_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_diedIn_merged_model_top5_borda_normal, indicies_borda_diedIn_merged_model, average_diedIn_merged_model_top5_borda_normal = create_results_list(
    diedIn_merged_model_borda_tokens_score, diedIn_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Died In Normal: {average_diedIn_merged_model_top5_borda_normal}')
print('\n')

results_list_diedIn_merged_model_top5_borda_spacy, indicies_borda_diedIn_merged_model_spacy, average_diedIn_merged_model_top5_borda_spacy = create_results_list(
    diedIn_merged_model_borda_spacy_tokens_score, diedIn_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Died In spacy: {average_diedIn_merged_model_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top5_spacy, _, average_diedIn_bert_top5_spacy = create_results_list(
    masked_diedIn_bert_original_spacy, diedIn_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Died In Bert Spacy: {average_diedIn_bert_top5_spacy}')

results_list_diedIn_bert_top5, _, average_diedIn_bert_top5 = create_results_list(
    masked_diedIn_bert_original, diedIn_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Died In Bert Normal: {average_diedIn_bert_top5}')

results_list_diedIn_roberta_top5_spacy, _, average_diedIn_roberta_top5_spacy = create_results_list(
    masked_diedIn_roberta_original_spacy, diedIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Died In roberta Spacy: {average_diedIn_roberta_top5_spacy}')

results_list_diedIn_roberta_top5, _, average_diedIn_roberta_top5 = create_results_list(
    masked_diedIn_roberta_original, diedIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Died In roberta Normal: {average_diedIn_roberta_top5}')

results_list_diedIn_electra_top5_spacy, _, average_diedIn_electra_top5_spacy = create_results_list(
    masked_diedIn_electra_original_spacy, diedIn_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Died In electra Spacy: {average_diedIn_electra_top5_spacy}')

results_list_diedIn_electra_top5, _, average_diedIn_electra_top5 = create_results_list(
    masked_diedIn_electra_original, diedIn_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Died In electra Normal: {average_diedIn_electra_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Died In Spacy: 0.715
Avg Precision K = 5 of the ranking of Died In Normal: 0.706


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Died In Normal: 0.707


Borda Spacy Score
Avg Precision K = 5 of the borda of Died In spacy: 0.717


Sem ranking
Avg Precision K = 5 of Died In Bert Spacy: 0.714
Avg Precision K = 5 of Died In Bert Normal: 0.72
Avg Precision K = 5 of Died In roberta Spacy: 0.64
Avg Precision K = 5 of Died In roberta Normal: 0.631
Avg Precision K = 5 of Died In electra Spacy: 0.58
Avg Precision K = 5 of Died In electra Normal: 0.57


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_diedIn_merged_model_top10_ranking_spacy, _, average_diedIn_merged_model_top10_ranking_spacy = create_results_list(
    ranking_diedIn_merged_model_spacy_tokens, diedIn_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Died In Spacy: {average_diedIn_merged_model_top10_ranking_spacy}')

results_list_diedIn_merged_model_top10_ranking_normal, _, average_diedIn_merged_model_top10_ranking_normal = create_results_list(
    ranking_diedIn_merged_model_normal_tokens, diedIn_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Died In Normal: {average_diedIn_merged_model_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_diedIn_merged_model_top10_borda_normal, indicies_borda_diedIn_merged_model, average_diedIn_merged_model_top10_borda_normal = create_results_list(
    diedIn_merged_model_borda_tokens_score, diedIn_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Died In Normal: {average_diedIn_merged_model_top10_borda_normal}')
print('\n')

results_list_diedIn_merged_model_top10_borda_spacy, indicies_borda_diedIn_merged_model_spacy, average_diedIn_merged_model_top10_borda_spacy = create_results_list(
    diedIn_merged_model_borda_spacy_tokens_score, diedIn_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Died In spacy: {average_diedIn_merged_model_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top10_spacy, _, average_diedIn_bert_top10_spacy = create_results_list(
    masked_diedIn_bert_original_spacy, diedIn_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Died In Bert Spacy: {average_diedIn_bert_top10_spacy}')

results_list_diedIn_bert_top10, _, average_diedIn_bert_top10 = create_results_list(
    masked_diedIn_bert_original, diedIn_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Died In Bert Normal: {average_diedIn_bert_top10}')

results_list_diedIn_roberta_top10_spacy, _, average_diedIn_roberta_top10_spacy = create_results_list(
    masked_diedIn_roberta_original_spacy, diedIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Died In roberta Spacy: {average_diedIn_roberta_top10_spacy}')

results_list_diedIn_roberta_top10, _, average_diedIn_roberta_top10 = create_results_list(
    masked_diedIn_roberta_original, diedIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Died In roberta Normal: {average_diedIn_roberta_top10}')

results_list_diedIn_electra_top10_spacy, _, average_diedIn_electra_top10_spacy = create_results_list(
    masked_diedIn_electra_original_spacy, diedIn_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Died In electra Spacy: {average_diedIn_electra_top10_spacy}')

results_list_diedIn_electra_top10, _, average_diedIn_electra_top10 = create_results_list(
    masked_diedIn_electra_original, diedIn_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Died In electra Normal: {average_diedIn_electra_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Died In Spacy: 0.762
Avg Precision K = 10 of the ranking of Died In Normal: 0.774


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Died In Normal: 0.772


Borda Spacy Score
Avg Precision K = 10 of the borda of Died In spacy: 0.767


Sem ranking
Avg Precision K = 10 of Died In Bert Spacy: 0.755
Avg Precision K = 10 of Died In Bert Normal: 0.78
Avg Precision K = 10 of Died In roberta Spacy: 0.667
Avg Precision K = 10 of Died In roberta Normal: 0.689
Avg Precision K = 10 of Died In electra Spacy: 0.631
Avg Precision K = 10 of Died In electra Normal: 0.652


#### Worksfor

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_worksfor_merged_model_top1_ranking_spacy, _, average_worksfor_merged_model_top1_ranking_spacy = create_results_list(
    ranking_worksfor_merged_model_spacy_tokens, worksfor_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Works For Spacy: {average_worksfor_merged_model_top1_ranking_spacy}')

results_list_worksfor_merged_model_top1_ranking_normal, _, average_worksfor_merged_model_top1_ranking_normal = create_results_list(
    ranking_worksfor_merged_model_normal_tokens, worksfor_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Works For Normal: {average_worksfor_merged_model_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_worksfor_merged_model_top1_borda_normal, indicies_borda_worksfor_merged_model, average_worksfor_merged_model_top1_borda_normal = create_results_list(
    worksfor_merged_model_borda_tokens_score, worksfor_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Works For Normal: {average_worksfor_merged_model_top1_borda_normal}')
print('\n')

results_list_worksfor_merged_model_top1_borda_spacy, indicies_borda_worksfor_merged_model_spacy, average_worksfor_merged_model_top1_borda_spacy = create_results_list(
    worksfor_merged_model_borda_spacy_tokens_score, worksfor_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Works For spacy: {average_worksfor_merged_model_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top1_spacy, _, average_worksfor_bert_top1_spacy = create_results_list(
    masked_worksfor_bert_original_spacy, worksfor_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born Works For Spacy: {average_worksfor_bert_top1_spacy}')

results_list_worksfor_bert_top1, _, average_worksfor_bert_top1 = create_results_list(
    masked_worksfor_bert_original, worksfor_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Works For Bert Normal: {average_worksfor_bert_top1}')

results_list_worksfor_roberta_top1_spacy, _, average_worksfor_roberta_top1_spacy = create_results_list(
    masked_worksfor_roberta_original_spacy, worksfor_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Works For roberta Spacy: {average_worksfor_roberta_top1_spacy}')

results_list_worksfor_roberta_top1, _, average_worksfor_roberta_top1 = create_results_list(
    masked_worksfor_roberta_original, worksfor_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Works For roberta Normal: {average_worksfor_roberta_top1}')

results_list_worksfor_electra_top1_spacy, _, average_worksfor_electra_top1_spacy = create_results_list(
    masked_worksfor_electra_original_spacy, worksfor_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Works For electra Spacy: {average_worksfor_electra_top1_spacy}')

results_list_worksfor_electra_top1, _, average_worksfor_electra_top1 = create_results_list(
    masked_worksfor_electra_original, worksfor_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Works For electra Normal: {average_worksfor_electra_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Works For Spacy: 0.646
Avg Precision K = 1 of the ranking of Works For Normal: 0.58


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Works For Normal: 0.578


Borda Spacy Score
Avg Precision K = 1 of the borda of Works For spacy: 0.636


Sem ranking
Avg Precision K = 1 of Born Works For Spacy: 0.586
Avg Precision K = 1 of Works For Bert Normal: 0.491
Avg Precision K = 1 of Works For roberta Spacy: 0.639
Avg Precision K = 1 of Works For roberta Normal: 0.594
Avg Precision K = 1 of Works For electra Spacy: 0.464
Avg Precision K = 1 of Works For electra Normal: 0.391


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_worksfor_merged_model_top5_ranking_spacy, _, average_worksfor_merged_model_top5_ranking_spacy = create_results_list(
    ranking_worksfor_merged_model_spacy_tokens, worksfor_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Works For Spacy: {average_worksfor_merged_model_top5_ranking_spacy}')

results_list_worksfor_merged_model_top5_ranking_normal, _, average_worksfor_merged_model_top5_ranking_normal = create_results_list(
    ranking_worksfor_merged_model_normal_tokens, worksfor_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Works For Normal: {average_worksfor_merged_model_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_worksfor_merged_model_top5_borda_normal, indicies_borda_worksfor_merged_model, average_worksfor_merged_model_top5_borda_normal = create_results_list(
    worksfor_merged_model_borda_tokens_score, worksfor_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Works For Normal: {average_worksfor_merged_model_top5_borda_normal}')
print('\n')

results_list_worksfor_merged_model_top5_borda_spacy, indicies_borda_worksfor_merged_model_spacy, average_worksfor_merged_model_top5_borda_spacy = create_results_list(
    worksfor_merged_model_borda_spacy_tokens_score, worksfor_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Works For spacy: {average_worksfor_merged_model_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top5_spacy, _, average_worksfor_bert_top5_spacy = create_results_list(
    masked_worksfor_bert_original_spacy, worksfor_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born Works For Spacy: {average_worksfor_bert_top5_spacy}')

results_list_worksfor_bert_top5, _, average_worksfor_bert_top5 = create_results_list(
    masked_worksfor_bert_original, worksfor_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Works For Bert Normal: {average_worksfor_bert_top5}')

results_list_worksfor_roberta_top5_spacy, _, average_worksfor_roberta_top5_spacy = create_results_list(
    masked_worksfor_roberta_original_spacy, worksfor_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Works For roberta Spacy: {average_worksfor_roberta_top5_spacy}')

results_list_worksfor_roberta_top5, _, average_worksfor_roberta_top5 = create_results_list(
    masked_worksfor_roberta_original, worksfor_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Works For roberta Normal: {average_worksfor_roberta_top5}')

results_list_worksfor_electra_top5_spacy, _, average_worksfor_electra_top5_spacy = create_results_list(
    masked_worksfor_electra_original_spacy, worksfor_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Works For electra Spacy: {average_worksfor_electra_top5_spacy}')

results_list_worksfor_electra_top5, _, average_worksfor_electra_top5 = create_results_list(
    masked_worksfor_electra_original, worksfor_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Works For electra Normal: {average_worksfor_electra_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Works For Spacy: 0.847
Avg Precision K = 5 of the ranking of Works For Normal: 0.826


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Works For Normal: 0.821


Borda Spacy Score
Avg Precision K = 5 of the borda of Works For spacy: 0.839


Sem ranking
Avg Precision K = 5 of Born Works For Spacy: 0.81
Avg Precision K = 5 of Works For Bert Normal: 0.784
Avg Precision K = 5 of Works For roberta Spacy: 0.797
Avg Precision K = 5 of Works For roberta Normal: 0.792
Avg Precision K = 5 of Works For electra Spacy: 0.686
Avg Precision K = 5 of Works For electra Normal: 0.628


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_worksfor_merged_model_top10_ranking_spacy, _, average_worksfor_merged_model_top10_ranking_spacy = create_results_list(
    ranking_worksfor_merged_model_spacy_tokens, worksfor_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Works For Spacy: {average_worksfor_merged_model_top10_ranking_spacy}')

results_list_worksfor_merged_model_top10_ranking_normal, _, average_worksfor_merged_model_top10_ranking_normal = create_results_list(
    ranking_worksfor_merged_model_normal_tokens, worksfor_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Works For Normal: {average_worksfor_merged_model_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_worksfor_merged_model_top10_borda_normal, indicies_borda_worksfor_merged_model, average_worksfor_merged_model_top10_borda_normal = create_results_list(
    worksfor_merged_model_borda_tokens_score, worksfor_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Works For Normal: {average_worksfor_merged_model_top10_borda_normal}')
print('\n')

results_list_worksfor_merged_model_top10_borda_spacy, indicies_borda_worksfor_merged_model_spacy, average_worksfor_merged_model_top10_borda_spacy = create_results_list(
    worksfor_merged_model_borda_spacy_tokens_score, worksfor_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Works For spacy: {average_worksfor_merged_model_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top10_spacy, _, average_worksfor_bert_top10_spacy = create_results_list(
    masked_worksfor_bert_original_spacy, worksfor_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born Works For Spacy: {average_worksfor_bert_top10_spacy}')

results_list_worksfor_bert_top10, _, average_worksfor_bert_top10 = create_results_list(
    masked_worksfor_bert_original, worksfor_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Works For Bert Normal: {average_worksfor_bert_top10}')

results_list_worksfor_roberta_top10_spacy, _, average_worksfor_roberta_top10_spacy = create_results_list(
    masked_worksfor_roberta_original_spacy, worksfor_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Works For roberta Spacy: {average_worksfor_roberta_top10_spacy}')

results_list_worksfor_roberta_top10, _, average_worksfor_roberta_top10 = create_results_list(
    masked_worksfor_roberta_original, worksfor_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Works For roberta Normal: {average_worksfor_roberta_top10}')

results_list_worksfor_electra_top10_spacy, _, average_worksfor_electra_top10_spacy = create_results_list(
    masked_worksfor_electra_original_spacy, worksfor_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Works For electra Spacy: {average_worksfor_electra_top10_spacy}')

results_list_worksfor_electra_top10, _, average_worksfor_electra_top10 = create_results_list(
    masked_worksfor_electra_original, worksfor_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Works For electra Normal: {average_worksfor_electra_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Works For Spacy: 0.871
Avg Precision K = 10 of the ranking of Works For Normal: 0.887


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Works For Normal: 0.884


Borda Spacy Score
Avg Precision K = 10 of the borda of Works For spacy: 0.871


Sem ranking
Avg Precision K = 10 of Born Works For Spacy: 0.815
Avg Precision K = 10 of Works For Bert Normal: 0.858
Avg Precision K = 10 of Works For roberta Spacy: 0.813
Avg Precision K = 10 of Works For roberta Normal: 0.855
Avg Precision K = 10 of Works For electra Spacy: 0.699
Avg Precision K = 10 of Works For electra Normal: 0.728


#### Capital

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_capital_merged_model_top1_ranking_spacy, _, average_capital_merged_model_top1_ranking_spacy = create_results_list(
    ranking_capital_merged_model_spacy_tokens, capital_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Capital Spacy: {average_capital_merged_model_top1_ranking_spacy}')

results_list_capital_merged_model_top1_ranking_normal, _, average_capital_merged_model_top1_ranking_normal = create_results_list(
    ranking_capital_merged_model_normal_tokens, capital_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Capital Normal: {average_capital_merged_model_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_capital_merged_model_top1_borda_normal, indicies_borda_capital_merged_model, average_capital_merged_model_top1_borda_normal = create_results_list(
    capital_merged_model_borda_tokens_score, capital_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Capital Normal: {average_capital_merged_model_top1_borda_normal}')
print('\n')

results_list_capital_merged_model_top1_borda_spacy, indicies_borda_capital_merged_model_spacy, average_capital_merged_model_top1_borda_spacy = create_results_list(
    capital_merged_model_borda_spacy_tokens_score, capital_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Capital spacy: {average_capital_merged_model_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top1_spacy, _, average_capital_bert_top1_spacy = create_results_list(
    masked_capital_bert_original_spacy, capital_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Capital Bert Spacy: {average_capital_bert_top1_spacy}')

results_list_capital_bert_top1, _, average_capital_bert_top1 = create_results_list(
    masked_capital_bert_original, capital_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Capital Bert Normal: {average_capital_bert_top1}')

results_list_capital_electra_top1_spacy, _, average_capital_electra_top1_spacy = create_results_list(
    masked_capital_electra_original_spacy, capital_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Capital electra Spacy: {average_capital_electra_top1_spacy}')

results_list_capital_electra_top1, _, average_capital_roberta_top1 = create_results_list(
    masked_capital_roberta_original, capital_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Capital roberta Normal: {average_capital_roberta_top1}')

results_list_capital_electra_top1_spacy, _, average_capital_electra_top1_spacy = create_results_list(
    masked_capital_electra_original_spacy, capital_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Capital electra Spacy: {average_capital_electra_top1_spacy}')

results_list_capital_electra_top1, _, average_capital_electra_top1 = create_results_list(
    masked_capital_electra_original, capital_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Capital electra Normal: {average_capital_electra_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Capital Spacy: 0.563
Avg Precision K = 1 of the ranking of Capital Normal: 0.544


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Capital Normal: 0.541


Borda Spacy Score
Avg Precision K = 1 of the borda of Capital spacy: 0.563


Sem ranking
Avg Precision K = 1 of Capital Bert Spacy: 0.554
Avg Precision K = 1 of Capital Bert Normal: 0.545
Avg Precision K = 1 of Capital electra Spacy: 0.375
Avg Precision K = 1 of Capital roberta Normal: 0.367
Avg Precision K = 1 of Capital electra Spacy: 0.375
Avg Precision K = 1 of Capital electra Normal: 0.365


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_capital_merged_model_top5_ranking_spacy, _, average_capital_merged_model_top5_ranking_spacy = create_results_list(
    ranking_capital_merged_model_spacy_tokens, capital_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Capital Spacy: {average_capital_merged_model_top5_ranking_spacy}')

results_list_capital_merged_model_top5_ranking_normal, _, average_capital_merged_model_top5_ranking_normal = create_results_list(
    ranking_capital_merged_model_normal_tokens, capital_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Capital Normal: {average_capital_merged_model_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_capital_merged_model_top5_borda_normal, indicies_borda_capital_merged_model, average_capital_merged_model_top5_borda_normal = create_results_list(
    capital_merged_model_borda_tokens_score, capital_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Capital Normal: {average_capital_merged_model_top5_borda_normal}')
print('\n')

results_list_capital_merged_model_top5_borda_spacy, indicies_borda_capital_merged_model_spacy, average_capital_merged_model_top5_borda_spacy = create_results_list(
    capital_merged_model_borda_spacy_tokens_score, capital_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Capital spacy: {average_capital_merged_model_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top5_spacy, _, average_capital_bert_top5_spacy = create_results_list(
    masked_capital_bert_original_spacy, capital_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Capital Bert Spacy: {average_capital_bert_top5_spacy}')

results_list_capital_bert_top5, _, average_capital_bert_top5 = create_results_list(
    masked_capital_bert_original, capital_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Capital Bert Normal: {average_capital_bert_top5}')

results_list_capital_electra_top5_spacy, _, average_capital_electra_top5_spacy = create_results_list(
    masked_capital_electra_original_spacy, capital_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Capital electra Spacy: {average_capital_electra_top5_spacy}')

results_list_capital_electra_top5, _, average_capital_roberta_top5 = create_results_list(
    masked_capital_roberta_original, capital_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Capital roberta Normal: {average_capital_roberta_top5}')

results_list_capital_electra_top5_spacy, _, average_capital_electra_top5_spacy = create_results_list(
    masked_capital_electra_original_spacy, capital_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Capital electra Spacy: {average_capital_electra_top5_spacy}')

results_list_capital_electra_top5, _, average_capital_electra_top5 = create_results_list(
    masked_capital_electra_original, capital_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Capital electra Normal: {average_capital_electra_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Capital Spacy: 0.752
Avg Precision K = 5 of the ranking of Capital Normal: 0.776


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Capital Normal: 0.772


Borda Spacy Score
Avg Precision K = 5 of the borda of Capital spacy: 0.752


Sem ranking
Avg Precision K = 5 of Capital Bert Spacy: 0.722
Avg Precision K = 5 of Capital Bert Normal: 0.75
Avg Precision K = 5 of Capital electra Spacy: 0.54
Avg Precision K = 5 of Capital roberta Normal: 0.498
Avg Precision K = 5 of Capital electra Spacy: 0.54
Avg Precision K = 5 of Capital electra Normal: 0.554


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_capital_merged_model_top10_ranking_spacy, _, average_capital_merged_model_top10_ranking_spacy = create_results_list(
    ranking_capital_merged_model_spacy_tokens, capital_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Capital Spacy: {average_capital_merged_model_top10_ranking_spacy}')

results_list_capital_merged_model_top10_ranking_normal, _, average_capital_merged_model_top10_ranking_normal = create_results_list(
    ranking_capital_merged_model_normal_tokens, capital_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Capital Normal: {average_capital_merged_model_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_capital_merged_model_top10_borda_normal, indicies_borda_capital_merged_model, average_capital_merged_model_top10_borda_normal = create_results_list(
    capital_merged_model_borda_tokens_score, capital_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Capital Normal: {average_capital_merged_model_top10_borda_normal}')
print('\n')

results_list_capital_merged_model_top10_borda_spacy, indicies_borda_capital_merged_model_spacy, average_capital_merged_model_top10_borda_spacy = create_results_list(
    capital_merged_model_borda_spacy_tokens_score, capital_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Capital spacy: {average_capital_merged_model_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top10_spacy, _, average_capital_bert_top10_spacy = create_results_list(
    masked_capital_bert_original_spacy, capital_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Capital Bert Spacy: {average_capital_bert_top10_spacy}')

results_list_capital_bert_top10, _, average_capital_bert_top10 = create_results_list(
    masked_capital_bert_original, capital_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Capital Bert Normal: {average_capital_bert_top10}')

results_list_capital_electra_top10_spacy, _, average_capital_electra_top10_spacy = create_results_list(
    masked_capital_electra_original_spacy, capital_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Capital electra Spacy: {average_capital_electra_top10_spacy}')

results_list_capital_electra_top10, _, average_capital_roberta_top10 = create_results_list(
    masked_capital_roberta_original, capital_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Capital roberta Normal: {average_capital_roberta_top10}')

results_list_capital_electra_top10_spacy, _, average_capital_electra_top10_spacy = create_results_list(
    masked_capital_electra_original_spacy, capital_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Capital electra Spacy: {average_capital_electra_top10_spacy}')

results_list_capital_electra_top10, _, average_capital_electra_top10 = create_results_list(
    masked_capital_electra_original, capital_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Capital electra Normal: {average_capital_electra_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Capital Spacy: 0.779
Avg Precision K = 10 of the ranking of Capital Normal: 0.81


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Capital Normal: 0.812


Borda Spacy Score
Avg Precision K = 10 of the borda of Capital spacy: 0.778


Sem ranking
Avg Precision K = 10 of Capital Bert Spacy: 0.736
Avg Precision K = 10 of Capital Bert Normal: 0.785
Avg Precision K = 10 of Capital electra Spacy: 0.584
Avg Precision K = 10 of Capital roberta Normal: 0.519
Avg Precision K = 10 of Capital electra Spacy: 0.584
Avg Precision K = 10 of Capital electra Normal: 0.615


#### Citizen

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_citizen_merged_model_top1_ranking_spacy, _, average_citizen_merged_model_top1_ranking_spacy = create_results_list(
    ranking_citizen_merged_model_spacy_tokens, citizen_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Citizen Spacy: {average_citizen_merged_model_top1_ranking_spacy}')

results_list_citizen_merged_model_top1_ranking_normal, _, average_citizen_merged_model_top1_ranking_normal = create_results_list(
    ranking_citizen_merged_model_normal_tokens, citizen_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Citizen Normal: {average_citizen_merged_model_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_merged_model_top1_borda_normal, indicies_borda_citizen_merged_model, average_citizen_merged_model_top1_borda_normal = create_results_list(
    citizen_merged_model_borda_tokens_score, citizen_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Citizen Normal: {average_citizen_merged_model_top1_borda_normal}')
print('\n')

results_list_citizen_merged_model_top1_borda_spacy, indicies_borda_citizen_merged_model_spacy, average_citizen_merged_model_top1_borda_spacy = create_results_list(
    citizen_merged_model_borda_spacy_tokens_score, citizen_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Citizen spacy: {average_citizen_merged_model_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top1_spacy, _, average_citizen_bert_top1_spacy = create_results_list(
    masked_citizen_bert_original_spacy, citizen_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Citizen Bert Spacy: {average_citizen_bert_top1_spacy}')

results_list_citizen_bert_top1, _, average_citizen_bert_top1 = create_results_list(
    masked_citizen_bert_original, citizen_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Citizen Bert Normal: {average_citizen_bert_top1}')

results_list_citizen_roberta_top1_spacy, _, average_citizen_roberta_top1_spacy = create_results_list(
    masked_citizen_roberta_original_spacy, citizen_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Citizen roberta Spacy: {average_citizen_roberta_top1_spacy}')

results_list_citizen_roberta_top1, _, average_citizen_roberta_top1 = create_results_list(
    masked_citizen_roberta_original, citizen_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Citizen roberta Normal: {average_citizen_roberta_top1}')

results_list_citizen_electra_top1_spacy, _, average_citizen_electra_top1_spacy = create_results_list(
    masked_citizen_electra_original_spacy, citizen_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Citizen electra Spacy: {average_citizen_electra_top1_spacy}')

results_list_citizen_electra_top1, _, average_citizen_electra_top1 = create_results_list(
    masked_citizen_electra_original, citizen_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Citizen electra Normal: {average_citizen_electra_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Citizen Spacy: 0.433
Avg Precision K = 1 of the ranking of Citizen Normal: 0.597


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Citizen Normal: 0.598


Borda Spacy Score
Avg Precision K = 1 of the borda of Citizen spacy: 0.434


Sem ranking
Avg Precision K = 1 of Citizen Bert Spacy: 0.423
Avg Precision K = 1 of Citizen Bert Normal: 0.58
Avg Precision K = 1 of Citizen roberta Spacy: 0.409
Avg Precision K = 1 of Citizen roberta Normal: 0.567
Avg Precision K = 1 of Citizen electra Spacy: 0.327
Avg Precision K = 1 of Citizen electra Normal: 0.483


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_citizen_merged_model_top5_ranking_spacy, _, average_citizen_merged_model_top5_ranking_spacy = create_results_list(
    ranking_citizen_merged_model_spacy_tokens, citizen_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Citizen Spacy: {average_citizen_merged_model_top5_ranking_spacy}')

results_list_citizen_merged_model_top5_ranking_normal, _, average_citizen_merged_model_top5_ranking_normal = create_results_list(
    ranking_citizen_merged_model_normal_tokens, citizen_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Citizen Normal: {average_citizen_merged_model_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_merged_model_top5_borda_normal, indicies_borda_citizen_merged_model, average_citizen_merged_model_top5_borda_normal = create_results_list(
    citizen_merged_model_borda_tokens_score, citizen_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Citizen Normal: {average_citizen_merged_model_top5_borda_normal}')
print('\n')

results_list_citizen_merged_model_top5_borda_spacy, indicies_borda_citizen_merged_model_spacy, average_citizen_merged_model_top5_borda_spacy = create_results_list(
    citizen_merged_model_borda_spacy_tokens_score, citizen_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Citizen spacy: {average_citizen_merged_model_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top5_spacy, _, average_citizen_bert_top5_spacy = create_results_list(
    masked_citizen_bert_original_spacy, citizen_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Citizen Bert Spacy: {average_citizen_bert_top5_spacy}')

results_list_citizen_bert_top5, _, average_citizen_bert_top5 = create_results_list(
    masked_citizen_bert_original, citizen_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Citizen Bert Normal: {average_citizen_bert_top5}')

results_list_citizen_roberta_top5_spacy, _, average_citizen_roberta_top5_spacy = create_results_list(
    masked_citizen_roberta_original_spacy, citizen_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Citizen roberta Spacy: {average_citizen_roberta_top5_spacy}')

results_list_citizen_roberta_top5, _, average_citizen_roberta_top5 = create_results_list(
    masked_citizen_roberta_original, citizen_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Citizen roberta Normal: {average_citizen_roberta_top5}')

results_list_citizen_electra_top5_spacy, _, average_citizen_electra_top5_spacy = create_results_list(
    masked_citizen_electra_original_spacy, citizen_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Citizen electra Spacy: {average_citizen_electra_top5_spacy}')

results_list_citizen_electra_top5, _, average_citizen_electra_top5 = create_results_list(
    masked_citizen_electra_original, citizen_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Citizen electra Normal: {average_citizen_electra_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Citizen Spacy: 0.508
Avg Precision K = 5 of the ranking of Citizen Normal: 0.71


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Citizen Normal: 0.711


Borda Spacy Score
Avg Precision K = 5 of the borda of Citizen spacy: 0.506


Sem ranking
Avg Precision K = 5 of Citizen Bert Spacy: 0.488
Avg Precision K = 5 of Citizen Bert Normal: 0.732
Avg Precision K = 5 of Citizen roberta Spacy: 0.472
Avg Precision K = 5 of Citizen roberta Normal: 0.709
Avg Precision K = 5 of Citizen electra Spacy: 0.396
Avg Precision K = 5 of Citizen electra Normal: 0.602


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_citizen_merged_model_top10_ranking_spacy, _, average_citizen_merged_model_top10_ranking_spacy = create_results_list(
    ranking_citizen_merged_model_spacy_tokens, citizen_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Citizen Spacy: {average_citizen_merged_model_top10_ranking_spacy}')

results_list_citizen_merged_model_top10_ranking_normal, _, average_citizen_merged_model_top10_ranking_normal = create_results_list(
    ranking_citizen_merged_model_normal_tokens, citizen_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Citizen Normal: {average_citizen_merged_model_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_citizen_merged_model_top10_borda_normal, indicies_borda_citizen_merged_model, average_citizen_merged_model_top10_borda_normal = create_results_list(
    citizen_merged_model_borda_tokens_score, citizen_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Citizen Normal: {average_citizen_merged_model_top10_borda_normal}')
print('\n')

results_list_citizen_merged_model_top10_borda_spacy, indicies_borda_citizen_merged_model_spacy, average_citizen_merged_model_top10_borda_spacy = create_results_list(
    citizen_merged_model_borda_spacy_tokens_score, citizen_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Citizen spacy: {average_citizen_merged_model_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top10_spacy, _, average_citizen_bert_top10_spacy = create_results_list(
    masked_citizen_bert_original_spacy, citizen_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Citizen Bert Spacy: {average_citizen_bert_top10_spacy}')

results_list_citizen_bert_top10, _, average_citizen_bert_top10 = create_results_list(
    masked_citizen_bert_original, citizen_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Citizen Bert Normal: {average_citizen_bert_top10}')

results_list_citizen_roberta_top10_spacy, _, average_citizen_roberta_top10_spacy = create_results_list(
    masked_citizen_roberta_original_spacy, citizen_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Citizen roberta Spacy: {average_citizen_roberta_top10_spacy}')

results_list_citizen_roberta_top10, _, average_citizen_roberta_top10 = create_results_list(
    masked_citizen_roberta_original, citizen_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Citizen roberta Normal: {average_citizen_roberta_top10}')

results_list_citizen_electra_top10_spacy, _, average_citizen_electra_top10_spacy = create_results_list(
    masked_citizen_electra_original_spacy, citizen_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Citizen electra Spacy: {average_citizen_electra_top10_spacy}')

results_list_citizen_electra_top10, _, average_citizen_electra_top10 = create_results_list(
    masked_citizen_electra_original, citizen_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Citizen electra Normal: {average_citizen_electra_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Citizen Spacy: 0.516
Avg Precision K = 10 of the ranking of Citizen Normal: 0.758


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Citizen Normal: 0.762


Borda Spacy Score
Avg Precision K = 10 of the borda of Citizen spacy: 0.515


Sem ranking
Avg Precision K = 10 of Citizen Bert Spacy: 0.491
Avg Precision K = 10 of Citizen Bert Normal: 0.802
Avg Precision K = 10 of Citizen roberta Spacy: 0.472
Avg Precision K = 10 of Citizen roberta Normal: 0.763
Avg Precision K = 10 of Citizen electra Spacy: 0.401
Avg Precision K = 10 of Citizen electra Normal: 0.662


#### Language

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_language_merged_model_top1_ranking_spacy, _, average_language_merged_model_top1_ranking_spacy = create_results_list(
    ranking_language_merged_model_spacy_tokens, language_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of language Spacy: {average_language_merged_model_top1_ranking_spacy}')

results_list_language_merged_model_top1_ranking_normal, _, average_language_merged_model_top1_ranking_normal = create_results_list(
    ranking_language_merged_model_normal_tokens, language_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of language Normal: {average_language_merged_model_top1_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_merged_model_top1_borda_normal, indicies_borda_language_merged_model, average_language_merged_model_top1_borda_normal = create_results_list(
    language_merged_model_borda_tokens_score, language_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of language Normal: {average_language_merged_model_top1_borda_normal}')
print('\n')

results_list_language_merged_model_top1_borda_spacy, indicies_borda_language_merged_model_spacy, average_language_merged_model_top1_borda_spacy = create_results_list(
    language_merged_model_borda_spacy_tokens_score, language_merged_model_spacy.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of language spacy: {average_language_merged_model_top1_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_bert_top1_spacy, _, average_language_bert_top1_spacy = create_results_list(
    masked_language_bert_original_spacy, language_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language Bert Spacy: {average_language_bert_top1_spacy}')

results_list_language_bert_top1, _, average_language_bert_top1 = create_results_list(
    masked_language_bert_original, language_bert.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language Bert Normal: {average_language_bert_top1}')

results_list_language_roberta_top1_spacy, _, average_language_roberta_top1_spacy = create_results_list(
    masked_language_roberta_original_spacy, language_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language roberta Spacy: {average_language_roberta_top1_spacy}')

results_list_language_roberta_top1, _, average_language_roberta_top1 = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language roberta Normal: {average_language_roberta_top1}')

results_list_language_electra_top1_spacy, _, average_language_electra_top1_spacy = create_results_list(
    masked_language_electra_original_spacy, language_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language electra Spacy: {average_language_electra_top1_spacy}')

results_list_language_electra_top1, _, average_language_electra_top1 = create_results_list(
    masked_language_electra_original, language_electra.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of language electra Normal: {average_language_electra_top1}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of language Spacy: 0.705
Avg Precision K = 1 of the ranking of language Normal: 0.737


Borda Count
Borda Score
Avg Precision K = 1 of the borda of language Normal: 0.731


Borda Spacy Score
Avg Precision K = 1 of the borda of language spacy: 0.69


Sem ranking
Avg Precision K = 1 of language Bert Spacy: 0.674
Avg Precision K = 1 of language Bert Normal: 0.738
Avg Precision K = 1 of language roberta Spacy: 0.614
Avg Precision K = 1 of language roberta Normal: 0.639
Avg Precision K = 1 of language electra Spacy: 0.455
Avg Precision K = 1 of language electra Normal: 0.429


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_language_merged_model_top5_ranking_spacy, _, average_language_merged_model_top5_ranking_spacy = create_results_list(
    ranking_language_merged_model_spacy_tokens, language_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of language Spacy: {average_language_merged_model_top5_ranking_spacy}')

results_list_language_merged_model_top5_ranking_normal, _, average_language_merged_model_top5_ranking_normal = create_results_list(
    ranking_language_merged_model_normal_tokens, language_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of language Normal: {average_language_merged_model_top5_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_merged_model_top5_borda_normal, indicies_borda_language_merged_model, average_language_merged_model_top5_borda_normal = create_results_list(
    language_merged_model_borda_tokens_score, language_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of language Normal: {average_language_merged_model_top5_borda_normal}')
print('\n')

results_list_language_merged_model_top5_borda_spacy, indicies_borda_language_merged_model_spacy, average_language_merged_model_top5_borda_spacy = create_results_list(
    language_merged_model_borda_spacy_tokens_score, language_merged_model_spacy.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of language spacy: {average_language_merged_model_top5_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_bert_top5_spacy, _, average_language_bert_top5_spacy = create_results_list(
    masked_language_bert_original_spacy, language_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language Bert Spacy: {average_language_bert_top5_spacy}')

results_list_language_bert_top5, _, average_language_bert_top5 = create_results_list(
    masked_language_bert_original, language_bert.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language Bert Normal: {average_language_bert_top5}')

results_list_language_roberta_top5_spacy, _, average_language_roberta_top5_spacy = create_results_list(
    masked_language_roberta_original_spacy, language_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language roberta Spacy: {average_language_roberta_top5_spacy}')

results_list_language_roberta_top5, _, average_language_roberta_top5 = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language roberta Normal: {average_language_roberta_top5}')

results_list_language_electra_top5_spacy, _, average_language_electra_top5_spacy = create_results_list(
    masked_language_electra_original_spacy, language_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language electra Spacy: {average_language_electra_top5_spacy}')

results_list_language_electra_top5, _, average_language_electra_top5 = create_results_list(
    masked_language_electra_original, language_electra.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of language electra Normal: {average_language_electra_top5}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of language Spacy: 0.796
Avg Precision K = 5 of the ranking of language Normal: 0.879


Borda Count
Borda Score
Avg Precision K = 5 of the borda of language Normal: 0.878


Borda Spacy Score
Avg Precision K = 5 of the borda of language spacy: 0.795


Sem ranking
Avg Precision K = 5 of language Bert Spacy: 0.77
Avg Precision K = 5 of language Bert Normal: 0.885
Avg Precision K = 5 of language roberta Spacy: 0.702
Avg Precision K = 5 of language roberta Normal: 0.785
Avg Precision K = 5 of language electra Spacy: 0.636
Avg Precision K = 5 of language electra Normal: 0.699


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_language_merged_model_top10_ranking_spacy, _, average_language_merged_model_top10_ranking_spacy = create_results_list(
    ranking_language_merged_model_spacy_tokens, language_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of language Spacy: {average_language_merged_model_top10_ranking_spacy}')

results_list_language_merged_model_top10_ranking_normal, _, average_language_merged_model_top10_ranking_normal = create_results_list(
    ranking_language_merged_model_normal_tokens, language_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of language Normal: {average_language_merged_model_top10_ranking_normal}')
print('\n')

print('Borda Count')
results_list_language_merged_model_top10_borda_normal, indicies_borda_language_merged_model, average_language_merged_model_top10_borda_normal = create_results_list(
    language_merged_model_borda_tokens_score, language_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of language Normal: {average_language_merged_model_top10_borda_normal}')
print('\n')

results_list_language_merged_model_top10_borda_spacy, indicies_borda_language_merged_model_spacy, average_language_merged_model_top10_borda_spacy = create_results_list(
    language_merged_model_borda_spacy_tokens_score, language_merged_model_spacy.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of language spacy: {average_language_merged_model_top10_borda_spacy}')
print('\n')

print('Sem ranking')
results_list_language_bert_top10_spacy, _, average_language_bert_top10_spacy = create_results_list(
    masked_language_bert_original_spacy, language_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language Bert Spacy: {average_language_bert_top10_spacy}')

results_list_language_bert_top10, _, average_language_bert_top10 = create_results_list(
    masked_language_bert_original, language_bert.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language Bert Normal: {average_language_bert_top10}')

results_list_language_roberta_top10_spacy, _, average_language_roberta_top10_spacy = create_results_list(
    masked_language_roberta_original_spacy, language_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language roberta Spacy: {average_language_roberta_top10_spacy}')

results_list_language_roberta_top10, _, average_language_roberta_top10 = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language roberta Normal: {average_language_roberta_top10}')

results_list_language_electra_top10_spacy, _, average_language_electra_top10_spacy = create_results_list(
    masked_language_electra_original_spacy, language_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language electra Spacy: {average_language_electra_top10_spacy}')

results_list_language_electra_top10, _, average_language_electra_top10 = create_results_list(
    masked_language_electra_original, language_electra.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of language electra Normal: {average_language_electra_top10}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of language Spacy: 0.806
Avg Precision K = 10 of the ranking of language Normal: 0.919


Borda Count
Borda Score
Avg Precision K = 10 of the borda of language Normal: 0.919


Borda Spacy Score
Avg Precision K = 10 of the borda of language spacy: 0.806


Sem ranking
Avg Precision K = 10 of language Bert Spacy: 0.779
Avg Precision K = 10 of language Bert Normal: 0.916
Avg Precision K = 10 of language roberta Spacy: 0.703
Avg Precision K = 10 of language roberta Normal: 0.831
Avg Precision K = 10 of language electra Spacy: 0.657
Avg Precision K = 10 of language electra Normal: 0.781


# One and Few Shot

## One Shot

### Bert

#### BornIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_bornIn_bert_top1_ranking_spacy_one, _, average_bornIn_bert_top1_ranking_spacy_one = create_results_list(
    ranking_bornIn_bert_spacy_tokens_one, bornIn_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Spacy One Shoot: {average_bornIn_bert_top1_ranking_spacy_one}')

results_list_bornIn_bert_top1_ranking_normal_one, _, average_bornIn_bert_top1_ranking_normal_one = create_results_list(
    ranking_bornIn_bert_normal_tokens_one, bornIn_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Normal One Shoot: {average_bornIn_bert_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_bornIn_bert_top1_borda_normal_one, _, average_bornIn_bert_top1_borda_normal_one = create_results_list(
    bornIn_bert_borda_tokens_score_one, bornIn_bert_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Born In Normal One Shoot: {average_bornIn_bert_top1_borda_normal_one}')
print('\n')

results_list_bornIn_bert_top1_borda_spacy_one, _, average_bornIn_bert_top1_borda_spacy_one = create_results_list(
    bornIn_bert_borda_spacy_tokens_score_one, bornIn_bert_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Born In Spacy One Shoot: {average_bornIn_bert_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top1_spacy_one, _, average_bornIn_bert_top1_spacy_one = create_results_list(
    masked_bornIn_bert_original_spacy_one, bornIn_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Spacy One Shoot: {average_bornIn_bert_top1_spacy_one}')

results_list_bornIn_bert_top1_one, _, average_bornIn_bert_top1_one = create_results_list(
    masked_bornIn_bert_original_one, bornIn_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Normal: {average_bornIn_bert_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Born In Spacy One Shoot: 0.213
Avg Precision K = 1 of the ranking of Born In Normal One Shoot: 0.258


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Born In Normal One Shoot: 0.256


Borda Spacy Score
Avg Precision K = 1 of the borda of Born In Spacy One Shoot: 0.217


Sem ranking
Avg Precision K = 1 of Born In Spacy One Shoot: 0.347
Avg Precision K = 1 of Born In Normal: 0.335


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_bornIn_bert_top5_ranking_spacy_one, _, average_bornIn_bert_top5_ranking_spacy_one = create_results_list(
    ranking_bornIn_bert_spacy_tokens_one, bornIn_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Spacy One Shoot: {average_bornIn_bert_top5_ranking_spacy_one}')

results_list_bornIn_bert_top5_ranking_normal_one, _, average_bornIn_bert_top5_ranking_normal_one = create_results_list(
    ranking_bornIn_bert_normal_tokens_one, bornIn_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Normal One Shoot: {average_bornIn_bert_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_bornIn_bert_top5_borda_normal_one, _, average_bornIn_bert_top5_borda_normal_one = create_results_list(
    bornIn_bert_borda_tokens_score_one, bornIn_bert_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Born In Normal One Shoot: {average_bornIn_bert_top5_borda_normal_one}')
print('\n')

results_list_bornIn_bert_top5_borda_spacy_one, _, average_bornIn_bert_top5_borda_spacy_one = create_results_list(
    bornIn_bert_borda_spacy_tokens_score_one, bornIn_bert_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Born In Spacy One Shoot: {average_bornIn_bert_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top5_spacy_one, _, average_bornIn_bert_top5_spacy_one = create_results_list(
    masked_bornIn_bert_original_spacy_one, bornIn_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Spacy One Shoot: {average_bornIn_bert_top5_spacy_one}')

results_list_bornIn_bert_top5_one, _, average_bornIn_bert_top5_one = create_results_list(
    masked_bornIn_bert_original_one, bornIn_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Normal: {average_bornIn_bert_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Born In Spacy One Shoot: 0.511
Avg Precision K = 5 of the ranking of Born In Normal One Shoot: 0.551


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Born In Normal One Shoot: 0.547


Borda Spacy Score
Avg Precision K = 5 of the borda of Born In Spacy One Shoot: 0.513


Sem ranking
Avg Precision K = 5 of Born In Spacy One Shoot: 0.555
Avg Precision K = 5 of Born In Normal: 0.548


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_bornIn_bert_top10_ranking_spacy_one, _, average_bornIn_bert_top10_ranking_spacy_one = create_results_list(
    ranking_bornIn_bert_spacy_tokens_one, bornIn_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Spacy One Shoot: {average_bornIn_bert_top10_ranking_spacy_one}')

results_list_bornIn_bert_top10_ranking_normal_one, _, average_bornIn_bert_top10_ranking_normal_one = create_results_list(
    ranking_bornIn_bert_normal_tokens_one, bornIn_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Normal One Shoot: {average_bornIn_bert_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_bornIn_bert_top10_borda_normal_one, _, average_bornIn_bert_top10_borda_normal_one = create_results_list(
    bornIn_bert_borda_tokens_score_one, bornIn_bert_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Born In Normal One Shoot: {average_bornIn_bert_top10_borda_normal_one}')
print('\n')

results_list_bornIn_bert_top10_borda_spacy_one, _, average_bornIn_bert_top10_borda_spacy_one = create_results_list(
    bornIn_bert_borda_spacy_tokens_score_one, bornIn_bert_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Born In Spacy One Shoot: {average_bornIn_bert_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top10_spacy_one, _, average_bornIn_bert_top10_spacy_one = create_results_list(
    masked_bornIn_bert_original_spacy_one, bornIn_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Spacy One Shoot: {average_bornIn_bert_top10_spacy_one}')

results_list_bornIn_bert_top10_one, _, average_bornIn_bert_top10_one = create_results_list(
    masked_bornIn_bert_original_one, bornIn_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Normal: {average_bornIn_bert_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Born In Spacy One Shoot: 0.619
Avg Precision K = 10 of the ranking of Born In Normal One Shoot: 0.639


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Born In Normal One Shoot: 0.637


Borda Spacy Score
Avg Precision K = 10 of the borda of Born In Spacy One Shoot: 0.608


Sem ranking
Avg Precision K = 10 of Born In Spacy One Shoot: 0.63
Avg Precision K = 10 of Born In Normal: 0.637


#### DiedIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_diedIn_bert_top1_ranking_spacy_one, _, average_diedIn_bert_top1_ranking_spacy_one = create_results_list(
    ranking_diedIn_bert_spacy_tokens_one, diedIn_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_diedIn_bert_top1_ranking_spacy_one}')

results_list_diedIn_bert_top1_ranking_normal_one, _, average_diedIn_bert_top1_ranking_normal_one = create_results_list(
    ranking_diedIn_bert_normal_tokens_one, diedIn_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_diedIn_bert_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_diedIn_bert_top1_borda_normal_one, _, average_diedIn_bert_top1_borda_normal_one = create_results_list(
    diedIn_bert_borda_tokens_score_one, diedIn_bert_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_diedIn_bert_top1_borda_normal_one}')
print('\n')

results_list_diedIn_bert_top1_borda_spacy_one, _, average_diedIn_bert_top1_borda_spacy_one = create_results_list(
    diedIn_bert_borda_spacy_tokens_score_one, diedIn_bert_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_diedIn_bert_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top1_spacy_one, _, average_diedIn_bert_top1_spacy_one = create_results_list(
    masked_diedIn_bert_original_spacy_one, diedIn_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_diedIn_bert_top1_spacy_one}')

results_list_diedIn_bert_top1_one, _, average_diedIn_bert_top1_one = create_results_list(
    masked_diedIn_bert_original_one, diedIn_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_diedIn_bert_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.436
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.44


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.44


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.437


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.486
Avg Precision K = 1 of died In Normal: 0.465


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_diedIn_bert_top5_ranking_spacy_one, _, average_diedIn_bert_top5_ranking_spacy_one = create_results_list(
    ranking_diedIn_bert_spacy_tokens_one, diedIn_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_diedIn_bert_top5_ranking_spacy_one}')

results_list_diedIn_bert_top5_ranking_normal_one, _, average_diedIn_bert_top5_ranking_normal_one = create_results_list(
    ranking_diedIn_bert_normal_tokens_one, diedIn_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_diedIn_bert_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_diedIn_bert_top5_borda_normal_one, _, average_diedIn_bert_top5_borda_normal_one = create_results_list(
    diedIn_bert_borda_tokens_score_one, diedIn_bert_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_diedIn_bert_top5_borda_normal_one}')
print('\n')

results_list_diedIn_bert_top5_borda_spacy_one, _, average_diedIn_bert_top5_borda_spacy_one = create_results_list(
    diedIn_bert_borda_spacy_tokens_score_one, diedIn_bert_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_diedIn_bert_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top5_spacy_one, _, average_diedIn_bert_top5_spacy_one = create_results_list(
    masked_diedIn_bert_original_spacy_one, diedIn_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_diedIn_bert_top5_spacy_one}')

results_list_diedIn_bert_top5_one, _, average_diedIn_bert_top5_one = create_results_list(
    masked_diedIn_bert_original_one, diedIn_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_diedIn_bert_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.687
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.74


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.733


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.684


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.702
Avg Precision K = 5 of died In Normal: 0.711


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_diedIn_bert_top10_ranking_spacy_one, _, average_diedIn_bert_top10_ranking_spacy_one = create_results_list(
    ranking_diedIn_bert_spacy_tokens_one, diedIn_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_diedIn_bert_top10_ranking_spacy_one}')

results_list_diedIn_bert_top10_ranking_normal_one, _, average_diedIn_bert_top10_ranking_normal_one = create_results_list(
    ranking_diedIn_bert_normal_tokens_one, diedIn_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_diedIn_bert_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_diedIn_bert_top10_borda_normal_one, _, average_diedIn_bert_top10_borda_normal_one = create_results_list(
    diedIn_bert_borda_tokens_score_one, diedIn_bert_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_diedIn_bert_top10_borda_normal_one}')
print('\n')

results_list_diedIn_bert_top10_borda_spacy_one, _, average_diedIn_bert_top10_borda_spacy_one = create_results_list(
    diedIn_bert_borda_spacy_tokens_score_one, diedIn_bert_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_diedIn_bert_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top10_spacy_one, _, average_diedIn_bert_top10_spacy_one = create_results_list(
    masked_diedIn_bert_original_spacy_one, diedIn_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_diedIn_bert_top10_spacy_one}')

results_list_diedIn_bert_top10_one, _, average_diedIn_bert_top10_one = create_results_list(
    masked_diedIn_bert_original_one, diedIn_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_diedIn_bert_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.772
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.816


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.81


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.771


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.751
Avg Precision K = 10 of died In Normal: 0.775


#### Worksfor

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_worksfor_bert_top1_ranking_spacy_one, _, average_worksfor_bert_top1_ranking_spacy_one = create_results_list(
    ranking_worksfor_bert_spacy_tokens_one, worksfor_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_worksfor_bert_top1_ranking_spacy_one}')

results_list_worksfor_bert_top1_ranking_normal_one, _, average_worksfor_bert_top1_ranking_normal_one = create_results_list(
    ranking_worksfor_bert_normal_tokens_one, worksfor_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_worksfor_bert_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_worksfor_bert_top1_borda_normal_one, _, average_worksfor_bert_top1_borda_normal_one = create_results_list(
    worksfor_bert_borda_tokens_score_one, worksfor_bert_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_worksfor_bert_top1_borda_normal_one}')
print('\n')

results_list_worksfor_bert_top1_borda_spacy_one, _, average_worksfor_bert_top1_borda_spacy_one = create_results_list(
    worksfor_bert_borda_spacy_tokens_score_one, worksfor_bert_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_worksfor_bert_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top1_spacy_one, _, average_worksfor_bert_top1_spacy_one = create_results_list(
    masked_worksfor_bert_original_spacy_one, worksfor_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_worksfor_bert_top1_spacy_one}')

results_list_worksfor_bert_top1_one, _, average_worksfor_bert_top1_one = create_results_list(
    masked_worksfor_bert_original_one, worksfor_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_worksfor_bert_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.604
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.591


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.583


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.599


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.604
Avg Precision K = 1 of died In Normal: 0.512


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_worksfor_bert_top5_ranking_spacy_one, _, average_worksfor_bert_top5_ranking_spacy_one = create_results_list(
    ranking_worksfor_bert_spacy_tokens_one, worksfor_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_worksfor_bert_top5_ranking_spacy_one}')

results_list_worksfor_bert_top5_ranking_normal_one, _, average_worksfor_bert_top5_ranking_normal_one = create_results_list(
    ranking_worksfor_bert_normal_tokens_one, worksfor_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_worksfor_bert_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_worksfor_bert_top5_borda_normal_one, _, average_worksfor_bert_top5_borda_normal_one = create_results_list(
    worksfor_bert_borda_tokens_score_one, worksfor_bert_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_worksfor_bert_top5_borda_normal_one}')
print('\n')

results_list_worksfor_bert_top5_borda_spacy_one, _, average_worksfor_bert_top5_borda_spacy_one = create_results_list(
    worksfor_bert_borda_spacy_tokens_score_one, worksfor_bert_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_worksfor_bert_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top5_spacy_one, _, average_worksfor_bert_top5_spacy_one = create_results_list(
    masked_worksfor_bert_original_spacy_one, worksfor_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_worksfor_bert_top5_spacy_one}')

results_list_worksfor_bert_top5_one, _, average_worksfor_bert_top5_one = create_results_list(
    masked_worksfor_bert_original_one, worksfor_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_worksfor_bert_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.836
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.831


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.839


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.836


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.802
Avg Precision K = 5 of died In Normal: 0.792


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_worksfor_bert_top10_ranking_spacy_one, _, average_worksfor_bert_top10_ranking_spacy_one = create_results_list(
    ranking_worksfor_bert_spacy_tokens_one, worksfor_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_worksfor_bert_top10_ranking_spacy_one}')

results_list_worksfor_bert_top10_ranking_normal_one, _, average_worksfor_bert_top10_ranking_normal_one = create_results_list(
    ranking_worksfor_bert_normal_tokens_one, worksfor_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_worksfor_bert_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_worksfor_bert_top10_borda_normal_one, _, average_worksfor_bert_top10_borda_normal_one = create_results_list(
    worksfor_bert_borda_tokens_score_one, worksfor_bert_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_worksfor_bert_top10_borda_normal_one}')
print('\n')

results_list_worksfor_bert_top10_borda_spacy_one, _, average_worksfor_bert_top10_borda_spacy_one = create_results_list(
    worksfor_bert_borda_spacy_tokens_score_one, worksfor_bert_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_worksfor_bert_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top10_spacy_one, _, average_worksfor_bert_top10_spacy_one = create_results_list(
    masked_worksfor_bert_original_spacy_one, worksfor_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_worksfor_bert_top10_spacy_one}')

results_list_worksfor_bert_top10_one, _, average_worksfor_bert_top10_one = create_results_list(
    masked_worksfor_bert_original_one, worksfor_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_worksfor_bert_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.876
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.921


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.918


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.876


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.821
Avg Precision K = 10 of died In Normal: 0.86


#### Capital

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_capital_bert_top1_ranking_spacy_one, _, average_capital_bert_top1_ranking_spacy_one = create_results_list(
    ranking_capital_bert_spacy_tokens_one, capital_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_capital_bert_top1_ranking_spacy_one}')

results_list_capital_bert_top1_ranking_normal_one, _, average_capital_bert_top1_ranking_normal_one = create_results_list(
    ranking_capital_bert_normal_tokens_one, capital_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_capital_bert_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_capital_bert_top1_borda_normal_one, _, average_capital_bert_top1_borda_normal_one = create_results_list(
    capital_bert_borda_tokens_score_one, capital_bert_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_capital_bert_top1_borda_normal_one}')
print('\n')

results_list_capital_bert_top1_borda_spacy_one, _, average_capital_bert_top1_borda_spacy_one = create_results_list(
    capital_bert_borda_spacy_tokens_score_one, capital_bert_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_capital_bert_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top1_spacy_one, _, average_capital_bert_top1_spacy_one = create_results_list(
    masked_capital_bert_original_spacy_one, capital_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_capital_bert_top1_spacy_one}')

results_list_capital_bert_top1_one, _, average_capital_bert_top1_one = create_results_list(
    masked_capital_bert_original_one, capital_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_capital_bert_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.343
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.41


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.41


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.341


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.455
Avg Precision K = 1 of died In Normal: 0.457


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_capital_bert_top5_ranking_spacy_one, _, average_capital_bert_top5_ranking_spacy_one = create_results_list(
    ranking_capital_bert_spacy_tokens_one, capital_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_capital_bert_top5_ranking_spacy_one}')

results_list_capital_bert_top5_ranking_normal_one, _, average_capital_bert_top5_ranking_normal_one = create_results_list(
    ranking_capital_bert_normal_tokens_one, capital_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_capital_bert_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_capital_bert_top5_borda_normal_one, _, average_capital_bert_top5_borda_normal_one = create_results_list(
    capital_bert_borda_tokens_score_one, capital_bert_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_capital_bert_top5_borda_normal_one}')
print('\n')

results_list_capital_bert_top5_borda_spacy_one, _, average_capital_bert_top5_borda_spacy_one = create_results_list(
    capital_bert_borda_spacy_tokens_score_one, capital_bert_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_capital_bert_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top5_spacy_one, _, average_capital_bert_top5_spacy_one = create_results_list(
    masked_capital_bert_original_spacy_one, capital_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_capital_bert_top5_spacy_one}')

results_list_capital_bert_top5_one, _, average_capital_bert_top5_one = create_results_list(
    masked_capital_bert_original_one, capital_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_capital_bert_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.593
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.675


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.672


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.596


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.638
Avg Precision K = 5 of died In Normal: 0.655


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_capital_bert_top10_ranking_spacy_one, _, average_capital_bert_top10_ranking_spacy_one = create_results_list(
    ranking_capital_bert_spacy_tokens_one, capital_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_capital_bert_top10_ranking_spacy_one}')

results_list_capital_bert_top10_ranking_normal_one, _, average_capital_bert_top10_ranking_normal_one = create_results_list(
    ranking_capital_bert_normal_tokens_one, capital_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_capital_bert_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_capital_bert_top10_borda_normal_one, _, average_capital_bert_top10_borda_normal_one = create_results_list(
    capital_bert_borda_tokens_score_one, capital_bert_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_capital_bert_top10_borda_normal_one}')
print('\n')

results_list_capital_bert_top10_borda_spacy_one, _, average_capital_bert_top10_borda_spacy_one = create_results_list(
    capital_bert_borda_spacy_tokens_score_one, capital_bert_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_capital_bert_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top10_spacy_one, _, average_capital_bert_top10_spacy_one = create_results_list(
    masked_capital_bert_original_spacy_one, capital_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_capital_bert_top10_spacy_one}')

results_list_capital_bert_top10_one, _, average_capital_bert_top10_one = create_results_list(
    masked_capital_bert_original_one, capital_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_capital_bert_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.659
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.74


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.742


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.658


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.668
Avg Precision K = 10 of died In Normal: 0.707


#### Citizen

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_citizen_bert_top1_ranking_spacy_one, _, average_citizen_bert_top1_ranking_spacy_one = create_results_list(
    ranking_citizen_bert_spacy_tokens_one, citizen_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_citizen_bert_top1_ranking_spacy_one}')

results_list_citizen_bert_top1_ranking_normal_one, _, average_citizen_bert_top1_ranking_normal_one = create_results_list(
    ranking_citizen_bert_normal_tokens_one, citizen_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_citizen_bert_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_citizen_bert_top1_borda_normal_one, _, average_citizen_bert_top1_borda_normal_one = create_results_list(
    citizen_bert_borda_tokens_score_one, citizen_bert_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_citizen_bert_top1_borda_normal_one}')
print('\n')

results_list_citizen_bert_top1_borda_spacy_one, _, average_citizen_bert_top1_borda_spacy_one = create_results_list(
    citizen_bert_borda_spacy_tokens_score_one, citizen_bert_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_citizen_bert_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top1_spacy_one, _, average_citizen_bert_top1_spacy_one = create_results_list(
    masked_citizen_bert_original_spacy_one, citizen_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_citizen_bert_top1_spacy_one}')

results_list_citizen_bert_top1_one, _, average_citizen_bert_top1_one = create_results_list(
    masked_citizen_bert_original_one, citizen_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_citizen_bert_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.449
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.646


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.645


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.451


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.414
Avg Precision K = 1 of died In Normal: 0.573


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_citizen_bert_top5_ranking_spacy_one, _, average_citizen_bert_top5_ranking_spacy_one = create_results_list(
    ranking_citizen_bert_spacy_tokens_one, citizen_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_citizen_bert_top5_ranking_spacy_one}')

results_list_citizen_bert_top5_ranking_normal_one, _, average_citizen_bert_top5_ranking_normal_one = create_results_list(
    ranking_citizen_bert_normal_tokens_one, citizen_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_citizen_bert_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_citizen_bert_top5_borda_normal_one, _, average_citizen_bert_top5_borda_normal_one = create_results_list(
    citizen_bert_borda_tokens_score_one, citizen_bert_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_citizen_bert_top5_borda_normal_one}')
print('\n')

results_list_citizen_bert_top5_borda_spacy_one, _, average_citizen_bert_top5_borda_spacy_one = create_results_list(
    citizen_bert_borda_spacy_tokens_score_one, citizen_bert_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_citizen_bert_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top5_spacy_one, _, average_citizen_bert_top5_spacy_one = create_results_list(
    masked_citizen_bert_original_spacy_one, citizen_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_citizen_bert_top5_spacy_one}')

results_list_citizen_bert_top5_one, _, average_citizen_bert_top5_one = create_results_list(
    masked_citizen_bert_original_one, citizen_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_citizen_bert_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.551
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.835


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.836


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.549


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.478
Avg Precision K = 5 of died In Normal: 0.722


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_citizen_bert_top10_ranking_spacy_one, _, average_citizen_bert_top10_ranking_spacy_one = create_results_list(
    ranking_citizen_bert_spacy_tokens_one, citizen_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_citizen_bert_top10_ranking_spacy_one}')

results_list_citizen_bert_top10_ranking_normal_one, _, average_citizen_bert_top10_ranking_normal_one = create_results_list(
    ranking_citizen_bert_normal_tokens_one, citizen_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_citizen_bert_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_citizen_bert_top10_borda_normal_one, _, average_citizen_bert_top10_borda_normal_one = create_results_list(
    citizen_bert_borda_tokens_score_one, citizen_bert_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_citizen_bert_top10_borda_normal_one}')
print('\n')

results_list_citizen_bert_top10_borda_spacy_one, _, average_citizen_bert_top10_borda_spacy_one = create_results_list(
    citizen_bert_borda_spacy_tokens_score_one, citizen_bert_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_citizen_bert_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top10_spacy_one, _, average_citizen_bert_top10_spacy_one = create_results_list(
    masked_citizen_bert_original_spacy_one, citizen_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_citizen_bert_top10_spacy_one}')

results_list_citizen_bert_top10_one, _, average_citizen_bert_top10_one = create_results_list(
    masked_citizen_bert_original_one, citizen_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_citizen_bert_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.568
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.885


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.884


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.567


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.482
Avg Precision K = 10 of died In Normal: 0.79


#### Language

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_language_bert_top1_ranking_spacy_one, _, average_language_bert_top1_ranking_spacy_one = create_results_list(
    ranking_language_bert_spacy_tokens_one, language_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_language_bert_top1_ranking_spacy_one}')

results_list_language_bert_top1_ranking_normal_one, _, average_language_bert_top1_ranking_normal_one = create_results_list(
    ranking_language_bert_normal_tokens_one, language_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_language_bert_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_language_bert_top1_borda_normal_one, _, average_language_bert_top1_borda_normal_one = create_results_list(
    language_bert_borda_tokens_score_one, language_bert_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_language_bert_top1_borda_normal_one}')
print('\n')

results_list_language_bert_top1_borda_spacy_one, _, average_language_bert_top1_borda_spacy_one = create_results_list(
    language_bert_borda_spacy_tokens_score_one, language_bert_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_language_bert_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_language_bert_top1_spacy_one, _, average_language_bert_top1_spacy_one = create_results_list(
    masked_language_bert_original_spacy_one, language_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_language_bert_top1_spacy_one}')

results_list_language_bert_top1_one, _, average_language_bert_top1_one = create_results_list(
    masked_language_bert_original_one, language_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_language_bert_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.443
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.683


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.682


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.439


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.657
Avg Precision K = 1 of died In Normal: 0.709


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_language_bert_top5_ranking_spacy_one, _, average_language_bert_top5_ranking_spacy_one = create_results_list(
    ranking_language_bert_spacy_tokens_one, language_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_language_bert_top5_ranking_spacy_one}')

results_list_language_bert_top5_ranking_normal_one, _, average_language_bert_top5_ranking_normal_one = create_results_list(
    ranking_language_bert_normal_tokens_one, language_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_language_bert_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_language_bert_top5_borda_normal_one, _, average_language_bert_top5_borda_normal_one = create_results_list(
    language_bert_borda_tokens_score_one, language_bert_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_language_bert_top5_borda_normal_one}')
print('\n')

results_list_language_bert_top5_borda_spacy_one, _, average_language_bert_top5_borda_spacy_one = create_results_list(
    language_bert_borda_spacy_tokens_score_one, language_bert_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_language_bert_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_language_bert_top5_spacy_one, _, average_language_bert_top5_spacy_one = create_results_list(
    masked_language_bert_original_spacy_one, language_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_language_bert_top5_spacy_one}')

results_list_language_bert_top5_one, _, average_language_bert_top5_one = create_results_list(
    masked_language_bert_original_one, language_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_language_bert_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.602
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.892


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.89


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.601


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.766
Avg Precision K = 5 of died In Normal: 0.871


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_language_bert_top10_ranking_spacy_one, _, average_language_bert_top10_ranking_spacy_one = create_results_list(
    ranking_language_bert_spacy_tokens_one, language_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_language_bert_top10_ranking_spacy_one}')

results_list_language_bert_top10_ranking_normal_one, _, average_language_bert_top10_ranking_normal_one = create_results_list(
    ranking_language_bert_normal_tokens_one, language_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_language_bert_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_language_bert_top10_borda_normal_one, _, average_language_bert_top10_borda_normal_one = create_results_list(
    language_bert_borda_tokens_score_one, language_bert_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_language_bert_top10_borda_normal_one}')
print('\n')

results_list_language_bert_top10_borda_spacy_one, _, average_language_bert_top10_borda_spacy_one = create_results_list(
    language_bert_borda_spacy_tokens_score_one, language_bert_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_language_bert_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_language_bert_top10_spacy_one, _, average_language_bert_top10_spacy_one = create_results_list(
    masked_language_bert_original_spacy_one, language_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_language_bert_top10_spacy_one}')

results_list_language_bert_top10_one, _, average_language_bert_top10_one = create_results_list(
    masked_language_bert_original_one, language_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_language_bert_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.642
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.939


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.938


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.64


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.771
Avg Precision K = 10 of died In Normal: 0.907


### Roberta

#### BornIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_bornIn_roberta_top1_ranking_spacy_one, _, average_bornIn_roberta_top1_ranking_spacy_one = create_results_list(
    ranking_bornIn_roberta_spacy_tokens_one, bornIn_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_bornIn_roberta_top1_ranking_spacy_one}')

results_list_bornIn_roberta_top1_ranking_normal_one, _, average_bornIn_roberta_top1_ranking_normal_one = create_results_list(
    ranking_bornIn_roberta_normal_tokens_one, bornIn_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_bornIn_roberta_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_bornIn_roberta_top1_borda_normal_one, _, average_bornIn_roberta_top1_borda_normal_one = create_results_list(
    bornIn_roberta_borda_tokens_score_one, bornIn_roberta_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_bornIn_roberta_top1_borda_normal_one}')
print('\n')

results_list_bornIn_roberta_top1_borda_spacy_one, _, average_bornIn_roberta_top1_borda_spacy_one = create_results_list(
    bornIn_roberta_borda_spacy_tokens_score_one, bornIn_roberta_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_bornIn_roberta_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_bornIn_roberta_top1_spacy_one, _, average_bornIn_roberta_top1_spacy_one = create_results_list(
    masked_bornIn_roberta_original_spacy_one, bornIn_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_bornIn_roberta_top1_spacy_one}')

results_list_bornIn_roberta_top1_one, _, average_bornIn_roberta_top1_one = create_results_list(
    masked_bornIn_roberta_original_one, bornIn_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_bornIn_roberta_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.239
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.28


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.272


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.242


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.306
Avg Precision K = 1 of died In Normal: 0.298


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_bornIn_roberta_top5_ranking_spacy_one, _, average_bornIn_roberta_top5_ranking_spacy_one = create_results_list(
    ranking_bornIn_roberta_spacy_tokens_one, bornIn_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_bornIn_roberta_top5_ranking_spacy_one}')

results_list_bornIn_roberta_top5_ranking_normal_one, _, average_bornIn_roberta_top5_ranking_normal_one = create_results_list(
    ranking_bornIn_roberta_normal_tokens_one, bornIn_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_bornIn_roberta_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_bornIn_roberta_top5_borda_normal_one, _, average_bornIn_roberta_top5_borda_normal_one = create_results_list(
    bornIn_roberta_borda_tokens_score_one, bornIn_roberta_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_bornIn_roberta_top5_borda_normal_one}')
print('\n')

results_list_bornIn_roberta_top5_borda_spacy_one, _, average_bornIn_roberta_top5_borda_spacy_one = create_results_list(
    bornIn_roberta_borda_spacy_tokens_score_one, bornIn_roberta_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_bornIn_roberta_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_bornIn_roberta_top5_spacy_one, _, average_bornIn_roberta_top5_spacy_one = create_results_list(
    masked_bornIn_roberta_original_spacy_one, bornIn_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_bornIn_roberta_top5_spacy_one}')

results_list_bornIn_roberta_top5_one, _, average_bornIn_roberta_top5_one = create_results_list(
    masked_bornIn_roberta_original_one, bornIn_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_bornIn_roberta_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.486
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.504


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.497


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.485


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.508
Avg Precision K = 5 of died In Normal: 0.505


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_bornIn_roberta_top10_ranking_spacy_one, _, average_bornIn_roberta_top10_ranking_spacy_one = create_results_list(
    ranking_bornIn_roberta_spacy_tokens_one, bornIn_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_bornIn_roberta_top10_ranking_spacy_one}')

results_list_bornIn_roberta_top10_ranking_normal_one, _, average_bornIn_roberta_top10_ranking_normal_one = create_results_list(
    ranking_bornIn_roberta_normal_tokens_one, bornIn_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_bornIn_roberta_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_bornIn_roberta_top10_borda_normal_one, _, average_bornIn_roberta_top10_borda_normal_one = create_results_list(
    bornIn_roberta_borda_tokens_score_one, bornIn_roberta_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_bornIn_roberta_top10_borda_normal_one}')
print('\n')

results_list_bornIn_roberta_top10_borda_spacy_one, _, average_bornIn_roberta_top10_borda_spacy_one = create_results_list(
    bornIn_roberta_borda_spacy_tokens_score_one, bornIn_roberta_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_bornIn_roberta_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_bornIn_roberta_top10_spacy_one, _, average_bornIn_roberta_top10_spacy_one = create_results_list(
    masked_bornIn_roberta_original_spacy_one, bornIn_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_bornIn_roberta_top10_spacy_one}')

results_list_bornIn_roberta_top10_one, _, average_bornIn_roberta_top10_one = create_results_list(
    masked_bornIn_roberta_original_one, bornIn_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_bornIn_roberta_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.55
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.577


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.574


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.541


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.552
Avg Precision K = 10 of died In Normal: 0.558


#### DiedIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_diedIn_roberta_top1_ranking_spacy_one, _, average_diedIn_roberta_top1_ranking_spacy_one = create_results_list(
    ranking_diedIn_roberta_spacy_tokens_one, diedIn_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_diedIn_roberta_top1_ranking_spacy_one}')

results_list_diedIn_roberta_top1_ranking_normal_one, _, average_diedIn_roberta_top1_ranking_normal_one = create_results_list(
    ranking_diedIn_roberta_normal_tokens_one, diedIn_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_diedIn_roberta_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_diedIn_roberta_top1_borda_normal_one, _, average_diedIn_roberta_top1_borda_normal_one = create_results_list(
    diedIn_roberta_borda_tokens_score_one, diedIn_roberta_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_diedIn_roberta_top1_borda_normal_one}')
print('\n')

results_list_diedIn_roberta_top1_borda_spacy_one, _, average_diedIn_roberta_top1_borda_spacy_one = create_results_list(
    diedIn_roberta_borda_spacy_tokens_score_one, diedIn_roberta_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_diedIn_roberta_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_diedIn_roberta_top1_spacy_one, _, average_diedIn_roberta_top1_spacy_one = create_results_list(
    masked_diedIn_roberta_original_spacy_one, diedIn_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_diedIn_roberta_top1_spacy_one}')

results_list_diedIn_roberta_top1_one, _, average_diedIn_roberta_top1_one = create_results_list(
    masked_diedIn_roberta_original_one, diedIn_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_diedIn_roberta_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.382
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.396


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.394


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.371


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.423
Avg Precision K = 1 of died In Normal: 0.407


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_diedIn_roberta_top5_ranking_spacy_one, _, average_diedIn_roberta_top5_ranking_spacy_one = create_results_list(
    ranking_diedIn_roberta_spacy_tokens_one, diedIn_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_diedIn_roberta_top5_ranking_spacy_one}')

results_list_diedIn_roberta_top5_ranking_normal_one, _, average_diedIn_roberta_top5_ranking_normal_one = create_results_list(
    ranking_diedIn_roberta_normal_tokens_one, diedIn_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_diedIn_roberta_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_diedIn_roberta_top5_borda_normal_one, _, average_diedIn_roberta_top5_borda_normal_one = create_results_list(
    diedIn_roberta_borda_tokens_score_one, diedIn_roberta_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_diedIn_roberta_top5_borda_normal_one}')
print('\n')

results_list_diedIn_roberta_top5_borda_spacy_one, _, average_diedIn_roberta_top5_borda_spacy_one = create_results_list(
    diedIn_roberta_borda_spacy_tokens_score_one, diedIn_roberta_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_diedIn_roberta_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_diedIn_roberta_top5_spacy_one, _, average_diedIn_roberta_top5_spacy_one = create_results_list(
    masked_diedIn_roberta_original_spacy_one, diedIn_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_diedIn_roberta_top5_spacy_one}')

results_list_diedIn_roberta_top5_one, _, average_diedIn_roberta_top5_one = create_results_list(
    masked_diedIn_roberta_original_one, diedIn_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_diedIn_roberta_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.6
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.633


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.631


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.598


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.619
Avg Precision K = 5 of died In Normal: 0.608


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_diedIn_roberta_top10_ranking_spacy_one, _, average_diedIn_roberta_top10_ranking_spacy_one = create_results_list(
    ranking_diedIn_roberta_spacy_tokens_one, diedIn_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_diedIn_roberta_top10_ranking_spacy_one}')

results_list_diedIn_roberta_top10_ranking_normal_one, _, average_diedIn_roberta_top10_ranking_normal_one = create_results_list(
    ranking_diedIn_roberta_normal_tokens_one, diedIn_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_diedIn_roberta_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_diedIn_roberta_top10_borda_normal_one, _, average_diedIn_roberta_top10_borda_normal_one = create_results_list(
    diedIn_roberta_borda_tokens_score_one, diedIn_roberta_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_diedIn_roberta_top10_borda_normal_one}')
print('\n')

results_list_diedIn_roberta_top10_borda_spacy_one, _, average_diedIn_roberta_top10_borda_spacy_one = create_results_list(
    diedIn_roberta_borda_spacy_tokens_score_one, diedIn_roberta_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_diedIn_roberta_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_diedIn_roberta_top10_spacy_one, _, average_diedIn_roberta_top10_spacy_one = create_results_list(
    masked_diedIn_roberta_original_spacy_one, diedIn_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_diedIn_roberta_top10_spacy_one}')

results_list_diedIn_roberta_top10_one, _, average_diedIn_roberta_top10_one = create_results_list(
    masked_diedIn_roberta_original_one, diedIn_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_diedIn_roberta_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.671
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.705


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.693


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.672


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.661
Avg Precision K = 10 of died In Normal: 0.682


#### Worksfor

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_worksfor_roberta_top1_ranking_spacy_one, _, average_worksfor_roberta_top1_ranking_spacy_one = create_results_list(
    ranking_worksfor_roberta_spacy_tokens_one, worksfor_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_worksfor_roberta_top1_ranking_spacy_one}')

results_list_worksfor_roberta_top1_ranking_normal_one, _, average_worksfor_roberta_top1_ranking_normal_one = create_results_list(
    ranking_worksfor_roberta_normal_tokens_one, worksfor_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_worksfor_roberta_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_worksfor_roberta_top1_borda_normal_one, _, average_worksfor_roberta_top1_borda_normal_one = create_results_list(
    worksfor_roberta_borda_tokens_score_one, worksfor_roberta_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_worksfor_roberta_top1_borda_normal_one}')
print('\n')

results_list_worksfor_roberta_top1_borda_spacy_one, _, average_worksfor_roberta_top1_borda_spacy_one = create_results_list(
    worksfor_roberta_borda_spacy_tokens_score_one, worksfor_roberta_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_worksfor_roberta_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_worksfor_roberta_top1_spacy_one, _, average_worksfor_roberta_top1_spacy_one = create_results_list(
    masked_worksfor_roberta_original_spacy_one, worksfor_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_worksfor_roberta_top1_spacy_one}')

results_list_worksfor_roberta_top1_one, _, average_worksfor_roberta_top1_one = create_results_list(
    masked_worksfor_roberta_original_one, worksfor_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_worksfor_roberta_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.683
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.654


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.646


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.675


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.673
Avg Precision K = 1 of died In Normal: 0.604


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_worksfor_roberta_top5_ranking_spacy_one, _, average_worksfor_roberta_top5_ranking_spacy_one = create_results_list(
    ranking_worksfor_roberta_spacy_tokens_one, worksfor_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_worksfor_roberta_top5_ranking_spacy_one}')

results_list_worksfor_roberta_top5_ranking_normal_one, _, average_worksfor_roberta_top5_ranking_normal_one = create_results_list(
    ranking_worksfor_roberta_normal_tokens_one, worksfor_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_worksfor_roberta_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_worksfor_roberta_top5_borda_normal_one, _, average_worksfor_roberta_top5_borda_normal_one = create_results_list(
    worksfor_roberta_borda_tokens_score_one, worksfor_roberta_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_worksfor_roberta_top5_borda_normal_one}')
print('\n')

results_list_worksfor_roberta_top5_borda_spacy_one, _, average_worksfor_roberta_top5_borda_spacy_one = create_results_list(
    worksfor_roberta_borda_spacy_tokens_score_one, worksfor_roberta_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_worksfor_roberta_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_worksfor_roberta_top5_spacy_one, _, average_worksfor_roberta_top5_spacy_one = create_results_list(
    masked_worksfor_roberta_original_spacy_one, worksfor_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_worksfor_roberta_top5_spacy_one}')

results_list_worksfor_roberta_top5_one, _, average_worksfor_roberta_top5_one = create_results_list(
    masked_worksfor_roberta_original_one, worksfor_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_worksfor_roberta_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.836
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.86


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.863


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.836


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.834
Avg Precision K = 5 of died In Normal: 0.826


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_worksfor_roberta_top10_ranking_spacy_one, _, average_worksfor_roberta_top10_ranking_spacy_one = create_results_list(
    ranking_worksfor_roberta_spacy_tokens_one, worksfor_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_worksfor_roberta_top10_ranking_spacy_one}')

results_list_worksfor_roberta_top10_ranking_normal_one, _, average_worksfor_roberta_top10_ranking_normal_one = create_results_list(
    ranking_worksfor_roberta_normal_tokens_one, worksfor_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_worksfor_roberta_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_worksfor_roberta_top10_borda_normal_one, _, average_worksfor_roberta_top10_borda_normal_one = create_results_list(
    worksfor_roberta_borda_tokens_score_one, worksfor_roberta_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_worksfor_roberta_top10_borda_normal_one}')
print('\n')

results_list_worksfor_roberta_top10_borda_spacy_one, _, average_worksfor_roberta_top10_borda_spacy_one = create_results_list(
    worksfor_roberta_borda_spacy_tokens_score_one, worksfor_roberta_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_worksfor_roberta_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_worksfor_roberta_top10_spacy_one, _, average_worksfor_roberta_top10_spacy_one = create_results_list(
    masked_worksfor_roberta_original_spacy_one, worksfor_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_worksfor_roberta_top10_spacy_one}')

results_list_worksfor_roberta_top10_one, _, average_worksfor_roberta_top10_one = create_results_list(
    masked_worksfor_roberta_original_one, worksfor_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_worksfor_roberta_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.876
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.905


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.905


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.879


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.852
Avg Precision K = 10 of died In Normal: 0.894


#### Capital

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_capital_roberta_top1_ranking_spacy_one, _, average_capital_roberta_top1_ranking_spacy_one = create_results_list(
    ranking_capital_roberta_spacy_tokens_one, capital_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_capital_roberta_top1_ranking_spacy_one}')

results_list_capital_roberta_top1_ranking_normal_one, _, average_capital_roberta_top1_ranking_normal_one = create_results_list(
    ranking_capital_roberta_normal_tokens_one, capital_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_capital_roberta_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_capital_roberta_top1_borda_normal_one, _, average_capital_roberta_top1_borda_normal_one = create_results_list(
    capital_roberta_borda_tokens_score_one, capital_roberta_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_capital_roberta_top1_borda_normal_one}')
print('\n')

results_list_capital_roberta_top1_borda_spacy_one, _, average_capital_roberta_top1_borda_spacy_one = create_results_list(
    capital_roberta_borda_spacy_tokens_score_one, capital_roberta_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_capital_roberta_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_capital_roberta_top1_spacy_one, _, average_capital_roberta_top1_spacy_one = create_results_list(
    masked_capital_roberta_original_spacy_one, capital_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_capital_roberta_top1_spacy_one}')

results_list_capital_roberta_top1_one, _, average_capital_roberta_top1_one = create_results_list(
    masked_capital_roberta_original_one, capital_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_capital_roberta_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.273
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.306


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.293


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.273


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.33
Avg Precision K = 1 of died In Normal: 0.326


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_capital_roberta_top5_ranking_spacy_one, _, average_capital_roberta_top5_ranking_spacy_one = create_results_list(
    ranking_capital_roberta_spacy_tokens_one, capital_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_capital_roberta_top5_ranking_spacy_one}')

results_list_capital_roberta_top5_ranking_normal_one, _, average_capital_roberta_top5_ranking_normal_one = create_results_list(
    ranking_capital_roberta_normal_tokens_one, capital_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_capital_roberta_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_capital_roberta_top5_borda_normal_one, _, average_capital_roberta_top5_borda_normal_one = create_results_list(
    capital_roberta_borda_tokens_score_one, capital_roberta_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_capital_roberta_top5_borda_normal_one}')
print('\n')

results_list_capital_roberta_top5_borda_spacy_one, _, average_capital_roberta_top5_borda_spacy_one = create_results_list(
    capital_roberta_borda_spacy_tokens_score_one, capital_roberta_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_capital_roberta_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_capital_roberta_top5_spacy_one, _, average_capital_roberta_top5_spacy_one = create_results_list(
    masked_capital_roberta_original_spacy_one, capital_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_capital_roberta_top5_spacy_one}')

results_list_capital_roberta_top5_one, _, average_capital_roberta_top5_one = create_results_list(
    masked_capital_roberta_original_one, capital_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_capital_roberta_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.418
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.483


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.476


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.418


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.43
Avg Precision K = 5 of died In Normal: 0.447


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_capital_roberta_top10_ranking_spacy_one, _, average_capital_roberta_top10_ranking_spacy_one = create_results_list(
    ranking_capital_roberta_spacy_tokens_one, capital_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_capital_roberta_top10_ranking_spacy_one}')

results_list_capital_roberta_top10_ranking_normal_one, _, average_capital_roberta_top10_ranking_normal_one = create_results_list(
    ranking_capital_roberta_normal_tokens_one, capital_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_capital_roberta_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_capital_roberta_top10_borda_normal_one, _, average_capital_roberta_top10_borda_normal_one = create_results_list(
    capital_roberta_borda_tokens_score_one, capital_roberta_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_capital_roberta_top10_borda_normal_one}')
print('\n')

results_list_capital_roberta_top10_borda_spacy_one, _, average_capital_roberta_top10_borda_spacy_one = create_results_list(
    capital_roberta_borda_spacy_tokens_score_one, capital_roberta_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_capital_roberta_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_capital_roberta_top10_spacy_one, _, average_capital_roberta_top10_spacy_one = create_results_list(
    masked_capital_roberta_original_spacy_one, capital_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_capital_roberta_top10_spacy_one}')

results_list_capital_roberta_top10_one, _, average_capital_roberta_top10_one = create_results_list(
    masked_capital_roberta_original_one, capital_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_capital_roberta_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.45
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.521


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.522


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.453


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.449
Avg Precision K = 10 of died In Normal: 0.486


#### Citizen

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_citizen_roberta_top1_ranking_spacy_one, _, average_citizen_roberta_top1_ranking_spacy_one = create_results_list(
    ranking_citizen_roberta_spacy_tokens_one, citizen_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_citizen_roberta_top1_ranking_spacy_one}')

results_list_citizen_roberta_top1_ranking_normal_one, _, average_citizen_roberta_top1_ranking_normal_one = create_results_list(
    ranking_citizen_roberta_normal_tokens_one, citizen_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_citizen_roberta_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_citizen_roberta_top1_borda_normal_one, _, average_citizen_roberta_top1_borda_normal_one = create_results_list(
    citizen_roberta_borda_tokens_score_one, citizen_roberta_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_citizen_roberta_top1_borda_normal_one}')
print('\n')

results_list_citizen_roberta_top1_borda_spacy_one, _, average_citizen_roberta_top1_borda_spacy_one = create_results_list(
    citizen_roberta_borda_spacy_tokens_score_one, citizen_roberta_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_citizen_roberta_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_citizen_roberta_top1_spacy_one, _, average_citizen_roberta_top1_spacy_one = create_results_list(
    masked_citizen_roberta_original_spacy_one, citizen_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_citizen_roberta_top1_spacy_one}')

results_list_citizen_roberta_top1_one, _, average_citizen_roberta_top1_one = create_results_list(
    masked_citizen_roberta_original_one, citizen_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_citizen_roberta_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.44
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.632


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.63


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.439


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.405
Avg Precision K = 1 of died In Normal: 0.551


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_citizen_roberta_top5_ranking_spacy_one, _, average_citizen_roberta_top5_ranking_spacy_one = create_results_list(
    ranking_citizen_roberta_spacy_tokens_one, citizen_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_citizen_roberta_top5_ranking_spacy_one}')

results_list_citizen_roberta_top5_ranking_normal_one, _, average_citizen_roberta_top5_ranking_normal_one = create_results_list(
    ranking_citizen_roberta_normal_tokens_one, citizen_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_citizen_roberta_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_citizen_roberta_top5_borda_normal_one, _, average_citizen_roberta_top5_borda_normal_one = create_results_list(
    citizen_roberta_borda_tokens_score_one, citizen_roberta_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_citizen_roberta_top5_borda_normal_one}')
print('\n')

results_list_citizen_roberta_top5_borda_spacy_one, _, average_citizen_roberta_top5_borda_spacy_one = create_results_list(
    citizen_roberta_borda_spacy_tokens_score_one, citizen_roberta_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_citizen_roberta_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_citizen_roberta_top5_spacy_one, _, average_citizen_roberta_top5_spacy_one = create_results_list(
    masked_citizen_roberta_original_spacy_one, citizen_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_citizen_roberta_top5_spacy_one}')

results_list_citizen_roberta_top5_one, _, average_citizen_roberta_top5_one = create_results_list(
    masked_citizen_roberta_original_one, citizen_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_citizen_roberta_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.54
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.824


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.828


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.541


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.472
Avg Precision K = 5 of died In Normal: 0.707


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_citizen_roberta_top10_ranking_spacy_one, _, average_citizen_roberta_top10_ranking_spacy_one = create_results_list(
    ranking_citizen_roberta_spacy_tokens_one, citizen_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_citizen_roberta_top10_ranking_spacy_one}')

results_list_citizen_roberta_top10_ranking_normal_one, _, average_citizen_roberta_top10_ranking_normal_one = create_results_list(
    ranking_citizen_roberta_normal_tokens_one, citizen_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_citizen_roberta_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_citizen_roberta_top10_borda_normal_one, _, average_citizen_roberta_top10_borda_normal_one = create_results_list(
    citizen_roberta_borda_tokens_score_one, citizen_roberta_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_citizen_roberta_top10_borda_normal_one}')
print('\n')

results_list_citizen_roberta_top10_borda_spacy_one, _, average_citizen_roberta_top10_borda_spacy_one = create_results_list(
    citizen_roberta_borda_spacy_tokens_score_one, citizen_roberta_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_citizen_roberta_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_citizen_roberta_top10_spacy_one, _, average_citizen_roberta_top10_spacy_one = create_results_list(
    masked_citizen_roberta_original_spacy_one, citizen_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_citizen_roberta_top10_spacy_one}')

results_list_citizen_roberta_top10_one, _, average_citizen_roberta_top10_one = create_results_list(
    masked_citizen_roberta_original_one, citizen_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_citizen_roberta_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.56
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.871


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.869


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.56


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.473
Avg Precision K = 10 of died In Normal: 0.773


#### Language

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_language_roberta_top1_ranking_spacy_one, _, average_language_roberta_top1_ranking_spacy_one = create_results_list(
    ranking_language_roberta_spacy_tokens_one, language_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_language_roberta_top1_ranking_spacy_one}')

results_list_language_roberta_top1_ranking_normal_one, _, average_language_roberta_top1_ranking_normal_one = create_results_list(
    ranking_language_roberta_normal_tokens_one, language_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_language_roberta_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_language_roberta_top1_borda_normal_one, _, average_language_roberta_top1_borda_normal_one = create_results_list(
    language_roberta_borda_tokens_score_one, language_roberta_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_language_roberta_top1_borda_normal_one}')
print('\n')

results_list_language_roberta_top1_borda_spacy_one, _, average_language_roberta_top1_borda_spacy_one = create_results_list(
    language_roberta_borda_spacy_tokens_score_one, language_roberta_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_language_roberta_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_language_roberta_top1_spacy_one, _, average_language_roberta_top1_spacy_one = create_results_list(
    masked_language_roberta_original_spacy_one, language_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_language_roberta_top1_spacy_one}')

results_list_language_roberta_top1_one, _, average_language_roberta_top1_one = create_results_list(
    masked_language_roberta_original_one, language_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_language_roberta_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.406
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.603


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.594


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.395


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.59
Avg Precision K = 1 of died In Normal: 0.619


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_language_roberta_top5_ranking_spacy_one, _, average_language_roberta_top5_ranking_spacy_one = create_results_list(
    ranking_language_roberta_spacy_tokens_one, language_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_language_roberta_top5_ranking_spacy_one}')

results_list_language_roberta_top5_ranking_normal_one, _, average_language_roberta_top5_ranking_normal_one = create_results_list(
    ranking_language_roberta_normal_tokens_one, language_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_language_roberta_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_language_roberta_top5_borda_normal_one, _, average_language_roberta_top5_borda_normal_one = create_results_list(
    language_roberta_borda_tokens_score_one, language_roberta_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_language_roberta_top5_borda_normal_one}')
print('\n')

results_list_language_roberta_top5_borda_spacy_one, _, average_language_roberta_top5_borda_spacy_one = create_results_list(
    language_roberta_borda_spacy_tokens_score_one, language_roberta_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_language_roberta_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_language_roberta_top5_spacy_one, _, average_language_roberta_top5_spacy_one = create_results_list(
    masked_language_roberta_original_spacy_one, language_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_language_roberta_top5_spacy_one}')

results_list_language_roberta_top5_one, _, average_language_roberta_top5_one = create_results_list(
    masked_language_roberta_original_one, language_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_language_roberta_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.568
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.831


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.828


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.57


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.704
Avg Precision K = 5 of died In Normal: 0.795


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_language_roberta_top10_ranking_spacy_one, _, average_language_roberta_top10_ranking_spacy_one = create_results_list(
    ranking_language_roberta_spacy_tokens_one, language_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_language_roberta_top10_ranking_spacy_one}')

results_list_language_roberta_top10_ranking_normal_one, _, average_language_roberta_top10_ranking_normal_one = create_results_list(
    ranking_language_roberta_normal_tokens_one, language_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_language_roberta_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_language_roberta_top10_borda_normal_one, _, average_language_roberta_top10_borda_normal_one = create_results_list(
    language_roberta_borda_tokens_score_one, language_roberta_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_language_roberta_top10_borda_normal_one}')
print('\n')

results_list_language_roberta_top10_borda_spacy_one, _, average_language_roberta_top10_borda_spacy_one = create_results_list(
    language_roberta_borda_spacy_tokens_score_one, language_roberta_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_language_roberta_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_language_roberta_top10_spacy_one, _, average_language_roberta_top10_spacy_one = create_results_list(
    masked_language_roberta_original_spacy_one, language_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_language_roberta_top10_spacy_one}')

results_list_language_roberta_top10_one, _, average_language_roberta_top10_one = create_results_list(
    masked_language_roberta_original_one, language_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_language_roberta_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.602
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.871


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.875


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.602


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.709
Avg Precision K = 10 of died In Normal: 0.838


### Electra

#### BornIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_bornIn_electra_top1_ranking_spacy_one, _, average_bornIn_electra_top1_ranking_spacy_one = create_results_list(
    ranking_bornIn_electra_spacy_tokens_one, bornIn_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_bornIn_electra_top1_ranking_spacy_one}')

results_list_bornIn_electra_top1_ranking_normal_one, _, average_bornIn_electra_top1_ranking_normal_one = create_results_list(
    ranking_bornIn_electra_normal_tokens_one, bornIn_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_bornIn_electra_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_bornIn_electra_top1_borda_normal_one, _, average_bornIn_electra_top1_borda_normal_one = create_results_list(
    bornIn_electra_borda_tokens_score_one, bornIn_electra_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_bornIn_electra_top1_borda_normal_one}')
print('\n')

results_list_bornIn_electra_top1_borda_spacy_one, _, average_bornIn_electra_top1_borda_spacy_one = create_results_list(
    bornIn_electra_borda_spacy_tokens_score_one, bornIn_electra_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_bornIn_electra_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_bornIn_electra_top1_spacy_one, _, average_bornIn_electra_top1_spacy_one = create_results_list(
    masked_bornIn_electra_original_spacy_one, bornIn_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_bornIn_electra_top1_spacy_one}')

results_list_bornIn_electra_top1_one, _, average_bornIn_electra_top1_one = create_results_list(
    masked_bornIn_electra_original_one, bornIn_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_bornIn_electra_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.145
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.15


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.15


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.141


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.193
Avg Precision K = 1 of died In Normal: 0.182


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_bornIn_electra_top5_ranking_spacy_one, _, average_bornIn_electra_top5_ranking_spacy_one = create_results_list(
    ranking_bornIn_electra_spacy_tokens_one, bornIn_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_bornIn_electra_top5_ranking_spacy_one}')

results_list_bornIn_electra_top5_ranking_normal_one, _, average_bornIn_electra_top5_ranking_normal_one = create_results_list(
    ranking_bornIn_electra_normal_tokens_one, bornIn_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_bornIn_electra_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_bornIn_electra_top5_borda_normal_one, _, average_bornIn_electra_top5_borda_normal_one = create_results_list(
    bornIn_electra_borda_tokens_score_one, bornIn_electra_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_bornIn_electra_top5_borda_normal_one}')
print('\n')

results_list_bornIn_electra_top5_borda_spacy_one, _, average_bornIn_electra_top5_borda_spacy_one = create_results_list(
    bornIn_electra_borda_spacy_tokens_score_one, bornIn_electra_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_bornIn_electra_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_bornIn_electra_top5_spacy_one, _, average_bornIn_electra_top5_spacy_one = create_results_list(
    masked_bornIn_electra_original_spacy_one, bornIn_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_bornIn_electra_top5_spacy_one}')

results_list_bornIn_electra_top5_one, _, average_bornIn_electra_top5_one = create_results_list(
    masked_bornIn_electra_original_one, bornIn_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_bornIn_electra_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.403
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.422


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.413


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.393


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.413
Avg Precision K = 5 of died In Normal: 0.408


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_bornIn_electra_top10_ranking_spacy_one, _, average_bornIn_electra_top10_ranking_spacy_one = create_results_list(
    ranking_bornIn_electra_spacy_tokens_one, bornIn_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_bornIn_electra_top10_ranking_spacy_one}')

results_list_bornIn_electra_top10_ranking_normal_one, _, average_bornIn_electra_top10_ranking_normal_one = create_results_list(
    ranking_bornIn_electra_normal_tokens_one, bornIn_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_bornIn_electra_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_bornIn_electra_top10_borda_normal_one, _, average_bornIn_electra_top10_borda_normal_one = create_results_list(
    bornIn_electra_borda_tokens_score_one, bornIn_electra_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_bornIn_electra_top10_borda_normal_one}')
print('\n')

results_list_bornIn_electra_top10_borda_spacy_one, _, average_bornIn_electra_top10_borda_spacy_one = create_results_list(
    bornIn_electra_borda_spacy_tokens_score_one, bornIn_electra_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_bornIn_electra_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_bornIn_electra_top10_spacy_one, _, average_bornIn_electra_top10_spacy_one = create_results_list(
    masked_bornIn_electra_original_spacy_one, bornIn_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_bornIn_electra_top10_spacy_one}')

results_list_bornIn_electra_top10_one, _, average_bornIn_electra_top10_one = create_results_list(
    masked_bornIn_electra_original_one, bornIn_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_bornIn_electra_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.506
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.518


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.517


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.494


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.479
Avg Precision K = 10 of died In Normal: 0.481


#### DiedIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_diedIn_electra_top1_ranking_spacy_one, _, average_diedIn_electra_top1_ranking_spacy_one = create_results_list(
    ranking_diedIn_electra_spacy_tokens_one, diedIn_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_diedIn_electra_top1_ranking_spacy_one}')

results_list_diedIn_electra_top1_ranking_normal_one, _, average_diedIn_electra_top1_ranking_normal_one = create_results_list(
    ranking_diedIn_electra_normal_tokens_one, diedIn_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_diedIn_electra_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_diedIn_electra_top1_borda_normal_one, _, average_diedIn_electra_top1_borda_normal_one = create_results_list(
    diedIn_electra_borda_tokens_score_one, diedIn_electra_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_diedIn_electra_top1_borda_normal_one}')
print('\n')

results_list_diedIn_electra_top1_borda_spacy_one, _, average_diedIn_electra_top1_borda_spacy_one = create_results_list(
    diedIn_electra_borda_spacy_tokens_score_one, diedIn_electra_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_diedIn_electra_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_diedIn_electra_top1_spacy_one, _, average_diedIn_electra_top1_spacy_one = create_results_list(
    masked_diedIn_electra_original_spacy_one, diedIn_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_diedIn_electra_top1_spacy_one}')

results_list_diedIn_electra_top1_one, _, average_diedIn_electra_top1_one = create_results_list(
    masked_diedIn_electra_original_one, diedIn_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_diedIn_electra_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.296
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.303


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.302


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.294


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.295
Avg Precision K = 1 of died In Normal: 0.273


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_diedIn_electra_top5_ranking_spacy_one, _, average_diedIn_electra_top5_ranking_spacy_one = create_results_list(
    ranking_diedIn_electra_spacy_tokens_one, diedIn_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_diedIn_electra_top5_ranking_spacy_one}')

results_list_diedIn_electra_top5_ranking_normal_one, _, average_diedIn_electra_top5_ranking_normal_one = create_results_list(
    ranking_diedIn_electra_normal_tokens_one, diedIn_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_diedIn_electra_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_diedIn_electra_top5_borda_normal_one, _, average_diedIn_electra_top5_borda_normal_one = create_results_list(
    diedIn_electra_borda_tokens_score_one, diedIn_electra_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_diedIn_electra_top5_borda_normal_one}')
print('\n')

results_list_diedIn_electra_top5_borda_spacy_one, _, average_diedIn_electra_top5_borda_spacy_one = create_results_list(
    diedIn_electra_borda_spacy_tokens_score_one, diedIn_electra_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_diedIn_electra_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_diedIn_electra_top5_spacy_one, _, average_diedIn_electra_top5_spacy_one = create_results_list(
    masked_diedIn_electra_original_spacy_one, diedIn_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_diedIn_electra_top5_spacy_one}')

results_list_diedIn_electra_top5_one, _, average_diedIn_electra_top5_one = create_results_list(
    masked_diedIn_electra_original_one, diedIn_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_diedIn_electra_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.557
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.572


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.564


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.553


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.533
Avg Precision K = 5 of died In Normal: 0.522


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_diedIn_electra_top10_ranking_spacy_one, _, average_diedIn_electra_top10_ranking_spacy_one = create_results_list(
    ranking_diedIn_electra_spacy_tokens_one, diedIn_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_diedIn_electra_top10_ranking_spacy_one}')

results_list_diedIn_electra_top10_ranking_normal_one, _, average_diedIn_electra_top10_ranking_normal_one = create_results_list(
    ranking_diedIn_electra_normal_tokens_one, diedIn_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_diedIn_electra_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_diedIn_electra_top10_borda_normal_one, _, average_diedIn_electra_top10_borda_normal_one = create_results_list(
    diedIn_electra_borda_tokens_score_one, diedIn_electra_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_diedIn_electra_top10_borda_normal_one}')
print('\n')

results_list_diedIn_electra_top10_borda_spacy_one, _, average_diedIn_electra_top10_borda_spacy_one = create_results_list(
    diedIn_electra_borda_spacy_tokens_score_one, diedIn_electra_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_diedIn_electra_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_diedIn_electra_top10_spacy_one, _, average_diedIn_electra_top10_spacy_one = create_results_list(
    masked_diedIn_electra_original_spacy_one, diedIn_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_diedIn_electra_top10_spacy_one}')

results_list_diedIn_electra_top10_one, _, average_diedIn_electra_top10_one = create_results_list(
    masked_diedIn_electra_original_one, diedIn_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_diedIn_electra_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.662
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.686


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.671


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.659


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.599
Avg Precision K = 10 of died In Normal: 0.618


#### Worksfor

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_worksfor_electra_top1_ranking_spacy_one, _, average_worksfor_electra_top1_ranking_spacy_one = create_results_list(
    ranking_worksfor_electra_spacy_tokens_one, worksfor_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_worksfor_electra_top1_ranking_spacy_one}')

results_list_worksfor_electra_top1_ranking_normal_one, _, average_worksfor_electra_top1_ranking_normal_one = create_results_list(
    ranking_worksfor_electra_normal_tokens_one, worksfor_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_worksfor_electra_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_worksfor_electra_top1_borda_normal_one, _, average_worksfor_electra_top1_borda_normal_one = create_results_list(
    worksfor_electra_borda_tokens_score_one, worksfor_electra_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_worksfor_electra_top1_borda_normal_one}')
print('\n')

results_list_worksfor_electra_top1_borda_spacy_one, _, average_worksfor_electra_top1_borda_spacy_one = create_results_list(
    worksfor_electra_borda_spacy_tokens_score_one, worksfor_electra_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_worksfor_electra_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_worksfor_electra_top1_spacy_one, _, average_worksfor_electra_top1_spacy_one = create_results_list(
    masked_worksfor_electra_original_spacy_one, worksfor_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_worksfor_electra_top1_spacy_one}')

results_list_worksfor_electra_top1_one, _, average_worksfor_electra_top1_one = create_results_list(
    masked_worksfor_electra_original_one, worksfor_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_worksfor_electra_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.57
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.475


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.472


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.57


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.485
Avg Precision K = 1 of died In Normal: 0.396


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_worksfor_electra_top5_ranking_spacy_one, _, average_worksfor_electra_top5_ranking_spacy_one = create_results_list(
    ranking_worksfor_electra_spacy_tokens_one, worksfor_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_worksfor_electra_top5_ranking_spacy_one}')

results_list_worksfor_electra_top5_ranking_normal_one, _, average_worksfor_electra_top5_ranking_normal_one = create_results_list(
    ranking_worksfor_electra_normal_tokens_one, worksfor_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_worksfor_electra_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_worksfor_electra_top5_borda_normal_one, _, average_worksfor_electra_top5_borda_normal_one = create_results_list(
    worksfor_electra_borda_tokens_score_one, worksfor_electra_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_worksfor_electra_top5_borda_normal_one}')
print('\n')

results_list_worksfor_electra_top5_borda_spacy_one, _, average_worksfor_electra_top5_borda_spacy_one = create_results_list(
    worksfor_electra_borda_spacy_tokens_score_one, worksfor_electra_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_worksfor_electra_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_worksfor_electra_top5_spacy_one, _, average_worksfor_electra_top5_spacy_one = create_results_list(
    masked_worksfor_electra_original_spacy_one, worksfor_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_worksfor_electra_top5_spacy_one}')

results_list_worksfor_electra_top5_one, _, average_worksfor_electra_top5_one = create_results_list(
    masked_worksfor_electra_original_one, worksfor_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_worksfor_electra_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.763
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.712


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.715


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.765


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.686
Avg Precision K = 5 of died In Normal: 0.646


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_worksfor_electra_top10_ranking_spacy_one, _, average_worksfor_electra_top10_ranking_spacy_one = create_results_list(
    ranking_worksfor_electra_spacy_tokens_one, worksfor_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_worksfor_electra_top10_ranking_spacy_one}')

results_list_worksfor_electra_top10_ranking_normal_one, _, average_worksfor_electra_top10_ranking_normal_one = create_results_list(
    ranking_worksfor_electra_normal_tokens_one, worksfor_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_worksfor_electra_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_worksfor_electra_top10_borda_normal_one, _, average_worksfor_electra_top10_borda_normal_one = create_results_list(
    worksfor_electra_borda_tokens_score_one, worksfor_electra_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_worksfor_electra_top10_borda_normal_one}')
print('\n')

results_list_worksfor_electra_top10_borda_spacy_one, _, average_worksfor_electra_top10_borda_spacy_one = create_results_list(
    worksfor_electra_borda_spacy_tokens_score_one, worksfor_electra_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_worksfor_electra_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_worksfor_electra_top10_spacy_one, _, average_worksfor_electra_top10_spacy_one = create_results_list(
    masked_worksfor_electra_original_spacy_one, worksfor_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_worksfor_electra_top10_spacy_one}')

results_list_worksfor_electra_top10_one, _, average_worksfor_electra_top10_one = create_results_list(
    masked_worksfor_electra_original_one, worksfor_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_worksfor_electra_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.805
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.81


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.802


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.805


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.704
Avg Precision K = 10 of died In Normal: 0.731


#### Capital

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_capital_electra_top1_ranking_spacy_one, _, average_capital_electra_top1_ranking_spacy_one = create_results_list(
    ranking_capital_electra_spacy_tokens_one, capital_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_capital_electra_top1_ranking_spacy_one}')

results_list_capital_electra_top1_ranking_normal_one, _, average_capital_electra_top1_ranking_normal_one = create_results_list(
    ranking_capital_electra_normal_tokens_one, capital_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_capital_electra_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_capital_electra_top1_borda_normal_one, _, average_capital_electra_top1_borda_normal_one = create_results_list(
    capital_electra_borda_tokens_score_one, capital_electra_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_capital_electra_top1_borda_normal_one}')
print('\n')

results_list_capital_electra_top1_borda_spacy_one, _, average_capital_electra_top1_borda_spacy_one = create_results_list(
    capital_electra_borda_spacy_tokens_score_one, capital_electra_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_capital_electra_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_capital_electra_top1_spacy_one, _, average_capital_electra_top1_spacy_one = create_results_list(
    masked_capital_electra_original_spacy_one, capital_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_capital_electra_top1_spacy_one}')

results_list_capital_electra_top1_one, _, average_capital_electra_top1_one = create_results_list(
    masked_capital_electra_original_one, capital_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_capital_electra_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.196
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.199


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.196


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.189


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.242
Avg Precision K = 1 of died In Normal: 0.237


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_capital_electra_top5_ranking_spacy_one, _, average_capital_electra_top5_ranking_spacy_one = create_results_list(
    ranking_capital_electra_spacy_tokens_one, capital_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_capital_electra_top5_ranking_spacy_one}')

results_list_capital_electra_top5_ranking_normal_one, _, average_capital_electra_top5_ranking_normal_one = create_results_list(
    ranking_capital_electra_normal_tokens_one, capital_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_capital_electra_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_capital_electra_top5_borda_normal_one, _, average_capital_electra_top5_borda_normal_one = create_results_list(
    capital_electra_borda_tokens_score_one, capital_electra_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_capital_electra_top5_borda_normal_one}')
print('\n')

results_list_capital_electra_top5_borda_spacy_one, _, average_capital_electra_top5_borda_spacy_one = create_results_list(
    capital_electra_borda_spacy_tokens_score_one, capital_electra_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_capital_electra_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_capital_electra_top5_spacy_one, _, average_capital_electra_top5_spacy_one = create_results_list(
    masked_capital_electra_original_spacy_one, capital_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_capital_electra_top5_spacy_one}')

results_list_capital_electra_top5_one, _, average_capital_electra_top5_one = create_results_list(
    masked_capital_electra_original_one, capital_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_capital_electra_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.411
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.465


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.46


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.408


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.405
Avg Precision K = 5 of died In Normal: 0.411


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_capital_electra_top10_ranking_spacy_one, _, average_capital_electra_top10_ranking_spacy_one = create_results_list(
    ranking_capital_electra_spacy_tokens_one, capital_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_capital_electra_top10_ranking_spacy_one}')

results_list_capital_electra_top10_ranking_normal_one, _, average_capital_electra_top10_ranking_normal_one = create_results_list(
    ranking_capital_electra_normal_tokens_one, capital_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_capital_electra_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_capital_electra_top10_borda_normal_one, _, average_capital_electra_top10_borda_normal_one = create_results_list(
    capital_electra_borda_tokens_score_one, capital_electra_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_capital_electra_top10_borda_normal_one}')
print('\n')

results_list_capital_electra_top10_borda_spacy_one, _, average_capital_electra_top10_borda_spacy_one = create_results_list(
    capital_electra_borda_spacy_tokens_score_one, capital_electra_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_capital_electra_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_capital_electra_top10_spacy_one, _, average_capital_electra_top10_spacy_one = create_results_list(
    masked_capital_electra_original_spacy_one, capital_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_capital_electra_top10_spacy_one}')

results_list_capital_electra_top10_one, _, average_capital_electra_top10_one = create_results_list(
    masked_capital_electra_original_one, capital_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_capital_electra_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.494
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.545


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.543


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.492


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.462
Avg Precision K = 10 of died In Normal: 0.485


#### Citizen

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_citizen_electra_top1_ranking_spacy_one, _, average_citizen_electra_top1_ranking_spacy_one = create_results_list(
    ranking_citizen_electra_spacy_tokens_one, citizen_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_citizen_electra_top1_ranking_spacy_one}')

results_list_citizen_electra_top1_ranking_normal_one, _, average_citizen_electra_top1_ranking_normal_one = create_results_list(
    ranking_citizen_electra_normal_tokens_one, citizen_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_citizen_electra_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_citizen_electra_top1_borda_normal_one, _, average_citizen_electra_top1_borda_normal_one = create_results_list(
    citizen_electra_borda_tokens_score_one, citizen_electra_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_citizen_electra_top1_borda_normal_one}')
print('\n')

results_list_citizen_electra_top1_borda_spacy_one, _, average_citizen_electra_top1_borda_spacy_one = create_results_list(
    citizen_electra_borda_spacy_tokens_score_one, citizen_electra_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_citizen_electra_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_citizen_electra_top1_spacy_one, _, average_citizen_electra_top1_spacy_one = create_results_list(
    masked_citizen_electra_original_spacy_one, citizen_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_citizen_electra_top1_spacy_one}')

results_list_citizen_electra_top1_one, _, average_citizen_electra_top1_one = create_results_list(
    masked_citizen_electra_original_one, citizen_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_citizen_electra_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.298
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.419


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.416


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.286


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.277
Avg Precision K = 1 of died In Normal: 0.382


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_citizen_electra_top5_ranking_spacy_one, _, average_citizen_electra_top5_ranking_spacy_one = create_results_list(
    ranking_citizen_electra_spacy_tokens_one, citizen_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_citizen_electra_top5_ranking_spacy_one}')

results_list_citizen_electra_top5_ranking_normal_one, _, average_citizen_electra_top5_ranking_normal_one = create_results_list(
    ranking_citizen_electra_normal_tokens_one, citizen_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_citizen_electra_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_citizen_electra_top5_borda_normal_one, _, average_citizen_electra_top5_borda_normal_one = create_results_list(
    citizen_electra_borda_tokens_score_one, citizen_electra_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_citizen_electra_top5_borda_normal_one}')
print('\n')

results_list_citizen_electra_top5_borda_spacy_one, _, average_citizen_electra_top5_borda_spacy_one = create_results_list(
    citizen_electra_borda_spacy_tokens_score_one, citizen_electra_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_citizen_electra_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_citizen_electra_top5_spacy_one, _, average_citizen_electra_top5_spacy_one = create_results_list(
    masked_citizen_electra_original_spacy_one, citizen_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_citizen_electra_top5_spacy_one}')

results_list_citizen_electra_top5_one, _, average_citizen_electra_top5_one = create_results_list(
    masked_citizen_electra_original_one, citizen_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_citizen_electra_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.456
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.645


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.641


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.456


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.357
Avg Precision K = 5 of died In Normal: 0.56


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_citizen_electra_top10_ranking_spacy_one, _, average_citizen_electra_top10_ranking_spacy_one = create_results_list(
    ranking_citizen_electra_spacy_tokens_one, citizen_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_citizen_electra_top10_ranking_spacy_one}')

results_list_citizen_electra_top10_ranking_normal_one, _, average_citizen_electra_top10_ranking_normal_one = create_results_list(
    ranking_citizen_electra_normal_tokens_one, citizen_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_citizen_electra_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_citizen_electra_top10_borda_normal_one, _, average_citizen_electra_top10_borda_normal_one = create_results_list(
    citizen_electra_borda_tokens_score_one, citizen_electra_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_citizen_electra_top10_borda_normal_one}')
print('\n')

results_list_citizen_electra_top10_borda_spacy_one, _, average_citizen_electra_top10_borda_spacy_one = create_results_list(
    citizen_electra_borda_spacy_tokens_score_one, citizen_electra_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_citizen_electra_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_citizen_electra_top10_spacy_one, _, average_citizen_electra_top10_spacy_one = create_results_list(
    masked_citizen_electra_original_spacy_one, citizen_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_citizen_electra_top10_spacy_one}')

results_list_citizen_electra_top10_one, _, average_citizen_electra_top10_one = create_results_list(
    masked_citizen_electra_original_one, citizen_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_citizen_electra_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.494
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.737


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.735


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.493


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.366
Avg Precision K = 10 of died In Normal: 0.613


#### Language

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_language_electra_top1_ranking_spacy_one, _, average_language_electra_top1_ranking_spacy_one = create_results_list(
    ranking_language_electra_spacy_tokens_one, language_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy One Shoot: {average_language_electra_top1_ranking_spacy_one}')

results_list_language_electra_top1_ranking_normal_one, _, average_language_electra_top1_ranking_normal_one = create_results_list(
    ranking_language_electra_normal_tokens_one, language_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal One Shoot: {average_language_electra_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_language_electra_top1_borda_normal_one, _, average_language_electra_top1_borda_normal_one = create_results_list(
    language_electra_borda_tokens_score_one, language_electra_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal One Shoot: {average_language_electra_top1_borda_normal_one}')
print('\n')

results_list_language_electra_top1_borda_spacy_one, _, average_language_electra_top1_borda_spacy_one = create_results_list(
    language_electra_borda_spacy_tokens_score_one, language_electra_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy One Shoot: {average_language_electra_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_language_electra_top1_spacy_one, _, average_language_electra_top1_spacy_one = create_results_list(
    masked_language_electra_original_spacy_one, language_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy One Shoot: {average_language_electra_top1_spacy_one}')

results_list_language_electra_top1_one, _, average_language_electra_top1_one = create_results_list(
    masked_language_electra_original_one, language_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_language_electra_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy One Shoot: 0.328
Avg Precision K = 1 of the ranking of died In Normal One Shoot: 0.402


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal One Shoot: 0.4


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy One Shoot: 0.324


Sem ranking
Avg Precision K = 1 of died In Spacy One Shoot: 0.396
Avg Precision K = 1 of died In Normal: 0.362


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_language_electra_top5_ranking_spacy_one, _, average_language_electra_top5_ranking_spacy_one = create_results_list(
    ranking_language_electra_spacy_tokens_one, language_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy One Shoot: {average_language_electra_top5_ranking_spacy_one}')

results_list_language_electra_top5_ranking_normal_one, _, average_language_electra_top5_ranking_normal_one = create_results_list(
    ranking_language_electra_normal_tokens_one, language_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal One Shoot: {average_language_electra_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_language_electra_top5_borda_normal_one, _, average_language_electra_top5_borda_normal_one = create_results_list(
    language_electra_borda_tokens_score_one, language_electra_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal One Shoot: {average_language_electra_top5_borda_normal_one}')
print('\n')

results_list_language_electra_top5_borda_spacy_one, _, average_language_electra_top5_borda_spacy_one = create_results_list(
    language_electra_borda_spacy_tokens_score_one, language_electra_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy One Shoot: {average_language_electra_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_language_electra_top5_spacy_one, _, average_language_electra_top5_spacy_one = create_results_list(
    masked_language_electra_original_spacy_one, language_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy One Shoot: {average_language_electra_top5_spacy_one}')

results_list_language_electra_top5_one, _, average_language_electra_top5_one = create_results_list(
    masked_language_electra_original_one, language_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_language_electra_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy One Shoot: 0.547
Avg Precision K = 5 of the ranking of died In Normal One Shoot: 0.742


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal One Shoot: 0.735


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy One Shoot: 0.543


Sem ranking
Avg Precision K = 5 of died In Spacy One Shoot: 0.63
Avg Precision K = 5 of died In Normal: 0.674


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_language_electra_top10_ranking_spacy_one, _, average_language_electra_top10_ranking_spacy_one = create_results_list(
    ranking_language_electra_spacy_tokens_one, language_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy One Shoot: {average_language_electra_top10_ranking_spacy_one}')

results_list_language_electra_top10_ranking_normal_one, _, average_language_electra_top10_ranking_normal_one = create_results_list(
    ranking_language_electra_normal_tokens_one, language_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal One Shoot: {average_language_electra_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_language_electra_top10_borda_normal_one, _, average_language_electra_top10_borda_normal_one = create_results_list(
    language_electra_borda_tokens_score_one, language_electra_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal One Shoot: {average_language_electra_top10_borda_normal_one}')
print('\n')

results_list_language_electra_top10_borda_spacy_one, _, average_language_electra_top10_borda_spacy_one = create_results_list(
    language_electra_borda_spacy_tokens_score_one, language_electra_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy One Shoot: {average_language_electra_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_language_electra_top10_spacy_one, _, average_language_electra_top10_spacy_one = create_results_list(
    masked_language_electra_original_spacy_one, language_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy One Shoot: {average_language_electra_top10_spacy_one}')

results_list_language_electra_top10_one, _, average_language_electra_top10_one = create_results_list(
    masked_language_electra_original_one, language_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_language_electra_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy One Shoot: 0.588
Avg Precision K = 10 of the ranking of died In Normal One Shoot: 0.84


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal One Shoot: 0.836


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy One Shoot: 0.587


Sem ranking
Avg Precision K = 10 of died In Spacy One Shoot: 0.647
Avg Precision K = 10 of died In Normal: 0.762


### Merged Models

#### BornIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_bornIn_merged_model_top1_ranking_spacy_one, _, average_bornIn_merged_model_top1_ranking_spacy_one = create_results_list(
    ranking_bornin_merged_model_spacy_tokens_one, bornIn_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Spacy: {average_bornIn_merged_model_top1_ranking_spacy_one}')

results_list_bornIn_merged_model_top1_ranking_normal_one, _, average_bornIn_merged_model_top1_ranking_normal_one = create_results_list(
    ranking_bornIn_merged_model_normal_tokens_one, bornIn_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Normal: {average_bornIn_merged_model_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_bornIn_merged_model_top1_borda_normal_one, _, average_bornIn_merged_model_top1_borda_normal_one = create_results_list(
    bornIn_merged_model_borda_tokens_score_one, bornIn_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Born In Normal: {average_bornIn_merged_model_top1_borda_normal_one}')
print('\n')

results_list_bornIn_merged_model_top1_borda_spacy_one, _, average_bornIn_merged_model_top1_borda_spacy_one = create_results_list(
    bornIn_merged_model_borda_spacy_tokens_score_one, bornIn_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Born In spacy: {average_bornIn_merged_model_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top1_spacy_one, _, average_bornIn_bert_top1_spacy_one = create_results_list(
    masked_bornIn_bert_original_spacy_one, bornIn_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Bert Spacy: {average_bornIn_bert_top1_spacy_one}')

results_list_bornIn_bert_top1_one, _, average_bornIn_bert_top1_one = create_results_list(
    masked_bornIn_bert_original_one, bornIn_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Bert Normal: {average_bornIn_bert_top1_one}')

results_list_bornIn_roberta_top1_spacy_one, _, average_bornIn_roberta_top1_spacy_one = create_results_list(
    masked_bornIn_roberta_original_spacy_one, bornIn_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In roberta Spacy: {average_bornIn_roberta_top1_spacy_one}')

results_list_bornIn_roberta_top1_one, _, average_bornIn_roberta_top1_one = create_results_list(
    masked_bornIn_roberta_original, bornIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In roberta Normal: {average_bornIn_roberta_top1_one}')

results_list_bornIn_electra_top1_spacy_one, _, average_bornIn_electra_top1_spacy_one = create_results_list(
    masked_bornIn_electra_original_spacy_one, bornIn_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In electra Spacy: {average_bornIn_electra_top1_spacy_one}')

results_list_bornIn_electra_top1_one, _, average_bornIn_electra_top1_one = create_results_list(
    masked_bornIn_electra_original_one, bornIn_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In electra Normal: {average_bornIn_electra_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Born In Spacy: 0.339
Avg Precision K = 1 of the ranking of Born In Normal: 0.328


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Born In Normal: 0.327


Borda Spacy Score
Avg Precision K = 1 of the borda of Born In spacy: 0.345


Sem ranking
Avg Precision K = 1 of Born In Bert Spacy: 0.347
Avg Precision K = 1 of Born In Bert Normal: 0.335
Avg Precision K = 1 of Born In roberta Spacy: 0.306
Avg Precision K = 1 of Born In roberta Normal: 0.329
Avg Precision K = 1 of Born In electra Spacy: 0.193
Avg Precision K = 1 of Born In electra Normal: 0.182


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_bornIn_merged_model_top5_ranking_spacy_one, _, average_bornIn_merged_model_top5_ranking_spacy_one = create_results_list(
    ranking_bornin_merged_model_spacy_tokens_one, bornIn_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Spacy: {average_bornIn_merged_model_top5_ranking_spacy_one}')

results_list_bornIn_merged_model_top5_ranking_normal_one, _, average_bornIn_merged_model_top5_ranking_normal_one = create_results_list(
    ranking_bornIn_merged_model_normal_tokens_one, bornIn_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Normal: {average_bornIn_merged_model_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_bornIn_merged_model_top5_borda_normal_one, _, average_bornIn_merged_model_top5_borda_normal_one = create_results_list(
    bornIn_merged_model_borda_tokens_score_one, bornIn_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Born In Normal: {average_bornIn_merged_model_top5_borda_normal_one}')
print('\n')

results_list_bornIn_merged_model_top5_borda_spacy_one, _, average_bornIn_merged_model_top5_borda_spacy_one = create_results_list(
    bornIn_merged_model_borda_spacy_tokens_score_one, bornIn_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Born In spacy: {average_bornIn_merged_model_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top5_spacy_one, _, average_bornIn_bert_top5_spacy_one = create_results_list(
    masked_bornIn_bert_original_spacy_one, bornIn_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Bert Spacy: {average_bornIn_bert_top5_spacy_one}')

results_list_bornIn_bert_top5_one, _, average_bornIn_bert_top5_one = create_results_list(
    masked_bornIn_bert_original_one, bornIn_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Bert Normal: {average_bornIn_bert_top5_one}')

results_list_bornIn_roberta_top5_spacy_one, _, average_bornIn_roberta_top5_spacy_one = create_results_list(
    masked_bornIn_roberta_original_spacy_one, bornIn_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In roberta Spacy: {average_bornIn_roberta_top5_spacy_one}')

results_list_bornIn_roberta_top5_one, _, average_bornIn_roberta_top5_one = create_results_list(
    masked_bornIn_roberta_original, bornIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In roberta Normal: {average_bornIn_roberta_top5_one}')

results_list_bornIn_electra_top5_spacy_one, _, average_bornIn_electra_top5_spacy_one = create_results_list(
    masked_bornIn_electra_original_spacy_one, bornIn_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In electra Spacy: {average_bornIn_electra_top5_spacy_one}')

results_list_bornIn_electra_top5_one, _, average_bornIn_electra_top5_one = create_results_list(
    masked_bornIn_electra_original_one, bornIn_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In electra Normal: {average_bornIn_electra_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Born In Spacy: 0.583
Avg Precision K = 5 of the ranking of Born In Normal: 0.56


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Born In Normal: 0.558


Borda Spacy Score
Avg Precision K = 5 of the borda of Born In spacy: 0.583


Sem ranking
Avg Precision K = 5 of Born In Bert Spacy: 0.555
Avg Precision K = 5 of Born In Bert Normal: 0.548
Avg Precision K = 5 of Born In roberta Spacy: 0.508
Avg Precision K = 5 of Born In roberta Normal: 0.52
Avg Precision K = 5 of Born In electra Spacy: 0.413
Avg Precision K = 5 of Born In electra Normal: 0.408


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_bornIn_merged_model_top10_ranking_spacy_one, _, average_bornIn_merged_model_top10_ranking_spacy_one = create_results_list(
    ranking_bornin_merged_model_spacy_tokens_one, bornIn_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Spacy: {average_bornIn_merged_model_top10_ranking_spacy_one}')

results_list_bornIn_merged_model_top10_ranking_normal_one, _, average_bornIn_merged_model_top10_ranking_normal_one = create_results_list(
    ranking_bornIn_merged_model_normal_tokens_one, bornIn_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Normal: {average_bornIn_merged_model_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_bornIn_merged_model_top10_borda_normal_one, _, average_bornIn_merged_model_top10_borda_normal_one = create_results_list(
    bornIn_merged_model_borda_tokens_score_one, bornIn_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Born In Normal: {average_bornIn_merged_model_top10_borda_normal_one}')
print('\n')

results_list_bornIn_merged_model_top10_borda_spacy_one, _, average_bornIn_merged_model_top10_borda_spacy_one = create_results_list(
    bornIn_merged_model_borda_spacy_tokens_score_one, bornIn_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Born In spacy: {average_bornIn_merged_model_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top10_spacy_one, _, average_bornIn_bert_top10_spacy_one = create_results_list(
    masked_bornIn_bert_original_spacy_one, bornIn_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Bert Spacy: {average_bornIn_bert_top10_spacy_one}')

results_list_bornIn_bert_top10_one, _, average_bornIn_bert_top10_one = create_results_list(
    masked_bornIn_bert_original_one, bornIn_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Bert Normal: {average_bornIn_bert_top10_one}')

results_list_bornIn_roberta_top10_spacy_one, _, average_bornIn_roberta_top10_spacy_one = create_results_list(
    masked_bornIn_roberta_original_spacy_one, bornIn_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In roberta Spacy: {average_bornIn_roberta_top10_spacy_one}')

results_list_bornIn_roberta_top10_one, _, average_bornIn_roberta_top10_one = create_results_list(
    masked_bornIn_roberta_original, bornIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In roberta Normal: {average_bornIn_roberta_top10_one}')

results_list_bornIn_electra_top10_spacy_one, _, average_bornIn_electra_top10_spacy_one = create_results_list(
    masked_bornIn_electra_original_spacy_one, bornIn_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In electra Spacy: {average_bornIn_electra_top10_spacy_one}')

results_list_bornIn_electra_top10_one, _, average_bornIn_electra_top10_one = create_results_list(
    masked_bornIn_electra_original_one, bornIn_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In electra Normal: {average_bornIn_electra_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Born In Spacy: 0.658
Avg Precision K = 10 of the ranking of Born In Normal: 0.655


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Born In Normal: 0.654


Borda Spacy Score
Avg Precision K = 10 of the borda of Born In spacy: 0.656


Sem ranking
Avg Precision K = 10 of Born In Bert Spacy: 0.63
Avg Precision K = 10 of Born In Bert Normal: 0.637
Avg Precision K = 10 of Born In roberta Spacy: 0.552
Avg Precision K = 10 of Born In roberta Normal: 0.577
Avg Precision K = 10 of Born In electra Spacy: 0.479
Avg Precision K = 10 of Born In electra Normal: 0.481


#### DiedIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_diedIn_merged_model_top1_ranking_spacy_one, _, average_diedIn_merged_model_top1_ranking_spacy_one = create_results_list(
    ranking_diedIn_merged_model_spacy_tokens_one, diedIn_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy: {average_diedIn_merged_model_top1_ranking_spacy_one}')

results_list_diedIn_merged_model_top1_ranking_normal_one, _, average_diedIn_merged_model_top1_ranking_normal_one = create_results_list(
    ranking_diedIn_merged_model_normal_tokens_one, diedIn_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal: {average_diedIn_merged_model_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_diedIn_merged_model_top1_borda_normal_one, _, average_diedIn_merged_model_top1_borda_normal_one = create_results_list(
    diedIn_merged_model_borda_tokens_score_one, diedIn_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal: {average_diedIn_merged_model_top1_borda_normal_one}')
print('\n')

results_list_diedIn_merged_model_top1_borda_spacy_one, _, average_diedIn_merged_model_top1_borda_spacy_one = create_results_list(
    diedIn_merged_model_borda_spacy_tokens_score_one, diedIn_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In spacy: {average_diedIn_merged_model_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top1_spacy_one, _, average_diedIn_bert_top1_spacy_one = create_results_list(
    masked_diedIn_bert_original_spacy_one, diedIn_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Spacy: {average_diedIn_bert_top1_spacy_one}')

results_list_diedIn_bert_top1_one, _, average_diedIn_bert_top1_one = create_results_list(
    masked_diedIn_bert_original_one, diedIn_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Normal: {average_diedIn_bert_top1_one}')

results_list_diedIn_roberta_top1_spacy_one, _, average_diedIn_roberta_top1_spacy_one = create_results_list(
    masked_diedIn_roberta_original_spacy_one, diedIn_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Spacy: {average_diedIn_roberta_top1_spacy_one}')

results_list_diedIn_roberta_top1_one, _, average_diedIn_roberta_top1_one = create_results_list(
    masked_diedIn_roberta_original, diedIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Normal: {average_diedIn_roberta_top1_one}')

results_list_diedIn_electra_top1_spacy_one, _, average_diedIn_electra_top1_spacy_one = create_results_list(
    masked_diedIn_electra_original_spacy_one, diedIn_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Spacy: {average_diedIn_electra_top1_spacy_one}')

results_list_diedIn_electra_top1_one, _, average_diedIn_electra_top1_one = create_results_list(
    masked_diedIn_electra_original_one, diedIn_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Normal: {average_diedIn_electra_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy: 0.479
Avg Precision K = 1 of the ranking of died In Normal: 0.449


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal: 0.45


Borda Spacy Score
Avg Precision K = 1 of the borda of died In spacy: 0.479


Sem ranking
Avg Precision K = 1 of died In Bert Spacy: 0.486
Avg Precision K = 1 of died In Bert Normal: 0.465
Avg Precision K = 1 of died In roberta Spacy: 0.423
Avg Precision K = 1 of died In roberta Normal: 0.421
Avg Precision K = 1 of died In electra Spacy: 0.295
Avg Precision K = 1 of died In electra Normal: 0.273


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_diedIn_merged_model_top5_ranking_spacy_one, _, average_diedIn_merged_model_top5_ranking_spacy_one = create_results_list(
    ranking_diedIn_merged_model_spacy_tokens_one, diedIn_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy: {average_diedIn_merged_model_top5_ranking_spacy_one}')

results_list_diedIn_merged_model_top5_ranking_normal_one, _, average_diedIn_merged_model_top5_ranking_normal_one = create_results_list(
    ranking_diedIn_merged_model_normal_tokens_one, diedIn_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal: {average_diedIn_merged_model_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_diedIn_merged_model_top5_borda_normal_one, _, average_diedIn_merged_model_top5_borda_normal_one = create_results_list(
    diedIn_merged_model_borda_tokens_score_one, diedIn_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal: {average_diedIn_merged_model_top5_borda_normal_one}')
print('\n')

results_list_diedIn_merged_model_top5_borda_spacy_one, _, average_diedIn_merged_model_top5_borda_spacy_one = create_results_list(
    diedIn_merged_model_borda_spacy_tokens_score_one, diedIn_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In spacy: {average_diedIn_merged_model_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top5_spacy_one, _, average_diedIn_bert_top5_spacy_one = create_results_list(
    masked_diedIn_bert_original_spacy_one, diedIn_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Spacy: {average_diedIn_bert_top5_spacy_one}')

results_list_diedIn_bert_top5_one, _, average_diedIn_bert_top5_one = create_results_list(
    masked_diedIn_bert_original_one, diedIn_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Normal: {average_diedIn_bert_top5_one}')

results_list_diedIn_roberta_top5_spacy_one, _, average_diedIn_roberta_top5_spacy_one = create_results_list(
    masked_diedIn_roberta_original_spacy_one, diedIn_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Spacy: {average_diedIn_roberta_top5_spacy_one}')

results_list_diedIn_roberta_top5_one, _, average_diedIn_roberta_top5_one = create_results_list(
    masked_diedIn_roberta_original, diedIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Normal: {average_diedIn_roberta_top5_one}')

results_list_diedIn_electra_top5_spacy_one, _, average_diedIn_electra_top5_spacy_one = create_results_list(
    masked_diedIn_electra_original_spacy_one, diedIn_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Spacy: {average_diedIn_electra_top5_spacy_one}')

results_list_diedIn_electra_top5_one, _, average_diedIn_electra_top5_one = create_results_list(
    masked_diedIn_electra_original_one, diedIn_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Normal: {average_diedIn_electra_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy: 0.699
Avg Precision K = 5 of the ranking of died In Normal: 0.682


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal: 0.68


Borda Spacy Score
Avg Precision K = 5 of the borda of died In spacy: 0.692


Sem ranking
Avg Precision K = 5 of died In Bert Spacy: 0.702
Avg Precision K = 5 of died In Bert Normal: 0.711
Avg Precision K = 5 of died In roberta Spacy: 0.619
Avg Precision K = 5 of died In roberta Normal: 0.631
Avg Precision K = 5 of died In electra Spacy: 0.533
Avg Precision K = 5 of died In electra Normal: 0.522


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_diedIn_merged_model_top10_ranking_spacy_one, _, average_diedIn_merged_model_top10_ranking_spacy_one = create_results_list(
    ranking_diedIn_merged_model_spacy_tokens_one, diedIn_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy: {average_diedIn_merged_model_top10_ranking_spacy_one}')

results_list_diedIn_merged_model_top10_ranking_normal_one, _, average_diedIn_merged_model_top10_ranking_normal_one = create_results_list(
    ranking_diedIn_merged_model_normal_tokens_one, diedIn_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal: {average_diedIn_merged_model_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_diedIn_merged_model_top10_borda_normal_one, _, average_diedIn_merged_model_top10_borda_normal_one = create_results_list(
    diedIn_merged_model_borda_tokens_score_one, diedIn_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal: {average_diedIn_merged_model_top10_borda_normal_one}')
print('\n')

results_list_diedIn_merged_model_top10_borda_spacy_one, _, average_diedIn_merged_model_top10_borda_spacy_one = create_results_list(
    diedIn_merged_model_borda_spacy_tokens_score_one, diedIn_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In spacy: {average_diedIn_merged_model_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top10_spacy_one, _, average_diedIn_bert_top10_spacy_one = create_results_list(
    masked_diedIn_bert_original_spacy_one, diedIn_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Spacy: {average_diedIn_bert_top10_spacy_one}')

results_list_diedIn_bert_top10_one, _, average_diedIn_bert_top10_one = create_results_list(
    masked_diedIn_bert_original_one, diedIn_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Normal: {average_diedIn_bert_top10_one}')

results_list_diedIn_roberta_top10_spacy_one, _, average_diedIn_roberta_top10_spacy_one = create_results_list(
    masked_diedIn_roberta_original_spacy_one, diedIn_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Spacy: {average_diedIn_roberta_top10_spacy_one}')

results_list_diedIn_roberta_top10_one, _, average_diedIn_roberta_top10_one = create_results_list(
    masked_diedIn_roberta_original, diedIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Normal: {average_diedIn_roberta_top10_one}')

results_list_diedIn_electra_top10_spacy_one, _, average_diedIn_electra_top10_spacy_one = create_results_list(
    masked_diedIn_electra_original_spacy_one, diedIn_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Spacy: {average_diedIn_electra_top10_spacy_one}')

results_list_diedIn_electra_top10_one, _, average_diedIn_electra_top10_one = create_results_list(
    masked_diedIn_electra_original_one, diedIn_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Normal: {average_diedIn_electra_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy: 0.761
Avg Precision K = 10 of the ranking of died In Normal: 0.767


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal: 0.766


Borda Spacy Score
Avg Precision K = 10 of the borda of died In spacy: 0.764


Sem ranking
Avg Precision K = 10 of died In Bert Spacy: 0.751
Avg Precision K = 10 of died In Bert Normal: 0.775
Avg Precision K = 10 of died In roberta Spacy: 0.661
Avg Precision K = 10 of died In roberta Normal: 0.689
Avg Precision K = 10 of died In electra Spacy: 0.599
Avg Precision K = 10 of died In electra Normal: 0.618


#### Worksfor

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_worksfor_merged_model_top1_ranking_spacy_one, _, average_worksfor_merged_model_top1_ranking_spacy_one = create_results_list(
    ranking_worksfor_merged_model_spacy_tokens_one, worksfor_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy: {average_worksfor_merged_model_top1_ranking_spacy_one}')

results_list_worksfor_merged_model_top1_ranking_normal_one, _, average_worksfor_merged_model_top1_ranking_normal_one = create_results_list(
    ranking_worksfor_merged_model_normal_tokens_one, worksfor_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal: {average_worksfor_merged_model_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_worksfor_merged_model_top1_borda_normal_one, _, average_worksfor_merged_model_top1_borda_normal_one = create_results_list(
    worksfor_merged_model_borda_tokens_score_one, worksfor_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal: {average_worksfor_merged_model_top1_borda_normal_one}')
print('\n')

results_list_worksfor_merged_model_top1_borda_spacy_one, _, average_worksfor_merged_model_top1_borda_spacy_one = create_results_list(
    worksfor_merged_model_borda_spacy_tokens_score_one, worksfor_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In spacy: {average_worksfor_merged_model_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top1_spacy_one, _, average_worksfor_bert_top1_spacy_one = create_results_list(
    masked_worksfor_bert_original_spacy_one, worksfor_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Spacy: {average_worksfor_bert_top1_spacy_one}')

results_list_worksfor_bert_top1_one, _, average_worksfor_bert_top1_one = create_results_list(
    masked_worksfor_bert_original_one, worksfor_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Normal: {average_worksfor_bert_top1_one}')

results_list_worksfor_roberta_top1_spacy_one, _, average_worksfor_roberta_top1_spacy_one = create_results_list(
    masked_worksfor_roberta_original_spacy_one, worksfor_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Spacy: {average_worksfor_roberta_top1_spacy_one}')

results_list_worksfor_roberta_top1_one, _, average_worksfor_roberta_top1_one = create_results_list(
    masked_worksfor_roberta_original, worksfor_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Normal: {average_worksfor_roberta_top1_one}')

results_list_worksfor_electra_top1_spacy_one, _, average_worksfor_electra_top1_spacy_one = create_results_list(
    masked_worksfor_electra_original_spacy_one, worksfor_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Spacy: {average_worksfor_electra_top1_spacy_one}')

results_list_worksfor_electra_top1_one, _, average_worksfor_electra_top1_one = create_results_list(
    masked_worksfor_electra_original_one, worksfor_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Normal: {average_worksfor_electra_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy: 0.662
Avg Precision K = 1 of the ranking of died In Normal: 0.573


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal: 0.583


Borda Spacy Score
Avg Precision K = 1 of the borda of died In spacy: 0.654


Sem ranking
Avg Precision K = 1 of died In Bert Spacy: 0.604
Avg Precision K = 1 of died In Bert Normal: 0.512
Avg Precision K = 1 of died In roberta Spacy: 0.673
Avg Precision K = 1 of died In roberta Normal: 0.594
Avg Precision K = 1 of died In electra Spacy: 0.485
Avg Precision K = 1 of died In electra Normal: 0.396


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_worksfor_merged_model_top5_ranking_spacy_one, _, average_worksfor_merged_model_top5_ranking_spacy_one = create_results_list(
    ranking_worksfor_merged_model_spacy_tokens_one, worksfor_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy: {average_worksfor_merged_model_top5_ranking_spacy_one}')

results_list_worksfor_merged_model_top5_ranking_normal_one, _, average_worksfor_merged_model_top5_ranking_normal_one = create_results_list(
    ranking_worksfor_merged_model_normal_tokens_one, worksfor_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal: {average_worksfor_merged_model_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_worksfor_merged_model_top5_borda_normal_one, _, average_worksfor_merged_model_top5_borda_normal_one = create_results_list(
    worksfor_merged_model_borda_tokens_score_one, worksfor_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal: {average_worksfor_merged_model_top5_borda_normal_one}')
print('\n')

results_list_worksfor_merged_model_top5_borda_spacy_one, _, average_worksfor_merged_model_top5_borda_spacy_one = create_results_list(
    worksfor_merged_model_borda_spacy_tokens_score_one, worksfor_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In spacy: {average_worksfor_merged_model_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top5_spacy_one, _, average_worksfor_bert_top5_spacy_one = create_results_list(
    masked_worksfor_bert_original_spacy_one, worksfor_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Spacy: {average_worksfor_bert_top5_spacy_one}')

results_list_worksfor_bert_top5_one, _, average_worksfor_bert_top5_one = create_results_list(
    masked_worksfor_bert_original_one, worksfor_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Normal: {average_worksfor_bert_top5_one}')

results_list_worksfor_roberta_top5_spacy_one, _, average_worksfor_roberta_top5_spacy_one = create_results_list(
    masked_worksfor_roberta_original_spacy_one, worksfor_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Spacy: {average_worksfor_roberta_top5_spacy_one}')

results_list_worksfor_roberta_top5_one, _, average_worksfor_roberta_top5_one = create_results_list(
    masked_worksfor_roberta_original, worksfor_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Normal: {average_worksfor_roberta_top5_one}')

results_list_worksfor_electra_top5_spacy_one, _, average_worksfor_electra_top5_spacy_one = create_results_list(
    masked_worksfor_electra_original_spacy_one, worksfor_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Spacy: {average_worksfor_electra_top5_spacy_one}')

results_list_worksfor_electra_top5_one, _, average_worksfor_electra_top5_one = create_results_list(
    masked_worksfor_electra_original_one, worksfor_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Normal: {average_worksfor_electra_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy: 0.858
Avg Precision K = 5 of the ranking of died In Normal: 0.842


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal: 0.839


Borda Spacy Score
Avg Precision K = 5 of the borda of died In spacy: 0.868


Sem ranking
Avg Precision K = 5 of died In Bert Spacy: 0.802
Avg Precision K = 5 of died In Bert Normal: 0.792
Avg Precision K = 5 of died In roberta Spacy: 0.834
Avg Precision K = 5 of died In roberta Normal: 0.792
Avg Precision K = 5 of died In electra Spacy: 0.686
Avg Precision K = 5 of died In electra Normal: 0.646


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_worksfor_merged_model_top10_ranking_spacy_one, _, average_worksfor_merged_model_top10_ranking_spacy_one = create_results_list(
    ranking_worksfor_merged_model_spacy_tokens_one, worksfor_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy: {average_worksfor_merged_model_top10_ranking_spacy_one}')

results_list_worksfor_merged_model_top10_ranking_normal_one, _, average_worksfor_merged_model_top10_ranking_normal_one = create_results_list(
    ranking_worksfor_merged_model_normal_tokens_one, worksfor_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal: {average_worksfor_merged_model_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_worksfor_merged_model_top10_borda_normal_one, _, average_worksfor_merged_model_top10_borda_normal_one = create_results_list(
    worksfor_merged_model_borda_tokens_score_one, worksfor_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal: {average_worksfor_merged_model_top10_borda_normal_one}')
print('\n')

results_list_worksfor_merged_model_top10_borda_spacy_one, _, average_worksfor_merged_model_top10_borda_spacy_one = create_results_list(
    worksfor_merged_model_borda_spacy_tokens_score_one, worksfor_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In spacy: {average_worksfor_merged_model_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top10_spacy_one, _, average_worksfor_bert_top10_spacy_one = create_results_list(
    masked_worksfor_bert_original_spacy_one, worksfor_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Spacy: {average_worksfor_bert_top10_spacy_one}')

results_list_worksfor_bert_top10_one, _, average_worksfor_bert_top10_one = create_results_list(
    masked_worksfor_bert_original_one, worksfor_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Normal: {average_worksfor_bert_top10_one}')

results_list_worksfor_roberta_top10_spacy_one, _, average_worksfor_roberta_top10_spacy_one = create_results_list(
    masked_worksfor_roberta_original_spacy_one, worksfor_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Spacy: {average_worksfor_roberta_top10_spacy_one}')

results_list_worksfor_roberta_top10_one, _, average_worksfor_roberta_top10_one = create_results_list(
    masked_worksfor_roberta_original, worksfor_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Normal: {average_worksfor_roberta_top10_one}')

results_list_worksfor_electra_top10_spacy_one, _, average_worksfor_electra_top10_spacy_one = create_results_list(
    masked_worksfor_electra_original_spacy_one, worksfor_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Spacy: {average_worksfor_electra_top10_spacy_one}')

results_list_worksfor_electra_top10_one, _, average_worksfor_electra_top10_one = create_results_list(
    masked_worksfor_electra_original_one, worksfor_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Normal: {average_worksfor_electra_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy: 0.887
Avg Precision K = 10 of the ranking of died In Normal: 0.902


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal: 0.902


Borda Spacy Score
Avg Precision K = 10 of the borda of died In spacy: 0.887


Sem ranking
Avg Precision K = 10 of died In Bert Spacy: 0.821
Avg Precision K = 10 of died In Bert Normal: 0.86
Avg Precision K = 10 of died In roberta Spacy: 0.852
Avg Precision K = 10 of died In roberta Normal: 0.855
Avg Precision K = 10 of died In electra Spacy: 0.704
Avg Precision K = 10 of died In electra Normal: 0.731


#### Capital

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_capital_merged_model_top1_ranking_spacy_one, _, average_capital_merged_model_top1_ranking_spacy_one = create_results_list(
    ranking_capital_merged_model_spacy_tokens_one, capital_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy: {average_capital_merged_model_top1_ranking_spacy_one}')

results_list_capital_merged_model_top1_ranking_normal_one, _, average_capital_merged_model_top1_ranking_normal_one = create_results_list(
    ranking_capital_merged_model_normal_tokens_one, capital_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal: {average_capital_merged_model_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_capital_merged_model_top1_borda_normal_one, _, average_capital_merged_model_top1_borda_normal_one = create_results_list(
    capital_merged_model_borda_tokens_score_one, capital_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal: {average_capital_merged_model_top1_borda_normal_one}')
print('\n')

results_list_capital_merged_model_top1_borda_spacy_one, _, average_capital_merged_model_top1_borda_spacy_one = create_results_list(
    capital_merged_model_borda_spacy_tokens_score_one, capital_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In spacy: {average_capital_merged_model_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top1_spacy_one, _, average_capital_bert_top1_spacy_one = create_results_list(
    masked_capital_bert_original_spacy_one, capital_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Spacy: {average_capital_bert_top1_spacy_one}')

results_list_capital_bert_top1_one, _, average_capital_bert_top1_one = create_results_list(
    masked_capital_bert_original_one, capital_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Normal: {average_capital_bert_top1_one}')

results_list_capital_roberta_top1_spacy_one, _, average_capital_roberta_top1_spacy_one = create_results_list(
    masked_capital_roberta_original_spacy_one, capital_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Spacy: {average_capital_roberta_top1_spacy_one}')

results_list_capital_roberta_top1_one, _, average_capital_roberta_top1_one = create_results_list(
    masked_capital_roberta_original, capital_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Normal: {average_capital_roberta_top1_one}')

results_list_capital_electra_top1_spacy_one, _, average_capital_electra_top1_spacy_one = create_results_list(
    masked_capital_electra_original_spacy_one, capital_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Spacy: {average_capital_electra_top1_spacy_one}')

results_list_capital_electra_top1_one, _, average_capital_electra_top1_one = create_results_list(
    masked_capital_electra_original_one, capital_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Normal: {average_capital_electra_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy: 0.465
Avg Precision K = 1 of the ranking of died In Normal: 0.449


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal: 0.452


Borda Spacy Score
Avg Precision K = 1 of the borda of died In spacy: 0.468


Sem ranking
Avg Precision K = 1 of died In Bert Spacy: 0.455
Avg Precision K = 1 of died In Bert Normal: 0.457
Avg Precision K = 1 of died In roberta Spacy: 0.33
Avg Precision K = 1 of died In roberta Normal: 0.367
Avg Precision K = 1 of died In electra Spacy: 0.242
Avg Precision K = 1 of died In electra Normal: 0.237


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_capital_merged_model_top5_ranking_spacy_one, _, average_capital_merged_model_top5_ranking_spacy_one = create_results_list(
    ranking_capital_merged_model_spacy_tokens_one, capital_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy: {average_capital_merged_model_top5_ranking_spacy_one}')

results_list_capital_merged_model_top5_ranking_normal_one, _, average_capital_merged_model_top5_ranking_normal_one = create_results_list(
    ranking_capital_merged_model_normal_tokens_one, capital_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal: {average_capital_merged_model_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_capital_merged_model_top5_borda_normal_one, _, average_capital_merged_model_top5_borda_normal_one = create_results_list(
    capital_merged_model_borda_tokens_score_one, capital_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal: {average_capital_merged_model_top5_borda_normal_one}')
print('\n')

results_list_capital_merged_model_top5_borda_spacy_one, _, average_capital_merged_model_top5_borda_spacy_one = create_results_list(
    capital_merged_model_borda_spacy_tokens_score_one, capital_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In spacy: {average_capital_merged_model_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top5_spacy_one, _, average_capital_bert_top5_spacy_one = create_results_list(
    masked_capital_bert_original_spacy_one, capital_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Spacy: {average_capital_bert_top5_spacy_one}')

results_list_capital_bert_top5_one, _, average_capital_bert_top5_one = create_results_list(
    masked_capital_bert_original_one, capital_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Normal: {average_capital_bert_top5_one}')

results_list_capital_roberta_top5_spacy_one, _, average_capital_roberta_top5_spacy_one = create_results_list(
    masked_capital_roberta_original_spacy_one, capital_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Spacy: {average_capital_roberta_top5_spacy_one}')

results_list_capital_roberta_top5_one, _, average_capital_roberta_top5_one = create_results_list(
    masked_capital_roberta_original, capital_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Normal: {average_capital_roberta_top5_one}')

results_list_capital_electra_top5_spacy_one, _, average_capital_electra_top5_spacy_one = create_results_list(
    masked_capital_electra_original_spacy_one, capital_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Spacy: {average_capital_electra_top5_spacy_one}')

results_list_capital_electra_top5_one, _, average_capital_electra_top5_one = create_results_list(
    masked_capital_electra_original_one, capital_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Normal: {average_capital_electra_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy: 0.662
Avg Precision K = 5 of the ranking of died In Normal: 0.668


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal: 0.661


Borda Spacy Score
Avg Precision K = 5 of the borda of died In spacy: 0.665


Sem ranking
Avg Precision K = 5 of died In Bert Spacy: 0.638
Avg Precision K = 5 of died In Bert Normal: 0.655
Avg Precision K = 5 of died In roberta Spacy: 0.43
Avg Precision K = 5 of died In roberta Normal: 0.498
Avg Precision K = 5 of died In electra Spacy: 0.405
Avg Precision K = 5 of died In electra Normal: 0.411


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_capital_merged_model_top10_ranking_spacy_one, _, average_capital_merged_model_top10_ranking_spacy_one = create_results_list(
    ranking_capital_merged_model_spacy_tokens_one, capital_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy: {average_capital_merged_model_top10_ranking_spacy_one}')

results_list_capital_merged_model_top10_ranking_normal_one, _, average_capital_merged_model_top10_ranking_normal_one = create_results_list(
    ranking_capital_merged_model_normal_tokens_one, capital_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal: {average_capital_merged_model_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_capital_merged_model_top10_borda_normal_one, _, average_capital_merged_model_top10_borda_normal_one = create_results_list(
    capital_merged_model_borda_tokens_score_one, capital_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal: {average_capital_merged_model_top10_borda_normal_one}')
print('\n')

results_list_capital_merged_model_top10_borda_spacy_one, _, average_capital_merged_model_top10_borda_spacy_one = create_results_list(
    capital_merged_model_borda_spacy_tokens_score_one, capital_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In spacy: {average_capital_merged_model_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top10_spacy_one, _, average_capital_bert_top10_spacy_one = create_results_list(
    masked_capital_bert_original_spacy_one, capital_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Spacy: {average_capital_bert_top10_spacy_one}')

results_list_capital_bert_top10_one, _, average_capital_bert_top10_one = create_results_list(
    masked_capital_bert_original_one, capital_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Normal: {average_capital_bert_top10_one}')

results_list_capital_roberta_top10_spacy_one, _, average_capital_roberta_top10_spacy_one = create_results_list(
    masked_capital_roberta_original_spacy_one, capital_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Spacy: {average_capital_roberta_top10_spacy_one}')

results_list_capital_roberta_top10_one, _, average_capital_roberta_top10_one = create_results_list(
    masked_capital_roberta_original, capital_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Normal: {average_capital_roberta_top10_one}')

results_list_capital_electra_top10_spacy_one, _, average_capital_electra_top10_spacy_one = create_results_list(
    masked_capital_electra_original_spacy_one, capital_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Spacy: {average_capital_electra_top10_spacy_one}')

results_list_capital_electra_top10_one, _, average_capital_electra_top10_one = create_results_list(
    masked_capital_electra_original_one, capital_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Normal: {average_capital_electra_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy: 0.709
Avg Precision K = 10 of the ranking of died In Normal: 0.734


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal: 0.737


Borda Spacy Score
Avg Precision K = 10 of the borda of died In spacy: 0.713


Sem ranking
Avg Precision K = 10 of died In Bert Spacy: 0.668
Avg Precision K = 10 of died In Bert Normal: 0.707
Avg Precision K = 10 of died In roberta Spacy: 0.449
Avg Precision K = 10 of died In roberta Normal: 0.519
Avg Precision K = 10 of died In electra Spacy: 0.462
Avg Precision K = 10 of died In electra Normal: 0.485


#### Citizen

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_citizen_merged_model_top1_ranking_spacy_one, _, average_citizen_merged_model_top1_ranking_spacy_one = create_results_list(
    ranking_citizen_merged_model_spacy_tokens_one, citizen_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy: {average_citizen_merged_model_top1_ranking_spacy_one}')

results_list_citizen_merged_model_top1_ranking_normal_one, _, average_citizen_merged_model_top1_ranking_normal_one = create_results_list(
    ranking_citizen_merged_model_normal_tokens_one, citizen_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal: {average_citizen_merged_model_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_citizen_merged_model_top1_borda_normal_one, _, average_citizen_merged_model_top1_borda_normal_one = create_results_list(
    citizen_merged_model_borda_tokens_score_one, citizen_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal: {average_citizen_merged_model_top1_borda_normal_one}')
print('\n')

results_list_citizen_merged_model_top1_borda_spacy_one, _, average_citizen_merged_model_top1_borda_spacy_one = create_results_list(
    citizen_merged_model_borda_spacy_tokens_score_one, citizen_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In spacy: {average_citizen_merged_model_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top1_spacy_one, _, average_citizen_bert_top1_spacy_one = create_results_list(
    masked_citizen_bert_original_spacy_one, citizen_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Spacy: {average_citizen_bert_top1_spacy_one}')

results_list_citizen_bert_top1_one, _, average_citizen_bert_top1_one = create_results_list(
    masked_citizen_bert_original_one, citizen_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Normal: {average_citizen_bert_top1_one}')

results_list_citizen_roberta_top1_spacy_one, _, average_citizen_roberta_top1_spacy_one = create_results_list(
    masked_citizen_roberta_original_spacy_one, citizen_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Spacy: {average_citizen_roberta_top1_spacy_one}')

results_list_citizen_roberta_top1_one, _, average_citizen_roberta_top1_one = create_results_list(
    masked_citizen_roberta_original, citizen_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Normal: {average_citizen_roberta_top1_one}')

results_list_citizen_electra_top1_spacy_one, _, average_citizen_electra_top1_spacy_one = create_results_list(
    masked_citizen_electra_original_spacy_one, citizen_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Spacy: {average_citizen_electra_top1_spacy_one}')

results_list_citizen_electra_top1_one, _, average_citizen_electra_top1_one = create_results_list(
    masked_citizen_electra_original_one, citizen_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Normal: {average_citizen_electra_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy: 0.428
Avg Precision K = 1 of the ranking of died In Normal: 0.576


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal: 0.572


Borda Spacy Score
Avg Precision K = 1 of the borda of died In spacy: 0.423


Sem ranking
Avg Precision K = 1 of died In Bert Spacy: 0.414
Avg Precision K = 1 of died In Bert Normal: 0.573
Avg Precision K = 1 of died In roberta Spacy: 0.405
Avg Precision K = 1 of died In roberta Normal: 0.567
Avg Precision K = 1 of died In electra Spacy: 0.277
Avg Precision K = 1 of died In electra Normal: 0.382


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_citizen_merged_model_top5_ranking_spacy_one, _, average_citizen_merged_model_top5_ranking_spacy_one = create_results_list(
    ranking_citizen_merged_model_spacy_tokens_one, citizen_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy: {average_citizen_merged_model_top5_ranking_spacy_one}')

results_list_citizen_merged_model_top5_ranking_normal_one, _, average_citizen_merged_model_top5_ranking_normal_one = create_results_list(
    ranking_citizen_merged_model_normal_tokens_one, citizen_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal: {average_citizen_merged_model_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_citizen_merged_model_top5_borda_normal_one, _, average_citizen_merged_model_top5_borda_normal_one = create_results_list(
    citizen_merged_model_borda_tokens_score_one, citizen_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal: {average_citizen_merged_model_top5_borda_normal_one}')
print('\n')

results_list_citizen_merged_model_top5_borda_spacy_one, _, average_citizen_merged_model_top5_borda_spacy_one = create_results_list(
    citizen_merged_model_borda_spacy_tokens_score_one, citizen_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In spacy: {average_citizen_merged_model_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top5_spacy_one, _, average_citizen_bert_top5_spacy_one = create_results_list(
    masked_citizen_bert_original_spacy_one, citizen_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Spacy: {average_citizen_bert_top5_spacy_one}')

results_list_citizen_bert_top5_one, _, average_citizen_bert_top5_one = create_results_list(
    masked_citizen_bert_original_one, citizen_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Normal: {average_citizen_bert_top5_one}')

results_list_citizen_roberta_top5_spacy_one, _, average_citizen_roberta_top5_spacy_one = create_results_list(
    masked_citizen_roberta_original_spacy_one, citizen_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Spacy: {average_citizen_roberta_top5_spacy_one}')

results_list_citizen_roberta_top5_one, _, average_citizen_roberta_top5_one = create_results_list(
    masked_citizen_roberta_original, citizen_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Normal: {average_citizen_roberta_top5_one}')

results_list_citizen_electra_top5_spacy_one, _, average_citizen_electra_top5_spacy_one = create_results_list(
    masked_citizen_electra_original_spacy_one, citizen_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Spacy: {average_citizen_electra_top5_spacy_one}')

results_list_citizen_electra_top5_one, _, average_citizen_electra_top5_one = create_results_list(
    masked_citizen_electra_original_one, citizen_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Normal: {average_citizen_electra_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy: 0.503
Avg Precision K = 5 of the ranking of died In Normal: 0.701


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal: 0.699


Borda Spacy Score
Avg Precision K = 5 of the borda of died In spacy: 0.501


Sem ranking
Avg Precision K = 5 of died In Bert Spacy: 0.478
Avg Precision K = 5 of died In Bert Normal: 0.722
Avg Precision K = 5 of died In roberta Spacy: 0.472
Avg Precision K = 5 of died In roberta Normal: 0.709
Avg Precision K = 5 of died In electra Spacy: 0.357
Avg Precision K = 5 of died In electra Normal: 0.56


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_citizen_merged_model_top10_ranking_spacy_one, _, average_citizen_merged_model_top10_ranking_spacy_one = create_results_list(
    ranking_citizen_merged_model_spacy_tokens_one, citizen_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy: {average_citizen_merged_model_top10_ranking_spacy_one}')

results_list_citizen_merged_model_top10_ranking_normal_one, _, average_citizen_merged_model_top10_ranking_normal_one = create_results_list(
    ranking_citizen_merged_model_normal_tokens_one, citizen_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal: {average_citizen_merged_model_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_citizen_merged_model_top10_borda_normal_one, _, average_citizen_merged_model_top10_borda_normal_one = create_results_list(
    citizen_merged_model_borda_tokens_score_one, citizen_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal: {average_citizen_merged_model_top10_borda_normal_one}')
print('\n')

results_list_citizen_merged_model_top10_borda_spacy_one, _, average_citizen_merged_model_top10_borda_spacy_one = create_results_list(
    citizen_merged_model_borda_spacy_tokens_score_one, citizen_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In spacy: {average_citizen_merged_model_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top10_spacy_one, _, average_citizen_bert_top10_spacy_one = create_results_list(
    masked_citizen_bert_original_spacy_one, citizen_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Spacy: {average_citizen_bert_top10_spacy_one}')

results_list_citizen_bert_top10_one, _, average_citizen_bert_top10_one = create_results_list(
    masked_citizen_bert_original_one, citizen_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Normal: {average_citizen_bert_top10_one}')

results_list_citizen_roberta_top10_spacy_one, _, average_citizen_roberta_top10_spacy_one = create_results_list(
    masked_citizen_roberta_original_spacy_one, citizen_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Spacy: {average_citizen_roberta_top10_spacy_one}')

results_list_citizen_roberta_top10_one, _, average_citizen_roberta_top10_one = create_results_list(
    masked_citizen_roberta_original, citizen_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Normal: {average_citizen_roberta_top10_one}')

results_list_citizen_electra_top10_spacy_one, _, average_citizen_electra_top10_spacy_one = create_results_list(
    masked_citizen_electra_original_spacy_one, citizen_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Spacy: {average_citizen_electra_top10_spacy_one}')

results_list_citizen_electra_top10_one, _, average_citizen_electra_top10_one = create_results_list(
    masked_citizen_electra_original_one, citizen_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Normal: {average_citizen_electra_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy: 0.512
Avg Precision K = 10 of the ranking of died In Normal: 0.757


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal: 0.758


Borda Spacy Score
Avg Precision K = 10 of the borda of died In spacy: 0.512


Sem ranking
Avg Precision K = 10 of died In Bert Spacy: 0.482
Avg Precision K = 10 of died In Bert Normal: 0.79
Avg Precision K = 10 of died In roberta Spacy: 0.473
Avg Precision K = 10 of died In roberta Normal: 0.763
Avg Precision K = 10 of died In electra Spacy: 0.366
Avg Precision K = 10 of died In electra Normal: 0.613


#### Language

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_language_merged_model_top1_ranking_spacy_one, _, average_language_merged_model_top1_ranking_spacy_one = create_results_list(
    ranking_language_merged_model_spacy_tokens_one, language_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy: {average_language_merged_model_top1_ranking_spacy_one}')

results_list_language_merged_model_top1_ranking_normal_one, _, average_language_merged_model_top1_ranking_normal_one = create_results_list(
    ranking_language_merged_model_normal_tokens_one, language_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal: {average_language_merged_model_top1_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_language_merged_model_top1_borda_normal_one, _, average_language_merged_model_top1_borda_normal_one = create_results_list(
    language_merged_model_borda_tokens_score_one, language_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal: {average_language_merged_model_top1_borda_normal_one}')
print('\n')

results_list_language_merged_model_top1_borda_spacy_one, _, average_language_merged_model_top1_borda_spacy_one = create_results_list(
    language_merged_model_borda_spacy_tokens_score_one, language_merged_model_spacy_one.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In spacy: {average_language_merged_model_top1_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_language_bert_top1_spacy_one, _, average_language_bert_top1_spacy_one = create_results_list(
    masked_language_bert_original_spacy_one, language_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Spacy: {average_language_bert_top1_spacy_one}')

results_list_language_bert_top1_one, _, average_language_bert_top1_one = create_results_list(
    masked_language_bert_original_one, language_bert_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Normal: {average_language_bert_top1_one}')

results_list_language_roberta_top1_spacy_one, _, average_language_roberta_top1_spacy_one = create_results_list(
    masked_language_roberta_original_spacy_one, language_roberta_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Spacy: {average_language_roberta_top1_spacy_one}')

results_list_language_roberta_top1_one, _, average_language_roberta_top1_one = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Normal: {average_language_roberta_top1_one}')

results_list_language_electra_top1_spacy_one, _, average_language_electra_top1_spacy_one = create_results_list(
    masked_language_electra_original_spacy_one, language_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Spacy: {average_language_electra_top1_spacy_one}')

results_list_language_electra_top1_one, _, average_language_electra_top1_one = create_results_list(
    masked_language_electra_original_one, language_electra_one.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Normal: {average_language_electra_top1_one}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy: 0.667
Avg Precision K = 1 of the ranking of died In Normal: 0.703


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal: 0.694


Borda Spacy Score
Avg Precision K = 1 of the borda of died In spacy: 0.659


Sem ranking
Avg Precision K = 1 of died In Bert Spacy: 0.657
Avg Precision K = 1 of died In Bert Normal: 0.709
Avg Precision K = 1 of died In roberta Spacy: 0.59
Avg Precision K = 1 of died In roberta Normal: 0.639
Avg Precision K = 1 of died In electra Spacy: 0.396
Avg Precision K = 1 of died In electra Normal: 0.362


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_language_merged_model_top5_ranking_spacy_one, _, average_language_merged_model_top5_ranking_spacy_one = create_results_list(
    ranking_language_merged_model_spacy_tokens_one, language_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy: {average_language_merged_model_top5_ranking_spacy_one}')

results_list_language_merged_model_top5_ranking_normal_one, _, average_language_merged_model_top5_ranking_normal_one = create_results_list(
    ranking_language_merged_model_normal_tokens_one, language_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal: {average_language_merged_model_top5_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_language_merged_model_top5_borda_normal_one, _, average_language_merged_model_top5_borda_normal_one = create_results_list(
    language_merged_model_borda_tokens_score_one, language_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal: {average_language_merged_model_top5_borda_normal_one}')
print('\n')

results_list_language_merged_model_top5_borda_spacy_one, _, average_language_merged_model_top5_borda_spacy_one = create_results_list(
    language_merged_model_borda_spacy_tokens_score_one, language_merged_model_spacy_one.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In spacy: {average_language_merged_model_top5_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_language_bert_top5_spacy_one, _, average_language_bert_top5_spacy_one = create_results_list(
    masked_language_bert_original_spacy_one, language_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Spacy: {average_language_bert_top5_spacy_one}')

results_list_language_bert_top5_one, _, average_language_bert_top5_one = create_results_list(
    masked_language_bert_original_one, language_bert_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Normal: {average_language_bert_top5_one}')

results_list_language_roberta_top5_spacy_one, _, average_language_roberta_top5_spacy_one = create_results_list(
    masked_language_roberta_original_spacy_one, language_roberta_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Spacy: {average_language_roberta_top5_spacy_one}')

results_list_language_roberta_top5_one, _, average_language_roberta_top5_one = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Normal: {average_language_roberta_top5_one}')

results_list_language_electra_top5_spacy_one, _, average_language_electra_top5_spacy_one = create_results_list(
    masked_language_electra_original_spacy_one, language_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Spacy: {average_language_electra_top5_spacy_one}')

results_list_language_electra_top5_one, _, average_language_electra_top5_one = create_results_list(
    masked_language_electra_original_one, language_electra_one.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Normal: {average_language_electra_top5_one}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy: 0.788
Avg Precision K = 5 of the ranking of died In Normal: 0.87


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal: 0.872


Borda Spacy Score
Avg Precision K = 5 of the borda of died In spacy: 0.788


Sem ranking
Avg Precision K = 5 of died In Bert Spacy: 0.766
Avg Precision K = 5 of died In Bert Normal: 0.871
Avg Precision K = 5 of died In roberta Spacy: 0.704
Avg Precision K = 5 of died In roberta Normal: 0.785
Avg Precision K = 5 of died In electra Spacy: 0.63
Avg Precision K = 5 of died In electra Normal: 0.674


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_language_merged_model_top10_ranking_spacy_one, _, average_language_merged_model_top10_ranking_spacy_one = create_results_list(
    ranking_language_merged_model_spacy_tokens_one, language_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy: {average_language_merged_model_top10_ranking_spacy_one}')

results_list_language_merged_model_top10_ranking_normal_one, _, average_language_merged_model_top10_ranking_normal_one = create_results_list(
    ranking_language_merged_model_normal_tokens_one, language_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal: {average_language_merged_model_top10_ranking_normal_one}')
print('\n')

print('Borda Count')
results_list_language_merged_model_top10_borda_normal_one, _, average_language_merged_model_top10_borda_normal_one = create_results_list(
    language_merged_model_borda_tokens_score_one, language_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal: {average_language_merged_model_top10_borda_normal_one}')
print('\n')

results_list_language_merged_model_top10_borda_spacy_one, _, average_language_merged_model_top10_borda_spacy_one = create_results_list(
    language_merged_model_borda_spacy_tokens_score_one, language_merged_model_spacy_one.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In spacy: {average_language_merged_model_top10_borda_spacy_one}')
print('\n')

print('Sem ranking')
results_list_language_bert_top10_spacy_one, _, average_language_bert_top10_spacy_one = create_results_list(
    masked_language_bert_original_spacy_one, language_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Spacy: {average_language_bert_top10_spacy_one}')

results_list_language_bert_top10_one, _, average_language_bert_top10_one = create_results_list(
    masked_language_bert_original_one, language_bert_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Normal: {average_language_bert_top10_one}')

results_list_language_roberta_top10_spacy_one, _, average_language_roberta_top10_spacy_one = create_results_list(
    masked_language_roberta_original_spacy_one, language_roberta_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Spacy: {average_language_roberta_top10_spacy_one}')

results_list_language_roberta_top10_one, _, average_language_roberta_top10_one = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Normal: {average_language_roberta_top10_one}')

results_list_language_electra_top10_spacy_one, _, average_language_electra_top10_spacy_one = create_results_list(
    masked_language_electra_original_spacy_one, language_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Spacy: {average_language_electra_top10_spacy_one}')

results_list_language_electra_top10_one, _, average_language_electra_top10_one = create_results_list(
    masked_language_electra_original_one, language_electra_one.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Normal: {average_language_electra_top10_one}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy: 0.804
Avg Precision K = 10 of the ranking of died In Normal: 0.914


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal: 0.913


Borda Spacy Score
Avg Precision K = 10 of the borda of died In spacy: 0.802


Sem ranking
Avg Precision K = 10 of died In Bert Spacy: 0.771
Avg Precision K = 10 of died In Bert Normal: 0.907
Avg Precision K = 10 of died In roberta Spacy: 0.709
Avg Precision K = 10 of died In roberta Normal: 0.831
Avg Precision K = 10 of died In electra Spacy: 0.647
Avg Precision K = 10 of died In electra Normal: 0.762


## Few Shot

### Bert

#### BornIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_bornIn_bert_top1_ranking_spacy_few, _, average_bornIn_bert_top1_ranking_spacy_few = create_results_list(
    ranking_bornIn_bert_spacy_tokens_few, bornIn_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Spacy few Shoot: {average_bornIn_bert_top1_ranking_spacy_few}')

results_list_bornIn_bert_top1_ranking_normal_few, _, average_bornIn_bert_top1_ranking_normal_few = create_results_list(
    ranking_bornIn_bert_normal_tokens_few, bornIn_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Normal few Shoot: {average_bornIn_bert_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_bornIn_bert_top1_borda_normal_few, _, average_bornIn_bert_top1_borda_normal_few = create_results_list(
    bornIn_bert_borda_tokens_score_few, bornIn_bert_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Born In Normal few Shoot: {average_bornIn_bert_top1_borda_normal_few}')
print('\n')

results_list_bornIn_bert_top1_borda_spacy_few, _, average_bornIn_bert_top1_borda_spacy_few = create_results_list(
    bornIn_bert_borda_spacy_tokens_score_few, bornIn_bert_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Born In Spacy few Shoot: {average_bornIn_bert_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top1_spacy_few, _, average_bornIn_bert_top1_spacy_few = create_results_list(
    masked_bornIn_bert_original_spacy_few, bornIn_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Spacy few Shoot: {average_bornIn_bert_top1_spacy_few}')

results_list_bornIn_bert_top1_few, _, average_bornIn_bert_top1_few = create_results_list(
    masked_bornIn_bert_original_few, bornIn_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Normal few Shoot: {average_bornIn_bert_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Born In Spacy few Shoot: 0.313
Avg Precision K = 1 of the ranking of Born In Normal few Shoot: 0.335


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Born In Normal few Shoot: 0.337


Borda Spacy Score
Avg Precision K = 1 of the borda of Born In Spacy few Shoot: 0.315


Sem ranking
Avg Precision K = 1 of Born In Spacy few Shoot: 0.379
Avg Precision K = 1 of Born In Normal few Shoot: 0.367


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_bornIn_bert_top5_ranking_spacy_few, _, average_bornIn_bert_top5_ranking_spacy_few = create_results_list(
    ranking_bornIn_bert_spacy_tokens_few, bornIn_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Spacy few Shoot: {average_bornIn_bert_top5_ranking_spacy_few}')

results_list_bornIn_bert_top5_ranking_normal_few, _, average_bornIn_bert_top5_ranking_normal_few = create_results_list(
    ranking_bornIn_bert_normal_tokens_few, bornIn_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Normal few Shoot: {average_bornIn_bert_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_bornIn_bert_top5_borda_normal_few, _, average_bornIn_bert_top5_borda_normal_few = create_results_list(
    bornIn_bert_borda_tokens_score_few, bornIn_bert_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Born In Normal few Shoot: {average_bornIn_bert_top5_borda_normal_few}')
print('\n')

results_list_bornIn_bert_top5_borda_spacy_few, _, average_bornIn_bert_top5_borda_spacy_few = create_results_list(
    bornIn_bert_borda_spacy_tokens_score_few, bornIn_bert_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Born In Spacy few Shoot: {average_bornIn_bert_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top5_spacy_few, _, average_bornIn_bert_top5_spacy_few = create_results_list(
    masked_bornIn_bert_original_spacy_few, bornIn_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Spacy few Shoot: {average_bornIn_bert_top5_spacy_few}')

results_list_bornIn_bert_top5_few, _, average_bornIn_bert_top5_few = create_results_list(
    masked_bornIn_bert_original_few, bornIn_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Normal: {average_bornIn_bert_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Born In Spacy few Shoot: 0.588
Avg Precision K = 5 of the ranking of Born In Normal few Shoot: 0.618


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Born In Normal few Shoot: 0.616


Borda Spacy Score
Avg Precision K = 5 of the borda of Born In Spacy few Shoot: 0.591


Sem ranking
Avg Precision K = 5 of Born In Spacy few Shoot: 0.601
Avg Precision K = 5 of Born In Normal: 0.594


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_bornIn_bert_top10_ranking_spacy_few, _, average_bornIn_bert_top10_ranking_spacy_few = create_results_list(
    ranking_bornIn_bert_spacy_tokens_few, bornIn_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Spacy few Shoot: {average_bornIn_bert_top10_ranking_spacy_few}')

results_list_bornIn_bert_top10_ranking_normal_few, _, average_bornIn_bert_top10_ranking_normal_few = create_results_list(
    ranking_bornIn_bert_normal_tokens_few, bornIn_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Normal few Shoot: {average_bornIn_bert_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_bornIn_bert_top10_borda_normal_few, _, average_bornIn_bert_top10_borda_normal_few = create_results_list(
    bornIn_bert_borda_tokens_score_few, bornIn_bert_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Born In Normal few Shoot: {average_bornIn_bert_top10_borda_normal_few}')
print('\n')

results_list_bornIn_bert_top10_borda_spacy_few, _, average_bornIn_bert_top10_borda_spacy_few = create_results_list(
    bornIn_bert_borda_spacy_tokens_score_few, bornIn_bert_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Born In Spacy few Shoot: {average_bornIn_bert_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top10_spacy_few, _, average_bornIn_bert_top10_spacy_few = create_results_list(
    masked_bornIn_bert_original_spacy_few, bornIn_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Spacy few Shoot: {average_bornIn_bert_top10_spacy_few}')

results_list_bornIn_bert_top10_few, _, average_bornIn_bert_top10_few = create_results_list(
    masked_bornIn_bert_original_few, bornIn_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Normal: {average_bornIn_bert_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Born In Spacy few Shoot: 0.659
Avg Precision K = 10 of the ranking of Born In Normal few Shoot: 0.687


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Born In Normal few Shoot: 0.688


Borda Spacy Score
Avg Precision K = 10 of the borda of Born In Spacy few Shoot: 0.658


Sem ranking
Avg Precision K = 10 of Born In Spacy few Shoot: 0.659
Avg Precision K = 10 of Born In Normal: 0.665


#### DiedIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_diedIn_bert_top1_ranking_spacy_few, _, average_diedIn_bert_top1_ranking_spacy_few = create_results_list(
    ranking_diedIn_bert_spacy_tokens_few, diedIn_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_diedIn_bert_top1_ranking_spacy_few}')

results_list_diedIn_bert_top1_ranking_normal_few, _, average_diedIn_bert_top1_ranking_normal_few = create_results_list(
    ranking_diedIn_bert_normal_tokens_few, diedIn_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_diedIn_bert_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_diedIn_bert_top1_borda_normal_few, _, average_diedIn_bert_top1_borda_normal_few = create_results_list(
    diedIn_bert_borda_tokens_score_few, diedIn_bert_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_diedIn_bert_top1_borda_normal_few}')
print('\n')

results_list_diedIn_bert_top1_borda_spacy_few, _, average_diedIn_bert_top1_borda_spacy_few = create_results_list(
    diedIn_bert_borda_spacy_tokens_score_few, diedIn_bert_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_diedIn_bert_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top1_spacy_few, _, average_diedIn_bert_top1_spacy_few = create_results_list(
    masked_diedIn_bert_original_spacy_few, diedIn_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_diedIn_bert_top1_spacy_few}')

results_list_diedIn_bert_top1_few, _, average_diedIn_bert_top1_few = create_results_list(
    masked_diedIn_bert_original_few, diedIn_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_diedIn_bert_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.445
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.453


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.456


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.445


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.49
Avg Precision K = 1 of died In Normal: 0.469


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_diedIn_bert_top5_ranking_spacy_few, _, average_diedIn_bert_top5_ranking_spacy_few = create_results_list(
    ranking_diedIn_bert_spacy_tokens_few, diedIn_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_diedIn_bert_top5_ranking_spacy_few}')

results_list_diedIn_bert_top5_ranking_normal_few, _, average_diedIn_bert_top5_ranking_normal_few = create_results_list(
    ranking_diedIn_bert_normal_tokens_few, diedIn_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_diedIn_bert_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_diedIn_bert_top5_borda_normal_few, _, average_diedIn_bert_top5_borda_normal_few = create_results_list(
    diedIn_bert_borda_tokens_score_few, diedIn_bert_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_diedIn_bert_top5_borda_normal_few}')
print('\n')

results_list_diedIn_bert_top5_borda_spacy_few, _, average_diedIn_bert_top5_borda_spacy_few = create_results_list(
    diedIn_bert_borda_spacy_tokens_score_few, diedIn_bert_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_diedIn_bert_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top5_spacy_few, _, average_diedIn_bert_top5_spacy_few = create_results_list(
    masked_diedIn_bert_original_spacy_few, diedIn_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_diedIn_bert_top5_spacy_few}')

results_list_diedIn_bert_top5_few, _, average_diedIn_bert_top5_few = create_results_list(
    masked_diedIn_bert_original_few, diedIn_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_diedIn_bert_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.693
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.751


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.745


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.696


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.712
Avg Precision K = 5 of died In Normal: 0.719


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_diedIn_bert_top10_ranking_spacy_few, _, average_diedIn_bert_top10_ranking_spacy_few = create_results_list(
    ranking_diedIn_bert_spacy_tokens_few, diedIn_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_diedIn_bert_top10_ranking_spacy_few}')

results_list_diedIn_bert_top10_ranking_normal_few, _, average_diedIn_bert_top10_ranking_normal_few = create_results_list(
    ranking_diedIn_bert_normal_tokens_few, diedIn_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_diedIn_bert_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_diedIn_bert_top10_borda_normal_few, _, average_diedIn_bert_top10_borda_normal_few = create_results_list(
    diedIn_bert_borda_tokens_score_few, diedIn_bert_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_diedIn_bert_top10_borda_normal_few}')
print('\n')

results_list_diedIn_bert_top10_borda_spacy_few, _, average_diedIn_bert_top10_borda_spacy_few = create_results_list(
    diedIn_bert_borda_spacy_tokens_score_few, diedIn_bert_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_diedIn_bert_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top10_spacy_few, _, average_diedIn_bert_top10_spacy_few = create_results_list(
    masked_diedIn_bert_original_spacy_few, diedIn_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_diedIn_bert_top10_spacy_few}')

results_list_diedIn_bert_top10_few, _, average_diedIn_bert_top10_few = create_results_list(
    masked_diedIn_bert_original_few, diedIn_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_diedIn_bert_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.782
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.821


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.822


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.78


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.764
Avg Precision K = 10 of died In Normal: 0.79


#### Worksfor

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_worksfor_bert_top1_ranking_spacy_few, _, average_worksfor_bert_top1_ranking_spacy_few = create_results_list(
    ranking_worksfor_bert_spacy_tokens_few, worksfor_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_worksfor_bert_top1_ranking_spacy_few}')

results_list_worksfor_bert_top1_ranking_normal_few, _, average_worksfor_bert_top1_ranking_normal_few = create_results_list(
    ranking_worksfor_bert_normal_tokens_few, worksfor_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_worksfor_bert_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_worksfor_bert_top1_borda_normal_few, _, average_worksfor_bert_top1_borda_normal_few = create_results_list(
    worksfor_bert_borda_tokens_score_few, worksfor_bert_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_worksfor_bert_top1_borda_normal_few}')
print('\n')

results_list_worksfor_bert_top1_borda_spacy_few, _, average_worksfor_bert_top1_borda_spacy_few = create_results_list(
    worksfor_bert_borda_spacy_tokens_score_few, worksfor_bert_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_worksfor_bert_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top1_spacy_few, _, average_worksfor_bert_top1_spacy_few = create_results_list(
    masked_worksfor_bert_original_spacy_few, worksfor_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_worksfor_bert_top1_spacy_few}')

results_list_worksfor_bert_top1_few, _, average_worksfor_bert_top1_few = create_results_list(
    masked_worksfor_bert_original_few, worksfor_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_worksfor_bert_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.013
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.615


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.612


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.013


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.013
Avg Precision K = 1 of died In Normal: 0.549


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_worksfor_bert_top5_ranking_spacy_few, _, average_worksfor_bert_top5_ranking_spacy_few = create_results_list(
    ranking_worksfor_bert_spacy_tokens_few, worksfor_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_worksfor_bert_top5_ranking_spacy_few}')

results_list_worksfor_bert_top5_ranking_normal_few, _, average_worksfor_bert_top5_ranking_normal_few = create_results_list(
    ranking_worksfor_bert_normal_tokens_few, worksfor_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_worksfor_bert_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_worksfor_bert_top5_borda_normal_few, _, average_worksfor_bert_top5_borda_normal_few = create_results_list(
    worksfor_bert_borda_tokens_score_few, worksfor_bert_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_worksfor_bert_top5_borda_normal_few}')
print('\n')

results_list_worksfor_bert_top5_borda_spacy_few, _, average_worksfor_bert_top5_borda_spacy_few = create_results_list(
    worksfor_bert_borda_spacy_tokens_score_few, worksfor_bert_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_worksfor_bert_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top5_spacy_few, _, average_worksfor_bert_top5_spacy_few = create_results_list(
    masked_worksfor_bert_original_spacy_few, worksfor_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_worksfor_bert_top5_spacy_few}')

results_list_worksfor_bert_top5_few, _, average_worksfor_bert_top5_few = create_results_list(
    masked_worksfor_bert_original_few, worksfor_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_worksfor_bert_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.013
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.855


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.86


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.013


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.013
Avg Precision K = 5 of died In Normal: 0.813


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_worksfor_bert_top10_ranking_spacy_few, _, average_worksfor_bert_top10_ranking_spacy_few = create_results_list(
    ranking_worksfor_bert_spacy_tokens_few, worksfor_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_worksfor_bert_top10_ranking_spacy_few}')

results_list_worksfor_bert_top10_ranking_normal_few, _, average_worksfor_bert_top10_ranking_normal_few = create_results_list(
    ranking_worksfor_bert_normal_tokens_few, worksfor_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_worksfor_bert_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_worksfor_bert_top10_borda_normal_few, _, average_worksfor_bert_top10_borda_normal_few = create_results_list(
    worksfor_bert_borda_tokens_score_few, worksfor_bert_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_worksfor_bert_top10_borda_normal_few}')
print('\n')

results_list_worksfor_bert_top10_borda_spacy_few, _, average_worksfor_bert_top10_borda_spacy_few = create_results_list(
    worksfor_bert_borda_spacy_tokens_score_few, worksfor_bert_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_worksfor_bert_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top10_spacy_few, _, average_worksfor_bert_top10_spacy_few = create_results_list(
    masked_worksfor_bert_original_spacy_few, worksfor_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_worksfor_bert_top10_spacy_few}')

results_list_worksfor_bert_top10_few, _, average_worksfor_bert_top10_few = create_results_list(
    masked_worksfor_bert_original_few, worksfor_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_worksfor_bert_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.013
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.918


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.913


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.013


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.013
Avg Precision K = 10 of died In Normal: 0.871


#### Capital

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_capital_bert_top1_ranking_spacy_few, _, average_capital_bert_top1_ranking_spacy_few = create_results_list(
    ranking_capital_bert_spacy_tokens_few, capital_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_capital_bert_top1_ranking_spacy_few}')

results_list_capital_bert_top1_ranking_normal_few, _, average_capital_bert_top1_ranking_normal_few = create_results_list(
    ranking_capital_bert_normal_tokens_few, capital_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_capital_bert_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_capital_bert_top1_borda_normal_few, _, average_capital_bert_top1_borda_normal_few = create_results_list(
    capital_bert_borda_tokens_score_few, capital_bert_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_capital_bert_top1_borda_normal_few}')
print('\n')

results_list_capital_bert_top1_borda_spacy_few, _, average_capital_bert_top1_borda_spacy_few = create_results_list(
    capital_bert_borda_spacy_tokens_score_few, capital_bert_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_capital_bert_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top1_spacy_few, _, average_capital_bert_top1_spacy_few = create_results_list(
    masked_capital_bert_original_spacy_few, capital_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_capital_bert_top1_spacy_few}')

results_list_capital_bert_top1_few, _, average_capital_bert_top1_few = create_results_list(
    masked_capital_bert_original_few, capital_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_capital_bert_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.433
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.483


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.482


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.43


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.509
Avg Precision K = 1 of died In Normal: 0.508


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_capital_bert_top5_ranking_spacy_few, _, average_capital_bert_top5_ranking_spacy_few = create_results_list(
    ranking_capital_bert_spacy_tokens_few, capital_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_capital_bert_top5_ranking_spacy_few}')

results_list_capital_bert_top5_ranking_normal_few, _, average_capital_bert_top5_ranking_normal_few = create_results_list(
    ranking_capital_bert_normal_tokens_few, capital_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_capital_bert_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_capital_bert_top5_borda_normal_few, _, average_capital_bert_top5_borda_normal_few = create_results_list(
    capital_bert_borda_tokens_score_few, capital_bert_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_capital_bert_top5_borda_normal_few}')
print('\n')

results_list_capital_bert_top5_borda_spacy_few, _, average_capital_bert_top5_borda_spacy_few = create_results_list(
    capital_bert_borda_spacy_tokens_score_few, capital_bert_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_capital_bert_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top5_spacy_few, _, average_capital_bert_top5_spacy_few = create_results_list(
    masked_capital_bert_original_spacy_few, capital_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_capital_bert_top5_spacy_few}')

results_list_capital_bert_top5_few, _, average_capital_bert_top5_few = create_results_list(
    masked_capital_bert_original_few, capital_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_capital_bert_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.629
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.737


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.737


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.629


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.696
Avg Precision K = 5 of died In Normal: 0.726


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_capital_bert_top10_ranking_spacy_few, _, average_capital_bert_top10_ranking_spacy_few = create_results_list(
    ranking_capital_bert_spacy_tokens_few, capital_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_capital_bert_top10_ranking_spacy_few}')

results_list_capital_bert_top10_ranking_normal_few, _, average_capital_bert_top10_ranking_normal_few = create_results_list(
    ranking_capital_bert_normal_tokens_few, capital_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_capital_bert_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_capital_bert_top10_borda_normal_few, _, average_capital_bert_top10_borda_normal_few = create_results_list(
    capital_bert_borda_tokens_score_few, capital_bert_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_capital_bert_top10_borda_normal_few}')
print('\n')

results_list_capital_bert_top10_borda_spacy_few, _, average_capital_bert_top10_borda_spacy_few = create_results_list(
    capital_bert_borda_spacy_tokens_score_few, capital_bert_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_capital_bert_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top10_spacy_few, _, average_capital_bert_top10_spacy_few = create_results_list(
    masked_capital_bert_original_spacy_few, capital_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_capital_bert_top10_spacy_few}')

results_list_capital_bert_top10_few, _, average_capital_bert_top10_few = create_results_list(
    masked_capital_bert_original_few, capital_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_capital_bert_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.678
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.789


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.782


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.685


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.723
Avg Precision K = 10 of died In Normal: 0.765


#### Citizen

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_citizen_bert_top1_ranking_spacy_few, _, average_citizen_bert_top1_ranking_spacy_few = create_results_list(
    ranking_citizen_bert_spacy_tokens_few, citizen_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_citizen_bert_top1_ranking_spacy_few}')

results_list_citizen_bert_top1_ranking_normal_few, _, average_citizen_bert_top1_ranking_normal_few = create_results_list(
    ranking_citizen_bert_normal_tokens_few, citizen_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_citizen_bert_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_citizen_bert_top1_borda_normal_few, _, average_citizen_bert_top1_borda_normal_few = create_results_list(
    citizen_bert_borda_tokens_score_few, citizen_bert_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_citizen_bert_top1_borda_normal_few}')
print('\n')

results_list_citizen_bert_top1_borda_spacy_few, _, average_citizen_bert_top1_borda_spacy_few = create_results_list(
    citizen_bert_borda_spacy_tokens_score_few, citizen_bert_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_citizen_bert_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top1_spacy_few, _, average_citizen_bert_top1_spacy_few = create_results_list(
    masked_citizen_bert_original_spacy_few, citizen_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_citizen_bert_top1_spacy_few}')

results_list_citizen_bert_top1_few, _, average_citizen_bert_top1_few = create_results_list(
    masked_citizen_bert_original_few, citizen_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_citizen_bert_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.461
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.654


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.65


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.46


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.434
Avg Precision K = 1 of died In Normal: 0.578


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_citizen_bert_top5_ranking_spacy_few, _, average_citizen_bert_top5_ranking_spacy_few = create_results_list(
    ranking_citizen_bert_spacy_tokens_few, citizen_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_citizen_bert_top5_ranking_spacy_few}')

results_list_citizen_bert_top5_ranking_normal_few, _, average_citizen_bert_top5_ranking_normal_few = create_results_list(
    ranking_citizen_bert_normal_tokens_few, citizen_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_citizen_bert_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_citizen_bert_top5_borda_normal_few, _, average_citizen_bert_top5_borda_normal_few = create_results_list(
    citizen_bert_borda_tokens_score_few, citizen_bert_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_citizen_bert_top5_borda_normal_few}')
print('\n')

results_list_citizen_bert_top5_borda_spacy_few, _, average_citizen_bert_top5_borda_spacy_few = create_results_list(
    citizen_bert_borda_spacy_tokens_score_few, citizen_bert_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_citizen_bert_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top5_spacy_few, _, average_citizen_bert_top5_spacy_few = create_results_list(
    masked_citizen_bert_original_spacy_few, citizen_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_citizen_bert_top5_spacy_few}')

results_list_citizen_bert_top5_few, _, average_citizen_bert_top5_few = create_results_list(
    masked_citizen_bert_original_few, citizen_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_citizen_bert_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.559
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.849


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.846


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.557


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.495
Avg Precision K = 5 of died In Normal: 0.745


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_citizen_bert_top10_ranking_spacy_few, _, average_citizen_bert_top10_ranking_spacy_few = create_results_list(
    ranking_citizen_bert_spacy_tokens_few, citizen_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_citizen_bert_top10_ranking_spacy_few}')

results_list_citizen_bert_top10_ranking_normal_few, _, average_citizen_bert_top10_ranking_normal_few = create_results_list(
    ranking_citizen_bert_normal_tokens_few, citizen_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_citizen_bert_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_citizen_bert_top10_borda_normal_few, _, average_citizen_bert_top10_borda_normal_few = create_results_list(
    citizen_bert_borda_tokens_score_few, citizen_bert_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_citizen_bert_top10_borda_normal_few}')
print('\n')

results_list_citizen_bert_top10_borda_spacy_few, _, average_citizen_bert_top10_borda_spacy_few = create_results_list(
    citizen_bert_borda_spacy_tokens_score_few, citizen_bert_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_citizen_bert_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top10_spacy_few, _, average_citizen_bert_top10_spacy_few = create_results_list(
    masked_citizen_bert_original_spacy_few, citizen_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_citizen_bert_top10_spacy_few}')

results_list_citizen_bert_top10_few, _, average_citizen_bert_top10_few = create_results_list(
    masked_citizen_bert_original_few, citizen_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_citizen_bert_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.573
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.893


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.893


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.573


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.497
Avg Precision K = 10 of died In Normal: 0.807


#### Language

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_language_bert_top1_ranking_spacy_few, _, average_language_bert_top1_ranking_spacy_few = create_results_list(
    ranking_language_bert_spacy_tokens_few, language_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_language_bert_top1_ranking_spacy_few}')

results_list_language_bert_top1_ranking_normal_few, _, average_language_bert_top1_ranking_normal_few = create_results_list(
    ranking_language_bert_normal_tokens_few, language_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_language_bert_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_language_bert_top1_borda_normal_few, _, average_language_bert_top1_borda_normal_few = create_results_list(
    language_bert_borda_tokens_score_few, language_bert_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_language_bert_top1_borda_normal_few}')
print('\n')

results_list_language_bert_top1_borda_spacy_few, _, average_language_bert_top1_borda_spacy_few = create_results_list(
    language_bert_borda_spacy_tokens_score_few, language_bert_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_language_bert_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_language_bert_top1_spacy_few, _, average_language_bert_top1_spacy_few = create_results_list(
    masked_language_bert_original_spacy_few, language_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_language_bert_top1_spacy_few}')

results_list_language_bert_top1_few, _, average_language_bert_top1_few = create_results_list(
    masked_language_bert_original_few, language_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_language_bert_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.46
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.684


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.681


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.459


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.665
Avg Precision K = 1 of died In Normal: 0.718


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_language_bert_top5_ranking_spacy_few, _, average_language_bert_top5_ranking_spacy_few = create_results_list(
    ranking_language_bert_spacy_tokens_few, language_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_language_bert_top5_ranking_spacy_few}')

results_list_language_bert_top5_ranking_normal_few, _, average_language_bert_top5_ranking_normal_few = create_results_list(
    ranking_language_bert_normal_tokens_few, language_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_language_bert_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_language_bert_top5_borda_normal_few, _, average_language_bert_top5_borda_normal_few = create_results_list(
    language_bert_borda_tokens_score_few, language_bert_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_language_bert_top5_borda_normal_few}')
print('\n')

results_list_language_bert_top5_borda_spacy_few, _, average_language_bert_top5_borda_spacy_few = create_results_list(
    language_bert_borda_spacy_tokens_score_few, language_bert_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_language_bert_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_language_bert_top5_spacy_few, _, average_language_bert_top5_spacy_few = create_results_list(
    masked_language_bert_original_spacy_few, language_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_language_bert_top5_spacy_few}')

results_list_language_bert_top5_few, _, average_language_bert_top5_few = create_results_list(
    masked_language_bert_original_few, language_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_language_bert_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.611
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.913


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.914


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.611


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.777
Avg Precision K = 5 of died In Normal: 0.888


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_language_bert_top10_ranking_spacy_few, _, average_language_bert_top10_ranking_spacy_few = create_results_list(
    ranking_language_bert_spacy_tokens_few, language_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_language_bert_top10_ranking_spacy_few}')

results_list_language_bert_top10_ranking_normal_few, _, average_language_bert_top10_ranking_normal_few = create_results_list(
    ranking_language_bert_normal_tokens_few, language_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_language_bert_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_language_bert_top10_borda_normal_few, _, average_language_bert_top10_borda_normal_few = create_results_list(
    language_bert_borda_tokens_score_few, language_bert_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_language_bert_top10_borda_normal_few}')
print('\n')

results_list_language_bert_top10_borda_spacy_few, _, average_language_bert_top10_borda_spacy_few = create_results_list(
    language_bert_borda_spacy_tokens_score_few, language_bert_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_language_bert_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_language_bert_top10_spacy_few, _, average_language_bert_top10_spacy_few = create_results_list(
    masked_language_bert_original_spacy_few, language_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_language_bert_top10_spacy_few}')

results_list_language_bert_top10_few, _, average_language_bert_top10_few = create_results_list(
    masked_language_bert_original_few, language_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_language_bert_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.647
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.941


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.945


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.648


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.783
Avg Precision K = 10 of died In Normal: 0.924


### Roberta

#### BornIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_bornIn_roberta_top1_ranking_spacy_few, _, average_bornIn_roberta_top1_ranking_spacy_few = create_results_list(
    ranking_bornIn_roberta_spacy_tokens_few, bornIn_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_bornIn_roberta_top1_ranking_spacy_few}')

results_list_bornIn_roberta_top1_ranking_normal_few, _, average_bornIn_roberta_top1_ranking_normal_few = create_results_list(
    ranking_bornIn_roberta_normal_tokens_few, bornIn_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_bornIn_roberta_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_bornIn_roberta_top1_borda_normal_few, _, average_bornIn_roberta_top1_borda_normal_few = create_results_list(
    bornIn_roberta_borda_tokens_score_few, bornIn_roberta_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_bornIn_roberta_top1_borda_normal_few}')
print('\n')

results_list_bornIn_roberta_top1_borda_spacy_few, _, average_bornIn_roberta_top1_borda_spacy_few = create_results_list(
    bornIn_roberta_borda_spacy_tokens_score_few, bornIn_roberta_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_bornIn_roberta_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_bornIn_roberta_top1_spacy_few, _, average_bornIn_roberta_top1_spacy_few = create_results_list(
    masked_bornIn_roberta_original_spacy_few, bornIn_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_bornIn_roberta_top1_spacy_few}')

results_list_bornIn_roberta_top1_few, _, average_bornIn_roberta_top1_few = create_results_list(
    masked_bornIn_roberta_original_few, bornIn_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_bornIn_roberta_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.285
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.311


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.304


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.282


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.332
Avg Precision K = 1 of died In Normal: 0.325


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_bornIn_roberta_top5_ranking_spacy_few, _, average_bornIn_roberta_top5_ranking_spacy_few = create_results_list(
    ranking_bornIn_roberta_spacy_tokens_few, bornIn_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_bornIn_roberta_top5_ranking_spacy_few}')

results_list_bornIn_roberta_top5_ranking_normal_few, _, average_bornIn_roberta_top5_ranking_normal_few = create_results_list(
    ranking_bornIn_roberta_normal_tokens_few, bornIn_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_bornIn_roberta_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_bornIn_roberta_top5_borda_normal_few, _, average_bornIn_roberta_top5_borda_normal_few = create_results_list(
    bornIn_roberta_borda_tokens_score_few, bornIn_roberta_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_bornIn_roberta_top5_borda_normal_few}')
print('\n')

results_list_bornIn_roberta_top5_borda_spacy_few, _, average_bornIn_roberta_top5_borda_spacy_few = create_results_list(
    bornIn_roberta_borda_spacy_tokens_score_few, bornIn_roberta_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_bornIn_roberta_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_bornIn_roberta_top5_spacy_few, _, average_bornIn_roberta_top5_spacy_few = create_results_list(
    masked_bornIn_roberta_original_spacy_few, bornIn_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_bornIn_roberta_top5_spacy_few}')

results_list_bornIn_roberta_top5_few, _, average_bornIn_roberta_top5_few = create_results_list(
    masked_bornIn_roberta_original_few, bornIn_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_bornIn_roberta_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.502
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.54


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.529


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.497


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.525
Avg Precision K = 5 of died In Normal: 0.52


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_bornIn_roberta_top10_ranking_spacy_few, _, average_bornIn_roberta_top10_ranking_spacy_few = create_results_list(
    ranking_bornIn_roberta_spacy_tokens_few, bornIn_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_bornIn_roberta_top10_ranking_spacy_few}')

results_list_bornIn_roberta_top10_ranking_normal_few, _, average_bornIn_roberta_top10_ranking_normal_few = create_results_list(
    ranking_bornIn_roberta_normal_tokens_few, bornIn_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_bornIn_roberta_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_bornIn_roberta_top10_borda_normal_few, _, average_bornIn_roberta_top10_borda_normal_few = create_results_list(
    bornIn_roberta_borda_tokens_score_few, bornIn_roberta_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_bornIn_roberta_top10_borda_normal_few}')
print('\n')

results_list_bornIn_roberta_top10_borda_spacy_few, _, average_bornIn_roberta_top10_borda_spacy_few = create_results_list(
    bornIn_roberta_borda_spacy_tokens_score_few, bornIn_roberta_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_bornIn_roberta_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_bornIn_roberta_top10_spacy_few, _, average_bornIn_roberta_top10_spacy_few = create_results_list(
    masked_bornIn_roberta_original_spacy_few, bornIn_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_bornIn_roberta_top10_spacy_few}')

results_list_bornIn_roberta_top10_few, _, average_bornIn_roberta_top10_few = create_results_list(
    masked_bornIn_roberta_original_few, bornIn_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_bornIn_roberta_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.565
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.589


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.595


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.56


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.575
Avg Precision K = 10 of died In Normal: 0.581


#### DiedIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_diedIn_roberta_top1_ranking_spacy_few, _, average_diedIn_roberta_top1_ranking_spacy_few = create_results_list(
    ranking_diedIn_roberta_spacy_tokens_few, diedIn_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_diedIn_roberta_top1_ranking_spacy_few}')

results_list_diedIn_roberta_top1_ranking_normal_few, _, average_diedIn_roberta_top1_ranking_normal_few = create_results_list(
    ranking_diedIn_roberta_normal_tokens_few, diedIn_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_diedIn_roberta_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_diedIn_roberta_top1_borda_normal_few, _, average_diedIn_roberta_top1_borda_normal_few = create_results_list(
    diedIn_roberta_borda_tokens_score_few, diedIn_roberta_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_diedIn_roberta_top1_borda_normal_few}')
print('\n')

results_list_diedIn_roberta_top1_borda_spacy_few, _, average_diedIn_roberta_top1_borda_spacy_few = create_results_list(
    diedIn_roberta_borda_spacy_tokens_score_few, diedIn_roberta_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_diedIn_roberta_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_diedIn_roberta_top1_spacy_few, _, average_diedIn_roberta_top1_spacy_few = create_results_list(
    masked_diedIn_roberta_original_spacy_few, diedIn_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_diedIn_roberta_top1_spacy_few}')

results_list_diedIn_roberta_top1_few, _, average_diedIn_roberta_top1_few = create_results_list(
    masked_diedIn_roberta_original_few, diedIn_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_diedIn_roberta_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.399
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.424


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.419


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.399


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.449
Avg Precision K = 1 of died In Normal: 0.43


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_diedIn_roberta_top5_ranking_spacy_few, _, average_diedIn_roberta_top5_ranking_spacy_few = create_results_list(
    ranking_diedIn_roberta_spacy_tokens_few, diedIn_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_diedIn_roberta_top5_ranking_spacy_few}')

results_list_diedIn_roberta_top5_ranking_normal_few, _, average_diedIn_roberta_top5_ranking_normal_few = create_results_list(
    ranking_diedIn_roberta_normal_tokens_few, diedIn_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_diedIn_roberta_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_diedIn_roberta_top5_borda_normal_few, _, average_diedIn_roberta_top5_borda_normal_few = create_results_list(
    diedIn_roberta_borda_tokens_score_few, diedIn_roberta_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_diedIn_roberta_top5_borda_normal_few}')
print('\n')

results_list_diedIn_roberta_top5_borda_spacy_few, _, average_diedIn_roberta_top5_borda_spacy_few = create_results_list(
    diedIn_roberta_borda_spacy_tokens_score_few, diedIn_roberta_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_diedIn_roberta_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_diedIn_roberta_top5_spacy_few, _, average_diedIn_roberta_top5_spacy_few = create_results_list(
    masked_diedIn_roberta_original_spacy_few, diedIn_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_diedIn_roberta_top5_spacy_few}')

results_list_diedIn_roberta_top5_few, _, average_diedIn_roberta_top5_few = create_results_list(
    masked_diedIn_roberta_original_few, diedIn_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_diedIn_roberta_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.617
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.651


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.648


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.615


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.632
Avg Precision K = 5 of died In Normal: 0.633


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_diedIn_roberta_top10_ranking_spacy_few, _, average_diedIn_roberta_top10_ranking_spacy_few = create_results_list(
    ranking_diedIn_roberta_spacy_tokens_few, diedIn_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_diedIn_roberta_top10_ranking_spacy_few}')

results_list_diedIn_roberta_top10_ranking_normal_few, _, average_diedIn_roberta_top10_ranking_normal_few = create_results_list(
    ranking_diedIn_roberta_normal_tokens_few, diedIn_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_diedIn_roberta_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_diedIn_roberta_top10_borda_normal_few, _, average_diedIn_roberta_top10_borda_normal_few = create_results_list(
    diedIn_roberta_borda_tokens_score_few, diedIn_roberta_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_diedIn_roberta_top10_borda_normal_few}')
print('\n')

results_list_diedIn_roberta_top10_borda_spacy_few, _, average_diedIn_roberta_top10_borda_spacy_few = create_results_list(
    diedIn_roberta_borda_spacy_tokens_score_few, diedIn_roberta_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_diedIn_roberta_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_diedIn_roberta_top10_spacy_few, _, average_diedIn_roberta_top10_spacy_few = create_results_list(
    masked_diedIn_roberta_original_spacy_few, diedIn_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_diedIn_roberta_top10_spacy_few}')

results_list_diedIn_roberta_top10_few, _, average_diedIn_roberta_top10_few = create_results_list(
    masked_diedIn_roberta_original_few, diedIn_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_diedIn_roberta_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.675
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.71


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.711


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.679


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.67
Avg Precision K = 10 of died In Normal: 0.692


#### Worksfor

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_worksfor_roberta_top1_ranking_spacy_few, _, average_worksfor_roberta_top1_ranking_spacy_few = create_results_list(
    ranking_worksfor_roberta_spacy_tokens_few, worksfor_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_worksfor_roberta_top1_ranking_spacy_few}')

results_list_worksfor_roberta_top1_ranking_normal_few, _, average_worksfor_roberta_top1_ranking_normal_few = create_results_list(
    ranking_worksfor_roberta_normal_tokens_few, worksfor_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_worksfor_roberta_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_worksfor_roberta_top1_borda_normal_few, _, average_worksfor_roberta_top1_borda_normal_few = create_results_list(
    worksfor_roberta_borda_tokens_score_few, worksfor_roberta_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_worksfor_roberta_top1_borda_normal_few}')
print('\n')

results_list_worksfor_roberta_top1_borda_spacy_few, _, average_worksfor_roberta_top1_borda_spacy_few = create_results_list(
    worksfor_roberta_borda_spacy_tokens_score_few, worksfor_roberta_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_worksfor_roberta_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_worksfor_roberta_top1_spacy_few, _, average_worksfor_roberta_top1_spacy_few = create_results_list(
    masked_worksfor_roberta_original_spacy_few, worksfor_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_worksfor_roberta_top1_spacy_few}')

results_list_worksfor_roberta_top1_few, _, average_worksfor_roberta_top1_few = create_results_list(
    masked_worksfor_roberta_original_few, worksfor_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_worksfor_roberta_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.702
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.704


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.697


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.691


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.691
Avg Precision K = 1 of died In Normal: 0.662


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_worksfor_roberta_top5_ranking_spacy_few, _, average_worksfor_roberta_top5_ranking_spacy_few = create_results_list(
    ranking_worksfor_roberta_spacy_tokens_few, worksfor_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_worksfor_roberta_top5_ranking_spacy_few}')

results_list_worksfor_roberta_top5_ranking_normal_few, _, average_worksfor_roberta_top5_ranking_normal_few = create_results_list(
    ranking_worksfor_roberta_normal_tokens_few, worksfor_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_worksfor_roberta_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_worksfor_roberta_top5_borda_normal_few, _, average_worksfor_roberta_top5_borda_normal_few = create_results_list(
    worksfor_roberta_borda_tokens_score_few, worksfor_roberta_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_worksfor_roberta_top5_borda_normal_few}')
print('\n')

results_list_worksfor_roberta_top5_borda_spacy_few, _, average_worksfor_roberta_top5_borda_spacy_few = create_results_list(
    worksfor_roberta_borda_spacy_tokens_score_few, worksfor_roberta_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_worksfor_roberta_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_worksfor_roberta_top5_spacy_few, _, average_worksfor_roberta_top5_spacy_few = create_results_list(
    masked_worksfor_roberta_original_spacy_few, worksfor_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_worksfor_roberta_top5_spacy_few}')

results_list_worksfor_roberta_top5_few, _, average_worksfor_roberta_top5_few = create_results_list(
    masked_worksfor_roberta_original_few, worksfor_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_worksfor_roberta_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.85
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.889


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.887


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.85


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.842
Avg Precision K = 5 of died In Normal: 0.858


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_worksfor_roberta_top10_ranking_spacy_few, _, average_worksfor_roberta_top10_ranking_spacy_few = create_results_list(
    ranking_worksfor_roberta_spacy_tokens_few, worksfor_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_worksfor_roberta_top10_ranking_spacy_few}')

results_list_worksfor_roberta_top10_ranking_normal_few, _, average_worksfor_roberta_top10_ranking_normal_few = create_results_list(
    ranking_worksfor_roberta_normal_tokens_few, worksfor_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_worksfor_roberta_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_worksfor_roberta_top10_borda_normal_few, _, average_worksfor_roberta_top10_borda_normal_few = create_results_list(
    worksfor_roberta_borda_tokens_score_few, worksfor_roberta_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_worksfor_roberta_top10_borda_normal_few}')
print('\n')

results_list_worksfor_roberta_top10_borda_spacy_few, _, average_worksfor_roberta_top10_borda_spacy_few = create_results_list(
    worksfor_roberta_borda_spacy_tokens_score_few, worksfor_roberta_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_worksfor_roberta_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_worksfor_roberta_top10_spacy_few, _, average_worksfor_roberta_top10_spacy_few = create_results_list(
    masked_worksfor_roberta_original_spacy_few, worksfor_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_worksfor_roberta_top10_spacy_few}')

results_list_worksfor_roberta_top10_few, _, average_worksfor_roberta_top10_few = create_results_list(
    masked_worksfor_roberta_original_few, worksfor_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_worksfor_roberta_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.892
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.939


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.945


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.889


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.858
Avg Precision K = 10 of died In Normal: 0.905


#### Capital

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_capital_roberta_top1_ranking_spacy_few, _, average_capital_roberta_top1_ranking_spacy_few = create_results_list(
    ranking_capital_roberta_spacy_tokens_few, capital_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_capital_roberta_top1_ranking_spacy_few}')

results_list_capital_roberta_top1_ranking_normal_few, _, average_capital_roberta_top1_ranking_normal_few = create_results_list(
    ranking_capital_roberta_normal_tokens_few, capital_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_capital_roberta_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_capital_roberta_top1_borda_normal_few, _, average_capital_roberta_top1_borda_normal_few = create_results_list(
    capital_roberta_borda_tokens_score_few, capital_roberta_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_capital_roberta_top1_borda_normal_few}')
print('\n')

results_list_capital_roberta_top1_borda_spacy_few, _, average_capital_roberta_top1_borda_spacy_few = create_results_list(
    capital_roberta_borda_spacy_tokens_score_few, capital_roberta_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_capital_roberta_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_capital_roberta_top1_spacy_few, _, average_capital_roberta_top1_spacy_few = create_results_list(
    masked_capital_roberta_original_spacy_few, capital_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_capital_roberta_top1_spacy_few}')

results_list_capital_roberta_top1_few, _, average_capital_roberta_top1_few = create_results_list(
    masked_capital_roberta_original_few, capital_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_capital_roberta_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.306
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.335


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.328


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.307


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.346
Avg Precision K = 1 of died In Normal: 0.355


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_capital_roberta_top5_ranking_spacy_few, _, average_capital_roberta_top5_ranking_spacy_few = create_results_list(
    ranking_capital_roberta_spacy_tokens_few, capital_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_capital_roberta_top5_ranking_spacy_few}')

results_list_capital_roberta_top5_ranking_normal_few, _, average_capital_roberta_top5_ranking_normal_few = create_results_list(
    ranking_capital_roberta_normal_tokens_few, capital_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_capital_roberta_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_capital_roberta_top5_borda_normal_few, _, average_capital_roberta_top5_borda_normal_few = create_results_list(
    capital_roberta_borda_tokens_score_few, capital_roberta_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_capital_roberta_top5_borda_normal_few}')
print('\n')

results_list_capital_roberta_top5_borda_spacy_few, _, average_capital_roberta_top5_borda_spacy_few = create_results_list(
    capital_roberta_borda_spacy_tokens_score_few, capital_roberta_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_capital_roberta_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_capital_roberta_top5_spacy_few, _, average_capital_roberta_top5_spacy_few = create_results_list(
    masked_capital_roberta_original_spacy_few, capital_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_capital_roberta_top5_spacy_few}')

results_list_capital_roberta_top5_few, _, average_capital_roberta_top5_few = create_results_list(
    masked_capital_roberta_original_few, capital_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_capital_roberta_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.43
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.496


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.496


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.429


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.442
Avg Precision K = 5 of died In Normal: 0.469


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_capital_roberta_top10_ranking_spacy_few, _, average_capital_roberta_top10_ranking_spacy_few = create_results_list(
    ranking_capital_roberta_spacy_tokens_few, capital_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_capital_roberta_top10_ranking_spacy_few}')

results_list_capital_roberta_top10_ranking_normal_few, _, average_capital_roberta_top10_ranking_normal_few = create_results_list(
    ranking_capital_roberta_normal_tokens_few, capital_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_capital_roberta_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_capital_roberta_top10_borda_normal_few, _, average_capital_roberta_top10_borda_normal_few = create_results_list(
    capital_roberta_borda_tokens_score_few, capital_roberta_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_capital_roberta_top10_borda_normal_few}')
print('\n')

results_list_capital_roberta_top10_borda_spacy_few, _, average_capital_roberta_top10_borda_spacy_few = create_results_list(
    capital_roberta_borda_spacy_tokens_score_few, capital_roberta_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_capital_roberta_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_capital_roberta_top10_spacy_few, _, average_capital_roberta_top10_spacy_few = create_results_list(
    masked_capital_roberta_original_spacy_few, capital_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_capital_roberta_top10_spacy_few}')

results_list_capital_roberta_top10_few, _, average_capital_roberta_top10_few = create_results_list(
    masked_capital_roberta_original_few, capital_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_capital_roberta_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.465
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.531


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.527


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.463


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.459
Avg Precision K = 10 of died In Normal: 0.495


#### Citizen

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_citizen_roberta_top1_ranking_spacy_few, _, average_citizen_roberta_top1_ranking_spacy_few = create_results_list(
    ranking_citizen_roberta_spacy_tokens_few, citizen_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_citizen_roberta_top1_ranking_spacy_few}')

results_list_citizen_roberta_top1_ranking_normal_few, _, average_citizen_roberta_top1_ranking_normal_few = create_results_list(
    ranking_citizen_roberta_normal_tokens_few, citizen_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_citizen_roberta_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_citizen_roberta_top1_borda_normal_few, _, average_citizen_roberta_top1_borda_normal_few = create_results_list(
    citizen_roberta_borda_tokens_score_few, citizen_roberta_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_citizen_roberta_top1_borda_normal_few}')
print('\n')

results_list_citizen_roberta_top1_borda_spacy_few, _, average_citizen_roberta_top1_borda_spacy_few = create_results_list(
    citizen_roberta_borda_spacy_tokens_score_few, citizen_roberta_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_citizen_roberta_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_citizen_roberta_top1_spacy_few, _, average_citizen_roberta_top1_spacy_few = create_results_list(
    masked_citizen_roberta_original_spacy_few, citizen_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_citizen_roberta_top1_spacy_few}')

results_list_citizen_roberta_top1_few, _, average_citizen_roberta_top1_few = create_results_list(
    masked_citizen_roberta_original_few, citizen_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_citizen_roberta_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.45
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.642


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.634


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.45


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.423
Avg Precision K = 1 of died In Normal: 0.558


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_citizen_roberta_top5_ranking_spacy_few, _, average_citizen_roberta_top5_ranking_spacy_few = create_results_list(
    ranking_citizen_roberta_spacy_tokens_few, citizen_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_citizen_roberta_top5_ranking_spacy_few}')

results_list_citizen_roberta_top5_ranking_normal_few, _, average_citizen_roberta_top5_ranking_normal_few = create_results_list(
    ranking_citizen_roberta_normal_tokens_few, citizen_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_citizen_roberta_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_citizen_roberta_top5_borda_normal_few, _, average_citizen_roberta_top5_borda_normal_few = create_results_list(
    citizen_roberta_borda_tokens_score_few, citizen_roberta_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_citizen_roberta_top5_borda_normal_few}')
print('\n')

results_list_citizen_roberta_top5_borda_spacy_few, _, average_citizen_roberta_top5_borda_spacy_few = create_results_list(
    citizen_roberta_borda_spacy_tokens_score_few, citizen_roberta_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_citizen_roberta_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_citizen_roberta_top5_spacy_few, _, average_citizen_roberta_top5_spacy_few = create_results_list(
    masked_citizen_roberta_original_spacy_few, citizen_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_citizen_roberta_top5_spacy_few}')

results_list_citizen_roberta_top5_few, _, average_citizen_roberta_top5_few = create_results_list(
    masked_citizen_roberta_original_few, citizen_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_citizen_roberta_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.555
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.836


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.831


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.555


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.48
Avg Precision K = 5 of died In Normal: 0.71


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_citizen_roberta_top10_ranking_spacy_few, _, average_citizen_roberta_top10_ranking_spacy_few = create_results_list(
    ranking_citizen_roberta_spacy_tokens_few, citizen_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_citizen_roberta_top10_ranking_spacy_few}')

results_list_citizen_roberta_top10_ranking_normal_few, _, average_citizen_roberta_top10_ranking_normal_few = create_results_list(
    ranking_citizen_roberta_normal_tokens_few, citizen_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_citizen_roberta_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_citizen_roberta_top10_borda_normal_few, _, average_citizen_roberta_top10_borda_normal_few = create_results_list(
    citizen_roberta_borda_tokens_score_few, citizen_roberta_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_citizen_roberta_top10_borda_normal_few}')
print('\n')

results_list_citizen_roberta_top10_borda_spacy_few, _, average_citizen_roberta_top10_borda_spacy_few = create_results_list(
    citizen_roberta_borda_spacy_tokens_score_few, citizen_roberta_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_citizen_roberta_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_citizen_roberta_top10_spacy_few, _, average_citizen_roberta_top10_spacy_few = create_results_list(
    masked_citizen_roberta_original_spacy_few, citizen_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_citizen_roberta_top10_spacy_few}')

results_list_citizen_roberta_top10_few, _, average_citizen_roberta_top10_few = create_results_list(
    masked_citizen_roberta_original_few, citizen_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_citizen_roberta_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.568
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.882


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.884


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.568


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.482
Avg Precision K = 10 of died In Normal: 0.783


#### Language

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_language_roberta_top1_ranking_spacy_few, _, average_language_roberta_top1_ranking_spacy_few = create_results_list(
    ranking_language_roberta_spacy_tokens_few, language_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_language_roberta_top1_ranking_spacy_few}')

results_list_language_roberta_top1_ranking_normal_few, _, average_language_roberta_top1_ranking_normal_few = create_results_list(
    ranking_language_roberta_normal_tokens_few, language_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_language_roberta_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_language_roberta_top1_borda_normal_few, _, average_language_roberta_top1_borda_normal_few = create_results_list(
    language_roberta_borda_tokens_score_few, language_roberta_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_language_roberta_top1_borda_normal_few}')
print('\n')

results_list_language_roberta_top1_borda_spacy_few, _, average_language_roberta_top1_borda_spacy_few = create_results_list(
    language_roberta_borda_spacy_tokens_score_few, language_roberta_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_language_roberta_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_language_roberta_top1_spacy_few, _, average_language_roberta_top1_spacy_few = create_results_list(
    masked_language_roberta_original_spacy_few, language_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_language_roberta_top1_spacy_few}')

results_list_language_roberta_top1_few, _, average_language_roberta_top1_few = create_results_list(
    masked_language_roberta_original_few, language_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_language_roberta_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.435
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.621


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.614


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.432


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.611
Avg Precision K = 1 of died In Normal: 0.642


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_language_roberta_top5_ranking_spacy_few, _, average_language_roberta_top5_ranking_spacy_few = create_results_list(
    ranking_language_roberta_spacy_tokens_few, language_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_language_roberta_top5_ranking_spacy_few}')

results_list_language_roberta_top5_ranking_normal_few, _, average_language_roberta_top5_ranking_normal_few = create_results_list(
    ranking_language_roberta_normal_tokens_few, language_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_language_roberta_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_language_roberta_top5_borda_normal_few, _, average_language_roberta_top5_borda_normal_few = create_results_list(
    language_roberta_borda_tokens_score_few, language_roberta_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_language_roberta_top5_borda_normal_few}')
print('\n')

results_list_language_roberta_top5_borda_spacy_few, _, average_language_roberta_top5_borda_spacy_few = create_results_list(
    language_roberta_borda_spacy_tokens_score_few, language_roberta_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_language_roberta_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_language_roberta_top5_spacy_few, _, average_language_roberta_top5_spacy_few = create_results_list(
    masked_language_roberta_original_spacy_few, language_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_language_roberta_top5_spacy_few}')

results_list_language_roberta_top5_few, _, average_language_roberta_top5_few = create_results_list(
    masked_language_roberta_original_few, language_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_language_roberta_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.573
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.842


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.838


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.572


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.71
Avg Precision K = 5 of died In Normal: 0.814


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_language_roberta_top10_ranking_spacy_few, _, average_language_roberta_top10_ranking_spacy_few = create_results_list(
    ranking_language_roberta_spacy_tokens_few, language_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_language_roberta_top10_ranking_spacy_few}')

results_list_language_roberta_top10_ranking_normal_few, _, average_language_roberta_top10_ranking_normal_few = create_results_list(
    ranking_language_roberta_normal_tokens_few, language_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_language_roberta_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_language_roberta_top10_borda_normal_few, _, average_language_roberta_top10_borda_normal_few = create_results_list(
    language_roberta_borda_tokens_score_few, language_roberta_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_language_roberta_top10_borda_normal_few}')
print('\n')

results_list_language_roberta_top10_borda_spacy_few, _, average_language_roberta_top10_borda_spacy_few = create_results_list(
    language_roberta_borda_spacy_tokens_score_few, language_roberta_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_language_roberta_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_language_roberta_top10_spacy_few, _, average_language_roberta_top10_spacy_few = create_results_list(
    masked_language_roberta_original_spacy_few, language_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_language_roberta_top10_spacy_few}')

results_list_language_roberta_top10_few, _, average_language_roberta_top10_few = create_results_list(
    masked_language_roberta_original_few, language_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_language_roberta_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.607
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.881


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.881


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.607


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.715
Avg Precision K = 10 of died In Normal: 0.847


### Electra

#### BornIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_bornIn_electra_top1_ranking_spacy_few, _, average_bornIn_electra_top1_ranking_spacy_few = create_results_list(
    ranking_bornIn_electra_spacy_tokens_few, bornIn_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_bornIn_electra_top1_ranking_spacy_few}')

results_list_bornIn_electra_top1_ranking_normal_few, _, average_bornIn_electra_top1_ranking_normal_few = create_results_list(
    ranking_bornIn_electra_normal_tokens_few, bornIn_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_bornIn_electra_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_bornIn_electra_top1_borda_normal_few, _, average_bornIn_electra_top1_borda_normal_few = create_results_list(
    bornIn_electra_borda_tokens_score_few, bornIn_electra_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_bornIn_electra_top1_borda_normal_few}')
print('\n')

results_list_bornIn_electra_top1_borda_spacy_few, _, average_bornIn_electra_top1_borda_spacy_few = create_results_list(
    bornIn_electra_borda_spacy_tokens_score_few, bornIn_electra_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_bornIn_electra_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_bornIn_electra_top1_spacy_few, _, average_bornIn_electra_top1_spacy_few = create_results_list(
    masked_bornIn_electra_original_spacy_few, bornIn_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_bornIn_electra_top1_spacy_few}')

results_list_bornIn_electra_top1_few, _, average_bornIn_electra_top1_few = create_results_list(
    masked_bornIn_electra_original_few, bornIn_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_bornIn_electra_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.251
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.256


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.251


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.245


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.269
Avg Precision K = 1 of died In Normal: 0.259


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_bornIn_electra_top5_ranking_spacy_few, _, average_bornIn_electra_top5_ranking_spacy_few = create_results_list(
    ranking_bornIn_electra_spacy_tokens_few, bornIn_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_bornIn_electra_top5_ranking_spacy_few}')

results_list_bornIn_electra_top5_ranking_normal_few, _, average_bornIn_electra_top5_ranking_normal_few = create_results_list(
    ranking_bornIn_electra_normal_tokens_few, bornIn_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_bornIn_electra_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_bornIn_electra_top5_borda_normal_few, _, average_bornIn_electra_top5_borda_normal_few = create_results_list(
    bornIn_electra_borda_tokens_score_few, bornIn_electra_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_bornIn_electra_top5_borda_normal_few}')
print('\n')

results_list_bornIn_electra_top5_borda_spacy_few, _, average_bornIn_electra_top5_borda_spacy_few = create_results_list(
    bornIn_electra_borda_spacy_tokens_score_few, bornIn_electra_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_bornIn_electra_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_bornIn_electra_top5_spacy_few, _, average_bornIn_electra_top5_spacy_few = create_results_list(
    masked_bornIn_electra_original_spacy_few, bornIn_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_bornIn_electra_top5_spacy_few}')

results_list_bornIn_electra_top5_few, _, average_bornIn_electra_top5_few = create_results_list(
    masked_bornIn_electra_original_few, bornIn_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_bornIn_electra_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.472
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.494


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.487


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.468


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.45
Avg Precision K = 5 of died In Normal: 0.443


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_bornIn_electra_top10_ranking_spacy_few, _, average_bornIn_electra_top10_ranking_spacy_few = create_results_list(
    ranking_bornIn_electra_spacy_tokens_few, bornIn_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_bornIn_electra_top10_ranking_spacy_few}')

results_list_bornIn_electra_top10_ranking_normal_few, _, average_bornIn_electra_top10_ranking_normal_few = create_results_list(
    ranking_bornIn_electra_normal_tokens_few, bornIn_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_bornIn_electra_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_bornIn_electra_top10_borda_normal_few, _, average_bornIn_electra_top10_borda_normal_few = create_results_list(
    bornIn_electra_borda_tokens_score_few, bornIn_electra_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_bornIn_electra_top10_borda_normal_few}')
print('\n')

results_list_bornIn_electra_top10_borda_spacy_few, _, average_bornIn_electra_top10_borda_spacy_few = create_results_list(
    bornIn_electra_borda_spacy_tokens_score_few, bornIn_electra_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_bornIn_electra_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_bornIn_electra_top10_spacy_few, _, average_bornIn_electra_top10_spacy_few = create_results_list(
    masked_bornIn_electra_original_spacy_few, bornIn_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_bornIn_electra_top10_spacy_few}')

results_list_bornIn_electra_top10_few, _, average_bornIn_electra_top10_few = create_results_list(
    masked_bornIn_electra_original_few, bornIn_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_bornIn_electra_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.546
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.564


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.569


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.545


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.515
Avg Precision K = 10 of died In Normal: 0.518


#### DiedIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_diedIn_electra_top1_ranking_spacy_few, _, average_diedIn_electra_top1_ranking_spacy_few = create_results_list(
    ranking_diedIn_electra_spacy_tokens_few, diedIn_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_diedIn_electra_top1_ranking_spacy_few}')

results_list_diedIn_electra_top1_ranking_normal_few, _, average_diedIn_electra_top1_ranking_normal_few = create_results_list(
    ranking_diedIn_electra_normal_tokens_few, diedIn_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_diedIn_electra_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_diedIn_electra_top1_borda_normal_few, _, average_diedIn_electra_top1_borda_normal_few = create_results_list(
    diedIn_electra_borda_tokens_score_few, diedIn_electra_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_diedIn_electra_top1_borda_normal_few}')
print('\n')

results_list_diedIn_electra_top1_borda_spacy_few, _, average_diedIn_electra_top1_borda_spacy_few = create_results_list(
    diedIn_electra_borda_spacy_tokens_score_few, diedIn_electra_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_diedIn_electra_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_diedIn_electra_top1_spacy_few, _, average_diedIn_electra_top1_spacy_few = create_results_list(
    masked_diedIn_electra_original_spacy_few, diedIn_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_diedIn_electra_top1_spacy_few}')

results_list_diedIn_electra_top1_few, _, average_diedIn_electra_top1_few = create_results_list(
    masked_diedIn_electra_original_few, diedIn_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_diedIn_electra_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.332
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.334


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.33


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.336


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.326
Avg Precision K = 1 of died In Normal: 0.308


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_diedIn_electra_top5_ranking_spacy_few, _, average_diedIn_electra_top5_ranking_spacy_few = create_results_list(
    ranking_diedIn_electra_spacy_tokens_few, diedIn_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_diedIn_electra_top5_ranking_spacy_few}')

results_list_diedIn_electra_top5_ranking_normal_few, _, average_diedIn_electra_top5_ranking_normal_few = create_results_list(
    ranking_diedIn_electra_normal_tokens_few, diedIn_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_diedIn_electra_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_diedIn_electra_top5_borda_normal_few, _, average_diedIn_electra_top5_borda_normal_few = create_results_list(
    diedIn_electra_borda_tokens_score_few, diedIn_electra_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_diedIn_electra_top5_borda_normal_few}')
print('\n')

results_list_diedIn_electra_top5_borda_spacy_few, _, average_diedIn_electra_top5_borda_spacy_few = create_results_list(
    diedIn_electra_borda_spacy_tokens_score_few, diedIn_electra_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_diedIn_electra_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_diedIn_electra_top5_spacy_few, _, average_diedIn_electra_top5_spacy_few = create_results_list(
    masked_diedIn_electra_original_spacy_few, diedIn_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_diedIn_electra_top5_spacy_few}')

results_list_diedIn_electra_top5_few, _, average_diedIn_electra_top5_few = create_results_list(
    masked_diedIn_electra_original_few, diedIn_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_diedIn_electra_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.584
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.594


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.599


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.584


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.567
Avg Precision K = 5 of died In Normal: 0.558


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_diedIn_electra_top10_ranking_spacy_few, _, average_diedIn_electra_top10_ranking_spacy_few = create_results_list(
    ranking_diedIn_electra_spacy_tokens_few, diedIn_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_diedIn_electra_top10_ranking_spacy_few}')

results_list_diedIn_electra_top10_ranking_normal_few, _, average_diedIn_electra_top10_ranking_normal_few = create_results_list(
    ranking_diedIn_electra_normal_tokens_few, diedIn_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_diedIn_electra_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_diedIn_electra_top10_borda_normal_few, _, average_diedIn_electra_top10_borda_normal_few = create_results_list(
    diedIn_electra_borda_tokens_score_few, diedIn_electra_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_diedIn_electra_top10_borda_normal_few}')
print('\n')

results_list_diedIn_electra_top10_borda_spacy_few, _, average_diedIn_electra_top10_borda_spacy_few = create_results_list(
    diedIn_electra_borda_spacy_tokens_score_few, diedIn_electra_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_diedIn_electra_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_diedIn_electra_top10_spacy_few, _, average_diedIn_electra_top10_spacy_few = create_results_list(
    masked_diedIn_electra_original_spacy_few, diedIn_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_diedIn_electra_top10_spacy_few}')

results_list_diedIn_electra_top10_few, _, average_diedIn_electra_top10_few = create_results_list(
    masked_diedIn_electra_original_few, diedIn_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_diedIn_electra_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.669
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.704


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.703


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.672


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.628
Avg Precision K = 10 of died In Normal: 0.647


#### Worksfor

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_worksfor_electra_top1_ranking_spacy_few, _, average_worksfor_electra_top1_ranking_spacy_few = create_results_list(
    ranking_worksfor_electra_spacy_tokens_few, worksfor_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_worksfor_electra_top1_ranking_spacy_few}')

results_list_worksfor_electra_top1_ranking_normal_few, _, average_worksfor_electra_top1_ranking_normal_few = create_results_list(
    ranking_worksfor_electra_normal_tokens_few, worksfor_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_worksfor_electra_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_worksfor_electra_top1_borda_normal_few, _, average_worksfor_electra_top1_borda_normal_few = create_results_list(
    worksfor_electra_borda_tokens_score_few, worksfor_electra_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_worksfor_electra_top1_borda_normal_few}')
print('\n')

results_list_worksfor_electra_top1_borda_spacy_few, _, average_worksfor_electra_top1_borda_spacy_few = create_results_list(
    worksfor_electra_borda_spacy_tokens_score_few, worksfor_electra_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_worksfor_electra_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_worksfor_electra_top1_spacy_few, _, average_worksfor_electra_top1_spacy_few = create_results_list(
    masked_worksfor_electra_original_spacy_few, worksfor_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_worksfor_electra_top1_spacy_few}')

results_list_worksfor_electra_top1_few, _, average_worksfor_electra_top1_few = create_results_list(
    masked_worksfor_electra_original_few, worksfor_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_worksfor_electra_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.594
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.517


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.504


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.586


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.536
Avg Precision K = 1 of died In Normal: 0.443


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_worksfor_electra_top5_ranking_spacy_few, _, average_worksfor_electra_top5_ranking_spacy_few = create_results_list(
    ranking_worksfor_electra_spacy_tokens_few, worksfor_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_worksfor_electra_top5_ranking_spacy_few}')

results_list_worksfor_electra_top5_ranking_normal_few, _, average_worksfor_electra_top5_ranking_normal_few = create_results_list(
    ranking_worksfor_electra_normal_tokens_few, worksfor_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_worksfor_electra_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_worksfor_electra_top5_borda_normal_few, _, average_worksfor_electra_top5_borda_normal_few = create_results_list(
    worksfor_electra_borda_tokens_score_few, worksfor_electra_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_worksfor_electra_top5_borda_normal_few}')
print('\n')

results_list_worksfor_electra_top5_borda_spacy_few, _, average_worksfor_electra_top5_borda_spacy_few = create_results_list(
    worksfor_electra_borda_spacy_tokens_score_few, worksfor_electra_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_worksfor_electra_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_worksfor_electra_top5_spacy_few, _, average_worksfor_electra_top5_spacy_few = create_results_list(
    masked_worksfor_electra_original_spacy_few, worksfor_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_worksfor_electra_top5_spacy_few}')

results_list_worksfor_electra_top5_few, _, average_worksfor_electra_top5_few = create_results_list(
    masked_worksfor_electra_original_few, worksfor_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_worksfor_electra_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.784
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.757


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.765


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.781


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.741
Avg Precision K = 5 of died In Normal: 0.715


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_worksfor_electra_top10_ranking_spacy_few, _, average_worksfor_electra_top10_ranking_spacy_few = create_results_list(
    ranking_worksfor_electra_spacy_tokens_few, worksfor_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_worksfor_electra_top10_ranking_spacy_few}')

results_list_worksfor_electra_top10_ranking_normal_few, _, average_worksfor_electra_top10_ranking_normal_few = create_results_list(
    ranking_worksfor_electra_normal_tokens_few, worksfor_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_worksfor_electra_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_worksfor_electra_top10_borda_normal_few, _, average_worksfor_electra_top10_borda_normal_few = create_results_list(
    worksfor_electra_borda_tokens_score_few, worksfor_electra_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_worksfor_electra_top10_borda_normal_few}')
print('\n')

results_list_worksfor_electra_top10_borda_spacy_few, _, average_worksfor_electra_top10_borda_spacy_few = create_results_list(
    worksfor_electra_borda_spacy_tokens_score_few, worksfor_electra_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_worksfor_electra_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_worksfor_electra_top10_spacy_few, _, average_worksfor_electra_top10_spacy_few = create_results_list(
    masked_worksfor_electra_original_spacy_few, worksfor_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_worksfor_electra_top10_spacy_few}')

results_list_worksfor_electra_top10_few, _, average_worksfor_electra_top10_few = create_results_list(
    masked_worksfor_electra_original_few, worksfor_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_worksfor_electra_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.813
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.831


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.842


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.813


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.755
Avg Precision K = 10 of died In Normal: 0.784


#### Capital

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_capital_electra_top1_ranking_spacy_few, _, average_capital_electra_top1_ranking_spacy_few = create_results_list(
    ranking_capital_electra_spacy_tokens_few, capital_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_capital_electra_top1_ranking_spacy_few}')

results_list_capital_electra_top1_ranking_normal_few, _, average_capital_electra_top1_ranking_normal_few = create_results_list(
    ranking_capital_electra_normal_tokens_few, capital_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_capital_electra_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_capital_electra_top1_borda_normal_few, _, average_capital_electra_top1_borda_normal_few = create_results_list(
    capital_electra_borda_tokens_score_few, capital_electra_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_capital_electra_top1_borda_normal_few}')
print('\n')

results_list_capital_electra_top1_borda_spacy_few, _, average_capital_electra_top1_borda_spacy_few = create_results_list(
    capital_electra_borda_spacy_tokens_score_few, capital_electra_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_capital_electra_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_capital_electra_top1_spacy_few, _, average_capital_electra_top1_spacy_few = create_results_list(
    masked_capital_electra_original_spacy_few, capital_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_capital_electra_top1_spacy_few}')

results_list_capital_electra_top1_few, _, average_capital_electra_top1_few = create_results_list(
    masked_capital_electra_original_few, capital_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_capital_electra_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.273
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.278


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.276


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.274


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.284
Avg Precision K = 1 of died In Normal: 0.284


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_capital_electra_top5_ranking_spacy_few, _, average_capital_electra_top5_ranking_spacy_few = create_results_list(
    ranking_capital_electra_spacy_tokens_few, capital_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_capital_electra_top5_ranking_spacy_few}')

results_list_capital_electra_top5_ranking_normal_few, _, average_capital_electra_top5_ranking_normal_few = create_results_list(
    ranking_capital_electra_normal_tokens_few, capital_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_capital_electra_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_capital_electra_top5_borda_normal_few, _, average_capital_electra_top5_borda_normal_few = create_results_list(
    capital_electra_borda_tokens_score_few, capital_electra_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_capital_electra_top5_borda_normal_few}')
print('\n')

results_list_capital_electra_top5_borda_spacy_few, _, average_capital_electra_top5_borda_spacy_few = create_results_list(
    capital_electra_borda_spacy_tokens_score_few, capital_electra_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_capital_electra_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_capital_electra_top5_spacy_few, _, average_capital_electra_top5_spacy_few = create_results_list(
    masked_capital_electra_original_spacy_few, capital_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_capital_electra_top5_spacy_few}')

results_list_capital_electra_top5_few, _, average_capital_electra_top5_few = create_results_list(
    masked_capital_electra_original_few, capital_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_capital_electra_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.479
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.527


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.53


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.468


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.45
Avg Precision K = 5 of died In Normal: 0.459


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_capital_electra_top10_ranking_spacy_few, _, average_capital_electra_top10_ranking_spacy_few = create_results_list(
    ranking_capital_electra_spacy_tokens_few, capital_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_capital_electra_top10_ranking_spacy_few}')

results_list_capital_electra_top10_ranking_normal_few, _, average_capital_electra_top10_ranking_normal_few = create_results_list(
    ranking_capital_electra_normal_tokens_few, capital_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_capital_electra_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_capital_electra_top10_borda_normal_few, _, average_capital_electra_top10_borda_normal_few = create_results_list(
    capital_electra_borda_tokens_score_few, capital_electra_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_capital_electra_top10_borda_normal_few}')
print('\n')

results_list_capital_electra_top10_borda_spacy_few, _, average_capital_electra_top10_borda_spacy_few = create_results_list(
    capital_electra_borda_spacy_tokens_score_few, capital_electra_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_capital_electra_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_capital_electra_top10_spacy_few, _, average_capital_electra_top10_spacy_few = create_results_list(
    masked_capital_electra_original_spacy_few, capital_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_capital_electra_top10_spacy_few}')

results_list_capital_electra_top10_few, _, average_capital_electra_top10_few = create_results_list(
    masked_capital_electra_original_few, capital_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_capital_electra_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.55
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.619


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.616


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.547


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.509
Avg Precision K = 10 of died In Normal: 0.532


#### Citizen

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_citizen_electra_top1_ranking_spacy_few, _, average_citizen_electra_top1_ranking_spacy_few = create_results_list(
    ranking_citizen_electra_spacy_tokens_few, citizen_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_citizen_electra_top1_ranking_spacy_few}')

results_list_citizen_electra_top1_ranking_normal_few, _, average_citizen_electra_top1_ranking_normal_few = create_results_list(
    ranking_citizen_electra_normal_tokens_few, citizen_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_citizen_electra_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_citizen_electra_top1_borda_normal_few, _, average_citizen_electra_top1_borda_normal_few = create_results_list(
    citizen_electra_borda_tokens_score_few, citizen_electra_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_citizen_electra_top1_borda_normal_few}')
print('\n')

results_list_citizen_electra_top1_borda_spacy_few, _, average_citizen_electra_top1_borda_spacy_few = create_results_list(
    citizen_electra_borda_spacy_tokens_score_few, citizen_electra_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_citizen_electra_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_citizen_electra_top1_spacy_few, _, average_citizen_electra_top1_spacy_few = create_results_list(
    masked_citizen_electra_original_spacy_few, citizen_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_citizen_electra_top1_spacy_few}')

results_list_citizen_electra_top1_few, _, average_citizen_electra_top1_few = create_results_list(
    masked_citizen_electra_original_few, citizen_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_citizen_electra_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.37
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.514


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.514


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.363


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.307
Avg Precision K = 1 of died In Normal: 0.459


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_citizen_electra_top5_ranking_spacy_few, _, average_citizen_electra_top5_ranking_spacy_few = create_results_list(
    ranking_citizen_electra_spacy_tokens_few, citizen_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_citizen_electra_top5_ranking_spacy_few}')

results_list_citizen_electra_top5_ranking_normal_few, _, average_citizen_electra_top5_ranking_normal_few = create_results_list(
    ranking_citizen_electra_normal_tokens_few, citizen_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_citizen_electra_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_citizen_electra_top5_borda_normal_few, _, average_citizen_electra_top5_borda_normal_few = create_results_list(
    citizen_electra_borda_tokens_score_few, citizen_electra_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_citizen_electra_top5_borda_normal_few}')
print('\n')

results_list_citizen_electra_top5_borda_spacy_few, _, average_citizen_electra_top5_borda_spacy_few = create_results_list(
    citizen_electra_borda_spacy_tokens_score_few, citizen_electra_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_citizen_electra_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_citizen_electra_top5_spacy_few, _, average_citizen_electra_top5_spacy_few = create_results_list(
    masked_citizen_electra_original_spacy_few, citizen_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_citizen_electra_top5_spacy_few}')

results_list_citizen_electra_top5_few, _, average_citizen_electra_top5_few = create_results_list(
    masked_citizen_electra_original_few, citizen_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_citizen_electra_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.497
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.714


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.71


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.492


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.388
Avg Precision K = 5 of died In Normal: 0.593


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_citizen_electra_top10_ranking_spacy_few, _, average_citizen_electra_top10_ranking_spacy_few = create_results_list(
    ranking_citizen_electra_spacy_tokens_few, citizen_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_citizen_electra_top10_ranking_spacy_few}')

results_list_citizen_electra_top10_ranking_normal_few, _, average_citizen_electra_top10_ranking_normal_few = create_results_list(
    ranking_citizen_electra_normal_tokens_few, citizen_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_citizen_electra_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_citizen_electra_top10_borda_normal_few, _, average_citizen_electra_top10_borda_normal_few = create_results_list(
    citizen_electra_borda_tokens_score_few, citizen_electra_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_citizen_electra_top10_borda_normal_few}')
print('\n')

results_list_citizen_electra_top10_borda_spacy_few, _, average_citizen_electra_top10_borda_spacy_few = create_results_list(
    citizen_electra_borda_spacy_tokens_score_few, citizen_electra_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_citizen_electra_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_citizen_electra_top10_spacy_few, _, average_citizen_electra_top10_spacy_few = create_results_list(
    masked_citizen_electra_original_spacy_few, citizen_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_citizen_electra_top10_spacy_few}')

results_list_citizen_electra_top10_few, _, average_citizen_electra_top10_few = create_results_list(
    masked_citizen_electra_original_few, citizen_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_citizen_electra_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.526
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.792


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.782


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.526


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.393
Avg Precision K = 10 of died In Normal: 0.656


#### Language

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_language_electra_top1_ranking_spacy_few, _, average_language_electra_top1_ranking_spacy_few = create_results_list(
    ranking_language_electra_spacy_tokens_few, language_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy few Shoot: {average_language_electra_top1_ranking_spacy_few}')

results_list_language_electra_top1_ranking_normal_few, _, average_language_electra_top1_ranking_normal_few = create_results_list(
    ranking_language_electra_normal_tokens_few, language_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal few Shoot: {average_language_electra_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_language_electra_top1_borda_normal_few, _, average_language_electra_top1_borda_normal_few = create_results_list(
    language_electra_borda_tokens_score_few, language_electra_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal few Shoot: {average_language_electra_top1_borda_normal_few}')
print('\n')

results_list_language_electra_top1_borda_spacy_few, _, average_language_electra_top1_borda_spacy_few = create_results_list(
    language_electra_borda_spacy_tokens_score_few, language_electra_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In Spacy few Shoot: {average_language_electra_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_language_electra_top1_spacy_few, _, average_language_electra_top1_spacy_few = create_results_list(
    masked_language_electra_original_spacy_few, language_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Spacy few Shoot: {average_language_electra_top1_spacy_few}')

results_list_language_electra_top1_few, _, average_language_electra_top1_few = create_results_list(
    masked_language_electra_original_few, language_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Normal: {average_language_electra_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy few Shoot: 0.316
Avg Precision K = 1 of the ranking of died In Normal few Shoot: 0.381


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal few Shoot: 0.379


Borda Spacy Score
Avg Precision K = 1 of the borda of died In Spacy few Shoot: 0.31


Sem ranking
Avg Precision K = 1 of died In Spacy few Shoot: 0.4
Avg Precision K = 1 of died In Normal: 0.371


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_language_electra_top5_ranking_spacy_few, _, average_language_electra_top5_ranking_spacy_few = create_results_list(
    ranking_language_electra_spacy_tokens_few, language_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy few Shoot: {average_language_electra_top5_ranking_spacy_few}')

results_list_language_electra_top5_ranking_normal_few, _, average_language_electra_top5_ranking_normal_few = create_results_list(
    ranking_language_electra_normal_tokens_few, language_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal few Shoot: {average_language_electra_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_language_electra_top5_borda_normal_few, _, average_language_electra_top5_borda_normal_few = create_results_list(
    language_electra_borda_tokens_score_few, language_electra_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal few Shoot: {average_language_electra_top5_borda_normal_few}')
print('\n')

results_list_language_electra_top5_borda_spacy_few, _, average_language_electra_top5_borda_spacy_few = create_results_list(
    language_electra_borda_spacy_tokens_score_few, language_electra_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In Spacy few Shoot: {average_language_electra_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_language_electra_top5_spacy_few, _, average_language_electra_top5_spacy_few = create_results_list(
    masked_language_electra_original_spacy_few, language_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Spacy few Shoot: {average_language_electra_top5_spacy_few}')

results_list_language_electra_top5_few, _, average_language_electra_top5_few = create_results_list(
    masked_language_electra_original_few, language_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Normal: {average_language_electra_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy few Shoot: 0.535
Avg Precision K = 5 of the ranking of died In Normal few Shoot: 0.719


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal few Shoot: 0.715


Borda Spacy Score
Avg Precision K = 5 of the borda of died In Spacy few Shoot: 0.53


Sem ranking
Avg Precision K = 5 of died In Spacy few Shoot: 0.624
Avg Precision K = 5 of died In Normal: 0.673


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_language_electra_top10_ranking_spacy_few, _, average_language_electra_top10_ranking_spacy_few = create_results_list(
    ranking_language_electra_spacy_tokens_few, language_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy few Shoot: {average_language_electra_top10_ranking_spacy_few}')

results_list_language_electra_top10_ranking_normal_few, _, average_language_electra_top10_ranking_normal_few = create_results_list(
    ranking_language_electra_normal_tokens_few, language_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal few Shoot: {average_language_electra_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_language_electra_top10_borda_normal_few, _, average_language_electra_top10_borda_normal_few = create_results_list(
    language_electra_borda_tokens_score_few, language_electra_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal few Shoot: {average_language_electra_top10_borda_normal_few}')
print('\n')

results_list_language_electra_top10_borda_spacy_few, _, average_language_electra_top10_borda_spacy_few = create_results_list(
    language_electra_borda_spacy_tokens_score_few, language_electra_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In Spacy few Shoot: {average_language_electra_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_language_electra_top10_spacy_few, _, average_language_electra_top10_spacy_few = create_results_list(
    masked_language_electra_original_spacy_few, language_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Spacy few Shoot: {average_language_electra_top10_spacy_few}')

results_list_language_electra_top10_few, _, average_language_electra_top10_few = create_results_list(
    masked_language_electra_original_few, language_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Normal: {average_language_electra_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy few Shoot: 0.583
Avg Precision K = 10 of the ranking of died In Normal few Shoot: 0.828


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal few Shoot: 0.827


Borda Spacy Score
Avg Precision K = 10 of the borda of died In Spacy few Shoot: 0.581


Sem ranking
Avg Precision K = 10 of died In Spacy few Shoot: 0.649
Avg Precision K = 10 of died In Normal: 0.768


### Merged Models

#### BornIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_bornIn_merged_model_top1_ranking_spacy_few, _, average_bornIn_merged_model_top1_ranking_spacy_few = create_results_list(
    ranking_bornin_merged_model_spacy_tokens_few, bornIn_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Spacy: {average_bornIn_merged_model_top1_ranking_spacy_few}')

results_list_bornIn_merged_model_top1_ranking_normal_few, _, average_bornIn_merged_model_top1_ranking_normal_few = create_results_list(
    ranking_bornIn_merged_model_normal_tokens_few, bornIn_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of Born In Normal: {average_bornIn_merged_model_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_bornIn_merged_model_top1_borda_normal_few, _, average_bornIn_merged_model_top1_borda_normal_few = create_results_list(
    bornIn_merged_model_borda_tokens_score_few, bornIn_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of Born In Normal: {average_bornIn_merged_model_top1_borda_normal_few}')
print('\n')

results_list_bornIn_merged_model_top1_borda_spacy_few, _, average_bornIn_merged_model_top1_borda_spacy_few = create_results_list(
    bornIn_merged_model_borda_spacy_tokens_score_few, bornIn_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of Born In spacy: {average_bornIn_merged_model_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top1_spacy_few, _, average_bornIn_bert_top1_spacy_few = create_results_list(
    masked_bornIn_bert_original_spacy_few, bornIn_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Bert Spacy: {average_bornIn_bert_top1_spacy_few}')

results_list_bornIn_bert_top1_few, _, average_bornIn_bert_top1_few = create_results_list(
    masked_bornIn_bert_original_few, bornIn_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In Bert Normal: {average_bornIn_bert_top1_few}')

results_list_bornIn_roberta_top1_spacy_few, _, average_bornIn_roberta_top1_spacy_few = create_results_list(
    masked_bornIn_roberta_original_spacy_few, bornIn_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In roberta Spacy: {average_bornIn_roberta_top1_spacy_few}')

results_list_bornIn_roberta_top1_few, _, average_bornIn_roberta_top1_few = create_results_list(
    masked_bornIn_roberta_original, bornIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In roberta Normal: {average_bornIn_roberta_top1_few}')

results_list_bornIn_electra_top1_spacy_few, _, average_bornIn_electra_top1_spacy_few = create_results_list(
    masked_bornIn_electra_original_spacy_few, bornIn_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In electra Spacy: {average_bornIn_electra_top1_spacy_few}')

results_list_bornIn_electra_top1_few, _, average_bornIn_electra_top1_few = create_results_list(
    masked_bornIn_electra_original_few, bornIn_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of Born In electra Normal: {average_bornIn_electra_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of Born In Spacy: 0.389
Avg Precision K = 1 of the ranking of Born In Normal: 0.372


Borda Count
Borda Score
Avg Precision K = 1 of the borda of Born In Normal: 0.373


Borda Spacy Score
Avg Precision K = 1 of the borda of Born In spacy: 0.382


Sem ranking
Avg Precision K = 1 of Born In Bert Spacy: 0.379
Avg Precision K = 1 of Born In Bert Normal: 0.367
Avg Precision K = 1 of Born In roberta Spacy: 0.332
Avg Precision K = 1 of Born In roberta Normal: 0.329
Avg Precision K = 1 of Born In electra Spacy: 0.269
Avg Precision K = 1 of Born In electra Normal: 0.259


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_bornIn_merged_model_top5_ranking_spacy_few, _, average_bornIn_merged_model_top5_ranking_spacy_few = create_results_list(
    ranking_bornin_merged_model_spacy_tokens_few, bornIn_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Spacy: {average_bornIn_merged_model_top5_ranking_spacy_few}')

results_list_bornIn_merged_model_top5_ranking_normal_few, _, average_bornIn_merged_model_top5_ranking_normal_few = create_results_list(
    ranking_bornIn_merged_model_normal_tokens_few, bornIn_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of Born In Normal: {average_bornIn_merged_model_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_bornIn_merged_model_top5_borda_normal_few, _, average_bornIn_merged_model_top5_borda_normal_few = create_results_list(
    bornIn_merged_model_borda_tokens_score_few, bornIn_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of Born In Normal: {average_bornIn_merged_model_top5_borda_normal_few}')
print('\n')

results_list_bornIn_merged_model_top5_borda_spacy_few, _, average_bornIn_merged_model_top5_borda_spacy_few = create_results_list(
    bornIn_merged_model_borda_spacy_tokens_score_few, bornIn_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of Born In spacy: {average_bornIn_merged_model_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top5_spacy_few, _, average_bornIn_bert_top5_spacy_few = create_results_list(
    masked_bornIn_bert_original_spacy_few, bornIn_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Bert Spacy: {average_bornIn_bert_top5_spacy_few}')

results_list_bornIn_bert_top5_few, _, average_bornIn_bert_top5_few = create_results_list(
    masked_bornIn_bert_original_few, bornIn_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In Bert Normal: {average_bornIn_bert_top5_few}')

results_list_bornIn_roberta_top5_spacy_few, _, average_bornIn_roberta_top5_spacy_few = create_results_list(
    masked_bornIn_roberta_original_spacy_few, bornIn_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In roberta Spacy: {average_bornIn_roberta_top5_spacy_few}')

results_list_bornIn_roberta_top5_few, _, average_bornIn_roberta_top5_few = create_results_list(
    masked_bornIn_roberta_original, bornIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In roberta Normal: {average_bornIn_roberta_top5_few}')

results_list_bornIn_electra_top5_spacy_few, _, average_bornIn_electra_top5_spacy_few = create_results_list(
    masked_bornIn_electra_original_spacy_few, bornIn_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In electra Spacy: {average_bornIn_electra_top5_spacy_few}')

results_list_bornIn_electra_top5_few, _, average_bornIn_electra_top5_few = create_results_list(
    masked_bornIn_electra_original_few, bornIn_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of Born In electra Normal: {average_bornIn_electra_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of Born In Spacy: 0.612
Avg Precision K = 5 of the ranking of Born In Normal: 0.592


Borda Count
Borda Score
Avg Precision K = 5 of the borda of Born In Normal: 0.597


Borda Spacy Score
Avg Precision K = 5 of the borda of Born In spacy: 0.609


Sem ranking
Avg Precision K = 5 of Born In Bert Spacy: 0.601
Avg Precision K = 5 of Born In Bert Normal: 0.594
Avg Precision K = 5 of Born In roberta Spacy: 0.525
Avg Precision K = 5 of Born In roberta Normal: 0.52
Avg Precision K = 5 of Born In electra Spacy: 0.45
Avg Precision K = 5 of Born In electra Normal: 0.443


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_bornIn_merged_model_top10_ranking_spacy_few, _, average_bornIn_merged_model_top10_ranking_spacy_few = create_results_list(
    ranking_bornin_merged_model_spacy_tokens_few, bornIn_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Spacy: {average_bornIn_merged_model_top10_ranking_spacy_few}')

results_list_bornIn_merged_model_top10_ranking_normal_few, _, average_bornIn_merged_model_top10_ranking_normal_few = create_results_list(
    ranking_bornIn_merged_model_normal_tokens_few, bornIn_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of Born In Normal: {average_bornIn_merged_model_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_bornIn_merged_model_top10_borda_normal_few, _, average_bornIn_merged_model_top10_borda_normal_few = create_results_list(
    bornIn_merged_model_borda_tokens_score_few, bornIn_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of Born In Normal: {average_bornIn_merged_model_top10_borda_normal_few}')
print('\n')

results_list_bornIn_merged_model_top10_borda_spacy_few, _, average_bornIn_merged_model_top10_borda_spacy_few = create_results_list(
    bornIn_merged_model_borda_spacy_tokens_score_few, bornIn_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of Born In spacy: {average_bornIn_merged_model_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_bornIn_bert_top10_spacy_few, _, average_bornIn_bert_top10_spacy_few = create_results_list(
    masked_bornIn_bert_original_spacy_few, bornIn_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Bert Spacy: {average_bornIn_bert_top10_spacy_few}')

results_list_bornIn_bert_top10_few, _, average_bornIn_bert_top10_few = create_results_list(
    masked_bornIn_bert_original_few, bornIn_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In Bert Normal: {average_bornIn_bert_top10_few}')

results_list_bornIn_roberta_top10_spacy_few, _, average_bornIn_roberta_top10_spacy_few = create_results_list(
    masked_bornIn_roberta_original_spacy_few, bornIn_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In roberta Spacy: {average_bornIn_roberta_top10_spacy_few}')

results_list_bornIn_roberta_top10_few, _, average_bornIn_roberta_top10_few = create_results_list(
    masked_bornIn_roberta_original, bornIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In roberta Normal: {average_bornIn_roberta_top10_few}')

results_list_bornIn_electra_top10_spacy_few, _, average_bornIn_electra_top10_spacy_few = create_results_list(
    masked_bornIn_electra_original_spacy_few, bornIn_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In electra Spacy: {average_bornIn_electra_top10_spacy_few}')

results_list_bornIn_electra_top10_few, _, average_bornIn_electra_top10_few = create_results_list(
    masked_bornIn_electra_original_few, bornIn_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of Born In electra Normal: {average_bornIn_electra_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of Born In Spacy: 0.684
Avg Precision K = 10 of the ranking of Born In Normal: 0.674


Borda Count
Borda Score
Avg Precision K = 10 of the borda of Born In Normal: 0.673


Borda Spacy Score
Avg Precision K = 10 of the borda of Born In spacy: 0.684


Sem ranking
Avg Precision K = 10 of Born In Bert Spacy: 0.659
Avg Precision K = 10 of Born In Bert Normal: 0.665
Avg Precision K = 10 of Born In roberta Spacy: 0.575
Avg Precision K = 10 of Born In roberta Normal: 0.577
Avg Precision K = 10 of Born In electra Spacy: 0.515
Avg Precision K = 10 of Born In electra Normal: 0.518


#### DiedIn

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_diedIn_merged_model_top1_ranking_spacy_few, _, average_diedIn_merged_model_top1_ranking_spacy_few = create_results_list(
    ranking_diedIn_merged_model_spacy_tokens_one, diedIn_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy: {average_diedIn_merged_model_top1_ranking_spacy_few}')

results_list_diedIn_merged_model_top1_ranking_normal_few, _, average_diedIn_merged_model_top1_ranking_normal_few = create_results_list(
    ranking_diedIn_merged_model_normal_tokens_few, diedIn_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal: {average_diedIn_merged_model_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_diedIn_merged_model_top1_borda_normal_few, _, average_diedIn_merged_model_top1_borda_normal_few = create_results_list(
    diedIn_merged_model_borda_tokens_score_few, diedIn_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal: {average_diedIn_merged_model_top1_borda_normal_few}')
print('\n')

results_list_diedIn_merged_model_top1_borda_spacy_few, _, average_diedIn_merged_model_top1_borda_spacy_few = create_results_list(
    diedIn_merged_model_borda_spacy_tokens_score_few, diedIn_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In spacy: {average_diedIn_merged_model_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top1_spacy_few, _, average_diedIn_bert_top1_spacy_few = create_results_list(
    masked_diedIn_bert_original_spacy_few, diedIn_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Spacy: {average_diedIn_bert_top1_spacy_few}')

results_list_diedIn_bert_top1_few, _, average_diedIn_bert_top1_few = create_results_list(
    masked_diedIn_bert_original_few, diedIn_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Normal: {average_diedIn_bert_top1_few}')

results_list_diedIn_roberta_top1_spacy_few, _, average_diedIn_roberta_top1_spacy_few = create_results_list(
    masked_diedIn_roberta_original_spacy_few, diedIn_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Spacy: {average_diedIn_roberta_top1_spacy_few}')

results_list_diedIn_roberta_top1_few, _, average_diedIn_roberta_top1_few = create_results_list(
    masked_diedIn_roberta_original, diedIn_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Normal: {average_diedIn_roberta_top1_few}')

results_list_diedIn_electra_top1_spacy_few, _, average_diedIn_electra_top1_spacy_few = create_results_list(
    masked_diedIn_electra_original_spacy_few, diedIn_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Spacy: {average_diedIn_electra_top1_spacy_few}')

results_list_diedIn_electra_top1_few, _, average_diedIn_electra_top1_few = create_results_list(
    masked_diedIn_electra_original_few, diedIn_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Normal: {average_diedIn_electra_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy: 0.479
Avg Precision K = 1 of the ranking of died In Normal: 0.47


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal: 0.471


Borda Spacy Score
Avg Precision K = 1 of the borda of died In spacy: 0.498


Sem ranking
Avg Precision K = 1 of died In Bert Spacy: 0.49
Avg Precision K = 1 of died In Bert Normal: 0.469
Avg Precision K = 1 of died In roberta Spacy: 0.449
Avg Precision K = 1 of died In roberta Normal: 0.421
Avg Precision K = 1 of died In electra Spacy: 0.326
Avg Precision K = 1 of died In electra Normal: 0.308


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_diedIn_merged_model_top5_ranking_spacy_few, _, average_diedIn_merged_model_top5_ranking_spacy_few = create_results_list(
    ranking_diedIn_merged_model_spacy_tokens_one, diedIn_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy: {average_diedIn_merged_model_top5_ranking_spacy_few}')

results_list_diedIn_merged_model_top5_ranking_normal_few, _, average_diedIn_merged_model_top5_ranking_normal_few = create_results_list(
    ranking_diedIn_merged_model_normal_tokens_few, diedIn_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal: {average_diedIn_merged_model_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_diedIn_merged_model_top5_borda_normal_few, _, average_diedIn_merged_model_top5_borda_normal_few = create_results_list(
    diedIn_merged_model_borda_tokens_score_few, diedIn_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal: {average_diedIn_merged_model_top5_borda_normal_few}')
print('\n')

results_list_diedIn_merged_model_top5_borda_spacy_few, _, average_diedIn_merged_model_top5_borda_spacy_few = create_results_list(
    diedIn_merged_model_borda_spacy_tokens_score_few, diedIn_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In spacy: {average_diedIn_merged_model_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top5_spacy_few, _, average_diedIn_bert_top5_spacy_few = create_results_list(
    masked_diedIn_bert_original_spacy_few, diedIn_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Spacy: {average_diedIn_bert_top5_spacy_few}')

results_list_diedIn_bert_top5_few, _, average_diedIn_bert_top5_few = create_results_list(
    masked_diedIn_bert_original_few, diedIn_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Normal: {average_diedIn_bert_top5_few}')

results_list_diedIn_roberta_top5_spacy_few, _, average_diedIn_roberta_top5_spacy_few = create_results_list(
    masked_diedIn_roberta_original_spacy_few, diedIn_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Spacy: {average_diedIn_roberta_top5_spacy_few}')

results_list_diedIn_roberta_top5_few, _, average_diedIn_roberta_top5_few = create_results_list(
    masked_diedIn_roberta_original, diedIn_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Normal: {average_diedIn_roberta_top5_few}')

results_list_diedIn_electra_top5_spacy_few, _, average_diedIn_electra_top5_spacy_few = create_results_list(
    masked_diedIn_electra_original_spacy_few, diedIn_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Spacy: {average_diedIn_electra_top5_spacy_few}')

results_list_diedIn_electra_top5_few, _, average_diedIn_electra_top5_few = create_results_list(
    masked_diedIn_electra_original_few, diedIn_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Normal: {average_diedIn_electra_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy: 0.699
Avg Precision K = 5 of the ranking of died In Normal: 0.702


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal: 0.701


Borda Spacy Score
Avg Precision K = 5 of the borda of died In spacy: 0.711


Sem ranking
Avg Precision K = 5 of died In Bert Spacy: 0.712
Avg Precision K = 5 of died In Bert Normal: 0.719
Avg Precision K = 5 of died In roberta Spacy: 0.632
Avg Precision K = 5 of died In roberta Normal: 0.631
Avg Precision K = 5 of died In electra Spacy: 0.567
Avg Precision K = 5 of died In electra Normal: 0.558


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_diedIn_merged_model_top10_ranking_spacy_few, _, average_diedIn_merged_model_top10_ranking_spacy_few = create_results_list(
    ranking_diedIn_merged_model_spacy_tokens_one, diedIn_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy: {average_diedIn_merged_model_top10_ranking_spacy_few}')

results_list_diedIn_merged_model_top10_ranking_normal_few, _, average_diedIn_merged_model_top10_ranking_normal_few = create_results_list(
    ranking_diedIn_merged_model_normal_tokens_few, diedIn_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal: {average_diedIn_merged_model_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_diedIn_merged_model_top10_borda_normal_few, _, average_diedIn_merged_model_top10_borda_normal_few = create_results_list(
    diedIn_merged_model_borda_tokens_score_few, diedIn_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal: {average_diedIn_merged_model_top10_borda_normal_few}')
print('\n')

results_list_diedIn_merged_model_top10_borda_spacy_few, _, average_diedIn_merged_model_top10_borda_spacy_few = create_results_list(
    diedIn_merged_model_borda_spacy_tokens_score_few, diedIn_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In spacy: {average_diedIn_merged_model_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_diedIn_bert_top10_spacy_few, _, average_diedIn_bert_top10_spacy_few = create_results_list(
    masked_diedIn_bert_original_spacy_few, diedIn_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Spacy: {average_diedIn_bert_top10_spacy_few}')

results_list_diedIn_bert_top10_few, _, average_diedIn_bert_top10_few = create_results_list(
    masked_diedIn_bert_original_few, diedIn_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Normal: {average_diedIn_bert_top10_few}')

results_list_diedIn_roberta_top10_spacy_few, _, average_diedIn_roberta_top10_spacy_few = create_results_list(
    masked_diedIn_roberta_original_spacy_few, diedIn_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Spacy: {average_diedIn_roberta_top10_spacy_few}')

results_list_diedIn_roberta_top10_few, _, average_diedIn_roberta_top10_few = create_results_list(
    masked_diedIn_roberta_original, diedIn_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Normal: {average_diedIn_roberta_top10_few}')

results_list_diedIn_electra_top10_spacy_few, _, average_diedIn_electra_top10_spacy_few = create_results_list(
    masked_diedIn_electra_original_spacy_few, diedIn_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Spacy: {average_diedIn_electra_top10_spacy_few}')

results_list_diedIn_electra_top10_few, _, average_diedIn_electra_top10_few = create_results_list(
    masked_diedIn_electra_original_few, diedIn_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Normal: {average_diedIn_electra_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy: 0.761
Avg Precision K = 10 of the ranking of died In Normal: 0.777


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal: 0.778


Borda Spacy Score
Avg Precision K = 10 of the borda of died In spacy: 0.768


Sem ranking
Avg Precision K = 10 of died In Bert Spacy: 0.764
Avg Precision K = 10 of died In Bert Normal: 0.79
Avg Precision K = 10 of died In roberta Spacy: 0.67
Avg Precision K = 10 of died In roberta Normal: 0.689
Avg Precision K = 10 of died In electra Spacy: 0.628
Avg Precision K = 10 of died In electra Normal: 0.647


#### Worksfor

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_worksfor_merged_model_top1_ranking_spacy_few, _, average_worksfor_merged_model_top1_ranking_spacy_few = create_results_list(
    ranking_worksfor_merged_model_spacy_tokens_few, worksfor_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy: {average_worksfor_merged_model_top1_ranking_spacy_few}')

results_list_worksfor_merged_model_top1_ranking_normal_few, _, average_worksfor_merged_model_top1_ranking_normal_few = create_results_list(
    ranking_worksfor_merged_model_normal_tokens_few, worksfor_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal: {average_worksfor_merged_model_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_worksfor_merged_model_top1_borda_normal_few, _, average_worksfor_merged_model_top1_borda_normal_few = create_results_list(
    worksfor_merged_model_borda_tokens_score_few, worksfor_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal: {average_worksfor_merged_model_top1_borda_normal_few}')
print('\n')

results_list_worksfor_merged_model_top1_borda_spacy_few, _, average_worksfor_merged_model_top1_borda_spacy_few = create_results_list(
    worksfor_merged_model_borda_spacy_tokens_score_few, worksfor_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In spacy: {average_worksfor_merged_model_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top1_spacy_few, _, average_worksfor_bert_top1_spacy_few = create_results_list(
    masked_worksfor_bert_original_spacy_few, worksfor_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Spacy: {average_worksfor_bert_top1_spacy_few}')

results_list_worksfor_bert_top1_few, _, average_worksfor_bert_top1_few = create_results_list(
    masked_worksfor_bert_original_few, worksfor_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Normal: {average_worksfor_bert_top1_few}')

results_list_worksfor_roberta_top1_spacy_few, _, average_worksfor_roberta_top1_spacy_few = create_results_list(
    masked_worksfor_roberta_original_spacy_few, worksfor_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Spacy: {average_worksfor_roberta_top1_spacy_few}')

results_list_worksfor_roberta_top1_few, _, average_worksfor_roberta_top1_few = create_results_list(
    masked_worksfor_roberta_original, worksfor_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Normal: {average_worksfor_roberta_top1_few}')

results_list_worksfor_electra_top1_spacy_few, _, average_worksfor_electra_top1_spacy_few = create_results_list(
    masked_worksfor_electra_original_spacy_few, worksfor_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Spacy: {average_worksfor_electra_top1_spacy_few}')

results_list_worksfor_electra_top1_few, _, average_worksfor_electra_top1_few = create_results_list(
    masked_worksfor_electra_original_few, worksfor_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Normal: {average_worksfor_electra_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy: 0.718
Avg Precision K = 1 of the ranking of died In Normal: 0.631


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal: 0.631


Borda Spacy Score
Avg Precision K = 1 of the borda of died In spacy: 0.718


Sem ranking
Avg Precision K = 1 of died In Bert Spacy: 0.013
Avg Precision K = 1 of died In Bert Normal: 0.549
Avg Precision K = 1 of died In roberta Spacy: 0.691
Avg Precision K = 1 of died In roberta Normal: 0.594
Avg Precision K = 1 of died In electra Spacy: 0.536
Avg Precision K = 1 of died In electra Normal: 0.443


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_worksfor_merged_model_top5_ranking_spacy_few, _, average_worksfor_merged_model_top5_ranking_spacy_few = create_results_list(
    ranking_worksfor_merged_model_spacy_tokens_few, worksfor_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy: {average_worksfor_merged_model_top5_ranking_spacy_few}')

results_list_worksfor_merged_model_top5_ranking_normal_few, _, average_worksfor_merged_model_top5_ranking_normal_few = create_results_list(
    ranking_worksfor_merged_model_normal_tokens_few, worksfor_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal: {average_worksfor_merged_model_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_worksfor_merged_model_top5_borda_normal_few, _, average_worksfor_merged_model_top5_borda_normal_few = create_results_list(
    worksfor_merged_model_borda_tokens_score_few, worksfor_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal: {average_worksfor_merged_model_top5_borda_normal_few}')
print('\n')

results_list_worksfor_merged_model_top5_borda_spacy_few, _, average_worksfor_merged_model_top5_borda_spacy_few = create_results_list(
    worksfor_merged_model_borda_spacy_tokens_score_few, worksfor_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In spacy: {average_worksfor_merged_model_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top5_spacy_few, _, average_worksfor_bert_top5_spacy_few = create_results_list(
    masked_worksfor_bert_original_spacy_few, worksfor_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Spacy: {average_worksfor_bert_top5_spacy_few}')

results_list_worksfor_bert_top5_few, _, average_worksfor_bert_top5_few = create_results_list(
    masked_worksfor_bert_original_few, worksfor_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Normal: {average_worksfor_bert_top5_few}')

results_list_worksfor_roberta_top5_spacy_few, _, average_worksfor_roberta_top5_spacy_few = create_results_list(
    masked_worksfor_roberta_original_spacy_few, worksfor_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Spacy: {average_worksfor_roberta_top5_spacy_few}')

results_list_worksfor_roberta_top5_few, _, average_worksfor_roberta_top5_few = create_results_list(
    masked_worksfor_roberta_original, worksfor_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Normal: {average_worksfor_roberta_top5_few}')

results_list_worksfor_electra_top5_spacy_few, _, average_worksfor_electra_top5_spacy_few = create_results_list(
    masked_worksfor_electra_original_spacy_few, worksfor_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Spacy: {average_worksfor_electra_top5_spacy_few}')

results_list_worksfor_electra_top5_few, _, average_worksfor_electra_top5_few = create_results_list(
    masked_worksfor_electra_original_few, worksfor_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Normal: {average_worksfor_electra_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy: 0.86
Avg Precision K = 5 of the ranking of died In Normal: 0.873


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal: 0.871


Borda Spacy Score
Avg Precision K = 5 of the borda of died In spacy: 0.863


Sem ranking
Avg Precision K = 5 of died In Bert Spacy: 0.013
Avg Precision K = 5 of died In Bert Normal: 0.813
Avg Precision K = 5 of died In roberta Spacy: 0.842
Avg Precision K = 5 of died In roberta Normal: 0.792
Avg Precision K = 5 of died In electra Spacy: 0.741
Avg Precision K = 5 of died In electra Normal: 0.715


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_worksfor_merged_model_top10_ranking_spacy_few, _, average_worksfor_merged_model_top10_ranking_spacy_few = create_results_list(
    ranking_worksfor_merged_model_spacy_tokens_few, worksfor_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy: {average_worksfor_merged_model_top10_ranking_spacy_few}')

results_list_worksfor_merged_model_top10_ranking_normal_few, _, average_worksfor_merged_model_top10_ranking_normal_few = create_results_list(
    ranking_worksfor_merged_model_normal_tokens_few, worksfor_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal: {average_worksfor_merged_model_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_worksfor_merged_model_top10_borda_normal_few, _, average_worksfor_merged_model_top10_borda_normal_few = create_results_list(
    worksfor_merged_model_borda_tokens_score_few, worksfor_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal: {average_worksfor_merged_model_top10_borda_normal_few}')
print('\n')

results_list_worksfor_merged_model_top10_borda_spacy_few, _, average_worksfor_merged_model_top10_borda_spacy_few = create_results_list(
    worksfor_merged_model_borda_spacy_tokens_score_few, worksfor_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In spacy: {average_worksfor_merged_model_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_worksfor_bert_top10_spacy_few, _, average_worksfor_bert_top10_spacy_few = create_results_list(
    masked_worksfor_bert_original_spacy_few, worksfor_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Spacy: {average_worksfor_bert_top10_spacy_few}')

results_list_worksfor_bert_top10_few, _, average_worksfor_bert_top10_few = create_results_list(
    masked_worksfor_bert_original_few, worksfor_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Normal: {average_worksfor_bert_top10_few}')

results_list_worksfor_roberta_top10_spacy_few, _, average_worksfor_roberta_top10_spacy_few = create_results_list(
    masked_worksfor_roberta_original_spacy_few, worksfor_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Spacy: {average_worksfor_roberta_top10_spacy_few}')

results_list_worksfor_roberta_top10_few, _, average_worksfor_roberta_top10_few = create_results_list(
    masked_worksfor_roberta_original, worksfor_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Normal: {average_worksfor_roberta_top10_few}')

results_list_worksfor_electra_top10_spacy_few, _, average_worksfor_electra_top10_spacy_few = create_results_list(
    masked_worksfor_electra_original_spacy_few, worksfor_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Spacy: {average_worksfor_electra_top10_spacy_few}')

results_list_worksfor_electra_top10_few, _, average_worksfor_electra_top10_few = create_results_list(
    masked_worksfor_electra_original_few, worksfor_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Normal: {average_worksfor_electra_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy: 0.892
Avg Precision K = 10 of the ranking of died In Normal: 0.921


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal: 0.921


Borda Spacy Score
Avg Precision K = 10 of the borda of died In spacy: 0.889


Sem ranking
Avg Precision K = 10 of died In Bert Spacy: 0.013
Avg Precision K = 10 of died In Bert Normal: 0.871
Avg Precision K = 10 of died In roberta Spacy: 0.858
Avg Precision K = 10 of died In roberta Normal: 0.855
Avg Precision K = 10 of died In electra Spacy: 0.755
Avg Precision K = 10 of died In electra Normal: 0.784


#### Capital

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_capital_merged_model_top1_ranking_spacy_few, _, average_capital_merged_model_top1_ranking_spacy_few = create_results_list(
    ranking_capital_merged_model_spacy_tokens_few, capital_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy: {average_capital_merged_model_top1_ranking_spacy_few}')

results_list_capital_merged_model_top1_ranking_normal_few, _, average_capital_merged_model_top1_ranking_normal_few = create_results_list(
    ranking_capital_merged_model_normal_tokens_few, capital_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal: {average_capital_merged_model_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_capital_merged_model_top1_borda_normal_few, _, average_capital_merged_model_top1_borda_normal_few = create_results_list(
    capital_merged_model_borda_tokens_score_few, capital_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal: {average_capital_merged_model_top1_borda_normal_few}')
print('\n')

results_list_capital_merged_model_top1_borda_spacy_few, _, average_capital_merged_model_top1_borda_spacy_few = create_results_list(
    capital_merged_model_borda_spacy_tokens_score_few, capital_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In spacy: {average_capital_merged_model_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top1_spacy_few, _, average_capital_bert_top1_spacy_few = create_results_list(
    masked_capital_bert_original_spacy_few, capital_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Spacy: {average_capital_bert_top1_spacy_few}')

results_list_capital_bert_top1_few, _, average_capital_bert_top1_few = create_results_list(
    masked_capital_bert_original_few, capital_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Normal: {average_capital_bert_top1_few}')

results_list_capital_roberta_top1_spacy_few, _, average_capital_roberta_top1_spacy_few = create_results_list(
    masked_capital_roberta_original_spacy_few, capital_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Spacy: {average_capital_roberta_top1_spacy_few}')

results_list_capital_roberta_top1_few, _, average_capital_roberta_top1_few = create_results_list(
    masked_capital_roberta_original, capital_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Normal: {average_capital_roberta_top1_few}')

results_list_capital_electra_top1_spacy_few, _, average_capital_electra_top1_spacy_few = create_results_list(
    masked_capital_electra_original_spacy_few, capital_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Spacy: {average_capital_electra_top1_spacy_few}')

results_list_capital_electra_top1_few, _, average_capital_electra_top1_few = create_results_list(
    masked_capital_electra_original_few, capital_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Normal: {average_capital_electra_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy: 0.517
Avg Precision K = 1 of the ranking of died In Normal: 0.512


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal: 0.508


Borda Spacy Score
Avg Precision K = 1 of the borda of died In spacy: 0.518


Sem ranking
Avg Precision K = 1 of died In Bert Spacy: 0.509
Avg Precision K = 1 of died In Bert Normal: 0.508
Avg Precision K = 1 of died In roberta Spacy: 0.346
Avg Precision K = 1 of died In roberta Normal: 0.367
Avg Precision K = 1 of died In electra Spacy: 0.284
Avg Precision K = 1 of died In electra Normal: 0.284


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_capital_merged_model_top5_ranking_spacy_few, _, average_capital_merged_model_top5_ranking_spacy_few = create_results_list(
    ranking_capital_merged_model_spacy_tokens_few, capital_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy: {average_capital_merged_model_top5_ranking_spacy_few}')

results_list_capital_merged_model_top5_ranking_normal_few, _, average_capital_merged_model_top5_ranking_normal_few = create_results_list(
    ranking_capital_merged_model_normal_tokens_few, capital_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal: {average_capital_merged_model_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_capital_merged_model_top5_borda_normal_few, _, average_capital_merged_model_top5_borda_normal_few = create_results_list(
    capital_merged_model_borda_tokens_score_few, capital_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal: {average_capital_merged_model_top5_borda_normal_few}')
print('\n')

results_list_capital_merged_model_top5_borda_spacy_few, _, average_capital_merged_model_top5_borda_spacy_few = create_results_list(
    capital_merged_model_borda_spacy_tokens_score_few, capital_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In spacy: {average_capital_merged_model_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top5_spacy_few, _, average_capital_bert_top5_spacy_few = create_results_list(
    masked_capital_bert_original_spacy_few, capital_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Spacy: {average_capital_bert_top5_spacy_few}')

results_list_capital_bert_top5_few, _, average_capital_bert_top5_few = create_results_list(
    masked_capital_bert_original_few, capital_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Normal: {average_capital_bert_top5_few}')

results_list_capital_roberta_top5_spacy_few, _, average_capital_roberta_top5_spacy_few = create_results_list(
    masked_capital_roberta_original_spacy_few, capital_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Spacy: {average_capital_roberta_top5_spacy_few}')

results_list_capital_roberta_top5_few, _, average_capital_roberta_top5_few = create_results_list(
    masked_capital_roberta_original, capital_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Normal: {average_capital_roberta_top5_few}')

results_list_capital_electra_top5_spacy_few, _, average_capital_electra_top5_spacy_few = create_results_list(
    masked_capital_electra_original_spacy_few, capital_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Spacy: {average_capital_electra_top5_spacy_few}')

results_list_capital_electra_top5_few, _, average_capital_electra_top5_few = create_results_list(
    masked_capital_electra_original_few, capital_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Normal: {average_capital_electra_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy: 0.704
Avg Precision K = 5 of the ranking of died In Normal: 0.722


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal: 0.719


Borda Spacy Score
Avg Precision K = 5 of the borda of died In spacy: 0.707


Sem ranking
Avg Precision K = 5 of died In Bert Spacy: 0.696
Avg Precision K = 5 of died In Bert Normal: 0.726
Avg Precision K = 5 of died In roberta Spacy: 0.442
Avg Precision K = 5 of died In roberta Normal: 0.498
Avg Precision K = 5 of died In electra Spacy: 0.45
Avg Precision K = 5 of died In electra Normal: 0.459


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_capital_merged_model_top10_ranking_spacy_few, _, average_capital_merged_model_top10_ranking_spacy_few = create_results_list(
    ranking_capital_merged_model_spacy_tokens_few, capital_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy: {average_capital_merged_model_top10_ranking_spacy_few}')

results_list_capital_merged_model_top10_ranking_normal_few, _, average_capital_merged_model_top10_ranking_normal_few = create_results_list(
    ranking_capital_merged_model_normal_tokens_few, capital_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal: {average_capital_merged_model_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_capital_merged_model_top10_borda_normal_few, _, average_capital_merged_model_top10_borda_normal_few = create_results_list(
    capital_merged_model_borda_tokens_score_few, capital_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal: {average_capital_merged_model_top10_borda_normal_few}')
print('\n')

results_list_capital_merged_model_top10_borda_spacy_few, _, average_capital_merged_model_top10_borda_spacy_few = create_results_list(
    capital_merged_model_borda_spacy_tokens_score_few, capital_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In spacy: {average_capital_merged_model_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_capital_bert_top10_spacy_few, _, average_capital_bert_top10_spacy_few = create_results_list(
    masked_capital_bert_original_spacy_few, capital_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Spacy: {average_capital_bert_top10_spacy_few}')

results_list_capital_bert_top10_few, _, average_capital_bert_top10_few = create_results_list(
    masked_capital_bert_original_few, capital_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Normal: {average_capital_bert_top10_few}')

results_list_capital_roberta_top10_spacy_few, _, average_capital_roberta_top10_spacy_few = create_results_list(
    masked_capital_roberta_original_spacy_few, capital_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Spacy: {average_capital_roberta_top10_spacy_few}')

results_list_capital_roberta_top10_few, _, average_capital_roberta_top10_few = create_results_list(
    masked_capital_roberta_original, capital_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Normal: {average_capital_roberta_top10_few}')

results_list_capital_electra_top10_spacy_few, _, average_capital_electra_top10_spacy_few = create_results_list(
    masked_capital_electra_original_spacy_few, capital_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Spacy: {average_capital_electra_top10_spacy_few}')

results_list_capital_electra_top10_few, _, average_capital_electra_top10_few = create_results_list(
    masked_capital_electra_original_few, capital_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Normal: {average_capital_electra_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy: 0.743
Avg Precision K = 10 of the ranking of died In Normal: 0.775


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal: 0.776


Borda Spacy Score
Avg Precision K = 10 of the borda of died In spacy: 0.739


Sem ranking
Avg Precision K = 10 of died In Bert Spacy: 0.723
Avg Precision K = 10 of died In Bert Normal: 0.765
Avg Precision K = 10 of died In roberta Spacy: 0.459
Avg Precision K = 10 of died In roberta Normal: 0.519
Avg Precision K = 10 of died In electra Spacy: 0.509
Avg Precision K = 10 of died In electra Normal: 0.532


#### Citizen

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_citizen_merged_model_top1_ranking_spacy_few, _, average_citizen_merged_model_top1_ranking_spacy_few = create_results_list(
    ranking_citizen_merged_model_spacy_tokens_few, citizen_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy: {average_citizen_merged_model_top1_ranking_spacy_few}')

results_list_citizen_merged_model_top1_ranking_normal_few, _, average_citizen_merged_model_top1_ranking_normal_few = create_results_list(
    ranking_citizen_merged_model_normal_tokens_few, citizen_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal: {average_citizen_merged_model_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_citizen_merged_model_top1_borda_normal_few, _, average_citizen_merged_model_top1_borda_normal_few = create_results_list(
    citizen_merged_model_borda_tokens_score_few, citizen_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal: {average_citizen_merged_model_top1_borda_normal_few}')
print('\n')

results_list_citizen_merged_model_top1_borda_spacy_few, _, average_citizen_merged_model_top1_borda_spacy_few = create_results_list(
    citizen_merged_model_borda_spacy_tokens_score_few, citizen_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In spacy: {average_citizen_merged_model_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top1_spacy_few, _, average_citizen_bert_top1_spacy_few = create_results_list(
    masked_citizen_bert_original_spacy_few, citizen_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Spacy: {average_citizen_bert_top1_spacy_few}')

results_list_citizen_bert_top1_few, _, average_citizen_bert_top1_few = create_results_list(
    masked_citizen_bert_original_few, citizen_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Normal: {average_citizen_bert_top1_few}')

results_list_citizen_roberta_top1_spacy_few, _, average_citizen_roberta_top1_spacy_few = create_results_list(
    masked_citizen_roberta_original_spacy_few, citizen_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Spacy: {average_citizen_roberta_top1_spacy_few}')

results_list_citizen_roberta_top1_few, _, average_citizen_roberta_top1_few = create_results_list(
    masked_citizen_roberta_original, citizen_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Normal: {average_citizen_roberta_top1_few}')

results_list_citizen_electra_top1_spacy_few, _, average_citizen_electra_top1_spacy_few = create_results_list(
    masked_citizen_electra_original_spacy_few, citizen_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Spacy: {average_citizen_electra_top1_spacy_few}')

results_list_citizen_electra_top1_few, _, average_citizen_electra_top1_few = create_results_list(
    masked_citizen_electra_original_few, citizen_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Normal: {average_citizen_electra_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy: 0.455
Avg Precision K = 1 of the ranking of died In Normal: 0.597


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal: 0.597


Borda Spacy Score
Avg Precision K = 1 of the borda of died In spacy: 0.449


Sem ranking
Avg Precision K = 1 of died In Bert Spacy: 0.434
Avg Precision K = 1 of died In Bert Normal: 0.578
Avg Precision K = 1 of died In roberta Spacy: 0.423
Avg Precision K = 1 of died In roberta Normal: 0.567
Avg Precision K = 1 of died In electra Spacy: 0.307
Avg Precision K = 1 of died In electra Normal: 0.459


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_citizen_merged_model_top5_ranking_spacy_few, _, average_citizen_merged_model_top5_ranking_spacy_few = create_results_list(
    ranking_citizen_merged_model_spacy_tokens_few, citizen_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy: {average_citizen_merged_model_top5_ranking_spacy_few}')

results_list_citizen_merged_model_top5_ranking_normal_few, _, average_citizen_merged_model_top5_ranking_normal_few = create_results_list(
    ranking_citizen_merged_model_normal_tokens_few, citizen_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal: {average_citizen_merged_model_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_citizen_merged_model_top5_borda_normal_few, _, average_citizen_merged_model_top5_borda_normal_few = create_results_list(
    citizen_merged_model_borda_tokens_score_few, citizen_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal: {average_citizen_merged_model_top5_borda_normal_few}')
print('\n')

results_list_citizen_merged_model_top5_borda_spacy_few, _, average_citizen_merged_model_top5_borda_spacy_few = create_results_list(
    citizen_merged_model_borda_spacy_tokens_score_few, citizen_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In spacy: {average_citizen_merged_model_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top5_spacy_few, _, average_citizen_bert_top5_spacy_few = create_results_list(
    masked_citizen_bert_original_spacy_few, citizen_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Spacy: {average_citizen_bert_top5_spacy_few}')

results_list_citizen_bert_top5_few, _, average_citizen_bert_top5_few = create_results_list(
    masked_citizen_bert_original_few, citizen_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Normal: {average_citizen_bert_top5_few}')

results_list_citizen_roberta_top5_spacy_few, _, average_citizen_roberta_top5_spacy_few = create_results_list(
    masked_citizen_roberta_original_spacy_few, citizen_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Spacy: {average_citizen_roberta_top5_spacy_few}')

results_list_citizen_roberta_top5_few, _, average_citizen_roberta_top5_few = create_results_list(
    masked_citizen_roberta_original, citizen_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Normal: {average_citizen_roberta_top5_few}')

results_list_citizen_electra_top5_spacy_few, _, average_citizen_electra_top5_spacy_few = create_results_list(
    masked_citizen_electra_original_spacy_few, citizen_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Spacy: {average_citizen_electra_top5_spacy_few}')

results_list_citizen_electra_top5_few, _, average_citizen_electra_top5_few = create_results_list(
    masked_citizen_electra_original_few, citizen_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Normal: {average_citizen_electra_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy: 0.516
Avg Precision K = 5 of the ranking of died In Normal: 0.709


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal: 0.711


Borda Spacy Score
Avg Precision K = 5 of the borda of died In spacy: 0.517


Sem ranking
Avg Precision K = 5 of died In Bert Spacy: 0.495
Avg Precision K = 5 of died In Bert Normal: 0.745
Avg Precision K = 5 of died In roberta Spacy: 0.48
Avg Precision K = 5 of died In roberta Normal: 0.709
Avg Precision K = 5 of died In electra Spacy: 0.388
Avg Precision K = 5 of died In electra Normal: 0.593


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_citizen_merged_model_top10_ranking_spacy_few, _, average_citizen_merged_model_top10_ranking_spacy_few = create_results_list(
    ranking_citizen_merged_model_spacy_tokens_few, citizen_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy: {average_citizen_merged_model_top10_ranking_spacy_few}')

results_list_citizen_merged_model_top10_ranking_normal_few, _, average_citizen_merged_model_top10_ranking_normal_few = create_results_list(
    ranking_citizen_merged_model_normal_tokens_few, citizen_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal: {average_citizen_merged_model_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_citizen_merged_model_top10_borda_normal_few, _, average_citizen_merged_model_top10_borda_normal_few = create_results_list(
    citizen_merged_model_borda_tokens_score_few, citizen_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal: {average_citizen_merged_model_top10_borda_normal_few}')
print('\n')

results_list_citizen_merged_model_top10_borda_spacy_few, _, average_citizen_merged_model_top10_borda_spacy_few = create_results_list(
    citizen_merged_model_borda_spacy_tokens_score_few, citizen_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In spacy: {average_citizen_merged_model_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_citizen_bert_top10_spacy_few, _, average_citizen_bert_top10_spacy_few = create_results_list(
    masked_citizen_bert_original_spacy_few, citizen_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Spacy: {average_citizen_bert_top10_spacy_few}')

results_list_citizen_bert_top10_few, _, average_citizen_bert_top10_few = create_results_list(
    masked_citizen_bert_original_few, citizen_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Normal: {average_citizen_bert_top10_few}')

results_list_citizen_roberta_top10_spacy_few, _, average_citizen_roberta_top10_spacy_few = create_results_list(
    masked_citizen_roberta_original_spacy_few, citizen_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Spacy: {average_citizen_roberta_top10_spacy_few}')

results_list_citizen_roberta_top10_few, _, average_citizen_roberta_top10_few = create_results_list(
    masked_citizen_roberta_original, citizen_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Normal: {average_citizen_roberta_top10_few}')

results_list_citizen_electra_top10_spacy_few, _, average_citizen_electra_top10_spacy_few = create_results_list(
    masked_citizen_electra_original_spacy_few, citizen_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Spacy: {average_citizen_electra_top10_spacy_few}')

results_list_citizen_electra_top10_few, _, average_citizen_electra_top10_few = create_results_list(
    masked_citizen_electra_original_few, citizen_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Normal: {average_citizen_electra_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy: 0.524
Avg Precision K = 10 of the ranking of died In Normal: 0.768


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal: 0.768


Borda Spacy Score
Avg Precision K = 10 of the borda of died In spacy: 0.525


Sem ranking
Avg Precision K = 10 of died In Bert Spacy: 0.497
Avg Precision K = 10 of died In Bert Normal: 0.807
Avg Precision K = 10 of died In roberta Spacy: 0.482
Avg Precision K = 10 of died In roberta Normal: 0.763
Avg Precision K = 10 of died In electra Spacy: 0.393
Avg Precision K = 10 of died In electra Normal: 0.656


#### Language

In [ ]:
print('K = 1')
print('Com Ranking')
results_list_language_merged_model_top1_ranking_spacy_few, _, average_language_merged_model_top1_ranking_spacy_few = create_results_list(
    ranking_language_merged_model_spacy_tokens_few, language_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Spacy: {average_language_merged_model_top1_ranking_spacy_few}')

results_list_language_merged_model_top1_ranking_normal_few, _, average_language_merged_model_top1_ranking_normal_few = create_results_list(
    ranking_language_merged_model_normal_tokens_few, language_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of the ranking of died In Normal: {average_language_merged_model_top1_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_language_merged_model_top1_borda_normal_few, _, average_language_merged_model_top1_borda_normal_few = create_results_list(
    language_merged_model_borda_tokens_score_few, language_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print('Borda Score')
print(f'Avg Precision K = 1 of the borda of died In Normal: {average_language_merged_model_top1_borda_normal_few}')
print('\n')

results_list_language_merged_model_top1_borda_spacy_few, _, average_language_merged_model_top1_borda_spacy_few = create_results_list(
    language_merged_model_borda_spacy_tokens_score_few, language_merged_model_spacy_few.answers, 1
    ) # top-1 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 1 of the borda of died In spacy: {average_language_merged_model_top1_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_language_bert_top1_spacy_few, _, average_language_bert_top1_spacy_few = create_results_list(
    masked_language_bert_original_spacy_few, language_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Spacy: {average_language_bert_top1_spacy_few}')

results_list_language_bert_top1_few, _, average_language_bert_top1_few = create_results_list(
    masked_language_bert_original_few, language_bert_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In Bert Normal: {average_language_bert_top1_few}')

results_list_language_roberta_top1_spacy_few, _, average_language_roberta_top1_spacy_few = create_results_list(
    masked_language_roberta_original_spacy_few, language_roberta_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Spacy: {average_language_roberta_top1_spacy_few}')

results_list_language_roberta_top1_few, _, average_language_roberta_top1_few = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In roberta Normal: {average_language_roberta_top1_few}')

results_list_language_electra_top1_spacy_few, _, average_language_electra_top1_spacy_few = create_results_list(
    masked_language_electra_original_spacy_few, language_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Spacy: {average_language_electra_top1_spacy_few}')

results_list_language_electra_top1_few, _, average_language_electra_top1_few = create_results_list(
    masked_language_electra_original_few, language_electra_few.answers, 1
    ) # top-1 accuracy
print(f'Avg Precision K = 1 of died In electra Normal: {average_language_electra_top1_few}')

K = 1
Com Ranking
Avg Precision K = 1 of the ranking of died In Spacy: 0.683
Avg Precision K = 1 of the ranking of died In Normal: 0.725


Borda Count
Borda Score
Avg Precision K = 1 of the borda of died In Normal: 0.721


Borda Spacy Score
Avg Precision K = 1 of the borda of died In spacy: 0.676


Sem ranking
Avg Precision K = 1 of died In Bert Spacy: 0.665
Avg Precision K = 1 of died In Bert Normal: 0.718
Avg Precision K = 1 of died In roberta Spacy: 0.611
Avg Precision K = 1 of died In roberta Normal: 0.639
Avg Precision K = 1 of died In electra Spacy: 0.4
Avg Precision K = 1 of died In electra Normal: 0.371


In [ ]:
print('K = 5')
print('Com Ranking')
results_list_language_merged_model_top5_ranking_spacy_few, _, average_language_merged_model_top5_ranking_spacy_few = create_results_list(
    ranking_language_merged_model_spacy_tokens_few, language_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Spacy: {average_language_merged_model_top5_ranking_spacy_few}')

results_list_language_merged_model_top5_ranking_normal_few, _, average_language_merged_model_top5_ranking_normal_few = create_results_list(
    ranking_language_merged_model_normal_tokens_few, language_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of the ranking of died In Normal: {average_language_merged_model_top5_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_language_merged_model_top5_borda_normal_few, _, average_language_merged_model_top5_borda_normal_few = create_results_list(
    language_merged_model_borda_tokens_score_few, language_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print('Borda Score')
print(f'Avg Precision K = 5 of the borda of died In Normal: {average_language_merged_model_top5_borda_normal_few}')
print('\n')

results_list_language_merged_model_top5_borda_spacy_few, _, average_language_merged_model_top5_borda_spacy_few = create_results_list(
    language_merged_model_borda_spacy_tokens_score_few, language_merged_model_spacy_few.answers, 5
    ) # top-5 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 5 of the borda of died In spacy: {average_language_merged_model_top5_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_language_bert_top5_spacy_few, _, average_language_bert_top5_spacy_few = create_results_list(
    masked_language_bert_original_spacy_few, language_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Spacy: {average_language_bert_top5_spacy_few}')

results_list_language_bert_top5_few, _, average_language_bert_top5_few = create_results_list(
    masked_language_bert_original_few, language_bert_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In Bert Normal: {average_language_bert_top5_few}')

results_list_language_roberta_top5_spacy_few, _, average_language_roberta_top5_spacy_few = create_results_list(
    masked_language_roberta_original_spacy_few, language_roberta_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Spacy: {average_language_roberta_top5_spacy_few}')

results_list_language_roberta_top5_few, _, average_language_roberta_top5_few = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In roberta Normal: {average_language_roberta_top5_few}')

results_list_language_electra_top5_spacy_few, _, average_language_electra_top5_spacy_few = create_results_list(
    masked_language_electra_original_spacy_few, language_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Spacy: {average_language_electra_top5_spacy_few}')

results_list_language_electra_top5_few, _, average_language_electra_top5_few = create_results_list(
    masked_language_electra_original_few, language_electra_few.answers, 5
    ) # top-5 accuracy
print(f'Avg Precision K = 5 of died In electra Normal: {average_language_electra_top5_few}')

K = 5
Com Ranking
Avg Precision K = 5 of the ranking of died In Spacy: 0.791
Avg Precision K = 5 of the ranking of died In Normal: 0.876


Borda Count
Borda Score
Avg Precision K = 5 of the borda of died In Normal: 0.877


Borda Spacy Score
Avg Precision K = 5 of the borda of died In spacy: 0.793


Sem ranking
Avg Precision K = 5 of died In Bert Spacy: 0.777
Avg Precision K = 5 of died In Bert Normal: 0.888
Avg Precision K = 5 of died In roberta Spacy: 0.71
Avg Precision K = 5 of died In roberta Normal: 0.785
Avg Precision K = 5 of died In electra Spacy: 0.624
Avg Precision K = 5 of died In electra Normal: 0.673


In [ ]:
print('K = 10')
print('Com Ranking')
results_list_language_merged_model_top10_ranking_spacy_few, _, average_language_merged_model_top10_ranking_spacy_few = create_results_list(
    ranking_language_merged_model_spacy_tokens_few, language_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Spacy: {average_language_merged_model_top10_ranking_spacy_few}')

results_list_language_merged_model_top10_ranking_normal_few, _, average_language_merged_model_top10_ranking_normal_few = create_results_list(
    ranking_language_merged_model_normal_tokens_few, language_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of the ranking of died In Normal: {average_language_merged_model_top10_ranking_normal_few}')
print('\n')

print('Borda Count')
results_list_language_merged_model_top10_borda_normal_few, _, average_language_merged_model_top10_borda_normal_few = create_results_list(
    language_merged_model_borda_tokens_score_few, language_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print('Borda Score')
print(f'Avg Precision K = 10 of the borda of died In Normal: {average_language_merged_model_top10_borda_normal_few}')
print('\n')

results_list_language_merged_model_top10_borda_spacy_few, _, average_language_merged_model_top10_borda_spacy_few = create_results_list(
    language_merged_model_borda_spacy_tokens_score_few, language_merged_model_spacy_few.answers, 10
    ) # top-10 accuracy
print('Borda Spacy Score')
print(f'Avg Precision K = 10 of the borda of died In spacy: {average_language_merged_model_top10_borda_spacy_few}')
print('\n')

print('Sem ranking')
results_list_language_bert_top10_spacy_few, _, average_language_bert_top10_spacy_few = create_results_list(
    masked_language_bert_original_spacy_few, language_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Spacy: {average_language_bert_top10_spacy_few}')

results_list_language_bert_top10_few, _, average_language_bert_top10_few = create_results_list(
    masked_language_bert_original_few, language_bert_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In Bert Normal: {average_language_bert_top10_few}')

results_list_language_roberta_top10_spacy_few, _, average_language_roberta_top10_spacy_few = create_results_list(
    masked_language_roberta_original_spacy_few, language_roberta_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Spacy: {average_language_roberta_top10_spacy_few}')

results_list_language_roberta_top10_few, _, average_language_roberta_top10_few = create_results_list(
    masked_language_roberta_original, language_roberta.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In roberta Normal: {average_language_roberta_top10_few}')

results_list_language_electra_top10_spacy_few, _, average_language_electra_top10_spacy_few = create_results_list(
    masked_language_electra_original_spacy_few, language_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Spacy: {average_language_electra_top10_spacy_few}')

results_list_language_electra_top10_few, _, average_language_electra_top10_few = create_results_list(
    masked_language_electra_original_few, language_electra_few.answers, 10
    ) # top-10 accuracy
print(f'Avg Precision K = 10 of died In electra Normal: {average_language_electra_top10_few}')

K = 10
Com Ranking
Avg Precision K = 10 of the ranking of died In Spacy: 0.801
Avg Precision K = 10 of the ranking of died In Normal: 0.925


Borda Count
Borda Score
Avg Precision K = 10 of the borda of died In Normal: 0.925


Borda Spacy Score
Avg Precision K = 10 of the borda of died In spacy: 0.801


Sem ranking
Avg Precision K = 10 of died In Bert Spacy: 0.783
Avg Precision K = 10 of died In Bert Normal: 0.924
Avg Precision K = 10 of died In roberta Spacy: 0.715
Avg Precision K = 10 of died In roberta Normal: 0.831
Avg Precision K = 10 of died In electra Spacy: 0.649
Avg Precision K = 10 of died In electra Normal: 0.768


# Excel

## Sem Shot

### Bert

In [ ]:
wb = load_workbook(filename='/home/rafael/tese/Slide/testes_2_triple_accuracy.xlsx')
wb.sheetnames

['bert_few_shot',
 'bert_one_shot',
 'bert',
 'roberta_few_shot',
 'roberta_one_shot',
 'roberta',
 'electra_few_shot',
 'electra_one_shot',
 'electra',
 'merge_models_few_shot',
 'merge_models_one_shot',
 'merge_models']

In [ ]:
# Select the active worksheet
sheet = wb['bert']

# Insert data into specific cells
#Born In
## Ranking
### Normal
sheet['F4'] = average_bornIn_bert_top1_ranking_normal
sheet['F5'] = average_bornIn_bert_top5_ranking_normal
sheet['F6'] = average_bornIn_bert_top10_ranking_normal
### Spacy
sheet['G4'] = average_bornIn_bert_top1_ranking_spacy
sheet['G5'] = average_bornIn_bert_top5_ranking_spacy
sheet['G6'] = average_bornIn_bert_top10_ranking_spacy

## Borda Count
### Normal
sheet['D4'] = average_bornIn_bert_top1_borda_normal
sheet['D5'] = average_bornIn_bert_top5_borda_normal
sheet['D6'] = average_bornIn_bert_top10_borda_normal
### Spacy
sheet['E4'] = average_bornIn_bert_top1_borda_spacy
sheet['E5'] = average_bornIn_bert_top5_borda_spacy
sheet['E6'] = average_bornIn_bert_top10_borda_spacy

## Prompts Originais
### Normal
sheet['B4'] = average_bornIn_bert_top1
sheet['B5'] = average_bornIn_bert_top5
sheet['B6'] = average_bornIn_bert_top10
### Spacy
sheet['C4'] = average_bornIn_bert_top1_spacy
sheet['C5'] = average_bornIn_bert_top5_spacy
sheet['C6'] = average_bornIn_bert_top10_spacy

#Died In
## Ranking
### Normal
sheet['I4'] = average_diedIn_bert_top1_ranking_normal
sheet['I5'] = average_diedIn_bert_top5_ranking_normal
sheet['I6'] = average_diedIn_bert_top10_ranking_normal
### Spacy
sheet['M4'] = average_diedIn_bert_top1_ranking_spacy
sheet['M5'] = average_diedIn_bert_top5_ranking_spacy
sheet['M6'] = average_diedIn_bert_top10_ranking_spacy

## Borda Count
### Normal
sheet['J4'] = average_diedIn_bert_top1_borda_normal
sheet['J5'] = average_diedIn_bert_top5_borda_normal
sheet['J6'] = average_diedIn_bert_top10_borda_normal
### Spacy
sheet['K4'] = average_diedIn_bert_top1_borda_spacy
sheet['K5'] = average_diedIn_bert_top5_borda_spacy
sheet['K6'] = average_diedIn_bert_top10_borda_spacy

## Prompts Originais
### Normal
sheet['H4'] = average_diedIn_bert_top1
sheet['H5'] = average_diedIn_bert_top5
sheet['H6'] = average_diedIn_bert_top10
### Spacy
sheet['I4'] = average_diedIn_bert_top1_spacy
sheet['I5'] = average_diedIn_bert_top5_spacy
sheet['I6'] = average_diedIn_bert_top10_spacy

#Works For
## Ranking
### Normal
sheet['R4'] = average_worksfor_bert_top1_ranking_normal
sheet['R5'] = average_worksfor_bert_top5_ranking_normal
sheet['R6'] = average_worksfor_bert_top10_ranking_normal
### Spacy
sheet['S4'] = average_worksfor_bert_top1_ranking_spacy
sheet['S5'] = average_worksfor_bert_top5_ranking_spacy
sheet['S6'] = average_worksfor_bert_top10_ranking_spacy

## Borda Count
### Normal
sheet['P4'] = average_worksfor_bert_top1_borda_normal
sheet['P5'] = average_worksfor_bert_top5_borda_normal
sheet['P6'] = average_worksfor_bert_top10_borda_normal
### Spacy
sheet['Q4'] = average_worksfor_bert_top1_borda_spacy
sheet['Q5'] = average_worksfor_bert_top5_borda_spacy
sheet['Q6'] = average_worksfor_bert_top10_borda_spacy

## Prompts Originais
### Normal
sheet['N4'] = average_worksfor_bert_top1
sheet['N5'] = average_worksfor_bert_top5
sheet['N6'] = average_worksfor_bert_top10
### Spacy
sheet['O4'] = average_worksfor_bert_top1_spacy
sheet['O5'] = average_worksfor_bert_top5_spacy
sheet['O6'] = average_worksfor_bert_top10_spacy

#Capital
## Ranking
### Normal
sheet['F12'] = average_capital_bert_top1_ranking_normal
sheet['F13'] = average_capital_bert_top5_ranking_normal
sheet['F14'] = average_capital_bert_top10_ranking_normal
### Spacy
sheet['G12'] = average_capital_bert_top1_ranking_spacy
sheet['G13'] = average_capital_bert_top5_ranking_spacy
sheet['G14'] = average_capital_bert_top10_ranking_spacy

## Borda Count
### Normal
sheet['D12'] = average_capital_bert_top1_borda_normal
sheet['D13'] = average_capital_bert_top5_borda_normal
sheet['D14'] = average_capital_bert_top10_borda_normal
### Spacy
sheet['E12'] = average_capital_bert_top1_borda_spacy
sheet['E13'] = average_capital_bert_top5_borda_spacy
sheet['E14'] = average_capital_bert_top10_borda_spacy

## Prompts Originais
### Normal
sheet['B12'] = average_capital_bert_top1
sheet['B13'] = average_capital_bert_top5
sheet['B14'] = average_capital_bert_top10
### Spacy
sheet['C12'] = average_capital_bert_top1_spacy
sheet['C13'] = average_capital_bert_top5_spacy
sheet['C14'] = average_capital_bert_top10_spacy

#Citizen
## Ranking
### Normal
sheet['L12'] = average_citizen_bert_top1_ranking_normal
sheet['L13'] = average_citizen_bert_top5_ranking_normal
sheet['L14'] = average_citizen_bert_top10_ranking_normal
### Spacy
sheet['M12'] = average_citizen_bert_top1_ranking_spacy
sheet['M13'] = average_citizen_bert_top5_ranking_spacy
sheet['M14'] = average_citizen_bert_top10_ranking_spacy

## Borda Count
### Normal
sheet['J12'] = average_citizen_bert_top1_borda_normal
sheet['J13'] = average_citizen_bert_top5_borda_normal
sheet['J14'] = average_citizen_bert_top10_borda_normal
### Spacy
sheet['K12'] = average_citizen_bert_top1_borda_spacy
sheet['K13'] = average_citizen_bert_top5_borda_spacy
sheet['K14'] = average_citizen_bert_top10_borda_spacy

## Prompts Originais
### Normal
sheet['H12'] = average_citizen_bert_top1
sheet['H13'] = average_citizen_bert_top5
sheet['H14'] = average_citizen_bert_top10
### Spacy
sheet['I12'] = average_citizen_bert_top1_spacy
sheet['I13'] = average_citizen_bert_top5_spacy
sheet['I14'] = average_citizen_bert_top10_spacy


#Oficial Language
## Ranking
### Normal
sheet['R12'] = average_language_bert_top1_ranking_normal
sheet['R13'] = average_language_bert_top5_ranking_normal
sheet['R14'] = average_language_bert_top10_ranking_normal
### Spacy
sheet['S12'] = average_language_bert_top1_ranking_spacy
sheet['S13'] = average_language_bert_top5_ranking_spacy
sheet['S14'] = average_language_bert_top10_ranking_spacy

## Borda Count
### Normal
sheet['P12'] = average_language_bert_top1_borda_normal
sheet['P13'] = average_language_bert_top5_borda_normal
sheet['P14'] = average_language_bert_top10_borda_normal
### Spacy
sheet['Q12'] = average_language_bert_top1_borda_spacy
sheet['Q13'] = average_language_bert_top5_borda_spacy
sheet['Q14'] = average_language_bert_top10_borda_spacy

# Save the changes
wb.save('/home/rafael/tese/Slide/testes_2.xlsx')

### Roberta

In [ ]:
# Select the active worksheet
sheet = wb['roberta']

# Insert data into specific cells
#Born In
## Ranking
### Normal
sheet['F4'] = average_bornIn_roberta_top1_ranking_normal
sheet['F5'] = average_bornIn_roberta_top5_ranking_normal
sheet['F6'] = average_bornIn_roberta_top10_ranking_normal
### Spacy
sheet['G4'] = average_bornIn_roberta_top1_ranking_spacy
sheet['G5'] = average_bornIn_roberta_top5_ranking_spacy
sheet['G6'] = average_bornIn_roberta_top10_ranking_spacy

## Borda Count
### Normal
sheet['D4'] = average_bornIn_roberta_top1_borda_normal
sheet['D5'] = average_bornIn_roberta_top5_borda_normal
sheet['D6'] = average_bornIn_roberta_top10_borda_normal
### Spacy
sheet['E4'] = average_bornIn_roberta_top1_borda_spacy
sheet['E5'] = average_bornIn_roberta_top5_borda_spacy
sheet['E6'] = average_bornIn_roberta_top10_borda_spacy

## Prompts Originais
### Normal
sheet['B4'] = average_bornIn_roberta_top1
sheet['B5'] = average_bornIn_roberta_top5
sheet['B6'] = average_bornIn_roberta_top10
### Spacy
sheet['C4'] = average_bornIn_roberta_top1_spacy
sheet['C5'] = average_bornIn_roberta_top5_spacy
sheet['C6'] = average_bornIn_roberta_top10_spacy

#Died In
## Ranking
### Normal
sheet['I4'] = average_diedIn_roberta_top1_ranking_normal
sheet['I5'] = average_diedIn_roberta_top5_ranking_normal
sheet['I6'] = average_diedIn_roberta_top10_ranking_normal
### Spacy
sheet['M4'] = average_diedIn_roberta_top1_ranking_spacy
sheet['M5'] = average_diedIn_roberta_top5_ranking_spacy
sheet['M6'] = average_diedIn_roberta_top10_ranking_spacy

## Borda Count
### Normal
sheet['J4'] = average_diedIn_roberta_top1_borda_normal
sheet['J5'] = average_diedIn_roberta_top5_borda_normal
sheet['J6'] = average_diedIn_roberta_top10_borda_normal
### Spacy
sheet['K4'] = average_diedIn_roberta_top1_borda_spacy
sheet['K5'] = average_diedIn_roberta_top5_borda_spacy
sheet['K6'] = average_diedIn_roberta_top10_borda_spacy

## Prompts Originais
### Normal
sheet['H4'] = average_diedIn_roberta_top1
sheet['H5'] = average_diedIn_roberta_top5
sheet['H6'] = average_diedIn_roberta_top10
### Spacy
sheet['I4'] = average_diedIn_roberta_top1_spacy
sheet['I5'] = average_diedIn_roberta_top5_spacy
sheet['I6'] = average_diedIn_roberta_top10_spacy

#Works For
## Ranking
### Normal
sheet['R4'] = average_worksfor_roberta_top1_ranking_normal
sheet['R5'] = average_worksfor_roberta_top5_ranking_normal
sheet['R6'] = average_worksfor_roberta_top10_ranking_normal
### Spacy
sheet['S4'] = average_worksfor_roberta_top1_ranking_spacy
sheet['S5'] = average_worksfor_roberta_top5_ranking_spacy
sheet['S6'] = average_worksfor_roberta_top10_ranking_spacy

## Borda Count
### Normal
sheet['P4'] = average_worksfor_roberta_top1_borda_normal
sheet['P5'] = average_worksfor_roberta_top5_borda_normal
sheet['P6'] = average_worksfor_roberta_top10_borda_normal
### Spacy
sheet['Q4'] = average_worksfor_roberta_top1_borda_spacy
sheet['Q5'] = average_worksfor_roberta_top5_borda_spacy
sheet['Q6'] = average_worksfor_roberta_top10_borda_spacy

## Prompts Originais
### Normal
sheet['N4'] = average_worksfor_roberta_top1
sheet['N5'] = average_worksfor_roberta_top5
sheet['N6'] = average_worksfor_roberta_top10
### Spacy
sheet['O4'] = average_worksfor_roberta_top1_spacy
sheet['O5'] = average_worksfor_roberta_top5_spacy
sheet['O6'] = average_worksfor_roberta_top10_spacy

#Capital
## Ranking
### Normal
sheet['F12'] = average_capital_roberta_top1_ranking_normal
sheet['F13'] = average_capital_roberta_top5_ranking_normal
sheet['F14'] = average_capital_roberta_top10_ranking_normal
### Spacy
sheet['G12'] = average_capital_roberta_top1_ranking_spacy
sheet['G13'] = average_capital_roberta_top5_ranking_spacy
sheet['G14'] = average_capital_roberta_top10_ranking_spacy

## Borda Count
### Normal
sheet['D12'] = average_capital_roberta_top1_borda_normal
sheet['D13'] = average_capital_roberta_top5_borda_normal
sheet['D14'] = average_capital_roberta_top10_borda_normal
### Spacy
sheet['E12'] = average_capital_roberta_top1_borda_spacy
sheet['E13'] = average_capital_roberta_top5_borda_spacy
sheet['E14'] = average_capital_roberta_top10_borda_spacy

## Prompts Originais
### Normal
sheet['B12'] = average_capital_roberta_top1
sheet['B13'] = average_capital_roberta_top5
sheet['B14'] = average_capital_roberta_top10
### Spacy
sheet['C12'] = average_capital_roberta_top1_spacy
sheet['C13'] = average_capital_roberta_top5_spacy
sheet['C14'] = average_capital_roberta_top10_spacy

#Citizen
## Ranking
### Normal
sheet['L12'] = average_citizen_roberta_top1_ranking_normal
sheet['L13'] = average_citizen_roberta_top5_ranking_normal
sheet['L14'] = average_citizen_roberta_top10_ranking_normal
### Spacy
sheet['M12'] = average_citizen_roberta_top1_ranking_spacy
sheet['M13'] = average_citizen_roberta_top5_ranking_spacy
sheet['M14'] = average_citizen_roberta_top10_ranking_spacy

## Borda Count
### Normal
sheet['J12'] = average_citizen_roberta_top1_borda_normal
sheet['J13'] = average_citizen_roberta_top5_borda_normal
sheet['J14'] = average_citizen_roberta_top10_borda_normal
### Spacy
sheet['K12'] = average_citizen_roberta_top1_borda_spacy
sheet['K13'] = average_citizen_roberta_top5_borda_spacy
sheet['K14'] = average_citizen_roberta_top10_borda_spacy

## Prompts Originais
### Normal
sheet['H12'] = average_citizen_roberta_top1
sheet['H13'] = average_citizen_roberta_top5
sheet['H14'] = average_citizen_roberta_top10
### Spacy
sheet['I12'] = average_citizen_roberta_top1_spacy
sheet['I13'] = average_citizen_roberta_top5_spacy
sheet['I14'] = average_citizen_roberta_top10_spacy


#Oficial Language
## Ranking
### Normal
sheet['R12'] = average_language_roberta_top1_ranking_normal
sheet['R13'] = average_language_roberta_top5_ranking_normal
sheet['R14'] = average_language_roberta_top10_ranking_normal
### Spacy
sheet['S12'] = average_language_roberta_top1_ranking_spacy
sheet['S13'] = average_language_roberta_top5_ranking_spacy
sheet['S14'] = average_language_roberta_top10_ranking_spacy

## Borda Count
### Normal
sheet['P12'] = average_language_roberta_top1_borda_normal
sheet['P13'] = average_language_roberta_top5_borda_normal
sheet['P14'] = average_language_roberta_top10_borda_normal
### Spacy
sheet['Q12'] = average_language_roberta_top1_borda_spacy
sheet['Q13'] = average_language_roberta_top5_borda_spacy
sheet['Q14'] = average_language_roberta_top10_borda_spacy

# Save the changes
wb.save('/home/rafael/tese/Slide/testes_2.xlsx')

### Electra

In [ ]:
# Select the active worksheet
sheet = wb['electra']

# Insert data into specific cells
#Born In
## Ranking
### Normal
sheet['F4'] = average_bornIn_electra_top1_ranking_normal
sheet['F5'] = average_bornIn_electra_top5_ranking_normal
sheet['F6'] = average_bornIn_electra_top10_ranking_normal
### Spacy
sheet['G4'] = average_bornIn_electra_top1_ranking_spacy
sheet['G5'] = average_bornIn_electra_top5_ranking_spacy
sheet['G6'] = average_bornIn_electra_top10_ranking_spacy

## Borda Count
### Normal
sheet['D4'] = average_bornIn_electra_top1_borda_normal
sheet['D5'] = average_bornIn_electra_top5_borda_normal
sheet['D6'] = average_bornIn_electra_top10_borda_normal
### Spacy
sheet['E4'] = average_bornIn_electra_top1_borda_spacy
sheet['E5'] = average_bornIn_electra_top5_borda_spacy
sheet['E6'] = average_bornIn_electra_top10_borda_spacy

## Prompts Originais
### Normal
sheet['B4'] = average_bornIn_electra_top1
sheet['B5'] = average_bornIn_electra_top5
sheet['B6'] = average_bornIn_electra_top10
### Spacy
sheet['C4'] = average_bornIn_electra_top1_spacy
sheet['C5'] = average_bornIn_electra_top5_spacy
sheet['C6'] = average_bornIn_electra_top10_spacy

#Died In
## Ranking
### Normal
sheet['I4'] = average_diedIn_electra_top1_ranking_normal
sheet['I5'] = average_diedIn_electra_top5_ranking_normal
sheet['I6'] = average_diedIn_electra_top10_ranking_normal
### Spacy
sheet['M4'] = average_diedIn_electra_top1_ranking_spacy
sheet['M5'] = average_diedIn_electra_top5_ranking_spacy
sheet['M6'] = average_diedIn_electra_top10_ranking_spacy

## Borda Count
### Normal
sheet['J4'] = average_diedIn_electra_top1_borda_normal
sheet['J5'] = average_diedIn_electra_top5_borda_normal
sheet['J6'] = average_diedIn_electra_top10_borda_normal
### Spacy
sheet['K4'] = average_diedIn_electra_top1_borda_spacy
sheet['K5'] = average_diedIn_electra_top5_borda_spacy
sheet['K6'] = average_diedIn_electra_top10_borda_spacy

## Prompts Originais
### Normal
sheet['H4'] = average_diedIn_electra_top1
sheet['H5'] = average_diedIn_electra_top5
sheet['H6'] = average_diedIn_electra_top10
### Spacy
sheet['I4'] = average_diedIn_electra_top1_spacy
sheet['I5'] = average_diedIn_electra_top5_spacy
sheet['I6'] = average_diedIn_electra_top10_spacy

#Works For
## Ranking
### Normal
sheet['R4'] = average_worksfor_electra_top1_ranking_normal
sheet['R5'] = average_worksfor_electra_top5_ranking_normal
sheet['R6'] = average_worksfor_electra_top10_ranking_normal
### Spacy
sheet['S4'] = average_worksfor_electra_top1_ranking_spacy
sheet['S5'] = average_worksfor_electra_top5_ranking_spacy
sheet['S6'] = average_worksfor_electra_top10_ranking_spacy

## Borda Count
### Normal
sheet['P4'] = average_worksfor_electra_top1_borda_normal
sheet['P5'] = average_worksfor_electra_top5_borda_normal
sheet['P6'] = average_worksfor_electra_top10_borda_normal
### Spacy
sheet['Q4'] = average_worksfor_electra_top1_borda_spacy
sheet['Q5'] = average_worksfor_electra_top5_borda_spacy
sheet['Q6'] = average_worksfor_electra_top10_borda_spacy

## Prompts Originais
### Normal
sheet['N4'] = average_worksfor_electra_top1
sheet['N5'] = average_worksfor_electra_top5
sheet['N6'] = average_worksfor_electra_top10
### Spacy
sheet['O4'] = average_worksfor_electra_top1_spacy
sheet['O5'] = average_worksfor_electra_top5_spacy
sheet['O6'] = average_worksfor_electra_top10_spacy

#Capital
## Ranking
### Normal
sheet['F12'] = average_capital_electra_top1_ranking_normal
sheet['F13'] = average_capital_electra_top5_ranking_normal
sheet['F14'] = average_capital_electra_top10_ranking_normal
### Spacy
sheet['G12'] = average_capital_electra_top1_ranking_spacy
sheet['G13'] = average_capital_electra_top5_ranking_spacy
sheet['G14'] = average_capital_electra_top10_ranking_spacy

## Borda Count
### Normal
sheet['D12'] = average_capital_electra_top1_borda_normal
sheet['D13'] = average_capital_electra_top5_borda_normal
sheet['D14'] = average_capital_electra_top10_borda_normal
### Spacy
sheet['E12'] = average_capital_electra_top1_borda_spacy
sheet['E13'] = average_capital_electra_top5_borda_spacy
sheet['E14'] = average_capital_electra_top10_borda_spacy

## Prompts Originais
### Normal
sheet['B12'] = average_capital_electra_top1
sheet['B13'] = average_capital_electra_top5
sheet['B14'] = average_capital_electra_top10
### Spacy
sheet['C12'] = average_capital_electra_top1_spacy
sheet['C13'] = average_capital_electra_top5_spacy
sheet['C14'] = average_capital_electra_top10_spacy

#Citizen
## Ranking
### Normal
sheet['L12'] = average_citizen_electra_top1_ranking_normal
sheet['L13'] = average_citizen_electra_top5_ranking_normal
sheet['L14'] = average_citizen_electra_top10_ranking_normal
### Spacy
sheet['M12'] = average_citizen_electra_top1_ranking_spacy
sheet['M13'] = average_citizen_electra_top5_ranking_spacy
sheet['M14'] = average_citizen_electra_top10_ranking_spacy

## Borda Count
### Normal
sheet['J12'] = average_citizen_electra_top1_borda_normal
sheet['J13'] = average_citizen_electra_top5_borda_normal
sheet['J14'] = average_citizen_electra_top10_borda_normal
### Spacy
sheet['K12'] = average_citizen_electra_top1_borda_spacy
sheet['K13'] = average_citizen_electra_top5_borda_spacy
sheet['K14'] = average_citizen_electra_top10_borda_spacy

## Prompts Originais
### Normal
sheet['H12'] = average_citizen_electra_top1
sheet['H13'] = average_citizen_electra_top5
sheet['H14'] = average_citizen_electra_top10
### Spacy
sheet['I12'] = average_citizen_electra_top1_spacy
sheet['I13'] = average_citizen_electra_top5_spacy
sheet['I14'] = average_citizen_electra_top10_spacy


#Oficial Language
## Ranking
### Normal
sheet['R12'] = average_language_electra_top1_ranking_normal
sheet['R13'] = average_language_electra_top5_ranking_normal
sheet['R14'] = average_language_electra_top10_ranking_normal
### Spacy
sheet['S12'] = average_language_electra_top1_ranking_spacy
sheet['S13'] = average_language_electra_top5_ranking_spacy
sheet['S14'] = average_language_electra_top10_ranking_spacy

## Borda Count
### Normal
sheet['P12'] = average_language_electra_top1_borda_normal
sheet['P13'] = average_language_electra_top5_borda_normal
sheet['P14'] = average_language_electra_top10_borda_normal
### Spacy
sheet['Q12'] = average_language_electra_top1_borda_spacy
sheet['Q13'] = average_language_electra_top5_borda_spacy
sheet['Q14'] = average_language_electra_top10_borda_spacy

# Save the changes
wb.save('/home/rafael/tese/Slide/testes_2.xlsx')

### Merged Models

In [ ]:
'''# Open the existing Excel file
wb = xw.Book('/home/rafael/tese/Slide/testes_2.xlsx')

# Select the active worksheet
sheet = wb.sheets['merge_models']

# Insert data into specific cells
#Born In
## Ranking
### Normal
sheet.range('B4').value = average_bornIn_merged_model_top1_ranking_normal
sheet.range('B5').value = average_bornIn_merged_model_top1_ranking_normal
sheet.range('B6').value = average_bornIn_merged_model_top1_ranking_normal
### Spacy
sheet.range('C4').value = average_bornIn_merged_model_top1_ranking_spacy
sheet.range('C5').value = average_bornIn_merged_model_top1_ranking_spacy
sheet.range('C6').value = average_bornIn_merged_model_top1_ranking_spacy

## Borda Count
### Normal
sheet.range('D4').value = average_bornIn_merged_model_top1_borda_normal
sheet.range('D5').value = average_bornIn_merged_model_top1_borda_normal
sheet.range('D6').value = average_bornIn_merged_model_top1_borda_normal
### Spacy
sheet.range('E4').value = average_bornIn_merged_model_top1_borda_spacy
sheet.range('E5').value = average_bornIn_merged_model_top1_borda_spacy
sheet.range('E6').value = average_bornIn_merged_model_top1_borda_spacy

## Prompts Originais
### Bert
#### Normal
sheet.range('F4').value = average_bornIn_bert_top1
sheet.range('F5').value = average_bornIn_bert_top5
sheet.range('F6').value = average_bornIn_bert_top10
#### Spacy
sheet.range('G4').value = average_bornIn_bert_top1_spacy
sheet.range('G5').value = average_bornIn_bert_top5_spacy
sheet.range('G6').value = average_bornIn_bert_top10_spacy

### Roberta
#### Normal
sheet.range('H4').value = average_bornIn_roberta_top1
sheet.range('H5').value = average_bornIn_roberta_top5
sheet.range('H6').value = average_bornIn_roberta_top10
#### Spacy
sheet.range('I4').value = average_bornIn_roberta_top1_spacy
sheet.range('I5').value = average_bornIn_roberta_top5_spacy
sheet.range('I6').value = average_bornIn_roberta_top10_spacy

### Electra
#### Normal
sheet.range('J4').value = average_bornIn_electra_top1
sheet.range('J5').value = average_bornIn_electra_top5
sheet.range('J6').value = average_bornIn_electra_top10
#### Spacy
sheet.range('K4').value = average_bornIn_electra_top1_spacy
sheet.range('K5').value = average_bornIn_electra_top5_spacy
sheet.range('K6').value = average_bornIn_electra_top10_spacy

#Died In
## Ranking
### Normal
sheet.range('B11').value = average_diedIn_merged_model_top1_ranking_normal
sheet.range('B12').value = average_diedIn_merged_model_top1_ranking_normal
sheet.range('B13').value = average_diedIn_merged_model_top1_ranking_normal
### Spacy
sheet.range('C11').value = average_diedIn_merged_model_top1_ranking_spacy
sheet.range('C12').value = average_diedIn_merged_model_top1_ranking_spacy
sheet.range('C13').value = average_diedIn_merged_model_top1_ranking_spacy

## Borda Count
### Normal
sheet.range('D11').value = average_diedIn_merged_model_top1_borda_normal
sheet.range('D12').value = average_diedIn_merged_model_top1_borda_normal
sheet.range('D13').value = average_diedIn_merged_model_top1_borda_normal
### Spacy
sheet.range('E11').value = average_diedIn_merged_model_top1_borda_spacy
sheet.range('E5').value = average_diedIn_merged_model_top1_borda_spacy
sheet.range('E13').value = average_diedIn_merged_model_top1_borda_spacy

## Prompts Originais
### Bert
#### Normal
sheet.range('F11').value = average_diedIn_bert_top1
sheet.range('F12').value = average_diedIn_bert_top5
sheet.range('F13').value = average_diedIn_bert_top10
#### Spacy
sheet.range('G11').value = average_diedIn_bert_top1_spacy
sheet.range('G12').value = average_diedIn_bert_top5_spacy
sheet.range('G13').value = average_diedIn_bert_top10_spacy

### Roberta
#### Normal
sheet.range('H11').value = average_diedIn_roberta_top1
sheet.range('H12').value = average_diedIn_roberta_top5
sheet.range('H13').value = average_diedIn_roberta_top10
#### Spacy
sheet.range('I11').value = average_diedIn_roberta_top1_spacy
sheet.range('I12').value = average_diedIn_roberta_top5_spacy
sheet.range('I13').value = average_diedIn_roberta_top10_spacy

### Electra
#### Normal
sheet.range('J11').value = average_diedIn_electra_top1
sheet.range('J12').value = average_diedIn_electra_top5
sheet.range('J13').value = average_diedIn_electra_top10
#### Spacy
sheet.range('K11').value = average_diedIn_electra_top1_spacy
sheet.range('K12').value = average_diedIn_electra_top5_spacy
sheet.range('K13').value = average_diedIn_electra_top10_spacy


#Works For
## Ranking
### Normal
sheet.range('B18').value = average_worksfor_merged_model_top1_ranking_normal
sheet.range('B19').value = average_worksfor_merged_model_top1_ranking_normal
sheet.range('B6').value = average_worksfor_merged_model_top1_ranking_normal
### Spacy
sheet.range('C18').value = average_worksfor_merged_model_top1_ranking_spacy
sheet.range('C19').value = average_worksfor_merged_model_top1_ranking_spacy
sheet.range('C20').value = average_worksfor_merged_model_top1_ranking_spacy

## Borda Count
### Normal
sheet.range('D18').value = average_worksfor_merged_model_top1_borda_normal
sheet.range('D19').value = average_worksfor_merged_model_top1_borda_normal
sheet.range('D20').value = average_worksfor_merged_model_top1_borda_normal
### Spacy
sheet.range('E18').value = average_worksfor_merged_model_top1_borda_spacy
sheet.range('E19').value = average_worksfor_merged_model_top1_borda_spacy
sheet.range('E20').value = average_worksfor_merged_model_top1_borda_spacy

## Prompts Originais
### Bert
#### Normal
sheet.range('F18').value = average_worksfor_bert_top1
sheet.range('F19').value = average_worksfor_bert_top5
sheet.range('F20').value = average_worksfor_bert_top10
#### Spacy
sheet.range('G18').value = average_worksfor_bert_top1_spacy
sheet.range('G19').value = average_worksfor_bert_top5_spacy
sheet.range('G20').value = average_worksfor_bert_top10_spacy

### Roberta
#### Normal
sheet.range('H18').value = average_worksfor_roberta_top1
sheet.range('H19').value = average_worksfor_roberta_top5
sheet.range('H20').value = average_worksfor_roberta_top10
#### Spacy
sheet.range('I18').value = average_worksfor_roberta_top1_spacy
sheet.range('I19').value = average_worksfor_roberta_top5_spacy
sheet.range('I20').value = average_worksfor_roberta_top10_spacy

### Electra
#### Normal
sheet.range('J18').value = average_worksfor_electra_top1
sheet.range('J19').value = average_worksfor_electra_top5
sheet.range('J20').value = average_worksfor_electra_top10
#### Spacy
sheet.range('K18').value = average_worksfor_electra_top1_spacy
sheet.range('K19').value = average_worksfor_electra_top5_spacy
sheet.range('K20').value = average_worksfor_electra_top10_spacy

#Capital
## Ranking
### Normal
sheet.range('B25').value = average_capital_merged_model_top1_ranking_normal
sheet.range('B26').value = average_capital_merged_model_top1_ranking_normal
sheet.range('B27').value = average_capital_merged_model_top1_ranking_normal
### Spacy
sheet.range('C25').value = average_capital_merged_model_top1_ranking_spacy
sheet.range('C26').value = average_capital_merged_model_top1_ranking_spacy
sheet.range('C27').value = average_capital_merged_model_top1_ranking_spacy

## Borda Count
### Normal
sheet.range('D25').value = average_capital_merged_model_top1_borda_normal
sheet.range('D26').value = average_capital_merged_model_top1_borda_normal
sheet.range('D6').value = average_capital_merged_model_top1_borda_normal
### Spacy
sheet.range('E25').value = average_capital_merged_model_top1_borda_spacy
sheet.range('E26').value = average_capital_merged_model_top1_borda_spacy
sheet.range('E27').value = average_capital_merged_model_top1_borda_spacy

## Prompts Originais
### Bert
#### Normal
sheet.range('F25').value = average_capital_bert_top1
sheet.range('F26').value = average_capital_bert_top5
sheet.range('F27').value = average_capital_bert_top10
#### Spacy
sheet.range('G25').value = average_capital_bert_top1_spacy
sheet.range('G26').value = average_capital_bert_top5_spacy
sheet.range('G27').value = average_capital_bert_top10_spacy

### Roberta
#### Normal
sheet.range('H25').value = average_capital_roberta_top1
sheet.range('H26').value = average_capital_roberta_top5
sheet.range('H27').value = average_capital_roberta_top10
#### Spacy
sheet.range('I25').value = average_capital_roberta_top1_spacy
sheet.range('I26').value = average_capital_roberta_top5_spacy
sheet.range('I27').value = average_capital_roberta_top10_spacy

### Electra
#### Normal
sheet.range('J25').value = average_capital_electra_top1
sheet.range('J26').value = average_capital_electra_top5
sheet.range('J27').value = average_capital_electra_top10
#### Spacy
sheet.range('K25').value = average_capital_electra_top1_spacy
sheet.range('K26').value = average_capital_electra_top5_spacy
sheet.range('K27').value = average_capital_electra_top10_spacy

#Citizen
## Ranking
### Normal
sheet.range('B32').value = average_citizen_merged_model_top1_ranking_normal
sheet.range('B33').value = average_citizen_merged_model_top1_ranking_normal
sheet.range('B34').value = average_citizen_merged_model_top1_ranking_normal
### Spacy
sheet.range('C32').value = average_citizen_merged_model_top1_ranking_spacy
sheet.range('C33').value = average_citizen_merged_model_top1_ranking_spacy
sheet.range('C34').value = average_citizen_merged_model_top1_ranking_spacy

## Borda Count
### Normal
sheet.range('D32').value = average_citizen_merged_model_top1_borda_normal
sheet.range('D33').value = average_citizen_merged_model_top1_borda_normal
sheet.range('D34').value = average_citizen_merged_model_top1_borda_normal
### Spacy
sheet.range('E32').value = average_citizen_merged_model_top1_borda_spacy
sheet.range('E33').value = average_citizen_merged_model_top1_borda_spacy
sheet.range('E34').value = average_citizen_merged_model_top1_borda_spacy

## Prompts Originais
### Bert
#### Normal
sheet.range('F32').value = average_capital_bert_top1
sheet.range('F33').value = average_capital_bert_top5
sheet.range('F27').value = average_capital_bert_top10
#### Spacy
sheet.range('G32').value = average_capital_bert_top1_spacy
sheet.range('G33').value = average_capital_bert_top5_spacy
sheet.range('G34').value = average_capital_bert_top10_spacy

### Roberta
#### Normal
sheet.range('H32').value = average_capital_roberta_top1
sheet.range('H33').value = average_capital_roberta_top5
sheet.range('H34').value = average_capital_roberta_top10
#### Spacy
sheet.range('I32').value = average_capital_roberta_top1_spacy
sheet.range('I33').value = average_capital_roberta_top5_spacy
sheet.range('I34').value = average_capital_roberta_top10_spacy

### Electra
#### Normal
sheet.range('J32').value = average_capital_electra_top1
sheet.range('J33').value = average_capital_electra_top5
sheet.range('J34').value = average_capital_electra_top10
#### Spacy
sheet.range('K32').value = average_capital_electra_top1_spacy
sheet.range('K33').value = average_capital_electra_top5_spacy
sheet.range('K34').value = average_capital_electra_top10_spacy

# Save the changes
wb.save('/home/rafael/tese/Slide/testes_2.xlsx')'''

"# Open the existing Excel file\nwb = xw.Book('/home/rafael/tese/Slide/testes_2.xlsx')\n\n# Select the active worksheet\nsheet = wb.sheets['merge_models']\n\n# Insert data into specific cells\n#Born In\n## Ranking\n### Normal\nsheet.range('B4').value = average_bornIn_merged_model_top1_ranking_normal\nsheet.range('B5').value = average_bornIn_merged_model_top1_ranking_normal\nsheet.range('B6').value = average_bornIn_merged_model_top1_ranking_normal\n### Spacy\nsheet.range('C4').value = average_bornIn_merged_model_top1_ranking_spacy\nsheet.range('C5').value = average_bornIn_merged_model_top1_ranking_spacy\nsheet.range('C6').value = average_bornIn_merged_model_top1_ranking_spacy\n\n## Borda Count\n### Normal\nsheet.range('D4').value = average_bornIn_merged_model_top1_borda_normal\nsheet.range('D5').value = average_bornIn_merged_model_top1_borda_normal\nsheet.range('D6').value = average_bornIn_merged_model_top1_borda_normal\n### Spacy\nsheet.range('E4').value = average_bornIn_merged_model_t

## One Shot

### Bert

In [ ]:
# Select the active worksheet
sheet = wb['bert_one_shot']

# Insert data into specific cells
#Born In
## Ranking
### Normal
sheet['F4'] = average_bornIn_bert_top1_ranking_normal_one
sheet['F5'] = average_bornIn_bert_top5_ranking_normal_one
sheet['F6'] = average_bornIn_bert_top10_ranking_normal_one
### Spacy
sheet['G4'] = average_bornIn_bert_top1_ranking_spacy_one
sheet['G5'] = average_bornIn_bert_top5_ranking_spacy_one
sheet['G6'] = average_bornIn_bert_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['D4'] = average_bornIn_bert_top1_borda_normal_one
sheet['D5'] = average_bornIn_bert_top5_borda_normal_one
sheet['D6'] = average_bornIn_bert_top10_borda_normal_one
### Spacy
sheet['E4'] = average_bornIn_bert_top1_borda_spacy_one
sheet['E5'] = average_bornIn_bert_top5_borda_spacy_one
sheet['E6'] = average_bornIn_bert_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['B4'] = average_bornIn_bert_top1_one
sheet['B5'] = average_bornIn_bert_top5_one
sheet['B6'] = average_bornIn_bert_top10_one
### Spacy
sheet['C4'] = average_bornIn_bert_top1_spacy_one
sheet['C5'] = average_bornIn_bert_top5_spacy_one
sheet['C6'] = average_bornIn_bert_top10_spacy_one

#Died In
## Ranking
### Normal
sheet['L4'] = average_diedIn_bert_top1_ranking_normal_one
sheet['L5'] = average_diedIn_bert_top5_ranking_normal_one
sheet['L6'] = average_diedIn_bert_top10_ranking_normal_one
### Spacy
sheet['M4'] = average_diedIn_bert_top1_ranking_spacy_one
sheet['M5'] = average_diedIn_bert_top5_ranking_spacy_one
sheet['M6'] = average_diedIn_bert_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['J4'] = average_diedIn_bert_top1_borda_normal_one
sheet['J5'] = average_diedIn_bert_top5_borda_normal_one
sheet['J6'] = average_diedIn_bert_top10_borda_normal_one
### Spacy
sheet['K4'] = average_diedIn_bert_top1_borda_spacy_one
sheet['K5'] = average_diedIn_bert_top5_borda_spacy_one
sheet['K6'] = average_diedIn_bert_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['H4'] = average_diedIn_bert_top1_one
sheet['H5'] = average_diedIn_bert_top5_one
sheet['H6'] = average_diedIn_bert_top10_one
### Spacy
sheet['I4'] = average_diedIn_bert_top1_spacy_one
sheet['I5'] = average_diedIn_bert_top5_spacy_one
sheet['I6'] = average_diedIn_bert_top10_spacy_one

#Works For
## Ranking
### Normal
sheet['R4'] = average_worksfor_bert_top1_ranking_normal_one
sheet['R5'] = average_worksfor_bert_top5_ranking_normal_one
sheet['R6'] = average_worksfor_bert_top10_ranking_normal_one
### Spacy
sheet['S4'] = average_worksfor_bert_top1_ranking_spacy_one
sheet['S5'] = average_worksfor_bert_top5_ranking_spacy_one
sheet['S6'] = average_worksfor_bert_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['P4'] = average_worksfor_bert_top1_borda_normal_one
sheet['P5'] = average_worksfor_bert_top5_borda_normal_one
sheet['P6'] = average_worksfor_bert_top10_borda_normal_one
### Spacy
sheet['Q4'] = average_worksfor_bert_top1_borda_spacy_one
sheet['Q5'] = average_worksfor_bert_top5_borda_spacy_one
sheet['Q6'] = average_worksfor_bert_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['N4'] = average_worksfor_bert_top1_one
sheet['N5'] = average_worksfor_bert_top5_one
sheet['N6'] = average_worksfor_bert_top10_one
### Spacy
sheet['O4'] = average_worksfor_bert_top1_spacy_one
sheet['O5'] = average_worksfor_bert_top5_spacy_one
sheet['O6'] = average_worksfor_bert_top10_spacy_one

#Capital
## Ranking
### Normal
sheet['F12'] = average_capital_bert_top1_ranking_normal_one
sheet['F13'] = average_capital_bert_top5_ranking_normal_one
sheet['F14'] = average_capital_bert_top10_ranking_normal_one
### Spacy
sheet['G12'] = average_capital_bert_top1_ranking_spacy_one
sheet['G13'] = average_capital_bert_top5_ranking_spacy_one
sheet['G14'] = average_capital_bert_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['D12'] = average_capital_bert_top1_borda_normal_one
sheet['D13'] = average_capital_bert_top5_borda_normal_one
sheet['D14'] = average_capital_bert_top10_borda_normal_one
### Spacy
sheet['E12'] = average_capital_bert_top1_borda_spacy_one
sheet['E13'] = average_capital_bert_top5_borda_spacy_one
sheet['E14'] = average_capital_bert_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['B12'] = average_capital_bert_top1_one
sheet['B13'] = average_capital_bert_top5_one
sheet['B14'] = average_capital_bert_top10_one
### Spacy
sheet['C12'] = average_capital_bert_top1_spacy_one
sheet['C13'] = average_capital_bert_top5_spacy_one
sheet['C14'] = average_capital_bert_top10_spacy_one

#Citizen
## Ranking
### Normal
sheet['L12'] = average_citizen_bert_top1_ranking_normal_one
sheet['L13'] = average_citizen_bert_top5_ranking_normal_one
sheet['L14'] = average_citizen_bert_top10_ranking_normal_one
### Spacy
sheet['M12'] = average_citizen_bert_top1_ranking_spacy_one
sheet['M13'] = average_citizen_bert_top5_ranking_spacy_one
sheet['M14'] = average_citizen_bert_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['J12'] = average_citizen_bert_top1_borda_normal_one
sheet['J13'] = average_citizen_bert_top5_borda_normal_one
sheet['J14'] = average_citizen_bert_top10_borda_normal_one
### Spacy
sheet['K12'] = average_citizen_bert_top1_borda_spacy_one
sheet['K13'] = average_citizen_bert_top5_borda_spacy_one
sheet['K14'] = average_citizen_bert_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['H12'] = average_citizen_bert_top1_one
sheet['H13'] = average_citizen_bert_top5_one
sheet['H14'] = average_citizen_bert_top10_one
### Spacy
sheet['I12'] = average_citizen_bert_top1_spacy_one
sheet['I13'] = average_citizen_bert_top5_spacy_one
sheet['I14'] = average_citizen_bert_top10_spacy_one

#Oficial Language
## Ranking
### Normal
sheet['R4'] = average_language_bert_top1_ranking_normal_one
sheet['R5'] = average_language_bert_top5_ranking_normal_one
sheet['R6'] = average_language_bert_top10_ranking_normal_one
### Spacy
sheet['S4'] = average_language_bert_top1_ranking_spacy_one
sheet['S5'] = average_language_bert_top5_ranking_spacy_one
sheet['S6'] = average_language_bert_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['P4'] = average_language_bert_top1_borda_normal_one
sheet['P5'] = average_language_bert_top5_borda_normal_one
sheet['P6'] = average_language_bert_top10_borda_normal_one
### Spacy
sheet['Q4'] = average_language_bert_top1_borda_spacy_one
sheet['Q5'] = average_language_bert_top5_borda_spacy_one
sheet['Q6'] = average_language_bert_top10_borda_spacy_one


# Save the changes
wb.save('/home/rafael/tese/Slide/testes_2.xlsx')

### Roberta

In [ ]:
# Select the active worksheet
sheet = wb['roberta_one_shot']

# Insert data into specific cells
#Born In
## Ranking
### Normal
sheet['F4'] = average_bornIn_roberta_top1_ranking_normal_one
sheet['F5'] = average_bornIn_roberta_top5_ranking_normal_one
sheet['F6'] = average_bornIn_roberta_top10_ranking_normal_one
### Spacy
sheet['G4'] = average_bornIn_roberta_top1_ranking_spacy_one
sheet['G5'] = average_bornIn_roberta_top5_ranking_spacy_one
sheet['G6'] = average_bornIn_roberta_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['D4'] = average_bornIn_roberta_top1_borda_normal_one
sheet['D5'] = average_bornIn_roberta_top5_borda_normal_one
sheet['D6'] = average_bornIn_roberta_top10_borda_normal_one
### Spacy
sheet['E4'] = average_bornIn_roberta_top1_borda_spacy_one
sheet['E5'] = average_bornIn_roberta_top5_borda_spacy_one
sheet['E6'] = average_bornIn_roberta_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['B4'] = average_bornIn_roberta_top1_one
sheet['B5'] = average_bornIn_roberta_top5_one
sheet['B6'] = average_bornIn_roberta_top10_one
### Spacy
sheet['C4'] = average_bornIn_roberta_top1_spacy_one
sheet['C5'] = average_bornIn_roberta_top5_spacy_one
sheet['C6'] = average_bornIn_roberta_top10_spacy_one

#Died In
## Ranking
### Normal
sheet['L4'] = average_diedIn_roberta_top1_ranking_normal_one
sheet['L5'] = average_diedIn_roberta_top5_ranking_normal_one
sheet['L6'] = average_diedIn_roberta_top10_ranking_normal_one
### Spacy
sheet['M4'] = average_diedIn_roberta_top1_ranking_spacy_one
sheet['M5'] = average_diedIn_roberta_top5_ranking_spacy_one
sheet['M6'] = average_diedIn_roberta_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['J4'] = average_diedIn_roberta_top1_borda_normal_one
sheet['J5'] = average_diedIn_roberta_top5_borda_normal_one
sheet['J6'] = average_diedIn_roberta_top10_borda_normal_one
### Spacy
sheet['K4'] = average_diedIn_roberta_top1_borda_spacy_one
sheet['K5'] = average_diedIn_roberta_top5_borda_spacy_one
sheet['K6'] = average_diedIn_roberta_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['H4'] = average_diedIn_roberta_top1_one
sheet['H5'] = average_diedIn_roberta_top5_one
sheet['H6'] = average_diedIn_roberta_top10_one
### Spacy
sheet['I4'] = average_diedIn_roberta_top1_spacy_one
sheet['I5'] = average_diedIn_roberta_top5_spacy_one
sheet['I6'] = average_diedIn_roberta_top10_spacy_one

#Works For
## Ranking
### Normal
sheet['R4'] = average_worksfor_roberta_top1_ranking_normal_one
sheet['R5'] = average_worksfor_roberta_top5_ranking_normal_one
sheet['R6'] = average_worksfor_roberta_top10_ranking_normal_one
### Spacy
sheet['S4'] = average_worksfor_roberta_top1_ranking_spacy_one
sheet['S5'] = average_worksfor_roberta_top5_ranking_spacy_one
sheet['S6'] = average_worksfor_roberta_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['P4'] = average_worksfor_roberta_top1_borda_normal_one
sheet['P5'] = average_worksfor_roberta_top5_borda_normal_one
sheet['P6'] = average_worksfor_roberta_top10_borda_normal_one
### Spacy
sheet['Q4'] = average_worksfor_roberta_top1_borda_spacy_one
sheet['Q5'] = average_worksfor_roberta_top5_borda_spacy_one
sheet['Q6'] = average_worksfor_roberta_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['N4'] = average_worksfor_roberta_top1_one
sheet['N5'] = average_worksfor_roberta_top5_one
sheet['N6'] = average_worksfor_roberta_top10_one
### Spacy
sheet['O4'] = average_worksfor_roberta_top1_spacy_one
sheet['O5'] = average_worksfor_roberta_top5_spacy_one
sheet['O6'] = average_worksfor_roberta_top10_spacy_one

#Capital
## Ranking
### Normal
sheet['F12'] = average_capital_roberta_top1_ranking_normal_one
sheet['F13'] = average_capital_roberta_top5_ranking_normal_one
sheet['F14'] = average_capital_roberta_top10_ranking_normal_one
### Spacy
sheet['G12'] = average_capital_roberta_top1_ranking_spacy_one
sheet['G13'] = average_capital_roberta_top5_ranking_spacy_one
sheet['G14'] = average_capital_roberta_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['D12'] = average_capital_roberta_top1_borda_normal_one
sheet['D13'] = average_capital_roberta_top5_borda_normal_one
sheet['D14'] = average_capital_roberta_top10_borda_normal_one
### Spacy
sheet['E12'] = average_capital_roberta_top1_borda_spacy_one
sheet['E13'] = average_capital_roberta_top5_borda_spacy_one
sheet['E14'] = average_capital_roberta_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['B12'] = average_capital_roberta_top1_one
sheet['B13'] = average_capital_roberta_top5_one
sheet['B14'] = average_capital_roberta_top10_one
### Spacy
sheet['C12'] = average_capital_roberta_top1_spacy_one
sheet['C13'] = average_capital_roberta_top5_spacy_one
sheet['C14'] = average_capital_roberta_top10_spacy_one

#Citizen
## Ranking
### Normal
sheet['L12'] = average_citizen_roberta_top1_ranking_normal_one
sheet['L13'] = average_citizen_roberta_top5_ranking_normal_one
sheet['L14'] = average_citizen_roberta_top10_ranking_normal_one
### Spacy
sheet['M12'] = average_citizen_roberta_top1_ranking_spacy_one
sheet['M13'] = average_citizen_roberta_top5_ranking_spacy_one
sheet['M14'] = average_citizen_roberta_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['J12'] = average_citizen_roberta_top1_borda_normal_one
sheet['J13'] = average_citizen_roberta_top5_borda_normal_one
sheet['J14'] = average_citizen_roberta_top10_borda_normal_one
### Spacy
sheet['K12'] = average_citizen_roberta_top1_borda_spacy_one
sheet['K13'] = average_citizen_roberta_top5_borda_spacy_one
sheet['K14'] = average_citizen_roberta_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['H12'] = average_citizen_roberta_top1_one
sheet['H13'] = average_citizen_roberta_top5_one
sheet['H14'] = average_citizen_roberta_top10_one
### Spacy
sheet['I12'] = average_citizen_roberta_top1_spacy_one
sheet['I13'] = average_citizen_roberta_top5_spacy_one
sheet['I14'] = average_citizen_roberta_top10_spacy_one

#Oficial Language
## Ranking
### Normal
sheet['R4'] = average_language_roberta_top1_ranking_normal_one
sheet['R5'] = average_language_roberta_top5_ranking_normal_one
sheet['R6'] = average_language_roberta_top10_ranking_normal_one
### Spacy
sheet['S4'] = average_language_roberta_top1_ranking_spacy_one
sheet['S5'] = average_language_roberta_top5_ranking_spacy_one
sheet['S6'] = average_language_roberta_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['P4'] = average_language_roberta_top1_borda_normal_one
sheet['P5'] = average_language_roberta_top5_borda_normal_one
sheet['P6'] = average_language_roberta_top10_borda_normal_one
### Spacy
sheet['Q4'] = average_language_roberta_top1_borda_spacy_one
sheet['Q5'] = average_language_roberta_top5_borda_spacy_one
sheet['Q6'] = average_language_roberta_top10_borda_spacy_one


# Save the changes
wb.save('/home/rafael/tese/Slide/testes_2.xlsx')

### Electra

In [ ]:
# Select the active worksheet
sheet = wb['electra_one_shot']

# Insert data into specific cells
#Born In
## Ranking
### Normal
sheet['F4'] = average_bornIn_electra_top1_ranking_normal_one
sheet['F5'] = average_bornIn_electra_top5_ranking_normal_one
sheet['F6'] = average_bornIn_electra_top10_ranking_normal_one
### Spacy
sheet['G4'] = average_bornIn_electra_top1_ranking_spacy_one
sheet['G5'] = average_bornIn_electra_top5_ranking_spacy_one
sheet['G6'] = average_bornIn_electra_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['D4'] = average_bornIn_electra_top1_borda_normal_one
sheet['D5'] = average_bornIn_electra_top5_borda_normal_one
sheet['D6'] = average_bornIn_electra_top10_borda_normal_one
### Spacy
sheet['E4'] = average_bornIn_electra_top1_borda_spacy_one
sheet['E5'] = average_bornIn_electra_top5_borda_spacy_one
sheet['E6'] = average_bornIn_electra_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['B4'] = average_bornIn_electra_top1_one
sheet['B5'] = average_bornIn_electra_top5_one
sheet['B6'] = average_bornIn_electra_top10_one
### Spacy
sheet['C4'] = average_bornIn_electra_top1_spacy_one
sheet['C5'] = average_bornIn_electra_top5_spacy_one
sheet['C6'] = average_bornIn_electra_top10_spacy_one

#Died In
## Ranking
### Normal
sheet['L4'] = average_diedIn_electra_top1_ranking_normal_one
sheet['L5'] = average_diedIn_electra_top5_ranking_normal_one
sheet['L6'] = average_diedIn_electra_top10_ranking_normal_one
### Spacy
sheet['M4'] = average_diedIn_electra_top1_ranking_spacy_one
sheet['M5'] = average_diedIn_electra_top5_ranking_spacy_one
sheet['M6'] = average_diedIn_electra_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['J4'] = average_diedIn_electra_top1_borda_normal_one
sheet['J5'] = average_diedIn_electra_top5_borda_normal_one
sheet['J6'] = average_diedIn_electra_top10_borda_normal_one
### Spacy
sheet['K4'] = average_diedIn_electra_top1_borda_spacy_one
sheet['K5'] = average_diedIn_electra_top5_borda_spacy_one
sheet['K6'] = average_diedIn_electra_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['H4'] = average_diedIn_electra_top1_one
sheet['H5'] = average_diedIn_electra_top5_one
sheet['H6'] = average_diedIn_electra_top10_one
### Spacy
sheet['I4'] = average_diedIn_electra_top1_spacy_one
sheet['I5'] = average_diedIn_electra_top5_spacy_one
sheet['I6'] = average_diedIn_electra_top10_spacy_one

#Works For
## Ranking
### Normal
sheet['R4'] = average_worksfor_electra_top1_ranking_normal_one
sheet['R5'] = average_worksfor_electra_top5_ranking_normal_one
sheet['R6'] = average_worksfor_electra_top10_ranking_normal_one
### Spacy
sheet['S4'] = average_worksfor_electra_top1_ranking_spacy_one
sheet['S5'] = average_worksfor_electra_top5_ranking_spacy_one
sheet['S6'] = average_worksfor_electra_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['P4'] = average_worksfor_electra_top1_borda_normal_one
sheet['P5'] = average_worksfor_electra_top5_borda_normal_one
sheet['P6'] = average_worksfor_electra_top10_borda_normal_one
### Spacy
sheet['Q4'] = average_worksfor_electra_top1_borda_spacy_one
sheet['Q5'] = average_worksfor_electra_top5_borda_spacy_one
sheet['Q6'] = average_worksfor_electra_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['N4'] = average_worksfor_electra_top1_one
sheet['N5'] = average_worksfor_electra_top5_one
sheet['N6'] = average_worksfor_electra_top10_one
### Spacy
sheet['O4'] = average_worksfor_electra_top1_spacy_one
sheet['O5'] = average_worksfor_electra_top5_spacy_one
sheet['O6'] = average_worksfor_electra_top10_spacy_one

#Capital
## Ranking
### Normal
sheet['F12'] = average_capital_electra_top1_ranking_normal_one
sheet['F13'] = average_capital_electra_top5_ranking_normal_one
sheet['F14'] = average_capital_electra_top10_ranking_normal_one
### Spacy
sheet['G12'] = average_capital_electra_top1_ranking_spacy_one
sheet['G13'] = average_capital_electra_top5_ranking_spacy_one
sheet['G14'] = average_capital_electra_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['D12'] = average_capital_electra_top1_borda_normal_one
sheet['D13'] = average_capital_electra_top5_borda_normal_one
sheet['D14'] = average_capital_electra_top10_borda_normal_one
### Spacy
sheet['E12'] = average_capital_electra_top1_borda_spacy_one
sheet['E13'] = average_capital_electra_top5_borda_spacy_one
sheet['E14'] = average_capital_electra_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['B12'] = average_capital_electra_top1_one
sheet['B13'] = average_capital_electra_top5_one
sheet['B14'] = average_capital_electra_top10_one
### Spacy
sheet['C12'] = average_capital_electra_top1_spacy_one
sheet['C13'] = average_capital_electra_top5_spacy_one
sheet['C14'] = average_capital_electra_top10_spacy_one

#Citizen
## Ranking
### Normal
sheet['L12'] = average_citizen_electra_top1_ranking_normal_one
sheet['L13'] = average_citizen_electra_top5_ranking_normal_one
sheet['L14'] = average_citizen_electra_top10_ranking_normal_one
### Spacy
sheet['M12'] = average_citizen_electra_top1_ranking_spacy_one
sheet['M13'] = average_citizen_electra_top5_ranking_spacy_one
sheet['M14'] = average_citizen_electra_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['J12'] = average_citizen_electra_top1_borda_normal_one
sheet['J13'] = average_citizen_electra_top5_borda_normal_one
sheet['J14'] = average_citizen_electra_top10_borda_normal_one
### Spacy
sheet['K12'] = average_citizen_electra_top1_borda_spacy_one
sheet['K13'] = average_citizen_electra_top5_borda_spacy_one
sheet['K14'] = average_citizen_electra_top10_borda_spacy_one

## Prompts Originais
### Normal
sheet['H12'] = average_citizen_electra_top1_one
sheet['H13'] = average_citizen_electra_top5_one
sheet['H14'] = average_citizen_electra_top10_one
### Spacy
sheet['I12'] = average_citizen_electra_top1_spacy_one
sheet['I13'] = average_citizen_electra_top5_spacy_one
sheet['I14'] = average_citizen_electra_top10_spacy_one

#Oficial Language
## Ranking
### Normal
sheet['R4'] = average_language_electra_top1_ranking_normal_one
sheet['R5'] = average_language_electra_top5_ranking_normal_one
sheet['R6'] = average_language_electra_top10_ranking_normal_one
### Spacy
sheet['S4'] = average_language_electra_top1_ranking_spacy_one
sheet['S5'] = average_language_electra_top5_ranking_spacy_one
sheet['S6'] = average_language_electra_top10_ranking_spacy_one

## Borda Count
### Normal
sheet['P4'] = average_language_electra_top1_borda_normal_one
sheet['P5'] = average_language_electra_top5_borda_normal_one
sheet['P6'] = average_language_electra_top10_borda_normal_one
### Spacy
sheet['Q4'] = average_language_electra_top1_borda_spacy_one
sheet['Q5'] = average_language_electra_top5_borda_spacy_one
sheet['Q6'] = average_language_electra_top10_borda_spacy_one


# Save the changes
wb.save('/home/rafael/tese/Slide/testes_2.xlsx')

### Merged Models

In [ ]:
'''# Open the existing Excel file
wb = xw.Book('/home/rafael/tese/Slide/testes_2.xlsx')

# Select the active worksheet
sheet = wb.sheets['merge_models_one_shot']

# Insert data into specific cells
#Born In
## Ranking
### Normal
sheet.range('B4').value = average_bornIn_merged_model_top1_ranking_normal_one
sheet.range('B5').value = average_bornIn_merged_model_top1_ranking_normal_one
sheet.range('B6').value = average_bornIn_merged_model_top1_ranking_normal_one
### Spacy
sheet.range('C4').value = average_bornIn_merged_model_top1_ranking_spacy_one
sheet.range('C5').value = average_bornIn_merged_model_top1_ranking_spacy_one
sheet.range('C6').value = average_bornIn_merged_model_top1_ranking_spacy_one

## Borda Count
### Normal
sheet.range('D4').value = average_bornIn_merged_model_top1_borda_normal_one
sheet.range('D5').value = average_bornIn_merged_model_top1_borda_normal_one
sheet.range('D6').value = average_bornIn_merged_model_top1_borda_normal_one
### Spacy
sheet.range('E4').value = average_bornIn_merged_model_top1_borda_spacy_one
sheet.range('E5').value = average_bornIn_merged_model_top1_borda_spacy_one
sheet.range('E6').value = average_bornIn_merged_model_top1_borda_spacy_one

## Prompts Originais
### Bert
#### Normal
sheet.range('F4').value = average_bornIn_bert_top1_one
sheet.range('F5').value = average_bornIn_bert_top5_one
sheet.range('F6').value = average_bornIn_bert_top10_one
#### Spacy
sheet.range('G4').value = average_bornIn_bert_top1_spacy_one
sheet.range('G5').value = average_bornIn_bert_top5_spacy_one
sheet.range('G6').value = average_bornIn_bert_top10_spacy_one

### Roberta
#### Normal
sheet.range('H4').value = average_bornIn_roberta_top1_one
sheet.range('H5').value = average_bornIn_roberta_top5_one
sheet.range('H6').value = average_bornIn_roberta_top10_one
#### Spacy
sheet.range('I4').value = average_bornIn_roberta_top1_spacy_one
sheet.range('I5').value = average_bornIn_roberta_top5_spacy_one
sheet.range('I6').value = average_bornIn_roberta_top10_spacy_one

### Electra
#### Normal
sheet.range('J4').value = average_bornIn_electra_top1_one
sheet.range('J5').value = average_bornIn_electra_top5_one
sheet.range('J6').value = average_bornIn_electra_top10_one
#### Spacy
sheet.range('K4').value = average_bornIn_electra_top1_spacy_one
sheet.range('K5').value = average_bornIn_electra_top5_spacy_one
sheet.range('K6').value = average_bornIn_electra_top10_spacy_one

#Died In
## Ranking
### Normal
sheet.range('B11').value = average_diedIn_merged_model_top1_ranking_normal_one
sheet.range('B12').value = average_diedIn_merged_model_top1_ranking_normal_one
sheet.range('B13').value = average_diedIn_merged_model_top1_ranking_normal_one
### Spacy
sheet.range('C11').value = average_diedIn_merged_model_top1_ranking_spacy_one
sheet.range('C12').value = average_diedIn_merged_model_top1_ranking_spacy_one
sheet.range('C13').value = average_diedIn_merged_model_top1_ranking_spacy_one

## Borda Count
### Normal
sheet.range('D11').value = average_diedIn_merged_model_top1_borda_normal_one
sheet.range('D12').value = average_diedIn_merged_model_top1_borda_normal_one
sheet.range('D13').value = average_diedIn_merged_model_top1_borda_normal_one
### Spacy
sheet.range('E11').value = average_diedIn_merged_model_top1_borda_spacy_one
sheet.range('E5').value = average_diedIn_merged_model_top1_borda_spacy_one
sheet.range('E13').value = average_diedIn_merged_model_top1_borda_spacy_one

## Prompts Originais
### Bert
#### Normal
sheet.range('F11').value = average_diedIn_bert_top1_one
sheet.range('F12').value = average_diedIn_bert_top5_one
sheet.range('F13').value = average_diedIn_bert_top10_one
#### Spacy
sheet.range('G11').value = average_diedIn_bert_top1_spacy_one
sheet.range('G12').value = average_diedIn_bert_top5_spacy_one
sheet.range('G13').value = average_diedIn_bert_top10_spacy_one

### Roberta
#### Normal
sheet.range('H11').value = average_diedIn_roberta_top1_one
sheet.range('H12').value = average_diedIn_roberta_top5_one
sheet.range('H13').value = average_diedIn_roberta_top10_one
#### Spacy
sheet.range('I11').value = average_diedIn_roberta_top1_spacy_one
sheet.range('I12').value = average_diedIn_roberta_top5_spacy_one
sheet.range('I13').value = average_diedIn_roberta_top10_spacy_one

### Electra
#### Normal
sheet.range('J11').value = average_diedIn_electra_top1_one
sheet.range('J12').value = average_diedIn_electra_top5_one
sheet.range('J13').value = average_diedIn_electra_top10_one
#### Spacy
sheet.range('K11').value = average_diedIn_electra_top1_spacy_one
sheet.range('K12').value = average_diedIn_electra_top5_spacy_one
sheet.range('K13').value = average_diedIn_electra_top10_spacy_one


#Works For
## Ranking
### Normal
sheet.range('B18').value = average_worksfor_merged_model_top1_ranking_normal_one
sheet.range('B19').value = average_worksfor_merged_model_top1_ranking_normal_one
sheet.range('B6').value = average_worksfor_merged_model_top1_ranking_normal_one
### Spacy
sheet.range('C18').value = average_worksfor_merged_model_top1_ranking_spacy_one
sheet.range('C19').value = average_worksfor_merged_model_top1_ranking_spacy_one
sheet.range('C20').value = average_worksfor_merged_model_top1_ranking_spacy_one

## Borda Count
### Normal
sheet.range('D18').value = average_worksfor_merged_model_top1_borda_normal_one
sheet.range('D19').value = average_worksfor_merged_model_top1_borda_normal_one
sheet.range('D20').value = average_worksfor_merged_model_top1_borda_normal_one
### Spacy
sheet.range('E18').value = average_worksfor_merged_model_top1_borda_spacy_one
sheet.range('E19').value = average_worksfor_merged_model_top1_borda_spacy_one
sheet.range('E20').value = average_worksfor_merged_model_top1_borda_spacy_one

## Prompts Originais
### Bert
#### Normal
sheet.range('F18').value = average_worksfor_bert_top1_one
sheet.range('F19').value = average_worksfor_bert_top5_one
sheet.range('F20').value = average_worksfor_bert_top10_one
#### Spacy
sheet.range('G18').value = average_worksfor_bert_top1_spacy_one
sheet.range('G19').value = average_worksfor_bert_top5_spacy_one
sheet.range('G20').value = average_worksfor_bert_top10_spacy_one

### Roberta
#### Normal
sheet.range('H18').value = average_worksfor_roberta_top1_one
sheet.range('H19').value = average_worksfor_roberta_top5_one
sheet.range('H20').value = average_worksfor_roberta_top10_one
#### Spacy
sheet.range('I18').value = average_worksfor_roberta_top1_spacy_one
sheet.range('I19').value = average_worksfor_roberta_top5_spacy_one
sheet.range('I20').value = average_worksfor_roberta_top10_spacy_one

### Electra
#### Normal
sheet.range('J18').value = average_worksfor_electra_top1_one
sheet.range('J19').value = average_worksfor_electra_top5_one
sheet.range('J20').value = average_worksfor_electra_top10_one
#### Spacy
sheet.range('K18').value = average_worksfor_electra_top1_spacy_one
sheet.range('K19').value = average_worksfor_electra_top5_spacy_one
sheet.range('K20').value = average_worksfor_electra_top10_spacy_one

#Capital
## Ranking
### Normal
sheet.range('B25').value = average_capital_merged_model_top1_ranking_normal_one
sheet.range('B26').value = average_capital_merged_model_top1_ranking_normal_one
sheet.range('B27').value = average_capital_merged_model_top1_ranking_normal_one
### Spacy
sheet.range('C25').value = average_capital_merged_model_top1_ranking_spacy_one
sheet.range('C26').value = average_capital_merged_model_top1_ranking_spacy_one
sheet.range('C27').value = average_capital_merged_model_top1_ranking_spacy_one

## Borda Count
### Normal
sheet.range('D25').value = average_capital_merged_model_top1_borda_normal_one
sheet.range('D26').value = average_capital_merged_model_top1_borda_normal_one
sheet.range('D6').value = average_capital_merged_model_top1_borda_normal_one
### Spacy
sheet.range('E25').value = average_capital_merged_model_top1_borda_spacy_one
sheet.range('E26').value = average_capital_merged_model_top1_borda_spacy_one
sheet.range('E27').value = average_capital_merged_model_top1_borda_spacy_one

## Prompts Originais
### Bert
#### Normal
sheet.range('F25').value = average_capital_bert_top1_one
sheet.range('F26').value = average_capital_bert_top5_one
sheet.range('F27').value = average_capital_bert_top10_one
#### Spacy
sheet.range('G25').value = average_capital_bert_top1_spacy_one
sheet.range('G26').value = average_capital_bert_top5_spacy_one
sheet.range('G27').value = average_capital_bert_top10_spacy_one

### Roberta
#### Normal
sheet.range('H25').value = average_capital_roberta_top1_one
sheet.range('H26').value = average_capital_roberta_top5_one
sheet.range('H27').value = average_capital_roberta_top10_one
#### Spacy
sheet.range('I25').value = average_capital_roberta_top1_spacy_one
sheet.range('I26').value = average_capital_roberta_top5_spacy_one
sheet.range('I27').value = average_capital_roberta_top10_spacy_one

### Electra
#### Normal
sheet.range('J25').value = average_capital_electra_top1_one
sheet.range('J26').value = average_capital_electra_top5_one
sheet.range('J27').value = average_capital_electra_top10_one
#### Spacy
sheet.range('K25').value = average_capital_electra_top1_spacy_one
sheet.range('K26').value = average_capital_electra_top5_spacy_one
sheet.range('K27').value = average_capital_electra_top10_spacy_one

#Citizen
## Ranking
### Normal
sheet.range('B32').value = average_citizen_merged_model_top1_ranking_normal_one
sheet.range('B33').value = average_citizen_merged_model_top1_ranking_normal_one
sheet.range('B34').value = average_citizen_merged_model_top1_ranking_normal_one
### Spacy
sheet.range('C32').value = average_citizen_merged_model_top1_ranking_spacy_one
sheet.range('C33').value = average_citizen_merged_model_top1_ranking_spacy_one
sheet.range('C34').value = average_citizen_merged_model_top1_ranking_spacy_one

## Borda Count
### Normal
sheet.range('D32').value = average_citizen_merged_model_top1_borda_normal_one
sheet.range('D33').value = average_citizen_merged_model_top1_borda_normal_one
sheet.range('D34').value = average_citizen_merged_model_top1_borda_normal_one
### Spacy
sheet.range('E32').value = average_citizen_merged_model_top1_borda_spacy_one
sheet.range('E33').value = average_citizen_merged_model_top1_borda_spacy_one
sheet.range('E34').value = average_citizen_merged_model_top1_borda_spacy_one

## Prompts Originais
### Bert
#### Normal
sheet.range('F32').value = average_capital_bert_top1_one
sheet.range('F33').value = average_capital_bert_top5_one
sheet.range('F27').value = average_capital_bert_top10_one
#### Spacy
sheet.range('G32').value = average_capital_bert_top1_spacy_one
sheet.range('G33').value = average_capital_bert_top5_spacy_one
sheet.range('G34').value = average_capital_bert_top10_spacy_one

### Roberta
#### Normal
sheet.range('H32').value = average_capital_roberta_top1_one
sheet.range('H33').value = average_capital_roberta_top5_one
sheet.range('H34').value = average_capital_roberta_top10_one
#### Spacy
sheet.range('I32').value = average_capital_roberta_top1_spacy_one
sheet.range('I33').value = average_capital_roberta_top5_spacy_one
sheet.range('I34').value = average_capital_roberta_top10_spacy_one

### Electra
#### Normal
sheet.range('J32').value = average_capital_electra_top1_one
sheet.range('J33').value = average_capital_electra_top5_one
sheet.range('J34').value = average_capital_electra_top10_one
#### Spacy
sheet.range('K32').value = average_capital_electra_top1_spacy_one
sheet.range('K33').value = average_capital_electra_top5_spacy_one
sheet.range('K34').value = average_capital_electra_top10_spacy_one

# Save the changes
wb.save('/home/rafael/tese/Slide/testes_2.xlsx')'''

"# Open the existing Excel file\nwb = xw.Book('/home/rafael/tese/Slide/testes_2.xlsx')\n\n# Select the active worksheet\nsheet = wb.sheets['merge_models_one_shot']\n\n# Insert data into specific cells\n#Born In\n## Ranking\n### Normal\nsheet.range('B4').value = average_bornIn_merged_model_top1_ranking_normal_one\nsheet.range('B5').value = average_bornIn_merged_model_top1_ranking_normal_one\nsheet.range('B6').value = average_bornIn_merged_model_top1_ranking_normal_one\n### Spacy\nsheet.range('C4').value = average_bornIn_merged_model_top1_ranking_spacy_one\nsheet.range('C5').value = average_bornIn_merged_model_top1_ranking_spacy_one\nsheet.range('C6').value = average_bornIn_merged_model_top1_ranking_spacy_one\n\n## Borda Count\n### Normal\nsheet.range('D4').value = average_bornIn_merged_model_top1_borda_normal_one\nsheet.range('D5').value = average_bornIn_merged_model_top1_borda_normal_one\nsheet.range('D6').value = average_bornIn_merged_model_top1_borda_normal_one\n### Spacy\nsheet.rang

## Few Shoot

### Bert

In [ ]:
# Select the active worksheet
sheet = wb['bert_few_shot']

# Insert data into specific cells
#Born In
## Ranking
### Normal
sheet['F4'] = average_bornIn_bert_top1_ranking_normal_few
sheet['F5'] = average_bornIn_bert_top5_ranking_normal_few
sheet['F6'] = average_bornIn_bert_top10_ranking_normal_few
### Spacy
sheet['G4'] = average_bornIn_bert_top1_ranking_spacy_few
sheet['G5'] = average_bornIn_bert_top5_ranking_spacy_few
sheet['G6'] = average_bornIn_bert_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['D4'] = average_bornIn_bert_top1_borda_normal_few
sheet['D5'] = average_bornIn_bert_top5_borda_normal_few
sheet['D6'] = average_bornIn_bert_top10_borda_normal_few
### Spacy
sheet['E4'] = average_bornIn_bert_top1_borda_spacy_few
sheet['E5'] = average_bornIn_bert_top5_borda_spacy_few
sheet['E6'] = average_bornIn_bert_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['B4'] = average_bornIn_bert_top1_few
sheet['B5'] = average_bornIn_bert_top5_few
sheet['B6'] = average_bornIn_bert_top10_few
### Spacy
sheet['C4'] = average_bornIn_bert_top1_spacy_few
sheet['C5'] = average_bornIn_bert_top5_spacy_few
sheet['C6'] = average_bornIn_bert_top10_spacy_few

#Died In
## Ranking
### Normal
sheet['L4'] = average_diedIn_bert_top1_ranking_normal_few
sheet['L5'] = average_diedIn_bert_top5_ranking_normal_few
sheet['L6'] = average_diedIn_bert_top10_ranking_normal_few
### Spacy
sheet['M4'] = average_diedIn_bert_top1_ranking_spacy_few
sheet['M5'] = average_diedIn_bert_top5_ranking_spacy_few
sheet['M6'] = average_diedIn_bert_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['J4'] = average_diedIn_bert_top1_borda_normal_few
sheet['J5'] = average_diedIn_bert_top5_borda_normal_few
sheet['J6'] = average_diedIn_bert_top10_borda_normal_few
### Spacy
sheet['K4'] = average_diedIn_bert_top1_borda_spacy_few
sheet['K5'] = average_diedIn_bert_top5_borda_spacy_few
sheet['K6'] = average_diedIn_bert_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['H4'] = average_diedIn_bert_top1_few
sheet['H5'] = average_diedIn_bert_top5_few
sheet['H6'] = average_diedIn_bert_top10_few
### Spacy
sheet['I4'] = average_diedIn_bert_top1_spacy_few
sheet['I5'] = average_diedIn_bert_top5_spacy_few
sheet['I6'] = average_diedIn_bert_top10_spacy_few

#Works For
## Ranking
### Normal
sheet['R4'] = average_worksfor_bert_top1_ranking_normal_few
sheet['R5'] = average_worksfor_bert_top5_ranking_normal_few
sheet['R6'] = average_worksfor_bert_top10_ranking_normal_few
### Spacy
sheet['S4'] = average_worksfor_bert_top1_ranking_spacy_few
sheet['S5'] = average_worksfor_bert_top5_ranking_spacy_few
sheet['S6'] = average_worksfor_bert_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['P4'] = average_worksfor_bert_top1_borda_normal_few
sheet['P5'] = average_worksfor_bert_top5_borda_normal_few
sheet['P6'] = average_worksfor_bert_top10_borda_normal_few
### Spacy
sheet['Q4'] = average_worksfor_bert_top1_borda_spacy_few
sheet['Q5'] = average_worksfor_bert_top5_borda_spacy_few
sheet['Q6'] = average_worksfor_bert_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['N4'] = average_worksfor_bert_top1_few
sheet['N5'] = average_worksfor_bert_top5_few
sheet['N6'] = average_worksfor_bert_top10_few
### Spacy
sheet['O4'] = average_worksfor_bert_top1_spacy_few
sheet['O5'] = average_worksfor_bert_top5_spacy_few
sheet['O6'] = average_worksfor_bert_top10_spacy_few

#Capital
## Ranking
### Normal
sheet['F12'] = average_capital_bert_top1_ranking_normal_few
sheet['F13'] = average_capital_bert_top5_ranking_normal_few
sheet['F14'] = average_capital_bert_top10_ranking_normal_few
### Spacy
sheet['G12'] = average_capital_bert_top1_ranking_spacy_few
sheet['G13'] = average_capital_bert_top5_ranking_spacy_few
sheet['G14'] = average_capital_bert_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['D12'] = average_capital_bert_top1_borda_normal_few
sheet['D13'] = average_capital_bert_top5_borda_normal_few
sheet['D14'] = average_capital_bert_top10_borda_normal_few
### Spacy
sheet['E12'] = average_capital_bert_top1_borda_spacy_few
sheet['E13'] = average_capital_bert_top5_borda_spacy_few
sheet['E14'] = average_capital_bert_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['B12'] = average_capital_bert_top1_few
sheet['B13'] = average_capital_bert_top5_few
sheet['B14'] = average_capital_bert_top10_few
### Spacy
sheet['C12'] = average_capital_bert_top1_spacy_few
sheet['C13'] = average_capital_bert_top5_spacy_few
sheet['C14'] = average_capital_bert_top10_spacy_few

#Citizen
## Ranking
### Normal
sheet['L12'] = average_citizen_bert_top1_ranking_normal_few
sheet['L13'] = average_citizen_bert_top5_ranking_normal_few
sheet['L14'] = average_citizen_bert_top10_ranking_normal_few
### Spacy
sheet['M12'] = average_citizen_bert_top1_ranking_spacy_few
sheet['M13'] = average_citizen_bert_top5_ranking_spacy_few
sheet['M14'] = average_citizen_bert_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['J12'] = average_citizen_bert_top1_borda_normal_few
sheet['J13'] = average_citizen_bert_top5_borda_normal_few
sheet['J14'] = average_citizen_bert_top10_borda_normal_few
### Spacy
sheet['K12'] = average_citizen_bert_top1_borda_spacy_few
sheet['K13'] = average_citizen_bert_top5_borda_spacy_few
sheet['K14'] = average_citizen_bert_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['H12'] = average_citizen_bert_top1_few
sheet['H13'] = average_citizen_bert_top5_few
sheet['H14'] = average_citizen_bert_top10_few
### Spacy
sheet['I12'] = average_citizen_bert_top1_spacy_few
sheet['I13'] = average_citizen_bert_top5_spacy_few
sheet['I14'] = average_citizen_bert_top10_spacy_few

#Oficial Language
## Ranking
### Normal
sheet['R4'] = average_language_bert_top1_ranking_normal_few
sheet['R5'] = average_language_bert_top5_ranking_normal_few
sheet['R6'] = average_language_bert_top10_ranking_normal_few
### Spacy
sheet['S4'] = average_language_bert_top1_ranking_spacy_few
sheet['S5'] = average_language_bert_top5_ranking_spacy_few
sheet['S6'] = average_language_bert_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['P4'] = average_language_bert_top1_borda_normal_few
sheet['P5'] = average_language_bert_top5_borda_normal_few
sheet['P6'] = average_language_bert_top10_borda_normal_few
### Spacy
sheet['Q4'] = average_language_bert_top1_borda_spacy_few
sheet['Q5'] = average_language_bert_top5_borda_spacy_few
sheet['Q6'] = average_language_bert_top10_borda_spacy_few


# Save the changes
wb.save('/home/rafael/tese/Slide/testes_2.xlsx')

### Roberta

In [ ]:
# Select the active worksheet
sheet = wb['roberta_few_shot']

# Insert data into specific cells
#Born In
## Ranking
### Normal
sheet['F4'] = average_bornIn_roberta_top1_ranking_normal_few
sheet['F5'] = average_bornIn_roberta_top5_ranking_normal_few
sheet['F6'] = average_bornIn_roberta_top10_ranking_normal_few
### Spacy
sheet['G4'] = average_bornIn_roberta_top1_ranking_spacy_few
sheet['G5'] = average_bornIn_roberta_top5_ranking_spacy_few
sheet['G6'] = average_bornIn_roberta_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['D4'] = average_bornIn_roberta_top1_borda_normal_few
sheet['D5'] = average_bornIn_roberta_top5_borda_normal_few
sheet['D6'] = average_bornIn_roberta_top10_borda_normal_few
### Spacy
sheet['E4'] = average_bornIn_roberta_top1_borda_spacy_few
sheet['E5'] = average_bornIn_roberta_top5_borda_spacy_few
sheet['E6'] = average_bornIn_roberta_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['B4'] = average_bornIn_roberta_top1_few
sheet['B5'] = average_bornIn_roberta_top5_few
sheet['B6'] = average_bornIn_roberta_top10_few
### Spacy
sheet['C4'] = average_bornIn_roberta_top1_spacy_few
sheet['C5'] = average_bornIn_roberta_top5_spacy_few
sheet['C6'] = average_bornIn_roberta_top10_spacy_few

#Died In
## Ranking
### Normal
sheet['L4'] = average_diedIn_roberta_top1_ranking_normal_few
sheet['L5'] = average_diedIn_roberta_top5_ranking_normal_few
sheet['L6'] = average_diedIn_roberta_top10_ranking_normal_few
### Spacy
sheet['M4'] = average_diedIn_roberta_top1_ranking_spacy_few
sheet['M5'] = average_diedIn_roberta_top5_ranking_spacy_few
sheet['M6'] = average_diedIn_roberta_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['J4'] = average_diedIn_roberta_top1_borda_normal_few
sheet['J5'] = average_diedIn_roberta_top5_borda_normal_few
sheet['J6'] = average_diedIn_roberta_top10_borda_normal_few
### Spacy
sheet['K4'] = average_diedIn_roberta_top1_borda_spacy_few
sheet['K5'] = average_diedIn_roberta_top5_borda_spacy_few
sheet['K6'] = average_diedIn_roberta_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['H4'] = average_diedIn_roberta_top1_few
sheet['H5'] = average_diedIn_roberta_top5_few
sheet['H6'] = average_diedIn_roberta_top10_few
### Spacy
sheet['I4'] = average_diedIn_roberta_top1_spacy_few
sheet['I5'] = average_diedIn_roberta_top5_spacy_few
sheet['I6'] = average_diedIn_roberta_top10_spacy_few

#Works For
## Ranking
### Normal
sheet['R4'] = average_worksfor_roberta_top1_ranking_normal_few
sheet['R5'] = average_worksfor_roberta_top5_ranking_normal_few
sheet['R6'] = average_worksfor_roberta_top10_ranking_normal_few
### Spacy
sheet['S4'] = average_worksfor_roberta_top1_ranking_spacy_few
sheet['S5'] = average_worksfor_roberta_top5_ranking_spacy_few
sheet['S6'] = average_worksfor_roberta_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['P4'] = average_worksfor_roberta_top1_borda_normal_few
sheet['P5'] = average_worksfor_roberta_top5_borda_normal_few
sheet['P6'] = average_worksfor_roberta_top10_borda_normal_few
### Spacy
sheet['Q4'] = average_worksfor_roberta_top1_borda_spacy_few
sheet['Q5'] = average_worksfor_roberta_top5_borda_spacy_few
sheet['Q6'] = average_worksfor_roberta_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['N4'] = average_worksfor_roberta_top1_few
sheet['N5'] = average_worksfor_roberta_top5_few
sheet['N6'] = average_worksfor_roberta_top10_few
### Spacy
sheet['O4'] = average_worksfor_roberta_top1_spacy_few
sheet['O5'] = average_worksfor_roberta_top5_spacy_few
sheet['O6'] = average_worksfor_roberta_top10_spacy_few

#Capital
## Ranking
### Normal
sheet['F12'] = average_capital_roberta_top1_ranking_normal_few
sheet['F13'] = average_capital_roberta_top5_ranking_normal_few
sheet['F14'] = average_capital_roberta_top10_ranking_normal_few
### Spacy
sheet['G12'] = average_capital_roberta_top1_ranking_spacy_few
sheet['G13'] = average_capital_roberta_top5_ranking_spacy_few
sheet['G14'] = average_capital_roberta_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['D12'] = average_capital_roberta_top1_borda_normal_few
sheet['D13'] = average_capital_roberta_top5_borda_normal_few
sheet['D14'] = average_capital_roberta_top10_borda_normal_few
### Spacy
sheet['E12'] = average_capital_roberta_top1_borda_spacy_few
sheet['E13'] = average_capital_roberta_top5_borda_spacy_few
sheet['E14'] = average_capital_roberta_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['B12'] = average_capital_roberta_top1_few
sheet['B13'] = average_capital_roberta_top5_few
sheet['B14'] = average_capital_roberta_top10_few
### Spacy
sheet['C12'] = average_capital_roberta_top1_spacy_few
sheet['C13'] = average_capital_roberta_top5_spacy_few
sheet['C14'] = average_capital_roberta_top10_spacy_few

#Citizen
## Ranking
### Normal
sheet['L12'] = average_citizen_roberta_top1_ranking_normal_few
sheet['L13'] = average_citizen_roberta_top5_ranking_normal_few
sheet['L14'] = average_citizen_roberta_top10_ranking_normal_few
### Spacy
sheet['M12'] = average_citizen_roberta_top1_ranking_spacy_few
sheet['M13'] = average_citizen_roberta_top5_ranking_spacy_few
sheet['M14'] = average_citizen_roberta_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['J12'] = average_citizen_roberta_top1_borda_normal_few
sheet['J13'] = average_citizen_roberta_top5_borda_normal_few
sheet['J14'] = average_citizen_roberta_top10_borda_normal_few
### Spacy
sheet['K12'] = average_citizen_roberta_top1_borda_spacy_few
sheet['K13'] = average_citizen_roberta_top5_borda_spacy_few
sheet['K14'] = average_citizen_roberta_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['H12'] = average_citizen_roberta_top1_few
sheet['H13'] = average_citizen_roberta_top5_few
sheet['H14'] = average_citizen_roberta_top10_few
### Spacy
sheet['I12'] = average_citizen_roberta_top1_spacy_few
sheet['I13'] = average_citizen_roberta_top5_spacy_few
sheet['I14'] = average_citizen_roberta_top10_spacy_few

#Oficial Language
## Ranking
### Normal
sheet['R4'] = average_language_roberta_top1_ranking_normal_few
sheet['R5'] = average_language_roberta_top5_ranking_normal_few
sheet['R6'] = average_language_roberta_top10_ranking_normal_few
### Spacy
sheet['S4'] = average_language_roberta_top1_ranking_spacy_few
sheet['S5'] = average_language_roberta_top5_ranking_spacy_few
sheet['S6'] = average_language_roberta_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['P4'] = average_language_roberta_top1_borda_normal_few
sheet['P5'] = average_language_roberta_top5_borda_normal_few
sheet['P6'] = average_language_roberta_top10_borda_normal_few
### Spacy
sheet['Q4'] = average_language_roberta_top1_borda_spacy_few
sheet['Q5'] = average_language_roberta_top5_borda_spacy_few
sheet['Q6'] = average_language_roberta_top10_borda_spacy_few


# Save the changes
wb.save('/home/rafael/tese/Slide/testes_2.xlsx')

### Electra

In [ ]:
# Select the active worksheet
sheet = wb['electra_few_shot']

# Insert data into specific cells
#Born In
## Ranking
### Normal
sheet['F4'] = average_bornIn_electra_top1_ranking_normal_few
sheet['F5'] = average_bornIn_electra_top5_ranking_normal_few
sheet['F6'] = average_bornIn_electra_top10_ranking_normal_few
### Spacy
sheet['G4'] = average_bornIn_electra_top1_ranking_spacy_few
sheet['G5'] = average_bornIn_electra_top5_ranking_spacy_few
sheet['G6'] = average_bornIn_electra_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['D4'] = average_bornIn_electra_top1_borda_normal_few
sheet['D5'] = average_bornIn_electra_top5_borda_normal_few
sheet['D6'] = average_bornIn_electra_top10_borda_normal_few
### Spacy
sheet['E4'] = average_bornIn_electra_top1_borda_spacy_few
sheet['E5'] = average_bornIn_electra_top5_borda_spacy_few
sheet['E6'] = average_bornIn_electra_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['B4'] = average_bornIn_electra_top1_few
sheet['B5'] = average_bornIn_electra_top5_few
sheet['B6'] = average_bornIn_electra_top10_few
### Spacy
sheet['C4'] = average_bornIn_electra_top1_spacy_few
sheet['C5'] = average_bornIn_electra_top5_spacy_few
sheet['C6'] = average_bornIn_electra_top10_spacy_few

#Died In
## Ranking
### Normal
sheet['L4'] = average_diedIn_electra_top1_ranking_normal_few
sheet['L5'] = average_diedIn_electra_top5_ranking_normal_few
sheet['L6'] = average_diedIn_electra_top10_ranking_normal_few
### Spacy
sheet['M4'] = average_diedIn_electra_top1_ranking_spacy_few
sheet['M5'] = average_diedIn_electra_top5_ranking_spacy_few
sheet['M6'] = average_diedIn_electra_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['J4'] = average_diedIn_electra_top1_borda_normal_few
sheet['J5'] = average_diedIn_electra_top5_borda_normal_few
sheet['J6'] = average_diedIn_electra_top10_borda_normal_few
### Spacy
sheet['K4'] = average_diedIn_electra_top1_borda_spacy_few
sheet['K5'] = average_diedIn_electra_top5_borda_spacy_few
sheet['K6'] = average_diedIn_electra_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['H4'] = average_diedIn_electra_top1_few
sheet['H5'] = average_diedIn_electra_top5_few
sheet['H6'] = average_diedIn_electra_top10_few
### Spacy
sheet['I4'] = average_diedIn_electra_top1_spacy_few
sheet['I5'] = average_diedIn_electra_top5_spacy_few
sheet['I6'] = average_diedIn_electra_top10_spacy_few

#Works For
## Ranking
### Normal
sheet['R4'] = average_worksfor_electra_top1_ranking_normal_few
sheet['R5'] = average_worksfor_electra_top5_ranking_normal_few
sheet['R6'] = average_worksfor_electra_top10_ranking_normal_few
### Spacy
sheet['S4'] = average_worksfor_electra_top1_ranking_spacy_few
sheet['S5'] = average_worksfor_electra_top5_ranking_spacy_few
sheet['S6'] = average_worksfor_electra_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['P4'] = average_worksfor_electra_top1_borda_normal_few
sheet['P5'] = average_worksfor_electra_top5_borda_normal_few
sheet['P6'] = average_worksfor_electra_top10_borda_normal_few
### Spacy
sheet['Q4'] = average_worksfor_electra_top1_borda_spacy_few
sheet['Q5'] = average_worksfor_electra_top5_borda_spacy_few
sheet['Q6'] = average_worksfor_electra_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['N4'] = average_worksfor_electra_top1_few
sheet['N5'] = average_worksfor_electra_top5_few
sheet['N6'] = average_worksfor_electra_top10_few
### Spacy
sheet['O4'] = average_worksfor_electra_top1_spacy_few
sheet['O5'] = average_worksfor_electra_top5_spacy_few
sheet['O6'] = average_worksfor_electra_top10_spacy_few

#Capital
## Ranking
### Normal
sheet['F12'] = average_capital_electra_top1_ranking_normal_few
sheet['F13'] = average_capital_electra_top5_ranking_normal_few
sheet['F14'] = average_capital_electra_top10_ranking_normal_few
### Spacy
sheet['G12'] = average_capital_electra_top1_ranking_spacy_few
sheet['G13'] = average_capital_electra_top5_ranking_spacy_few
sheet['G14'] = average_capital_electra_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['D12'] = average_capital_electra_top1_borda_normal_few
sheet['D13'] = average_capital_electra_top5_borda_normal_few
sheet['D14'] = average_capital_electra_top10_borda_normal_few
### Spacy
sheet['E12'] = average_capital_electra_top1_borda_spacy_few
sheet['E13'] = average_capital_electra_top5_borda_spacy_few
sheet['E14'] = average_capital_electra_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['B12'] = average_capital_electra_top1_few
sheet['B13'] = average_capital_electra_top5_few
sheet['B14'] = average_capital_electra_top10_few
### Spacy
sheet['C12'] = average_capital_electra_top1_spacy_few
sheet['C13'] = average_capital_electra_top5_spacy_few
sheet['C14'] = average_capital_electra_top10_spacy_few

#Citizen
## Ranking
### Normal
sheet['L12'] = average_citizen_electra_top1_ranking_normal_few
sheet['L13'] = average_citizen_electra_top5_ranking_normal_few
sheet['L14'] = average_citizen_electra_top10_ranking_normal_few
### Spacy
sheet['M12'] = average_citizen_electra_top1_ranking_spacy_few
sheet['M13'] = average_citizen_electra_top5_ranking_spacy_few
sheet['M14'] = average_citizen_electra_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['J12'] = average_citizen_electra_top1_borda_normal_few
sheet['J13'] = average_citizen_electra_top5_borda_normal_few
sheet['J14'] = average_citizen_electra_top10_borda_normal_few
### Spacy
sheet['K12'] = average_citizen_electra_top1_borda_spacy_few
sheet['K13'] = average_citizen_electra_top5_borda_spacy_few
sheet['K14'] = average_citizen_electra_top10_borda_spacy_few

## Prompts Originais
### Normal
sheet['H12'] = average_citizen_electra_top1_few
sheet['H13'] = average_citizen_electra_top5_few
sheet['H14'] = average_citizen_electra_top10_few
### Spacy
sheet['I12'] = average_citizen_electra_top1_spacy_few
sheet['I13'] = average_citizen_electra_top5_spacy_few
sheet['I14'] = average_citizen_electra_top10_spacy_few

#Oficial Language
## Ranking
### Normal
sheet['R4'] = average_language_electra_top1_ranking_normal_few
sheet['R5'] = average_language_electra_top5_ranking_normal_few
sheet['R6'] = average_language_electra_top10_ranking_normal_few
### Spacy
sheet['S4'] = average_language_electra_top1_ranking_spacy_few
sheet['S5'] = average_language_electra_top5_ranking_spacy_few
sheet['S6'] = average_language_electra_top10_ranking_spacy_few

## Borda Count
### Normal
sheet['P4'] = average_language_electra_top1_borda_normal_few
sheet['P5'] = average_language_electra_top5_borda_normal_few
sheet['P6'] = average_language_electra_top10_borda_normal_few
### Spacy
sheet['Q4'] = average_language_electra_top1_borda_spacy_few
sheet['Q5'] = average_language_electra_top5_borda_spacy_few
sheet['Q6'] = average_language_electra_top10_borda_spacy_few


# Save the changes
wb.save('/home/rafael/tese/Slide/testes_2.xlsx')

### Merged Models

In [ ]:
'''# Open the existing Excel file
wb = xw.Book('/home/rafael/tese/Slide/testes_2.xlsx')

# Select the active worksheet
sheet = wb.sheets['merge_models_few_shot']

# Insert data into specific cells
#Born In
## Ranking
### Normal
sheet.range('B4').value = average_bornIn_merged_model_top1_ranking_normal_few
sheet.range('B5').value = average_bornIn_merged_model_top1_ranking_normal_few
sheet.range('B6').value = average_bornIn_merged_model_top1_ranking_normal_few
### Spacy
sheet.range('C4').value = average_bornIn_merged_model_top1_ranking_spacy_few
sheet.range('C5').value = average_bornIn_merged_model_top1_ranking_spacy_few
sheet.range('C6').value = average_bornIn_merged_model_top1_ranking_spacy_few

## Borda Count
### Normal
sheet.range('D4').value = average_bornIn_merged_model_top1_borda_normal_few
sheet.range('D5').value = average_bornIn_merged_model_top1_borda_normal_few
sheet.range('D6').value = average_bornIn_merged_model_top1_borda_normal_few
### Spacy
sheet.range('E4').value = average_bornIn_merged_model_top1_borda_spacy_few
sheet.range('E5').value = average_bornIn_merged_model_top1_borda_spacy_few
sheet.range('E6').value = average_bornIn_merged_model_top1_borda_spacy_few

## Prompts Originais
### Bert
#### Normal
sheet.range('F4').value = average_bornIn_bert_top1_few
sheet.range('F5').value = average_bornIn_bert_top5_few
sheet.range('F6').value = average_bornIn_bert_top10_few
#### Spacy
sheet.range('G4').value = average_bornIn_bert_top1_spacy_few
sheet.range('G5').value = average_bornIn_bert_top5_spacy_few
sheet.range('G6').value = average_bornIn_bert_top10_spacy_few

### Roberta
#### Normal
sheet.range('H4').value = average_bornIn_roberta_top1_few
sheet.range('H5').value = average_bornIn_roberta_top5_few
sheet.range('H6').value = average_bornIn_roberta_top10_few
#### Spacy
sheet.range('I4').value = average_bornIn_roberta_top1_spacy_few
sheet.range('I5').value = average_bornIn_roberta_top5_spacy_few
sheet.range('I6').value = average_bornIn_roberta_top10_spacy_few

### Electra
#### Normal
sheet.range('J4').value = average_bornIn_electra_top1_few
sheet.range('J5').value = average_bornIn_electra_top5_few
sheet.range('J6').value = average_bornIn_electra_top10_few
#### Spacy
sheet.range('K4').value = average_bornIn_electra_top1_spacy_few
sheet.range('K5').value = average_bornIn_electra_top5_spacy_few
sheet.range('K6').value = average_bornIn_electra_top10_spacy_few

#Died In
## Ranking
### Normal
sheet.range('B11').value = average_diedIn_merged_model_top1_ranking_normal_few
sheet.range('B12').value = average_diedIn_merged_model_top1_ranking_normal_few
sheet.range('B13').value = average_diedIn_merged_model_top1_ranking_normal_few
### Spacy
sheet.range('C11').value = average_diedIn_merged_model_top1_ranking_spacy_few
sheet.range('C12').value = average_diedIn_merged_model_top1_ranking_spacy_few
sheet.range('C13').value = average_diedIn_merged_model_top1_ranking_spacy_few

## Borda Count
### Normal
sheet.range('D11').value = average_diedIn_merged_model_top1_borda_normal_few
sheet.range('D12').value = average_diedIn_merged_model_top1_borda_normal_few
sheet.range('D13').value = average_diedIn_merged_model_top1_borda_normal_few
### Spacy
sheet.range('E11').value = average_diedIn_merged_model_top1_borda_spacy_few
sheet.range('E5').value = average_diedIn_merged_model_top1_borda_spacy_few
sheet.range('E13').value = average_diedIn_merged_model_top1_borda_spacy_few

## Prompts Originais
### Bert
#### Normal
sheet.range('F11').value = average_diedIn_bert_top1_few
sheet.range('F12').value = average_diedIn_bert_top5_few
sheet.range('F13').value = average_diedIn_bert_top10_few
#### Spacy
sheet.range('G11').value = average_diedIn_bert_top1_spacy_few
sheet.range('G12').value = average_diedIn_bert_top5_spacy_few
sheet.range('G13').value = average_diedIn_bert_top10_spacy_few

### Roberta
#### Normal
sheet.range('H11').value = average_diedIn_roberta_top1_few
sheet.range('H12').value = average_diedIn_roberta_top5_few
sheet.range('H13').value = average_diedIn_roberta_top10_few
#### Spacy
sheet.range('I11').value = average_diedIn_roberta_top1_spacy_few
sheet.range('I12').value = average_diedIn_roberta_top5_spacy_few
sheet.range('I13').value = average_diedIn_roberta_top10_spacy_few

### Electra
#### Normal
sheet.range('J11').value = average_diedIn_electra_top1_few
sheet.range('J12').value = average_diedIn_electra_top5_few
sheet.range('J13').value = average_diedIn_electra_top10_few
#### Spacy
sheet.range('K11').value = average_diedIn_electra_top1_spacy_few
sheet.range('K12').value = average_diedIn_electra_top5_spacy_few
sheet.range('K13').value = average_diedIn_electra_top10_spacy_few


#Works For
## Ranking
### Normal
sheet.range('B18').value = average_worksfor_merged_model_top1_ranking_normal_few
sheet.range('B19').value = average_worksfor_merged_model_top1_ranking_normal_few
sheet.range('B6').value = average_worksfor_merged_model_top1_ranking_normal_few
### Spacy
sheet.range('C18').value = average_worksfor_merged_model_top1_ranking_spacy_few
sheet.range('C19').value = average_worksfor_merged_model_top1_ranking_spacy_few
sheet.range('C20').value = average_worksfor_merged_model_top1_ranking_spacy_few

## Borda Count
### Normal
sheet.range('D18').value = average_worksfor_merged_model_top1_borda_normal_few
sheet.range('D19').value = average_worksfor_merged_model_top1_borda_normal_few
sheet.range('D20').value = average_worksfor_merged_model_top1_borda_normal_few
### Spacy
sheet.range('E18').value = average_worksfor_merged_model_top1_borda_spacy_few
sheet.range('E19').value = average_worksfor_merged_model_top1_borda_spacy_few
sheet.range('E20').value = average_worksfor_merged_model_top1_borda_spacy_few

## Prompts Originais
### Bert
#### Normal
sheet.range('F18').value = average_worksfor_bert_top1_few
sheet.range('F19').value = average_worksfor_bert_top5_few
sheet.range('F20').value = average_worksfor_bert_top10_few
#### Spacy
sheet.range('G18').value = average_worksfor_bert_top1_spacy_few
sheet.range('G19').value = average_worksfor_bert_top5_spacy_few
sheet.range('G20').value = average_worksfor_bert_top10_spacy_few

### Roberta
#### Normal
sheet.range('H18').value = average_worksfor_roberta_top1_few
sheet.range('H19').value = average_worksfor_roberta_top5_few
sheet.range('H20').value = average_worksfor_roberta_top10_few
#### Spacy
sheet.range('I18').value = average_worksfor_roberta_top1_spacy_few
sheet.range('I19').value = average_worksfor_roberta_top5_spacy_few
sheet.range('I20').value = average_worksfor_roberta_top10_spacy_few

### Electra
#### Normal
sheet.range('J18').value = average_worksfor_electra_top1_few
sheet.range('J19').value = average_worksfor_electra_top5_few
sheet.range('J20').value = average_worksfor_electra_top10_few
#### Spacy
sheet.range('K18').value = average_worksfor_electra_top1_spacy_few
sheet.range('K19').value = average_worksfor_electra_top5_spacy_few
sheet.range('K20').value = average_worksfor_electra_top10_spacy_few

#Capital
## Ranking
### Normal
sheet.range('B25').value = average_capital_merged_model_top1_ranking_normal_few
sheet.range('B26').value = average_capital_merged_model_top1_ranking_normal_few
sheet.range('B27').value = average_capital_merged_model_top1_ranking_normal_few
### Spacy
sheet.range('C25').value = average_capital_merged_model_top1_ranking_spacy_few
sheet.range('C26').value = average_capital_merged_model_top1_ranking_spacy_few
sheet.range('C27').value = average_capital_merged_model_top1_ranking_spacy_few

## Borda Count
### Normal
sheet.range('D25').value = average_capital_merged_model_top1_borda_normal_few
sheet.range('D26').value = average_capital_merged_model_top1_borda_normal_few
sheet.range('D6').value = average_capital_merged_model_top1_borda_normal_few
### Spacy
sheet.range('E25').value = average_capital_merged_model_top1_borda_spacy_few
sheet.range('E26').value = average_capital_merged_model_top1_borda_spacy_few
sheet.range('E27').value = average_capital_merged_model_top1_borda_spacy_few

## Prompts Originais
### Bert
#### Normal
sheet.range('F25').value = average_capital_bert_top1_few
sheet.range('F26').value = average_capital_bert_top5_few
sheet.range('F27').value = average_capital_bert_top10_few
#### Spacy
sheet.range('G25').value = average_capital_bert_top1_spacy_few
sheet.range('G26').value = average_capital_bert_top5_spacy_few
sheet.range('G27').value = average_capital_bert_top10_spacy_few

### Roberta
#### Normal
sheet.range('H25').value = average_capital_roberta_top1_few
sheet.range('H26').value = average_capital_roberta_top5_few
sheet.range('H27').value = average_capital_roberta_top10_few
#### Spacy
sheet.range('I25').value = average_capital_roberta_top1_spacy_few
sheet.range('I26').value = average_capital_roberta_top5_spacy_few
sheet.range('I27').value = average_capital_roberta_top10_spacy_few

### Electra
#### Normal
sheet.range('J25').value = average_capital_electra_top1_few
sheet.range('J26').value = average_capital_electra_top5_few
sheet.range('J27').value = average_capital_electra_top10_few
#### Spacy
sheet.range('K25').value = average_capital_electra_top1_spacy_few
sheet.range('K26').value = average_capital_electra_top5_spacy_few
sheet.range('K27').value = average_capital_electra_top10_spacy_few

#Citizen
## Ranking
### Normal
sheet.range('B32').value = average_citizen_merged_model_top1_ranking_normal_few
sheet.range('B33').value = average_citizen_merged_model_top1_ranking_normal_few
sheet.range('B34').value = average_citizen_merged_model_top1_ranking_normal_few
### Spacy
sheet.range('C32').value = average_citizen_merged_model_top1_ranking_spacy_few
sheet.range('C33').value = average_citizen_merged_model_top1_ranking_spacy_few
sheet.range('C34').value = average_citizen_merged_model_top1_ranking_spacy_few

## Borda Count
### Normal
sheet.range('D32').value = average_citizen_merged_model_top1_borda_normal_few
sheet.range('D33').value = average_citizen_merged_model_top1_borda_normal_few
sheet.range('D34').value = average_citizen_merged_model_top1_borda_normal_few
### Spacy
sheet.range('E32').value = average_citizen_merged_model_top1_borda_spacy_few
sheet.range('E33').value = average_citizen_merged_model_top1_borda_spacy_few
sheet.range('E34').value = average_citizen_merged_model_top1_borda_spacy_few

## Prompts Originais
### Bert
#### Normal
sheet.range('F32').value = average_capital_bert_top1_few
sheet.range('F33').value = average_capital_bert_top5_few
sheet.range('F27').value = average_capital_bert_top10_few
#### Spacy
sheet.range('G32').value = average_capital_bert_top1_spacy_few
sheet.range('G33').value = average_capital_bert_top5_spacy_few
sheet.range('G34').value = average_capital_bert_top10_spacy_few

### Roberta
#### Normal
sheet.range('H32').value = average_capital_roberta_top1_few
sheet.range('H33').value = average_capital_roberta_top5_few
sheet.range('H34').value = average_capital_roberta_top10_few
#### Spacy
sheet.range('I32').value = average_capital_roberta_top1_spacy_few
sheet.range('I33').value = average_capital_roberta_top5_spacy_few
sheet.range('I34').value = average_capital_roberta_top10_spacy_few

### Electra
#### Normal
sheet.range('J32').value = average_capital_electra_top1_few
sheet.range('J33').value = average_capital_electra_top5_few
sheet.range('J34').value = average_capital_electra_top10_few
#### Spacy
sheet.range('K32').value = average_capital_electra_top1_spacy_few
sheet.range('K33').value = average_capital_electra_top5_spacy_few
sheet.range('K34').value = average_capital_electra_top10_spacy_few

# Save the changes
wb.save('/home/rafael/tese/Slide/testes_2.xlsx')'''

"# Open the existing Excel file\nwb = xw.Book('/home/rafael/tese/Slide/testes_2.xlsx')\n\n# Select the active worksheet\nsheet = wb.sheets['merge_models_few_shot']\n\n# Insert data into specific cells\n#Born In\n## Ranking\n### Normal\nsheet.range('B4').value = average_bornIn_merged_model_top1_ranking_normal_few\nsheet.range('B5').value = average_bornIn_merged_model_top1_ranking_normal_few\nsheet.range('B6').value = average_bornIn_merged_model_top1_ranking_normal_few\n### Spacy\nsheet.range('C4').value = average_bornIn_merged_model_top1_ranking_spacy_few\nsheet.range('C5').value = average_bornIn_merged_model_top1_ranking_spacy_few\nsheet.range('C6').value = average_bornIn_merged_model_top1_ranking_spacy_few\n\n## Borda Count\n### Normal\nsheet.range('D4').value = average_bornIn_merged_model_top1_borda_normal_few\nsheet.range('D5').value = average_bornIn_merged_model_top1_borda_normal_few\nsheet.range('D6').value = average_bornIn_merged_model_top1_borda_normal_few\n### Spacy\nsheet.rang